In [7]:
%config Completer.use_jedi = False
import pandas as pd
import numpy as np
from scipy.stats import hmean
import matplotlib.pyplot as plt
import os
from collections import Counter

In [8]:
def csv_parser(log_file, results_file):
    results_list = []
    with open(log_file,"r") as fp:
        for line in fp:
            split = line.split()
            split2 = line.split(",")
            if(len(split)>0):
                if("pmpakos@" in split[0]):
                    matrix_name = ''
                    # print("---------")
            if(len(split)>2):
                if((split[0]=="INFO:") & (split[1]=="loading") & (split[2]=="Mtx")):
                    print(split[-1])
                    matrix_name = split[-1].split("/")[-1]
                    matrix_name = matrix_name.replace(".mtx","")
            if(len(split2)>2):
                if(split2[1]!=" matrix"):
                    line=line.replace("app",matrix_name).strip("DATA_CSV:,")
                    # line=line.replace(",","\t")
                    results_list.append(line)
                    print(line)

    file = open(results_file,"w")
    for res in results_list:
        file.write(res)
    file.close()

In [746]:
%%time
def clean_matrix_generation_log2(start_gen):
    fw_list = []
    cnt = 0
    with open(start_gen) as fp:
        prev_line = "NULL"
        param_df = pd.DataFrame()
        param_df["param"] = []
        param_df["name"] = []
        
        for line in fp:
            if(">>>> " in line):
                line = line.strip(">>>> ").replace(" ",",")#.replace(".mtx","")
                line_spl = line.split(",")
                line_spl[0] = line_spl[0].replace(".0_","_")
                line_new = ",".join(line_spl)
                param_df = param_df.append({"param": prev_line, "name": line_spl[0]}, ignore_index = True)
                if(line not in fw_list):
                    fw_list.append(line)
                else: # duplicate found
                    cnt+=1
            prev_line = line.strip("\n")
    print("duplicates found :", cnt)
    return param_df

def find_duplicates(distr_memrange):
    start_gen = "./generation_stats/"+distr_memrange+"_log.txt"
    param_df = clean_matrix_generation_log2(start_gen)

    counter = Counter(param_df["name"])
    filtered_counter = dict()
    for (key, value) in counter.items():
        if(value > 1):
            filtered_counter[key] = value

    filtered_counter = dict(sorted(filtered_counter.items(), key=lambda item: item[1]))
    duplicate_matrices = filtered_counter.keys()
    # print(len(duplicate_matrices))

    fw = open("duplicate_matrices_neighbours_generator_"+distr_memrange+".txt","w")
    for matrix in duplicate_matrices:
        fw.write(matrix+"\n")
        lista = list(param_df[param_df["name"] == matrix]["param"])
        for a in lista:
            fw.write('\t'+a+"\n")        

# distr = "normal_"
# mem_range = "4-32"
# mem_range = "32-512"
# mem_range = "512-2048"
# distr_memrange = distr + mem_range
distr_memrange = "validation_matrices_10_samples_30_range_twins"
find_duplicates(distr_memrange)

duplicates found : 193
CPU times: user 4.73 s, sys: 0 ns, total: 4.73 s
Wall time: 4.84 s


In [13]:
def clean_matrix_generation_log(start_gen, results_gen):
    fw = open(results_gen,"w")
    fw_list = []
    cnt = 0
    with open(start_gen) as fp:
        prev_line = "NULL"
        param_df = pd.DataFrame()
        
        for line in fp:
            if(">>>> " in line):
                param_df.append({"param": prev_line, "name": line}, ignore_index = True)
                if(line not in fw_list):
                    fw_list.append(line)
                    line = line.strip(">>>> ").replace(" ",",")#.replace(".mtx","")
                    line_spl = line.split(",")
                    line_spl[0] = line_spl[0].replace(".0_","_")
                    # line_spl[0] = line_spl[0].split("_")
                    # del line_spl[0][3]
                    # line_spl[0] = "_".join(line_spl[0])
                    line_new = ",".join(line_spl)
                    fw.write(line_new)
                else: # duplicate found
                    cnt+=1
                    
            prev_line = line
    print("duplicates found :", cnt)
    fw.close()

def clean_results(start_list, results_list):
    for t1,t2 in zip(start_list,results_list):
        fw = open(t2,"w")
        with open(t1) as fp:
            for line in fp:
                if("DATA_CSV:," in line):
                    line_new = line.strip("DATA_CSV:,").replace(".0_","_")
                    fw.write(line_new)
        fw.close()
    
def read_results_single(result_file):
    header = ["matrix_name","original rows","original cols","original NNZs",
              "padded rows","padded cols","padded NNZs","padding overhead[%]",
              "num of runs","total run time[sec]","time[ms]/run","performance[GFLOPs]","performance (padded)[GFLOPs]","mem_footprint[MB]"]
    df = pd.read_table(result_file, delimiter =",", names=header)
    return df

def read_results(results_list, results_gen, results_csv):
    # this header was used for old result reporting
    # header = ["matrix_name","mem_range","avg_nnz_per_row","std_nnz_per_row","avg_bw","std_bw","avg_sc","std_sc"]
    # v2 dgal generator
    header = ["matrix_name", "distribution", "placement", "seed", 
              "nr_rows", "nr_cols", "nr_nnz", "density", "mem_footprint", "mem_range", 
              "avg_nnz_per_row", "std_nnz_per_row", 
              "avg_bw", "std_bw", "avg_bw_scaled", "std_bw_scaled", 
              "avg_sc", "std_sc", "avg_sc_scaled", "std_sc_scaled", 
              "skew_coeff", "avg_num_neighbours", "cross_row_similarity"]
    df_gen = pd.read_table(results_gen, delimiter =",", names=header) 
    
    dataframes = []
    for result_file in results_list:
        dataframes.append(read_results_single(result_file))
    matrices = dataframes[0]["matrix_name"].unique()

    n_runs = len(results_list)
    results_final = []
    cntt=0
    for matrix in matrices:
        # print(matrix)
        selected_gen = df_gen[df_gen["matrix_name"]==matrix]
        if(len(selected_gen)==0):
            print('\t', matrix)
            cntt+=1
        else:
            matrix_name = np.asarray(selected_gen["matrix_name"])[0]
            distribution = np.asarray(selected_gen["distribution"])[0]
            placement = np.asarray(selected_gen["placement"])[0]
            seed = np.asarray(selected_gen["seed"])[0]
            nr_rows = np.asarray(selected_gen["nr_rows"])[0]
            nr_cols = np.asarray(selected_gen["nr_cols"])[0]
            nr_nnz = np.asarray(selected_gen["nr_nnz"])[0]
            density = np.asarray(selected_gen["density"])[0]
            # if I want to show mem_footprint of CSR double representation
            mem_footprint = np.asarray(selected_gen["mem_footprint"])[0]
            
            mem_range = np.asarray(selected_gen["mem_range"])[0]
            avg_nnz_per_row = np.asarray(selected_gen["avg_nnz_per_row"])[0]
            std_nnz_per_row = np.asarray(selected_gen["std_nnz_per_row"])[0]
            avg_bw = np.asarray(selected_gen["avg_bw"])[0]
            std_bw = np.asarray(selected_gen["std_bw"])[0]
            avg_bw_scaled = np.asarray(selected_gen["avg_bw_scaled"])[0]
            std_bw_scaled = np.asarray(selected_gen["std_bw_scaled"])[0]
            avg_sc = np.asarray(selected_gen["avg_sc"])[0]
            std_sc = np.asarray(selected_gen["std_sc"])[0]
            avg_sc_scaled = np.asarray(selected_gen["avg_sc_scaled"])[0]
            std_sc_scaled = np.asarray(selected_gen["std_sc_scaled"])[0]
            skew_coeff = np.asarray(selected_gen["skew_coeff"])[0]
            avg_num_neighbours = np.asarray(selected_gen["avg_num_neighbours"])[0]
            cross_row_similarity = np.asarray(selected_gen["cross_row_similarity"])[0]

            matrix_list = matrix.split("_")
            mtx_name = matrix_list[0]

            # nr_rows = int(matrix_list[1])
            # nr_cols = int(matrix_list[2])

            # if(len(matrix_list)==7):#random
            #     placement = "random"
            #     diagonal_factor = 1
            # else: #diagonal
            #     placement = "diagonal"
            #     diagonal_factor = matrix_list[6]

            # seed = matrix_list[-1].strip("n").strip("g")

            # distr = list(filter(lambda x: x.isalpha(), matrix_list[-1]))[0]
            # if(distr=="g"):
            #     distribution="gamma"
            # else:
            #     distribution="normal"

            # nr_nnz = np.asarray(dataframes[0][dataframes[0]["matrix_name"]==matrix]["original NNZs"])[0]
            # density = nr_nnz/(nr_rows*nr_cols)*100

            runtime_iter = np.zeros(shape=(n_runs,1))
            perf_padded = np.zeros(shape=(n_runs,1))
            # mem_footprint = np.zeros(shape=(n_runs,1))

            for run in range(0,n_runs):
                df = dataframes[run]
                runtime_iter[run] = np.asarray(df[df["matrix_name"]==matrix]["time[ms]/run"])[0]
                perf_padded[run] = np.asarray(df[df["matrix_name"]==matrix]["performance (padded)[GFLOPs]"])[0]
                # mem_footprint[run] = np.asarray(df[df["matrix_name"]==matrix]["mem_footprint[MB]"])[0]
            if(perf_padded[0][0]<0): # why? nobody knows why
                continue

            runtime_iter_hm = hmean(runtime_iter,axis=0)[0]
            perf_padded_hm = hmean(perf_padded,axis=0)[0]
            # if I want to show mem_footprint of Xilinx sparse double representation
            # mem_footprint = hmean(mem_footprint,axis=0)[0]

            # selected_gen = df_gen[df_gen["matrix_name"]==matrix]
            # mem_range = np.asarray(selected_gen["mem_range"])[0]
            # avg_nnz_per_row = np.asarray(selected_gen["avg_nnz_per_row"])[0]
            # std_nnz_per_row = np.asarray(selected_gen["std_nnz_per_row"])[0]
            # avg_bw = np.asarray(selected_gen["avg_bw"])[0]
            # std_bw = np.asarray(selected_gen["std_bw"])[0]
            # avg_sc = np.asarray(selected_gen["avg_sc"])[0]
            # std_sc = np.asarray(selected_gen["std_sc"])[0]/nr_cols

            W_avg = 33
            J_estimated = W_avg*runtime_iter_hm

            line_list = [matrix_name, distribution, placement, seed,
                         nr_rows, nr_cols, nr_nnz, density, mem_footprint, mem_range,
                         avg_nnz_per_row, std_nnz_per_row, 
                         avg_bw, std_bw, avg_bw_scaled, std_bw_scaled,
                         avg_sc, std_sc, avg_sc_scaled, std_sc_scaled,
                         skew_coeff, avg_num_neighbours, cross_row_similarity, 
                         "Xilinx_SpMV", runtime_iter_hm, perf_padded_hm, W_avg, J_estimated
                        ]
            line = ",".join(str(x) for x in line_list) + "\n"
            results_final.append(line)
        
    file = open(results_csv,"w")
    for line in results_final:
        file.write(line)
    file.close()
    print("Results for",len(results_final),"matrices for",results_csv.split("/")[-1])
    if(cntt>0):
        print(cntt, "matrices not found")

def extract_results_of_distr_memrange(distr_memrange):
    start_gen = "./generation_stats/"+distr_memrange+"_log.txt"
    results_gen = "./generation_stats/"+distr_memrange+"_log_CLEAN.txt"
    clean_matrix_generation_log(start_gen, results_gen)

    start_list = [
        "./dirty/"+distr_memrange+"_run1.txt",
    ]

    results_list = [
        "./clean/"+distr_memrange+"_run_CLEAN1.txt",
    ]
    clean_results(start_list, results_list)

    results_csv = "./results_"+distr_memrange+".csv"
    read_results(results_list, results_gen, results_csv)


In [619]:
%%time

distr_base = "normal"
# mem_ranges = ["4-8","8-16","16-32","32-64","64-128","128-256","256-512","512-1024","1024-2048"]
mem_ranges = ["4-32", "32-512", "512-2048"]

for mem_range in mem_ranges:
    distr_memrange = distr_base + "_" + mem_range
    extract_results_of_distr_memrange(distr_memrange)

filenames = ["results_"+distr_base+"_"+mr+".csv" for mr in mem_ranges]

with open('../Benchmarks/xilinx_spmv_4-2048_dtype-D.csv', 'w') as outfile:
    for fname in filenames:
        with open(fname) as infile:
            outfile.write(infile.read())

duplicates found : 780
Results for 2805 matrices for results_normal_4-32.csv
duplicates found : 100
Results for 2991 matrices for results_normal_32-512.csv
duplicates found : 0
Results for 1425 matrices for results_normal_512-2048.csv
CPU times: user 42.4 s, sys: 153 ms, total: 42.5 s
Wall time: 43.2 s


In [896]:
%%time
distr_memrange = "validation_matrices_10_samples_30_range_twins"
extract_results_of_distr_memrange(distr_memrange)

filename = "results_"+distr_memrange+".csv"

with open('../Benchmarks/xilinx_spmv_'+distr_memrange+'_dtype-D.csv', 'w') as outfile:
    with open(filename) as infile:
        outfile.write(infile.read())

duplicates found : 386
Results for 3502 matrices for results_validation_matrices_10_samples_30_range_twins.csv
CPU times: user 18.7 s, sys: 17.1 ms, total: 18.8 s
Wall time: 19.4 s


In [ ]:
# %%time
# distr_memrange = "validation_matrices_synthetic_small"
# extract_results_of_distr_memrange(distr_memrange)
# filename = "results_"+distr_memrange+".csv"

# with open('../Benchmarks/xilinx_spmv_'+distr_memrange+'_dtype-D.csv', 'w') as outfile:
#     with open(filename) as infile:
#             outfile.write(infile.read())

---

---

In [15]:
def read_results2(results_list, results_csv):
    dataframes = []
    for result_file in results_list:
        dataframes.append(read_results_single(result_file))
    # get matrices sorted by mem_footprint and then begin processing them (so that I do not have to sort them later...)
    matrices = dataframes[0]["matrix_name"].unique()
    # matrices = dataframes[0].groupby(["matrix name","mem_footprint[MB]"]).size().reset_index().sort_values("mem_footprint[MB]").reset_index(drop=True)["matrix name"]
    
    n_runs = len(results_list)
    results_final = []
    for matrix in matrices:
        nr_rows = np.asarray(dataframes[0][dataframes[0]["matrix_name"]==matrix]["original rows"])[0]
        nr_cols = np.asarray(dataframes[0][dataframes[0]["matrix_name"]==matrix]["original cols"])[0]        
        nr_nnz = np.asarray(dataframes[0][dataframes[0]["matrix_name"]==matrix]["original NNZs"])[0]
        density = nr_nnz/(nr_rows*nr_cols)*100
        runtime_iter = np.zeros(shape=(n_runs,1))
        perf_padded = np.zeros(shape=(n_runs,1))
        mem_footprint = np.zeros(shape=(n_runs,1))

        for run in range(0,n_runs):
            df = dataframes[run]
            runtime_iter[run] = np.asarray(df[df["matrix_name"]==matrix]["time[ms]/run"])[0]
            perf_padded[run] = np.asarray(df[df["matrix_name"]==matrix]["performance (padded)[GFLOPs]"])[0]
            mem_footprint[run] = np.asarray(df[df["matrix_name"]==matrix]["mem_footprint[MB]"])[0]
        if(perf_padded[0][0]<0): # why? nobody knows why
            continue

        runtime_iter_hm = hmean(runtime_iter,axis=0)[0]
        perf_padded_hm = hmean(perf_padded,axis=0)[0]
        mem_footprint = hmean(mem_footprint,axis=0)[0]

        W_avg = 33
        J_estimated = W_avg*runtime_iter_hm
        
        line_list = [matrix, nr_rows, nr_cols, nr_nnz, density, mem_footprint, 
                     "Xilinx_SpMV", runtime_iter_hm, perf_padded_hm, W_avg, J_estimated
                    ]
        line = ",".join(str(x) for x in line_list) + "\n"
        results_final.append(line)

    file = open(results_csv,"w")
    for line in results_final:
        file.write(line)
    file.close()

In [ ]:
# start_list = [
#     "./dirty/validation_matrices_run1.txt",
# ]

# results_list = [
#     "./clean/validation_matrices_run1.txt",
# ]
# clean_results(start_list, results_list)

# results_csv = "./results_validation_matrices.csv"
# read_results2(results_list, results_csv)

---

In [751]:
def split_equally(starter_file, nummer, extra):
    cnt_m = sum(1 for line in open(starter_file,"r"))
    print(cnt_m, nummer)
    if(cnt_m%nummer != 0):
        step = cnt_m//(nummer-1)
    else:
        step = cnt_m//(nummer)
    fp = open(starter_file,"r")
    lines = fp.readlines()
    cnt_file = 0
    for num in range(nummer):
        partial_file = starter_file.split(".txt")[0].replace(extra,"") + "_part" + str(num+1) + ".txt"
        fw = open(partial_file,"w")
        cnt_m = 0
        while(cnt_m < step and cnt_file < len(lines)):
            curr_line = lines[cnt_file]
            fw.write(curr_line)
            cnt_file += 1
            cnt_m += 1  # int(curr_line.split(" ")[0])
        fw.close()
        print(partial_file.split("/")[-1], "\t", cnt_file, "\t", cnt_m)
    fp.close()
    
# distr_memrange = "4-32"
extra = ""
# starter_file = "../matrix_generation_parameters/synthetic_matrices_small_dataset_"+extra+distr_memrange+".txt"
starter_file = "../matrix_generation_parameters/validation_matrices_10_samples_30_range_diagonal_twins.txt"
nummer = 34

split_equally(starter_file, nummer, extra)

2151 34
validation_matrices_10_samples_30_range_diagonal_twins_part1.txt 	 65 	 65
validation_matrices_10_samples_30_range_diagonal_twins_part2.txt 	 130 	 65
validation_matrices_10_samples_30_range_diagonal_twins_part3.txt 	 195 	 65
validation_matrices_10_samples_30_range_diagonal_twins_part4.txt 	 260 	 65
validation_matrices_10_samples_30_range_diagonal_twins_part5.txt 	 325 	 65
validation_matrices_10_samples_30_range_diagonal_twins_part6.txt 	 390 	 65
validation_matrices_10_samples_30_range_diagonal_twins_part7.txt 	 455 	 65
validation_matrices_10_samples_30_range_diagonal_twins_part8.txt 	 520 	 65
validation_matrices_10_samples_30_range_diagonal_twins_part9.txt 	 585 	 65
validation_matrices_10_samples_30_range_diagonal_twins_part10.txt 	 650 	 65
validation_matrices_10_samples_30_range_diagonal_twins_part11.txt 	 715 	 65
validation_matrices_10_samples_30_range_diagonal_twins_part12.txt 	 780 	 65
validation_matrices_10_samples_30_range_diagonal_twins_part13.txt 	 845 	 65
v

---

In [890]:
def clean_progress_log(start_gen, clean_gen, flag_write = True):
    if(flag_write):
        fw = open(clean_gen,"w")
    cnt, cnt2 = 0, 0
    with open(start_gen) as fp:
        for line in fp:
            # print(line)
            if( flag_write and ("%|" not in line) and not(line=="\n") and ("MEMORY USED :" not in line) and ("->" not in line)):
                fw.write(line)
            if("MATRIX FINISHED." in line):
                cnt+=1
            if("partition done." in line):
                cnt2+=1
    if(flag_write):
        fw.close()
    return cnt, cnt2
    
def clean_progress_log_wrapper(distr_memrange):
    prefix = "../matrix_generation_parameters/"
    max_ID = 35
    cnt_total = 0
    cnt2_total = 0
    cnt3_total = 0
    for ID in range(1,max_ID,1):
        # param_set = prefix+"synthetic_matrices_small_dataset_"+distr_memrange+"_part"+str(ID)+".txt"
        param_set = prefix+"validation_matrices_10_samples_30_range_diagonal_twins"+"_part"+str(ID)+".txt"
        if(os.path.exists(param_set)):
            cnt3 = sum(1 for line in open(param_set,"r"))
            cnt3_total += cnt3
            cnt, cnt2 = 0, 0
            start_gen = prefix+"progress_"+distr_memrange+"_"+str(ID)
            if(os.path.exists(start_gen)):
                clean_gen = prefix+"progress_"+distr_memrange+"_"+str(ID)+"_clean"
                cnt, cnt2 = clean_progress_log(start_gen, clean_gen)
            if(cnt3>0):
                readyness = "" if (cnt<cnt3) else "\tREADY "+str(ID)
                # print(ID,"\t",cnt,'/',cnt3, "ready.\t(", cnt2, "successful )\t", np.round(cnt/cnt3*100,2), "% completion\t", cnt3-cnt, "remaining",readyness)
                print(ID,"\t",cnt,'/',cnt3, "ready\t(", cnt2, "successful )\t", cnt3-cnt, "remaining",readyness)
            cnt_total += cnt
            cnt2_total += cnt2
            
    if(cnt_total>0):
        print(cnt_total,"/",cnt3_total,"ready\t\t(",cnt2_total,"successful - success rate", np.round(cnt2_total/cnt_total*100,2),"% )")
    print(cnt3_total-cnt_total, "remain")
    # final_log = "../FPGA_stuff/generation_stats/normal_" + distr_memrange + "_log.txt"
    final_log = "../FPGA_stuff/generation_stats/validation_matrices_10_samples_30_range_twins" + "_log.txt"
    cnt, cnt2 = clean_progress_log(final_log, "", False)
    print("\nFINAL_LOG\n\t",cnt, "ready\t(", cnt2, "successful - success rate", np.round(cnt2/cnt*100,2),"% )")

distr_memrange = ""
clean_progress_log_wrapper(distr_memrange)


0 remain

FINAL_LOG
	 4302 ready	( 3882 successful - success rate 90.24 % )


In [895]:








megali=["synthetic_1000005_1000005_2190550_avg2.19054_std25.45314_random_bw119819.1947_skew1501.82644_neigh0.0864_cross0.84356_n14.mtx","synthetic_1000005_1000005_2378888_avg2.37888_std25.47359_random_bw127104.78039_skew1514.4215_neigh0.103_cross0.88018_n14.mtx","synthetic_1000005_1000005_2551748_avg2.55174_std25.499_random_bw119896.54265_skew1534.81764_neigh0.1086_cross0.87541_n14.mtx","synthetic_1000005_1000005_2793946_avg2.79393_std25.52481_random_bw139709.49965_skew1513.71115_neigh0.10178_cross0.85786_n14.mtx","synthetic_1000005_1000005_3068666_avg3.06865_std25.54963_random_bw116338.41941_skew1480.43289_neigh0.11916_cross0.88158_n14.mtx","synthetic_1000005_1000005_3110917_avg3.1109_std33.11754_random_bw103532.18733_skew1510.459_neigh0.11814_cross0.85466_n14.mtx","synthetic_1000005_1000005_3116965_avg3.11695_std31.43703_random_bw107163.38446_skew1507.52625_neigh0.11804_cross0.86266_n14.mtx","synthetic_1000005_1000005_3121903_avg3.12189_std25.45302_random_bw110119.5356_skew1053.81063_neigh0.11769_cross0.86713_n14.mtx","synthetic_1000005_1000005_3127972_avg3.12796_std29.75772_random_bw112018.04327_skew1502.5376_neigh0.12161_cross0.87113_n14.mtx","synthetic_1000005_1000005_3132968_avg3.13295_std25.47429_random_bw113902.70335_skew1150.31021_neigh0.12143_cross0.87472_n14.mtx","synthetic_1000005_1000005_3136144_avg3.13613_std18.05255_random_bw135367.82089_skew1498.61976_neigh0.21685_cross0.93223_n14.mtx","synthetic_1000005_1000005_3144021_avg3.14401_std28.07823_random_bw116953.11281_skew1494.86263_neigh0.12731_cross0.88232_n14.mtx","synthetic_1000005_1000005_3144795_avg3.14478_std25.49712_random_bw117373.02426_skew1245.51038_neigh0.12684_cross0.88308_n14.mtx","synthetic_1000005_1000005_3156371_avg3.15636_std25.52223_random_bw120795.31056_skew1340.10381_neigh0.13144_cross0.89113_n14.mtx","synthetic_1000005_1000005_3160475_avg3.16046_std19.71515_random_bw135013.30133_skew1487.0749_neigh0.21383_cross0.93153_n14.mtx","synthetic_1000005_1000005_3162276_avg3.16226_std26.40103_random_bw122511.6205_skew1486.2274_neigh0.13472_cross0.89571_n14.mtx","synthetic_1000005_1000005_3166512_avg3.1665_std25.54915_random_bw123598.70118_skew1434.65625_neigh0.13874_cross0.8991_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_random_bw104073.79051_skew1482.18441_neigh0.14395_cross0.90267_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_random_bw112547.15199_skew1482.18441_neigh0.14364_cross0.90269_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_random_bw120556.18558_skew1482.18441_neigh0.14326_cross0.90269_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_random_bw121722.82145_skew1482.18441_neigh0.1869_cross0.90333_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_random_bw122150.26353_skew1482.18441_neigh0.05805_cross0.96156_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_random_bw122426.6617_skew1482.18441_neigh0.17652_cross0.90307_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_random_bw123138.58643_skew1482.18441_neigh0.16628_cross0.90292_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_random_bw123297.83473_skew1482.18441_neigh0.11813_cross0.93173_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_random_bw123697.2713_skew1482.18441_neigh0.1568_cross0.90295_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_random_bw124260.11141_skew1482.18441_neigh0.14853_cross0.90276_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_random_bw124950.43144_skew1482.18441_neigh0.13931_cross0.90257_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_random_bw125531.54795_skew1482.18441_neigh0.15898_cross0.87296_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_random_bw125571.10503_skew1482.18441_neigh0.12939_cross0.90242_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_random_bw126188.03769_skew1482.18441_neigh0.11966_cross0.90226_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_random_bw126648.45731_skew1482.18441_neigh0.11069_cross0.90206_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_random_bw127142.75789_skew1482.18441_neigh0.17733_cross0.81362_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_random_bw127147.10169_skew1482.18441_neigh0.10042_cross0.90174_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_random_bw128104.19842_skew1482.18441_neigh0.18795_cross0.75388_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_random_bw128465.7325_skew1482.18441_neigh0.19495_cross0.69438_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_random_bw128711.29975_skew1482.18441_neigh0.20001_cross0.63303_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_random_bw128912.58669_skew1482.18441_neigh0.14293_cross0.90268_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_random_bw137896.76343_skew1482.18441_neigh0.14246_cross0.90269_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_random_bw145843.60046_skew1482.18441_neigh0.14298_cross0.90268_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_random_bw154831.75347_skew1482.18441_neigh0.14358_cross0.90267_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_random_bw162157.8696_skew1482.18441_neigh0.14347_cross0.90267_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_random_bw87649.18931_skew1482.18441_neigh0.14287_cross0.90272_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_random_bw95510.6301_skew1482.18441_neigh0.14423_cross0.9027_n14.mtx","synthetic_1000005_1000005_3174621_avg3.17461_std25.57736_random_bw125956.24319_skew1529.58406_neigh0.14718_cross0.90575_n14.mtx","synthetic_1000005_1000005_3177914_avg3.1779_std24.72495_random_bw127016.47781_skew1478.909_neigh0.15361_cross0.90914_n14.mtx","synthetic_1000005_1000005_3179023_avg3.17901_std21.38191_random_bw134207.95085_skew1478.39273_neigh0.19999_cross0.92826_n14.mtx","synthetic_1000005_1000005_3180578_avg3.18056_std25.60874_random_bw128092.87982_skew1625.12766_neigh0.15702_cross0.91168_n14.mtx","synthetic_1000005_1000005_3184160_avg3.18414_std25.64113_random_bw130081.20538_skew1721.91199_neigh0.16859_cross0.9164_n14.mtx","synthetic_1000005_1000005_3185020_avg3.185_std23.052_random_bw131609.81549_skew1475.60722_neigh0.17602_cross0.92056_n14.mtx","synthetic_1000005_1000005_3185310_avg3.18529_std25.71304_random_bw132686.94329_skew1917.81812_neigh0.18507_cross0.92381_n14.mtx","synthetic_1000005_1000005_3185554_avg3.18554_std25.67645_random_bw131621.35039_skew1819.4146_neigh0.17691_cross0.9205_n14.mtx","synthetic_1000005_1000005_3238430_avg3.23841_std25.57761_random_bw128487.05961_skew1499.42591_neigh0.17003_cross0.91626_n14.mtx","synthetic_1000005_1000005_3338044_avg3.33803_std25.60993_random_bw120102.95798_skew1548.7177_neigh0.17012_cross0.91371_n14.mtx","synthetic_1000005_1000005_3554394_avg3.55438_std25.64567_random_bw113500.84038_skew1542.73078_neigh0.13151_cross0.88485_n14.mtx","synthetic_1000005_1000005_3911756_avg3.91174_std25.67945_random_bw133527.84178_skew1481.71748_neigh0.13986_cross0.88512_n14.mtx","synthetic_1000005_1000005_4154204_avg4.15418_std25.71363_random_bw158208.83055_skew1470.76946_neigh0.19591_cross0.92055_n14.mtx","synthetic_100993_100993_2185846_avg21.64354_std13.77891_random_bw54912.02009_skew4.03614_neigh1.01465_cross0.56058_n14.mtx","synthetic_1033338_1033338_3275830_avg3.17014_std25.55637_random_bw129053.64703_skew1482.52894_neigh0.14465_cross0.90234_n14.mtx","synthetic_109072_109072_2360564_avg21.64225_std13.77936_random_bw59445.13864_skew4.08265_neigh1.0157_cross0.55937_n14.mtx","synthetic_1100005_1100005_3487516_avg3.17045_std25.54298_random_bw137079.35531_skew1482.38345_neigh0.14227_cross0.90274_n14.mtx","synthetic_1166672_1166672_3699263_avg3.17078_std25.53274_random_bw145853.05163_skew1482.23015_neigh0.14123_cross0.90291_n14.mtx","synthetic_117152_117152_2535420_avg21.64214_std13.77711_random_bw63786.53108_skew4.08268_neigh1.01456_cross0.56071_n14.mtx","synthetic_119698_119698_670830_avg5.60435_std4.47718_random_bw38463.99053_skew62.8789_neigh0.60838_cross0.60967_n14.mtx","synthetic_121192_121192_1836052_avg15.14994_std12.36381_random_bw63604.68267_skew3.8185_neigh0.98085_cross0.56421_n14.mtx","synthetic_121192_121192_2010928_avg16.59291_std12.82631_random_bw63387.43514_skew3.94187_neigh0.99258_cross0.55956_n14.mtx","synthetic_121192_121192_2185768_avg18.03558_std13.23353_random_bw65140.22976_skew4.04558_neigh0.99835_cross0.55476_n14.mtx","synthetic_121192_121192_2360629_avg19.47842_std13.60481_random_bw64734.37292_skew4.13389_neigh1.00738_cross0.55096_n14.mtx","synthetic_121192_121192_2535413_avg20.92063_std13.77469_random_bw66438.45794_skew4.11457_neigh1.01319_cross0.55608_n14.mtx","synthetic_121192_121192_2622629_avg21.64028_std13.77138_random_bw65424.26549_skew3.85206_neigh1.01668_cross0.55626_n14.mtx","synthetic_121192_121192_2622663_avg21.64056_std13.63311_random_bw64870.55905_skew3.52853_neigh1.01573_cross0.55103_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_random_bw46185.35154_skew4.08299_neigh1.01504_cross0.5599_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_random_bw50573.38166_skew4.08299_neigh1.01509_cross0.56075_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_random_bw54972.82661_skew4.08299_neigh1.01519_cross0.5598_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_random_bw59411.92528_skew4.08299_neigh1.01561_cross0.56068_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_random_bw61692.98851_skew4.08299_neigh0.98464_cross0.72926_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_random_bw62953.41461_skew4.08299_neigh0.99544_cross0.69172_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_random_bw63724.66045_skew4.08299_neigh1.0157_cross0.55944_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_random_bw63863.58232_skew4.08299_neigh1.00127_cross0.65566_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_random_bw64806.59055_skew4.08299_neigh1.00789_cross0.61815_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_random_bw65523.57715_skew4.08299_neigh1.01329_cross0.58066_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_random_bw65666.88089_skew4.08299_neigh0.77995_cross0.56148_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_random_bw65781.74026_skew4.08299_neigh1.11633_cross0.56174_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_random_bw65912.90063_skew4.08299_neigh0.8475_cross0.55891_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_random_bw66094.34436_skew4.08299_neigh0.91339_cross0.55811_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_random_bw66297.82562_skew4.08299_neigh1.01673_cross0.5433_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_random_bw66303.01327_skew4.08299_neigh0.71059_cross0.55994_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_random_bw66344.93551_skew4.08299_neigh1.18402_cross0.56026_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_random_bw66632.5966_skew4.08299_neigh0.98135_cross0.55939_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_random_bw66915.08177_skew4.08299_neigh1.04961_cross0.5616_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_random_bw66999.72835_skew4.08299_neigh1.02006_cross0.50617_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_random_bw67029.04955_skew4.08299_neigh1.25119_cross0.56135_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_random_bw67482.8569_skew4.08299_neigh1.02304_cross0.46991_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_random_bw68050.39675_skew4.08299_neigh1.02635_cross0.4318_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_random_bw68172.92334_skew4.08299_neigh1.01556_cross0.56046_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_random_bw68534.78082_skew4.08299_neigh1.31693_cross0.56196_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_random_bw68594.96245_skew4.08299_neigh1.02897_cross0.39354_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_random_bw72555.87469_skew4.08299_neigh1.0152_cross0.56039_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_random_bw76889.20434_skew4.08299_neigh1.01471_cross0.5605_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_random_bw81259.52524_skew4.08299_neigh1.01612_cross0.56016_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_random_bw85714.50154_skew4.08299_neigh1.01527_cross0.56045_n14.mtx","synthetic_121192_121192_2622724_avg21.64107_std13.77599_random_bw65800.92968_skew3.99051_neigh1.01427_cross0.5606_n14.mtx","synthetic_121192_121192_2622736_avg21.64116_std14.23425_random_bw65388.53812_skew4.08291_neigh1.01644_cross0.55562_n14.mtx","synthetic_121192_121192_2622750_avg21.64128_std13.768_random_bw65046.54528_skew3.71321_neigh1.01557_cross0.55052_n14.mtx","synthetic_121192_121192_2622777_avg21.6415_std15.59481_random_bw64466.94691_skew4.08283_neigh1.01521_cross0.55446_n14.mtx","synthetic_121192_121192_2622792_avg21.64163_std13.77856_random_bw66101.8109_skew4.129_neigh1.01554_cross0.56191_n14.mtx","synthetic_121192_121192_2622812_avg21.64179_std16.1555_random_bw64390.16957_skew3.99034_neigh1.01546_cross0.55795_n14.mtx","synthetic_121192_121192_2622828_avg21.64192_std13.43338_random_bw65117.16008_skew3.29722_neigh1.01575_cross0.55268_n14.mtx","synthetic_121192_121192_2622837_avg21.642_std15.01875_random_bw64761.9232_skew4.12892_neigh1.01546_cross0.55076_n14.mtx","synthetic_121192_121192_2622876_avg21.64232_std16.69445_random_bw64326.40631_skew3.94402_neigh1.01578_cross0.55631_n14.mtx","synthetic_121192_121192_2622972_avg21.64311_std13.78148_random_bw66363.1673_skew4.31347_neigh1.01343_cross0.566_n14.mtx","synthetic_121192_121192_2623047_avg21.64373_std13.32114_random_bw66419.48364_skew4.0361_neigh1.01466_cross0.56615_n14.mtx","synthetic_121192_121192_2623109_avg21.64424_std13.78493_random_bw66631.25408_skew4.4518_neigh1.01539_cross0.56882_n14.mtx","synthetic_121192_121192_2623156_avg21.64463_std11.48856_random_bw68448.4577_skew3.89729_neigh1.01326_cross0.58356_n14.mtx","synthetic_121192_121192_2623189_avg21.6449_std13.78718_random_bw67331.69717_skew4.77503_neigh1.01343_cross0.57336_n14.mtx","synthetic_121192_121192_2623197_avg21.64497_std13.78635_random_bw66921.85212_skew4.59021_neigh1.01484_cross0.57212_n14.mtx","synthetic_121192_121192_2623249_avg21.6454_std12.40537_random_bw67456.81523_skew3.98951_neigh1.01486_cross0.57624_n14.mtx","synthetic_121192_121192_2623583_avg21.64815_std9.65791_random_bw70183.15791_skew3.8503_neigh1.01496_cross0.59679_n14.mtx","synthetic_121192_121192_2623613_avg21.6484_std10.57459_random_bw69356.8168_skew3.85024_neigh1.0146_cross0.5911_n14.mtx","synthetic_121192_121192_2710314_avg22.3638_std13.78076_random_bw66994.6287_skew4.05281_neigh1.01691_cross0.56487_n14.mtx","synthetic_121192_121192_2885490_avg23.80924_std13.78209_random_bw67434.57931_skew3.99806_neigh1.02152_cross0.5719_n14.mtx","synthetic_121192_121192_3060321_avg25.25184_std13.78548_random_bw68906.15443_skew3.95013_neigh1.02431_cross0.57782_n14.mtx","synthetic_121192_121192_3235599_avg26.69812_std13.78831_random_bw69042.29829_skew3.86926_neigh1.02849_cross0.58477_n14.mtx","synthetic_121192_121192_3410529_avg28.14154_std13.78985_random_bw69312.50343_skew3.86825_neigh1.03157_cross0.58839_n14.mtx","synthetic_1233339_1233339_3909668_avg3.16999_std25.52165_random_bw154195.41372_skew1482.60253_neigh0.14111_cross0.90327_n14.mtx","synthetic_125231_125231_2710355_avg21.64284_std13.77742_random_bw68141.31404_skew4.08251_neigh1.01464_cross0.56127_n14.mtx","synthetic_1300006_1300006_4120758_avg3.1698_std25.51264_random_bw162585.00942_skew1482.69019_neigh0.14075_cross0.90351_n14.mtx","synthetic_131098_131098_734679_avg5.60404_std4.47302_random_bw41879.66163_skew62.88244_neigh0.60932_cross0.60986_n14.mtx","synthetic_133311_133311_2884795_avg21.63959_std13.78017_random_bw72606.70986_skew4.08328_neigh1.01353_cross0.56105_n14.mtx","synthetic_141390_141390_3059797_avg21.64083_std13.77882_random_bw76927.21946_skew4.08298_neigh1.01482_cross0.56115_n14.mtx","synthetic_142498_142498_798541_avg5.60388_std4.46852_random_bw45521.18383_skew62.88436_neigh0.61099_cross0.60948_n14.mtx","synthetic_144550_144550_890999_avg6.16395_std4.44624_random_bw249.77523_skew7.27392_neigh0.18177_cross0.33337_n14.mtx","synthetic_14839_14839_1040332_avg70.10796_std6.31579_random_bw1146.69634_skew0.34079_neigh1.78621_cross0.91392_n14.mtx","synthetic_149470_149470_3234141_avg21.63739_std13.78086_random_bw81415.43761_skew4.08379_neigh1.01461_cross0.56019_n14.mtx","synthetic_153898_153898_862342_avg5.60333_std4.46589_random_bw49364.34344_skew62.89053_neigh0.61051_cross0.60952_n14.mtx","synthetic_157549_157549_3408821_avg21.63658_std13.78244_random_bw85829.82001_skew4.08398_neigh1.01388_cross0.56058_n14.mtx","synthetic_158316_158316_975794_avg6.16358_std4.44586_random_bw273.88663_skew7.27441_neigh0.17907_cross0.32957_n14.mtx","synthetic_16253_16253_1139653_avg70.11955_std6.30855_random_bw1286.71322_skew0.34057_neigh1.7819_cross0.91551_n14.mtx","synthetic_165298_165298_926272_avg5.60365_std4.4624_random_bw52893.04048_skew62.88694_neigh0.60862_cross0.6098_n14.mtx","synthetic_170998_170998_1008507_avg5.89777_std3.14995_random_bw47719.98025_skew59.87046_neigh0.63807_cross0.61706_n14.mtx","synthetic_170998_170998_1037773_avg6.06892_std4.43265_random_bw48509.54384_skew59.96638_neigh0.66301_cross0.62469_n14.mtx","synthetic_170998_170998_1043193_avg6.10062_std4.44271_random_bw48724.69743_skew63.58365_neigh0.67297_cross0.62698_n14.mtx","synthetic_170998_170998_1092151_avg6.38692_std4.47337_random_bw49002.39485_skew64.44623_neigh0.64099_cross0.61384_n14.mtx","synthetic_170998_170998_1211686_avg7.08597_std4.4649_random_bw52652.61671_skew64.76379_neigh0.68861_cross0.62669_n14.mtx","synthetic_170998_170998_1212281_avg7.08945_std4.45045_random_bw52659.56747_skew61.3462_neigh0.68857_cross0.62682_n14.mtx","synthetic_170998_170998_703429_avg4.11367_std4.4083_random_bw66949.59583_skew60.01611_neigh0.59084_cross0.62104_n14.mtx","synthetic_170998_170998_709969_avg4.15191_std4.42013_random_bw67041.59758_skew65.23451_neigh0.60621_cross0.62439_n14.mtx","synthetic_170998_170998_813246_avg4.75588_std4.44226_random_bw49547.73476_skew61.86954_neigh0.577_cross0.60721_n14.mtx","synthetic_170998_170998_875832_avg5.12188_std4.42226_random_bw52588.79732_skew62.06273_neigh0.64594_cross0.6236_n14.mtx","synthetic_170998_170998_882159_avg5.15888_std4.44207_random_bw52806.32402_skew66.06876_neigh0.63734_cross0.62272_n14.mtx","synthetic_170998_170998_944721_avg5.52475_std5.76053_random_bw53759.0296_skew63.79933_neigh0.61291_cross0.61003_n14.mtx","synthetic_170998_170998_945632_avg5.53008_std5.47116_random_bw53929.01913_skew63.7369_neigh0.61326_cross0.61142_n14.mtx","synthetic_170998_170998_946661_avg5.53609_std4.44002_random_bw54131.28407_skew44.88072_neigh0.61289_cross0.6094_n14.mtx","synthetic_170998_170998_947518_avg5.54111_std5.18255_random_bw54056.59899_skew63.60804_neigh0.61159_cross0.61005_n14.mtx","synthetic_170998_170998_948659_avg5.54778_std4.44165_random_bw54194.39357_skew48.9299_neigh0.61004_cross0.61069_n14.mtx","synthetic_170998_170998_950710_avg5.55977_std4.8941_random_bw54225.54433_skew63.39112_neigh0.60981_cross0.60991_n14.mtx","synthetic_170998_170998_951053_avg5.56178_std4.44852_random_bw54287.96161_skew53.11938_neigh0.6081_cross0.60983_n14.mtx","synthetic_170998_170998_953667_avg5.57707_std4.45309_random_bw54491.631_skew57.09507_neigh0.6098_cross0.60996_n14.mtx","synthetic_170998_170998_955297_avg5.5866_std4.60347_random_bw54609.35347_skew63.08194_neigh0.6098_cross0.61023_n14.mtx","synthetic_170998_170998_956707_avg5.59484_std4.45769_random_bw54693.36679_skew61.0214_neigh0.61022_cross0.60929_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_random_bw38321.91494_skew62.88573_neigh0.60926_cross0.60964_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_random_bw41929.48688_skew62.88573_neigh0.60983_cross0.6094_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_random_bw45628.56654_skew62.88573_neigh0.61026_cross0.60953_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_random_bw46903.96538_skew62.88573_neigh0.54722_cross0.60807_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_random_bw47843.37714_skew62.88573_neigh0.50741_cross0.60854_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_random_bw48523.58357_skew62.88573_neigh0.46873_cross0.60766_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_random_bw49181.26028_skew62.88573_neigh0.60917_cross0.60964_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_random_bw49272.78155_skew62.88573_neigh0.42658_cross0.60457_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_random_bw49297.41096_skew62.88573_neigh0.79067_cross0.61249_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_random_bw50512.55887_skew62.88573_neigh0.75263_cross0.61034_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_random_bw51880.4182_skew62.88573_neigh0.71336_cross0.6105_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_random_bw52905.44733_skew62.88573_neigh0.60897_cross0.60969_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_random_bw53028.52728_skew62.88573_neigh0.67336_cross0.60943_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_random_bw53125.78092_skew62.88573_neigh0.55319_cross0.78965_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_random_bw53660.70344_skew62.88573_neigh0.56909_cross0.75077_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_random_bw54099.133_skew62.88573_neigh0.58565_cross0.70928_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_random_bw54240.77009_skew62.88573_neigh0.63324_cross0.60935_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_random_bw54411.68621_skew62.88573_neigh0.59727_cross0.66902_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_random_bw54696.32419_skew62.88573_neigh0.60505_cross0.63095_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_random_bw54839.18485_skew62.88573_neigh0.61271_cross0.58999_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_random_bw54975.41519_skew62.88573_neigh0.61926_cross0.54811_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_random_bw55027.14865_skew62.88573_neigh0.62632_cross0.50615_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_random_bw55105.55545_skew62.88573_neigh0.63063_cross0.46746_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_random_bw55236.3212_skew62.88573_neigh0.63219_cross0.42616_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_random_bw55398.06761_skew62.88573_neigh0.58791_cross0.60932_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_random_bw56503.9465_skew62.88573_neigh0.60809_cross0.60967_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_random_bw60168.38237_skew62.88573_neigh0.60916_cross0.60962_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_random_bw64070.5493_skew62.88573_neigh0.60935_cross0.60965_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_random_bw67352.62944_skew62.88573_neigh0.60959_cross0.60963_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_random_bw71097.05963_skew62.88573_neigh0.60883_cross0.60961_n14.mtx","synthetic_170998_170998_959835_avg5.61314_std4.46266_random_bw54839.73947_skew64.91681_neigh0.60778_cross0.61012_n14.mtx","synthetic_170998_170998_961623_avg5.62359_std4.31537_random_bw54830.15481_skew62.66038_neigh0.60743_cross0.6101_n14.mtx","synthetic_170998_170998_963061_avg5.632_std4.4671_random_bw54958.67312_skew68.77981_neigh0.6102_cross0.6093_n14.mtx","synthetic_170998_170998_966597_avg5.65268_std4.47351_random_bw55278.98278_skew72.59341_neigh0.60946_cross0.60959_n14.mtx","synthetic_170998_170998_970106_avg5.6732_std4.48093_random_bw55352.2479_skew76.55763_neigh0.61119_cross0.61051_n14.mtx","synthetic_170998_170998_970315_avg5.67442_std4.02645_random_bw55373.82904_skew62.09011_neigh0.61062_cross0.61139_n14.mtx","synthetic_170998_170998_973557_avg5.69338_std4.48437_random_bw55499.03393_skew80.32248_neigh0.61487_cross0.61054_n14.mtx","synthetic_170998_170998_981215_avg5.73817_std3.73499_random_bw55852.32383_skew61.38927_neigh0.61949_cross0.61025_n14.mtx","synthetic_170998_170998_994346_avg5.81496_std3.44543_random_bw56542.54022_skew60.73734_neigh0.6252_cross0.61459_n14.mtx","synthetic_172083_172083_1060867_avg6.16486_std4.44424_random_bw297.78023_skew7.2727_neigh0.17596_cross0.32947_n14.mtx","synthetic_17666_17666_1238955_avg70.13217_std6.31199_random_bw1373.26344_skew0.34033_neigh1.78005_cross0.91685_n14.mtx","synthetic_176697_176697_990167_avg5.60376_std4.45819_random_bw56447.76777_skew62.88571_neigh0.60833_cross0.60971_n14.mtx","synthetic_185639_185639_15003294_avg80.81973_std87.6573_random_bw25664.94771_skew8.39127_neigh1.21561_cross0.82365_n14.mtx","synthetic_185639_185639_15003763_avg80.82226_std142.24862_random_bw20258.95107_skew7.19081_neigh1.26005_cross0.95405_n14.mtx","synthetic_185639_185639_15004156_avg80.82437_std94.60907_random_bw25577.96053_skew8.30412_neigh1.21738_cross0.82741_n14.mtx","synthetic_185639_185639_15004647_avg80.82702_std101.43013_random_bw25579.20672_skew8.20484_neigh1.21614_cross0.83107_n14.mtx","synthetic_185639_185639_15005689_avg80.83263_std108.09033_random_bw25508.21416_skew8.08049_neigh1.21648_cross0.83534_n14.mtx","synthetic_185639_185639_15006242_avg80.83561_std119.38413_random_bw25324.46226_skew8.66158_neigh1.21359_cross0.83963_n14.mtx","synthetic_185639_185639_15006431_avg80.83663_std114.55767_random_bw25341.18275_skew7.94397_neigh1.21465_cross0.84203_n14.mtx","synthetic_185639_185639_15006681_avg80.83798_std118.31728_random_bw25323.95677_skew8.14174_neigh1.2126_cross0.84303_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_random_bw17706.01026_skew7.8818_neigh1.2138_cross0.84521_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_random_bw19404.35313_skew7.8818_neigh1.21315_cross0.84474_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_random_bw21104.27872_skew7.8818_neigh1.21478_cross0.84522_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_random_bw22750.22312_skew7.8818_neigh1.21263_cross0.84532_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_random_bw24441.96582_skew7.8818_neigh1.21268_cross0.84507_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_random_bw26179.82365_skew7.8818_neigh1.21167_cross0.84498_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_random_bw27821.86705_skew7.8818_neigh1.21199_cross0.84555_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_random_bw29515.52781_skew7.8818_neigh1.21272_cross0.84497_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_random_bw31178.6333_skew7.8818_neigh1.21304_cross0.84523_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_random_bw32880.54075_skew7.8818_neigh1.21042_cross0.84511_n14.mtx","synthetic_185639_185639_15007305_avg80.84134_std120.8084_random_bw25218.21287_skew7.80738_neigh1.21325_cross0.84823_n14.mtx","synthetic_185639_185639_15008187_avg80.84609_std126.81395_random_bw25055.73053_skew7.64606_neigh1.20604_cross0.8579_n14.mtx","synthetic_185639_185639_15008702_avg80.84886_std137.983_random_bw24238.46969_skew7.32417_neigh1.20104_cross0.89187_n14.mtx","synthetic_185639_185639_15008859_avg80.84971_std132.54683_random_bw24711.02466_skew7.48488_neigh1.20312_cross0.87083_n14.mtx","synthetic_185639_185639_18505682_avg99.68639_std122.27517_random_bw25744.70036_skew8.23897_neigh1.22874_cross0.82863_n14.mtx","synthetic_185639_185639_19505424_avg105.0718_std123.33624_random_bw25716.49132_skew8.2984_neigh1.23114_cross0.82664_n14.mtx","synthetic_185850_185850_1145706_avg6.16468_std4.44467_random_bw321.67519_skew7.27293_neigh0.17292_cross0.32599_n14.mtx","synthetic_188097_188097_1054064_avg5.60383_std4.45656_random_bw60049.05112_skew62.88486_neigh0.60992_cross0.60918_n14.mtx","synthetic_19080_19080_1337993_avg70.12542_std6.31221_random_bw1489.05252_skew0.34046_neigh1.78444_cross0.91485_n14.mtx","synthetic_199497_199497_1118014_avg5.60416_std4.45398_random_bw64023.47114_skew62.88107_neigh0.60994_cross0.60927_n14.mtx","synthetic_199616_199616_1230682_avg6.16525_std4.44318_random_bw345.0451_skew7.27217_neigh0.1707_cross0.32355_n14.mtx","synthetic_20493_20493_1437302_avg70.13624_std6.31204_random_bw1605.41268_skew0.34025_neigh1.77707_cross0.91532_n14.mtx","synthetic_206500_206500_1060353_avg5.13488_std4.44292_random_bw335.27304_skew7.3741_neigh0.15993_cross0.3084_n14.mtx","synthetic_206500_206500_1145534_avg5.54738_std4.44341_random_bw356.84439_skew7.2922_neigh0.16358_cross0.31502_n14.mtx","synthetic_206500_206500_1230425_avg5.95847_std4.44436_random_bw374.92322_skew7.22358_neigh0.16711_cross0.31892_n14.mtx","synthetic_206500_206500_1272393_avg6.16171_std5.76962_random_bw331.0109_skew7.43922_neigh0.1768_cross0.30559_n14.mtx","synthetic_206500_206500_1272581_avg6.16262_std4.44309_random_bw352.77997_skew6.30209_neigh0.16944_cross0.31989_n14.mtx","synthetic_206500_206500_1272606_avg6.16274_std4.44382_random_bw350.20787_skew5.97741_neigh0.16939_cross0.31707_n14.mtx","synthetic_206500_206500_1272648_avg6.16294_std5.47591_random_bw338.0763_skew7.43753_neigh0.17479_cross0.31105_n14.mtx","synthetic_206500_206500_1272653_avg6.16297_std5.1813_random_bw344.41462_skew7.27523_neigh0.17252_cross0.31341_n14.mtx","synthetic_206500_206500_1272686_avg6.16313_std4.88791_random_bw350.39832_skew7.27502_neigh0.17198_cross0.31772_n14.mtx","synthetic_206500_206500_1272775_avg6.16356_std4.44367_random_bw347.43103_skew5.48976_neigh0.16995_cross0.31545_n14.mtx","synthetic_206500_206500_1272904_avg6.16418_std4.4462_random_bw361.82239_skew8.73365_neigh0.1696_cross0.32814_n14.mtx","synthetic_206500_206500_1272958_avg6.16445_std4.59126_random_bw354.96969_skew7.27325_neigh0.16996_cross0.32281_n14.mtx","synthetic_206500_206500_1272981_avg6.16456_std4.44526_random_bw362.54571_skew9.05749_neigh0.16989_cross0.32918_n14.mtx","synthetic_206500_206500_1272983_avg6.16457_std4.4441_random_bw354.47146_skew6.62422_neigh0.16961_cross0.32233_n14.mtx","synthetic_206500_206500_1272990_avg6.1646_std4.44515_random_bw357.90535_skew7.43526_neigh0.16959_cross0.32445_n14.mtx","synthetic_206500_206500_1272993_avg6.16462_std4.44589_random_bw359.27284_skew7.92189_neigh0.16953_cross0.32657_n14.mtx","synthetic_206500_206500_1272999_avg6.16464_std4.44572_random_bw360.70137_skew8.24628_neigh0.16983_cross0.32797_n14.mtx","synthetic_206500_206500_1273034_avg6.16481_std4.29748_random_bw359.14318_skew7.27276_neigh0.17002_cross0.32629_n14.mtx","synthetic_206500_206500_1273051_avg6.1649_std3.71025_random_bw366.49966_skew7.27265_neigh0.16733_cross0.33042_n14.mtx","synthetic_206500_206500_1273053_avg6.16491_std4.44447_random_bw356.03003_skew7.11042_neigh0.1699_cross0.32349_n14.mtx","synthetic_206500_206500_1273074_avg6.16501_std4.00298_random_bw362.98579_skew7.2725_neigh0.16799_cross0.32983_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_random_bw250.53829_skew7.27211_neigh0.18184_cross0.33208_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_random_bw272.80182_skew7.27211_neigh0.17887_cross0.32932_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_random_bw297.33833_skew7.27211_neigh0.17494_cross0.32838_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_random_bw321.61163_skew7.27211_neigh0.17327_cross0.32563_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_random_bw345.2926_skew7.27211_neigh0.17126_cross0.32436_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_random_bw353.07637_skew7.27211_neigh0.1683_cross0.41181_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_random_bw354.05036_skew7.27211_neigh0.1697_cross0.39234_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_random_bw355.04431_skew7.27211_neigh0.16961_cross0.37276_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_random_bw355.0493_skew7.27211_neigh0.193_cross0.3226_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_random_bw356.01345_skew7.27211_neigh0.16921_cross0.35329_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_random_bw356.02066_skew7.27211_neigh0.18376_cross0.32185_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_random_bw356.56768_skew7.27211_neigh0.17513_cross0.32339_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_random_bw357.15197_skew7.27211_neigh0.16977_cross0.3328_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_random_bw357.48545_skew7.27211_neigh0.16999_cross0.31374_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_random_bw357.59221_skew7.27211_neigh0.16543_cross0.32304_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_random_bw358.45388_skew7.27211_neigh0.17021_cross0.29537_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_random_bw358.52034_skew7.27211_neigh0.15599_cross0.32162_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_random_bw358.88324_skew7.27211_neigh0.17043_cross0.27574_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_random_bw359.59658_skew7.27211_neigh0.17102_cross0.25585_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_random_bw359.63777_skew7.27211_neigh0.14622_cross0.32225_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_random_bw360.3102_skew7.27211_neigh0.17173_cross0.23608_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_random_bw360.37211_skew7.27211_neigh0.13735_cross0.3228_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_random_bw361.09723_skew7.27211_neigh0.12774_cross0.32289_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_random_bw368.19459_skew7.27211_neigh0.16931_cross0.32373_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_random_bw378.37779_skew7.27211_neigh0.21109_cross0.3219_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_random_bw379.42115_skew7.27211_neigh0.20197_cross0.32156_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_random_bw392.56075_skew7.27211_neigh0.16763_cross0.32173_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_random_bw416.95627_skew7.27211_neigh0.16635_cross0.32052_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_random_bw440.8278_skew7.27211_neigh0.16504_cross0.3198_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_random_bw464.28694_skew7.27211_neigh0.16438_cross0.31804_n14.mtx","synthetic_206500_206500_1273351_avg6.16635_std3.41298_random_bw369.41645_skew7.2707_neigh0.16589_cross0.33282_n14.mtx","synthetic_206500_206500_1274354_avg6.17121_std3.11913_random_bw372.02783_skew7.10214_neigh0.16567_cross0.33662_n14.mtx","synthetic_206500_206500_1315280_avg6.36939_std4.44461_random_bw364.31629_skew7.32104_neigh0.1716_cross0.32663_n14.mtx","synthetic_206500_206500_1400260_avg6.78092_std4.44568_random_bw375.96645_skew7.25847_neigh0.17353_cross0.33102_n14.mtx","synthetic_206500_206500_1485240_avg7.19245_std4.44584_random_bw368.40782_skew7.20305_neigh0.17827_cross0.33719_n14.mtx","synthetic_206500_206500_1570037_avg7.60308_std4.44659_random_bw376.86972_skew7.15458_neigh0.18053_cross0.33804_n14.mtx","synthetic_206500_206500_1655223_avg8.01561_std4.44684_random_bw384.89388_skew7.23394_neigh0.18176_cross0.34264_n14.mtx","synthetic_206500_206500_890883_avg4.3142_std4.3742_random_bw314.87298_skew7.34453_neigh0.15043_cross0.29772_n14.mtx","synthetic_206500_206500_975718_avg4.72503_std4.44295_random_bw342.64695_skew7.46556_neigh0.15368_cross0.29915_n14.mtx","synthetic_210897_210897_1181860_avg5.60397_std4.45232_random_bw67447.98409_skew62.88331_neigh0.61013_cross0.60909_n14.mtx","synthetic_21200_21200_1040239_avg49.06788_std6.3116_random_bw2270.10316_skew0.48774_neigh1.73665_cross0.90005_n14.mtx","synthetic_21200_21200_1139550_avg53.75236_std6.31054_random_bw2327.97646_skew0.4325_neigh1.747_cross0.90406_n14.mtx","synthetic_21200_21200_1238795_avg58.43373_std6.30753_random_bw2370.53675_skew0.4033_neigh1.76263_cross0.90541_n14.mtx","synthetic_21200_21200_1338002_avg63.1133_std6.31089_random_bw1621.46392_skew0.37847_neigh1.76689_cross0.91021_n14.mtx","synthetic_21200_21200_1437342_avg67.79915_std6.31153_random_bw1625.12618_skew0.35695_neigh1.78043_cross0.91191_n14.mtx","synthetic_21200_21200_1486357_avg70.11118_std8.20133_random_bw1621.03675_skew0.44057_neigh1.79499_cross0.90156_n14.mtx","synthetic_21200_21200_1486477_avg70.11684_std7.77973_random_bw1623.75863_skew0.41193_neigh1.78416_cross0.90649_n14.mtx","synthetic_21200_21200_1486586_avg70.12198_std7.35959_random_bw1633.7316_skew0.39756_neigh1.78517_cross0.90911_n14.mtx","synthetic_21200_21200_1486691_avg70.12693_std6.94018_random_bw1638.06472_skew0.36895_neigh1.78923_cross0.90946_n14.mtx","synthetic_21200_21200_1486777_avg70.13099_std6.51934_random_bw1665.03075_skew0.35461_neigh1.77737_cross0.91541_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_random_bw1091.14792_skew0.3403_neigh1.27638_cross0.91529_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_random_bw1099.35217_skew0.3403_neigh1.38725_cross0.91539_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_random_bw1129.65231_skew0.3403_neigh1.4963_cross0.91405_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_random_bw1150.42368_skew0.3403_neigh1.6111_cross0.91717_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_random_bw1155.31632_skew0.3403_neigh1.7853_cross0.91405_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_random_bw1291.37047_skew0.3403_neigh1.78149_cross0.91544_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_random_bw1309.76505_skew0.3403_neigh1.7269_cross0.91625_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_random_bw1369.88212_skew0.3403_neigh1.7807_cross0.91642_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_random_bw1489.33288_skew0.3403_neigh1.78357_cross0.91451_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_random_bw1563.73939_skew0.3403_neigh1.71868_cross0.94884_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_random_bw1590.50175_skew0.3403_neigh1.72943_cross0.94422_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_random_bw1604.78255_skew0.3403_neigh1.77801_cross0.91536_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_random_bw1650.62693_skew0.3403_neigh1.86722_cross0.64927_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_random_bw1663.23962_skew0.3403_neigh1.86073_cross0.70503_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_random_bw1668.34939_skew0.3403_neigh1.77532_cross0.91565_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_random_bw1670.25646_skew0.3403_neigh1.85028_cross0.77171_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_random_bw1674.10208_skew0.3403_neigh1.83639_cross0.82849_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_random_bw1677.25429_skew0.3403_neigh1.80579_cross0.88625_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_random_bw1710.35316_skew0.3403_neigh1.78264_cross0.91384_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_random_bw1844.95325_skew0.3403_neigh1.77183_cross0.91655_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_random_bw1925.18679_skew0.3403_neigh1.77933_cross0.91379_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_random_bw2023.01226_skew0.3403_neigh1.85164_cross0.91467_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_random_bw2027.65887_skew0.3403_neigh1.77965_cross0.91342_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_random_bw2126.54439_skew0.3403_neigh1.7794_cross0.91462_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_random_bw2183.93132_skew0.3403_neigh1.83524_cross0.91315_n14.mtx","synthetic_21200_21200_1487000_avg70.14151_std6.10324_random_bw1671.56_skew0.32589_neigh1.77648_cross0.91706_n14.mtx","synthetic_21200_21200_1487077_avg70.14514_std5.67801_random_bw1678.95703_skew0.29731_neigh1.77486_cross0.91968_n14.mtx","synthetic_21200_21200_1487192_avg70.15057_std5.25576_random_bw1669.85401_skew0.28295_neigh1.78043_cross0.92058_n14.mtx","synthetic_21200_21200_1487315_avg70.15637_std4.84276_random_bw1691.18377_skew0.25434_neigh1.76784_cross0.925_n14.mtx","synthetic_21200_21200_1487478_avg70.16406_std4.4186_random_bw1723.39901_skew0.23995_neigh1.76461_cross0.92734_n14.mtx","synthetic_21200_21200_1536543_avg72.47844_std6.30853_random_bw1661.22915_skew0.32453_neigh1.78331_cross0.91588_n14.mtx","synthetic_21200_21200_1635721_avg77.15665_std6.31046_random_bw1691.41788_skew0.30903_neigh1.78778_cross0.91802_n14.mtx","synthetic_21200_21200_1735090_avg81.84387_std6.31393_random_bw1717.89467_skew0.29515_neigh1.79144_cross0.92217_n14.mtx","synthetic_21200_21200_1834315_avg86.52429_std6.30851_random_bw1449.33458_skew0.27132_neigh1.8011_cross0.92178_n14.mtx","synthetic_21200_21200_1933465_avg91.20118_std6.31043_random_bw1477.8358_skew0.26095_neigh1.80472_cross0.92273_n14.mtx","synthetic_213383_213383_1315556_avg6.16523_std4.44392_random_bw368.70315_skew7.27219_neigh0.16899_cross0.32206_n14.mtx","synthetic_21906_21906_1536320_avg70.13238_std6.31756_random_bw1709.96585_skew0.34032_neigh1.77856_cross0.91434_n14.mtx","synthetic_222297_222297_1245675_avg5.60365_std4.45068_random_bw71187.12274_skew62.88691_neigh0.61086_cross0.60946_n14.mtx","synthetic_227150_227150_1400378_avg6.16499_std4.4444_random_bw392.60485_skew7.27252_neigh0.16823_cross0.32207_n14.mtx","synthetic_23320_23320_1635398_avg70.12856_std6.31946_random_bw1840.50772_skew0.3404_neigh1.7719_cross0.91648_n14.mtx","synthetic_240916_240916_1485276_avg6.16512_std4.4437_random_bw416.93905_skew7.27235_neigh0.16633_cross0.32069_n14.mtx","synthetic_24733_24733_1734543_avg70.13072_std6.31636_random_bw1933.712_skew0.34035_neigh1.7778_cross0.91451_n14.mtx","synthetic_254683_254683_1570192_avg6.16528_std4.4433_random_bw441.04844_skew7.27213_neigh0.16638_cross0.31989_n14.mtx","synthetic_26146_26146_1833536_avg70.12683_std6.31243_random_bw2034.00092_skew0.34043_neigh1.7785_cross0.91319_n14.mtx","synthetic_268450_268450_1655008_avg6.16505_std4.44456_random_bw465.05871_skew7.27244_neigh0.16485_cross0.31978_n14.mtx","synthetic_27120_27120_1234696_avg45.52714_std31.4396_random_bw657.74856_skew2.64618_neigh1.22988_cross0.77937_n14.mtx","synthetic_27560_27560_1933075_avg70.1406_std6.31297_random_bw2125.47631_skew0.34017_neigh1.77961_cross0.91464_n14.mtx","synthetic_29703_29703_1352183_avg45.52345_std31.43457_random_bw718.17419_skew2.64647_neigh1.22769_cross0.77551_n14.mtx","synthetic_32286_32286_1469958_avg45.52927_std31.42835_random_bw783.22802_skew2.66797_neigh1.22513_cross0.77351_n14.mtx","synthetic_32784_32784_1657863_avg50.56927_std27.82469_random_bw5242.21303_skew3.09339_neigh1.63697_cross0.76668_n14.mtx","synthetic_34406_34406_1341834_avg39.0_std0.0_random_bw8401.7213_skew0.0_neigh1.4059_cross0.80649_n14.mtx","synthetic_34869_34869_1587838_avg45.53724_std31.4194_random_bw844.81465_skew2.66733_neigh1.22431_cross0.77337_n14.mtx","synthetic_35906_35906_1816148_avg50.58063_std27.80623_random_bw5716.92046_skew3.09248_neigh1.63607_cross0.7699_n14.mtx","synthetic_368077_368077_1472308_avg4.0_std0.0_random_bw504.70969_skew0.0_neigh0.37372_cross0.9891_n14.mtx","synthetic_37452_37452_1706036_avg45.5526_std31.4087_random_bw906.67014_skew2.66609_neigh1.22181_cross0.77055_n14.mtx","synthetic_37683_37683_1469637_avg39.0_std0.0_random_bw9214.2749_skew0.0_neigh1.40447_cross0.80565_n14.mtx","synthetic_38120_38120_14004492_avg367.37912_std206.50003_random_bw17131.47558_skew1.32185_neigh1.97443_cross0.91972_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_random_bw12163.96299_skew1.57045_neigh1.97477_cross0.91968_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_random_bw13487.94121_skew1.57045_neigh1.97466_cross0.91853_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_random_bw14593.25708_skew1.57045_neigh1.97453_cross0.91966_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_random_bw15458.63701_skew1.57045_neigh1.97488_cross0.91833_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_random_bw16697.59053_skew1.57045_neigh1.97466_cross0.91971_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_random_bw17311.49276_skew1.57045_neigh1.97473_cross0.91875_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_random_bw17930.28565_skew1.57045_neigh1.97458_cross0.91885_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_random_bw18890.56044_skew1.57045_neigh1.97464_cross0.91976_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_random_bw19998.78673_skew1.57045_neigh1.97496_cross0.91885_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_random_bw21254.44919_skew1.57045_neigh1.9747_cross0.9186_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_random_bw22981.65732_skew1.57045_neigh1.97438_cross0.91938_n14.mtx","synthetic_38120_38120_15009192_avg393.73536_std249.23315_random_bw20164.63922_skew1.65915_neigh1.95556_cross0.89934_n14.mtx","synthetic_38120_38120_15509350_avg406.85598_std254.10081_random_bw18952.83022_skew1.83638_neigh1.9601_cross0.87267_n14.mtx","synthetic_38120_38120_15831412_avg415.30462_std256.81933_random_bw18361.46692_skew2.01225_neigh1.9625_cross0.85222_n14.mtx","synthetic_38120_38120_16013257_avg420.07495_std256.51996_random_bw18132.49037_skew2.18991_neigh1.96341_cross0.83427_n14.mtx","synthetic_38120_38120_17911818_avg469.8798_std294.64729_random_bw17768.78305_skew1.77305_neigh1.96177_cross0.87832_n14.mtx","synthetic_38120_38120_17943763_avg470.71781_std404.16901_random_bw13028.28431_skew3.74169_neigh1.974_cross0.55914_n14.mtx","synthetic_38120_38120_19284934_avg505.90068_std314.8817_random_bw17404.61952_skew1.89583_neigh1.96456_cross0.86292_n14.mtx","synthetic_38120_38120_20568467_avg539.57154_std333.79776_random_bw17206.90546_skew2.00609_neigh1.96636_cross0.85062_n14.mtx","synthetic_38744_38744_1161488_avg29.97853_std22.5272_random_bw1034.84256_skew1.93543_neigh1.22772_cross0.85244_n14.mtx","synthetic_38744_38744_1307882_avg33.75702_std24.88267_random_bw998.77878_skew2.08084_neigh1.17487_cross0.82838_n14.mtx","synthetic_38744_38744_1449216_avg37.40491_std27.07152_random_bw962.94693_skew2.28834_neigh1.19126_cross0.80394_n14.mtx","synthetic_38744_38744_1579684_avg40.77235_std28.98643_random_bw952.64469_skew2.45823_neigh1.20264_cross0.78378_n14.mtx","synthetic_38744_38744_1703850_avg43.97713_std30.65815_random_bw943.0718_skew2.61552_neigh1.21338_cross0.77875_n14.mtx","synthetic_38744_38744_1707520_avg44.07186_std32.46486_random_bw995.81409_skew2.10856_neigh1.20504_cross0.82864_n14.mtx","synthetic_38744_38744_1730367_avg44.66155_std32.51286_random_bw976.76174_skew2.22425_neigh1.21085_cross0.80896_n14.mtx","synthetic_38744_38744_1736329_avg44.81543_std26.74959_random_bw958.24236_skew1.96773_neigh1.21246_cross0.79103_n14.mtx","synthetic_38744_38744_1746236_avg45.07113_std32.41542_random_bw952.99461_skew2.35026_neigh1.21614_cross0.79836_n14.mtx","synthetic_38744_38744_1747768_avg45.11068_std28.0118_random_bw950.83561_skew2.12565_neigh1.21055_cross0.78825_n14.mtx","synthetic_38744_38744_1755242_avg45.30358_std29.13201_random_bw947.62389_skew2.28892_neigh1.21485_cross0.78295_n14.mtx","synthetic_38744_38744_1756611_avg45.33892_std32.15335_random_bw944.97597_skew2.48486_neigh1.21552_cross0.78836_n14.mtx","synthetic_38744_38744_1760377_avg45.43612_std30.12952_random_bw943.2046_skew2.43339_neigh1.21615_cross0.78132_n14.mtx","synthetic_38744_38744_1762627_avg45.49419_std31.70473_random_bw945.51417_skew2.60486_neigh1.21499_cross0.77366_n14.mtx","synthetic_38744_38744_1763457_avg45.51562_std31.01391_random_bw938.21335_skew2.60316_neigh1.21504_cross0.77558_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_random_bw1012.91606_skew2.6669_neigh1.23666_cross0.60626_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_random_bw1028.13896_skew2.6669_neigh1.24007_cross0.5571_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_random_bw1030.11568_skew2.6669_neigh1.2189_cross0.77356_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_random_bw1090.87397_skew2.6669_neigh1.21727_cross0.76922_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_random_bw1152.64756_skew2.6669_neigh1.2162_cross0.77082_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_random_bw1218.42621_skew2.6669_neigh1.21339_cross0.77016_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_random_bw659.5718_skew2.6669_neigh1.23127_cross0.77639_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_random_bw720.8265_skew2.6669_neigh1.23049_cross0.7745_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_random_bw783.97236_skew2.6669_neigh1.22458_cross0.77252_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_random_bw790.44606_skew2.6669_neigh1.18687_cross0.89462_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_random_bw799.34238_skew2.6669_neigh1.18845_cross0.8908_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_random_bw843.39606_skew2.6669_neigh1.22114_cross0.77348_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_random_bw875.30678_skew2.6669_neigh1.20367_cross0.84315_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_random_bw905.63736_skew2.6669_neigh1.21901_cross0.77413_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_random_bw922.58182_skew2.6669_neigh1.21558_cross0.79594_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_random_bw932.78459_skew2.6669_neigh1.03232_cross0.77348_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_random_bw935.45003_skew2.6669_neigh1.17793_cross0.77396_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_random_bw936.96549_skew2.6669_neigh1.25866_cross0.77239_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_random_bw939.4777_skew2.6669_neigh0.88501_cross0.77405_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_random_bw940.09284_skew2.6669_neigh0.96014_cross0.7732_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_random_bw940.09785_skew2.6669_neigh1.10807_cross0.77234_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_random_bw943.56437_skew2.6669_neigh1.32906_cross0.77341_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_random_bw946.71836_skew2.6669_neigh1.40773_cross0.77338_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_random_bw952.76706_skew2.6669_neigh1.22251_cross0.74756_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_random_bw956.49716_skew2.6669_neigh1.48195_cross0.77307_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_random_bw963.6331_skew2.6669_neigh1.22154_cross0.77193_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_random_bw974.85482_skew2.6669_neigh1.23062_cross0.69785_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_random_bw975.63003_skew2.6669_neigh1.56256_cross0.77213_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_random_bw995.95674_skew2.6669_neigh1.23589_cross0.65094_n14.mtx","synthetic_38744_38744_1765368_avg45.56494_std31.78636_random_bw934.44368_skew2.75289_neigh1.21894_cross0.77198_n14.mtx","synthetic_38744_38744_1765877_avg45.57808_std31.07073_random_bw933.8017_skew2.72986_neigh1.22255_cross0.76872_n14.mtx","synthetic_38744_38744_1766496_avg45.59405_std32.481_random_bw931.2818_skew2.88209_neigh1.21975_cross0.76797_n14.mtx","synthetic_38744_38744_1767073_avg45.60895_std26.86577_random_bw926.3303_skew3.122_neigh1.22395_cross0.74655_n14.mtx","synthetic_38744_38744_1767160_avg45.61119_std30.25452_random_bw933.41929_skew2.83678_neigh1.21906_cross0.76087_n14.mtx","synthetic_38744_38744_1767330_avg45.61558_std33.08901_random_bw930.75617_skew3.03371_neigh1.22002_cross0.7649_n14.mtx","synthetic_38744_38744_1767431_avg45.61819_std29.2665_random_bw931.34116_skew2.94579_neigh1.22054_cross0.7542_n14.mtx","synthetic_38744_38744_1767460_avg45.61893_std28.1289_random_bw924.86287_skew3.03341_neigh1.22219_cross0.75561_n14.mtx","synthetic_38744_38744_1767651_avg45.62386_std33.63482_random_bw928.32175_skew3.16449_neigh1.22329_cross0.76327_n14.mtx","synthetic_38744_38744_1767760_avg45.62668_std34.12282_random_bw928.16813_skew3.31765_neigh1.22071_cross0.76095_n14.mtx","synthetic_38744_38744_1824565_avg47.09284_std32.11946_random_bw938.10585_skew2.7373_neigh1.22202_cross0.76943_n14.mtx","synthetic_38744_38744_1943737_avg50.16872_std33.41096_random_bw937.69409_skew2.82709_neigh1.22997_cross0.76198_n14.mtx","synthetic_38744_38744_2062005_avg53.22127_std34.5782_random_bw928.18411_skew2.90821_neigh1.23817_cross0.75999_n14.mtx","synthetic_38744_38744_2179882_avg56.26373_std35.63951_random_bw931.14637_skew2.98125_neigh1.24047_cross0.75443_n14.mtx","synthetic_38744_38744_2297582_avg59.30162_std36.62621_random_bw928.90731_skew3.03024_neigh1.24558_cross0.75336_n14.mtx","synthetic_39029_39029_1974236_avg50.58382_std27.80163_random_bw6199.88865_skew3.11199_neigh1.63568_cross0.76891_n14.mtx","synthetic_40035_40035_1824084_avg45.56223_std31.3971_random_bw969.35456_skew2.66532_neigh1.21894_cross0.77386_n14.mtx","synthetic_403132_403132_1612528_avg4.0_std0.0_random_bw553.55446_skew0.0_neigh0.37461_cross0.98908_n14.mtx","synthetic_40960_40960_1597440_avg39.0_std0.0_random_bw9995.91001_skew0.0_neigh1.40666_cross0.80631_n14.mtx","synthetic_42151_42151_2133812_avg50.62305_std27.78297_random_bw6697.26877_skew3.12855_neigh1.63324_cross0.77066_n14.mtx","synthetic_42618_42618_1942443_avg45.578_std31.38541_random_bw1030.63919_skew2.66405_neigh1.21482_cross0.7709_n14.mtx","synthetic_43715_43715_2801362_avg64.0824_std14.01594_random_bw314.71989_skew0.93501_neigh1.5866_cross0.83319_n14.mtx","synthetic_438187_438187_1752748_avg4.0_std0.0_random_bw605.4017_skew0.0_neigh0.36958_cross0.98958_n14.mtx","synthetic_44236_44236_1725204_avg39.0_std0.0_random_bw10814.82767_skew0.0_neigh1.40668_cross0.80597_n14.mtx","synthetic_45201_45201_2060190_avg45.57842_std31.38204_random_bw1097.18524_skew2.68596_neigh1.21466_cross0.77308_n14.mtx","synthetic_45273_45273_2292008_avg50.62638_std27.77577_random_bw7230.17452_skew3.12828_neigh1.63423_cross0.77066_n14.mtx","synthetic_46835_46835_1659331_avg35.42929_std24.19453_random_bw7661.97799_skew2.92331_neigh1.59887_cross0.77081_n14.mtx","synthetic_46835_46835_1817944_avg38.81593_std25.43637_random_bw7818.83557_skew3.04473_neigh1.61137_cross0.76248_n14.mtx","synthetic_46835_46835_1976036_avg42.19144_std26.52871_random_bw7433.66969_skew3.14776_neigh1.62142_cross0.75764_n14.mtx","synthetic_46835_46835_2134025_avg45.56475_std27.52318_random_bw7560.21004_skew3.21378_neigh1.62711_cross0.75172_n14.mtx","synthetic_46835_46835_2292218_avg48.94241_std27.77072_random_bw7378.23094_skew3.16816_neigh1.63162_cross0.7647_n14.mtx","synthetic_46835_46835_2370905_avg50.6225_std27.74621_random_bw7334.7539_skew2.81253_neigh1.63507_cross0.75275_n14.mtx","synthetic_46835_46835_2371001_avg50.62455_std27.39541_random_bw7354.04117_skew2.69386_neigh1.63657_cross0.75406_n14.mtx","synthetic_46835_46835_2371113_avg50.62695_std32.27909_random_bw7361.07123_skew3.10848_neigh1.63754_cross0.75502_n14.mtx","synthetic_46835_46835_2371156_avg50.62786_std27.75732_random_bw7391.41134_skew2.91089_neigh1.63514_cross0.76051_n14.mtx","synthetic_46835_46835_2371164_avg50.62803_std30.41679_random_bw7309.21574_skew3.22691_neigh1.63831_cross0.75152_n14.mtx","synthetic_46835_46835_2371201_avg50.62882_std33.12436_random_bw7335.62733_skew3.04908_neigh1.63667_cross0.75847_n14.mtx","synthetic_46835_46835_2371202_avg50.62885_std27.76569_random_bw7416.01217_skew3.00957_neigh1.63418_cross0.76445_n14.mtx","synthetic_46835_46835_2371250_avg50.62987_std31.37317_random_bw7354.67208_skew3.1675_neigh1.63654_cross0.75207_n14.mtx","synthetic_46835_46835_2371308_avg50.63111_std28.69655_random_bw7409.30012_skew3.1674_neigh1.638_cross0.76266_n14.mtx","synthetic_46835_46835_2371384_avg50.63273_std27.77434_random_bw7430.84482_skew3.08826_neigh1.63371_cross0.76567_n14.mtx","synthetic_46835_46835_2371440_avg50.63393_std27.7783_random_bw7484.63769_skew3.18692_neigh1.63258_cross0.77158_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_random_bw5235.2982_skew3.12766_neigh1.63485_cross0.7661_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_random_bw5745.43256_skew3.12766_neigh1.63461_cross0.76806_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_random_bw6229.28771_skew3.12766_neigh1.63338_cross0.76748_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_random_bw6661.50364_skew3.12766_neigh1.555_cross0.88602_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_random_bw6744.12087_skew3.12766_neigh1.63266_cross0.76858_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_random_bw7030.71878_skew3.12766_neigh1.60145_cross0.84428_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_random_bw7051.92288_skew3.12766_neigh1.1472_cross0.76824_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_random_bw7076.9514_skew3.12766_neigh1.25341_cross0.77005_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_random_bw7108.27014_skew3.12766_neigh1.36391_cross0.76831_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_random_bw7209.39063_skew3.12766_neigh1.57945_cross0.76788_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_random_bw7217.16063_skew3.12766_neigh1.47014_cross0.76969_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_random_bw7237.79114_skew3.12766_neigh1.63478_cross0.7678_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_random_bw7362.32181_skew3.12766_neigh1.62253_cross0.79466_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_random_bw7533.90896_skew3.12766_neigh1.68673_cross0.77234_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_random_bw7581.89267_skew3.12766_neigh1.64059_cross0.74279_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_random_bw7694.42259_skew3.12766_neigh1.63441_cross0.76753_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_random_bw7725.2259_skew3.12766_neigh1.64727_cross0.69381_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_random_bw7852.92027_skew3.12766_neigh1.65613_cross0.64131_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_random_bw7964.00444_skew3.12766_neigh1.66283_cross0.58936_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_random_bw8050.7121_skew3.12766_neigh1.66655_cross0.53913_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_random_bw8187.29021_skew3.12766_neigh1.63352_cross0.76751_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_random_bw8518.3466_skew3.12766_neigh1.89786_cross0.7699_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_random_bw8759.33535_skew3.12766_neigh1.63533_cross0.76743_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_random_bw9196.04473_skew3.12766_neigh1.63338_cross0.76885_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_random_bw9256.19861_skew3.12766_neigh1.79462_cross0.76946_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_random_bw9716.25818_skew3.12766_neigh1.63456_cross0.76651_n14.mtx","synthetic_46835_46835_2371578_avg50.63687_std27.78811_random_bw7532.72126_skew3.36441_neigh1.63252_cross0.77731_n14.mtx","synthetic_46835_46835_2371669_avg50.63882_std26.85583_random_bw7496.12448_skew3.10752_neigh1.63286_cross0.77698_n14.mtx","synthetic_46835_46835_2371693_avg50.63933_std27.78141_random_bw7520.67403_skew3.26546_neigh1.63151_cross0.77728_n14.mtx","synthetic_46835_46835_2371771_avg50.64099_std27.79721_random_bw7596.83937_skew3.60102_neigh1.63043_cross0.7826_n14.mtx","synthetic_46835_46835_2371874_avg50.64319_std27.79128_random_bw7573.47232_skew3.48234_neigh1.63324_cross0.78289_n14.mtx","synthetic_46835_46835_2371944_avg50.64469_std25.00834_random_bw7661.37113_skew3.02806_neigh1.62991_cross0.78443_n14.mtx","synthetic_46835_46835_2372249_avg50.6512_std23.16377_random_bw7721.62859_skew3.0078_neigh1.63_cross0.7957_n14.mtx","synthetic_46835_46835_2372391_avg50.65423_std21.31556_random_bw7865.19097_skew2.98782_neigh1.6257_cross0.80882_n14.mtx","synthetic_46835_46835_2372900_avg50.6651_std19.46618_random_bw7985.40892_skew2.96723_neigh1.62267_cross0.81739_n14.mtx","synthetic_46835_46835_2450659_avg52.32538_std27.78086_random_bw7558.44102_skew3.1089_neigh1.63528_cross0.77453_n14.mtx","synthetic_46835_46835_2609003_avg55.70627_std27.78687_random_bw7460.64719_skew3.03904_neigh1.63822_cross0.78391_n14.mtx","synthetic_46835_46835_2767329_avg59.08677_std27.79319_random_bw7641.74269_skew3.01105_neigh1.64146_cross0.79361_n14.mtx","synthetic_46835_46835_2925939_avg62.47334_std27.79699_random_bw7575.97229_skew3.00171_neigh1.64383_cross0.80098_n14.mtx","synthetic_46835_46835_3084303_avg65.85466_std27.80201_random_bw7666.96814_skew2.97846_neigh1.64609_cross0.80651_n14.mtx","synthetic_473242_473242_1892968_avg4.0_std0.0_random_bw643.34994_skew0.0_neigh0.37806_cross0.99029_n14.mtx","synthetic_47513_47513_1853007_avg39.0_std0.0_random_bw11630.5911_skew0.0_neigh1.40553_cross0.80427_n14.mtx","synthetic_47784_47784_2177960_avg45.57927_std31.38054_random_bw1153.05592_skew2.68589_neigh1.21393_cross0.77217_n14.mtx","synthetic_47879_47879_3068257_avg64.08356_std14.01465_random_bw343.15784_skew0.93497_neigh1.58587_cross0.83121_n14.mtx","synthetic_48396_48396_2450061_avg50.62528_std27.78261_random_bw7683.6283_skew3.14813_neigh1.6334_cross0.76917_n14.mtx","synthetic_49152_49152_1327104_avg27.0_std0.0_random_bw11888.55758_skew0.0_neigh1.39065_cross0.80136_n14.mtx","synthetic_49152_49152_1474560_avg30.0_std0.0_random_bw12201.67973_skew0.0_neigh1.39972_cross0.80562_n14.mtx","synthetic_49152_49152_1622016_avg33.0_std0.0_random_bw12114.60683_skew0.0_neigh1.3977_cross0.80501_n14.mtx","synthetic_49152_49152_1720320_avg35.0_std0.0_random_bw11976.30068_skew0.0_neigh1.40142_cross0.80302_n14.mtx","synthetic_49152_49152_1867776_avg38.0_std0.0_random_bw11962.14925_skew0.0_neigh1.40582_cross0.80314_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_random_bw10814.92206_skew0.0_neigh1.40656_cross0.80518_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_random_bw11602.43638_skew0.0_neigh1.40627_cross0.80459_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_random_bw11997.39925_skew0.0_neigh1.17226_cross0.80303_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_random_bw11999.34074_skew0.0_neigh1.41023_cross0.99003_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_random_bw12019.23385_skew0.0_neigh1.26333_cross0.80653_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_random_bw12021.09155_skew0.0_neigh1.35943_cross0.80353_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_random_bw12028.01392_skew0.0_neigh1.40369_cross0.56602_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_random_bw12033.44261_skew0.0_neigh1.40424_cross0.72534_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_random_bw12034.67322_skew0.0_neigh1.40695_cross0.78004_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_random_bw12036.00016_skew0.0_neigh1.45087_cross0.8025_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_random_bw12044.37571_skew0.0_neigh1.40455_cross0.62007_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_random_bw12045.43215_skew0.0_neigh1.40443_cross0.67088_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_random_bw12050.9789_skew0.0_neigh1.07727_cross0.806_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_random_bw12058.91376_skew0.0_neigh1.40774_cross0.83098_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_random_bw12073.32371_skew0.0_neigh1.40561_cross0.80444_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_random_bw12085.13057_skew0.0_neigh0.9811_cross0.8051_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_random_bw12096.1748_skew0.0_neigh1.40517_cross0.88682_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_random_bw12100.19297_skew0.0_neigh1.40732_cross0.93736_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_random_bw12100.23661_skew0.0_neigh1.54428_cross0.80491_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_random_bw12254.45363_skew0.0_neigh1.63837_cross0.80419_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_random_bw12401.00248_skew0.0_neigh1.40613_cross0.80523_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_random_bw12966.94114_skew0.0_neigh1.73293_cross0.80316_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_random_bw13193.13125_skew0.0_neigh1.40663_cross0.80616_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_random_bw14038.84108_skew0.0_neigh1.40505_cross0.80485_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_random_bw14826.38826_skew0.0_neigh1.4046_cross0.80484_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_random_bw15659.63725_skew0.0_neigh1.40594_cross0.80484_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_random_bw17726.56396_skew0.0_neigh1.82624_cross0.8031_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_random_bw8412.1531_skew0.0_neigh1.40648_cross0.80556_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_random_bw9217.67674_skew0.0_neigh1.40509_cross0.80535_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_random_bw9999.7901_skew0.0_neigh1.40701_cross0.8066_n14.mtx","synthetic_49152_49152_1966080_avg40.0_std0.0_random_bw12103.00734_skew0.0_neigh1.40894_cross0.80074_n14.mtx","synthetic_49152_49152_2113536_avg43.0_std0.0_random_bw12029.22158_skew0.0_neigh1.40822_cross0.80163_n14.mtx","synthetic_49152_49152_2260992_avg46.0_std0.0_random_bw11993.41239_skew0.0_neigh1.41259_cross0.8045_n14.mtx","synthetic_49152_49152_2359296_avg48.0_std0.0_random_bw12095.14152_skew0.0_neigh1.41171_cross0.80651_n14.mtx","synthetic_49152_49152_2506752_avg51.0_std0.0_random_bw12095.6193_skew0.0_neigh1.41564_cross0.80645_n14.mtx","synthetic_50367_50367_2296192_avg45.58922_std31.37796_random_bw1217.66939_skew2.68508_neigh1.21323_cross0.76936_n14.mtx","synthetic_50790_50790_1980810_avg39.0_std0.0_random_bw12402.57342_skew0.0_neigh1.40626_cross0.80547_n14.mtx","synthetic_508297_508297_2033188_avg4.0_std0.0_random_bw695.22929_skew0.0_neigh0.36764_cross0.99098_n14.mtx","synthetic_51518_51518_2608832_avg50.63923_std27.762_random_bw8203.55775_skew3.14698_neigh1.63544_cross0.76613_n14.mtx","synthetic_52042_52042_3335826_avg64.09873_std14.0_random_bw372.49024_skew0.93452_neigh1.58046_cross0.82963_n14.mtx","synthetic_525825_525825_1577449_avg2.99995_std0.00703_random_bw859.08869_skew2e-05_neigh0.3293_cross0.99111_n14.mtx","synthetic_525825_525825_1577475_avg3.0_std0.0_random_bw858.55443_skew0.0_neigh0.33003_cross0.99112_n14.mtx","synthetic_525825_525825_1583911_avg3.01224_std0.10995_random_bw797.16394_skew0.32792_neigh0.26282_cross0.98853_n14.mtx","synthetic_525825_525825_2046881_avg3.8927_std0.30949_random_bw622.52163_skew0.02756_neigh0.1362_cross0.96883_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_random_bw505.76605_skew0.0_neigh0.37416_cross0.99125_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_random_bw554.92392_skew0.0_neigh0.37192_cross0.99125_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_random_bw606.93324_skew0.0_neigh0.37258_cross0.99124_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_random_bw644.53081_skew0.0_neigh0.37119_cross0.99121_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_random_bw670.66979_skew0.0_neigh0.48412_cross0.99123_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_random_bw683.49694_skew0.0_neigh0.45923_cross0.9912_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_random_bw695.20509_skew0.0_neigh0.4342_cross0.9912_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_random_bw702.53172_skew0.0_neigh0.37156_cross0.99124_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_random_bw705.68994_skew0.0_neigh0.40842_cross0.99122_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_random_bw714.99184_skew0.0_neigh0.38412_cross0.99123_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_random_bw717.31533_skew0.0_neigh0.3775_cross0.95838_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_random_bw719.01499_skew0.0_neigh0.37393_cross0.8926_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_random_bw719.86053_skew0.0_neigh0.37171_cross0.99223_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_random_bw719.98069_skew0.0_neigh0.37726_cross0.76297_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_random_bw720.4176_skew0.0_neigh0.37153_cross0.99122_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_random_bw720.57144_skew0.0_neigh0.37696_cross0.69668_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_random_bw720.84278_skew0.0_neigh0.37761_cross0.82791_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_random_bw726.03436_skew0.0_neigh0.35739_cross0.9912_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_random_bw737.14928_skew0.0_neigh0.33322_cross0.99119_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_random_bw737.4945_skew0.0_neigh0.3712_cross0.99123_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_random_bw744.2048_skew0.0_neigh0.30997_cross0.99121_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_random_bw756.76312_skew0.0_neigh0.28196_cross0.99121_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_random_bw766.29148_skew0.0_neigh0.26012_cross0.9912_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_random_bw792.51217_skew0.0_neigh0.37127_cross0.99121_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_random_bw833.10523_skew0.0_neigh0.37123_cross0.99121_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_random_bw890.92849_skew0.0_neigh0.37114_cross0.9912_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_random_bw939.84754_skew0.0_neigh0.3724_cross0.99121_n14.mtx","synthetic_525825_525825_2146618_avg4.08238_std0.27494_random_bw490.88949_skew0.22478_neigh0.18414_cross0.97729_n14.mtx","synthetic_525825_525825_2619767_avg4.9822_std0.13221_random_bw654.36998_skew0.00357_neigh0.31537_cross0.98835_n14.mtx","synthetic_525825_525825_2629125_avg5.0_std0.0_random_bw685.43441_skew0.0_neigh0.39914_cross0.9912_n14.mtx","synthetic_525825_525825_2629143_avg5.00003_std0.00585_random_bw685.37638_skew0.19999_neigh0.3989_cross0.9912_n14.mtx","synthetic_54067_54067_2108613_avg39.0_std0.0_random_bw13195.94642_skew0.0_neigh1.40596_cross0.80602_n14.mtx","synthetic_543352_543352_2173408_avg4.0_std0.0_random_bw737.48539_skew0.0_neigh0.3826_cross0.99144_n14.mtx","synthetic_54640_54640_2767687_avg50.65313_std27.75459_random_bw8703.93389_skew3.14584_neigh1.6342_cross0.76941_n14.mtx","synthetic_56205_56205_3603711_avg64.11727_std13.99803_random_bw400.50616_skew0.93396_neigh1.57644_cross0.82872_n14.mtx","synthetic_57344_57344_2236416_avg39.0_std0.0_random_bw14033.40515_skew0.0_neigh1.40406_cross0.80608_n14.mtx","synthetic_57763_57763_2926741_avg50.66809_std27.74629_random_bw9179.53278_skew3.16436_neigh1.63113_cross0.77117_n14.mtx","synthetic_578407_578407_2313628_avg4.0_std0.0_random_bw786.14351_skew0.0_neigh0.37361_cross0.99189_n14.mtx","synthetic_60369_60369_3871327_avg64.12773_std13.99319_random_bw430.33024_skew0.93364_neigh1.57281_cross0.82791_n14.mtx","synthetic_60620_60620_2364180_avg39.0_std0.0_random_bw14835.70082_skew0.0_neigh1.4037_cross0.80546_n14.mtx","synthetic_60885_60885_3084540_avg50.66174_std27.74397_random_bw9701.3275_skew3.16488_neigh1.63253_cross0.76879_n14.mtx","synthetic_613462_613462_2453848_avg4.0_std0.0_random_bw842.88409_skew0.0_neigh0.38222_cross0.9922_n14.mtx","synthetic_62451_62451_2802494_avg44.87509_std13.98221_random_bw433.69895_skew1.33983_neigh1.53816_cross0.79072_n14.mtx","synthetic_62451_62451_3069539_avg49.15116_std13.98917_random_bw442.35894_skew1.21765_neigh1.54755_cross0.80178_n14.mtx","synthetic_62451_62451_3336828_avg53.43114_std13.99111_random_bw442.79608_skew1.11487_neigh1.55465_cross0.80814_n14.mtx","synthetic_62451_62451_3603802_avg57.70607_std13.99012_random_bw440.31634_skew1.04485_neigh1.56479_cross0.81778_n14.mtx","synthetic_62451_62451_3870981_avg61.98429_std13.98816_random_bw440.14062_skew0.96824_neigh1.57158_cross0.82571_n14.mtx","synthetic_62451_62451_4003815_avg64.1113_std18.18532_random_bw433.90471_skew1.2149_neigh1.57849_cross0.80426_n14.mtx","synthetic_62451_62451_4004004_avg64.11433_std17.25509_random_bw437.19148_skew1.1524_neigh1.57787_cross0.80809_n14.mtx","synthetic_62451_62451_4004101_avg64.11588_std16.32075_random_bw439.33923_skew1.08997_neigh1.57568_cross0.81415_n14.mtx","synthetic_62451_62451_4004374_avg64.12025_std15.38744_random_bw440.62742_skew1.02744_neigh1.5754_cross0.81871_n14.mtx","synthetic_62451_62451_4004540_avg64.12291_std14.45551_random_bw443.23719_skew0.96498_neigh1.573_cross0.82479_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_random_bw314.5764_skew0.93375_neigh1.58679_cross0.83355_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_random_bw343.48287_skew0.93375_neigh1.58569_cross0.83066_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_random_bw372.86596_skew0.93375_neigh1.57974_cross0.82953_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_random_bw400.84485_skew0.93375_neigh1.57624_cross0.82892_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_random_bw430.23839_skew0.93375_neigh1.57299_cross0.82774_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_random_bw458.26891_skew0.93375_neigh1.57086_cross0.82777_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_random_bw485.32971_skew0.93375_neigh1.56912_cross0.82716_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_random_bw514.6847_skew0.93375_neigh1.56848_cross0.82621_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_random_bw541.61408_skew0.93375_neigh1.56544_cross0.82639_n14.mtx","synthetic_62451_62451_4004768_avg64.12656_std12.59183_random_bw445.64555_skew0.84011_neigh1.57031_cross0.83683_n14.mtx","synthetic_62451_62451_4004839_avg64.1277_std13.52372_random_bw443.69469_skew0.90245_neigh1.57245_cross0.83204_n14.mtx","synthetic_62451_62451_4005045_avg64.131_std11.65916_random_bw450.22189_skew0.77761_neigh1.56813_cross0.8418_n14.mtx","synthetic_62451_62451_4005321_avg64.13542_std10.72592_random_bw451.83634_skew0.71512_neigh1.56389_cross0.84902_n14.mtx","synthetic_62451_62451_4005505_avg64.13836_std9.79513_random_bw454.37802_skew0.65268_neigh1.56244_cross0.85409_n14.mtx","synthetic_62451_62451_4138184_avg66.26289_std13.99067_random_bw440.8832_skew0.90152_neigh1.57532_cross0.83038_n14.mtx","synthetic_62451_62451_4405420_avg70.54202_std13.98985_random_bw441.29735_skew0.85705_neigh1.58148_cross0.83621_n14.mtx","synthetic_62451_62451_4672484_avg74.8184_std13.9889_random_bw446.77627_skew0.80437_neigh1.58533_cross0.84209_n14.mtx","synthetic_62451_62451_4939666_avg79.09667_std13.99063_random_bw446.66801_skew0.75734_neigh1.59109_cross0.84662_n14.mtx","synthetic_62451_62451_5206981_avg83.37706_std13.99111_random_bw448.16022_skew0.7151_neigh1.59447_cross0.85123_n14.mtx","synthetic_63897_63897_2491983_avg39.0_std0.0_random_bw15621.79835_skew0.0_neigh1.40542_cross0.80575_n14.mtx","synthetic_64532_64532_4137790_avg64.11997_std13.98737_random_bw458.31169_skew0.93387_neigh1.5704_cross0.82793_n14.mtx","synthetic_648517_648517_2594068_avg4.0_std0.0_random_bw886.62276_skew0.0_neigh0.37472_cross0.99258_n14.mtx","synthetic_683572_683572_2734288_avg4.0_std0.0_random_bw924.48268_skew0.0_neigh0.37975_cross0.9929_n14.mtx","synthetic_68696_68696_4403097_avg64.09539_std13.99668_random_bw485.33335_skew0.93462_neigh1.56937_cross0.82685_n14.mtx","synthetic_700003_700003_2220092_avg3.17155_std25.65674_random_bw87391.5121_skew1481.87283_neigh0.14428_cross0.90055_n14.mtx","synthetic_72859_72859_4668546_avg64.07645_std14.00516_random_bw513.63554_skew0.93519_neigh1.56865_cross0.82618_n14.mtx","synthetic_766670_766670_2431595_avg3.17163_std25.62988_random_bw95810.1159_skew1481.83288_neigh0.14367_cross0.90161_n14.mtx","synthetic_77022_77022_4936326_avg64.08982_std14.01488_random_bw542.85377_skew0.93478_neigh1.56641_cross0.8258_n14.mtx","synthetic_81186_81186_5203811_avg64.09739_std14.01954_random_bw572.64595_skew0.93456_neigh1.56212_cross0.82312_n14.mtx","synthetic_833337_833337_2642605_avg3.17111_std25.60679_random_bw103837.45496_skew1482.07595_neigh0.14411_cross0.90125_n14.mtx","synthetic_84834_84834_1835279_avg21.63377_std13.7861_random_bw46227.07554_skew4.03842_neigh1.01517_cross0.55965_n14.mtx","synthetic_900004_900004_2853888_avg3.17097_std25.58712_random_bw112429.88878_skew1482.14118_neigh0.14011_cross0.90178_n14.mtx","synthetic_92913_92913_2009995_avg21.63309_std13.78454_random_bw50425.51531_skew4.03858_neigh1.01427_cross0.56119_n14.mtx","synthetic_966671_966671_3064966_avg3.17064_std25.57082_random_bw120972.93224_skew1482.29662_neigh0.14283_cross0.90232_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_random_bw12314.98414_skew1.0109_neigh1.69561_cross0.82051_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_random_bw13218.95443_skew1.0109_neigh1.63894_cross0.89577_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_random_bw13302.83667_skew1.0109_neigh1.69559_cross0.82053_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_random_bw13936.21876_skew1.0109_neigh1.41417_cross0.8198_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_random_bw14004.39261_skew1.0109_neigh1.52687_cross0.8201_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_random_bw14289.87368_skew1.0109_neigh1.69536_cross0.81999_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_random_bw14335.05046_skew1.0109_neigh1.68453_cross0.84715_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_random_bw14787.04212_skew1.0109_neigh1.69513_cross0.82047_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_random_bw15130.18363_skew1.0109_neigh1.7031_cross0.79329_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_random_bw16264.19116_skew1.0109_neigh1.69516_cross0.82009_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_random_bw16429.30637_skew1.0109_neigh1.72541_cross0.63001_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_random_bw16689.96273_skew1.0109_neigh1.72925_cross0.57549_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_random_bw17252.2932_skew1.0109_neigh1.69476_cross0.8205_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_random_bw18227.45253_skew1.0109_neigh1.69515_cross0.82059_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_random_bw20444.65782_skew1.0109_neigh1.91237_cross0.8206_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_random_bw27298.86084_skew1.0109_neigh1.86289_cross0.82084_n14.mtx","synthetic_1006441_1006441_22414673_avg22.27122_std29.74727_random_bw254733.54551_skew313.62123_neigh1.26289_cross0.75654_n14.mtx","synthetic_1019593_1019593_74066178_avg72.64289_std15.80935_random_bw15286.94394_skew1.03736_neigh1.69506_cross0.8204_n14.mtx","synthetic_102778_102778_7408141_avg72.07905_std19.01282_random_bw5890.51486_skew1.19204_neigh1.65342_cross0.76938_n14.mtx","synthetic_1047423_1047423_51169710_avg48.85296_std11.94333_random_bw189507.21493_skew1.10837_neigh1.69026_cross0.8037_n14.mtx","synthetic_1060229_1060229_12992547_avg12.25447_std37.61142_random_bw24980.56284_skew632.64621_neigh1.32193_cross0.78513_n14.mtx","synthetic_1063952_1063952_23695040_avg22.27078_std29.72463_random_bw269168.5156_skew313.62752_neigh1.26394_cross0.75677_n14.mtx","synthetic_108003_108003_5987303_avg55.43645_std11.04356_random_bw4243.01828_skew0.89406_neigh1.62997_cross0.78726_n14.mtx","synthetic_108334_108334_7808284_avg72.07602_std19.0204_random_bw6217.09331_skew1.19213_neigh1.65302_cross0.76997_n14.mtx","synthetic_1085373_1085373_78800672_avg72.60239_std15.80629_random_bw16260.36911_skew1.02473_neigh1.69518_cross0.82006_n14.mtx","synthetic_1110903_1110903_54291613_avg48.87161_std11.93583_random_bw200760.47456_skew1.10756_neigh1.68972_cross0.80443_n14.mtx","synthetic_1121463_1121463_24976074_avg22.27097_std29.7046_random_bw283780.07078_skew313.62476_neigh1.26318_cross0.75684_n14.mtx","synthetic_112749_112749_5727652_avg50.80002_std19.54637_random_bw3458.33754_skew1.7559_neigh1.22842_cross0.71885_n14.mtx","synthetic_1151153_1151153_83612943_avg72.63408_std15.81129_random_bw17250.83068_skew1.01008_neigh1.69488_cross0.82066_n14.mtx","synthetic_1152423_1152423_14123090_avg12.25513_std37.58092_random_bw27186.1267_skew632.61237_neigh1.32195_cross0.78618_n14.mtx","synthetic_117394_117394_6507908_avg55.43646_std11.04171_random_bw4592.46353_skew0.89406_neigh1.63085_cross0.78812_n14.mtx","synthetic_1174383_1174383_57377479_avg48.85755_std11.94355_random_bw212160.93638_skew1.10817_neigh1.68973_cross0.80444_n14.mtx","synthetic_1216933_1216933_88363297_avg72.61147_std15.79644_random_bw18231.88915_skew1.03825_neigh1.69444_cross0.82088_n14.mtx","synthetic_123486_123486_6272129_avg50.79223_std19.54882_random_bw3787.53287_skew1.75633_neigh1.22949_cross0.71628_n14.mtx","synthetic_1237863_1237863_60496296_avg48.87156_std11.94811_random_bw223383.86547_skew1.20987_neigh1.69022_cross0.80452_n14.mtx","synthetic_1244617_1244617_15251343_avg12.25384_std37.55512_random_bw29349.06445_skew632.67869_neigh1.32246_cross0.78617_n14.mtx","synthetic_126786_126786_7028939_avg55.43939_std11.04404_random_bw4958.06756_skew0.89396_neigh1.63099_cross0.78932_n14.mtx","synthetic_1282713_1282713_93191153_avg72.6516_std15.81312_random_bw19218.72077_skew1.10594_neigh1.69435_cross0.82082_n14.mtx","synthetic_129947_129947_10504776_avg80.83893_std117.70867_random_bw17726.53455_skew7.85712_neigh1.21404_cross0.84553_n14.mtx","synthetic_1336811_1336811_16381252_avg12.25398_std37.5327_random_bw31479.27429_skew632.6718_neigh1.32292_cross0.78634_n14.mtx","synthetic_134225_134225_6816930_avg50.78733_std19.56874_random_bw4113.48627_skew1.77628_neigh1.22748_cross0.71779_n14.mtx","synthetic_136178_136178_7548659_avg55.43229_std11.05483_random_bw5329.09974_skew0.91224_neigh1.63079_cross0.78836_n14.mtx","synthetic_1382908_1382908_11844497_avg8.56492_std37.37396_random_bw40487.08926_skew633.56515_neigh1.20543_cross0.77916_n14.mtx","synthetic_1382908_1382908_12976596_avg9.38356_std37.40231_random_bw42638.77576_skew633.40762_neigh1.23946_cross0.78166_n14.mtx","synthetic_1382908_1382908_14113272_avg10.2055_std37.43376_random_bw29869.67508_skew632.97168_neigh1.26037_cross0.782_n14.mtx","synthetic_1382908_1382908_15242471_avg11.02204_std37.4678_random_bw31130.67806_skew633.00226_neigh1.26986_cross0.78321_n14.mtx","synthetic_1382908_1382908_16349573_avg11.8226_std37.50438_random_bw32032.95325_skew633.88554_neigh1.28645_cross0.78343_n14.mtx","synthetic_1382908_1382908_16848093_avg12.18309_std26.47929_random_bw31499.6008_skew636.35882_neigh1.38178_cross0.79033_n14.mtx","synthetic_1382908_1382908_16896659_avg12.21821_std28.92386_random_bw31578.67454_skew634.52686_neigh1.37766_cross0.7904_n14.mtx","synthetic_1382908_1382908_16922141_avg12.23664_std48.62398_random_bw32402.87716_skew633.48814_neigh1.28006_cross0.77973_n14.mtx","synthetic_1382908_1382908_16923406_avg12.23755_std46.1541_random_bw32550.44114_skew633.44071_neigh1.28373_cross0.78136_n14.mtx","synthetic_1382908_1382908_16923784_avg12.23782_std37.37393_random_bw32578.59823_skew443.68692_neigh1.28762_cross0.78233_n14.mtx","synthetic_1382908_1382908_16926342_avg12.23967_std43.68598_random_bw32633.88821_skew633.41236_neigh1.29181_cross0.78314_n14.mtx","synthetic_1382908_1382908_16927023_avg12.24017_std37.40259_random_bw32650.87537_skew485.75812_neigh1.29364_cross0.78327_n14.mtx","synthetic_1382908_1382908_16931896_avg12.24369_std37.43372_random_bw32726.41259_skew527.76219_neigh1.30038_cross0.78452_n14.mtx","synthetic_1382908_1382908_16931967_avg12.24374_std41.21881_random_bw32683.4221_skew633.2016_neigh1.29995_cross0.78417_n14.mtx","synthetic_1382908_1382908_16933417_avg12.24479_std31.37528_random_bw31839.54587_skew633.1473_neigh1.36655_cross0.78964_n14.mtx","synthetic_1382908_1382908_16938002_avg12.2481_std37.46714_random_bw32660.61927_skew569.78219_neigh1.30898_cross0.7854_n14.mtx","synthetic_1382908_1382908_16941719_avg12.25079_std38.75385_random_bw32663.46745_skew632.83654_neigh1.31388_cross0.78566_n14.mtx","synthetic_1382908_1382908_16943948_avg12.2524_std37.50354_random_bw32608.3439_skew611.69607_neigh1.31792_cross0.78598_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_random_bw16831.66287_skew632.66322_neigh1.72456_cross0.78841_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_random_bw22849.17172_skew632.66322_neigh1.32203_cross0.78644_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_random_bw25001.14321_skew632.66322_neigh1.32215_cross0.78651_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_random_bw26972.08395_skew632.66322_neigh1.63487_cross0.7871_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_random_bw27152.96238_skew632.66322_neigh1.32223_cross0.78644_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_random_bw28588.16229_skew632.66322_neigh0.52059_cross0.98566_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_random_bw28608.91048_skew632.66322_neigh1.10181_cross0.78592_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_random_bw28827.3735_skew632.66322_neigh0.92504_cross0.78425_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_random_bw29311.03639_skew632.66322_neigh1.3223_cross0.7864_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_random_bw29812.11226_skew632.66322_neigh1.01255_cross0.78502_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_random_bw30178.2994_skew632.66322_neigh1.18993_cross0.78639_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_random_bw31055.2687_skew632.66322_neigh1.36651_cross0.78696_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_random_bw31485.89914_skew632.66322_neigh1.32204_cross0.78641_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_random_bw31866.08413_skew632.66322_neigh1.3625_cross0.55123_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_random_bw31960.20546_skew632.66322_neigh1.35743_cross0.60367_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_random_bw32103.51377_skew632.66322_neigh1.3514_cross0.65603_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_random_bw32298.61541_skew632.66322_neigh1.342_cross0.70831_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_random_bw32527.47818_skew632.66322_neigh1.32971_cross0.7607_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_random_bw32649.42139_skew632.66322_neigh1.31221_cross0.81225_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_random_bw33004.14191_skew632.66322_neigh0.99375_cross0.96683_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_random_bw33018.43863_skew632.66322_neigh1.28269_cross0.8644_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_random_bw33385.39612_skew632.66322_neigh1.21925_cross0.91646_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_random_bw33686.12799_skew632.66322_neigh1.32166_cross0.7864_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_random_bw33939.34987_skew632.66322_neigh1.27774_cross0.78619_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_random_bw35035.98789_skew632.66322_neigh1.545_cross0.78785_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_random_bw41464.2511_skew632.66322_neigh1.45537_cross0.78695_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_random_bw42380.53696_skew632.66322_neigh1.32182_cross0.78634_n14.mtx","synthetic_1382908_1382908_16948048_avg12.25537_std37.72372_random_bw32047.73507_skew822.06782_neigh1.35662_cross0.78905_n14.mtx","synthetic_1382908_1382908_16948709_avg12.25585_std37.54211_random_bw32537.49715_skew653.62631_neigh1.32642_cross0.78675_n14.mtx","synthetic_1382908_1382908_16950422_avg12.25709_std36.29175_random_bw32502.49439_skew632.51111_neigh1.33052_cross0.78757_n14.mtx","synthetic_1382908_1382908_16950660_avg12.25726_std33.83118_random_bw32159.50342_skew632.50221_neigh1.34985_cross0.78899_n14.mtx","synthetic_1382908_1382908_16951601_avg12.25794_std37.67423_random_bw32164.80648_skew779.80014_neigh1.34967_cross0.78884_n14.mtx","synthetic_1382908_1382908_16951652_avg12.25798_std37.58358_random_bw32449.74613_skew695.60771_neigh1.33402_cross0.7879_n14.mtx","synthetic_1382908_1382908_16952713_avg12.25874_std37.62729_random_bw32268.27374_skew737.65652_neigh1.34279_cross0.78851_n14.mtx","synthetic_1382908_1382908_17445616_avg12.61517_std37.54329_random_bw32804.62199_skew635.05973_neigh1.31534_cross0.78529_n14.mtx","synthetic_1382908_1382908_18572951_avg13.43036_std37.58404_random_bw33577.21407_skew634.94726_neigh1.34983_cross0.78702_n14.mtx","synthetic_1382908_1382908_19756490_avg14.28619_std37.62781_random_bw28948.7818_skew633.10877_neigh1.36875_cross0.78902_n14.mtx","synthetic_1382908_1382908_20949793_avg15.14909_std37.6751_random_bw30001.28368_skew631.18333_neigh1.36337_cross0.78796_n14.mtx","synthetic_1382908_1382908_22070012_avg15.95913_std37.72535_random_bw30789.33457_skew631.49052_neigh1.34689_cross0.78668_n14.mtx","synthetic_140874_140874_10153211_avg72.07299_std11.05163_random_bw5566.17624_skew0.7066_neigh1.64681_cross0.80416_n14.mtx","synthetic_140874_140874_5465261_avg38.79538_std11.0515_random_bw5472.25499_skew1.31986_neigh1.60427_cross0.75763_n14.mtx","synthetic_140874_140874_5986164_avg42.49304_std11.0536_random_bw5615.53216_skew1.18859_neigh1.61127_cross0.76558_n14.mtx","synthetic_140874_140874_6506839_avg46.18907_std11.05195_random_bw5467.46501_skew1.10006_neigh1.61774_cross0.77319_n14.mtx","synthetic_140874_140874_7027827_avg49.88732_std11.05353_random_bw5578.52051_skew1.02456_neigh1.62272_cross0.78142_n14.mtx","synthetic_140874_140874_7548897_avg53.58616_std11.05321_random_bw5458.59888_skew0.9408_neigh1.6305_cross0.78364_n14.mtx","synthetic_140874_140874_7807971_avg55.42521_std14.36638_random_bw5391.79174_skew1.18312_neigh1.63652_cross0.76654_n14.mtx","synthetic_140874_140874_7808175_avg55.42666_std13.63005_random_bw5409.69514_skew1.12894_neigh1.63453_cross0.77116_n14.mtx","synthetic_140874_140874_7808374_avg55.42807_std12.89549_random_bw5451.61362_skew1.07476_neigh1.63304_cross0.77494_n14.mtx","synthetic_140874_140874_7808713_avg55.43048_std12.15675_random_bw5470.09146_skew1.00251_neigh1.63286_cross0.77934_n14.mtx","synthetic_140874_140874_7809044_avg55.43283_std11.42187_random_bw5501.94588_skew0.9483_neigh1.63246_cross0.78383_n14.mtx","synthetic_140874_140874_7809186_avg55.43383_std10.68354_random_bw5528.77301_skew0.89415_neigh1.63008_cross0.79051_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_random_bw3868.73909_skew0.91217_neigh1.62925_cross0.78945_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_random_bw4235.02181_skew0.91217_neigh1.62951_cross0.78894_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_random_bw4605.71603_skew0.91217_neigh1.63013_cross0.78855_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_random_bw4770.72113_skew0.91217_neigh1.56132_cross0.89987_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_random_bw4956.26364_skew0.91217_neigh1.63116_cross0.78903_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_random_bw5095.18835_skew0.91217_neigh1.59792_cross0.86514_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_random_bw5267.47496_skew0.91217_neigh1.14546_cross0.78537_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_random_bw5270.93334_skew0.91217_neigh1.256_cross0.78678_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_random_bw5285.20601_skew0.91217_neigh1.3622_cross0.78648_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_random_bw5319.89205_skew0.91217_neigh1.46878_cross0.78637_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_random_bw5329.16386_skew0.91217_neigh1.63054_cross0.78839_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_random_bw5371.77114_skew0.91217_neigh1.57638_cross0.78738_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_random_bw5396.20773_skew0.91217_neigh1.62321_cross0.81439_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_random_bw5522.95748_skew0.91217_neigh1.6306_cross0.7874_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_random_bw5613.49824_skew0.91217_neigh1.63776_cross0.76192_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_random_bw5693.34228_skew0.91217_neigh1.63074_cross0.78739_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_random_bw5768.02808_skew0.91217_neigh1.64697_cross0.7083_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_random_bw5802.87171_skew0.91217_neigh1.68497_cross0.78689_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_random_bw5877.42657_skew0.91217_neigh1.65421_cross0.65718_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_random_bw5969.91225_skew0.91217_neigh1.65949_cross0.60613_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_random_bw6034.73118_skew0.91217_neigh1.66318_cross0.55387_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_random_bw6073.33464_skew0.91217_neigh1.63029_cross0.78753_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_random_bw6436.17949_skew0.91217_neigh1.62906_cross0.78757_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_random_bw6525.25347_skew0.91217_neigh1.90151_cross0.7872_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_random_bw6689.33888_skew0.91217_neigh1.89964_cross0.78632_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_random_bw6800.23242_skew0.91217_neigh1.62946_cross0.78737_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_random_bw7164.09343_skew0.91217_neigh1.62998_cross0.78797_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_random_bw7287.18556_skew0.91217_neigh1.79126_cross0.78857_n14.mtx","synthetic_140874_140874_7809480_avg55.43592_std9.94668_random_bw5548.2026_skew0.82192_neigh1.62952_cross0.79532_n14.mtx","synthetic_140874_140874_7809970_avg55.4394_std9.2113_random_bw5619.82513_skew0.7677_neigh1.62745_cross0.7992_n14.mtx","synthetic_140874_140874_7810161_avg55.44076_std8.47606_random_bw5628.17949_skew0.6955_neigh1.62718_cross0.80558_n14.mtx","synthetic_140874_140874_7810358_avg55.44215_std7.74028_random_bw5680.4538_skew0.64135_neigh1.62737_cross0.8106_n14.mtx","synthetic_140874_140874_7812605_avg55.4581_std11.08015_random_bw6222.03617_skew2.11947_neigh1.61513_cross0.85242_n14.mtx","synthetic_140874_140874_7812630_avg55.45828_std11.08_random_bw6217.1087_skew2.06537_neigh1.61471_cross0.85345_n14.mtx","synthetic_140874_140874_8069568_avg57.28217_std11.05207_random_bw5567.22519_skew0.8854_neigh1.63271_cross0.78944_n14.mtx","synthetic_140874_140874_8590481_avg60.97989_std11.05315_random_bw5493.64159_skew0.83667_neigh1.63609_cross0.79354_n14.mtx","synthetic_140874_140874_9111507_avg64.67841_std11.05222_random_bw5571.17536_skew0.77803_neigh1.64018_cross0.79828_n14.mtx","synthetic_140874_140874_9632262_avg68.37502_std11.0533_random_bw5500.1344_skew0.7404_neigh1.64395_cross0.80172_n14.mtx","synthetic_142323_142323_11504673_avg80.83495_std117.71151_random_bw19450.52367_skew7.85755_neigh1.21346_cross0.84523_n14.mtx","synthetic_1429004_1429004_17510920_avg12.25393_std37.51291_random_bw33714.51176_skew632.67408_neigh1.3215_cross0.7862_n14.mtx","synthetic_144963_144963_7361596_avg50.78259_std19.57819_random_bw4444.28282_skew1.77654_neigh1.22789_cross0.7157_n14.mtx","synthetic_145569_145569_8068734_avg55.42893_std11.05658_random_bw5696.50499_skew0.91236_neigh1.63059_cross0.78723_n14.mtx","synthetic_1521198_1521198_18638978_avg12.25283_std37.49579_random_bw35839.82327_skew632.73123_neigh1.3217_cross0.78709_n14.mtx","synthetic_152542_152542_8133417_avg53.3192_std4.76067_random_bw10831.6453_skew3.36991_neigh1.81197_cross0.94119_n14.mtx","synthetic_154699_154699_12503850_avg80.82696_std117.71492_random_bw21128.24826_skew7.85843_neigh1.21484_cross0.84433_n14.mtx","synthetic_154961_154961_8588351_avg55.42266_std11.06218_random_bw6071.63273_skew0.91258_neigh1.62979_cross0.78766_n14.mtx","synthetic_155701_155701_7905533_avg50.77381_std19.58895_random_bw4775.17387_skew1.77702_neigh1.22646_cross0.71616_n14.mtx","synthetic_161070_161070_10085122_avg62.61329_std19.67482_random_bw5009.75586_skew1.44357_neigh1.23526_cross0.7413_n14.mtx","synthetic_161070_161070_10630527_avg65.99942_std19.67949_random_bw5044.97444_skew1.36366_neigh1.23861_cross0.74622_n14.mtx","synthetic_161070_161070_5763734_avg35.78403_std19.07947_random_bw4731.12239_skew2.52112_neigh1.21212_cross0.66116_n14.mtx","synthetic_161070_161070_6293192_avg39.07116_std19.27381_random_bw4796.72539_skew2.30167_neigh1.2143_cross0.67765_n14.mtx","synthetic_161070_161070_6828184_avg42.39265_std19.41502_random_bw4843.00839_skew2.13734_neigh1.21781_cross0.691_n14.mtx","synthetic_161070_161070_7366852_avg45.73696_std19.51164_random_bw4891.21219_skew1.97353_neigh1.22313_cross0.70301_n14.mtx","synthetic_161070_161070_7908198_avg49.0979_std19.57809_random_bw4902.19808_skew1.83108_neigh1.2252_cross0.71126_n14.mtx","synthetic_161070_161070_8177031_avg50.76694_std16.39176_random_bw5026.50779_skew1.48193_neigh1.22306_cross0.73804_n14.mtx","synthetic_161070_161070_8177317_avg50.76872_std15.08847_random_bw5069.23063_skew1.36366_neigh1.22345_cross0.74511_n14.mtx","synthetic_161070_161070_8177462_avg50.76962_std17.68784_random_bw4990.5517_skew1.59998_neigh1.2267_cross0.72883_n14.mtx","synthetic_161070_161070_8177930_avg50.77252_std13.78024_random_bw5097.10939_skew1.24531_neigh1.22217_cross0.75643_n14.mtx","synthetic_161070_161070_8178443_avg50.77571_std18.96773_random_bw4956.1697_skew1.71784_neigh1.22663_cross0.71981_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_random_bw3455.35798_skew1.77657_neigh1.22972_cross0.71662_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_random_bw3785.72739_skew1.77657_neigh1.22674_cross0.71723_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_random_bw4111.1518_skew1.77657_neigh1.22748_cross0.71758_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_random_bw4422.81445_skew1.77657_neigh1.19628_cross0.82221_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_random_bw4441.89847_skew1.77657_neigh1.22792_cross0.71561_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_random_bw4627.28018_skew1.77657_neigh1.21189_cross0.78656_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_random_bw4774.35335_skew1.77657_neigh1.22637_cross0.71614_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_random_bw4843.79053_skew1.77657_neigh1.22333_cross0.73927_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_random_bw4898.56338_skew1.77657_neigh1.10619_cross0.71643_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_random_bw4900.19763_skew1.77657_neigh0.94384_cross0.71595_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_random_bw4901.06241_skew1.77657_neigh0.86553_cross0.71558_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_random_bw4902.46508_skew1.77657_neigh1.02504_cross0.71613_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_random_bw4934.87943_skew1.77657_neigh1.18606_cross0.71564_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_random_bw4941.28083_skew1.77657_neigh1.22782_cross0.71491_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_random_bw4945.13241_skew1.77657_neigh1.26686_cross0.71552_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_random_bw4964.76983_skew1.77657_neigh1.34692_cross0.71497_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_random_bw4977.92274_skew1.77657_neigh1.42811_cross0.71687_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_random_bw5011.34549_skew1.77657_neigh1.50982_cross0.7154_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_random_bw5016.74181_skew1.77657_neigh1.22985_cross0.69227_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_random_bw5049.43708_skew1.77657_neigh1.58911_cross0.7181_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_random_bw5104.02865_skew1.77657_neigh1.22549_cross0.71514_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_random_bw5170.8755_skew1.77657_neigh1.23483_cross0.64645_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_random_bw5295.40467_skew1.77657_neigh1.2381_cross0.60018_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_random_bw5409.67965_skew1.77657_neigh1.24201_cross0.55228_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_random_bw5421.6343_skew1.77657_neigh1.22563_cross0.71654_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_random_bw5516.25936_skew1.77657_neigh1.24385_cross0.50438_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_random_bw5765.30102_skew1.77657_neigh1.22547_cross0.71569_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_random_bw6093.91118_skew1.77657_neigh1.22469_cross0.7147_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_random_bw6414.53412_skew1.77657_neigh1.22329_cross0.7164_n14.mtx","synthetic_161070_161070_8181013_avg50.79166_std20.23119_random_bw4921.85183_skew1.83511_neigh1.22714_cross0.71122_n14.mtx","synthetic_161070_161070_8181297_avg50.79343_std19.67479_random_bw5300.87224_skew2.30751_neigh1.21351_cross0.79956_n14.mtx","synthetic_161070_161070_8184643_avg50.8142_std21.47457_random_bw4878.01938_skew1.95193_neigh1.22869_cross0.70478_n14.mtx","synthetic_161070_161070_8190209_avg50.84876_std22.69412_random_bw4850.04792_skew2.06792_neigh1.22945_cross0.69379_n14.mtx","synthetic_161070_161070_8197860_avg50.89626_std23.8911_random_bw4817.59002_skew2.18295_neigh1.23133_cross0.68541_n14.mtx","synthetic_161070_161070_8207913_avg50.95867_std25.06294_random_bw4778.56517_skew2.29679_neigh1.23219_cross0.67699_n14.mtx","synthetic_161070_161070_8451173_avg52.46895_std19.62109_random_bw4948.49578_skew1.72542_neigh1.22786_cross0.71962_n14.mtx","synthetic_161070_161070_8995114_avg55.84599_std19.64654_random_bw4972.88478_skew1.61433_neigh1.23174_cross0.72925_n14.mtx","synthetic_161070_161070_9539943_avg59.22855_std19.66505_random_bw5000.60859_skew1.53256_neigh1.23372_cross0.73505_n14.mtx","synthetic_1613392_1613392_19770221_avg12.25382_std37.48086_random_bw38048.92289_skew632.67976_neigh1.3224_cross0.78669_n14.mtx","synthetic_164353_164353_9109408_avg55.42587_std11.0688_random_bw6425.46749_skew0.91246_neigh1.62988_cross0.78753_n14.mtx","synthetic_166438_166438_8452047_avg50.78195_std19.60687_random_bw5103.62439_skew1.77658_neigh1.22548_cross0.7151_n14.mtx","synthetic_167070_167070_8908050_avg53.31927_std4.75919_random_bw11844.75739_skew3.3699_neigh1.81234_cross0.94129_n14.mtx","synthetic_167075_167075_13504757_avg80.83051_std117.71597_random_bw22746.915_skew7.87041_neigh1.21275_cross0.84541_n14.mtx","synthetic_1705586_1705586_20897972_avg12.25266_std37.46728_random_bw40228.04846_skew632.73974_neigh1.32222_cross0.78649_n14.mtx","synthetic_173744_173744_9631043_avg55.43238_std11.0756_random_bw6798.57169_skew0.91224_neigh1.62874_cross0.78706_n14.mtx","synthetic_177177_177177_8998963_avg50.79081_std19.6143_random_bw5426.967_skew1.77609_neigh1.22568_cross0.7169_n14.mtx","synthetic_179451_179451_14506771_avg80.83973_std117.71232_random_bw24453.77012_skew7.8694_neigh1.21214_cross0.84477_n14.mtx","synthetic_1797780_1797780_22027651_avg12.2527_std37.45466_random_bw42402.70848_skew632.7381_neigh1.32199_cross0.78712_n14.mtx","synthetic_181598_181598_9682693_avg53.31938_std4.75813_random_bw12869.05691_skew3.36989_neigh1.81177_cross0.94114_n14.mtx","synthetic_183136_183136_10152501_avg55.43695_std11.07295_random_bw7165.08293_skew0.91208_neigh1.63012_cross0.78692_n14.mtx","synthetic_185639_185639_10500260_avg56.56279_std99.5812_random_bw19106.22939_skew7.18559_neigh1.25551_cross0.95442_n14.mtx","synthetic_185639_185639_11501046_avg61.95382_std109.06293_random_bw19420.97121_skew7.19966_neigh1.25396_cross0.95351_n14.mtx","synthetic_185639_185639_12507698_avg67.37646_std112.75287_random_bw24196.24969_skew7.4154_neigh1.18657_cross0.88107_n14.mtx","synthetic_185639_185639_13507262_avg72.76091_std115.00904_random_bw24859.95962_skew7.61726_neigh1.19956_cross0.86027_n14.mtx","synthetic_185639_185639_14507140_avg78.14705_std116.8824_random_bw25219.52172_skew7.80392_neigh1.20963_cross0.84859_n14.mtx","synthetic_185639_185639_15001505_avg80.81009_std108.51788_random_bw25511.99254_skew5.36059_neigh1.20064_cross0.8813_n14.mtx","synthetic_185639_185639_15005658_avg80.83247_std121.76167_random_bw25355.77027_skew10.22074_neigh1.21411_cross0.83223_n14.mtx","synthetic_185639_185639_15005807_avg80.83327_std121.07944_random_bw25384.15564_skew9.70104_neigh1.21342_cross0.83416_n14.mtx","synthetic_185639_185639_15006000_avg80.83431_std120.29592_random_bw25293.65767_skew9.18132_neigh1.21471_cross0.83668_n14.mtx","synthetic_185639_185639_15006279_avg80.83581_std111.39593_random_bw25316.2149_skew5.93999_neigh1.20276_cross0.86868_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_random_bw23672.1534_skew7.8818_neigh1.18863_cross0.87685_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_random_bw23954.47017_skew7.8818_neigh1.19585_cross0.87257_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_random_bw25265.00391_skew7.8818_neigh1.49293_cross0.84614_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_random_bw25293.93346_skew7.8818_neigh1.41168_cross0.84605_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_random_bw25297.50533_skew7.8818_neigh1.33097_cross0.84586_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_random_bw25337.47945_skew7.8818_neigh1.17285_cross0.84478_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_random_bw25357.35315_skew7.8818_neigh1.25202_cross0.84469_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_random_bw25422.38964_skew7.8818_neigh1.09355_cross0.84331_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_random_bw25459.31441_skew7.8818_neigh1.01249_cross0.84366_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_random_bw25490.88255_skew7.8818_neigh0.93357_cross0.84298_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_random_bw25498.90443_skew7.8818_neigh1.57331_cross0.84717_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_random_bw25568.41993_skew7.8818_neigh0.85133_cross0.84267_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_random_bw26253.68657_skew7.8818_neigh1.22152_cross0.81618_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_random_bw27626.6459_skew7.8818_neigh1.23351_cross0.76024_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_random_bw28571.00316_skew7.8818_neigh1.2369_cross0.70583_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_random_bw29320.71745_skew7.8818_neigh1.24185_cross0.64948_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_random_bw29911.00532_skew7.8818_neigh1.24556_cross0.59443_n14.mtx","synthetic_185639_185639_15007163_avg80.84057_std117.05079_random_bw25318.13414_skew7.60954_neigh1.20941_cross0.84686_n14.mtx","synthetic_185639_185639_15007417_avg80.84194_std113.68148_random_bw25329.82537_skew6.50848_neigh1.20564_cross0.85898_n14.mtx","synthetic_185639_185639_15007427_avg80.84199_std115.53272_random_bw25278.26792_skew7.06512_neigh1.20929_cross0.85246_n14.mtx","synthetic_185639_185639_15506741_avg83.5317_std118.48272_random_bw25422.1247_skew7.94271_neigh1.21538_cross0.84172_n14.mtx","synthetic_185639_185639_16506181_avg88.91548_std119.88133_random_bw25572.96292_skew8.06479_neigh1.21966_cross0.83617_n14.mtx","synthetic_185639_185639_17505764_avg94.30003_std121.13382_random_bw25625.10673_skew8.16224_neigh1.22371_cross0.83275_n14.mtx","synthetic_185639_185639_18505682_avg99.68639_std122.27517_random_bw25744.70036_skew8.23897_neigh1.22874_cross0.82863_n14.mtx","synthetic_187667_187667_12936634_avg68.93398_std101.26087_random_bw22637.37919_skew10.08307_neigh1.12453_cross0.83741_n14.mtx","synthetic_187915_187915_9545196_avg50.79529_std19.60651_random_bw5758.22991_skew1.77585_neigh1.22523_cross0.71562_n14.mtx","synthetic_191826_191826_15508397_avg80.84617_std117.70962_random_bw26139.99935_skew7.88106_neigh1.21134_cross0.84514_n14.mtx","synthetic_196126_196126_10457351_avg53.31955_std4.75831_random_bw14006.14808_skew3.36988_neigh1.81257_cross0.94078_n14.mtx","synthetic_198653_198653_10092676_avg50.80556_std19.60715_random_bw6093.46107_skew1.77529_neigh1.22471_cross0.71497_n14.mtx","synthetic_204202_204202_16508082_avg80.84192_std117.71063_random_bw27859.72659_skew7.88153_neigh1.21321_cross0.84455_n14.mtx","synthetic_205540_205540_14169500_avg68.93792_std101.25978_random_bw24815.66416_skew10.08243_neigh1.12421_cross0.83711_n14.mtx","synthetic_209391_209391_10637567_avg50.80241_std19.60148_random_bw6420.80853_skew1.77546_neigh1.22408_cross0.716_n14.mtx","synthetic_210654_210654_11231930_avg53.31933_std4.75723_random_bw14914.26662_skew3.3699_neigh1.81313_cross0.94073_n14.mtx","synthetic_216578_216578_17508080_avg80.83961_std117.71328_random_bw29528.4278_skew7.89416_neigh1.2132_cross0.8447_n14.mtx","synthetic_217918_217918_10485592_avg48.11714_std4.75335_random_bw15844.7796_skew3.36435_neigh1.77209_cross0.93961_n14.mtx","synthetic_217918_217918_11231492_avg51.53999_std4.76893_random_bw15995.38461_skew3.36554_neigh1.75717_cross0.93963_n14.mtx","synthetic_217918_217918_11582841_avg53.15229_std3.34176_random_bw14914.85908_skew3.38363_neigh1.83128_cross0.94011_n14.mtx","synthetic_217918_217918_11590559_avg53.18771_std3.65776_random_bw14955.00461_skew3.38071_neigh1.82908_cross0.93983_n14.mtx","synthetic_217918_217918_11598747_avg53.22528_std3.97324_random_bw15044.77643_skew3.37762_neigh1.82449_cross0.9406_n14.mtx","synthetic_217918_217918_11607157_avg53.26387_std4.28638_random_bw15200.0146_skew3.37445_neigh1.82082_cross0.9401_n14.mtx","synthetic_217918_217918_11608217_avg53.26874_std4.76281_random_bw15269.0692_skew4.08741_neigh1.81967_cross0.94009_n14.mtx","synthetic_217918_217918_11610489_avg53.27916_std4.76243_random_bw15240.4697_skew3.93626_neigh1.81896_cross0.94023_n14.mtx","synthetic_217918_217918_11612905_avg53.29025_std4.75996_random_bw15310.45462_skew3.76635_neigh1.81736_cross0.94035_n14.mtx","synthetic_217918_217918_11615389_avg53.30165_std4.60014_random_bw15350.22672_skew3.37135_neigh1.81586_cross0.94062_n14.mtx","synthetic_217918_217918_11615400_avg53.3017_std4.75902_random_bw15342.03914_skew3.61524_neigh1.81594_cross0.94062_n14.mtx","synthetic_217918_217918_11617960_avg53.31345_std4.75796_random_bw15405.69215_skew3.44541_neigh1.81369_cross0.94068_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_random_bw10737.64739_skew3.36989_neigh1.81318_cross0.94085_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_random_bw11891.67531_skew3.36989_neigh1.81288_cross0.94086_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_random_bw12724.60274_skew3.36989_neigh1.50678_cross0.94071_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_random_bw12730.20473_skew3.36989_neigh1.26613_cross0.9391_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_random_bw12738.48211_skew3.36989_neigh1.38666_cross0.94034_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_random_bw12837.98779_skew3.36989_neigh1.81264_cross0.9409_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_random_bw13058.23403_skew3.36989_neigh1.62674_cross0.94024_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_random_bw13376.58862_skew3.36989_neigh1.74964_cross0.94006_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_random_bw13808.95321_skew3.36989_neigh1.81302_cross0.94102_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_random_bw14783.21932_skew3.36989_neigh1.83882_cross0.66043_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_random_bw14834.61294_skew3.36989_neigh1.83738_cross0.72231_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_random_bw14844.13273_skew3.36989_neigh1.83615_cross0.78285_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_random_bw14910.1263_skew3.36989_neigh1.81269_cross0.94076_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_random_bw15016.8092_skew3.36989_neigh1.83172_cross0.84569_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_random_bw15289.18189_skew3.36989_neigh1.8215_cross0.90858_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_random_bw15805.89794_skew3.36989_neigh1.78683_cross0.97162_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_random_bw15930.0478_skew3.36989_neigh1.81219_cross0.94078_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_random_bw16309.5886_skew3.36989_neigh1.7473_cross0.98961_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_random_bw16886.23663_skew3.36989_neigh1.87576_cross0.93935_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_random_bw16921.52031_skew3.36989_neigh1.81297_cross0.94073_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_random_bw17934.76325_skew3.36989_neigh1.81325_cross0.94068_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_random_bw19000.93916_skew3.36989_neigh1.81255_cross0.94072_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_random_bw19930.43914_skew3.36989_neigh1.81307_cross0.94069_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_random_bw5768.22535_skew3.36989_neigh1.93911_cross0.93966_n14.mtx","synthetic_217918_217918_11620586_avg53.3255_std4.75691_random_bw15468.18448_skew3.29438_neigh1.81163_cross0.94092_n14.mtx","synthetic_217918_217918_11622834_avg53.33581_std4.91358_random_bw15511.53295_skew3.36855_neigh1.80929_cross0.94061_n14.mtx","synthetic_217918_217918_11623241_avg53.33768_std4.75547_random_bw15548.19386_skew3.12466_neigh1.8088_cross0.94062_n14.mtx","synthetic_217918_217918_11625780_avg53.34933_std4.75397_random_bw15624.44027_skew2.97381_neigh1.80583_cross0.94018_n14.mtx","synthetic_217918_217918_11628184_avg53.36036_std4.75308_random_bw15742.20805_skew2.80432_neigh1.80167_cross0.93967_n14.mtx","synthetic_217918_217918_11628620_avg53.36237_std5.22805_random_bw15759.47133_skew3.36637_neigh1.80061_cross0.93977_n14.mtx","synthetic_217918_217918_11630299_avg53.37007_std4.7531_random_bw15847.73759_skew2.65373_neigh1.79719_cross0.93963_n14.mtx","synthetic_217918_217918_11632309_avg53.37929_std5.54522_random_bw16023.97314_skew3.36499_neigh1.78977_cross0.9395_n14.mtx","synthetic_217918_217918_11634099_avg53.38751_std5.86172_random_bw16271.97895_skew3.36432_neigh1.77745_cross0.93875_n14.mtx","synthetic_217918_217918_11634741_avg53.39045_std6.17863_random_bw16478.31237_skew3.36408_neigh1.76344_cross0.93829_n14.mtx","synthetic_217918_217918_12031582_avg55.21151_std4.74931_random_bw15618.56314_skew3.36503_neigh1.82311_cross0.94108_n14.mtx","synthetic_217918_217918_12797820_avg58.72769_std4.7736_random_bw14173.92179_skew3.3591_neigh1.75513_cross0.93936_n14.mtx","synthetic_217918_217918_13561979_avg62.23432_std4.75526_random_bw13352.00925_skew3.37058_neigh1.8388_cross0.94017_n14.mtx","synthetic_217918_217918_14374351_avg65.9622_std4.76726_random_bw14674.102_skew3.36614_neigh1.78599_cross0.9393_n14.mtx","synthetic_217918_217918_15093236_avg69.26108_std4.76589_random_bw14165.84201_skew3.37475_neigh1.82997_cross0.94033_n14.mtx","synthetic_217918_217918_8143542_avg37.36975_std4.75333_random_bw21449.04695_skew3.36182_neigh1.7462_cross0.93898_n14.mtx","synthetic_217918_217918_8924884_avg40.95524_std4.76049_random_bw22967.50218_skew3.34621_neigh1.71344_cross0.93831_n14.mtx","synthetic_217918_217918_9684687_avg44.44189_std4.75752_random_bw14709.39543_skew3.36525_neigh1.77216_cross0.93925_n14.mtx","synthetic_223413_223413_15400297_avg68.93196_std101.25897_random_bw27005.84088_skew10.0979_neigh1.12422_cross0.83714_n14.mtx","synthetic_225181_225181_12006505_avg53.31935_std4.75652_random_bw15784.15411_skew3.3699_neigh1.8126_cross0.94063_n14.mtx","synthetic_228954_228954_18508358_avg80.83876_std117.71434_random_bw31230.9613_skew7.89425_neigh1.21214_cross0.84502_n14.mtx","synthetic_239709_239709_12781051_avg53.31903_std4.75618_random_bw17005.93316_skew3.36992_neigh1.81258_cross0.9406_n14.mtx","synthetic_241286_241286_16633118_avg68.93528_std101.25723_random_bw29154.84234_skew10.09737_neigh1.12198_cross0.83731_n14.mtx","synthetic_241330_241330_19509069_avg80.8398_std117.71435_random_bw32920.56295_skew7.89413_neigh1.21032_cross0.845_n14.mtx","synthetic_254237_254237_13555788_avg53.31949_std4.75562_random_bw17988.99306_skew3.36988_neigh1.81207_cross0.9405_n14.mtx","synthetic_25491_25491_3028356_avg118.80099_std31.7722_random_bw2627.38543_skew1.00335_neigh1.79456_cross0.81089_n14.mtx","synthetic_259159_259159_17866974_avg68.94213_std101.25583_random_bw31249.92925_skew10.11077_neigh1.12114_cross0.83763_n14.mtx","synthetic_26684_26684_10204693_avg382.42741_std245.44826_random_bw12208.0425_skew1.57042_neigh1.97487_cross0.9207_n14.mtx","synthetic_268096_268096_12941160_avg48.27062_std95.20699_random_bw27594.27638_skew9.2754_neigh1.08407_cross0.8943_n14.mtx","synthetic_268096_268096_14172610_avg52.86394_std97.03631_random_bw30537.79348_skew9.5554_neigh1.09138_cross0.86621_n14.mtx","synthetic_268096_268096_15403716_avg57.45597_std98.50696_random_bw31584.47846_skew9.75606_neigh1.10235_cross0.85247_n14.mtx","synthetic_268096_268096_16634948_avg62.04848_std99.72897_random_bw31984.02507_skew9.92694_neigh1.11286_cross0.84502_n14.mtx","synthetic_268096_268096_17866195_avg66.64104_std100.78017_random_bw32220.36555_skew10.05925_neigh1.12121_cross0.83959_n14.mtx","synthetic_268096_268096_18481037_avg68.9344_std103.42196_random_bw32441.44161_skew12.99882_neigh1.12449_cross0.83_n14.mtx","synthetic_268096_268096_18481246_avg68.93518_std102.1488_random_bw32365.8807_skew11.06931_neigh1.12496_cross0.83401_n14.mtx","synthetic_268096_268096_18481318_avg68.93545_std102.6344_random_bw32417.76157_skew11.72205_neigh1.12377_cross0.83241_n14.mtx","synthetic_268096_268096_18481428_avg68.93586_std103.0557_random_bw32473.21371_skew12.36025_neigh1.12371_cross0.83096_n14.mtx","synthetic_268096_268096_18481631_avg68.93662_std101.58036_random_bw32372.11142_skew10.43079_neigh1.12404_cross0.8362_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_random_bw30754.23082_skew10.11157_neigh1.10438_cross0.86329_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_random_bw31287.63614_skew10.11157_neigh1.12125_cross0.83758_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_random_bw32306.79739_skew10.11157_neigh1.15983_cross0.83743_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_random_bw32321.14848_skew10.11157_neigh1.23407_cross0.83798_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_random_bw32329.4436_skew10.11157_neigh1.01031_cross0.83738_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_random_bw32349.06004_skew10.11157_neigh1.38185_cross0.83852_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_random_bw32367.35543_skew10.11157_neigh1.08557_cross0.83678_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_random_bw32375.57571_skew10.11157_neigh0.93684_cross0.8365_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_random_bw32420.97615_skew10.11157_neigh1.30741_cross0.8385_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_random_bw32430.76354_skew10.11157_neigh0.86471_cross0.83616_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_random_bw32452.85556_skew10.11157_neigh0.79_cross0.83549_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_random_bw32557.34156_skew10.11157_neigh1.45611_cross0.83956_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_random_bw33506.09611_skew10.11157_neigh1.12211_cross0.83722_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_random_bw33650.79702_skew10.11157_neigh1.13371_cross0.80829_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_random_bw35478.95576_skew10.11157_neigh1.14236_cross0.75351_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_random_bw35561.27802_skew10.11157_neigh1.12238_cross0.83753_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_random_bw36758.75994_skew10.11157_neigh1.14871_cross0.69834_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_random_bw37755.70411_skew10.11157_neigh1.12209_cross0.83735_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_random_bw37857.24747_skew10.11157_neigh1.15306_cross0.64222_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_random_bw38714.99432_skew10.11157_neigh1.15592_cross0.5881_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_random_bw39930.72019_skew10.11157_neigh1.12081_cross0.83752_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_random_bw42070.66722_skew10.11157_neigh1.12184_cross0.83718_n14.mtx","synthetic_268096_268096_18482060_avg68.93822_std100.90606_random_bw32385.89976_skew9.79227_neigh1.12123_cross0.83833_n14.mtx","synthetic_268096_268096_18482363_avg68.93935_std100.10125_random_bw32276.82294_skew9.13935_neigh1.12203_cross0.84217_n14.mtx","synthetic_268096_268096_18482725_avg68.9407_std99.12285_random_bw32363.10438_skew8.47191_neigh1.12037_cross0.84459_n14.mtx","synthetic_268096_268096_18483147_avg68.94227_std97.91705_random_bw32207.13852_skew7.80447_neigh1.12058_cross0.85009_n14.mtx","synthetic_268096_268096_18483826_avg68.9448_std96.40124_random_bw32262.35735_skew7.10794_neigh1.11413_cross0.85517_n14.mtx","synthetic_268765_268765_14330411_avg53.31948_std4.75541_random_bw19027.02115_skew3.36988_neigh1.81207_cross0.94046_n14.mtx","synthetic_277032_277032_19096094_avg68.931_std101.25944_random_bw33505.32851_skew10.12707_neigh1.12297_cross0.83731_n14.mtx","synthetic_27919_27919_3319237_avg118.8881_std31.78107_random_bw2882.12425_skew1.00188_neigh1.79322_cross0.80796_n14.mtx","synthetic_283293_283293_15104954_avg53.31919_std4.75508_random_bw20060.27324_skew3.36991_neigh1.81213_cross0.94052_n14.mtx","synthetic_29225_29225_11176419_avg382.42666_std245.44724_random_bw13101.62433_skew1.57043_neigh1.97449_cross0.92145_n14.mtx","synthetic_294905_294905_20328825_avg68.93347_std101.25836_random_bw35580.71368_skew10.12667_neigh1.12383_cross0.83765_n14.mtx","synthetic_29496_29496_3296630_avg111.76532_std88.66285_random_bw13978.43416_skew3.35734_neigh1.89349_cross0.6274_n14.mtx","synthetic_2998_2998_6123474_avg2042.51968_std621.91091_random_bw2814.28252_skew0.46779_neigh1.82839_cross0.81924_n14.mtx","synthetic_30347_30347_3608195_avg118.89791_std31.76143_random_bw3114.986_skew1.00172_neigh1.79361_cross0.81079_n14.mtx","synthetic_312778_312778_21563736_avg68.94262_std101.25558_random_bw37752.7149_skew10.1397_neigh1.12125_cross0.83735_n14.mtx","synthetic_31766_31766_12148131_avg382.42558_std245.44713_random_bw14374.34849_skew1.57043_neigh1.97472_cross0.91791_n14.mtx","synthetic_32305_32305_3613009_avg111.84055_std88.75337_random_bw15339.91778_skew3.35441_neigh1.8933_cross0.62712_n14.mtx","synthetic_32775_32775_3898242_avg118.9395_std31.78937_random_bw3355.91933_skew1.00102_neigh1.79257_cross0.81174_n14.mtx","synthetic_3283001_3283001_13139084_avg4.00216_std1.21615_random_bw3180.83136_skew312.8308_neigh0.41641_cross0.6482_n14.mtx","synthetic_3284_3284_6742726_avg2053.20524_std666.12092_random_bw3090.98173_skew0.59945_neigh1.79209_cross0.78602_n14.mtx","synthetic_330651_330651_22801392_avg68.95909_std101.25183_random_bw39914.18466_skew10.13704_neigh1.12248_cross0.83693_n14.mtx","synthetic_34308_34308_13120227_avg382.42471_std245.44609_random_bw15373.72135_skew1.57044_neigh1.97444_cross0.91794_n14.mtx","synthetic_348524_348524_24034194_avg68.95994_std101.25128_random_bw42061.27886_skew10.1369_neigh1.12113_cross0.83729_n14.mtx","synthetic_35115_35115_3931279_avg111.95441_std88.67056_random_bw16758.071_skew3.34999_neigh1.89338_cross0.62737_n14.mtx","synthetic_35203_35203_4188100_avg118.96997_std31.72558_random_bw3622.57756_skew1.0005_neigh1.79296_cross0.80889_n14.mtx","synthetic_3569_3569_7446679_avg2086.48893_std694.14116_random_bw3307.12945_skew0.71053_neigh1.77466_cross0.76435_n14.mtx","synthetic_3595668_3595668_14390363_avg4.00214_std1.20637_random_bw3483.23396_skew312.83218_neigh0.41589_cross0.64799_n14.mtx","synthetic_36417_36417_3031503_avg83.24417_std31.62901_random_bw3778.50658_skew1.4266_neigh1.77891_cross0.76976_n14.mtx","synthetic_36417_36417_3320284_avg91.17401_std31.69422_random_bw3853.70876_skew1.30329_neigh1.78298_cross0.77984_n14.mtx","synthetic_36417_36417_3609556_avg99.11734_std31.72484_random_bw3797.36502_skew1.19941_neigh1.78594_cross0.79053_n14.mtx","synthetic_36417_36417_3899087_avg107.06777_std31.73599_random_bw3752.97905_skew1.11081_neigh1.78782_cross0.80067_n14.mtx","synthetic_36417_36417_4188724_avg115.02112_std31.74031_random_bw3797.6023_skew1.03441_neigh1.79206_cross0.80583_n14.mtx","synthetic_36417_36417_4330804_avg118.92259_std41.20033_random_bw3675.94291_skew1.30402_neigh1.79795_cross0.77953_n14.mtx","synthetic_36417_36417_4331206_avg118.93363_std39.11615_random_bw3697.36082_skew1.23654_neigh1.79608_cross0.7852_n14.mtx","synthetic_36417_36417_4331707_avg118.94739_std37.01905_random_bw3704.02521_skew1.16903_neigh1.79589_cross0.79365_n14.mtx","synthetic_36417_36417_4332360_avg118.96532_std34.9075_random_bw3710.06437_skew1.10145_neigh1.79431_cross0.8013_n14.mtx","synthetic_36417_36417_4333163_avg118.98737_std32.79928_random_bw3723.49809_skew1.03383_neigh1.79539_cross0.80667_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_random_bw2623.52629_skew1.00003_neigh1.79451_cross0.81215_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_random_bw2877.98784_skew1.00003_neigh1.79304_cross0.80927_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_random_bw3116.1024_skew1.00003_neigh1.79446_cross0.81024_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_random_bw3362.22435_skew1.00003_neigh1.79146_cross0.81185_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_random_bw3467.27641_skew1.00003_neigh1.77473_cross0.86839_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_random_bw3597.87423_skew1.00003_neigh1.61979_cross0.81098_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_random_bw3598.07988_skew1.00003_neigh1.50455_cross0.81028_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_random_bw3599.29363_skew1.00003_neigh1.27022_cross0.8107_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_random_bw3601.93981_skew1.00003_neigh1.38778_cross0.81049_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_random_bw3621.76799_skew1.00003_neigh1.79294_cross0.80868_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_random_bw3628.53802_skew1.00003_neigh1.78888_cross0.83631_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_random_bw3688.34179_skew1.00003_neigh1.73475_cross0.80937_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_random_bw3736.64409_skew1.00003_neigh1.79285_cross0.80866_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_random_bw3824.14642_skew1.00003_neigh1.79702_cross0.78418_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_random_bw3846.82772_skew1.00003_neigh1.79307_cross0.81062_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_random_bw3974.32482_skew1.00003_neigh1.80409_cross0.73066_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_random_bw4041.25675_skew1.00003_neigh1.85155_cross0.80883_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_random_bw4082.51341_skew1.00003_neigh1.80817_cross0.68078_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_random_bw4114.70692_skew1.00003_neigh1.79169_cross0.81094_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_random_bw4164.96568_skew1.00003_neigh1.81125_cross0.63039_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_random_bw4243.70544_skew1.00003_neigh1.81309_cross0.57648_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_random_bw4381.77057_skew1.00003_neigh1.79261_cross0.80821_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_random_bw4600.41277_skew1.00003_neigh1.79347_cross0.80785_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_random_bw4827.23959_skew1.00003_neigh1.79176_cross0.81018_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_random_bw5452.08949_skew1.00003_neigh1.94835_cross0.80771_n14.mtx","synthetic_36417_36417_4333866_avg119.00667_std30.68416_random_bw3732.46783_skew0.96628_neigh1.79286_cross0.81383_n14.mtx","synthetic_36417_36417_4334685_avg119.02916_std28.56617_random_bw3758.80797_skew0.89869_neigh1.79181_cross0.82202_n14.mtx","synthetic_36417_36417_4335436_avg119.04978_std26.44817_random_bw3785.04915_skew0.83117_neigh1.78962_cross0.82614_n14.mtx","synthetic_36417_36417_4336151_avg119.06942_std24.33643_random_bw3809.09534_skew0.76368_neigh1.78823_cross0.83433_n14.mtx","synthetic_36417_36417_4336875_avg119.0893_std22.21912_random_bw3824.95716_skew0.7046_neigh1.78872_cross0.84268_n14.mtx","synthetic_36417_36417_4478356_avg122.97433_std31.74172_random_bw3753.51149_skew0.96789_neigh1.79439_cross0.81446_n14.mtx","synthetic_36417_36417_4767999_avg130.92784_std31.7426_random_bw3736.97166_skew0.90945_neigh1.79577_cross0.81963_n14.mtx","synthetic_36417_36417_5057685_avg138.88253_std31.74387_random_bw3770.7999_skew0.85769_neigh1.79863_cross0.82586_n14.mtx","synthetic_36417_36417_5347290_avg146.83499_std31.74228_random_bw3749.94288_skew0.81156_neigh1.8002_cross0.83033_n14.mtx","synthetic_36417_36417_5636923_avg154.78823_std31.74188_random_bw3744.12662_skew0.77016_neigh1.80391_cross0.8341_n14.mtx","synthetic_36849_36849_14091918_avg382.42335_std245.44508_random_bw16368.21697_skew1.57045_neigh1.97457_cross0.91965_n14.mtx","synthetic_37630_37630_4477645_avg118.99136_std31.72824_random_bw3844.70563_skew1.00015_neigh1.793_cross0.81096_n14.mtx","synthetic_37924_37924_4245700_avg111.95285_std88.65794_random_bw18034.65215_skew3.35005_neigh1.89352_cross0.62758_n14.mtx","synthetic_38120_38120_10204584_avg267.69633_std171.81206_random_bw17167.82778_skew1.57008_neigh1.97267_cross0.91545_n14.mtx","synthetic_38120_38120_11176434_avg293.19082_std188.17435_random_bw18077.60719_skew1.5717_neigh1.97255_cross0.91793_n14.mtx","synthetic_38120_38120_12148318_avg318.6862_std204.53731_random_bw19160.42267_skew1.56993_neigh1.97325_cross0.91894_n14.mtx","synthetic_38120_38120_13120182_avg344.18106_std220.90077_random_bw19517.48116_skew1.57132_neigh1.97392_cross0.91784_n14.mtx","synthetic_38120_38120_14092040_avg369.67576_std237.2634_random_bw16796.84077_skew1.56982_neigh1.9745_cross0.91796_n14.mtx","synthetic_38120_38120_14186695_avg372.15884_std217.74813_random_bw17520.62065_skew1.39145_neigh1.97428_cross0.91992_n14.mtx","synthetic_38120_38120_14353565_avg376.53633_std228.90355_random_bw17193.09321_skew1.46191_neigh1.97454_cross0.91856_n14.mtx","synthetic_38120_38120_14506403_avg380.54572_std239.95763_random_bw16970.09357_skew1.5332_neigh1.97424_cross0.92361_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_random_bw16928.18211_skew1.57045_neigh1.97488_cross0.70725_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_random_bw16978.22802_skew1.57045_neigh1.97475_cross0.6468_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_random_bw16980.42762_skew1.57045_neigh1.97471_cross0.88897_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_random_bw17047.29614_skew1.57045_neigh1.9748_cross0.76884_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_random_bw17164.09806_skew1.57045_neigh1.9747_cross0.82515_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_random_bw17288.93704_skew1.57045_neigh1.97422_cross0.94898_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_random_bw17337.86668_skew1.57045_neigh1.9736_cross0.99006_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_random_bw17436.03269_skew1.57045_neigh1.90981_cross0.91796_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_random_bw17997.22335_skew1.57045_neigh1.77798_cross0.91958_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_random_bw18145.65771_skew1.57045_neigh1.65034_cross0.92138_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_random_bw18235.12718_skew1.57045_neigh1.51713_cross0.921_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_random_bw18240.43122_skew1.57045_neigh1.3892_cross0.9198_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_random_bw5058.0712_skew1.57045_neigh1.99412_cross0.92723_n14.mtx","synthetic_38120_38120_14646455_avg384.2197_std250.9036_random_bw17379.7798_skew1.60788_neigh1.97452_cross0.92139_n14.mtx","synthetic_38120_38120_14774683_avg387.5835_std261.73289_random_bw17213.87985_skew1.68071_neigh1.97461_cross0.91994_n14.mtx","synthetic_38120_38120_14892124_avg390.66432_std272.44037_random_bw17256.11215_skew1.75428_neigh1.97475_cross0.9194_n14.mtx","synthetic_38120_38120_15029193_avg394.26005_std283.24138_random_bw20307.43822_skew1.83569_neigh1.96002_cross0.918_n14.mtx","synthetic_38120_38120_15063906_avg395.17067_std253.62681_random_bw17391.16758_skew1.57104_neigh1.97506_cross0.9205_n14.mtx","synthetic_38120_38120_15246247_avg399.95401_std294.48908_random_bw20475.55488_skew1.94534_neigh1.95593_cross0.90935_n14.mtx","synthetic_38120_38120_16417883_avg430.68948_std273.30431_random_bw20513.51482_skew1.63763_neigh1.95832_cross0.8988_n14.mtx","synthetic_38120_38120_17911818_avg469.8798_std294.64729_random_bw17768.78305_skew1.77305_neigh1.96177_cross0.87832_n14.mtx","synthetic_3855_3855_8084357_avg2097.10947_std723.99546_random_bw3581.91751_skew0.83824_neigh1.7534_cross0.73513_n14.mtx","synthetic_3908334_3908334_15641636_avg4.00212_std1.19836_random_bw3788.22744_skew312.83338_neigh0.41514_cross0.64836_n14.mtx","synthetic_39390_39390_15063634_avg382.4228_std245.44522_random_bw17726.0756_skew1.57045_neigh1.97466_cross0.91891_n14.mtx","synthetic_40058_40058_4768887_avg119.04955_std31.78281_random_bw4114.1045_skew0.99917_neigh1.79221_cross0.81063_n14.mtx","synthetic_40733_40733_4573744_avg112.28596_std88.81893_random_bw19403.9372_skew3.33714_neigh1.89379_cross0.62734_n14.mtx","synthetic_4141_4141_8723864_avg2106.70466_std753.89371_random_bw3856.85245_skew0.96563_neigh1.73618_cross0.70892_n14.mtx","synthetic_41932_41932_16035737_avg382.42242_std245.44481_random_bw18664.87153_skew1.57046_neigh1.9748_cross0.9208_n14.mtx","synthetic_42138_42138_3666754_avg87.01775_std81.65352_random_bw19853.31741_skew4.24031_neigh1.89219_cross0.57666_n14.mtx","synthetic_42138_42138_3893764_avg92.40505_std83.38574_random_bw17974.59227_skew4.01055_neigh1.89331_cross0.57355_n14.mtx","synthetic_42138_42138_3928503_avg93.22946_std52.30654_random_bw27312.13121_skew1.38122_neigh1.71236_cross0.97097_n14.mtx","synthetic_42138_42138_4006220_avg95.07381_std54.75201_random_bw26439.50325_skew1.47176_neigh1.72394_cross0.96381_n14.mtx","synthetic_42138_42138_4072421_avg96.64486_std57.11798_random_bw25980.40367_skew1.5661_neigh1.74415_cross0.95788_n14.mtx","synthetic_42138_42138_4126809_avg97.93557_std85.02631_random_bw18557.95631_skew3.79907_neigh1.8936_cross0.5899_n14.mtx","synthetic_42138_42138_4128960_avg97.98662_std59.40057_random_bw25487.21109_skew1.65342_neigh1.7633_cross0.95074_n14.mtx","synthetic_42138_42138_4176699_avg99.11954_std61.5954_random_bw24959.75837_skew1.74416_neigh1.77059_cross0.94519_n14.mtx","synthetic_42138_42138_4365370_avg103.59699_std86.57531_random_bw19147.75084_skew3.60438_neigh1.89374_cross0.60541_n14.mtx","synthetic_42138_42138_4491637_avg106.5935_std66.96099_random_bw21515.96234_skew2.48989_neigh1.88186_cross0.70673_n14.mtx","synthetic_42138_42138_4533678_avg107.5912_std72.07832_random_bw21100.98446_skew2.69919_neigh1.88564_cross0.68688_n14.mtx","synthetic_42138_42138_4582731_avg108.7553_std77.01103_random_bw20786.03004_skew2.88947_neigh1.88766_cross0.66816_n14.mtx","synthetic_42138_42138_4608940_avg109.37728_std88.04095_random_bw19683.90524_skew3.42505_neigh1.89317_cross0.62053_n14.mtx","synthetic_42138_42138_4638390_avg110.07618_std81.79532_random_bw20354.8565_skew3.07899_neigh1.89046_cross0.65066_n14.mtx","synthetic_42138_42138_4699816_avg111.53391_std86.4582_random_bw20073.78231_skew3.24983_neigh1.89286_cross0.635_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_random_bw14002.42427_skew3.33605_neigh1.89425_cross0.62763_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_random_bw14320.37603_skew3.33605_neigh1.97024_cross0.65977_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_random_bw15315.12146_skew3.33605_neigh1.89405_cross0.62765_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_random_bw15931.56866_skew3.33605_neigh1.32686_cross0.62684_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_random_bw15959.80735_skew3.33605_neigh1.45483_cross0.62671_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_random_bw16095.01927_skew3.33605_neigh1.57976_cross0.62768_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_random_bw16296.61787_skew3.33605_neigh1.7053_cross0.62795_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_random_bw16555.88737_skew3.33605_neigh1.89387_cross0.6282_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_random_bw16953.95005_skew3.33605_neigh1.83072_cross0.62763_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_random_bw17957.18508_skew3.33605_neigh1.89403_cross0.62819_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_random_bw18224.68475_skew3.33605_neigh1.95842_cross0.66307_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_random_bw19284.41321_skew3.33605_neigh1.89382_cross0.62757_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_random_bw19954.55513_skew3.33605_neigh1.89279_cross0.63258_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_random_bw19982.02784_skew3.33605_neigh1.89327_cross0.62802_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_random_bw20091.36831_skew3.33605_neigh1.89497_cross0.60786_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_random_bw20234.19666_skew3.33605_neigh1.89725_cross0.563_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_random_bw20448.61299_skew3.33605_neigh1.89904_cross0.52249_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_random_bw20616.53341_skew3.33605_neigh1.90059_cross0.4835_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_random_bw20714.32773_skew3.33605_neigh1.90175_cross0.44229_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_random_bw20750.59585_skew3.33605_neigh1.8937_cross0.62968_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_random_bw21583.20827_skew3.33605_neigh1.89368_cross0.63603_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_random_bw22018.06151_skew3.33605_neigh1.89495_cross0.63936_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_random_bw22492.64431_skew3.33605_neigh1.89475_cross0.64482_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_random_bw22731.73506_skew3.33605_neigh1.89483_cross0.6471_n14.mtx","synthetic_42138_42138_4766804_avg113.12364_std90.99737_random_bw19857.30628_skew3.41994_neigh1.89446_cross0.62081_n14.mtx","synthetic_42138_42138_4838257_avg114.81933_std95.4594_random_bw19620.37349_skew3.5724_neigh1.89654_cross0.6073_n14.mtx","synthetic_42138_42138_4857446_avg115.27472_std89.41484_random_bw20263.25834_skew3.25939_neigh1.89365_cross0.63484_n14.mtx","synthetic_42138_42138_4913422_avg116.60311_std99.85384_random_bw19444.31781_skew3.72543_neigh1.89784_cross0.59642_n14.mtx","synthetic_42138_42138_4991872_avg118.46485_std104.19584_random_bw17244.71835_skew3.8622_neigh1.89896_cross0.58472_n14.mtx","synthetic_42138_42138_5073389_avg120.39938_std108.48726_random_bw17138.19232_skew4.00003_neigh1.90049_cross0.57502_n14.mtx","synthetic_42138_42138_5111019_avg121.2924_std90.68096_random_bw18636.57063_skew3.10578_neigh1.89429_cross0.64795_n14.mtx","synthetic_42138_42138_5368820_avg127.41041_std91.85872_random_bw19117.17587_skew2.96357_neigh1.89434_cross0.66066_n14.mtx","synthetic_42138_42138_5630518_avg133.62091_std92.94843_random_bw19407.61094_skew2.83174_neigh1.89425_cross0.67317_n14.mtx","synthetic_42138_42138_5895869_avg139.9181_std93.95081_random_bw19763.9757_skew2.70931_neigh1.8945_cross0.68576_n14.mtx","synthetic_4221001_4221001_16892920_avg4.00211_std1.19165_random_bw4086.05171_skew312.83427_neigh0.41605_cross0.64812_n14.mtx","synthetic_42486_42486_5059907_avg119.09587_std31.75092_random_bw4380.455_skew1.14113_neigh1.79204_cross0.80881_n14.mtx","synthetic_4284_4284_10161733_avg2372.01984_std415.11429_random_bw4053.5726_skew0.80606_neigh1.72262_cross0.77626_n14.mtx","synthetic_4284_4284_10296320_avg2403.43604_std664.88425_random_bw4000.62278_skew0.78245_neigh1.74482_cross0.76139_n14.mtx","synthetic_4284_4284_11128593_avg2597.71078_std612.25995_random_bw4020.83217_skew0.64914_neigh1.75863_cross0.79849_n14.mtx","synthetic_4284_4284_11955257_avg2790.67624_std565.77456_random_bw4036.47432_skew0.53511_neigh1.7748_cross0.83525_n14.mtx","synthetic_4284_4284_12776299_avg2982.32937_std524.4261_random_bw4061.7514_skew0.43646_neigh1.79331_cross0.868_n14.mtx","synthetic_4284_4284_5176835_avg1208.41153_std875.52905_random_bw3946.70121_skew2.54515_neigh1.71142_cross0.50146_n14.mtx","synthetic_4284_4284_6045664_avg1411.21942_std813.05724_random_bw3941.44468_skew2.03567_neigh1.70361_cross0.54989_n14.mtx","synthetic_4284_4284_6908449_avg1612.61648_std777.77715_random_bw3940.60668_skew1.65655_neigh1.70614_cross0.59717_n14.mtx","synthetic_4284_4284_7606767_avg1775.62255_std826.63023_random_bw3957.25163_skew1.41267_neigh1.72072_cross0.62851_n14.mtx","synthetic_4284_4284_7764407_avg1812.41993_std766.39496_random_bw3956.24113_skew1.36369_neigh1.71331_cross0.63304_n14.mtx","synthetic_4284_4284_7948514_avg1855.39542_std808.46009_random_bw3960.53291_skew1.30894_neigh1.72141_cross0.64445_n14.mtx","synthetic_4284_4284_8278029_avg1932.31303_std796.54977_random_bw3961.24043_skew1.21703_neigh1.72325_cross0.6619_n14.mtx","synthetic_4284_4284_8399652_avg1960.70308_std840.91619_random_bw3956.79902_skew1.18493_neigh1.7318_cross0.66546_n14.mtx","synthetic_4284_4284_8563923_avg1999.04832_std817.99013_random_bw3958.27801_skew1.14302_neigh1.73026_cross0.67074_n14.mtx","synthetic_4284_4284_8591948_avg2005.5901_std788.67453_random_bw3972.91223_skew1.13603_neigh1.72562_cross0.67423_n14.mtx","synthetic_4284_4284_8613657_avg2010.65756_std774.7613_random_bw3958.64706_skew1.13065_neigh1.72447_cross0.67632_n14.mtx","synthetic_4284_4284_8716487_avg2034.66083_std804.2751_random_bw3962.64192_skew1.10551_neigh1.73048_cross0.68001_n14.mtx","synthetic_4284_4284_8853517_avg2066.64729_std794.27725_random_bw3951.22479_skew1.07292_neigh1.7316_cross0.68495_n14.mtx","synthetic_4284_4284_8891530_avg2075.52054_std785.22497_random_bw3958.54809_skew1.06406_neigh1.7304_cross0.68853_n14.mtx","synthetic_4284_4284_8977954_avg2095.69421_std780.20568_random_bw3962.95075_skew1.04419_neigh1.73103_cross0.68744_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_random_bw2869.04809_skew1.03117_neigh1.83364_cross0.82024_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_random_bw3085.78268_skew1.03117_neigh1.80212_cross0.7937_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_random_bw3110.41713_skew1.03117_neigh1.99854_cross0.71407_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_random_bw3327.52498_skew1.03117_neigh1.77515_cross0.76336_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_random_bw3588.97456_skew1.03117_neigh1.75307_cross0.73669_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_random_bw3823.53688_skew1.03117_neigh1.73708_cross0.7132_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_random_bw3910.22712_skew1.03117_neigh1.7316_cross0.71867_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_random_bw3938.6781_skew1.03117_neigh1.73122_cross0.71141_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_random_bw3943.74907_skew1.03117_neigh1.73054_cross0.70283_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_random_bw3949.98413_skew1.03117_neigh1.7303_cross0.69957_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_random_bw3951.74276_skew1.03117_neigh1.70211_cross0.70719_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_random_bw3960.64356_skew1.03117_neigh1.82061_cross0.66726_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_random_bw3963.83333_skew1.03117_neigh1.75897_cross0.68514_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_random_bw3966.8373_skew1.03117_neigh1.72932_cross0.69638_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_random_bw3969.17367_skew1.03117_neigh1.88668_cross0.64196_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_random_bw3972.03175_skew1.03117_neigh1.64659_cross0.72312_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_random_bw3972.66503_skew1.03117_neigh1.72911_cross0.69369_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_random_bw3985.19958_skew1.03117_neigh1.5959_cross0.73688_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_random_bw3987.60574_skew1.03117_neigh1.7287_cross0.69077_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_random_bw4000.61111_skew1.03117_neigh1.72832_cross0.6858_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_random_bw4001.37395_skew1.03117_neigh1.54602_cross0.75029_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_random_bw4004.8268_skew1.03117_neigh1.9578_cross0.61085_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_random_bw4005.42204_skew1.03117_neigh1.72805_cross0.6856_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_random_bw4006.26377_skew1.03117_neigh1.72822_cross0.67994_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_random_bw4022.77614_skew1.03117_neigh1.49943_cross0.76079_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_random_bw4107.28268_skew1.03117_neigh1.72254_cross0.68493_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_random_bw4115.8254_skew1.03117_neigh1.72168_cross0.68593_n14.mtx","synthetic_4284_4284_9091186_avg2122.12558_std733.44441_random_bw3969.63002_skew1.01873_neigh1.72798_cross0.70101_n14.mtx","synthetic_4284_4284_9176988_avg2142.15406_std713.01815_random_bw3972.35014_skew0.99986_neigh1.72769_cross0.70804_n14.mtx","synthetic_4284_4284_9196527_avg2146.71499_std692.64997_random_bw3992.58567_skew0.99561_neigh1.72588_cross0.71021_n14.mtx","synthetic_4284_4284_9292087_avg2169.02124_std655.40775_random_bw3988.24556_skew0.97508_neigh1.72422_cross0.71568_n14.mtx","synthetic_4284_4284_9379803_avg2189.4965_std621.87428_random_bw4005.78431_skew0.95661_neigh1.72224_cross0.72544_n14.mtx","synthetic_4284_4284_9448827_avg2205.60854_std630.77463_random_bw4007.62162_skew0.94232_neigh1.72532_cross0.7253_n14.mtx","synthetic_4284_4284_9456745_avg2207.45682_std723.4564_random_bw3975.21195_skew0.94069_neigh1.73405_cross0.72004_n14.mtx","synthetic_4284_4284_9460554_avg2208.34594_std591.75846_random_bw3997.86905_skew0.93991_neigh1.72216_cross0.73111_n14.mtx","synthetic_4284_4284_9703417_avg2265.03665_std553.19719_random_bw4014.98599_skew0.89136_neigh1.72415_cross0.74686_n14.mtx","synthetic_4284_4284_9941769_avg2320.67437_std481.87027_random_bw4030.17951_skew0.84602_neigh1.72296_cross0.75864_n14.mtx","synthetic_43542_43542_4892832_avg112.3704_std88.7642_random_bw20697.51918_skew3.82333_neigh1.89327_cross0.62809_n14.mtx","synthetic_4426_4426_9354099_avg2113.44306_std776.70231_random_bw4087.97854_skew1.09421_neigh1.72575_cross0.6866_n14.mtx","synthetic_44473_44473_17007427_avg382.4214_std245.44406_random_bw19571.46993_skew1.57046_neigh1.97497_cross0.92019_n14.mtx","synthetic_44686_44686_9904566_avg221.64808_std96.54092_random_bw36393.77102_skew15.40889_neigh1.70401_cross0.85291_n14.mtx","synthetic_44914_44914_5350806_avg119.13448_std31.75488_random_bw4611.11074_skew1.14044_neigh1.79301_cross0.80788_n14.mtx","synthetic_4533668_4533668_18144199_avg4.0021_std1.18568_random_bw4391.88464_skew312.83513_neigh0.41656_cross0.6482_n14.mtx","synthetic_46351_46351_5209319_avg112.38849_std88.86396_random_bw22009.55751_skew3.82256_neigh1.89342_cross0.62869_n14.mtx","synthetic_46524_46524_7261288_avg156.07618_std357.60767_random_bw27277.78536_skew297.08521_neigh1.84057_cross0.94343_n14.mtx","synthetic_4690002_4690002_14082483_avg3.00266_std1.15153_random_bw5393.19648_skew319.7156_neigh0.36894_cross0.64814_n14.mtx","synthetic_4690002_4690002_14083544_avg3.00289_std1.14359_random_bw5393.22819_skew291.71835_neigh0.36895_cross0.64814_n14.mtx","synthetic_4690002_4690002_18765165_avg4.0011_std0.88465_random_bw4545.45148_skew312.91371_neigh0.41524_cross0.6488_n14.mtx","synthetic_4690002_4690002_18766060_avg4.00129_std0.94875_random_bw4545.42947_skew312.89874_neigh0.41525_cross0.6488_n14.mtx","synthetic_4690002_4690002_18767037_avg4.0015_std1.01413_random_bw4545.42698_skew312.88239_neigh0.41527_cross0.6488_n14.mtx","synthetic_4690002_4690002_18767925_avg4.00169_std1.23629_random_bw4545.43862_skew406.578_neigh0.4153_cross0.6488_n14.mtx","synthetic_4690002_4690002_18768094_avg4.00172_std1.08092_random_bw4545.42342_skew312.86472_neigh0.41527_cross0.6488_n14.mtx","synthetic_4690002_4690002_18768255_avg4.00176_std1.22278_random_bw4545.43101_skew385.58006_neigh0.41529_cross0.6488_n14.mtx","synthetic_4690002_4690002_18768639_avg4.00184_std1.21074_random_bw4545.44234_skew364.83169_neigh0.41529_cross0.6488_n14.mtx","synthetic_4690002_4690002_18769069_avg4.00193_std1.19917_random_bw4545.4511_skew344.08333_neigh0.4153_cross0.6488_n14.mtx","synthetic_4690002_4690002_18769236_avg4.00197_std1.1487_random_bw4545.4197_skew312.84562_neigh0.41529_cross0.6488_n14.mtx","synthetic_4690002_4690002_18769566_avg4.00204_std1.18832_random_bw4545.4388_skew323.33476_neigh0.41528_cross0.6488_n14.mtx","synthetic_4690002_4690002_18769585_avg4.00204_std1.18853_random_bw4545.45064_skew323.33443_neigh0.41531_cross0.6488_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_random_bw3185.35558_skew312.8356_neigh0.41546_cross0.64898_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_random_bw3466.9726_skew312.8356_neigh0.41544_cross0.64891_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_random_bw3796.51273_skew312.8356_neigh0.41538_cross0.64888_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_random_bw4075.34022_skew312.8356_neigh0.41535_cross0.64886_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_random_bw4201.23754_skew312.8356_neigh0.54031_cross0.64874_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_random_bw4280.60386_skew312.8356_neigh0.51221_cross0.64875_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_random_bw4358.8812_skew312.8356_neigh0.48441_cross0.64883_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_random_bw4407.96081_skew312.8356_neigh0.41528_cross0.64881_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_random_bw4434.70451_skew312.8356_neigh0.45658_cross0.6486_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_random_bw4508.95932_skew312.8356_neigh0.429_cross0.64869_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_random_bw4543.29273_skew312.8356_neigh0.41514_cross0.58411_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_random_bw4544.09158_skew312.8356_neigh0.4154_cross0.62714_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_random_bw4544.19255_skew312.8356_neigh0.41607_cross0.84262_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_random_bw4544.8615_skew312.8356_neigh0.41542_cross0.45413_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_random_bw4545.23971_skew312.8356_neigh0.41497_cross0.67016_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_random_bw4545.54195_skew312.8356_neigh0.41601_cross0.79982_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_random_bw4545.90204_skew312.8356_neigh0.41502_cross0.54071_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_random_bw4546.18642_skew312.8356_neigh0.4153_cross0.49753_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_random_bw4547.86071_skew312.8356_neigh0.41468_cross0.71352_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_random_bw4548.10776_skew312.8356_neigh0.41515_cross0.7566_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_random_bw4582.29087_skew312.8356_neigh0.40122_cross0.6487_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_random_bw4653.77434_skew312.8356_neigh0.37381_cross0.64862_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_random_bw4685.95172_skew312.8356_neigh0.41524_cross0.64878_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_random_bw4721.25686_skew312.8356_neigh0.34632_cross0.64851_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_random_bw4788.70054_skew312.8356_neigh0.31846_cross0.64863_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_random_bw4855.60684_skew312.8356_neigh0.29083_cross0.64871_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_random_bw5014.52865_skew312.8356_neigh0.41525_cross0.64877_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_random_bw5290.58011_skew312.8356_neigh0.41527_cross0.64873_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_random_bw5624.25737_skew312.8356_neigh0.41525_cross0.64872_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_random_bw5905.56921_skew312.8356_neigh0.4152_cross0.64868_n14.mtx","synthetic_4690002_4690002_18770121_avg4.00216_std1.17768_random_bw4545.45419_skew302.33648_neigh0.4153_cross0.6488_n14.mtx","synthetic_4690002_4690002_18770136_avg4.00216_std1.17836_random_bw4545.46735_skew302.5861_neigh0.4153_cross0.6488_n14.mtx","synthetic_4690002_4690002_18770459_avg4.00223_std1.21745_random_bw4545.43962_skew312.82517_neigh0.4153_cross0.6488_n14.mtx","synthetic_4690002_4690002_18770757_avg4.00229_std1.16774_random_bw4545.45494_skew281.58808_neigh0.4153_cross0.6488_n14.mtx","synthetic_4690002_4690002_18770794_avg4.0023_std1.16864_random_bw4545.45869_skew281.83738_neigh0.4153_cross0.6488_n14.mtx","synthetic_4690002_4690002_18771502_avg4.00245_std1.15845_random_bw4545.49654_skew260.58973_neigh0.41529_cross0.6488_n14.mtx","synthetic_4690002_4690002_18771567_avg4.00246_std1.15985_random_bw4545.48401_skew260.83867_neigh0.41529_cross0.6488_n14.mtx","synthetic_4690002_4690002_18771765_avg4.00251_std1.28647_random_bw4545.47248_skew312.80334_neigh0.41532_cross0.6488_n14.mtx","synthetic_4690002_4690002_18773155_avg4.0028_std1.35648_random_bw4545.50339_skew312.7801_neigh0.41532_cross0.6488_n14.mtx","synthetic_4690002_4690002_18773573_avg4.00289_std1.14442_random_bw4545.51523_skew219.34068_neigh0.4153_cross0.6488_n14.mtx","synthetic_4690002_4690002_18774622_avg4.00312_std1.42675_random_bw4545.47999_skew312.75559_neigh0.41534_cross0.6488_n14.mtx","synthetic_4690002_4690002_18776169_avg4.00345_std1.497_random_bw4545.44654_skew312.72973_neigh0.41535_cross0.6488_n14.mtx","synthetic_4690002_4690002_23458256_avg5.00176_std1.22307_random_bw4356.33337_skew308.6911_neigh0.44326_cross0.64833_n14.mtx","synthetic_4690002_4690002_23458625_avg5.00184_std1.21051_random_bw4356.3482_skew292.09232_neigh0.44326_cross0.64833_n14.mtx","synthetic_4690002_4690002_23459038_avg5.00192_std1.19839_random_bw4356.34599_skew275.29363_neigh0.44327_cross0.64833_n14.mtx","synthetic_4690002_4690002_28147894_avg6.00168_std1.23536_random_bw4777.70775_skew271.09045_neigh0.46196_cross0.6485_n14.mtx","synthetic_47014_47014_17979151_avg382.42121_std245.44364_random_bw21213.06834_skew1.57046_neigh1.97471_cross0.91776_n14.mtx","synthetic_4712_4712_10000031_avg2122.24767_std807.79622_random_bw4350.07343_skew1.22029_neigh1.71541_cross0.66543_n14.mtx","synthetic_47342_47342_5640490_avg119.14347_std31.78033_random_bw4829.64226_skew1.14028_neigh1.79166_cross0.81115_n14.mtx","synthetic_47684_47684_3704607_avg77.69078_std983.84259_random_bw1475.52561_skew612.76655_neigh1.33846_cross0.83857_n14.mtx","synthetic_4846335_4846335_19395478_avg4.00209_std1.18059_random_bw4694.95912_skew312.83587_neigh0.41556_cross0.64846_n14.mtx","synthetic_48942_48942_10847397_avg221.6378_std96.4853_random_bw40035.34643_skew15.40966_neigh1.70398_cross0.84816_n14.mtx","synthetic_49161_49161_5521770_avg112.32013_std88.82977_random_bw23368.76097_skew3.82549_neigh1.89374_cross0.62848_n14.mtx","synthetic_49556_49556_18951229_avg382.42047_std245.44344_random_bw21913.94158_skew1.57047_neigh1.97494_cross0.91632_n14.mtx","synthetic_4998_4998_10667079_avg2134.26951_std834.36333_random_bw4642.41957_skew1.34178_neigh1.70696_cross0.64169_n14.mtx","synthetic_50954_50954_7955612_avg156.13322_std368.01601_random_bw29951.25091_skew325.34951_neigh1.84107_cross0.94282_n14.mtx","synthetic_5159002_5159002_20646759_avg4.00208_std1.17597_random_bw5000.15374_skew312.8365_neigh0.41584_cross0.64845_n14.mtx","synthetic_51970_51970_5840112_avg112.37468_std88.844_random_bw24618.90708_skew3.82315_neigh1.89384_cross0.62918_n14.mtx","synthetic_52226_52226_4093416_avg78.37889_std1019.1394_random_bw1610.58794_skew665.32736_neigh1.33958_cross0.83862_n14.mtx","synthetic_5283_5283_11298374_avg2138.62843_std877.77445_random_bw4852.64547_skew1.47027_neigh1.70262_cross0.62729_n14.mtx","synthetic_53198_53198_11791218_avg221.64777_std96.43769_random_bw43503.74766_skew15.40892_neigh1.70456_cross0.84719_n14.mtx","synthetic_5471669_5471669_21898040_avg4.00208_std1.17221_random_bw5302.30865_skew312.83705_neigh0.41568_cross0.64833_n14.mtx","synthetic_54779_54779_6163317_avg112.5124_std88.83086_random_bw26024.3978_skew3.81725_neigh1.89353_cross0.62937_n14.mtx","synthetic_55385_55385_8650205_avg156.18317_std377.56925_random_bw32639.28862_skew353.61567_neigh1.83893_cross0.94232_n14.mtx","synthetic_5569_5569_11949345_avg2145.68953_std911.81863_random_bw5136.07919_skew1.59544_neigh1.69647_cross0.60886_n14.mtx","synthetic_56767_56767_4479663_avg78.91315_std1046.34011_random_bw1746.70014_skew718.36043_neigh1.34074_cross0.83789_n14.mtx","synthetic_57454_57454_12734908_avg221.65398_std96.39664_random_bw46842.79518_skew15.40846_neigh1.70371_cross0.85054_n14.mtx","synthetic_5784335_5784335_23149312_avg4.00207_std1.16847_random_bw5605.51869_skew312.83761_neigh0.41582_cross0.64832_n14.mtx","synthetic_58333_58333_4204101_avg72.07071_std19.00931_random_bw3352.53952_skew1.13679_neigh1.65318_cross0.7705_n14.mtx","synthetic_59816_59816_9344868_avg156.2269_std386.43932_random_bw35412.98975_skew381.87902_neigh1.83858_cross0.94305_n14.mtx","synthetic_603864_603864_13450351_avg22.27381_std30.02405_random_bw152862.83998_skew313.58473_neigh1.26422_cross0.75345_n14.mtx","synthetic_6097002_6097002_24400590_avg4.00206_std1.16523_random_bw5910.6367_skew312.83809_neigh0.41581_cross0.64818_n14.mtx","synthetic_61308_61308_4858098_avg79.24085_std1065.00271_random_bw1888.99212_skew772.69186_neigh1.34152_cross0.83808_n14.mtx","synthetic_61710_61710_13678136_avg221.65186_std96.36424_random_bw50412.03223_skew15.40862_neigh1.70378_cross0.84921_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_random_bw399.28715_skew0.93375_neigh1.57754_cross0.89197_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_random_bw427.54729_skew0.93375_neigh1.17987_cross0.83144_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_random_bw430.40577_skew0.93375_neigh1.2674_cross0.82956_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_random_bw430.53472_skew0.93375_neigh1.35298_cross0.83078_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_random_bw431.15425_skew0.93375_neigh1.56885_cross0.85081_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_random_bw432.49407_skew0.93375_neigh1.44041_cross0.82902_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_random_bw437.05305_skew0.93375_neigh1.52957_cross0.82898_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_random_bw443.57612_skew0.93375_neigh1.61781_cross0.82827_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_random_bw444.27433_skew0.93375_neigh1.57185_cross0.82753_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_random_bw453.26434_skew0.93375_neigh1.57391_cross0.80665_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_random_bw471.54689_skew0.93375_neigh1.58217_cross0.75806_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_random_bw472.59519_skew0.93375_neigh1.70735_cross0.82682_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_random_bw482.56044_skew0.93375_neigh1.58512_cross0.71559_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_random_bw493.41782_skew0.93375_neigh1.58831_cross0.66574_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_random_bw501.92205_skew0.93375_neigh1.59187_cross0.621_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_random_bw533.47514_skew0.93375_neigh1.8037_cross0.82219_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_random_bw573.2297_skew0.93375_neigh1.56189_cross0.82326_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_random_bw653.65655_skew0.93375_neigh1.90485_cross0.81986_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_random_bw683.09787_skew0.93375_neigh1.8984_cross0.81849_n14.mtx","synthetic_63838_63838_10847423_avg169.9211_std96.15794_random_bw50815.19321_skew15.38408_neigh1.69783_cross0.83843_n14.mtx","synthetic_63838_63838_11790856_avg184.69965_std96.20654_random_bw51110.24539_skew15.38877_neigh1.69913_cross0.84602_n14.mtx","synthetic_63838_63838_12734460_avg199.48087_std96.25772_random_bw51593.46717_skew15.39756_neigh1.70252_cross0.84507_n14.mtx","synthetic_63838_63838_13677970_avg214.26063_std96.31751_random_bw51973.46416_skew15.40525_neigh1.70373_cross0.84684_n14.mtx","synthetic_63838_63838_14148710_avg221.63461_std96.12924_random_bw51283.67085_skew11.06039_neigh1.70527_cross0.84272_n14.mtx","synthetic_63838_63838_14148875_avg221.63719_std96.16987_random_bw51535.13301_skew12.02579_neigh1.70395_cross0.8451_n14.mtx","synthetic_63838_63838_14148974_avg221.63874_std125.01008_random_bw50694.22474_skew15.38251_neigh1.706_cross0.83858_n14.mtx","synthetic_63838_63838_14149076_avg221.64034_std96.21451_random_bw51658.38851_skew12.99114_neigh1.70423_cross0.84718_n14.mtx","synthetic_63838_63838_14149188_avg221.64209_std118.63327_random_bw50889.44055_skew15.38678_neigh1.70517_cross0.84295_n14.mtx","synthetic_63838_63838_14149208_avg221.64241_std96.26387_random_bw51855.74965_skew13.96104_neigh1.70421_cross0.84669_n14.mtx","synthetic_63838_63838_14149296_avg221.64379_std112.26231_random_bw51274.57478_skew15.39568_neigh1.70495_cross0.84257_n14.mtx","synthetic_63838_63838_14149329_avg221.6443_std105.89153_random_bw51549.08313_skew15.40015_neigh1.70303_cross0.84806_n14.mtx","synthetic_63838_63838_14149335_avg221.6444_std96.31796_random_bw52140.13924_skew14.92641_neigh1.70443_cross0.84646_n14.mtx","synthetic_63838_63838_14149338_avg221.64444_std99.52693_random_bw52057.82066_skew15.40465_neigh1.70386_cross0.84887_n14.mtx","synthetic_63838_63838_14149538_avg221.64758_std93.16614_random_bw52341.45014_skew15.40893_neigh1.70422_cross0.84685_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_random_bw31553.90656_skew15.40883_neigh1.97927_cross0.90185_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_random_bw36435.81718_skew15.40883_neigh1.70415_cross0.85204_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_random_bw40001.91804_skew15.40883_neigh1.70412_cross0.84856_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_random_bw41994.67554_skew15.40883_neigh1.62327_cross0.97823_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_random_bw43525.69963_skew15.40883_neigh1.70435_cross0.8478_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_random_bw46863.79909_skew15.40883_neigh1.70408_cross0.85043_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_random_bw48556.48783_skew15.40883_neigh1.67822_cross0.93499_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_random_bw50400.40091_skew15.40883_neigh1.70389_cross0.84914_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_random_bw51440.8804_skew15.40883_neigh1.69826_cross0.87894_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_random_bw51860.61629_skew15.40883_neigh1.19383_cross0.8502_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_random_bw51860.91715_skew15.40883_neigh1.30912_cross0.84912_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_random_bw51934.71139_skew15.40883_neigh1.53425_cross0.84942_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_random_bw51974.98208_skew15.40883_neigh1.42134_cross0.8471_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_random_bw52133.12723_skew15.40883_neigh1.64881_cross0.84612_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_random_bw52424.50489_skew15.40883_neigh1.76191_cross0.85027_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_random_bw52642.58633_skew15.40883_neigh1.70867_cross0.81986_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_random_bw53211.08631_skew15.40883_neigh1.87632_cross0.84853_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_random_bw53321.33316_skew15.40883_neigh1.71205_cross0.7625_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_random_bw53759.27571_skew15.40883_neigh1.71483_cross0.70856_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_random_bw53829.21378_skew15.40883_neigh1.70392_cross0.84808_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_random_bw54127.70973_skew15.40883_neigh1.71701_cross0.64906_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_random_bw54345.60945_skew15.40883_neigh1.71839_cross0.59238_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_random_bw57263.41878_skew15.40883_neigh1.70375_cross0.85062_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_random_bw58199.76162_skew15.40883_neigh1.70478_cross0.85336_n14.mtx","synthetic_63838_63838_14149682_avg221.64983_std96.37657_random_bw52178.12439_skew15.887_neigh1.70295_cross0.84932_n14.mtx","synthetic_63838_63838_14149757_avg221.65101_std96.43499_random_bw52389.17814_skew16.85239_neigh1.70325_cross0.85077_n14.mtx","synthetic_63838_63838_14149836_avg221.65224_std86.81097_random_bw52747.35418_skew15.4131_neigh1.7044_cross0.85031_n14.mtx","synthetic_63838_63838_14149837_avg221.65226_std96.50454_random_bw52563.14853_skew17.81776_neigh1.70366_cross0.84972_n14.mtx","synthetic_63838_63838_14149882_avg221.65297_std80.46101_random_bw53023.30211_skew15.41756_neigh1.70467_cross0.84957_n14.mtx","synthetic_63838_63838_14149914_avg221.65347_std74.11814_random_bw53380.64075_skew15.42203_neigh1.70407_cross0.85405_n14.mtx","synthetic_63838_63838_14150121_avg221.65671_std67.78709_random_bw53636.29238_skew15.4263_neigh1.7049_cross0.85444_n14.mtx","synthetic_63838_63838_14150132_avg221.65688_std96.57304_random_bw52625.73423_skew18.78283_neigh1.70324_cross0.85136_n14.mtx","synthetic_63838_63838_14150259_avg221.65887_std96.64741_random_bw52705.54984_skew19.7481_neigh1.705_cross0.85168_n14.mtx","synthetic_63838_63838_14621335_avg229.03811_std96.37884_random_bw52333.38261_skew15.40775_neigh1.70591_cross0.8492_n14.mtx","synthetic_63838_63838_15564722_avg243.81594_std96.44368_random_bw52583.81193_skew15.41402_neigh1.70736_cross0.85221_n14.mtx","synthetic_63838_63838_16508065_avg258.59308_std96.5168_random_bw52719.01902_skew15.41575_neigh1.70764_cross0.8531_n14.mtx","synthetic_63838_63838_17451479_avg273.37133_std96.59121_random_bw53072.43958_skew15.41723_neigh1.70777_cross0.85229_n14.mtx","synthetic_63838_63838_18395041_avg288.1519_std96.67009_random_bw53224.04721_skew15.42189_neigh1.71047_cross0.85161_n14.mtx","synthetic_63838_63838_9903973_avg155.14228_std96.11613_random_bw50204.46286_skew15.37207_neigh1.69502_cross0.83641_n14.mtx","synthetic_63889_63889_4604376_avg72.06837_std18.98718_random_bw3672.48262_skew1.13686_neigh1.6535_cross0.76942_n14.mtx","synthetic_64247_64247_10039260_avg156.26037_std394.7431_random_bw37749.51279_skew410.15351_neigh1.83858_cross0.94311_n14.mtx","synthetic_65850_65850_5226388_avg79.36808_std1075.21227_random_bw2021.34421_skew828.67864_neigh1.34471_cross0.83794_n14.mtx","synthetic_65965_65965_14620660_avg221.64269_std96.33127_random_bw53946.07292_skew15.40929_neigh1.7042_cross0.84826_n14.mtx","synthetic_661375_661375_14730847_avg22.27306_std29.96346_random_bw167527.31499_skew313.59526_neigh1.26295_cross0.75385_n14.mtx","synthetic_66463_66463_10040261_avg151.06542_std395.7133_random_bw39089.18279_skew424.82214_neigh1.83705_cross0.94111_n14.mtx","synthetic_66463_66463_10375238_avg156.10547_std374.16577_random_bw37885.35229_skew297.97094_neigh1.84271_cross0.9369_n14.mtx","synthetic_66463_66463_10377678_avg156.14218_std378.88433_random_bw38300.96201_skew326.18897_neigh1.84277_cross0.93802_n14.mtx","synthetic_66463_66463_10378609_avg156.15619_std376.2538_random_bw40147.56841_skew424.61873_neigh1.83726_cross0.94556_n14.mtx","synthetic_66463_66463_10380253_avg156.18093_std381.55088_random_bw40132.11695_skew424.55132_neigh1.83814_cross0.94385_n14.mtx","synthetic_66463_66463_10380322_avg156.18197_std384.04911_random_bw38766.29347_skew354.38034_neigh1.84508_cross0.93857_n14.mtx","synthetic_66463_66463_10381996_avg156.20715_std386.69205_random_bw39470.08879_skew424.47988_neigh1.83953_cross0.9449_n14.mtx","synthetic_66463_66463_10382686_avg156.21753_std389.6586_random_bw38980.4995_skew382.5741_neigh1.84251_cross0.94018_n14.mtx","synthetic_66463_66463_10383864_avg156.23526_std391.65048_random_bw39602.28441_skew424.40333_neigh1.84213_cross0.94243_n14.mtx","synthetic_66463_66463_10384998_avg156.25232_std492.58539_random_bw37240.82965_skew424.35688_neigh1.85054_cross0.93288_n14.mtx","synthetic_66463_66463_10385329_avg156.2573_std395.72064_random_bw38976.54612_skew410.73756_neigh1.84231_cross0.94152_n14.mtx","synthetic_66463_66463_10385409_avg156.2585_std471.2363_random_bw37706.06005_skew424.34005_neigh1.84635_cross0.93626_n14.mtx","synthetic_66463_66463_10385572_avg156.26096_std396.41268_random_bw39282.12082_skew424.33337_neigh1.84007_cross0.94251_n14.mtx","synthetic_66463_66463_10385645_avg156.26206_std450.123_random_bw38391.32296_skew424.33038_neigh1.84642_cross0.93565_n14.mtx","synthetic_66463_66463_10386025_avg156.26777_std429.29508_random_bw38743.10672_skew424.31482_neigh1.84579_cross0.93782_n14.mtx","synthetic_66463_66463_10386363_avg156.27286_std408.80831_random_bw38921.95154_skew424.30098_neigh1.84281_cross0.94099_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_random_bw27380.53228_skew424.29959_neigh1.83971_cross0.94311_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_random_bw30115.66172_skew424.29959_neigh1.83841_cross0.94227_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_random_bw32802.33897_skew424.29959_neigh1.8391_cross0.94285_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_random_bw33891.3234_skew424.29959_neigh1.30566_cross0.9399_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_random_bw33914.40305_skew424.29959_neigh1.42021_cross0.9414_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_random_bw34216.51382_skew424.29959_neigh1.54427_cross0.94023_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_random_bw34386.70958_skew424.29959_neigh1.66268_cross0.94107_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_random_bw35355.46844_skew424.29959_neigh1.83904_cross0.94283_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_random_bw35880.75824_skew424.29959_neigh1.78051_cross0.94121_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_random_bw36358.1962_skew424.29959_neigh1.75925_cross0.97917_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_random_bw36828.67158_skew424.29959_neigh1.94766_cross0.95084_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_random_bw37162.55769_skew424.29959_neigh1.78345_cross0.9726_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_random_bw37752.50297_skew424.29959_neigh1.83874_cross0.94308_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_random_bw39749.89256_skew424.29959_neigh1.86406_cross0.91226_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_random_bw39793.17787_skew424.29959_neigh1.90076_cross0.72217_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_random_bw39841.02385_skew424.29959_neigh1.90364_cross0.65888_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_random_bw40073.12002_skew424.29959_neigh1.88547_cross0.84863_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_random_bw40078.15401_skew424.29959_neigh1.89472_cross0.78617_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_random_bw40874.04356_skew424.29959_neigh1.838_cross0.94255_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_random_bw43013.40304_skew424.29959_neigh1.92851_cross0.95044_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_random_bw43275.46909_skew424.29959_neigh1.8389_cross0.94246_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_random_bw45759.36917_skew424.29959_neigh1.83956_cross0.94233_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_random_bw48163.28926_skew424.29959_neigh1.84119_cross0.9421_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_random_bw50080.08679_skew424.29959_neigh1.84023_cross0.94336_n14.mtx","synthetic_66463_66463_10386663_avg156.27737_std388.73672_random_bw39544.24019_skew424.2887_neigh1.83892_cross0.9432_n14.mtx","synthetic_66463_66463_10387014_avg156.28265_std369.1724_random_bw40157.88734_skew424.27433_neigh1.8384_cross0.94372_n14.mtx","synthetic_66463_66463_10387408_avg156.28858_std350.24164_random_bw40130.74235_skew424.25819_neigh1.83783_cross0.94626_n14.mtx","synthetic_66463_66463_10387855_avg156.29531_std332.11193_random_bw40571.57874_skew424.23989_neigh1.8371_cross0.9482_n14.mtx","synthetic_66463_66463_10388533_avg156.30551_std315.01147_random_bw41077.64824_skew424.21214_neigh1.83814_cross0.94745_n14.mtx","synthetic_66463_66463_10730769_avg161.45478_std396.39605_random_bw39721.11865_skew410.65087_neigh1.8442_cross0.94219_n14.mtx","synthetic_66463_66463_11418942_avg171.80901_std391.59435_random_bw40372.93939_skew385.84235_neigh1.84536_cross0.94343_n14.mtx","synthetic_66463_66463_12107419_avg182.16781_std386.59381_random_bw38834.28252_skew363.84492_neigh1.84927_cross0.94558_n14.mtx","synthetic_66463_66463_12795815_avg192.52539_std381.413_random_bw39460.49056_skew344.2168_neigh1.85447_cross0.94527_n14.mtx","synthetic_66463_66463_13484307_avg202.88442_std376.07335_random_bw38668.20735_skew326.59046_neigh1.85481_cross0.94695_n14.mtx","synthetic_66463_66463_7269367_avg109.37464_std374.11902_random_bw41648.04419_skew424.85738_neigh1.81971_cross0.92957_n14.mtx","synthetic_66463_66463_7962072_avg119.79706_std378.84315_random_bw43046.45655_skew424.84518_neigh1.82641_cross0.93386_n14.mtx","synthetic_66463_66463_8654839_avg130.22041_std384.01598_random_bw39759.86055_skew424.83187_neigh1.83083_cross0.93703_n14.mtx","synthetic_66463_66463_9347522_avg140.64249_std389.6368_random_bw40907.57737_skew424.82436_neigh1.83448_cross0.93954_n14.mtx","synthetic_666542_666542_32559308_avg48.8481_std11.96425_random_bw120493.81988_skew1.12905_neigh1.69092_cross0.80241_n14.mtx","synthetic_68121_68121_3787555_avg55.6004_std1068.24938_random_bw1710.80564_skew856.74559_neigh1.75999_cross0.84954_n14.mtx","synthetic_68121_68121_4147401_avg60.88286_std1069.9982_random_bw1924.33426_skew857.05765_neigh1.62192_cross0.84311_n14.mtx","synthetic_68121_68121_4507272_avg66.16568_std1071.85693_random_bw2009.01365_skew857.28489_neigh1.51372_cross0.84049_n14.mtx","synthetic_68121_68121_4867182_avg71.44907_std1073.8353_random_bw2057.30139_skew857.47165_neigh1.43283_cross0.83917_n14.mtx","synthetic_68121_68121_5227217_avg76.7343_std1075.93952_random_bw2082.79121_skew857.61216_neigh1.37242_cross0.83802_n14.mtx","synthetic_68121_68121_5356033_avg78.62528_std1024.59921_random_bw2109.44585_skew865.40068_neigh1.23716_cross0.83754_n14.mtx","synthetic_68121_68121_5371179_avg78.84762_std1041.23556_random_bw2105.22274_skew862.95755_neigh1.25608_cross0.8375_n14.mtx","synthetic_68121_68121_5385572_avg79.05891_std1056.36091_random_bw2101.63207_skew860.64861_neigh1.27562_cross0.83773_n14.mtx","synthetic_68121_68121_5397801_avg79.23843_std1068.36979_random_bw2095.97678_skew858.6965_neigh1.29918_cross0.83772_n14.mtx","synthetic_68121_68121_5398333_avg79.24624_std1068.28806_random_bw2035.98597_skew601.36298_neigh1.51198_cross0.84041_n14.mtx","synthetic_68121_68121_5400082_avg79.27191_std1070.0369_random_bw2057.79974_skew658.4265_neigh1.46277_cross0.83918_n14.mtx","synthetic_68121_68121_5402340_avg79.30506_std1071.88875_random_bw2069.56856_skew715.38556_neigh1.42374_cross0.83882_n14.mtx","synthetic_68121_68121_5404162_avg79.33181_std1073.85709_random_bw2080.78824_skew772.35942_neigh1.38955_cross0.83863_n14.mtx","synthetic_68121_68121_5405884_avg79.35709_std1075.92117_random_bw2092.30701_skew857.41106_neigh1.32607_cross0.83815_n14.mtx","synthetic_68121_68121_5405960_avg79.3582_std834.61171_random_bw2118.46771_skew857.399_neigh1.17219_cross0.83969_n14.mtx","synthetic_68121_68121_5406317_avg79.36344_std1075.95011_random_bw2086.76499_skew829.23113_neigh1.35604_cross0.83848_n14.mtx","synthetic_68121_68121_5406423_avg79.365_std903.5954_random_bw2113.96985_skew857.32548_neigh1.20794_cross0.83738_n14.mtx","synthetic_68121_68121_5406595_avg79.36752_std972.82101_random_bw2105.47931_skew857.29818_neigh1.25674_cross0.83732_n14.mtx","synthetic_68121_68121_5406754_avg79.36986_std765.8736_random_bw2126.17506_skew857.27294_neigh1.12346_cross0.86663_n14.mtx","synthetic_68121_68121_5407048_avg79.37417_std1042.23736_random_bw2098.32533_skew857.22627_neigh1.3103_cross0.83794_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_random_bw1475.2481_skew857.20278_neigh1.35089_cross0.83825_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_random_bw1616.29047_skew857.20278_neigh1.3466_cross0.83808_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_random_bw1747.72823_skew857.20278_neigh1.34503_cross0.83777_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_random_bw1888.70535_skew857.20278_neigh1.34185_cross0.83764_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_random_bw2022.47778_skew857.20278_neigh1.34097_cross0.83789_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_random_bw2074.43904_skew857.20278_neigh1.34668_cross0.84315_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_random_bw2084.69378_skew857.20278_neigh1.47356_cross0.83912_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_random_bw2084.8877_skew857.20278_neigh1.41898_cross0.83878_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_random_bw2087.46808_skew857.20278_neigh1.44672_cross0.83882_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_random_bw2087.55843_skew857.20278_neigh1.38435_cross0.83848_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_random_bw2091.70347_skew857.20278_neigh1.24164_cross0.83704_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_random_bw2092.93644_skew857.20278_neigh1.27201_cross0.83748_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_random_bw2093.47124_skew857.20278_neigh1.29801_cross0.83744_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_random_bw2093.49917_skew857.20278_neigh1.35469_cross0.83806_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_random_bw2093.88641_skew857.20278_neigh1.32253_cross0.83718_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_random_bw2096.20556_skew857.20278_neigh1.21315_cross0.83699_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_random_bw2157.43003_skew857.20278_neigh1.33753_cross0.83793_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_random_bw2184.16415_skew857.20278_neigh1.33864_cross0.81059_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_random_bw2297.56153_skew857.20278_neigh1.34152_cross0.83789_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_random_bw2316.85899_skew857.20278_neigh1.33597_cross0.75893_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_random_bw2425.34644_skew857.20278_neigh1.33787_cross0.70358_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_random_bw2429.8828_skew857.20278_neigh1.33672_cross0.83792_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_random_bw2506.6977_skew857.20278_neigh1.33929_cross0.65052_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_random_bw2565.36768_skew857.20278_neigh1.33531_cross0.83829_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_random_bw2580.54001_skew857.20278_neigh1.34006_cross0.59378_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_random_bw2698.36137_skew857.20278_neigh1.33462_cross0.83799_n14.mtx","synthetic_68121_68121_5407447_avg79.38003_std1111.80732_random_bw2086.89475_skew857.16294_neigh1.369_cross0.83803_n14.mtx","synthetic_68121_68121_5407867_avg79.3862_std1181.49691_random_bw2070.29451_skew857.0963_neigh1.42499_cross0.83866_n14.mtx","synthetic_68121_68121_5408409_avg79.39415_std1251.28152_random_bw2052.9943_skew857.0103_neigh1.49396_cross0.83934_n14.mtx","synthetic_68121_68121_5409111_avg79.40446_std1321.14786_random_bw2021.97833_skew856.89895_neigh1.56017_cross0.84082_n14.mtx","synthetic_68121_68121_5409514_avg79.41037_std1391.07361_random_bw1983.1191_skew856.83504_neigh1.63329_cross0.84144_n14.mtx","synthetic_68121_68121_5584961_avg81.98589_std1075.92069_random_bw2099.22202_skew829.88685_neigh1.31053_cross0.83776_n14.mtx","synthetic_68121_68121_5934842_avg87.12206_std1068.36291_random_bw2110.99739_skew780.90298_neigh1.27284_cross0.83727_n14.mtx","synthetic_68121_68121_6280428_avg92.19518_std1056.3285_random_bw2123.14184_skew737.87809_neigh1.23532_cross0.83721_n14.mtx","synthetic_68121_68121_6623921_avg97.23758_std1041.17147_random_bw2125.39362_skew699.5625_neigh1.21008_cross0.83733_n14.mtx","synthetic_68121_68121_6966795_avg102.27089_std1024.4946_random_bw2134.44271_skew665.08399_neigh1.1864_cross0.83805_n14.mtx","synthetic_68678_68678_10731088_avg156.25219_std397.34109_random_bw40858.96562_skew424.88203_neigh1.83835_cross0.94238_n14.mtx","synthetic_690692_690692_50143771_avg72.59932_std15.84383_random_bw10372.13539_skew0.96972_neigh1.69572_cross0.81872_n14.mtx","synthetic_69445_69445_5001284_avg72.01791_std19.00309_random_bw3971.49729_skew1.13836_neigh1.65217_cross0.77151_n14.mtx","synthetic_70221_70221_15563153_avg221.63104_std96.30657_random_bw57348.75106_skew15.41016_neigh1.70385_cross0.85141_n14.mtx","synthetic_70391_70391_5585080_avg79.34367_std1076.52341_random_bw2156.59175_skew858.03517_neigh1.33809_cross0.83781_n14.mtx","synthetic_718886_718886_16011537_avg22.27271_std29.91357_random_bw181769.97437_skew313.60029_neigh1.26418_cross0.75622_n14.mtx","synthetic_730022_730022_35670999_avg48.86291_std11.95289_random_bw131807.11807_skew1.14887_neigh1.69169_cross0.8019_n14.mtx","synthetic_73109_73109_11420815_avg156.21627_std394.47456_random_bw43325.79208_skew424.97997_neigh1.83945_cross0.94217_n14.mtx","synthetic_74477_74477_16505933_avg221.62457_std96.28344_random_bw60807.87753_skew15.41064_neigh1.70429_cross0.84959_n14.mtx","synthetic_74933_74933_5942603_avg79.30555_std1075.58283_random_bw2296.59318_skew858.44801_neigh1.34268_cross0.83766_n14.mtx","synthetic_75000_75000_5399893_avg71.99857_std19.01052_random_bw4301.01067_skew1.13893_neigh1.6539_cross0.76935_n14.mtx","synthetic_756472_756472_54943929_avg72.63181_std15.80449_random_bw11344.89478_skew1.03767_neigh1.69516_cross0.81986_n14.mtx","synthetic_77540_77540_12111057_avg156.19109_std391.93861_random_bw45639.49656_skew425.04863_neigh1.84051_cross0.94184_n14.mtx","synthetic_776397_776397_17292444_avg22.27268_std29.86978_random_bw196406.45729_skew313.60063_neigh1.26218_cross0.75486_n14.mtx","synthetic_78733_78733_17449768_avg221.6322_std96.26215_random_bw64358.73616_skew15.41007_neigh1.70377_cross0.84821_n14.mtx","synthetic_793502_793502_38791634_avg48.88662_std11.94806_random_bw143383.51376_skew1.39329_neigh1.69098_cross0.8029_n14.mtx","synthetic_79474_79474_6302859_avg79.30718_std1074.75192_random_bw2429.36754_skew858.43036_neigh1.33896_cross0.83795_n14.mtx","synthetic_80556_80556_5802157_avg72.02638_std19.02772_random_bw4624.77567_skew1.13811_neigh1.6521_cross0.76889_n14.mtx","synthetic_81971_81971_12801291_avg156.16854_std389.68003_random_bw48091.50462_skew425.11016_neigh1.8392_cross0.9423_n14.mtx","synthetic_822252_822252_59710654_avg72.61844_std15.79674_random_bw12337.39137_skew1.02428_neigh1.69509_cross0.82013_n14.mtx","synthetic_82989_82989_18393238_avg221.63465_std96.24293_random_bw67768.63417_skew15.40989_neigh1.70426_cross0.84856_n14.mtx","synthetic_83334_83334_4201657_avg50.41948_std18.95017_random_bw4791.17217_skew1.61804_neigh1.63203_cross0.73132_n14.mtx","synthetic_83334_83334_4601152_avg55.21338_std18.98799_random_bw4722.67207_skew1.48128_neigh1.63833_cross0.74354_n14.mtx","synthetic_83334_83334_5001407_avg60.0164_std19.00589_random_bw4672.15651_skew1.36602_neigh1.64462_cross0.75232_n14.mtx","synthetic_83334_83334_5402033_avg64.82388_std19.01381_random_bw4770.34201_skew1.25226_neigh1.64891_cross0.75864_n14.mtx","synthetic_83334_83334_5802662_avg69.63139_std19.01513_random_bw4732.86769_skew1.16856_neigh1.65263_cross0.76629_n14.mtx","synthetic_83334_83334_6001725_avg72.02012_std24.68331_random_bw4665.54866_skew1.47153_neigh1.65687_cross0.7425_n14.mtx","synthetic_83334_83334_6001767_avg72.02063_std23.43414_random_bw4680.6887_skew1.40209_neigh1.65691_cross0.74737_n14.mtx","synthetic_83334_83334_6001894_avg72.02215_std22.17621_random_bw4719.37063_skew1.31873_neigh1.65546_cross0.7542_n14.mtx","synthetic_83334_83334_6002259_avg72.02653_std20.9136_random_bw4737.64665_skew1.24917_neigh1.65459_cross0.76108_n14.mtx","synthetic_83334_83334_6002747_avg72.03239_std19.64906_random_bw4762.0232_skew1.16569_neigh1.65259_cross0.76576_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_random_bw3350.26591_skew1.13786_neigh1.65428_cross0.77109_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_random_bw3665.49009_skew1.13786_neigh1.65236_cross0.77134_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_random_bw3982.36841_skew1.13786_neigh1.65216_cross0.77018_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_random_bw4220.40105_skew1.13786_neigh1.61436_cross0.87115_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_random_bw4298.73964_skew1.13786_neigh1.65361_cross0.7697_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_random_bw4395.07819_skew1.13786_neigh1.63026_cross0.84649_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_random_bw4592.28661_skew1.13786_neigh1.16508_cross0.77088_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_random_bw4606.03839_skew1.13786_neigh1.27327_cross0.76874_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_random_bw4608.30407_skew1.13786_neigh1.38168_cross0.77039_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_random_bw4614.10663_skew1.13786_neigh1.653_cross0.77104_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_random_bw4654.25229_skew1.13786_neigh1.4901_cross0.76949_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_random_bw4659.61285_skew1.13786_neigh1.6488_cross0.79591_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_random_bw4701.35313_skew1.13786_neigh1.59891_cross0.77081_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_random_bw4776.47572_skew1.13786_neigh1.65401_cross0.76988_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_random_bw4850.09408_skew1.13786_neigh1.65707_cross0.74566_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_random_bw4869.2196_skew1.13786_neigh1.70817_cross0.76996_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_random_bw4931.43106_skew1.13786_neigh1.65277_cross0.76964_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_random_bw4997.26823_skew1.13786_neigh1.66492_cross0.69556_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_random_bw5131.45013_skew1.13786_neigh1.66917_cross0.64549_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_random_bw5220.44701_skew1.13786_neigh1.67321_cross0.59335_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_random_bw5240.64593_skew1.13786_neigh1.65263_cross0.76939_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_random_bw5314.98523_skew1.13786_neigh1.67563_cross0.54557_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_random_bw5580.33847_skew1.13786_neigh1.65308_cross0.76857_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_random_bw5607.76149_skew1.13786_neigh1.81609_cross0.77014_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_random_bw5792.30158_skew1.13786_neigh1.92751_cross0.7686_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_random_bw5891.31062_skew1.13786_neigh1.65284_cross0.76938_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_random_bw5944.6548_skew1.13786_neigh1.92575_cross0.76959_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_random_bw6200.06394_skew1.13786_neigh1.65377_cross0.76946_n14.mtx","synthetic_83334_83334_6003209_avg72.03793_std18.38192_random_bw4782.87446_skew1.09612_neigh1.6521_cross0.77432_n14.mtx","synthetic_83334_83334_6003696_avg72.04378_std17.11472_random_bw4810.91187_skew1.01267_neigh1.65178_cross0.77955_n14.mtx","synthetic_83334_83334_6004305_avg72.05108_std15.84803_random_bw4849.17645_skew0.94307_neigh1.6506_cross0.7874_n14.mtx","synthetic_83334_83334_6004738_avg72.05628_std14.57979_random_bw4873.18033_skew0.87354_neigh1.6472_cross0.79352_n14.mtx","synthetic_83334_83334_6005175_avg72.06152_std13.31259_random_bw4900.61932_skew0.79014_neigh1.6502_cross0.79935_n14.mtx","synthetic_83334_83334_6203256_avg74.43848_std19.01659_random_bw4717.39456_skew1.09569_neigh1.65429_cross0.77391_n14.mtx","synthetic_83334_83334_6603858_avg79.24566_std19.01545_random_bw4770.86497_skew1.03166_neigh1.65742_cross0.7795_n14.mtx","synthetic_83334_83334_7004670_avg84.05537_std19.01725_random_bw4746.86476_skew0.97489_neigh1.66015_cross0.78394_n14.mtx","synthetic_83334_83334_7405486_avg88.86512_std19.01689_random_bw4748.30992_skew0.91301_neigh1.66314_cross0.78781_n14.mtx","synthetic_83334_83334_7806125_avg93.67275_std19.01502_random_bw4799.57248_skew0.86821_neigh1.66483_cross0.79526_n14.mtx","synthetic_833908_833908_18572841_avg22.27205_std29.833_random_bw210844.85277_skew313.60956_neigh1.26298_cross0.75563_n14.mtx","synthetic_84015_84015_6661114_avg79.28482_std1074.00684_random_bw2567.05804_skew858.67278_neigh1.33657_cross0.83743_n14.mtx","synthetic_856982_856982_41861338_avg48.84739_std11.94057_random_bw154675.82077_skew1.3338_neigh1.69011_cross0.80335_n14.mtx","synthetic_86111_86111_6203103_avg72.03613_std19.01524_random_bw4930.68174_skew1.13782_neigh1.65292_cross0.76969_n14.mtx","synthetic_862664_862664_13443097_avg15.58324_std29.57178_random_bw217449.07728_skew313.76132_neigh1.20059_cross0.75196_n14.mtx","synthetic_862664_862664_14783705_avg17.13727_std29.61764_random_bw211187.81291_skew312.46885_neigh1.21655_cross0.75346_n14.mtx","synthetic_862664_862664_15984607_avg18.52935_std29.67134_random_bw215438.01831_skew314.12161_neigh1.22333_cross0.75444_n14.mtx","synthetic_862664_862664_17373620_avg20.1395_std29.72434_random_bw211752.41659_skew312.11605_neigh1.23946_cross0.75537_n14.mtx","synthetic_862664_862664_18524582_avg21.47369_std29.78706_random_bw214344.29615_skew314.40918_neigh1.24035_cross0.75391_n14.mtx","synthetic_862664_862664_19089435_avg22.12847_std21.22646_random_bw218646.30801_skew315.65089_neigh1.27062_cross0.75619_n14.mtx","synthetic_862664_862664_19142097_avg22.18952_std25.02501_random_bw218419.55757_skew314.77975_neigh1.26903_cross0.75599_n14.mtx","synthetic_862664_862664_19162894_avg22.21362_std30.15004_random_bw218402.40102_skew408.47843_neigh1.26817_cross0.75575_n14.mtx","synthetic_862664_862664_19171989_avg22.22417_std26.93718_random_bw218359.93899_skew314.2874_neigh1.26727_cross0.7556_n14.mtx","synthetic_862664_862664_19173147_avg22.22551_std30.06905_random_bw218342.75064_skew387.38255_neigh1.26736_cross0.75562_n14.mtx","synthetic_862664_862664_19184120_avg22.23823_std29.99101_random_bw218334.93381_skew366.25046_neigh1.26648_cross0.75604_n14.mtx","synthetic_862664_862664_19195719_avg22.25168_std29.91773_random_bw218360.52038_skew345.17619_neigh1.26538_cross0.75561_n14.mtx","synthetic_862664_862664_19200670_avg22.25741_std28.85544_random_bw218339.10627_skew313.81644_neigh1.26454_cross0.75565_n14.mtx","synthetic_862664_862664_19207356_avg22.26516_std29.84926_random_bw218277.96985_skew324.12672_neigh1.26399_cross0.75549_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_random_bw152638.10671_skew313.61244_neigh1.26367_cross0.75505_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_random_bw167333.37581_skew313.61244_neigh1.26365_cross0.75511_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_random_bw181720.63529_skew313.61244_neigh1.26353_cross0.75502_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_random_bw196516.2028_skew313.61244_neigh1.26346_cross0.75516_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_random_bw211000.2943_skew313.61244_neigh1.26342_cross0.75507_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_random_bw213175.84954_skew313.61244_neigh0.96905_cross0.75596_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_random_bw213223.70119_skew313.61244_neigh0.88372_cross0.75578_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_random_bw213418.94648_skew313.61244_neigh1.05278_cross0.75588_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_random_bw213434.42462_skew313.61244_neigh1.13672_cross0.75617_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_random_bw213774.00756_skew313.61244_neigh1.22102_cross0.755_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_random_bw214411.79173_skew313.61244_neigh1.30492_cross0.75491_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_random_bw215370.12458_skew313.61244_neigh1.38935_cross0.75586_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_random_bw216786.80808_skew313.61244_neigh1.47408_cross0.75519_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_random_bw218507.74866_skew313.61244_neigh1.26845_cross0.52792_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_random_bw221443.99058_skew313.61244_neigh1.55823_cross0.75678_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_random_bw225527.89524_skew313.61244_neigh1.26339_cross0.75523_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_random_bw233014.20901_skew313.61244_neigh1.6426_cross0.75641_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_random_bw240175.84299_skew313.61244_neigh1.26362_cross0.75505_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_random_bw254512.86048_skew313.61244_neigh1.26353_cross0.7551_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_random_bw269234.4357_skew313.61244_neigh1.26372_cross0.75516_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_random_bw283733.51022_skew313.61244_neigh1.26362_cross0.75506_n14.mtx","synthetic_862664_862664_19218664_avg22.27827_std29.78506_random_bw218120.72798_skew303.10795_neigh1.26259_cross0.75534_n14.mtx","synthetic_862664_862664_19223712_avg22.28412_std30.77861_random_bw218094.95048_skew313.4391_neigh1.2615_cross0.75525_n14.mtx","synthetic_862664_862664_19229090_avg22.29036_std29.72467_random_bw217946.1315_skew282.12689_neigh1.26031_cross0.75517_n14.mtx","synthetic_862664_862664_19237224_avg22.29979_std29.6692_random_bw217791.40434_skew261.15496_neigh1.25666_cross0.75502_n14.mtx","synthetic_862664_862664_19238563_avg22.30134_std32.70575_random_bw217839.24923_skew313.19637_neigh1.25574_cross0.75505_n14.mtx","synthetic_862664_862664_19242590_avg22.30601_std29.61737_random_bw217546.12267_skew240.28029_neigh1.252_cross0.75446_n14.mtx","synthetic_862664_862664_19243461_avg22.30702_std38.50483_random_bw216241.00134_skew313.1164_neigh1.23809_cross0.7533_n14.mtx","synthetic_862664_862664_19244915_avg22.3087_std29.57096_random_bw217130.97238_skew219.45208_neigh1.24755_cross0.75425_n14.mtx","synthetic_862664_862664_19963054_avg23.14117_std29.84969_random_bw212519.73122_skew311.90553_neigh1.25719_cross0.75481_n14.mtx","synthetic_862664_862664_21064571_avg24.41805_std29.92055_random_bw214229.26362_skew314.66815_neigh1.25303_cross0.75497_n14.mtx","synthetic_862664_862664_22549917_avg26.13986_std29.9915_random_bw212791.27921_skew311.74076_neigh1.27038_cross0.75528_n14.mtx","synthetic_862664_862664_23605495_avg27.36349_std30.07249_random_bw214076.51411_skew314.82232_neigh1.26413_cross0.75466_n14.mtx","synthetic_862664_862664_25134553_avg29.13597_std30.15077_random_bw213148.94325_skew311.6376_neigh1.2775_cross0.75586_n14.mtx","synthetic_86401_86401_13491346_avg156.14803_std387.65694_random_bw50727.43053_skew425.16612_neigh1.83819_cross0.94244_n14.mtx","synthetic_88557_88557_7019496_avg79.26529_std1073.34158_random_bw2696.06683_skew858.88461_neigh1.33763_cross0.83799_n14.mtx","synthetic_888032_888032_64506105_avg72.63939_std15.82947_random_bw13315.63085_skew1.00993_neigh1.69523_cross0.81974_n14.mtx","synthetic_891419_891419_19853723_avg22.27204_std29.80105_random_bw225604.82234_skew313.60965_neigh1.26302_cross0.75611_n14.mtx","synthetic_91667_91667_6603697_avg72.04007_std19.01165_random_bw5241.08871_skew1.19322_neigh1.65278_cross0.76932_n14.mtx","synthetic_920462_920462_44975685_avg48.86208_std11.94475_random_bw166256.93189_skew1.2103_neigh1.69039_cross0.80408_n14.mtx","synthetic_948930_948930_21133875_avg22.27127_std29.77202_random_bw239983.61722_skew313.6206_neigh1.26346_cross0.75671_n14.mtx","synthetic_952203_952203_32554296_avg34.1884_std11.93673_random_bw208499.30602_skew1.69097_neigh1.6586_cross0.76805_n14.mtx","synthetic_952203_952203_35650826_avg37.44036_std11.95176_random_bw178182.3534_skew1.53737_neigh1.66758_cross0.77791_n14.mtx","synthetic_952203_952203_38750463_avg40.69559_std11.9572_random_bw182355.14926_skew1.4327_neigh1.67525_cross0.7865_n14.mtx","synthetic_952203_952203_41851500_avg43.95229_std11.95993_random_bw185907.68003_skew1.3207_neigh1.68123_cross0.79456_n14.mtx","synthetic_952203_952203_44952640_avg47.20909_std11.96013_random_bw170436.18015_skew1.22415_neigh1.68791_cross0.80043_n14.mtx","synthetic_952203_952203_46500626_avg48.83478_std13.95044_random_bw170085.21621_skew1.37536_neigh1.69447_cross0.78846_n14.mtx","synthetic_952203_952203_46500963_avg48.83514_std14.74428_random_bw169353.05265_skew1.45725_neigh1.69595_cross0.78191_n14.mtx","synthetic_952203_952203_46501138_avg48.83532_std15.536_random_bw168758.23927_skew1.53915_neigh1.69704_cross0.77638_n14.mtx","synthetic_952203_952203_46502122_avg48.83635_std13.15524_random_bw170806.64163_skew1.31385_neigh1.69248_cross0.79425_n14.mtx","synthetic_952203_952203_46502884_avg48.83715_std12.35905_random_bw171544.88603_skew1.23191_neigh1.69063_cross0.80082_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_random_bw120375.53352_skew1.19093_neigh1.69025_cross0.8034_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_random_bw131783.961_skew1.19093_neigh1.6902_cross0.80347_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_random_bw143412.56495_skew1.19093_neigh1.69053_cross0.8034_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_random_bw154853.41935_skew1.19093_neigh1.69028_cross0.80351_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_random_bw166308.11219_skew1.19093_neigh1.69031_cross0.80348_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_random_bw177753.75801_skew1.19093_neigh1.69007_cross0.80347_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_random_bw189177.26804_skew1.19093_neigh1.69023_cross0.80345_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_random_bw200570.66789_skew1.19093_neigh1.69027_cross0.80345_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_random_bw212041.87573_skew1.19093_neigh1.69043_cross0.80333_n14.mtx","synthetic_952203_952203_46504635_avg48.83899_std11.56217_random_bw172412.673_skew1.14992_neigh1.6892_cross0.80663_n14.mtx","synthetic_952203_952203_46504874_avg48.83924_std10.76526_random_bw173220.71164_skew1.06801_neigh1.68664_cross0.81356_n14.mtx","synthetic_952203_952203_46506617_avg48.84107_std9.96939_random_bw174360.33325_skew0.98603_neigh1.68579_cross0.81888_n14.mtx","synthetic_952203_952203_46507502_avg48.842_std9.17223_random_bw175367.02321_skew0.9041_neigh1.68327_cross0.82582_n14.mtx","synthetic_952203_952203_46508902_avg48.84347_std8.37556_random_bw177002.71627_skew0.82215_neigh1.68094_cross0.8324_n14.mtx","synthetic_952203_952203_48054277_avg50.46642_std11.96125_random_bw173372.13525_skew1.14004_neigh1.69321_cross0.80658_n14.mtx","synthetic_952203_952203_51155719_avg53.72354_std11.96081_random_bw176138.00989_skew1.08475_neigh1.69709_cross0.81142_n14.mtx","synthetic_952203_952203_54257059_avg56.98056_std11.96124_random_bw166937.37724_skew1.01823_neigh1.70097_cross0.81597_n14.mtx","synthetic_952203_952203_57358241_avg60.23741_std11.96072_random_bw169273.03323_skew0.95892_neigh1.70515_cross0.82049_n14.mtx","synthetic_952203_952203_60460102_avg63.49497_std11.9616_random_bw171406.13604_skew0.90566_neigh1.70892_cross0.82386_n14.mtx","synthetic_953812_953812_69251992_avg72.60549_std15.82731_random_bw14300.01986_skew1.05219_neigh1.69474_cross0.82044_n14.mtx","synthetic_968035_968035_11863180_avg12.25491_std37.64729_random_bw22809.30094_skew632.62368_neigh1.3222_cross0.78548_n14.mtx","synthetic_97223_97223_7006972_avg72.07114_std19.01384_random_bw5580.63884_skew1.19228_neigh1.65246_cross0.76899_n14.mtx","synthetic_983943_983943_48052837_avg48.83701_std11.95054_random_bw177774.01613_skew1.12953_neigh1.69013_cross0.804_n14.mtx","synthetic_98611_98611_5466825_avg55.43829_std11.03861_random_bw3858.831_skew0.894_neigh1.63088_cross0.78813_n14.mtx","synthetic_986703_986703_50142919_avg50.81865_std15.80386_random_bw14886.34812_skew1.44005_neigh1.67248_cross0.78656_n14.mtx","synthetic_986703_986703_54918080_avg55.65817_std15.81077_random_bw15208.31576_skew1.31772_neigh1.6788_cross0.79621_n14.mtx","synthetic_986703_986703_59694812_avg60.49927_std15.81312_random_bw14727.02983_skew1.2149_neigh1.68487_cross0.80451_n14.mtx","synthetic_986703_986703_64472911_avg65.34176_std15.81391_random_bw14462.32703_skew1.11197_neigh1.68893_cross0.81154_n14.mtx","synthetic_986703_986703_69250817_avg70.18405_std15.81417_random_bw14701.88268_skew1.0375_neigh1.6931_cross0.81692_n14.mtx","synthetic_986703_986703_71632861_avg72.5982_std20.55272_random_bw14534.71352_skew1.31411_neigh1.69992_cross0.79562_n14.mtx","synthetic_986703_986703_71633918_avg72.59927_std19.50045_random_bw14580.21846_skew1.2452_neigh1.69942_cross0.8008_n14.mtx","synthetic_986703_986703_71635339_avg72.60071_std18.448_random_bw14638.21943_skew1.17629_neigh1.6985_cross0.8063_n14.mtx","synthetic_986703_986703_71637178_avg72.60257_std17.39427_random_bw14693.79283_skew1.10736_neigh1.69733_cross0.81127_n14.mtx","synthetic_986703_986703_71638484_avg72.6039_std16.3415_random_bw14738.44751_skew1.03846_neigh1.69562_cross0.81775_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_random_bw10351.48927_skew1.0109_neigh1.69556_cross0.82059_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_random_bw11350.18676_skew1.0109_neigh1.69576_cross0.82027_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_random_bw13218.95443_skew1.0109_neigh1.63894_cross0.89577_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_random_bw13857.1301_skew1.0109_neigh1.1897_cross0.81891_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_random_bw13898.10027_skew1.0109_neigh1.30212_cross0.81916_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_random_bw14441.63744_skew1.0109_neigh1.63911_cross0.82048_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_random_bw14787.04212_skew1.0109_neigh1.69513_cross0.82047_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_random_bw15272.02993_skew1.0109_neigh1.69545_cross0.82033_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_random_bw15463.25016_skew1.0109_neigh1.75112_cross0.82045_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_random_bw15698.85861_skew1.0109_neigh1.71329_cross0.73895_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_random_bw16106.42649_skew1.0109_neigh1.72022_cross0.68496_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_random_bw19207.41674_skew1.0109_neigh1.69559_cross0.8201_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_random_bw20444.65782_skew1.0109_neigh1.91237_cross0.8206_n14.mtx","synthetic_986703_986703_71640715_avg72.60616_std15.28673_random_bw14825.68345_skew0.96953_neigh1.69453_cross0.82276_n14.mtx","synthetic_986703_986703_71642304_avg72.60777_std14.23366_random_bw14897.2269_skew0.90062_neigh1.69343_cross0.8287_n14.mtx","synthetic_986703_986703_71644194_avg72.60968_std13.17962_random_bw14980.02571_skew0.84548_neigh1.69135_cross0.83479_n14.mtx","synthetic_986703_986703_71646204_avg72.61172_std12.12567_random_bw15073.83895_skew0.77657_neigh1.68951_cross0.84052_n14.mtx","synthetic_986703_986703_71647231_avg72.61276_std11.07215_random_bw15188.71315_skew0.70769_neigh1.68786_cross0.84661_n14.mtx","synthetic_986703_986703_74029078_avg75.02671_std15.81428_random_bw14477.08036_skew0.97263_neigh1.69683_cross0.82287_n14.mtx","synthetic_986703_986703_78806613_avg79.86863_std15.81421_random_bw14655.966_skew0.91565_neigh1.69988_cross0.82778_n14.mtx","synthetic_986703_986703_83583783_avg84.71017_std15.81435_random_bw14520.34653_skew0.86518_neigh1.70339_cross0.83169_n14.mtx","synthetic_986703_986703_88361224_avg89.552_std15.81425_random_bw14683.12554_skew0.82017_neigh1.7057_cross0.83562_n14.mtx","synthetic_986703_986703_93139124_avg94.39428_std15.81414_random_bw14552.95978_skew0.76917_neigh1.70838_cross0.83944_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_random_bw35853.81968_skew632.66322_neigh1.32203_cross0.78644_n14.mtx","synthetic_268096_268096_19097540_avg71.23396_std101.70964_random_bw32428.45227_skew10.16041_neigh1.12478_cross0.8356_n14.mtx","synthetic_268096_268096_20328893_avg75.82692_std102.55303_random_bw32567.72158_skew10.2493_neigh1.12931_cross0.83183_n14.mtx","synthetic_4690002_4690002_18772478_avg4.00266_std1.15153_random_bw4545.51747_skew240.08974_neigh0.41531_cross0.6488_n14.mtx","synthetic_862664_862664_19114072_avg22.15703_std23.12053_random_bw218587.40518_skew315.24275_neigh1.26934_cross0.75615_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_random_bw218387.82176_skew313.61244_neigh1.26748_cross0.57934_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_random_bw223475.36954_skew1.19093_neigh1.69052_cross0.80348_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_random_bw37993.81919_skew632.66322_neigh1.32219_cross0.78642_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_random_bw40249.088_skew632.66322_neigh1.32209_cross0.7864_n14.mtx","synthetic_268096_268096_18479164_avg68.92741_std73.74951_random_bw32856.73895_skew10.4033_neigh1.1224_cross0.84096_n14.mtx","synthetic_268096_268096_18479645_avg68.92921_std80.36452_random_bw32647.3807_skew10.403_neigh1.12506_cross0.82761_n14.mtx","synthetic_268096_268096_18480123_avg68.93099_std86.42344_random_bw32612.95544_skew10.33017_neigh1.12468_cross0.82902_n14.mtx","synthetic_268096_268096_18480967_avg68.93414_std92.41882_random_bw32501.5288_skew10.25712_neigh1.12262_cross0.83284_n14.mtx","synthetic_268096_268096_18481691_avg68.93684_std98.3339_random_bw32411.34374_skew10.16964_neigh1.12316_cross0.83523_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_random_bw22631.95884_skew10.11157_neigh1.1236_cross0.83759_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_random_bw24810.92538_skew10.11157_neigh1.12535_cross0.83717_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_random_bw26946.62422_skew10.11157_neigh1.12451_cross0.83736_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_random_bw29104.95345_skew10.11157_neigh1.1245_cross0.83739_n14.mtx","synthetic_268096_268096_18482362_avg68.93934_std104.15593_random_bw32241.45427_skew10.0677_neigh1.12209_cross0.83937_n14.mtx","synthetic_268096_268096_18483075_avg68.942_std109.86851_random_bw32121.44382_skew9.95123_neigh1.1203_cross0.84327_n14.mtx","synthetic_268096_268096_18483963_avg68.94531_std115.46168_random_bw31912.0398_skew9.82017_neigh1.11692_cross0.84863_n14.mtx","synthetic_268096_268096_18484698_avg68.94806_std120.91939_random_bw31591.06942_skew9.68921_neigh1.11467_cross0.85538_n14.mtx","synthetic_268096_268096_18485774_avg68.95207_std126.22922_random_bw31186.72806_skew9.55806_neigh1.11278_cross0.86343_n14.mtx","synthetic_268096_268096_21560408_avg80.42048_std103.33442_random_bw32634.53398_skew10.30309_neigh1.13401_cross0.8296_n14.mtx","synthetic_268096_268096_22792018_avg85.01439_std104.0723_random_bw32673.47377_skew10.36278_neigh1.13741_cross0.82812_n14.mtx","synthetic_268096_268096_24023179_avg89.60663_std104.77794_random_bw32722.96392_skew10.40541_neigh1.1392_cross0.8264_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_random_bw209088.73821_skew313.61244_neigh1.15533_cross0.98278_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_random_bw216180.85999_skew313.61244_neigh1.23792_cross0.93242_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_random_bw217290.8255_skew313.61244_neigh1.2532_cross0.88202_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_random_bw217679.16493_skew313.61244_neigh1.25924_cross0.83135_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_random_bw218002.31666_skew313.61244_neigh1.26186_cross0.78113_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_random_bw218342.9789_skew313.61244_neigh1.26682_cross0.62984_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_random_bw218343.96868_skew313.61244_neigh1.26388_cross0.73052_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_random_bw218353.64903_skew313.61244_neigh1.26627_cross0.6806_n14.mtx","synthetic_862664_862664_19245171_avg22.309_std34.6357_random_bw217268.52825_skew313.08849_neigh1.25013_cross0.75439_n14.mtx","synthetic_862664_862664_19245832_avg22.30977_std36.57002_random_bw216933.33678_skew313.0777_neigh1.24283_cross0.75358_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_random_bw150854.23009_skew1.19093_neigh1.18385_cross0.80149_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_random_bw151895.95754_skew1.19093_neigh1.29667_cross0.80161_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_random_bw153481.11228_skew1.19093_neigh1.40932_cross0.80268_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_random_bw157146.27492_skew1.19093_neigh1.52169_cross0.80287_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_random_bw161158.68364_skew1.19093_neigh1.61052_cross0.88656_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_random_bw161480.87896_skew1.19093_neigh1.61762_cross0.88363_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_random_bw167259.59789_skew1.19093_neigh1.63354_cross0.80363_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_random_bw169055.00469_skew1.19093_neigh1.67727_cross0.83029_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_random_bw172056.91803_skew1.19093_neigh1.69026_cross0.80348_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_random_bw174466.89467_skew1.19093_neigh1.70003_cross0.77698_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_random_bw178531.20153_skew1.19093_neigh1.71349_cross0.72328_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_random_bw179918.61642_skew1.19093_neigh1.7464_cross0.80393_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_random_bw181221.45031_skew1.19093_neigh1.72367_cross0.66928_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_random_bw183413.76762_skew1.19093_neigh1.73057_cross0.61618_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_random_bw185134.26789_skew1.19093_neigh1.73599_cross0.56267_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_random_bw209833.19969_skew1.19093_neigh1.87978_cross0.80421_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_random_bw241430.72348_skew1.19093_neigh1.85806_cross0.80383_n14.mtx","synthetic_1060229_1060229_12992547_avg12.25447_std37.61142_diagonal_bw24877.30989_skew632.64621_neigh1.32175_cross0.78562_n14.mtx","synthetic_112749_112749_5727652_avg50.80002_std19.54637_diagonal_bw3425.86088_skew1.7559_neigh1.22834_cross0.71806_n14.mtx","synthetic_1152423_1152423_14123090_avg12.25513_std37.58092_diagonal_bw27066.48438_skew632.61237_neigh1.32182_cross0.78662_n14.mtx","synthetic_123486_123486_6272129_avg50.79223_std19.54882_diagonal_bw3753.58169_skew1.75633_neigh1.22934_cross0.71722_n14.mtx","synthetic_1244617_1244617_15251343_avg12.25384_std37.55512_diagonal_bw29219.49964_skew632.67869_neigh1.3223_cross0.78671_n14.mtx","synthetic_1336811_1336811_16381252_avg12.25398_std37.5327_diagonal_bw31339.89957_skew632.6718_neigh1.32267_cross0.78669_n14.mtx","synthetic_134225_134225_6816930_avg50.78733_std19.56874_diagonal_bw4082.86505_skew1.77628_neigh1.22794_cross0.71641_n14.mtx","synthetic_1382908_1382908_11844497_avg8.56492_std37.37396_diagonal_bw40250.63741_skew633.56515_neigh1.20442_cross0.7799_n14.mtx","synthetic_1382908_1382908_12976596_avg9.38356_std37.40231_diagonal_bw42376.82952_skew633.40762_neigh1.23873_cross0.78254_n14.mtx","synthetic_1382908_1382908_14113272_avg10.2055_std37.43376_diagonal_bw29741.04005_skew632.97168_neigh1.26021_cross0.78249_n14.mtx","synthetic_1382908_1382908_15242471_avg11.02204_std37.4678_diagonal_bw30995.34918_skew633.00226_neigh1.2701_cross0.78366_n14.mtx","synthetic_1382908_1382908_16349573_avg11.8226_std37.50438_diagonal_bw31873.17246_skew633.88554_neigh1.28656_cross0.78385_n14.mtx","synthetic_1382908_1382908_16848093_avg12.18309_std26.47929_diagonal_bw31429.05804_skew636.35882_neigh1.38049_cross0.79092_n14.mtx","synthetic_1382908_1382908_16896659_avg12.21821_std28.92386_diagonal_bw31501.87862_skew634.52686_neigh1.37656_cross0.79099_n14.mtx","synthetic_1382908_1382908_16922141_avg12.23664_std48.62398_diagonal_bw32239.82905_skew633.48814_neigh1.28004_cross0.7802_n14.mtx","synthetic_1382908_1382908_16923406_avg12.23755_std46.1541_diagonal_bw32382.62648_skew633.44071_neigh1.28391_cross0.78182_n14.mtx","synthetic_1382908_1382908_16926342_avg12.23967_std43.68598_diagonal_bw32452.39208_skew633.41236_neigh1.2921_cross0.78362_n14.mtx","synthetic_1382908_1382908_16931967_avg12.24374_std41.21881_diagonal_bw32524.36102_skew633.2016_neigh1.30009_cross0.7846_n14.mtx","synthetic_1382908_1382908_16933417_avg12.24479_std31.37528_diagonal_bw31740.52473_skew633.1473_neigh1.36561_cross0.7903_n14.mtx","synthetic_1382908_1382908_16941719_avg12.25079_std38.75385_diagonal_bw32512.7569_skew632.83654_neigh1.31404_cross0.78613_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_diagonal_bw22778.5047_skew632.66322_neigh1.32228_cross0.78682_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_diagonal_bw24917.9724_skew632.66322_neigh1.32217_cross0.78679_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_diagonal_bw27035.12761_skew632.66322_neigh1.32227_cross0.7868_n14.mtx","synthetic_1382908_1382908_16950422_avg12.25709_std36.29175_diagonal_bw32375.96458_skew632.51111_neigh1.33014_cross0.78815_n14.mtx","synthetic_1382908_1382908_16950660_avg12.25726_std33.83118_diagonal_bw32049.12593_skew632.50221_neigh1.34897_cross0.78969_n14.mtx","synthetic_1382908_1382908_17445616_avg12.61517_std37.54329_diagonal_bw32660.91296_skew635.05973_neigh1.31546_cross0.78564_n14.mtx","synthetic_1382908_1382908_18572951_avg13.43036_std37.58404_diagonal_bw33424.56483_skew634.94726_neigh1.34978_cross0.78747_n14.mtx","synthetic_1382908_1382908_19756490_avg14.28619_std37.62781_diagonal_bw28853.64125_skew633.10877_neigh1.36846_cross0.78954_n14.mtx","synthetic_1382908_1382908_20949793_avg15.14909_std37.6751_diagonal_bw29885.40906_skew631.18333_neigh1.36304_cross0.78832_n14.mtx","synthetic_1382908_1382908_22070012_avg15.95913_std37.72535_diagonal_bw30643.05341_skew631.49052_neigh1.34712_cross0.78696_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_diagonal_bw4732.21902_skew0.91217_neigh1.56188_cross0.89987_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_diagonal_bw5037.41918_skew0.91217_neigh1.59723_cross0.86565_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_diagonal_bw5204.83825_skew0.91217_neigh1.14624_cross0.78728_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_diagonal_bw5224.88197_skew0.91217_neigh1.25499_cross0.78723_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_diagonal_bw5232.21638_skew0.91217_neigh1.36051_cross0.78676_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_diagonal_bw5264.87446_skew0.91217_neigh1.46866_cross0.78769_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_diagonal_bw5313.69498_skew0.91217_neigh1.57687_cross0.78795_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_diagonal_bw5335.36823_skew0.91217_neigh1.6227_cross0.815_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_diagonal_bw5457.86719_skew0.91217_neigh1.63077_cross0.78851_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_diagonal_bw5549.51317_skew0.91217_neigh1.63866_cross0.76193_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_diagonal_bw5706.21311_skew0.91217_neigh1.64636_cross0.71043_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_diagonal_bw5738.5913_skew0.91217_neigh1.68499_cross0.78752_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_diagonal_bw5820.74673_skew0.91217_neigh1.65384_cross0.65812_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_diagonal_bw5912.46692_skew0.91217_neigh1.65889_cross0.60755_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_diagonal_bw5978.71195_skew0.91217_neigh1.66285_cross0.5548_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_diagonal_bw6442.07975_skew0.91217_neigh1.902_cross0.78882_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_diagonal_bw6597.81502_skew0.91217_neigh1.90014_cross0.78819_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_diagonal_bw7193.09661_skew0.91217_neigh1.79177_cross0.78885_n14.mtx","synthetic_140874_140874_7812605_avg55.4581_std11.08015_diagonal_bw6152.40279_skew2.11947_neigh1.6145_cross0.85299_n14.mtx","synthetic_140874_140874_7812630_avg55.45828_std11.08_diagonal_bw6149.91056_skew2.06537_neigh1.61493_cross0.85302_n14.mtx","synthetic_1429004_1429004_17510920_avg12.25393_std37.51291_diagonal_bw33558.90432_skew632.67408_neigh1.32147_cross0.78663_n14.mtx","synthetic_144963_144963_7361596_avg50.78259_std19.57819_diagonal_bw4399.88447_skew1.77654_neigh1.22836_cross0.71716_n14.mtx","synthetic_1521198_1521198_18638978_avg12.25283_std37.49579_diagonal_bw35682.83773_skew632.73123_neigh1.32136_cross0.78772_n14.mtx","synthetic_152542_152542_8133417_avg53.3192_std4.76067_diagonal_bw10659.33841_skew3.36991_neigh1.81127_cross0.94184_n14.mtx","synthetic_155701_155701_7905533_avg50.77381_std19.58895_diagonal_bw4730.89037_skew1.77702_neigh1.22735_cross0.71638_n14.mtx","synthetic_161070_161070_10085122_avg62.61329_std19.67482_diagonal_bw4970.48093_skew1.44357_neigh1.23583_cross0.74036_n14.mtx","synthetic_161070_161070_10630527_avg65.99942_std19.67949_diagonal_bw4997.9576_skew1.36366_neigh1.23787_cross0.74712_n14.mtx","synthetic_161070_161070_5763734_avg35.78403_std19.07947_diagonal_bw4691.45329_skew2.52112_neigh1.21157_cross0.6613_n14.mtx","synthetic_161070_161070_6293192_avg39.07116_std19.27381_diagonal_bw4754.31728_skew2.30167_neigh1.21361_cross0.6789_n14.mtx","synthetic_161070_161070_6828184_avg42.39265_std19.41502_diagonal_bw4805.07083_skew2.13734_neigh1.21786_cross0.69146_n14.mtx","synthetic_161070_161070_7366852_avg45.73696_std19.51164_diagonal_bw4846.14217_skew1.97353_neigh1.22207_cross0.70264_n14.mtx","synthetic_161070_161070_7908198_avg49.0979_std19.57809_diagonal_bw4854.49827_skew1.83108_neigh1.2254_cross0.71257_n14.mtx","synthetic_161070_161070_8177031_avg50.76694_std16.39176_diagonal_bw4982.53222_skew1.48193_neigh1.22352_cross0.73821_n14.mtx","synthetic_161070_161070_8177317_avg50.76872_std15.08847_diagonal_bw5015.21029_skew1.36366_neigh1.22314_cross0.74727_n14.mtx","synthetic_161070_161070_8177462_avg50.76962_std17.68784_diagonal_bw4942.64516_skew1.59998_neigh1.22734_cross0.72915_n14.mtx","synthetic_161070_161070_8177930_avg50.77252_std13.78024_diagonal_bw5053.39532_skew1.24531_neigh1.22227_cross0.75674_n14.mtx","synthetic_161070_161070_8178443_avg50.77571_std18.96773_diagonal_bw4913.07711_skew1.71784_neigh1.22757_cross0.72014_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_diagonal_bw3436.43137_skew1.77657_neigh1.22839_cross0.71742_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_diagonal_bw3766.73128_skew1.77657_neigh1.22745_cross0.71673_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_diagonal_bw4085.27859_skew1.77657_neigh1.22766_cross0.7165_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_diagonal_bw4397.09225_skew1.77657_neigh1.19684_cross0.82225_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_diagonal_bw4402.87749_skew1.77657_neigh1.22824_cross0.71675_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_diagonal_bw4582.03666_skew1.77657_neigh1.21427_cross0.78739_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_diagonal_bw4732.34129_skew1.77657_neigh1.22757_cross0.71639_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_diagonal_bw4806.31347_skew1.77657_neigh1.22333_cross0.7394_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_diagonal_bw4848.33162_skew1.77657_neigh0.945_cross0.71714_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_diagonal_bw4854.14888_skew1.77657_neigh0.86533_cross0.7155_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_diagonal_bw4858.05593_skew1.77657_neigh1.10606_cross0.71633_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_diagonal_bw4859.13659_skew1.77657_neigh1.02495_cross0.71552_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_diagonal_bw4887.50244_skew1.77657_neigh1.22794_cross0.7157_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_diagonal_bw4889.05284_skew1.77657_neigh1.18606_cross0.71625_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_diagonal_bw4905.36219_skew1.77657_neigh1.26702_cross0.71513_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_diagonal_bw4920.32821_skew1.77657_neigh1.34684_cross0.71421_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_diagonal_bw4938.19408_skew1.77657_neigh1.42871_cross0.71564_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_diagonal_bw4963.01172_skew1.77657_neigh1.50995_cross0.71659_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_diagonal_bw4974.48644_skew1.77657_neigh1.2296_cross0.6917_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_diagonal_bw5006.7359_skew1.77657_neigh1.58991_cross0.71725_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_diagonal_bw5047.52778_skew1.77657_neigh1.22518_cross0.71711_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_diagonal_bw5128.09611_skew1.77657_neigh1.23448_cross0.64524_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_diagonal_bw5252.15061_skew1.77657_neigh1.23809_cross0.60038_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_diagonal_bw5358.15437_skew1.77657_neigh1.24126_cross0.55385_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_diagonal_bw5368.754_skew1.77657_neigh1.22547_cross0.71707_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_diagonal_bw5466.81631_skew1.77657_neigh1.24378_cross0.50496_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_diagonal_bw5702.6109_skew1.77657_neigh1.22489_cross0.7154_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_diagonal_bw6025.13256_skew1.77657_neigh1.22428_cross0.71487_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_diagonal_bw6352.66382_skew1.77657_neigh1.22335_cross0.71563_n14.mtx","synthetic_161070_161070_8181013_avg50.79166_std20.23119_diagonal_bw4876.39439_skew1.83511_neigh1.22713_cross0.71077_n14.mtx","synthetic_161070_161070_8181297_avg50.79343_std19.67479_diagonal_bw5249.20404_skew2.30751_neigh1.21368_cross0.79983_n14.mtx","synthetic_161070_161070_8184643_avg50.8142_std21.47457_diagonal_bw4835.93918_skew1.95193_neigh1.22927_cross0.70445_n14.mtx","synthetic_161070_161070_8190209_avg50.84876_std22.69412_diagonal_bw4805.6814_skew2.06792_neigh1.23025_cross0.69419_n14.mtx","synthetic_161070_161070_8197860_avg50.89626_std23.8911_diagonal_bw4766.49097_skew2.18295_neigh1.23139_cross0.68671_n14.mtx","synthetic_161070_161070_8207913_avg50.95867_std25.06294_diagonal_bw4739.27473_skew2.29679_neigh1.2307_cross0.67748_n14.mtx","synthetic_161070_161070_8451173_avg52.46895_std19.62109_diagonal_bw4901.00951_skew1.72542_neigh1.22729_cross0.72045_n14.mtx","synthetic_161070_161070_8995114_avg55.84599_std19.64654_diagonal_bw4934.95017_skew1.61433_neigh1.23106_cross0.72911_n14.mtx","synthetic_161070_161070_9539943_avg59.22855_std19.66505_diagonal_bw4956.28317_skew1.53256_neigh1.23358_cross0.73578_n14.mtx","synthetic_1613392_1613392_19770221_avg12.25382_std37.48086_diagonal_bw37889.56893_skew632.67976_neigh1.3217_cross0.78728_n14.mtx","synthetic_166438_166438_8452047_avg50.78195_std19.60687_diagonal_bw5049.85631_skew1.77658_neigh1.22495_cross0.71681_n14.mtx","synthetic_167070_167070_8908050_avg53.31927_std4.75919_diagonal_bw11705.28266_skew3.3699_neigh1.8115_cross0.9416_n14.mtx","synthetic_1705586_1705586_20897972_avg12.25266_std37.46728_diagonal_bw40049.59136_skew632.73974_neigh1.32185_cross0.78691_n14.mtx","synthetic_177177_177177_8998963_avg50.79081_std19.6143_diagonal_bw5379.20051_skew1.77609_neigh1.22643_cross0.71605_n14.mtx","synthetic_1797780_1797780_22027651_avg12.2527_std37.45466_diagonal_bw42214.60549_skew632.7381_neigh1.32143_cross0.78759_n14.mtx","synthetic_181598_181598_9682693_avg53.31938_std4.75813_diagonal_bw12669.60339_skew3.36989_neigh1.81184_cross0.9412_n14.mtx","synthetic_185639_185639_15001505_avg80.81009_std108.51788_diagonal_bw24304.53698_skew5.36059_neigh1.20249_cross0.88135_n14.mtx","synthetic_185639_185639_15005658_avg80.83247_std121.76167_diagonal_bw24068.62026_skew10.22074_neigh1.21908_cross0.83189_n14.mtx","synthetic_185639_185639_15005807_avg80.83327_std121.07944_diagonal_bw24098.67153_skew9.70104_neigh1.21713_cross0.83417_n14.mtx","synthetic_185639_185639_15006000_avg80.83431_std120.29592_diagonal_bw24022.966_skew9.18132_neigh1.21687_cross0.83669_n14.mtx","synthetic_185639_185639_15006242_avg80.83561_std119.38413_diagonal_bw24043.49472_skew8.66158_neigh1.21482_cross0.83974_n14.mtx","synthetic_185639_185639_15006279_avg80.83581_std111.39593_diagonal_bw24110.85286_skew5.93999_neigh1.20322_cross0.86853_n14.mtx","synthetic_185639_185639_15006681_avg80.83798_std118.31728_diagonal_bw24061.44702_skew8.14174_neigh1.21351_cross0.84318_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_diagonal_bw18670.03953_skew7.8818_neigh1.21456_cross0.84468_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_diagonal_bw20186.89682_skew7.8818_neigh1.2138_cross0.8455_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_diagonal_bw21725.13769_skew7.8818_neigh1.21452_cross0.84491_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_diagonal_bw22498.75533_skew7.8818_neigh1.19263_cross0.87688_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_diagonal_bw22743.18636_skew7.8818_neigh1.19716_cross0.87274_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_diagonal_bw23239.31109_skew7.8818_neigh1.21284_cross0.84509_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_diagonal_bw23967.12665_skew7.8818_neigh1.49579_cross0.84608_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_diagonal_bw24008.68622_skew7.8818_neigh1.33536_cross0.84607_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_diagonal_bw24018.14304_skew7.8818_neigh1.41334_cross0.84578_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_diagonal_bw24034.31555_skew7.8818_neigh1.25453_cross0.84477_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_diagonal_bw24106.64346_skew7.8818_neigh1.1756_cross0.84461_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_diagonal_bw24114.69933_skew7.8818_neigh1.57356_cross0.8475_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_diagonal_bw24131.56928_skew7.8818_neigh1.09712_cross0.84415_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_diagonal_bw24182.54428_skew7.8818_neigh1.01304_cross0.84375_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_diagonal_bw24230.56658_skew7.8818_neigh0.93581_cross0.84308_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_diagonal_bw24274.8686_skew7.8818_neigh0.85549_cross0.84272_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_diagonal_bw24811.92989_skew7.8818_neigh1.21348_cross0.84512_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_diagonal_bw24955.57252_skew7.8818_neigh1.2234_cross0.81646_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_diagonal_bw26276.50771_skew7.8818_neigh1.23325_cross0.75987_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_diagonal_bw26290.01987_skew7.8818_neigh1.21444_cross0.84532_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_diagonal_bw27182.66132_skew7.8818_neigh1.23712_cross0.7061_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_diagonal_bw27777.91443_skew7.8818_neigh1.2129_cross0.84488_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_diagonal_bw27910.49869_skew7.8818_neigh1.24252_cross0.65023_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_diagonal_bw28492.91056_skew7.8818_neigh1.24563_cross0.59619_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_diagonal_bw29291.3444_skew7.8818_neigh1.21419_cross0.84494_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_diagonal_bw30735.2641_skew7.8818_neigh1.21293_cross0.84528_n14.mtx","synthetic_185639_185639_15007163_avg80.84057_std117.05079_diagonal_bw24046.68677_skew7.60954_neigh1.21368_cross0.84709_n14.mtx","synthetic_185639_185639_15007417_avg80.84194_std113.68148_diagonal_bw24066.61476_skew6.50848_neigh1.20615_cross0.85925_n14.mtx","synthetic_185639_185639_15007427_avg80.84199_std115.53272_diagonal_bw24030.31379_skew7.06512_neigh1.21132_cross0.85201_n14.mtx","synthetic_187915_187915_9545196_avg50.79529_std19.60651_diagonal_bw5703.99496_skew1.77585_neigh1.22534_cross0.71637_n14.mtx","synthetic_196126_196126_10457351_avg53.31955_std4.75831_diagonal_bw13779.54019_skew3.36988_neigh1.81248_cross0.94112_n14.mtx","synthetic_198653_198653_10092676_avg50.80556_std19.60715_diagonal_bw6040.72041_skew1.77529_neigh1.2246_cross0.71507_n14.mtx","synthetic_205540_205540_14169500_avg68.93792_std101.25978_diagonal_bw23682.46024_skew10.08243_neigh1.12517_cross0.83739_n14.mtx","synthetic_209391_209391_10637567_avg50.80241_std19.60148_diagonal_bw6367.17197_skew1.77546_neigh1.22433_cross0.71571_n14.mtx","synthetic_210654_210654_11231930_avg53.31933_std4.75723_diagonal_bw14684.9664_skew3.3699_neigh1.81334_cross0.94103_n14.mtx","synthetic_217918_217918_10485592_avg48.11714_std4.75335_diagonal_bw15572.84676_skew3.36435_neigh1.77256_cross0.93985_n14.mtx","synthetic_217918_217918_11231492_avg51.53999_std4.76893_diagonal_bw15690.56769_skew3.36554_neigh1.75769_cross0.93992_n14.mtx","synthetic_217918_217918_11582841_avg53.15229_std3.34176_diagonal_bw14752.54676_skew3.38363_neigh1.83064_cross0.94044_n14.mtx","synthetic_217918_217918_11590559_avg53.18771_std3.65776_diagonal_bw14759.15174_skew3.38071_neigh1.82883_cross0.94023_n14.mtx","synthetic_217918_217918_11598747_avg53.22528_std3.97324_diagonal_bw14804.58821_skew3.37762_neigh1.82457_cross0.94082_n14.mtx","synthetic_217918_217918_11607157_avg53.26387_std4.28638_diagonal_bw14926.5797_skew3.37445_neigh1.82058_cross0.94051_n14.mtx","synthetic_217918_217918_11608217_avg53.26874_std4.76281_diagonal_bw14978.94192_skew4.08741_neigh1.8196_cross0.94057_n14.mtx","synthetic_217918_217918_11610489_avg53.27916_std4.76243_diagonal_bw14965.80924_skew3.93626_neigh1.8186_cross0.94068_n14.mtx","synthetic_217918_217918_11612905_avg53.29025_std4.75996_diagonal_bw15022.46198_skew3.76635_neigh1.8171_cross0.94082_n14.mtx","synthetic_217918_217918_11615389_avg53.30165_std4.60014_diagonal_bw15069.01749_skew3.37135_neigh1.81567_cross0.941_n14.mtx","synthetic_217918_217918_11615400_avg53.3017_std4.75902_diagonal_bw15068.40239_skew3.61524_neigh1.81564_cross0.94109_n14.mtx","synthetic_217918_217918_11617960_avg53.31345_std4.75796_diagonal_bw15125.67497_skew3.44541_neigh1.81377_cross0.94083_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_diagonal_bw10615.26121_skew3.36989_neigh1.81293_cross0.941_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_diagonal_bw11723.96417_skew3.36989_neigh1.81304_cross0.94098_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_diagonal_bw12548.56527_skew3.36989_neigh1.50637_cross0.94082_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_diagonal_bw12555.41407_skew3.36989_neigh1.26528_cross0.93926_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_diagonal_bw12561.18815_skew3.36989_neigh1.38503_cross0.94038_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_diagonal_bw12662.63745_skew3.36989_neigh1.81249_cross0.94104_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_diagonal_bw12875.08402_skew3.36989_neigh1.6263_cross0.94024_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_diagonal_bw13192.62211_skew3.36989_neigh1.74925_cross0.94027_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_diagonal_bw13589.06211_skew3.36989_neigh1.8131_cross0.94142_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_diagonal_bw14637.1457_skew3.36989_neigh1.81291_cross0.94097_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_diagonal_bw14639.64249_skew3.36989_neigh1.83771_cross0.66265_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_diagonal_bw14678.79878_skew3.36989_neigh1.83641_cross0.72459_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_diagonal_bw14692.70001_skew3.36989_neigh1.83507_cross0.78486_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_diagonal_bw14839.20089_skew3.36989_neigh1.8307_cross0.84682_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_diagonal_bw15050.52926_skew3.36989_neigh1.82143_cross0.90888_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_diagonal_bw15508.12602_skew3.36989_neigh1.7873_cross0.97163_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_diagonal_bw15626.79599_skew3.36989_neigh1.81205_cross0.94094_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_diagonal_bw15937.73698_skew3.36989_neigh1.74855_cross0.98962_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_diagonal_bw16599.98545_skew3.36989_neigh1.81308_cross0.9411_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_diagonal_bw16602.40328_skew3.36989_neigh1.87566_cross0.93981_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_diagonal_bw17572.60761_skew3.36989_neigh1.81311_cross0.94094_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_diagonal_bw18584.92687_skew3.36989_neigh1.81265_cross0.94092_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_diagonal_bw19575.60228_skew3.36989_neigh1.81285_cross0.94097_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_diagonal_bw5775.00642_skew3.36989_neigh1.93946_cross0.94054_n14.mtx","synthetic_217918_217918_11620586_avg53.3255_std4.75691_diagonal_bw15174.3723_skew3.29438_neigh1.81188_cross0.94105_n14.mtx","synthetic_217918_217918_11622834_avg53.33581_std4.91358_diagonal_bw15211.80314_skew3.36855_neigh1.80941_cross0.94098_n14.mtx","synthetic_217918_217918_11623241_avg53.33768_std4.75547_diagonal_bw15225.00254_skew3.12466_neigh1.80948_cross0.9409_n14.mtx","synthetic_217918_217918_11625780_avg53.34933_std4.75397_diagonal_bw15317.68741_skew2.97381_neigh1.806_cross0.94062_n14.mtx","synthetic_217918_217918_11628184_avg53.36036_std4.75308_diagonal_bw15462.19266_skew2.80432_neigh1.8014_cross0.94016_n14.mtx","synthetic_217918_217918_11628620_avg53.36237_std5.22805_diagonal_bw15467.4387_skew3.36637_neigh1.80121_cross0.93991_n14.mtx","synthetic_217918_217918_11630299_avg53.37007_std4.7531_diagonal_bw15559.9952_skew2.65373_neigh1.79743_cross0.9399_n14.mtx","synthetic_217918_217918_11632309_avg53.37929_std5.54522_diagonal_bw15720.40304_skew3.36499_neigh1.78973_cross0.93981_n14.mtx","synthetic_217918_217918_11634099_avg53.38751_std5.86172_diagonal_bw15948.7068_skew3.36432_neigh1.77763_cross0.93909_n14.mtx","synthetic_217918_217918_11634741_avg53.39045_std6.17863_diagonal_bw16156.60265_skew3.36408_neigh1.76393_cross0.93894_n14.mtx","synthetic_217918_217918_12031582_avg55.21151_std4.74931_diagonal_bw15391.15638_skew3.36503_neigh1.82291_cross0.9415_n14.mtx","synthetic_217918_217918_12797820_avg58.72769_std4.7736_diagonal_bw13915.978_skew3.3591_neigh1.75599_cross0.93955_n14.mtx","synthetic_217918_217918_13561979_avg62.23432_std4.75526_diagonal_bw13183.43978_skew3.37058_neigh1.83856_cross0.94033_n14.mtx","synthetic_217918_217918_14374351_avg65.9622_std4.76726_diagonal_bw14447.91926_skew3.36614_neigh1.78633_cross0.93943_n14.mtx","synthetic_217918_217918_15093236_avg69.26108_std4.76589_diagonal_bw13990.39745_skew3.37475_neigh1.82922_cross0.94054_n14.mtx","synthetic_217918_217918_8143542_avg37.36975_std4.75333_diagonal_bw20866.53868_skew3.36182_neigh1.74733_cross0.9391_n14.mtx","synthetic_217918_217918_8924884_avg40.95524_std4.76049_diagonal_bw22294.09855_skew3.34621_neigh1.71612_cross0.93867_n14.mtx","synthetic_217918_217918_9684687_avg44.44189_std4.75752_diagonal_bw14468.16582_skew3.36525_neigh1.77178_cross0.93964_n14.mtx","synthetic_223413_223413_15400297_avg68.93196_std101.25897_diagonal_bw25758.67821_skew10.0979_neigh1.12654_cross0.83736_n14.mtx","synthetic_225181_225181_12006505_avg53.31935_std4.75652_diagonal_bw15512.77201_skew3.3699_neigh1.81265_cross0.94113_n14.mtx","synthetic_239709_239709_12781051_avg53.31903_std4.75618_diagonal_bw16764.10618_skew3.36992_neigh1.81204_cross0.94086_n14.mtx","synthetic_241286_241286_16633118_avg68.93528_std101.25723_diagonal_bw27818.08052_skew10.09737_neigh1.12625_cross0.83765_n14.mtx","synthetic_254237_254237_13555788_avg53.31949_std4.75562_diagonal_bw17672.99219_skew3.36988_neigh1.81182_cross0.94118_n14.mtx","synthetic_259159_259159_17866974_avg68.94213_std101.25583_diagonal_bw29844.7971_skew10.11077_neigh1.1228_cross0.83747_n14.mtx","synthetic_26684_26684_10204693_avg382.42741_std245.44826_diagonal_bw11448.26195_skew1.57042_neigh1.97454_cross0.92536_n14.mtx","synthetic_268096_268096_12941160_avg48.27062_std95.20699_diagonal_bw26357.31682_skew9.2754_neigh1.08518_cross0.89421_n14.mtx","synthetic_268096_268096_14172610_avg52.86394_std97.03631_diagonal_bw29123.12309_skew9.5554_neigh1.0936_cross0.86612_n14.mtx","synthetic_268096_268096_15403716_avg57.45597_std98.50696_diagonal_bw30125.15948_skew9.75606_neigh1.10358_cross0.85268_n14.mtx","synthetic_268096_268096_16634948_avg62.04848_std99.72897_diagonal_bw30499.11718_skew9.92694_neigh1.11411_cross0.84513_n14.mtx","synthetic_268096_268096_17866195_avg66.64104_std100.78017_diagonal_bw30751.27234_skew10.05925_neigh1.12044_cross0.83953_n14.mtx","synthetic_268096_268096_18479164_avg68.92741_std73.74951_diagonal_bw31377.65125_skew10.4033_neigh1.1237_cross0.84107_n14.mtx","synthetic_268096_268096_18479645_avg68.92921_std80.36452_diagonal_bw31191.46774_skew10.403_neigh1.12752_cross0.82739_n14.mtx","synthetic_268096_268096_18480123_avg68.93099_std86.42344_diagonal_bw31125.07403_skew10.33017_neigh1.12545_cross0.82944_n14.mtx","synthetic_268096_268096_18480967_avg68.93414_std92.41882_diagonal_bw31046.67467_skew10.25712_neigh1.12466_cross0.8325_n14.mtx","synthetic_268096_268096_18481691_avg68.93684_std98.3339_diagonal_bw30926.5338_skew10.16964_neigh1.12377_cross0.83538_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_diagonal_bw21923.44482_skew10.11157_neigh1.12703_cross0.83756_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_diagonal_bw23935.8137_skew10.11157_neigh1.12612_cross0.8371_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_diagonal_bw25895.422_skew10.11157_neigh1.12483_cross0.83763_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_diagonal_bw27899.12455_skew10.11157_neigh1.12567_cross0.83764_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_diagonal_bw29902.52638_skew10.11157_neigh1.12497_cross0.83752_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_diagonal_bw31924.98227_skew10.11157_neigh1.12445_cross0.83711_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_diagonal_bw33817.07914_skew10.11157_neigh1.12399_cross0.83739_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_diagonal_bw35779.99142_skew10.11157_neigh1.12322_cross0.83708_n14.mtx","synthetic_268096_268096_18482362_avg68.93934_std104.15593_diagonal_bw30765.9396_skew10.0677_neigh1.12333_cross0.83908_n14.mtx","synthetic_268096_268096_18483075_avg68.942_std109.86851_diagonal_bw30647.70205_skew9.95123_neigh1.12162_cross0.84333_n14.mtx","synthetic_268096_268096_18483963_avg68.94531_std115.46168_diagonal_bw30446.51037_skew9.82017_neigh1.11909_cross0.84887_n14.mtx","synthetic_268096_268096_18484698_avg68.94806_std120.91939_diagonal_bw30186.45871_skew9.68921_neigh1.11754_cross0.85523_n14.mtx","synthetic_268096_268096_18485774_avg68.95207_std126.22922_diagonal_bw29770.97031_skew9.55806_neigh1.11488_cross0.86335_n14.mtx","synthetic_268096_268096_19097540_avg71.23396_std101.70964_diagonal_bw30933.99411_skew10.16041_neigh1.12541_cross0.83599_n14.mtx","synthetic_268096_268096_20328893_avg75.82692_std102.55303_diagonal_bw31081.7585_skew10.2493_neigh1.1336_cross0.83222_n14.mtx","synthetic_268096_268096_21560408_avg80.42048_std103.33442_diagonal_bw31148.48253_skew10.30309_neigh1.13451_cross0.82948_n14.mtx","synthetic_268096_268096_22792018_avg85.01439_std104.0723_diagonal_bw31199.25223_skew10.36278_neigh1.13885_cross0.8282_n14.mtx","synthetic_268096_268096_24023179_avg89.60663_std104.77794_diagonal_bw31239.70547_skew10.40541_neigh1.14366_cross0.82667_n14.mtx","synthetic_268765_268765_14330411_avg53.31948_std4.75541_diagonal_bw18731.37211_skew3.36988_neigh1.81156_cross0.94066_n14.mtx","synthetic_277032_277032_19096094_avg68.931_std101.25944_diagonal_bw31953.72601_skew10.12707_neigh1.12352_cross0.83736_n14.mtx","synthetic_283293_283293_15104954_avg53.31919_std4.75508_diagonal_bw19749.01419_skew3.36991_neigh1.81186_cross0.9407_n14.mtx","synthetic_29225_29225_11176419_avg382.42666_std245.44724_diagonal_bw12425.97666_skew1.57043_neigh1.97381_cross0.92588_n14.mtx","synthetic_294905_294905_20328825_avg68.93347_std101.25836_diagonal_bw33931.21622_skew10.12667_neigh1.12413_cross0.83784_n14.mtx","synthetic_2998_2998_6123474_avg2042.51968_std621.91091_diagonal_bw2392.08572_skew0.46779_neigh1.915_cross0.85498_n14.mtx","synthetic_312778_312778_21563736_avg68.94262_std101.25558_diagonal_bw36065.10736_skew10.1397_neigh1.12116_cross0.83732_n14.mtx","synthetic_31766_31766_12148131_avg382.42558_std245.44713_diagonal_bw13817.64257_skew1.57043_neigh1.97384_cross0.91941_n14.mtx","synthetic_3284_3284_6742726_avg2053.20524_std666.12092_diagonal_bw2537.96711_skew0.59945_neigh1.8895_cross0.83825_n14.mtx","synthetic_330651_330651_22801392_avg68.95909_std101.25183_diagonal_bw38055.27574_skew10.13704_neigh1.1241_cross0.83714_n14.mtx","synthetic_34308_34308_13120227_avg382.42471_std245.44609_diagonal_bw14650.04739_skew1.57044_neigh1.97406_cross0.9232_n14.mtx","synthetic_348524_348524_24034194_avg68.95994_std101.25128_diagonal_bw40157.18736_skew10.1369_neigh1.12259_cross0.83707_n14.mtx","synthetic_3569_3569_7446679_avg2086.48893_std694.14116_diagonal_bw2698.16419_skew0.71053_neigh1.86781_cross0.82629_n14.mtx","synthetic_36849_36849_14091918_avg382.42335_std245.44508_diagonal_bw15781.99004_skew1.57045_neigh1.97399_cross0.92492_n14.mtx","synthetic_38120_38120_10204584_avg267.69633_std171.81206_diagonal_bw16895.89722_skew1.57008_neigh1.97152_cross0.92219_n14.mtx","synthetic_38120_38120_11176434_avg293.19082_std188.17435_diagonal_bw17618.12893_skew1.5717_neigh1.97171_cross0.9225_n14.mtx","synthetic_38120_38120_12148318_avg318.6862_std204.53731_diagonal_bw18097.15551_skew1.56993_neigh1.97265_cross0.92312_n14.mtx","synthetic_38120_38120_13120182_avg344.18106_std220.90077_diagonal_bw18722.69885_skew1.57132_neigh1.97302_cross0.92199_n14.mtx","synthetic_38120_38120_14004492_avg367.37912_std206.50003_diagonal_bw16517.99428_skew1.32185_neigh1.97296_cross0.92309_n14.mtx","synthetic_38120_38120_14092040_avg369.67576_std237.2634_diagonal_bw15900.27104_skew1.56982_neigh1.97398_cross0.92174_n14.mtx","synthetic_38120_38120_14186695_avg372.15884_std217.74813_diagonal_bw16610.97285_skew1.39145_neigh1.97336_cross0.92298_n14.mtx","synthetic_38120_38120_14353565_avg376.53633_std228.90355_diagonal_bw16362.02206_skew1.46191_neigh1.97392_cross0.9229_n14.mtx","synthetic_38120_38120_14506403_avg380.54572_std239.95763_diagonal_bw16221.47907_skew1.5332_neigh1.97371_cross0.92545_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_diagonal_bw11840.96207_skew1.57045_neigh1.97389_cross0.92172_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_diagonal_bw13051.93368_skew1.57045_neigh1.97364_cross0.9215_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_diagonal_bw14054.5128_skew1.57045_neigh1.97418_cross0.92287_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_diagonal_bw14901.13825_skew1.57045_neigh1.9738_cross0.92245_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_diagonal_bw15702.30456_skew1.57045_neigh1.97328_cross0.9903_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_diagonal_bw16005.2249_skew1.57045_neigh1.97376_cross0.92394_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_diagonal_bw16107.56653_skew1.57045_neigh1.90891_cross0.91992_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_diagonal_bw16165.12815_skew1.57045_neigh1.3922_cross0.92174_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_diagonal_bw16173.10614_skew1.57045_neigh1.64922_cross0.92071_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_diagonal_bw16181.2495_skew1.57045_neigh1.51874_cross0.92011_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_diagonal_bw16189.91522_skew1.57045_neigh1.77707_cross0.91956_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_diagonal_bw16251.80551_skew1.57045_neigh1.97384_cross0.89306_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_diagonal_bw16403.35273_skew1.57045_neigh1.97397_cross0.72233_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_diagonal_bw16412.02442_skew1.57045_neigh1.97389_cross0.66503_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_diagonal_bw16450.53809_skew1.57045_neigh1.9739_cross0.77998_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_diagonal_bw16454.6271_skew1.57045_neigh1.97336_cross0.95155_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_diagonal_bw16504.44638_skew1.57045_neigh1.97349_cross0.83444_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_diagonal_bw16601.00073_skew1.57045_neigh1.97376_cross0.92272_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_diagonal_bw16971.93463_skew1.57045_neigh1.9734_cross0.92126_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_diagonal_bw17802.86905_skew1.57045_neigh1.97345_cross0.9247_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_diagonal_bw18943.77943_skew1.57045_neigh1.97338_cross0.92328_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_diagonal_bw20020.58657_skew1.57045_neigh1.97348_cross0.92229_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_diagonal_bw21202.83549_skew1.57045_neigh1.97368_cross0.92139_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_diagonal_bw5783.47169_skew1.57045_neigh1.99394_cross0.9293_n14.mtx","synthetic_38120_38120_14646455_avg384.2197_std250.9036_diagonal_bw16369.04491_skew1.60788_neigh1.97362_cross0.92456_n14.mtx","synthetic_38120_38120_14774683_avg387.5835_std261.73289_diagonal_bw16271.8489_skew1.68071_neigh1.97337_cross0.92347_n14.mtx","synthetic_38120_38120_14892124_avg390.66432_std272.44037_diagonal_bw16642.15774_skew1.75428_neigh1.9735_cross0.92246_n14.mtx","synthetic_38120_38120_15009192_avg393.73536_std249.23315_diagonal_bw17334.85677_skew1.65915_neigh1.95771_cross0.90213_n14.mtx","synthetic_38120_38120_15029193_avg394.26005_std283.24138_diagonal_bw17593.26878_skew1.83569_neigh1.95985_cross0.92052_n14.mtx","synthetic_38120_38120_15063906_avg395.17067_std253.62681_diagonal_bw16677.30215_skew1.57104_neigh1.97408_cross0.92294_n14.mtx","synthetic_38120_38120_15246247_avg399.95401_std294.48908_diagonal_bw17717.6793_skew1.94534_neigh1.95613_cross0.91225_n14.mtx","synthetic_38120_38120_15509350_avg406.85598_std254.10081_diagonal_bw16470.84627_skew1.83638_neigh1.96076_cross0.87593_n14.mtx","synthetic_38120_38120_15831412_avg415.30462_std256.81933_diagonal_bw15755.80622_skew2.01225_neigh1.96337_cross0.85513_n14.mtx","synthetic_38120_38120_16013257_avg420.07495_std256.51996_diagonal_bw15554.95262_skew2.18991_neigh1.96477_cross0.83814_n14.mtx","synthetic_38120_38120_16417883_avg430.68948_std273.30431_diagonal_bw17866.36671_skew1.63763_neigh1.95894_cross0.90329_n14.mtx","synthetic_38120_38120_17911818_avg469.8798_std294.64729_diagonal_bw15437.19732_skew1.77305_neigh1.96314_cross0.88403_n14.mtx","synthetic_38120_38120_17943763_avg470.71781_std404.16901_diagonal_bw11533.34024_skew3.74169_neigh1.97374_cross0.57584_n14.mtx","synthetic_38120_38120_19284934_avg505.90068_std314.8817_diagonal_bw15240.20312_skew1.89583_neigh1.96535_cross0.86661_n14.mtx","synthetic_38120_38120_20568467_avg539.57154_std333.79776_diagonal_bw14935.54641_skew2.00609_neigh1.96694_cross0.85709_n14.mtx","synthetic_3855_3855_8084357_avg2097.10947_std723.99546_diagonal_bw2875.22931_skew0.83824_neigh1.84384_cross0.81071_n14.mtx","synthetic_39390_39390_15063634_avg382.4228_std245.44522_diagonal_bw17150.32615_skew1.57045_neigh1.97365_cross0.9222_n14.mtx","synthetic_4141_4141_8723864_avg2106.70466_std753.89371_diagonal_bw3055.31055_skew0.96563_neigh1.82243_cross0.79514_n14.mtx","synthetic_41932_41932_16035737_avg382.42242_std245.44481_diagonal_bw18021.62279_skew1.57046_neigh1.97387_cross0.92405_n14.mtx","synthetic_4284_4284_10161733_avg2372.01984_std415.11429_diagonal_bw3157.98039_skew0.80606_neigh1.8259_cross0.87786_n14.mtx","synthetic_4284_4284_10296320_avg2403.43604_std664.88425_diagonal_bw3183.64402_skew0.78245_neigh1.84578_cross0.84547_n14.mtx","synthetic_4284_4284_11128593_avg2597.71078_std612.25995_diagonal_bw3227.32843_skew0.64914_neigh1.86927_cross0.87428_n14.mtx","synthetic_4284_4284_11955257_avg2790.67624_std565.77456_diagonal_bw3299.09944_skew0.53511_neigh1.89073_cross0.89611_n14.mtx","synthetic_4284_4284_12776299_avg2982.32937_std524.4261_diagonal_bw3366.44678_skew0.43646_neigh1.91312_cross0.91453_n14.mtx","synthetic_4284_4284_5176835_avg1208.41153_std875.52905_diagonal_bw3144.46919_skew2.54515_neigh1.74013_cross0.56551_n14.mtx","synthetic_4284_4284_6045664_avg1411.21942_std813.05724_diagonal_bw3116.32236_skew2.03567_neigh1.73991_cross0.63076_n14.mtx","synthetic_4284_4284_6908449_avg1612.61648_std777.77715_diagonal_bw3104.0042_skew1.65655_neigh1.75254_cross0.68691_n14.mtx","synthetic_4284_4284_7606767_avg1775.62255_std826.63023_diagonal_bw3138.01167_skew1.41267_neigh1.77542_cross0.71893_n14.mtx","synthetic_4284_4284_7764407_avg1812.41993_std766.39496_diagonal_bw3107.61204_skew1.36369_neigh1.77486_cross0.73027_n14.mtx","synthetic_4284_4284_7948514_avg1855.39542_std808.46009_diagonal_bw3131.42437_skew1.30894_neigh1.78337_cross0.73476_n14.mtx","synthetic_4284_4284_8278029_avg1932.31303_std796.54977_diagonal_bw3133.04062_skew1.21703_neigh1.79149_cross0.75251_n14.mtx","synthetic_4284_4284_8399652_avg1960.70308_std840.91619_diagonal_bw3170.12582_skew1.18493_neigh1.79818_cross0.75117_n14.mtx","synthetic_4284_4284_8563923_avg1999.04832_std817.99013_diagonal_bw3151.49883_skew1.14302_neigh1.802_cross0.75986_n14.mtx","synthetic_4284_4284_8591948_avg2005.5901_std788.67453_diagonal_bw3136.71662_skew1.13603_neigh1.80156_cross0.76701_n14.mtx","synthetic_4284_4284_8613657_avg2010.65756_std774.7613_diagonal_bw3127.03525_skew1.13065_neigh1.8022_cross0.76595_n14.mtx","synthetic_4284_4284_8716487_avg2034.66083_std804.2751_diagonal_bw3151.36345_skew1.10551_neigh1.80589_cross0.76962_n14.mtx","synthetic_4284_4284_8853517_avg2066.64729_std794.27725_diagonal_bw3141.82166_skew1.07292_neigh1.80997_cross0.77415_n14.mtx","synthetic_4284_4284_8891530_avg2075.52054_std785.22497_diagonal_bw3124.03922_skew1.06406_neigh1.81154_cross0.77892_n14.mtx","synthetic_4284_4284_8977954_avg2095.69421_std780.20568_diagonal_bw3133.23389_skew1.04419_neigh1.81378_cross0.78051_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_diagonal_bw2587.0796_skew1.03117_neigh1.8919_cross0.84164_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_diagonal_bw2710.43301_skew1.03117_neigh1.87148_cross0.83111_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_diagonal_bw2823.22969_skew1.03117_neigh1.85445_cross0.8163_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_diagonal_bw2954.99697_skew1.03117_neigh1.83634_cross0.8091_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_diagonal_bw3067.13329_skew1.03117_neigh1.82242_cross0.79404_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_diagonal_bw3102.39846_skew1.03117_neigh1.81661_cross0.79393_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_diagonal_bw3107.22993_skew1.03117_neigh1.8162_cross0.79153_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_diagonal_bw3110.80742_skew1.03117_neigh1.81584_cross0.79256_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_diagonal_bw3114.21218_skew1.03117_neigh1.99853_cross0.70901_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_diagonal_bw3121.18417_skew1.03117_neigh1.81524_cross0.79023_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_diagonal_bw3126.92857_skew1.03117_neigh1.87269_cross0.76602_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_diagonal_bw3127.45448_skew1.03117_neigh1.91846_cross0.74602_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_diagonal_bw3130.74066_skew1.03117_neigh1.8144_cross0.78625_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_diagonal_bw3132.41083_skew1.03117_neigh1.8143_cross0.78794_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_diagonal_bw3133.53035_skew1.03117_neigh1.83285_cross0.77848_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_diagonal_bw3135.26284_skew1.03117_neigh1.79588_cross0.7941_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_diagonal_bw3135.64356_skew1.03117_neigh1.76094_cross0.80472_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_diagonal_bw3142.22782_skew1.03117_neigh1.81328_cross0.78516_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_diagonal_bw3144.31513_skew1.03117_neigh1.72858_cross0.81697_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_diagonal_bw3149.68371_skew1.03117_neigh1.69844_cross0.82642_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_diagonal_bw3156.4239_skew1.03117_neigh1.81211_cross0.78361_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_diagonal_bw3156.47082_skew1.03117_neigh1.96828_cross0.71812_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_diagonal_bw3162.04692_skew1.03117_neigh1.81086_cross0.78323_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_diagonal_bw3171.51494_skew1.03117_neigh1.66807_cross0.8333_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_diagonal_bw3178.75864_skew1.03117_neigh1.80961_cross0.78012_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_diagonal_bw3198.155_skew1.03117_neigh1.80553_cross0.78061_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_diagonal_bw3209.27381_skew1.03117_neigh1.80497_cross0.77778_n14.mtx","synthetic_4284_4284_9091186_avg2122.12558_std733.44441_diagonal_bw3133.95915_skew1.01873_neigh1.81407_cross0.79058_n14.mtx","synthetic_4284_4284_9176988_avg2142.15406_std713.01815_diagonal_bw3139.19981_skew0.99986_neigh1.8134_cross0.80089_n14.mtx","synthetic_4284_4284_9196527_avg2146.71499_std692.64997_diagonal_bw3137.96078_skew0.99561_neigh1.81239_cross0.80355_n14.mtx","synthetic_4284_4284_9292087_avg2169.02124_std655.40775_diagonal_bw3136.66106_skew0.97508_neigh1.81305_cross0.81195_n14.mtx","synthetic_4284_4284_9379803_avg2189.4965_std621.87428_diagonal_bw3140.22502_skew0.95661_neigh1.81248_cross0.82177_n14.mtx","synthetic_4284_4284_9448827_avg2205.60854_std630.77463_diagonal_bw3151.63002_skew0.94232_neigh1.81415_cross0.82352_n14.mtx","synthetic_4284_4284_9456745_avg2207.45682_std723.4564_diagonal_bw3150.4225_skew0.94069_neigh1.8231_cross0.80811_n14.mtx","synthetic_4284_4284_9460554_avg2208.34594_std591.75846_diagonal_bw3131.92063_skew0.93991_neigh1.81387_cross0.82831_n14.mtx","synthetic_4284_4284_9703417_avg2265.03665_std553.19719_diagonal_bw3154.62862_skew0.89136_neigh1.8169_cross0.84303_n14.mtx","synthetic_4284_4284_9941769_avg2320.67437_std481.87027_diagonal_bw3156.55415_skew0.84602_neigh1.82089_cross0.86146_n14.mtx","synthetic_4426_4426_9354099_avg2113.44306_std776.70231_diagonal_bw3225.23949_skew1.09421_neigh1.80628_cross0.77807_n14.mtx","synthetic_44473_44473_17007427_avg382.4214_std245.44406_diagonal_bw18588.49515_skew1.57046_neigh1.9742_cross0.92197_n14.mtx","synthetic_44686_44686_9904566_avg221.64808_std96.54092_diagonal_bw28996.12507_skew15.40889_neigh1.70385_cross0.85407_n14.mtx","synthetic_46524_46524_7261288_avg156.07618_std357.60767_diagonal_bw23117.62254_skew297.08521_neigh1.84312_cross0.94487_n14.mtx","synthetic_47014_47014_17979151_avg382.42121_std245.44364_diagonal_bw20028.40735_skew1.57046_neigh1.97384_cross0.92279_n14.mtx","synthetic_4712_4712_10000031_avg2122.24767_std807.79622_diagonal_bw3425.88222_skew1.22029_neigh1.78887_cross0.75824_n14.mtx","synthetic_48942_48942_10847397_avg221.6378_std96.4853_diagonal_bw31784.2128_skew15.40966_neigh1.7043_cross0.85189_n14.mtx","synthetic_49556_49556_18951229_avg382.42047_std245.44344_diagonal_bw21059.9188_skew1.57047_neigh1.97406_cross0.92093_n14.mtx","synthetic_4998_4998_10667079_avg2134.26951_std834.36333_diagonal_bw3626.9994_skew1.34178_neigh1.7719_cross0.74002_n14.mtx","synthetic_50954_50954_7955612_avg156.13322_std368.01601_diagonal_bw25406.9614_skew325.34951_neigh1.84494_cross0.94426_n14.mtx","synthetic_5283_5283_11298374_avg2138.62843_std877.77445_diagonal_bw3805.81677_skew1.47027_neigh1.7624_cross0.71965_n14.mtx","synthetic_53198_53198_11791218_avg221.64777_std96.43769_diagonal_bw34602.16726_skew15.40892_neigh1.70332_cross0.85286_n14.mtx","synthetic_55385_55385_8650205_avg156.18317_std377.56925_diagonal_bw27666.03716_skew353.61567_neigh1.84432_cross0.94383_n14.mtx","synthetic_5569_5569_11949345_avg2145.68953_std911.81863_diagonal_bw4008.83264_skew1.59544_neigh1.75126_cross0.70162_n14.mtx","synthetic_57454_57454_12734908_avg221.65398_std96.39664_diagonal_bw37281.76299_skew15.40846_neigh1.70414_cross0.85028_n14.mtx","synthetic_59816_59816_9344868_avg156.2269_std386.43932_diagonal_bw29761.58428_skew381.87902_neigh1.84288_cross0.94411_n14.mtx","synthetic_61710_61710_13678136_avg221.65186_std96.36424_diagonal_bw39991.18778_skew15.40862_neigh1.70436_cross0.85228_n14.mtx","synthetic_63838_63838_10847423_avg169.9211_std96.15794_diagonal_bw40401.27927_skew15.38408_neigh1.69606_cross0.84555_n14.mtx","synthetic_63838_63838_11790856_avg184.69965_std96.20654_diagonal_bw40776.4429_skew15.38877_neigh1.69867_cross0.84918_n14.mtx","synthetic_63838_63838_12734460_avg199.48087_std96.25772_diagonal_bw41109.50902_skew15.39756_neigh1.70139_cross0.84605_n14.mtx","synthetic_63838_63838_13677970_avg214.26063_std96.31751_diagonal_bw41355.45747_skew15.40525_neigh1.70246_cross0.84998_n14.mtx","synthetic_63838_63838_14148710_avg221.63461_std96.12924_diagonal_bw40820.71309_skew11.06039_neigh1.70496_cross0.84517_n14.mtx","synthetic_63838_63838_14148875_avg221.63719_std96.16987_diagonal_bw40978.10773_skew12.02579_neigh1.7049_cross0.84703_n14.mtx","synthetic_63838_63838_14148974_avg221.63874_std125.01008_diagonal_bw40403.16755_skew15.38251_neigh1.70576_cross0.84119_n14.mtx","synthetic_63838_63838_14149076_avg221.64034_std96.21451_diagonal_bw41078.1748_skew12.99114_neigh1.70372_cross0.8494_n14.mtx","synthetic_63838_63838_14149188_avg221.64209_std118.63327_diagonal_bw40496.69658_skew15.38678_neigh1.70525_cross0.84551_n14.mtx","synthetic_63838_63838_14149208_avg221.64241_std96.26387_diagonal_bw41309.92768_skew13.96104_neigh1.7036_cross0.84889_n14.mtx","synthetic_63838_63838_14149296_avg221.64379_std112.26231_diagonal_bw40884.59971_skew15.39568_neigh1.70481_cross0.84573_n14.mtx","synthetic_63838_63838_14149329_avg221.6443_std105.89153_diagonal_bw41036.28118_skew15.40015_neigh1.70463_cross0.8482_n14.mtx","synthetic_63838_63838_14149335_avg221.6444_std96.31796_diagonal_bw41384.37215_skew14.92641_neigh1.70468_cross0.85201_n14.mtx","synthetic_63838_63838_14149338_avg221.64444_std99.52693_diagonal_bw41338.26943_skew15.40465_neigh1.70463_cross0.85088_n14.mtx","synthetic_63838_63838_14149538_avg221.64758_std93.16614_diagonal_bw41616.56296_skew15.40893_neigh1.70332_cross0.8503_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_diagonal_bw31209.32828_skew15.40883_neigh1.70379_cross0.85318_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_diagonal_bw31263.79578_skew15.40883_neigh1.97907_cross0.90114_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_diagonal_bw33310.08077_skew15.40883_neigh1.62891_cross0.97834_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_diagonal_bw33741.33778_skew15.40883_neigh1.70407_cross0.84949_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_diagonal_bw36088.99159_skew15.40883_neigh1.70329_cross0.85235_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_diagonal_bw38238.34086_skew15.40883_neigh1.70401_cross0.8519_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_diagonal_bw38844.09753_skew15.40883_neigh1.68246_cross0.9336_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_diagonal_bw40356.3582_skew15.40883_neigh1.70389_cross0.85225_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_diagonal_bw40916.26257_skew15.40883_neigh1.69941_cross0.87831_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_diagonal_bw41070.66258_skew15.40883_neigh1.19535_cross0.84968_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_diagonal_bw41074.56236_skew15.40883_neigh1.30766_cross0.85115_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_diagonal_bw41141.45558_skew15.40883_neigh1.42075_cross0.85031_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_diagonal_bw41227.36727_skew15.40883_neigh1.53352_cross0.85087_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_diagonal_bw41364.42425_skew15.40883_neigh1.64744_cross0.85051_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_diagonal_bw41755.58307_skew15.40883_neigh1.76114_cross0.85034_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_diagonal_bw41906.82374_skew15.40883_neigh1.70747_cross0.82402_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_diagonal_bw42415.19562_skew15.40883_neigh1.71119_cross0.76651_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_diagonal_bw42495.96779_skew15.40883_neigh1.7039_cross0.85098_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_diagonal_bw42773.9898_skew15.40883_neigh1.8751_cross0.85514_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_diagonal_bw42815.11517_skew15.40883_neigh1.71375_cross0.7124_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_diagonal_bw43078.55498_skew15.40883_neigh1.71506_cross0.65509_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_diagonal_bw43268.40659_skew15.40883_neigh1.71684_cross0.60215_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_diagonal_bw44501.11614_skew15.40883_neigh1.70412_cross0.85016_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_diagonal_bw46019.35751_skew15.40883_neigh1.70471_cross0.84937_n14.mtx","synthetic_63838_63838_14149682_avg221.64983_std96.37657_diagonal_bw41560.95078_skew15.887_neigh1.70358_cross0.84917_n14.mtx","synthetic_63838_63838_14149757_avg221.65101_std96.43499_diagonal_bw41589.99978_skew16.85239_neigh1.70335_cross0.8529_n14.mtx","synthetic_63838_63838_14149836_avg221.65224_std86.81097_diagonal_bw41852.38988_skew15.4131_neigh1.70445_cross0.8526_n14.mtx","synthetic_63838_63838_14149837_avg221.65226_std96.50454_diagonal_bw41795.62602_skew17.81776_neigh1.7041_cross0.85102_n14.mtx","synthetic_63838_63838_14149882_avg221.65297_std80.46101_diagonal_bw42155.13125_skew15.41756_neigh1.70314_cross0.85556_n14.mtx","synthetic_63838_63838_14149914_avg221.65347_std74.11814_diagonal_bw42395.55904_skew15.42203_neigh1.70343_cross0.855_n14.mtx","synthetic_63838_63838_14150121_avg221.65671_std67.78709_diagonal_bw42629.81854_skew15.4263_neigh1.70372_cross0.85703_n14.mtx","synthetic_63838_63838_14150132_avg221.65688_std96.57304_diagonal_bw41766.83688_skew18.78283_neigh1.70227_cross0.85422_n14.mtx","synthetic_63838_63838_14150259_avg221.65887_std96.64741_diagonal_bw41910.74145_skew19.7481_neigh1.70367_cross0.85371_n14.mtx","synthetic_63838_63838_14621335_avg229.03811_std96.37884_diagonal_bw41539.90302_skew15.40775_neigh1.70411_cross0.85184_n14.mtx","synthetic_63838_63838_15564722_avg243.81594_std96.44368_diagonal_bw41752.45562_skew15.41402_neigh1.70623_cross0.85371_n14.mtx","synthetic_63838_63838_16508065_avg258.59308_std96.5168_diagonal_bw41932.57262_skew15.41575_neigh1.70761_cross0.85252_n14.mtx","synthetic_63838_63838_17451479_avg273.37133_std96.59121_diagonal_bw42040.04983_skew15.41723_neigh1.70991_cross0.85482_n14.mtx","synthetic_63838_63838_18395041_avg288.1519_std96.67009_diagonal_bw42202.0742_skew15.42189_neigh1.71077_cross0.85457_n14.mtx","synthetic_63838_63838_9903973_avg155.14228_std96.11613_diagonal_bw40026.20256_skew15.37207_neigh1.69445_cross0.84169_n14.mtx","synthetic_64247_64247_10039260_avg156.26037_std394.7431_diagonal_bw31668.91797_skew410.15351_neigh1.84448_cross0.94465_n14.mtx","synthetic_65965_65965_14620660_avg221.64269_std96.33127_diagonal_bw42883.51398_skew15.40929_neigh1.70416_cross0.8507_n14.mtx","synthetic_66463_66463_10040261_avg151.06542_std395.7133_diagonal_bw32962.39217_skew424.82214_neigh1.844_cross0.94181_n14.mtx","synthetic_66463_66463_10375238_avg156.10547_std374.16577_diagonal_bw32112.74231_skew297.97094_neigh1.84818_cross0.93876_n14.mtx","synthetic_66463_66463_10377678_avg156.14218_std378.88433_diagonal_bw32368.34001_skew326.18897_neigh1.84976_cross0.93964_n14.mtx","synthetic_66463_66463_10378609_avg156.15619_std376.2538_diagonal_bw33885.40087_skew424.61873_neigh1.84267_cross0.94663_n14.mtx","synthetic_66463_66463_10380253_avg156.18093_std381.55088_diagonal_bw33648.55795_skew424.55132_neigh1.84118_cross0.94627_n14.mtx","synthetic_66463_66463_10380322_avg156.18197_std384.04911_diagonal_bw32719.54181_skew354.38034_neigh1.847_cross0.94116_n14.mtx","synthetic_66463_66463_10381996_avg156.20715_std386.69205_diagonal_bw33367.73913_skew424.47988_neigh1.84_cross0.94697_n14.mtx","synthetic_66463_66463_10382686_avg156.21753_std389.6586_diagonal_bw32926.14501_skew382.5741_neigh1.84812_cross0.94145_n14.mtx","synthetic_66463_66463_10383864_avg156.23526_std391.65048_diagonal_bw33368.29388_skew424.40333_neigh1.84441_cross0.94475_n14.mtx","synthetic_66463_66463_10384998_avg156.25232_std492.58539_diagonal_bw31613.53189_skew424.35688_neigh1.85286_cross0.93515_n14.mtx","synthetic_66463_66463_10385329_avg156.2573_std395.72064_diagonal_bw32889.72072_skew410.73756_neigh1.8454_cross0.94344_n14.mtx","synthetic_66463_66463_10385409_avg156.2585_std471.2363_diagonal_bw32083.54764_skew424.34005_neigh1.85336_cross0.93664_n14.mtx","synthetic_66463_66463_10385572_avg156.26096_std396.41268_diagonal_bw33072.52193_skew424.33337_neigh1.84327_cross0.94513_n14.mtx","synthetic_66463_66463_10385645_avg156.26206_std450.123_diagonal_bw32496.96564_skew424.33038_neigh1.85048_cross0.93744_n14.mtx","synthetic_66463_66463_10386025_avg156.26777_std429.29508_diagonal_bw32700.40045_skew424.31482_neigh1.84744_cross0.94073_n14.mtx","synthetic_66463_66463_10386363_avg156.27286_std408.80831_diagonal_bw32967.75383_skew424.30098_neigh1.84617_cross0.94247_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_diagonal_bw24478.41774_skew424.29959_neigh1.8439_cross0.94323_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_diagonal_bw26438.0019_skew424.29959_neigh1.84659_cross0.94266_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_diagonal_bw28604.35611_skew424.29959_neigh1.8441_cross0.94365_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_diagonal_bw29170.51087_skew424.29959_neigh1.30538_cross0.94141_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_diagonal_bw29263.20915_skew424.29959_neigh1.42161_cross0.9415_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_diagonal_bw29455.90226_skew424.29959_neigh1.54434_cross0.94126_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_diagonal_bw29562.86522_skew424.29959_neigh1.66172_cross0.94313_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_diagonal_bw30146.88541_skew424.29959_neigh1.76709_cross0.97938_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_diagonal_bw30341.85309_skew424.29959_neigh1.8421_cross0.94456_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_diagonal_bw30743.21286_skew424.29959_neigh1.7836_cross0.94242_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_diagonal_bw30957.89412_skew424.29959_neigh1.78881_cross0.97358_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_diagonal_bw32012.39481_skew424.29959_neigh1.84485_cross0.94356_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_diagonal_bw33940.03652_skew424.29959_neigh1.8657_cross0.91479_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_diagonal_bw34285.62295_skew424.29959_neigh1.84502_cross0.94387_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_diagonal_bw34484.39386_skew424.29959_neigh1.88647_cross0.85463_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_diagonal_bw34739.67447_skew424.29959_neigh1.89449_cross0.79398_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_diagonal_bw34819.1538_skew424.29959_neigh1.89891_cross0.73344_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_diagonal_bw34936.4192_skew424.29959_neigh1.90192_cross0.6744_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_diagonal_bw35702.1289_skew424.29959_neigh1.84209_cross0.94568_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_diagonal_bw36237.71714_skew424.29959_neigh1.94624_cross0.95035_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_diagonal_bw37525.80485_skew424.29959_neigh1.84482_cross0.94433_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_diagonal_bw39170.09238_skew424.29959_neigh1.84527_cross0.94403_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_diagonal_bw40665.0048_skew424.29959_neigh1.84538_cross0.94433_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_diagonal_bw41811.7269_skew424.29959_neigh1.92884_cross0.94905_n14.mtx","synthetic_66463_66463_10386663_avg156.27737_std388.73672_diagonal_bw33293.45057_skew424.2887_neigh1.84329_cross0.94546_n14.mtx","synthetic_66463_66463_10387014_avg156.28265_std369.1724_diagonal_bw33710.6116_skew424.27433_neigh1.84189_cross0.94604_n14.mtx","synthetic_66463_66463_10387408_avg156.28858_std350.24164_diagonal_bw33963.66553_skew424.25819_neigh1.84411_cross0.94677_n14.mtx","synthetic_66463_66463_10387855_avg156.29531_std332.11193_diagonal_bw34351.45063_skew424.23989_neigh1.8405_cross0.94886_n14.mtx","synthetic_66463_66463_10388533_avg156.30551_std315.01147_diagonal_bw34714.95685_skew424.21214_neigh1.83978_cross0.94968_n14.mtx","synthetic_66463_66463_10730769_avg161.45478_std396.39605_diagonal_bw33389.55323_skew410.65087_neigh1.84718_cross0.94432_n14.mtx","synthetic_66463_66463_11418942_avg171.80901_std391.59435_diagonal_bw33904.10221_skew385.84235_neigh1.8474_cross0.94666_n14.mtx","synthetic_66463_66463_12107419_avg182.16781_std386.59381_diagonal_bw33078.68366_skew363.84492_neigh1.85114_cross0.94649_n14.mtx","synthetic_66463_66463_12795815_avg192.52539_std381.413_diagonal_bw33483.01449_skew344.2168_neigh1.85524_cross0.94648_n14.mtx","synthetic_66463_66463_13484307_avg202.88442_std376.07335_diagonal_bw32893.08663_skew326.59046_neigh1.85894_cross0.94691_n14.mtx","synthetic_66463_66463_7269367_avg109.37464_std374.11902_diagonal_bw34146.31739_skew424.85738_neigh1.83213_cross0.93119_n14.mtx","synthetic_66463_66463_7962072_avg119.79706_std378.84315_diagonal_bw35198.97952_skew424.84518_neigh1.835_cross0.93592_n14.mtx","synthetic_66463_66463_8654839_avg130.22041_std384.01598_diagonal_bw33135.07615_skew424.83187_neigh1.83541_cross0.94038_n14.mtx","synthetic_66463_66463_9347522_avg140.64249_std389.6368_diagonal_bw33861.58836_skew424.82436_neigh1.84023_cross0.94139_n14.mtx","synthetic_68678_68678_10731088_avg156.25219_std397.34109_diagonal_bw34504.32861_skew424.88203_neigh1.844_cross0.94379_n14.mtx","synthetic_70221_70221_15563153_avg221.63104_std96.30657_diagonal_bw45552.93294_skew15.41016_neigh1.70514_cross0.85192_n14.mtx","synthetic_73109_73109_11420815_avg156.21627_std394.47456_diagonal_bw36296.88892_skew424.97997_neigh1.84425_cross0.94399_n14.mtx","synthetic_74477_74477_16505933_avg221.62457_std96.28344_diagonal_bw48331.68664_skew15.41064_neigh1.70415_cross0.85307_n14.mtx","synthetic_77540_77540_12111057_avg156.19109_std391.93861_diagonal_bw38852.76655_skew425.04863_neigh1.84276_cross0.94411_n14.mtx","synthetic_78733_78733_17449768_avg221.6322_std96.26215_diagonal_bw51104.86271_skew15.41007_neigh1.7037_cross0.8514_n14.mtx","synthetic_81971_81971_12801291_avg156.16854_std389.68003_diagonal_bw40877.20774_skew425.11016_neigh1.84172_cross0.94425_n14.mtx","synthetic_82989_82989_18393238_avg221.63465_std96.24293_diagonal_bw53916.13093_skew15.40989_neigh1.70419_cross0.85143_n14.mtx","synthetic_86401_86401_13491346_avg156.14803_std387.65694_diagonal_bw43065.81676_skew425.16612_neigh1.845_cross0.94312_n14.mtx","synthetic_968035_968035_11863180_avg12.25491_std37.64729_diagonal_bw22702.75661_skew632.62368_neigh1.32267_cross0.78592_n14.mtx","synthetic_1382908_1382908_16923784_avg12.23782_std37.37393_diagonal_bw32417.85806_skew443.68692_neigh1.28774_cross0.78284_n14.mtx","synthetic_1382908_1382908_16927023_avg12.24017_std37.40259_diagonal_bw32489.12672_skew485.75812_neigh1.2937_cross0.78368_n14.mtx","synthetic_1382908_1382908_16931896_avg12.24369_std37.43372_diagonal_bw32563.1309_skew527.76219_neigh1.30041_cross0.78501_n14.mtx","synthetic_1382908_1382908_16938002_avg12.2481_std37.46714_diagonal_bw32505.8507_skew569.78219_neigh1.30908_cross0.78582_n14.mtx","synthetic_1382908_1382908_16943948_avg12.2524_std37.50354_diagonal_bw32473.5113_skew611.69607_neigh1.31734_cross0.7865_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_diagonal_bw16934.48122_skew632.66322_neigh1.72397_cross0.78921_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_diagonal_bw26955.89151_skew632.66322_neigh1.63424_cross0.788_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_diagonal_bw28433.29645_skew632.66322_neigh0.52815_cross0.98565_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_diagonal_bw28482.64659_skew632.66322_neigh1.10244_cross0.78637_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_diagonal_bw28699.51453_skew632.66322_neigh0.92649_cross0.7846_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_diagonal_bw29202.3537_skew632.66322_neigh1.32237_cross0.78681_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_diagonal_bw29676.91177_skew632.66322_neigh1.01331_cross0.78531_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_diagonal_bw30053.31884_skew632.66322_neigh1.19029_cross0.78701_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_diagonal_bw30933.71381_skew632.66322_neigh1.36619_cross0.78741_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_diagonal_bw31363.81879_skew632.66322_neigh1.32155_cross0.78695_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_diagonal_bw31759.41838_skew632.66322_neigh1.36147_cross0.55271_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_diagonal_bw31850.6332_skew632.66322_neigh1.35679_cross0.60476_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_diagonal_bw31986.10955_skew632.66322_neigh1.35083_cross0.65686_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_diagonal_bw32172.68164_skew632.66322_neigh1.34169_cross0.70926_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_diagonal_bw32389.12793_skew632.66322_neigh1.32961_cross0.76125_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_diagonal_bw32518.92977_skew632.66322_neigh1.31159_cross0.81268_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_diagonal_bw32792.31039_skew632.66322_neigh0.99726_cross0.96686_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_diagonal_bw32847.57488_skew632.66322_neigh1.28282_cross0.8647_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_diagonal_bw33206.98578_skew632.66322_neigh1.21973_cross0.91658_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_diagonal_bw33535.67693_skew632.66322_neigh1.32147_cross0.78686_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_diagonal_bw33796.51893_skew632.66322_neigh1.27722_cross0.78689_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_diagonal_bw34964.23666_skew632.66322_neigh1.54388_cross0.78867_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_diagonal_bw35676.67346_skew632.66322_neigh1.32177_cross0.78687_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_diagonal_bw37797.05653_skew632.66322_neigh1.32178_cross0.78697_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_diagonal_bw40050.68177_skew632.66322_neigh1.32136_cross0.78693_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_diagonal_bw41281.79686_skew632.66322_neigh1.45465_cross0.78781_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_diagonal_bw42127.99755_skew632.66322_neigh1.32137_cross0.787_n14.mtx","synthetic_1382908_1382908_16948048_avg12.25537_std37.72372_diagonal_bw31935.00008_skew822.06782_neigh1.35574_cross0.78975_n14.mtx","synthetic_1382908_1382908_16948709_avg12.25585_std37.54211_diagonal_bw32403.26495_skew653.62631_neigh1.32622_cross0.78731_n14.mtx","synthetic_1382908_1382908_16951601_avg12.25794_std37.67423_diagonal_bw32052.58183_skew779.80014_neigh1.34903_cross0.78953_n14.mtx","synthetic_1382908_1382908_16951652_avg12.25798_std37.58358_diagonal_bw32324.44939_skew695.60771_neigh1.33364_cross0.78844_n14.mtx","synthetic_1382908_1382908_16952713_avg12.25874_std37.62729_diagonal_bw32145.64971_skew737.65652_neigh1.34221_cross0.78918_n14.mtx","synthetic_187667_187667_12936634_avg68.93398_std101.26087_diagonal_bw21604.74821_skew10.08307_neigh1.12835_cross0.83707_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_diagonal_bw37687.1288_skew10.11157_neigh1.12382_cross0.83773_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_diagonal_bw39604.98722_skew10.11157_neigh1.12032_cross0.83705_n14.mtx","synthetic_268096_268096_18483826_avg68.9448_std96.40124_diagonal_bw30839.36433_skew7.10794_neigh1.1166_cross0.85528_n14.mtx","synthetic_1000005_1000005_2190550_avg2.19054_std25.45314_diagonal_bw114108.9876_skew1501.82644_neigh0.08498_cross0.84657_n14.mtx","synthetic_1000005_1000005_2378888_avg2.37888_std25.47359_diagonal_bw121468.2977_skew1514.4215_neigh0.10196_cross0.883_n14.mtx","synthetic_1000005_1000005_2551748_avg2.55174_std25.499_diagonal_bw114016.12502_skew1534.81764_neigh0.10873_cross0.87763_n14.mtx","synthetic_1000005_1000005_2793946_avg2.79393_std25.52481_diagonal_bw131905.46734_skew1513.71115_neigh0.10444_cross0.86052_n14.mtx","synthetic_1000005_1000005_3068666_avg3.06865_std25.54963_diagonal_bw112236.356_skew1480.43289_neigh0.12104_cross0.88326_n14.mtx","synthetic_1000005_1000005_3110917_avg3.1109_std33.11754_diagonal_bw99822.95915_skew1510.459_neigh0.11796_cross0.85632_n14.mtx","synthetic_1000005_1000005_3116965_avg3.11695_std31.43703_diagonal_bw103495.94606_skew1507.52625_neigh0.11943_cross0.86424_n14.mtx","synthetic_1000005_1000005_3121903_avg3.12189_std25.45302_diagonal_bw106120.25943_skew1053.81063_neigh0.11898_cross0.86904_n14.mtx","synthetic_1000005_1000005_3127972_avg3.12796_std29.75772_diagonal_bw108092.68293_skew1502.5376_neigh0.12362_cross0.87298_n14.mtx","synthetic_1000005_1000005_3132968_avg3.13295_std25.47429_diagonal_bw109696.38688_skew1150.31021_neigh0.12322_cross0.87652_n14.mtx","synthetic_1000005_1000005_3136144_avg3.13613_std18.05255_diagonal_bw131504.88603_skew1498.61976_neigh0.20987_cross0.9344_n14.mtx","synthetic_1000005_1000005_3144021_avg3.14401_std28.07823_diagonal_bw112779.06701_skew1494.86263_neigh0.12564_cross0.88411_n14.mtx","synthetic_1000005_1000005_3144795_avg3.14478_std25.49712_diagonal_bw113241.01438_skew1245.51038_neigh0.12721_cross0.885_n14.mtx","synthetic_1000005_1000005_3156371_avg3.15636_std25.52223_diagonal_bw116391.07562_skew1340.10381_neigh0.13204_cross0.89292_n14.mtx","synthetic_1000005_1000005_3160475_avg3.16046_std19.71515_diagonal_bw131174.03454_skew1487.0749_neigh0.20681_cross0.93372_n14.mtx","synthetic_1000005_1000005_3162276_avg3.16226_std26.40103_diagonal_bw118193.42178_skew1486.2274_neigh0.13377_cross0.89745_n14.mtx","synthetic_1000005_1000005_3166512_avg3.1665_std25.54915_diagonal_bw119317.10886_skew1434.65625_neigh0.13844_cross0.90081_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_diagonal_bw100875.1064_skew1482.18441_neigh0.14415_cross0.90421_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_diagonal_bw108834.1731_skew1482.18441_neigh0.14336_cross0.90436_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_diagonal_bw116472.78808_skew1482.18441_neigh0.14374_cross0.90463_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_diagonal_bw117096.97406_skew1482.18441_neigh0.06224_cross0.96168_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_diagonal_bw117677.40758_skew1482.18441_neigh0.18675_cross0.90502_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_diagonal_bw118297.28426_skew1482.18441_neigh0.17603_cross0.90478_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_diagonal_bw118732.2225_skew1482.18441_neigh0.11799_cross0.93285_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_diagonal_bw118876.87938_skew1482.18441_neigh0.164_cross0.9047_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_diagonal_bw119416.85487_skew1482.18441_neigh0.1575_cross0.90469_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_diagonal_bw119981.2095_skew1482.18441_neigh0.14756_cross0.90455_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_diagonal_bw120655.04241_skew1482.18441_neigh0.13912_cross0.90438_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_diagonal_bw121157.45525_skew1482.18441_neigh0.12935_cross0.90421_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_diagonal_bw121505.44901_skew1482.18441_neigh0.15658_cross0.87562_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_diagonal_bw121800.16162_skew1482.18441_neigh0.1198_cross0.90401_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_diagonal_bw122167.3375_skew1482.18441_neigh0.10989_cross0.90394_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_diagonal_bw122547.34699_skew1482.18441_neigh0.1007_cross0.90373_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_diagonal_bw122970.89533_skew1482.18441_neigh0.17404_cross0.8183_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_diagonal_bw123852.15593_skew1482.18441_neigh0.18429_cross0.76047_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_diagonal_bw124316.39867_skew1482.18441_neigh0.14315_cross0.90464_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_diagonal_bw124333.68929_skew1482.18441_neigh0.19183_cross0.70272_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_diagonal_bw124629.28556_skew1482.18441_neigh0.19573_cross0.64408_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_diagonal_bw132643.35795_skew1482.18441_neigh0.14112_cross0.90484_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_diagonal_bw139742.14787_skew1482.18441_neigh0.14113_cross0.9048_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_diagonal_bw147748.47168_skew1482.18441_neigh0.14113_cross0.90477_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_diagonal_bw154767.97818_skew1482.18441_neigh0.14355_cross0.90514_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_diagonal_bw85508.89004_skew1482.18441_neigh0.14435_cross0.90404_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_diagonal_bw92882.01052_skew1482.18441_neigh0.14352_cross0.90404_n14.mtx","synthetic_1000005_1000005_3174621_avg3.17461_std25.57736_diagonal_bw121783.7238_skew1529.58406_neigh0.14525_cross0.90761_n14.mtx","synthetic_1000005_1000005_3177914_avg3.1779_std24.72495_diagonal_bw122735.04959_skew1478.909_neigh0.15049_cross0.91097_n14.mtx","synthetic_1000005_1000005_3179023_avg3.17901_std21.38191_diagonal_bw130097.76418_skew1478.39273_neigh0.19486_cross0.93033_n14.mtx","synthetic_1000005_1000005_3180578_avg3.18056_std25.60874_diagonal_bw123725.69417_skew1625.12766_neigh0.15445_cross0.91356_n14.mtx","synthetic_1000005_1000005_3184160_avg3.18414_std25.64113_diagonal_bw125738.34862_skew1721.91199_neigh0.16506_cross0.91841_n14.mtx","synthetic_1000005_1000005_3185020_avg3.185_std23.052_diagonal_bw127168.47608_skew1475.60722_neigh0.17295_cross0.92261_n14.mtx","synthetic_1000005_1000005_3185310_avg3.18529_std25.71304_diagonal_bw128318.0595_skew1917.81812_neigh0.18146_cross0.92577_n14.mtx","synthetic_1000005_1000005_3185554_avg3.18554_std25.67645_diagonal_bw127162.91658_skew1819.4146_neigh0.17343_cross0.92255_n14.mtx","synthetic_1000005_1000005_3238430_avg3.23841_std25.57761_diagonal_bw124163.77369_skew1499.42591_neigh0.16758_cross0.91813_n14.mtx","synthetic_1000005_1000005_3338044_avg3.33803_std25.60993_diagonal_bw115730.26217_skew1548.7177_neigh0.16947_cross0.91526_n14.mtx","synthetic_1000005_1000005_3554394_avg3.55438_std25.64567_diagonal_bw109077.20175_skew1542.73078_neigh0.13309_cross0.88625_n14.mtx","synthetic_1000005_1000005_3911756_avg3.91174_std25.67945_diagonal_bw128127.29214_skew1481.71748_neigh0.14126_cross0.8869_n14.mtx","synthetic_1000005_1000005_4154204_avg4.15418_std25.71363_diagonal_bw152102.90185_skew1470.76946_neigh0.19404_cross0.9225_n14.mtx","synthetic_1006441_1006441_22414673_avg22.27122_std29.74727_diagonal_bw241757.44752_skew313.62123_neigh1.25654_cross0.76073_n14.mtx","synthetic_100993_100993_2185846_avg21.64354_std13.77891_diagonal_bw47261.74273_skew4.03614_neigh1.00741_cross0.57221_n14.mtx","synthetic_1019593_1019593_74066178_avg72.64289_std15.80935_diagonal_bw15214.38873_skew1.03736_neigh1.69545_cross0.82041_n14.mtx","synthetic_102778_102778_7408141_avg72.07905_std19.01282_diagonal_bw5790.43171_skew1.19204_neigh1.65271_cross0.77107_n14.mtx","synthetic_1033338_1033338_3275830_avg3.17014_std25.55637_diagonal_bw124512.44793_skew1482.52894_neigh0.14371_cross0.90431_n14.mtx","synthetic_1047423_1047423_51169710_avg48.85296_std11.94333_diagonal_bw179790.95101_skew1.10837_neigh1.69181_cross0.80579_n14.mtx","synthetic_1063952_1063952_23695040_avg22.27078_std29.72463_diagonal_bw255536.35333_skew313.62752_neigh1.25753_cross0.76084_n14.mtx","synthetic_108003_108003_5987303_avg55.43645_std11.04356_diagonal_bw4191.65781_skew0.89406_neigh1.63079_cross0.7877_n14.mtx","synthetic_108334_108334_7808284_avg72.07602_std19.0204_diagonal_bw6122.43647_skew1.19213_neigh1.65216_cross0.76954_n14.mtx","synthetic_1085373_1085373_78800672_avg72.60239_std15.80629_diagonal_bw16200.45996_skew1.02473_neigh1.69539_cross0.82034_n14.mtx","synthetic_109072_109072_2360564_avg21.64225_std13.77936_diagonal_bw51027.24323_skew4.08265_neigh1.00811_cross0.57094_n14.mtx","synthetic_1100005_1100005_3487516_avg3.17045_std25.54298_diagonal_bw132402.99662_skew1482.38345_neigh0.14166_cross0.90462_n14.mtx","synthetic_1110903_1110903_54291613_avg48.87161_std11.93583_diagonal_bw190510.69059_skew1.10756_neigh1.69155_cross0.80595_n14.mtx","synthetic_1121463_1121463_24976074_avg22.27097_std29.7046_diagonal_bw269393.05866_skew313.62476_neigh1.2569_cross0.76099_n14.mtx","synthetic_1151153_1151153_83612943_avg72.63408_std15.81129_diagonal_bw17174.81048_skew1.01008_neigh1.69486_cross0.82103_n14.mtx","synthetic_1166672_1166672_3699263_avg3.17078_std25.53274_diagonal_bw140872.67348_skew1482.23015_neigh0.13992_cross0.90484_n14.mtx","synthetic_117152_117152_2535420_avg21.64214_std13.77711_diagonal_bw54858.54236_skew4.08268_neigh1.00527_cross0.57159_n14.mtx","synthetic_117394_117394_6507908_avg55.43646_std11.04171_diagonal_bw4548.03296_skew0.89406_neigh1.63141_cross0.78802_n14.mtx","synthetic_1174383_1174383_57377479_avg48.85755_std11.94355_diagonal_bw201337.3527_skew1.10817_neigh1.69149_cross0.80605_n14.mtx","synthetic_119698_119698_670830_avg5.60435_std4.47718_diagonal_bw35851.2119_skew62.8789_neigh0.59319_cross0.62674_n14.mtx","synthetic_121192_121192_1836052_avg15.14994_std12.36381_diagonal_bw54781.70924_skew3.8185_neigh0.97072_cross0.58007_n14.mtx","synthetic_121192_121192_2010928_avg16.59291_std12.82631_diagonal_bw54606.41729_skew3.94187_neigh0.98248_cross0.57348_n14.mtx","synthetic_121192_121192_2185768_avg18.03558_std13.23353_diagonal_bw55828.6145_skew4.04558_neigh0.98987_cross0.56777_n14.mtx","synthetic_121192_121192_2360629_avg19.47842_std13.60481_diagonal_bw55574.55169_skew4.13389_neigh0.99709_cross0.56533_n14.mtx","synthetic_121192_121192_2535413_avg20.92063_std13.77469_diagonal_bw56884.2582_skew4.11457_neigh1.00373_cross0.57077_n14.mtx","synthetic_121192_121192_2622629_avg21.64028_std13.77138_diagonal_bw56242.87207_skew3.85206_neigh1.00809_cross0.56863_n14.mtx","synthetic_121192_121192_2622663_avg21.64056_std13.63311_diagonal_bw55697.66942_skew3.52853_neigh1.00738_cross0.56431_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_diagonal_bw41449.60518_skew4.08299_neigh1.00709_cross0.56924_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_diagonal_bw44969.04646_skew4.08299_neigh1.00918_cross0.5684_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_diagonal_bw48375.95076_skew4.08299_neigh1.0066_cross0.57186_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_diagonal_bw51808.58859_skew4.08299_neigh1.00555_cross0.57153_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_diagonal_bw52786.48945_skew4.08299_neigh0.98178_cross0.73314_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_diagonal_bw53882.38648_skew4.08299_neigh0.98833_cross0.69862_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_diagonal_bw54816.63871_skew4.08299_neigh0.99503_cross0.66136_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_diagonal_bw55046.60667_skew4.08299_neigh1.00618_cross0.57223_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_diagonal_bw55588.08012_skew4.08299_neigh1.00197_cross0.62866_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_diagonal_bw56289.99224_skew4.08299_neigh0.77156_cross0.57111_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_diagonal_bw56335.34997_skew4.08299_neigh1.00541_cross0.59144_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_diagonal_bw56445.73291_skew4.08299_neigh0.83909_cross0.57135_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_diagonal_bw56550.54337_skew4.08299_neigh0.70497_cross0.57228_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_diagonal_bw56640.70898_skew4.08299_neigh1.10568_cross0.57621_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_diagonal_bw56646.25647_skew4.08299_neigh0.90696_cross0.57222_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_diagonal_bw56981.50968_skew4.08299_neigh1.00923_cross0.55476_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_diagonal_bw57143.44331_skew4.08299_neigh1.1732_cross0.57642_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_diagonal_bw57206.83955_skew4.08299_neigh0.97383_cross0.57171_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_diagonal_bw57392.3198_skew4.08299_neigh1.03836_cross0.57342_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_diagonal_bw57680.27683_skew4.08299_neigh1.01286_cross0.52046_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_diagonal_bw57736.54601_skew4.08299_neigh1.2401_cross0.57851_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_diagonal_bw58118.42279_skew4.08299_neigh1.01366_cross0.48629_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_diagonal_bw58305.84042_skew4.08299_neigh1.00621_cross0.5718_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_diagonal_bw58608.29432_skew4.08299_neigh1.01533_cross0.452_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_diagonal_bw59091.75387_skew4.08299_neigh1.01816_cross0.41458_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_diagonal_bw59170.98197_skew4.08299_neigh1.30576_cross0.57913_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_diagonal_bw61351.56958_skew4.08299_neigh1.00459_cross0.57351_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_diagonal_bw64498.77718_skew4.08299_neigh1.00458_cross0.57593_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_diagonal_bw67553.05734_skew4.08299_neigh1.00396_cross0.57603_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_diagonal_bw70322.56861_skew4.08299_neigh1.00409_cross0.57935_n14.mtx","synthetic_121192_121192_2622724_avg21.64107_std13.77599_diagonal_bw56586.74825_skew3.99051_neigh1.00718_cross0.57135_n14.mtx","synthetic_121192_121192_2622736_avg21.64116_std14.23425_diagonal_bw56154.62111_skew4.08291_neigh1.00762_cross0.56767_n14.mtx","synthetic_121192_121192_2622750_avg21.64128_std13.768_diagonal_bw55889.48449_skew3.71321_neigh1.00743_cross0.56293_n14.mtx","synthetic_121192_121192_2622777_avg21.6415_std15.59481_diagonal_bw55368.29813_skew4.08283_neigh1.00631_cross0.56573_n14.mtx","synthetic_121192_121192_2622792_avg21.64163_std13.77856_diagonal_bw56873.57123_skew4.129_neigh1.0086_cross0.57384_n14.mtx","synthetic_121192_121192_2622812_avg21.64179_std16.1555_diagonal_bw55349.84176_skew3.99034_neigh1.00627_cross0.56748_n14.mtx","synthetic_121192_121192_2622828_avg21.64192_std13.43338_diagonal_bw55856.38154_skew3.29722_neigh1.00759_cross0.56535_n14.mtx","synthetic_121192_121192_2622837_avg21.642_std15.01875_diagonal_bw55585.0516_skew4.12892_neigh1.00673_cross0.56331_n14.mtx","synthetic_121192_121192_2622876_avg21.64232_std16.69445_diagonal_bw55240.42113_skew3.94402_neigh1.00765_cross0.5674_n14.mtx","synthetic_121192_121192_2622972_avg21.64311_std13.78148_diagonal_bw57067.76991_skew4.31347_neigh1.00707_cross0.57783_n14.mtx","synthetic_121192_121192_2623047_avg21.64373_std13.32114_diagonal_bw57121.20154_skew4.0361_neigh1.00729_cross0.57944_n14.mtx","synthetic_121192_121192_2623109_avg21.64424_std13.78493_diagonal_bw57338.33986_skew4.4518_neigh1.00545_cross0.58243_n14.mtx","synthetic_121192_121192_2623156_avg21.64463_std11.48856_diagonal_bw58934.13194_skew3.89729_neigh1.00538_cross0.59657_n14.mtx","synthetic_121192_121192_2623189_avg21.6449_std13.78718_diagonal_bw57991.06353_skew4.77503_neigh1.00635_cross0.58424_n14.mtx","synthetic_121192_121192_2623197_avg21.64497_std13.78635_diagonal_bw57626.49167_skew4.59021_neigh1.00568_cross0.58329_n14.mtx","synthetic_121192_121192_2623249_avg21.6454_std12.40537_diagonal_bw57996.48538_skew3.98951_neigh1.0059_cross0.58819_n14.mtx","synthetic_121192_121192_2623583_avg21.64815_std9.65791_diagonal_bw60405.36362_skew3.8503_neigh1.00451_cross0.61091_n14.mtx","synthetic_121192_121192_2623613_avg21.6484_std10.57459_diagonal_bw59660.52937_skew3.85024_neigh1.00452_cross0.60326_n14.mtx","synthetic_121192_121192_2710314_avg22.3638_std13.78076_diagonal_bw57516.41338_skew4.05281_neigh1.00876_cross0.57867_n14.mtx","synthetic_121192_121192_2885490_avg23.80924_std13.78209_diagonal_bw57922.69442_skew3.99806_neigh1.01487_cross0.58481_n14.mtx","synthetic_121192_121192_3060321_avg25.25184_std13.78548_diagonal_bw58954.02529_skew3.95013_neigh1.0179_cross0.59132_n14.mtx","synthetic_121192_121192_3235599_avg26.69812_std13.78831_diagonal_bw59260.45423_skew3.86926_neigh1.02285_cross0.59628_n14.mtx","synthetic_121192_121192_3410529_avg28.14154_std13.78985_diagonal_bw59475.58642_skew3.86825_neigh1.02443_cross0.59913_n14.mtx","synthetic_1216933_1216933_88363297_avg72.61147_std15.79644_diagonal_bw18153.04703_skew1.03825_neigh1.69455_cross0.82106_n14.mtx","synthetic_1233339_1233339_3909668_avg3.16999_std25.52165_diagonal_bw148870.18881_skew1482.60253_neigh0.14197_cross0.90505_n14.mtx","synthetic_1237863_1237863_60496296_avg48.87156_std11.94811_diagonal_bw211983.92253_skew1.20987_neigh1.69135_cross0.80663_n14.mtx","synthetic_125231_125231_2710355_avg21.64284_std13.77742_diagonal_bw58546.37263_skew4.08251_neigh1.00665_cross0.57313_n14.mtx","synthetic_126786_126786_7028939_avg55.43939_std11.04404_diagonal_bw4906.49056_skew0.89396_neigh1.63134_cross0.78947_n14.mtx","synthetic_1282713_1282713_93191153_avg72.6516_std15.81312_diagonal_bw19135.30286_skew1.10594_neigh1.69446_cross0.821_n14.mtx","synthetic_129947_129947_10504776_avg80.83893_std117.70867_diagonal_bw16837.76377_skew7.85712_neigh1.21519_cross0.8459_n14.mtx","synthetic_1300006_1300006_4120758_avg3.1698_std25.51264_diagonal_bw157091.52358_skew1482.69019_neigh0.13981_cross0.90518_n14.mtx","synthetic_131098_131098_734679_avg5.60404_std4.47302_diagonal_bw39029.11606_skew62.88244_neigh0.59297_cross0.62541_n14.mtx","synthetic_133311_133311_2884795_avg21.63959_std13.78017_diagonal_bw62354.80178_skew4.08328_neigh1.00679_cross0.57368_n14.mtx","synthetic_136178_136178_7548659_avg55.43229_std11.05483_diagonal_bw5281.69958_skew0.91224_neigh1.63102_cross0.78738_n14.mtx","synthetic_140874_140874_10153211_avg72.07299_std11.05163_diagonal_bw5512.27589_skew0.7066_neigh1.64608_cross0.80473_n14.mtx","synthetic_140874_140874_5465261_avg38.79538_std11.0515_diagonal_bw5412.9335_skew1.31986_neigh1.60389_cross0.75869_n14.mtx","synthetic_140874_140874_5986164_avg42.49304_std11.0536_diagonal_bw5548.31213_skew1.18859_neigh1.61095_cross0.76635_n14.mtx","synthetic_140874_140874_6506839_avg46.18907_std11.05195_diagonal_bw5412.39753_skew1.10006_neigh1.61828_cross0.77328_n14.mtx","synthetic_140874_140874_7027827_avg49.88732_std11.05353_diagonal_bw5512.6976_skew1.02456_neigh1.62363_cross0.78108_n14.mtx","synthetic_140874_140874_7548897_avg53.58616_std11.05321_diagonal_bw5415.30582_skew0.9408_neigh1.62912_cross0.7833_n14.mtx","synthetic_140874_140874_7807971_avg55.42521_std14.36638_diagonal_bw5335.16197_skew1.18312_neigh1.63649_cross0.76697_n14.mtx","synthetic_140874_140874_7808175_avg55.42666_std13.63005_diagonal_bw5359.29341_skew1.12894_neigh1.63441_cross0.77184_n14.mtx","synthetic_140874_140874_7808374_avg55.42807_std12.89549_diagonal_bw5396.31618_skew1.07476_neigh1.63344_cross0.77504_n14.mtx","synthetic_140874_140874_7808713_avg55.43048_std12.15675_diagonal_bw5408.74805_skew1.00251_neigh1.63324_cross0.78138_n14.mtx","synthetic_140874_140874_7809044_avg55.43283_std11.42187_diagonal_bw5439.57739_skew0.9483_neigh1.6324_cross0.78435_n14.mtx","synthetic_140874_140874_7809186_avg55.43383_std10.68354_diagonal_bw5480.44304_skew0.89415_neigh1.62983_cross0.78974_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_diagonal_bw3838.69228_skew0.91217_neigh1.63011_cross0.78907_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_diagonal_bw4204.41167_skew0.91217_neigh1.62943_cross0.78923_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_diagonal_bw4558.67931_skew0.91217_neigh1.63021_cross0.78941_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_diagonal_bw4913.35727_skew0.91217_neigh1.63146_cross0.78927_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_diagonal_bw5282.63594_skew0.91217_neigh1.63072_cross0.78718_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_diagonal_bw5457.86719_skew0.91217_neigh1.63077_cross0.78851_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_diagonal_bw5648.31754_skew0.91217_neigh1.62967_cross0.78702_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_diagonal_bw6002.23969_skew0.91217_neigh1.63068_cross0.78728_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_diagonal_bw6356.58993_skew0.91217_neigh1.62903_cross0.78853_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_diagonal_bw6710.26111_skew0.91217_neigh1.62896_cross0.78828_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_diagonal_bw7054.0657_skew0.91217_neigh1.63058_cross0.78783_n14.mtx","synthetic_140874_140874_7809480_avg55.43592_std9.94668_diagonal_bw5490.06378_skew0.82192_neigh1.63009_cross0.79615_n14.mtx","synthetic_140874_140874_7809970_avg55.4394_std9.2113_diagonal_bw5566.8335_skew0.7677_neigh1.62757_cross0.7999_n14.mtx","synthetic_140874_140874_7810161_avg55.44076_std8.47606_diagonal_bw5572.74971_skew0.6955_neigh1.62616_cross0.80687_n14.mtx","synthetic_140874_140874_7810358_avg55.44215_std7.74028_diagonal_bw5620.76373_skew0.64135_neigh1.62766_cross0.81104_n14.mtx","synthetic_140874_140874_8069568_avg57.28217_std11.05207_diagonal_bw5516.71932_skew0.8854_neigh1.63387_cross0.78936_n14.mtx","synthetic_140874_140874_8590481_avg60.97989_std11.05315_diagonal_bw5437.17752_skew0.83667_neigh1.63703_cross0.79384_n14.mtx","synthetic_140874_140874_9111507_avg64.67841_std11.05222_diagonal_bw5508.99952_skew0.77803_neigh1.6398_cross0.79875_n14.mtx","synthetic_140874_140874_9632262_avg68.37502_std11.0533_diagonal_bw5447.32524_skew0.7404_neigh1.64497_cross0.80063_n14.mtx","synthetic_141390_141390_3059797_avg21.64083_std13.77882_diagonal_bw66168.99875_skew4.08298_neigh1.00521_cross0.57194_n14.mtx","synthetic_142323_142323_11504673_avg80.83495_std117.71151_diagonal_bw18462.18834_skew7.85755_neigh1.21247_cross0.84559_n14.mtx","synthetic_142498_142498_798541_avg5.60388_std4.46852_diagonal_bw42535.5183_skew62.88436_neigh0.59531_cross0.62546_n14.mtx","synthetic_144550_144550_890999_avg6.16395_std4.44624_diagonal_bw249.77002_skew7.27392_neigh0.18253_cross0.33392_n14.mtx","synthetic_145569_145569_8068734_avg55.42893_std11.05658_diagonal_bw5649.76329_skew0.91236_neigh1.62963_cross0.7868_n14.mtx","synthetic_14839_14839_1040332_avg70.10796_std6.31579_diagonal_bw1132.91226_skew0.34079_neigh1.79083_cross0.91451_n14.mtx","synthetic_149470_149470_3234141_avg21.63739_std13.78086_diagonal_bw69927.19965_skew4.08379_neigh1.00664_cross0.57176_n14.mtx","synthetic_153898_153898_862342_avg5.60333_std4.46589_diagonal_bw45848.56757_skew62.89053_neigh0.59749_cross0.62567_n14.mtx","synthetic_154699_154699_12503850_avg80.82696_std117.71492_diagonal_bw20044.28137_skew7.85843_neigh1.21606_cross0.84532_n14.mtx","synthetic_154961_154961_8588351_avg55.42266_std11.06218_diagonal_bw6007.51607_skew0.91258_neigh1.63019_cross0.78673_n14.mtx","synthetic_157549_157549_3408821_avg21.63658_std13.78244_diagonal_bw73653.17583_skew4.08398_neigh1.00489_cross0.5734_n14.mtx","synthetic_158316_158316_975794_avg6.16358_std4.44586_diagonal_bw273.64732_skew7.27441_neigh0.1798_cross0.33007_n14.mtx","synthetic_16253_16253_1139653_avg70.11955_std6.30855_diagonal_bw1264.2658_skew0.34057_neigh1.78351_cross0.91717_n14.mtx","synthetic_164353_164353_9109408_avg55.42587_std11.0688_diagonal_bw6365.94759_skew0.91246_neigh1.6294_cross0.78797_n14.mtx","synthetic_165298_165298_926272_avg5.60365_std4.4624_diagonal_bw49243.13259_skew62.88694_neigh0.59497_cross0.62573_n14.mtx","synthetic_167075_167075_13504757_avg80.83051_std117.71597_diagonal_bw21622.74045_skew7.87041_neigh1.21592_cross0.84505_n14.mtx","synthetic_170998_170998_1008507_avg5.89777_std3.14995_diagonal_bw45171.22177_skew59.87046_neigh0.62353_cross0.6314_n14.mtx","synthetic_170998_170998_1037773_avg6.06892_std4.43265_diagonal_bw45901.38609_skew59.96638_neigh0.64574_cross0.63979_n14.mtx","synthetic_170998_170998_1043193_avg6.10062_std4.44271_diagonal_bw46214.72752_skew63.58365_neigh0.6538_cross0.64274_n14.mtx","synthetic_170998_170998_1092151_avg6.38692_std4.47337_diagonal_bw46053.98994_skew64.44623_neigh0.62763_cross0.62819_n14.mtx","synthetic_170998_170998_1211686_avg7.08597_std4.4649_diagonal_bw49555.83123_skew64.76379_neigh0.67173_cross0.6434_n14.mtx","synthetic_170998_170998_1212281_avg7.08945_std4.45045_diagonal_bw49559.75803_skew61.3462_neigh0.67177_cross0.64357_n14.mtx","synthetic_170998_170998_703429_avg4.11367_std4.4083_diagonal_bw62525.67356_skew60.01611_neigh0.55916_cross0.6516_n14.mtx","synthetic_170998_170998_709969_avg4.15191_std4.42013_diagonal_bw62829.82587_skew65.23451_neigh0.57062_cross0.65616_n14.mtx","synthetic_170998_170998_813246_avg4.75588_std4.44226_diagonal_bw46482.54362_skew61.86954_neigh0.56085_cross0.62458_n14.mtx","synthetic_170998_170998_875832_avg5.12188_std4.42226_diagonal_bw49819.5485_skew62.06273_neigh0.62237_cross0.64457_n14.mtx","synthetic_170998_170998_882159_avg5.15888_std4.44207_diagonal_bw49851.31179_skew66.06876_neigh0.61393_cross0.64338_n14.mtx","synthetic_170998_170998_944721_avg5.52475_std5.76053_diagonal_bw50240.49201_skew63.79933_neigh0.59642_cross0.62742_n14.mtx","synthetic_170998_170998_945632_avg5.53008_std5.47116_diagonal_bw50240.27614_skew63.7369_neigh0.5981_cross0.62739_n14.mtx","synthetic_170998_170998_946661_avg5.53609_std4.44002_diagonal_bw50440.90686_skew44.88072_neigh0.59713_cross0.62583_n14.mtx","synthetic_170998_170998_947518_avg5.54111_std5.18255_diagonal_bw50420.37847_skew63.60804_neigh0.59685_cross0.62681_n14.mtx","synthetic_170998_170998_948659_avg5.54778_std4.44165_diagonal_bw50506.82008_skew48.9299_neigh0.59655_cross0.62663_n14.mtx","synthetic_170998_170998_950710_avg5.55977_std4.8941_diagonal_bw50483.86054_skew63.39112_neigh0.59518_cross0.62663_n14.mtx","synthetic_170998_170998_951053_avg5.56178_std4.44852_diagonal_bw50556.26173_skew53.11938_neigh0.59346_cross0.62678_n14.mtx","synthetic_170998_170998_953667_avg5.57707_std4.45309_diagonal_bw50785.42323_skew57.09507_neigh0.59646_cross0.62658_n14.mtx","synthetic_170998_170998_955297_avg5.5866_std4.60347_diagonal_bw50921.85081_skew63.08194_neigh0.596_cross0.6275_n14.mtx","synthetic_170998_170998_956707_avg5.59484_std4.45769_diagonal_bw51000.98471_skew61.0214_neigh0.59632_cross0.62507_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_diagonal_bw36461.87693_skew62.88573_neigh0.59695_cross0.61997_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_diagonal_bw39610.08149_skew62.88573_neigh0.59906_cross0.62096_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_diagonal_bw43105.47571_skew62.88573_neigh0.59816_cross0.62237_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_diagonal_bw44031.98084_skew62.88573_neigh0.5372_cross0.62197_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_diagonal_bw44847.35264_skew62.88573_neigh0.49711_cross0.62325_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_diagonal_bw45384.2646_skew62.88573_neigh0.4588_cross0.62138_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_diagonal_bw45971.97334_skew62.88573_neigh0.41681_cross0.61967_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_diagonal_bw46159.87112_skew62.88573_neigh0.59674_cross0.62414_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_diagonal_bw46483.06345_skew62.88573_neigh0.77274_cross0.62785_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_diagonal_bw47421.7408_skew62.88573_neigh0.73666_cross0.62676_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_diagonal_bw48584.83352_skew62.88573_neigh0.69732_cross0.62642_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_diagonal_bw49003.35613_skew62.88573_neigh0.54697_cross0.79644_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_diagonal_bw49449.25941_skew62.88573_neigh0.59289_cross0.62664_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_diagonal_bw49562.93434_skew62.88573_neigh0.65726_cross0.6251_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_diagonal_bw49598.09636_skew62.88573_neigh0.56075_cross0.75943_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_diagonal_bw50162.74852_skew62.88573_neigh0.57537_cross0.72046_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_diagonal_bw50491.14134_skew62.88573_neigh0.58432_cross0.6826_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_diagonal_bw50571.15461_skew62.88573_neigh0.61583_cross0.62382_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_diagonal_bw50893.01638_skew62.88573_neigh0.5929_cross0.64621_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_diagonal_bw51129.15176_skew62.88573_neigh0.59642_cross0.60766_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_diagonal_bw51234.36906_skew62.88573_neigh0.60413_cross0.5687_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_diagonal_bw51395.69692_skew62.88573_neigh0.61075_cross0.52993_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_diagonal_bw51420.69302_skew62.88573_neigh0.61218_cross0.49298_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_diagonal_bw51535.92532_skew62.88573_neigh0.57482_cross0.62495_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_diagonal_bw51655.22982_skew62.88573_neigh0.61451_cross0.45451_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_diagonal_bw52368.46746_skew62.88573_neigh0.59522_cross0.62658_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_diagonal_bw55725.08564_skew62.88573_neigh0.59126_cross0.62765_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_diagonal_bw58848.50641_skew62.88573_neigh0.5924_cross0.62814_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_diagonal_bw61752.83746_skew62.88573_neigh0.59197_cross0.62973_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_diagonal_bw64753.49829_skew62.88573_neigh0.59013_cross0.63179_n14.mtx","synthetic_170998_170998_959835_avg5.61314_std4.46266_diagonal_bw51077.39694_skew64.91681_neigh0.59508_cross0.62613_n14.mtx","synthetic_170998_170998_961623_avg5.62359_std4.31537_diagonal_bw51156.06217_skew62.66038_neigh0.59268_cross0.62607_n14.mtx","synthetic_170998_170998_963061_avg5.632_std4.4671_diagonal_bw51143.10828_skew68.77981_neigh0.59568_cross0.62558_n14.mtx","synthetic_170998_170998_966597_avg5.65268_std4.47351_diagonal_bw51344.14215_skew72.59341_neigh0.59618_cross0.6253_n14.mtx","synthetic_170998_170998_970106_avg5.6732_std4.48093_diagonal_bw51465.29233_skew76.55763_neigh0.59896_cross0.62591_n14.mtx","synthetic_170998_170998_970315_avg5.67442_std4.02645_diagonal_bw51483.87979_skew62.09011_neigh0.59847_cross0.62615_n14.mtx","synthetic_170998_170998_973557_avg5.69338_std4.48437_diagonal_bw51658.71521_skew80.32248_neigh0.59938_cross0.6276_n14.mtx","synthetic_170998_170998_981215_avg5.73817_std3.73499_diagonal_bw52085.53194_skew61.38927_neigh0.60333_cross0.62711_n14.mtx","synthetic_170998_170998_994346_avg5.81496_std3.44543_diagonal_bw52702.41379_skew60.73734_neigh0.6117_cross0.63171_n14.mtx","synthetic_172083_172083_1060867_avg6.16486_std4.44424_diagonal_bw297.53188_skew7.2727_neigh0.17686_cross0.32932_n14.mtx","synthetic_173744_173744_9631043_avg55.43238_std11.0756_diagonal_bw6722.25909_skew0.91224_neigh1.62924_cross0.7879_n14.mtx","synthetic_17666_17666_1238955_avg70.13217_std6.31199_diagonal_bw1367.70486_skew0.34033_neigh1.78108_cross0.91647_n14.mtx","synthetic_176697_176697_990167_avg5.60376_std4.45819_diagonal_bw52568.63684_skew62.88571_neigh0.59492_cross0.62595_n14.mtx","synthetic_179451_179451_14506771_avg80.83973_std117.71232_diagonal_bw23249.97453_skew7.8694_neigh1.21265_cross0.84464_n14.mtx","synthetic_183136_183136_10152501_avg55.43695_std11.07295_diagonal_bw7090.3351_skew0.91208_neigh1.631_cross0.78675_n14.mtx","synthetic_185639_185639_10500260_avg56.56279_std99.5812_diagonal_bw18098.88818_skew7.18559_neigh1.25585_cross0.95445_n14.mtx","synthetic_185639_185639_11501046_avg61.95382_std109.06293_diagonal_bw18425.33593_skew7.19966_neigh1.25674_cross0.95406_n14.mtx","synthetic_185639_185639_12507698_avg67.37646_std112.75287_diagonal_bw23036.91435_skew7.4154_neigh1.18654_cross0.88126_n14.mtx","synthetic_185639_185639_13507262_avg72.76091_std115.00904_diagonal_bw23636.41108_skew7.61726_neigh1.19914_cross0.86016_n14.mtx","synthetic_185639_185639_14507140_avg78.14705_std116.8824_diagonal_bw23970.8135_skew7.80392_neigh1.21053_cross0.84821_n14.mtx","synthetic_185639_185639_15003294_avg80.81973_std87.6573_diagonal_bw24360.81189_skew8.39127_neigh1.21918_cross0.82425_n14.mtx","synthetic_185639_185639_15003763_avg80.82226_std142.24862_diagonal_bw19300.06598_skew7.19081_neigh1.26239_cross0.95424_n14.mtx","synthetic_185639_185639_15004156_avg80.82437_std94.60907_diagonal_bw24270.69581_skew8.30412_neigh1.21969_cross0.82786_n14.mtx","synthetic_185639_185639_15004647_avg80.82702_std101.43013_diagonal_bw24276.21957_skew8.20484_neigh1.21772_cross0.83109_n14.mtx","synthetic_185639_185639_15005689_avg80.83263_std108.09033_diagonal_bw24209.38904_skew8.08049_neigh1.21736_cross0.83603_n14.mtx","synthetic_185639_185639_15006431_avg80.83663_std114.55767_diagonal_bw24067.75636_skew7.94397_neigh1.21737_cross0.84232_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_diagonal_bw17089.64436_skew7.8818_neigh1.21539_cross0.84521_n14.mtx","synthetic_185639_185639_15007305_avg80.84134_std120.8084_diagonal_bw23932.70372_skew7.80738_neigh1.21348_cross0.84884_n14.mtx","synthetic_185639_185639_15008187_avg80.84609_std126.81395_diagonal_bw23820.75271_skew7.64606_neigh1.20923_cross0.85805_n14.mtx","synthetic_185639_185639_15008702_avg80.84886_std137.983_diagonal_bw23110.0529_skew7.32417_neigh1.20249_cross0.89205_n14.mtx","synthetic_185639_185639_15008859_avg80.84971_std132.54683_diagonal_bw23516.89351_skew7.48488_neigh1.20404_cross0.87118_n14.mtx","synthetic_185639_185639_15506741_avg83.5317_std118.48272_diagonal_bw24157.25407_skew7.94271_neigh1.21788_cross0.84141_n14.mtx","synthetic_185639_185639_16506181_avg88.91548_std119.88133_diagonal_bw24248.66297_skew8.06479_neigh1.22306_cross0.83613_n14.mtx","synthetic_185639_185639_17505764_avg94.30003_std121.13382_diagonal_bw24308.77245_skew8.16224_neigh1.22615_cross0.83329_n14.mtx","synthetic_185639_185639_18505682_avg99.68639_std122.27517_diagonal_bw24387.0841_skew8.23897_neigh1.22956_cross0.82911_n14.mtx","synthetic_185639_185639_19505424_avg105.0718_std123.33624_diagonal_bw24435.45274_skew8.2984_neigh1.23248_cross0.82657_n14.mtx","synthetic_185850_185850_1145706_avg6.16468_std4.44467_diagonal_bw321.4122_skew7.27293_neigh0.17353_cross0.32676_n14.mtx","synthetic_188097_188097_1054064_avg5.60383_std4.45656_diagonal_bw55856.29168_skew62.88486_neigh0.59686_cross0.62548_n14.mtx","synthetic_19080_19080_1337993_avg70.12542_std6.31221_diagonal_bw1466.79911_skew0.34046_neigh1.78524_cross0.91492_n14.mtx","synthetic_191826_191826_15508397_avg80.84617_std117.70962_diagonal_bw24854.19831_skew7.88106_neigh1.21284_cross0.84477_n14.mtx","synthetic_199497_199497_1118014_avg5.60416_std4.45398_diagonal_bw59520.93639_skew62.88107_neigh0.59561_cross0.62565_n14.mtx","synthetic_199616_199616_1230682_avg6.16525_std4.44318_diagonal_bw345.00688_skew7.27217_neigh0.17135_cross0.32384_n14.mtx","synthetic_204202_204202_16508082_avg80.84192_std117.71063_diagonal_bw26469.08637_skew7.88153_neigh1.21323_cross0.84417_n14.mtx","synthetic_20493_20493_1437302_avg70.13624_std6.31204_diagonal_bw1570.36622_skew0.34025_neigh1.78301_cross0.915_n14.mtx","synthetic_206500_206500_1060353_avg5.13488_std4.44292_diagonal_bw335.07862_skew7.3741_neigh0.16054_cross0.3088_n14.mtx","synthetic_206500_206500_1145534_avg5.54738_std4.44341_diagonal_bw356.74644_skew7.2922_neigh0.16406_cross0.31454_n14.mtx","synthetic_206500_206500_1230425_avg5.95847_std4.44436_diagonal_bw374.57129_skew7.22358_neigh0.16743_cross0.31868_n14.mtx","synthetic_206500_206500_1272393_avg6.16171_std5.76962_diagonal_bw330.69652_skew7.43922_neigh0.17725_cross0.30587_n14.mtx","synthetic_206500_206500_1272581_avg6.16262_std4.44309_diagonal_bw352.66015_skew6.30209_neigh0.16953_cross0.32032_n14.mtx","synthetic_206500_206500_1272606_avg6.16274_std4.44382_diagonal_bw349.99388_skew5.97741_neigh0.16985_cross0.31715_n14.mtx","synthetic_206500_206500_1272648_avg6.16294_std5.47591_diagonal_bw338.10897_skew7.43753_neigh0.17526_cross0.31073_n14.mtx","synthetic_206500_206500_1272653_avg6.16297_std5.1813_diagonal_bw344.20462_skew7.27523_neigh0.17301_cross0.31335_n14.mtx","synthetic_206500_206500_1272686_avg6.16313_std4.88791_diagonal_bw350.18601_skew7.27502_neigh0.1719_cross0.3183_n14.mtx","synthetic_206500_206500_1272775_avg6.16356_std4.44367_diagonal_bw347.38147_skew5.48976_neigh0.17032_cross0.31578_n14.mtx","synthetic_206500_206500_1272904_avg6.16418_std4.4462_diagonal_bw361.59791_skew8.73365_neigh0.17074_cross0.32864_n14.mtx","synthetic_206500_206500_1272958_avg6.16445_std4.59126_diagonal_bw354.70395_skew7.27325_neigh0.171_cross0.32279_n14.mtx","synthetic_206500_206500_1272981_avg6.16456_std4.44526_diagonal_bw362.3522_skew9.05749_neigh0.17062_cross0.32883_n14.mtx","synthetic_206500_206500_1272983_avg6.16457_std4.4441_diagonal_bw354.25261_skew6.62422_neigh0.17027_cross0.32275_n14.mtx","synthetic_206500_206500_1272990_avg6.1646_std4.44515_diagonal_bw357.75317_skew7.43526_neigh0.17014_cross0.3244_n14.mtx","synthetic_206500_206500_1272993_avg6.16462_std4.44589_diagonal_bw358.92702_skew7.92189_neigh0.17019_cross0.32662_n14.mtx","synthetic_206500_206500_1272999_avg6.16464_std4.44572_diagonal_bw360.42425_skew8.24628_neigh0.17053_cross0.3283_n14.mtx","synthetic_206500_206500_1273034_avg6.16481_std4.29748_diagonal_bw358.86067_skew7.27276_neigh0.17064_cross0.32624_n14.mtx","synthetic_206500_206500_1273051_avg6.1649_std3.71025_diagonal_bw366.16904_skew7.27265_neigh0.16791_cross0.33052_n14.mtx","synthetic_206500_206500_1273053_avg6.16491_std4.44447_diagonal_bw355.79495_skew7.11042_neigh0.17039_cross0.3241_n14.mtx","synthetic_206500_206500_1273074_avg6.16501_std4.00298_diagonal_bw362.83666_skew7.2725_neigh0.16845_cross0.33021_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_diagonal_bw250.50201_skew7.27211_neigh0.1828_cross0.332_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_diagonal_bw272.67229_skew7.27211_neigh0.18005_cross0.32953_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_diagonal_bw297.25621_skew7.27211_neigh0.17513_cross0.32858_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_diagonal_bw321.39573_skew7.27211_neigh0.17392_cross0.32618_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_diagonal_bw345.22322_skew7.27211_neigh0.17126_cross0.32461_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_diagonal_bw352.99268_skew7.27211_neigh0.16832_cross0.41146_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_diagonal_bw353.69447_skew7.27211_neigh0.17026_cross0.39266_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_diagonal_bw354.69502_skew7.27211_neigh0.19369_cross0.32315_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_diagonal_bw354.75358_skew7.27211_neigh0.16996_cross0.37247_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_diagonal_bw355.71822_skew7.27211_neigh0.18424_cross0.32217_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_diagonal_bw355.76259_skew7.27211_neigh0.16942_cross0.35332_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_diagonal_bw356.3296_skew7.27211_neigh0.17524_cross0.3238_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_diagonal_bw356.8881_skew7.27211_neigh0.16993_cross0.33263_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_diagonal_bw357.32531_skew7.27211_neigh0.17048_cross0.3136_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_diagonal_bw357.60403_skew7.27211_neigh0.16581_cross0.32313_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_diagonal_bw358.15523_skew7.27211_neigh0.17117_cross0.29493_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_diagonal_bw358.35721_skew7.27211_neigh0.15654_cross0.322_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_diagonal_bw358.64557_skew7.27211_neigh0.17114_cross0.27625_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_diagonal_bw359.47191_skew7.27211_neigh0.17162_cross0.25643_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_diagonal_bw359.53662_skew7.27211_neigh0.14621_cross0.32227_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_diagonal_bw360.09546_skew7.27211_neigh0.17221_cross0.23667_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_diagonal_bw360.26387_skew7.27211_neigh0.13732_cross0.32285_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_diagonal_bw360.84579_skew7.27211_neigh0.12827_cross0.32319_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_diagonal_bw367.88503_skew7.27211_neigh0.17018_cross0.32346_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_diagonal_bw378.11669_skew7.27211_neigh0.21111_cross0.3229_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_diagonal_bw379.08464_skew7.27211_neigh0.20234_cross0.32283_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_diagonal_bw392.32879_skew7.27211_neigh0.16826_cross0.32093_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_diagonal_bw416.65222_skew7.27211_neigh0.16705_cross0.32007_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_diagonal_bw440.35505_skew7.27211_neigh0.16563_cross0.32035_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_diagonal_bw463.79818_skew7.27211_neigh0.16463_cross0.31842_n14.mtx","synthetic_206500_206500_1273351_avg6.16635_std3.41298_diagonal_bw369.12411_skew7.2707_neigh0.16658_cross0.33288_n14.mtx","synthetic_206500_206500_1274354_avg6.17121_std3.11913_diagonal_bw371.78899_skew7.10214_neigh0.1662_cross0.33678_n14.mtx","synthetic_206500_206500_1315280_avg6.36939_std4.44461_diagonal_bw364.33156_skew7.32104_neigh0.17162_cross0.32684_n14.mtx","synthetic_206500_206500_1400260_avg6.78092_std4.44568_diagonal_bw375.85394_skew7.25847_neigh0.17409_cross0.33062_n14.mtx","synthetic_206500_206500_1485240_avg7.19245_std4.44584_diagonal_bw368.30232_skew7.20305_neigh0.17875_cross0.33711_n14.mtx","synthetic_206500_206500_1570037_avg7.60308_std4.44659_diagonal_bw376.69061_skew7.15458_neigh0.18122_cross0.33823_n14.mtx","synthetic_206500_206500_1655223_avg8.01561_std4.44684_diagonal_bw384.76944_skew7.23394_neigh0.18229_cross0.34249_n14.mtx","synthetic_206500_206500_890883_avg4.3142_std4.3742_diagonal_bw314.74509_skew7.34453_neigh0.14985_cross0.29741_n14.mtx","synthetic_206500_206500_975718_avg4.72503_std4.44295_diagonal_bw342.29814_skew7.46556_neigh0.15394_cross0.29927_n14.mtx","synthetic_210897_210897_1181860_avg5.60397_std4.45232_diagonal_bw62903.73817_skew62.88331_neigh0.59456_cross0.6267_n14.mtx","synthetic_21200_21200_1040239_avg49.06788_std6.3116_diagonal_bw2199.9967_skew0.48774_neigh1.7444_cross0.90083_n14.mtx","synthetic_21200_21200_1139550_avg53.75236_std6.31054_diagonal_bw2242.21934_skew0.4325_neigh1.75417_cross0.90404_n14.mtx","synthetic_21200_21200_1238795_avg58.43373_std6.30753_diagonal_bw2270.23382_skew0.4033_neigh1.7655_cross0.90641_n14.mtx","synthetic_21200_21200_1338002_avg63.1133_std6.31089_diagonal_bw1589.57203_skew0.37847_neigh1.76861_cross0.91301_n14.mtx","synthetic_21200_21200_1437342_avg67.79915_std6.31153_diagonal_bw1595.57099_skew0.35695_neigh1.77813_cross0.91446_n14.mtx","synthetic_21200_21200_1486357_avg70.11118_std8.20133_diagonal_bw1585.25472_skew0.44057_neigh1.79532_cross0.90224_n14.mtx","synthetic_21200_21200_1486477_avg70.11684_std7.77973_diagonal_bw1583.02868_skew0.41193_neigh1.79079_cross0.90683_n14.mtx","synthetic_21200_21200_1486586_avg70.12198_std7.35959_diagonal_bw1593.45448_skew0.39756_neigh1.78432_cross0.91026_n14.mtx","synthetic_21200_21200_1486691_avg70.12693_std6.94018_diagonal_bw1598.67321_skew0.36895_neigh1.79102_cross0.91039_n14.mtx","synthetic_21200_21200_1486777_avg70.13099_std6.51934_diagonal_bw1631.45156_skew0.35461_neigh1.78321_cross0.91501_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_diagonal_bw1081.97071_skew0.3403_neigh1.27549_cross0.91799_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_diagonal_bw1084.54524_skew0.3403_neigh1.38938_cross0.91731_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_diagonal_bw1117.74425_skew0.3403_neigh1.49561_cross0.91247_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_diagonal_bw1145.98717_skew0.3403_neigh1.61232_cross0.9156_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_diagonal_bw1149.7934_skew0.3403_neigh1.7861_cross0.91524_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_diagonal_bw1271.02222_skew0.3403_neigh1.78132_cross0.91674_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_diagonal_bw1300.1284_skew0.3403_neigh1.73006_cross0.9152_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_diagonal_bw1368.9767_skew0.3403_neigh1.77995_cross0.91561_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_diagonal_bw1468.32679_skew0.3403_neigh1.78426_cross0.91509_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_diagonal_bw1537.10948_skew0.3403_neigh1.72718_cross0.94892_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_diagonal_bw1564.65613_skew0.3403_neigh1.73429_cross0.94452_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_diagonal_bw1571.45608_skew0.3403_neigh1.78136_cross0.91469_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_diagonal_bw1625.49264_skew0.3403_neigh1.77718_cross0.91652_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_diagonal_bw1634.23774_skew0.3403_neigh1.86126_cross0.70933_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_diagonal_bw1635.55807_skew0.3403_neigh1.83717_cross0.82731_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_diagonal_bw1642.04132_skew0.3403_neigh1.85064_cross0.77163_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_diagonal_bw1646.38675_skew0.3403_neigh1.86599_cross0.6495_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_diagonal_bw1648.95406_skew0.3403_neigh1.80659_cross0.88791_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_diagonal_bw1682.76604_skew0.3403_neigh1.77909_cross0.91593_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_diagonal_bw1801.4084_skew0.3403_neigh1.77944_cross0.91386_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_diagonal_bw1870.38679_skew0.3403_neigh1.77964_cross0.91599_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_diagonal_bw1950.42142_skew0.3403_neigh1.85569_cross0.91574_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_diagonal_bw1986.16745_skew0.3403_neigh1.78558_cross0.9125_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_diagonal_bw2072.52415_skew0.3403_neigh1.78223_cross0.91562_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_diagonal_bw2091.85245_skew0.3403_neigh1.83963_cross0.91531_n14.mtx","synthetic_21200_21200_1487000_avg70.14151_std6.10324_diagonal_bw1634.66854_skew0.32589_neigh1.77365_cross0.91984_n14.mtx","synthetic_21200_21200_1487077_avg70.14514_std5.67801_diagonal_bw1633.00311_skew0.29731_neigh1.77437_cross0.91951_n14.mtx","synthetic_21200_21200_1487192_avg70.15057_std5.25576_diagonal_bw1652.73608_skew0.28295_neigh1.78041_cross0.92082_n14.mtx","synthetic_21200_21200_1487315_avg70.15637_std4.84276_diagonal_bw1652.45113_skew0.25434_neigh1.77351_cross0.92458_n14.mtx","synthetic_21200_21200_1487478_avg70.16406_std4.4186_diagonal_bw1687.16137_skew0.23995_neigh1.76694_cross0.92745_n14.mtx","synthetic_21200_21200_1536543_avg72.47844_std6.30853_diagonal_bw1645.73835_skew0.32453_neigh1.78455_cross0.91665_n14.mtx","synthetic_21200_21200_1635721_avg77.15665_std6.31046_diagonal_bw1656.96292_skew0.30903_neigh1.79294_cross0.9192_n14.mtx","synthetic_21200_21200_1735090_avg81.84387_std6.31393_diagonal_bw1680.13108_skew0.29515_neigh1.79626_cross0.9206_n14.mtx","synthetic_21200_21200_1834315_avg86.52429_std6.30851_diagonal_bw1434.02858_skew0.27132_neigh1.79882_cross0.92217_n14.mtx","synthetic_21200_21200_1933465_avg91.20118_std6.31043_diagonal_bw1436.35245_skew0.26095_neigh1.80915_cross0.92347_n14.mtx","synthetic_213383_213383_1315556_avg6.16523_std4.44392_diagonal_bw368.56408_skew7.27219_neigh0.16933_cross0.32183_n14.mtx","synthetic_216578_216578_17508080_avg80.83961_std117.71328_diagonal_bw28056.02979_skew7.89416_neigh1.2146_cross0.84473_n14.mtx","synthetic_21906_21906_1536320_avg70.13238_std6.31756_diagonal_bw1682.27098_skew0.34032_neigh1.78153_cross0.91497_n14.mtx","synthetic_222297_222297_1245675_avg5.60365_std4.45068_diagonal_bw66275.4602_skew62.88691_neigh0.59474_cross0.62554_n14.mtx","synthetic_227150_227150_1400378_avg6.16499_std4.4444_diagonal_bw392.40004_skew7.27252_neigh0.16863_cross0.32173_n14.mtx","synthetic_228954_228954_18508358_avg80.83876_std117.71434_diagonal_bw29655.86157_skew7.89425_neigh1.21497_cross0.8448_n14.mtx","synthetic_23320_23320_1635398_avg70.12856_std6.31946_diagonal_bw1799.99889_skew0.3404_neigh1.78021_cross0.91361_n14.mtx","synthetic_240916_240916_1485276_avg6.16512_std4.4437_diagonal_bw416.70601_skew7.27235_neigh0.16679_cross0.32047_n14.mtx","synthetic_241330_241330_19509069_avg80.8398_std117.71435_diagonal_bw31247.32631_skew7.89413_neigh1.21166_cross0.84524_n14.mtx","synthetic_24733_24733_1734543_avg70.13072_std6.31636_diagonal_bw1883.50532_skew0.34035_neigh1.77994_cross0.91558_n14.mtx","synthetic_254683_254683_1570192_avg6.16528_std4.4433_diagonal_bw440.76115_skew7.27213_neigh0.1672_cross0.32015_n14.mtx","synthetic_25491_25491_3028356_avg118.80099_std31.7722_diagonal_bw2553.87309_skew1.00335_neigh1.79544_cross0.81183_n14.mtx","synthetic_26146_26146_1833536_avg70.12683_std6.31243_diagonal_bw2003.83967_skew0.34043_neigh1.78323_cross0.91235_n14.mtx","synthetic_268096_268096_18481037_avg68.9344_std103.42196_diagonal_bw30970.77096_skew12.99882_neigh1.12598_cross0.8301_n14.mtx","synthetic_268096_268096_18481246_avg68.93518_std102.1488_diagonal_bw30884.51728_skew11.06931_neigh1.12482_cross0.83411_n14.mtx","synthetic_268096_268096_18481318_avg68.93545_std102.6344_diagonal_bw30907.47443_skew11.72205_neigh1.12565_cross0.83282_n14.mtx","synthetic_268096_268096_18481428_avg68.93586_std103.0557_diagonal_bw30967.38712_skew12.36025_neigh1.12626_cross0.83123_n14.mtx","synthetic_268096_268096_18481631_avg68.93662_std101.58036_diagonal_bw30873.57231_skew10.43079_neigh1.12458_cross0.83658_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_diagonal_bw29370.46931_skew10.11157_neigh1.10661_cross0.8633_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_diagonal_bw30799.97611_skew10.11157_neigh1.16245_cross0.83769_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_diagonal_bw30825.7074_skew10.11157_neigh1.23495_cross0.83815_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_diagonal_bw30839.54861_skew10.11157_neigh1.38307_cross0.83895_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_diagonal_bw30893.42836_skew10.11157_neigh1.01276_cross0.83715_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_diagonal_bw30904.3262_skew10.11157_neigh1.31024_cross0.83886_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_diagonal_bw30928.04662_skew10.11157_neigh1.08835_cross0.83649_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_diagonal_bw30929.83701_skew10.11157_neigh0.93859_cross0.83647_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_diagonal_bw30967.23835_skew10.11157_neigh0.86484_cross0.83608_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_diagonal_bw30999.29783_skew10.11157_neigh1.45859_cross0.83966_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_diagonal_bw31009.21338_skew10.11157_neigh0.79088_cross0.83554_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_diagonal_bw32084.18859_skew10.11157_neigh1.13434_cross0.80865_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_diagonal_bw33853.27533_skew10.11157_neigh1.14358_cross0.7543_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_diagonal_bw35112.29857_skew10.11157_neigh1.14998_cross0.69937_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_diagonal_bw36191.33218_skew10.11157_neigh1.15329_cross0.64297_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_diagonal_bw37019.2485_skew10.11157_neigh1.15704_cross0.58827_n14.mtx","synthetic_268096_268096_18482060_avg68.93822_std100.90606_diagonal_bw30904.29893_skew9.79227_neigh1.12389_cross0.83844_n14.mtx","synthetic_268096_268096_18482363_avg68.93935_std100.10125_diagonal_bw30812.69345_skew9.13935_neigh1.12221_cross0.84194_n14.mtx","synthetic_268096_268096_18482725_avg68.9407_std99.12285_diagonal_bw30864.65616_skew8.47191_neigh1.12038_cross0.84492_n14.mtx","synthetic_268096_268096_18483147_avg68.94227_std97.91705_diagonal_bw30780.25248_skew7.80447_neigh1.11844_cross0.84993_n14.mtx","synthetic_268450_268450_1655008_avg6.16505_std4.44456_diagonal_bw464.64015_skew7.27244_neigh0.16504_cross0.31934_n14.mtx","synthetic_27120_27120_1234696_avg45.52714_std31.4396_diagonal_bw652.95889_skew2.64618_neigh1.23287_cross0.77761_n14.mtx","synthetic_27560_27560_1933075_avg70.1406_std6.31297_diagonal_bw2085.91422_skew0.34017_neigh1.78453_cross0.915_n14.mtx","synthetic_27919_27919_3319237_avg118.8881_std31.78107_diagonal_bw2786.27282_skew1.00188_neigh1.79352_cross0.80999_n14.mtx","synthetic_29496_29496_3296630_avg111.76532_std88.66285_diagonal_bw11717.2995_skew3.35734_neigh1.89319_cross0.64015_n14.mtx","synthetic_29703_29703_1352183_avg45.52345_std31.43457_diagonal_bw714.4414_skew2.64647_neigh1.23245_cross0.77394_n14.mtx","synthetic_30347_30347_3608195_avg118.89791_std31.76143_diagonal_bw3042.07187_skew1.00172_neigh1.79272_cross0.81233_n14.mtx","synthetic_32286_32286_1469958_avg45.52927_std31.42835_diagonal_bw776.30704_skew2.66797_neigh1.22078_cross0.77601_n14.mtx","synthetic_32305_32305_3613009_avg111.84055_std88.75337_diagonal_bw12837.01436_skew3.35441_neigh1.89289_cross0.64054_n14.mtx","synthetic_32775_32775_3898242_avg118.9395_std31.78937_diagonal_bw3266.08442_skew1.00102_neigh1.79344_cross0.81117_n14.mtx","synthetic_32784_32784_1657863_avg50.56927_std27.82469_diagonal_bw4979.98978_skew3.09339_neigh1.63543_cross0.76823_n14.mtx","synthetic_3283001_3283001_13139084_avg4.00216_std1.21615_diagonal_bw3180.2682_skew312.8308_neigh0.41642_cross0.64828_n14.mtx","synthetic_34406_34406_1341834_avg39.0_std0.0_diagonal_bw7973.05903_skew0.0_neigh1.39882_cross0.80525_n14.mtx","synthetic_34869_34869_1587838_avg45.53724_std31.4194_diagonal_bw841.85663_skew2.66733_neigh1.22513_cross0.77194_n14.mtx","synthetic_35115_35115_3931279_avg111.95441_std88.67056_diagonal_bw13934.66499_skew3.34999_neigh1.89327_cross0.64172_n14.mtx","synthetic_35203_35203_4188100_avg118.96997_std31.72558_diagonal_bw3517.51751_skew1.0005_neigh1.79419_cross0.8104_n14.mtx","synthetic_35906_35906_1816148_avg50.58063_std27.80623_diagonal_bw5428.79923_skew3.09248_neigh1.63383_cross0.77123_n14.mtx","synthetic_3595668_3595668_14390363_avg4.00214_std1.20637_diagonal_bw3482.69612_skew312.83218_neigh0.41594_cross0.64805_n14.mtx","synthetic_36417_36417_3031503_avg83.24417_std31.62901_diagonal_bw3659.90647_skew1.4266_neigh1.77779_cross0.7712_n14.mtx","synthetic_36417_36417_3320284_avg91.17401_std31.69422_diagonal_bw3723.96647_skew1.30329_neigh1.78294_cross0.78319_n14.mtx","synthetic_36417_36417_3609556_avg99.11734_std31.72484_diagonal_bw3679.72145_skew1.19941_neigh1.78657_cross0.7906_n14.mtx","synthetic_36417_36417_3899087_avg107.06777_std31.73599_diagonal_bw3640.28097_skew1.11081_neigh1.79005_cross0.79883_n14.mtx","synthetic_36417_36417_4188724_avg115.02112_std31.74031_diagonal_bw3682.55345_skew1.03441_neigh1.79122_cross0.80743_n14.mtx","synthetic_36417_36417_4330804_avg118.92259_std41.20033_diagonal_bw3566.11415_skew1.30402_neigh1.79774_cross0.77959_n14.mtx","synthetic_36417_36417_4331206_avg118.93363_std39.11615_diagonal_bw3582.20153_skew1.23654_neigh1.79553_cross0.78805_n14.mtx","synthetic_36417_36417_4331707_avg118.94739_std37.01905_diagonal_bw3596.97108_skew1.16903_neigh1.79583_cross0.79348_n14.mtx","synthetic_36417_36417_4332360_avg118.96532_std34.9075_diagonal_bw3586.20139_skew1.10145_neigh1.79703_cross0.80194_n14.mtx","synthetic_36417_36417_4333163_avg118.98737_std32.79928_diagonal_bw3623.78502_skew1.03383_neigh1.79496_cross0.80499_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_diagonal_bw2579.24335_skew1.00003_neigh1.79431_cross0.81183_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_diagonal_bw2808.21114_skew1.00003_neigh1.79322_cross0.81044_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_diagonal_bw3060.87028_skew1.00003_neigh1.7932_cross0.81204_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_diagonal_bw3277.14828_skew1.00003_neigh1.79298_cross0.81293_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_diagonal_bw3375.93338_skew1.00003_neigh1.77805_cross0.86843_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_diagonal_bw3480.48944_skew1.00003_neigh1.38997_cross0.81257_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_diagonal_bw3491.53049_skew1.00003_neigh1.50396_cross0.81194_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_diagonal_bw3492.88901_skew1.00003_neigh1.27007_cross0.81135_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_diagonal_bw3503.97943_skew1.00003_neigh1.62041_cross0.81058_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_diagonal_bw3512.30892_skew1.00003_neigh1.7876_cross0.83745_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_diagonal_bw3516.78812_skew1.00003_neigh1.79411_cross0.811_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_diagonal_bw3588.56273_skew1.00003_neigh1.73452_cross0.81056_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_diagonal_bw3629.41058_skew1.00003_neigh1.79306_cross0.81098_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_diagonal_bw3710.52736_skew1.00003_neigh1.79803_cross0.78467_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_diagonal_bw3750.91707_skew1.00003_neigh1.79431_cross0.80913_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_diagonal_bw3858.63267_skew1.00003_neigh1.80498_cross0.73111_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_diagonal_bw3893.78263_skew1.00003_neigh1.85196_cross0.8121_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_diagonal_bw3967.32032_skew1.00003_neigh1.80868_cross0.68014_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_diagonal_bw3977.20534_skew1.00003_neigh1.79315_cross0.81223_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_diagonal_bw4055.86613_skew1.00003_neigh1.81065_cross0.62863_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_diagonal_bw4135.85089_skew1.00003_neigh1.81242_cross0.57712_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_diagonal_bw4215.83134_skew1.00003_neigh1.79293_cross0.81131_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_diagonal_bw4432.89683_skew1.00003_neigh1.79479_cross0.81053_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_diagonal_bw4655.25722_skew1.00003_neigh1.79359_cross0.80909_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_diagonal_bw5224.60291_skew1.00003_neigh1.94938_cross0.81229_n14.mtx","synthetic_36417_36417_4333866_avg119.00667_std30.68416_diagonal_bw3624.47093_skew0.96628_neigh1.79345_cross0.81402_n14.mtx","synthetic_36417_36417_4334685_avg119.02916_std28.56617_diagonal_bw3659.71673_skew0.89869_neigh1.79021_cross0.8221_n14.mtx","synthetic_36417_36417_4335436_avg119.04978_std26.44817_diagonal_bw3656.97312_skew0.83117_neigh1.79127_cross0.82819_n14.mtx","synthetic_36417_36417_4336151_avg119.06942_std24.33643_diagonal_bw3688.3458_skew0.76368_neigh1.79072_cross0.83561_n14.mtx","synthetic_36417_36417_4336875_avg119.0893_std22.21912_diagonal_bw3701.03968_skew0.7046_neigh1.78927_cross0.84338_n14.mtx","synthetic_36417_36417_4478356_avg122.97433_std31.74172_diagonal_bw3655.42475_skew0.96789_neigh1.79462_cross0.81529_n14.mtx","synthetic_36417_36417_4767999_avg130.92784_std31.7426_diagonal_bw3634.76154_skew0.90945_neigh1.79683_cross0.8202_n14.mtx","synthetic_36417_36417_5057685_avg138.88253_std31.74387_diagonal_bw3658.71557_skew0.85769_neigh1.80018_cross0.82673_n14.mtx","synthetic_36417_36417_5347290_avg146.83499_std31.74228_diagonal_bw3634.01996_skew0.81156_neigh1.80021_cross0.83143_n14.mtx","synthetic_36417_36417_5636923_avg154.78823_std31.74188_diagonal_bw3643.12294_skew0.77016_neigh1.80217_cross0.8324_n14.mtx","synthetic_368077_368077_1472308_avg4.0_std0.0_diagonal_bw504.58307_skew0.0_neigh0.37345_cross0.98911_n14.mtx","synthetic_37452_37452_1706036_avg45.5526_std31.4087_diagonal_bw902.84756_skew2.66609_neigh1.22018_cross0.77042_n14.mtx","synthetic_37630_37630_4477645_avg118.99136_std31.72824_diagonal_bw3752.49753_skew1.00015_neigh1.79415_cross0.80957_n14.mtx","synthetic_37683_37683_1469637_avg39.0_std0.0_diagonal_bw8731.54935_skew0.0_neigh1.39924_cross0.81006_n14.mtx","synthetic_37924_37924_4245700_avg111.95285_std88.65794_diagonal_bw15018.56603_skew3.35005_neigh1.89355_cross0.6412_n14.mtx","synthetic_38120_38120_14186695_avg372.15884_std217.74813_diagonal_bw16610.97285_skew1.39145_neigh1.97336_cross0.92298_n14.mtx","synthetic_38120_38120_14353565_avg376.53633_std228.90355_diagonal_bw16362.02206_skew1.46191_neigh1.97392_cross0.9229_n14.mtx","synthetic_38120_38120_14506403_avg380.54572_std239.95763_diagonal_bw16221.47907_skew1.5332_neigh1.97371_cross0.92545_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_diagonal_bw16107.56653_skew1.57045_neigh1.90891_cross0.91992_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_diagonal_bw16165.12815_skew1.57045_neigh1.3922_cross0.92174_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_diagonal_bw16173.10614_skew1.57045_neigh1.64922_cross0.92071_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_diagonal_bw16181.2495_skew1.57045_neigh1.51874_cross0.92011_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_diagonal_bw16189.91522_skew1.57045_neigh1.77707_cross0.91956_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_diagonal_bw16403.35273_skew1.57045_neigh1.97397_cross0.72233_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_diagonal_bw16412.02442_skew1.57045_neigh1.97389_cross0.66503_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_diagonal_bw16450.53809_skew1.57045_neigh1.9739_cross0.77998_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_diagonal_bw16504.44638_skew1.57045_neigh1.97349_cross0.83444_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_diagonal_bw5783.47169_skew1.57045_neigh1.99394_cross0.9293_n14.mtx","synthetic_38120_38120_14646455_avg384.2197_std250.9036_diagonal_bw16369.04491_skew1.60788_neigh1.97362_cross0.92456_n14.mtx","synthetic_38120_38120_14774683_avg387.5835_std261.73289_diagonal_bw16271.8489_skew1.68071_neigh1.97337_cross0.92347_n14.mtx","synthetic_38120_38120_14892124_avg390.66432_std272.44037_diagonal_bw16642.15774_skew1.75428_neigh1.9735_cross0.92246_n14.mtx","synthetic_38120_38120_15029193_avg394.26005_std283.24138_diagonal_bw17593.26878_skew1.83569_neigh1.95985_cross0.92052_n14.mtx","synthetic_38120_38120_15246247_avg399.95401_std294.48908_diagonal_bw17717.6793_skew1.94534_neigh1.95613_cross0.91225_n14.mtx","synthetic_38744_38744_1161488_avg29.97853_std22.5272_diagonal_bw1029.10133_skew1.93543_neigh1.22753_cross0.85246_n14.mtx","synthetic_38744_38744_1307882_avg33.75702_std24.88267_diagonal_bw994.82547_skew2.08084_neigh1.17474_cross0.82692_n14.mtx","synthetic_38744_38744_1449216_avg37.40491_std27.07152_diagonal_bw955.52689_skew2.28834_neigh1.19116_cross0.80482_n14.mtx","synthetic_38744_38744_1579684_avg40.77235_std28.98643_diagonal_bw944.42949_skew2.45823_neigh1.20967_cross0.78737_n14.mtx","synthetic_38744_38744_1703850_avg43.97713_std30.65815_diagonal_bw937.00284_skew2.61552_neigh1.2153_cross0.77679_n14.mtx","synthetic_38744_38744_1707520_avg44.07186_std32.46486_diagonal_bw992.99381_skew2.10856_neigh1.20651_cross0.82436_n14.mtx","synthetic_38744_38744_1730367_avg44.66155_std32.51286_diagonal_bw967.85456_skew2.22425_neigh1.21385_cross0.81258_n14.mtx","synthetic_38744_38744_1736329_avg44.81543_std26.74959_diagonal_bw952.63904_skew1.96773_neigh1.21351_cross0.79326_n14.mtx","synthetic_38744_38744_1746236_avg45.07113_std32.41542_diagonal_bw950.25712_skew2.35026_neigh1.21262_cross0.79957_n14.mtx","synthetic_38744_38744_1747768_avg45.11068_std28.0118_diagonal_bw947.08319_skew2.12565_neigh1.21504_cross0.78804_n14.mtx","synthetic_38744_38744_1755242_avg45.30358_std29.13201_diagonal_bw940.57346_skew2.28892_neigh1.21712_cross0.78411_n14.mtx","synthetic_38744_38744_1756611_avg45.33892_std32.15335_diagonal_bw942.33254_skew2.48486_neigh1.21428_cross0.78987_n14.mtx","synthetic_38744_38744_1760377_avg45.43612_std30.12952_diagonal_bw941.83703_skew2.43339_neigh1.22146_cross0.77671_n14.mtx","synthetic_38744_38744_1762627_avg45.49419_std31.70473_diagonal_bw939.03988_skew2.60486_neigh1.21755_cross0.77579_n14.mtx","synthetic_38744_38744_1763457_avg45.51562_std31.01391_diagonal_bw935.46596_skew2.60316_neigh1.22021_cross0.77524_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_diagonal_bw1007.61049_skew2.6669_neigh1.23901_cross0.60466_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_diagonal_bw1020.39872_skew2.6669_neigh1.23921_cross0.56066_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_diagonal_bw1025.26425_skew2.6669_neigh1.21375_cross0.77219_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_diagonal_bw1083.3129_skew2.6669_neigh1.21379_cross0.77025_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_diagonal_bw1146.61932_skew2.6669_neigh1.21114_cross0.77133_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_diagonal_bw1206.53601_skew2.6669_neigh1.21863_cross0.77168_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_diagonal_bw658.37758_skew2.6669_neigh1.22947_cross0.77507_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_diagonal_bw718.69366_skew2.6669_neigh1.22855_cross0.77459_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_diagonal_bw780.82743_skew2.6669_neigh1.22346_cross0.77387_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_diagonal_bw812.95816_skew2.6669_neigh1.18464_cross0.89437_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_diagonal_bw816.16013_skew2.6669_neigh1.18813_cross0.89061_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_diagonal_bw837.69265_skew2.6669_neigh1.22334_cross0.77418_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_diagonal_bw873.34813_skew2.6669_neigh1.20844_cross0.84314_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_diagonal_bw897.51727_skew2.6669_neigh1.2208_cross0.77498_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_diagonal_bw915.8694_skew2.6669_neigh1.21725_cross0.79709_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_diagonal_bw930.70419_skew2.6669_neigh1.10614_cross0.77727_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_diagonal_bw930.71709_skew2.6669_neigh1.18489_cross0.77237_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_diagonal_bw931.48136_skew2.6669_neigh1.03297_cross0.77123_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_diagonal_bw932.33357_skew2.6669_neigh0.96264_cross0.77189_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_diagonal_bw932.7993_skew2.6669_neigh1.25769_cross0.77371_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_diagonal_bw934.61429_skew2.6669_neigh0.88819_cross0.7721_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_diagonal_bw937.26642_skew2.6669_neigh1.33096_cross0.77342_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_diagonal_bw940.71841_skew2.6669_neigh1.40988_cross0.77173_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_diagonal_bw950.09139_skew2.6669_neigh1.48389_cross0.77368_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_diagonal_bw950.26476_skew2.6669_neigh1.2235_cross0.74756_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_diagonal_bw961.91957_skew2.6669_neigh1.22023_cross0.77204_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_diagonal_bw968.81104_skew2.6669_neigh1.56264_cross0.77172_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_diagonal_bw970.44453_skew2.6669_neigh1.2313_cross0.70063_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_diagonal_bw991.09919_skew2.6669_neigh1.23319_cross0.6528_n14.mtx","synthetic_38744_38744_1765368_avg45.56494_std31.78636_diagonal_bw930.97393_skew2.75289_neigh1.22265_cross0.77205_n14.mtx","synthetic_38744_38744_1765877_avg45.57808_std31.07073_diagonal_bw931.07658_skew2.72986_neigh1.22085_cross0.76806_n14.mtx","synthetic_38744_38744_1766496_avg45.59405_std32.481_diagonal_bw927.24148_skew2.88209_neigh1.22172_cross0.76806_n14.mtx","synthetic_38744_38744_1767073_avg45.60895_std26.86577_diagonal_bw920.15365_skew3.122_neigh1.22525_cross0.74801_n14.mtx","synthetic_38744_38744_1767160_avg45.61119_std30.25452_diagonal_bw926.58649_skew2.83678_neigh1.22133_cross0.7642_n14.mtx","synthetic_38744_38744_1767330_avg45.61558_std33.08901_diagonal_bw927.56388_skew3.03371_neigh1.22246_cross0.76288_n14.mtx","synthetic_38744_38744_1767431_avg45.61819_std29.2665_diagonal_bw927.4427_skew2.94579_neigh1.22246_cross0.75424_n14.mtx","synthetic_38744_38744_1767460_avg45.61893_std28.1289_diagonal_bw919.38078_skew3.03341_neigh1.22097_cross0.75479_n14.mtx","synthetic_38744_38744_1767651_avg45.62386_std33.63482_diagonal_bw924.81207_skew3.16449_neigh1.22491_cross0.76276_n14.mtx","synthetic_38744_38744_1767760_avg45.62668_std34.12282_diagonal_bw922.18836_skew3.31765_neigh1.22536_cross0.76056_n14.mtx","synthetic_38744_38744_1824565_avg47.09284_std32.11946_diagonal_bw936.71239_skew2.7373_neigh1.22296_cross0.76775_n14.mtx","synthetic_38744_38744_1943737_avg50.16872_std33.41096_diagonal_bw932.86026_skew2.82709_neigh1.22921_cross0.76384_n14.mtx","synthetic_38744_38744_2062005_avg53.22127_std34.5782_diagonal_bw922.50232_skew2.90821_neigh1.23925_cross0.76092_n14.mtx","synthetic_38744_38744_2179882_avg56.26373_std35.63951_diagonal_bw923.68405_skew2.98125_neigh1.24163_cross0.75814_n14.mtx","synthetic_38744_38744_2297582_avg59.30162_std36.62621_diagonal_bw923.20411_skew3.03024_neigh1.25019_cross0.75495_n14.mtx","synthetic_39029_39029_1974236_avg50.58382_std27.80163_diagonal_bw5889.07769_skew3.11199_neigh1.63432_cross0.77123_n14.mtx","synthetic_3908334_3908334_15641636_avg4.00212_std1.19836_diagonal_bw3787.4883_skew312.83338_neigh0.41515_cross0.64842_n14.mtx","synthetic_40035_40035_1824084_avg45.56223_std31.3971_diagonal_bw964.72262_skew2.66532_neigh1.22009_cross0.77447_n14.mtx","synthetic_40058_40058_4768887_avg119.04955_std31.78281_diagonal_bw3989.12307_skew0.99917_neigh1.79379_cross0.8114_n14.mtx","synthetic_403132_403132_1612528_avg4.0_std0.0_diagonal_bw553.42608_skew0.0_neigh0.37448_cross0.98906_n14.mtx","synthetic_40733_40733_4573744_avg112.28596_std88.81893_diagonal_bw16070.31044_skew3.33714_neigh1.8936_cross0.64178_n14.mtx","synthetic_40960_40960_1597440_avg39.0_std0.0_diagonal_bw9495.98555_skew0.0_neigh1.40202_cross0.80537_n14.mtx","synthetic_42138_42138_3666754_avg87.01775_std81.65352_diagonal_bw17018.78651_skew4.24031_neigh1.89191_cross0.58167_n14.mtx","synthetic_42138_42138_3893764_avg92.40505_std83.38574_diagonal_bw15269.74446_skew4.01055_neigh1.89198_cross0.59237_n14.mtx","synthetic_42138_42138_3928503_avg93.22946_std52.30654_diagonal_bw21754.9879_skew1.38122_neigh1.71883_cross0.97165_n14.mtx","synthetic_42138_42138_4006220_avg95.07381_std54.75201_diagonal_bw21199.48559_skew1.47176_neigh1.73848_cross0.96424_n14.mtx","synthetic_42138_42138_4072421_avg96.64486_std57.11798_diagonal_bw20552.35225_skew1.5661_neigh1.75712_cross0.95786_n14.mtx","synthetic_42138_42138_4126809_avg97.93557_std85.02631_diagonal_bw15674.99532_skew3.79907_neigh1.89239_cross0.60629_n14.mtx","synthetic_42138_42138_4128960_avg97.98662_std59.40057_diagonal_bw20131.7263_skew1.65342_neigh1.77031_cross0.9522_n14.mtx","synthetic_42138_42138_4176699_avg99.11954_std61.5954_diagonal_bw19886.75485_skew1.74416_neigh1.78051_cross0.94591_n14.mtx","synthetic_42138_42138_4365370_avg103.59699_std86.57531_diagonal_bw16089.94143_skew3.60438_neigh1.89274_cross0.62091_n14.mtx","synthetic_42138_42138_4491637_avg106.5935_std66.96099_diagonal_bw17746.25056_skew2.48989_neigh1.88275_cross0.71376_n14.mtx","synthetic_42138_42138_4533678_avg107.5912_std72.07832_diagonal_bw17488.78198_skew2.69919_neigh1.88644_cross0.69523_n14.mtx","synthetic_42138_42138_4582731_avg108.7553_std77.01103_diagonal_bw17151.27085_skew2.88947_neigh1.88851_cross0.67805_n14.mtx","synthetic_42138_42138_4608940_avg109.37728_std88.04095_diagonal_bw16405.47544_skew3.42505_neigh1.89321_cross0.63399_n14.mtx","synthetic_42138_42138_4638390_avg110.07618_std81.79532_diagonal_bw16937.41371_skew3.07899_neigh1.88999_cross0.6614_n14.mtx","synthetic_42138_42138_4699816_avg111.53391_std86.4582_diagonal_bw16776.08987_skew3.24983_neigh1.89212_cross0.64722_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_diagonal_bw12401.69942_skew3.33605_neigh1.89352_cross0.6363_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_diagonal_bw13394.34783_skew3.33605_neigh1.89357_cross0.63833_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_diagonal_bw13613.90223_skew3.33605_neigh1.32672_cross0.62991_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_diagonal_bw13637.14168_skew3.33605_neigh1.45441_cross0.63031_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_diagonal_bw13744.74961_skew3.33605_neigh1.57876_cross0.6321_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_diagonal_bw13939.96504_skew3.33605_neigh1.70489_cross0.63265_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_diagonal_bw14040.32778_skew3.33605_neigh1.96986_cross0.65808_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_diagonal_bw14295.7245_skew3.33605_neigh1.89407_cross0.63944_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_diagonal_bw14505.18821_skew3.33605_neigh1.83046_cross0.63541_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_diagonal_bw15310.55024_skew3.33605_neigh1.89386_cross0.6408_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_diagonal_bw16210.1421_skew3.33605_neigh1.89362_cross0.64161_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_diagonal_bw16611.63949_skew3.33605_neigh1.89329_cross0.64481_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_diagonal_bw16636.23077_skew3.33605_neigh1.89346_cross0.6403_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_diagonal_bw16806.17125_skew3.33605_neigh1.89434_cross0.62136_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_diagonal_bw17086.3858_skew3.33605_neigh1.89638_cross0.5859_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_diagonal_bw17169.41374_skew3.33605_neigh1.89337_cross0.64205_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_diagonal_bw17388.06735_skew3.33605_neigh1.89731_cross0.5472_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_diagonal_bw17574.48287_skew3.33605_neigh1.89818_cross0.5104_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_diagonal_bw17859.63285_skew3.33605_neigh1.89909_cross0.46925_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_diagonal_bw17951.80365_skew3.33605_neigh1.9579_cross0.66188_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_diagonal_bw17999.08176_skew3.33605_neigh1.89332_cross0.64342_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_diagonal_bw18723.11078_skew3.33605_neigh1.89352_cross0.64341_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_diagonal_bw19493.96504_skew3.33605_neigh1.89374_cross0.64434_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_diagonal_bw20097.98809_skew3.33605_neigh1.89382_cross0.64549_n14.mtx","synthetic_42138_42138_4766804_avg113.12364_std90.99737_diagonal_bw16569.85666_skew3.41994_neigh1.8942_cross0.63355_n14.mtx","synthetic_42138_42138_4838257_avg114.81933_std95.4594_diagonal_bw16396.72595_skew3.5724_neigh1.89578_cross0.62195_n14.mtx","synthetic_42138_42138_4857446_avg115.27472_std89.41484_diagonal_bw16809.13779_skew3.25939_neigh1.89318_cross0.64791_n14.mtx","synthetic_42138_42138_4913422_avg116.60311_std99.85384_diagonal_bw16243.33808_skew3.72543_neigh1.89696_cross0.61156_n14.mtx","synthetic_42138_42138_4991872_avg118.46485_std104.19584_diagonal_bw14758.12336_skew3.8622_neigh1.89775_cross0.60004_n14.mtx","synthetic_42138_42138_5073389_avg120.39938_std108.48726_diagonal_bw14653.196_skew4.00003_neigh1.89942_cross0.59075_n14.mtx","synthetic_42138_42138_5111019_avg121.2924_std90.68096_diagonal_bw15825.42845_skew3.10578_neigh1.89371_cross0.65868_n14.mtx","synthetic_42138_42138_5368820_avg127.41041_std91.85872_diagonal_bw16050.79954_skew2.96357_neigh1.89482_cross0.67034_n14.mtx","synthetic_42138_42138_5630518_avg133.62091_std92.94843_diagonal_bw16358.79403_skew2.83174_neigh1.89447_cross0.682_n14.mtx","synthetic_42138_42138_5895869_avg139.9181_std93.95081_diagonal_bw16631.46988_skew2.70931_neigh1.89472_cross0.69266_n14.mtx","synthetic_42151_42151_2133812_avg50.62305_std27.78297_diagonal_bw6371.02387_skew3.12855_neigh1.63402_cross0.77144_n14.mtx","synthetic_4221001_4221001_16892920_avg4.00211_std1.19165_diagonal_bw4085.33261_skew312.83427_neigh0.41605_cross0.64818_n14.mtx","synthetic_42486_42486_5059907_avg119.09587_std31.75092_diagonal_bw4233.6327_skew1.14113_neigh1.79264_cross0.81065_n14.mtx","synthetic_42618_42618_1942443_avg45.578_std31.38541_diagonal_bw1023.09081_skew2.66405_neigh1.21726_cross0.77332_n14.mtx","synthetic_43542_43542_4892832_avg112.3704_std88.7642_diagonal_bw17276.74331_skew3.82333_neigh1.89356_cross0.64204_n14.mtx","synthetic_43715_43715_2801362_avg64.0824_std14.01594_diagonal_bw317.77484_skew0.93501_neigh1.58611_cross0.83222_n14.mtx","synthetic_438187_438187_1752748_avg4.0_std0.0_diagonal_bw605.21006_skew0.0_neigh0.36936_cross0.98955_n14.mtx","synthetic_44236_44236_1725204_avg39.0_std0.0_diagonal_bw10260.92608_skew0.0_neigh1.40155_cross0.80735_n14.mtx","synthetic_44914_44914_5350806_avg119.13448_std31.75488_diagonal_bw4462.1276_skew1.14044_neigh1.7944_cross0.81059_n14.mtx","synthetic_45201_45201_2060190_avg45.57842_std31.38204_diagonal_bw1086.85529_skew2.68596_neigh1.21635_cross0.77336_n14.mtx","synthetic_45273_45273_2292008_avg50.62638_std27.77577_diagonal_bw6880.17573_skew3.12828_neigh1.63398_cross0.77235_n14.mtx","synthetic_4533668_4533668_18144199_avg4.0021_std1.18568_diagonal_bw4390.98196_skew312.83513_neigh0.41656_cross0.64825_n14.mtx","synthetic_46351_46351_5209319_avg112.38849_std88.86396_diagonal_bw18374.76208_skew3.82256_neigh1.89341_cross0.64137_n14.mtx","synthetic_46835_46835_1659331_avg35.42929_std24.19453_diagonal_bw7302.21917_skew2.92331_neigh1.5963_cross0.7705_n14.mtx","synthetic_46835_46835_1817944_avg38.81593_std25.43637_diagonal_bw7388.90979_skew3.04473_neigh1.61055_cross0.76366_n14.mtx","synthetic_46835_46835_1976036_avg42.19144_std26.52871_diagonal_bw7052.0227_skew3.14776_neigh1.62131_cross0.75772_n14.mtx","synthetic_46835_46835_2134025_avg45.56475_std27.52318_diagonal_bw7179.05959_skew3.21378_neigh1.62818_cross0.75329_n14.mtx","synthetic_46835_46835_2292218_avg48.94241_std27.77072_diagonal_bw6998.59428_skew3.16816_neigh1.62975_cross0.76816_n14.mtx","synthetic_46835_46835_2370905_avg50.6225_std27.74621_diagonal_bw6973.90507_skew2.81253_neigh1.63594_cross0.75611_n14.mtx","synthetic_46835_46835_2371001_avg50.62455_std27.39541_diagonal_bw6984.37081_skew2.69386_neigh1.6379_cross0.75534_n14.mtx","synthetic_46835_46835_2371113_avg50.62695_std32.27909_diagonal_bw6994.42105_skew3.10848_neigh1.63552_cross0.75717_n14.mtx","synthetic_46835_46835_2371156_avg50.62786_std27.75732_diagonal_bw7042.80683_skew2.91089_neigh1.63466_cross0.76079_n14.mtx","synthetic_46835_46835_2371164_avg50.62803_std30.41679_diagonal_bw6939.09181_skew3.22691_neigh1.63687_cross0.75372_n14.mtx","synthetic_46835_46835_2371201_avg50.62882_std33.12436_diagonal_bw6969.84853_skew3.04908_neigh1.63376_cross0.76209_n14.mtx","synthetic_46835_46835_2371202_avg50.62885_std27.76569_diagonal_bw7070.97963_skew3.00957_neigh1.6345_cross0.76617_n14.mtx","synthetic_46835_46835_2371250_avg50.62987_std31.37317_diagonal_bw6975.62833_skew3.1675_neigh1.6373_cross0.75463_n14.mtx","synthetic_46835_46835_2371308_avg50.63111_std28.69655_diagonal_bw7047.95307_skew3.1674_neigh1.63521_cross0.76523_n14.mtx","synthetic_46835_46835_2371384_avg50.63273_std27.77434_diagonal_bw7064.33473_skew3.08826_neigh1.63349_cross0.76857_n14.mtx","synthetic_46835_46835_2371440_avg50.63393_std27.7783_diagonal_bw7114.30981_skew3.18692_neigh1.63074_cross0.77345_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_diagonal_bw5050.44975_skew3.12766_neigh1.63384_cross0.7683_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_diagonal_bw5519.37801_skew3.12766_neigh1.63425_cross0.76948_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_diagonal_bw5974.71957_skew3.12766_neigh1.63482_cross0.76832_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_diagonal_bw6289.96966_skew3.12766_neigh1.56424_cross0.88608_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_diagonal_bw6432.35471_skew3.12766_neigh1.63211_cross0.77136_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_diagonal_bw6665.6247_skew3.12766_neigh1.60147_cross0.846_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_diagonal_bw6698.93127_skew3.12766_neigh1.14856_cross0.76971_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_diagonal_bw6757.40436_skew3.12766_neigh1.25689_cross0.76811_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_diagonal_bw6780.66794_skew3.12766_neigh1.36547_cross0.76752_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_diagonal_bw6866.76471_skew3.12766_neigh1.57918_cross0.77036_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_diagonal_bw6870.58687_skew3.12766_neigh1.47124_cross0.76996_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_diagonal_bw6888.78317_skew3.12766_neigh1.63511_cross0.76873_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_diagonal_bw6998.11442_skew3.12766_neigh1.62573_cross0.79459_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_diagonal_bw7163.16152_skew3.12766_neigh1.68835_cross0.77345_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_diagonal_bw7218.82703_skew3.12766_neigh1.6394_cross0.74525_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_diagonal_bw7311.94737_skew3.12766_neigh1.63543_cross0.76939_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_diagonal_bw7364.02791_skew3.12766_neigh1.64601_cross0.69592_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_diagonal_bw7473.56761_skew3.12766_neigh1.65519_cross0.64484_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_diagonal_bw7584.44582_skew3.12766_neigh1.66065_cross0.59521_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_diagonal_bw7668.03327_skew3.12766_neigh1.66479_cross0.54423_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_diagonal_bw7752.28955_skew3.12766_neigh1.63462_cross0.76859_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_diagonal_bw8144.77457_skew3.12766_neigh1.8993_cross0.77798_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_diagonal_bw8228.79031_skew3.12766_neigh1.63299_cross0.77184_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_diagonal_bw8649.96035_skew3.12766_neigh1.63342_cross0.77078_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_diagonal_bw8734.98268_skew3.12766_neigh1.79476_cross0.77224_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_diagonal_bw9093.75454_skew3.12766_neigh1.63511_cross0.76942_n14.mtx","synthetic_46835_46835_2371578_avg50.63687_std27.78811_diagonal_bw7158.8859_skew3.36441_neigh1.63233_cross0.77814_n14.mtx","synthetic_46835_46835_2371669_avg50.63882_std26.85583_diagonal_bw7120.5289_skew3.10752_neigh1.63199_cross0.77811_n14.mtx","synthetic_46835_46835_2371693_avg50.63933_std27.78141_diagonal_bw7136.54039_skew3.26546_neigh1.63148_cross0.77945_n14.mtx","synthetic_46835_46835_2371771_avg50.64099_std27.79721_diagonal_bw7216.51397_skew3.60102_neigh1.63057_cross0.78376_n14.mtx","synthetic_46835_46835_2371874_avg50.64319_std27.79128_diagonal_bw7207.36039_skew3.48234_neigh1.63236_cross0.78304_n14.mtx","synthetic_46835_46835_2371944_avg50.64469_std25.00834_diagonal_bw7255.44428_skew3.02806_neigh1.63083_cross0.78597_n14.mtx","synthetic_46835_46835_2372249_avg50.6512_std23.16377_diagonal_bw7332.95584_skew3.0078_neigh1.63031_cross0.79742_n14.mtx","synthetic_46835_46835_2372391_avg50.65423_std21.31556_diagonal_bw7497.52574_skew2.98782_neigh1.6247_cross0.80892_n14.mtx","synthetic_46835_46835_2372900_avg50.6651_std19.46618_diagonal_bw7594.71372_skew2.96723_neigh1.62451_cross0.818_n14.mtx","synthetic_46835_46835_2450659_avg52.32538_std27.78086_diagonal_bw7165.93817_skew3.1089_neigh1.6353_cross0.77626_n14.mtx","synthetic_46835_46835_2609003_avg55.70627_std27.78687_diagonal_bw7091.41887_skew3.03904_neigh1.63996_cross0.78555_n14.mtx","synthetic_46835_46835_2767329_avg59.08677_std27.79319_diagonal_bw7277.89561_skew3.01105_neigh1.64136_cross0.79423_n14.mtx","synthetic_46835_46835_2925939_avg62.47334_std27.79699_diagonal_bw7235.67689_skew3.00171_neigh1.64415_cross0.80009_n14.mtx","synthetic_46835_46835_3084303_avg65.85466_std27.80201_diagonal_bw7327.42293_skew2.97846_neigh1.64707_cross0.80559_n14.mtx","synthetic_4690002_4690002_14083544_avg3.00289_std1.14359_diagonal_bw5391.75836_skew291.71835_neigh0.36891_cross0.64826_n14.mtx","synthetic_4690002_4690002_18765165_avg4.0011_std0.88465_diagonal_bw4544.61296_skew312.91371_neigh0.41523_cross0.64886_n14.mtx","synthetic_4690002_4690002_18766060_avg4.00129_std0.94875_diagonal_bw4544.61433_skew312.89874_neigh0.41523_cross0.64886_n14.mtx","synthetic_4690002_4690002_18767037_avg4.0015_std1.01413_diagonal_bw4544.62177_skew312.88239_neigh0.41524_cross0.64886_n14.mtx","synthetic_4690002_4690002_18767925_avg4.00169_std1.23629_diagonal_bw4544.6141_skew406.578_neigh0.4153_cross0.64886_n14.mtx","synthetic_4690002_4690002_18768094_avg4.00172_std1.08092_diagonal_bw4544.60917_skew312.86472_neigh0.41525_cross0.64886_n14.mtx","synthetic_4690002_4690002_18768255_avg4.00176_std1.22278_diagonal_bw4544.60003_skew385.58006_neigh0.41528_cross0.64886_n14.mtx","synthetic_4690002_4690002_18768639_avg4.00184_std1.21074_diagonal_bw4544.61468_skew364.83169_neigh0.41531_cross0.64886_n14.mtx","synthetic_4690002_4690002_18769069_avg4.00193_std1.19917_diagonal_bw4544.58175_skew344.08333_neigh0.41529_cross0.64886_n14.mtx","synthetic_4690002_4690002_18769236_avg4.00197_std1.1487_diagonal_bw4544.58612_skew312.84562_neigh0.41528_cross0.64886_n14.mtx","synthetic_4690002_4690002_18769566_avg4.00204_std1.18832_diagonal_bw4544.53443_skew323.33476_neigh0.41529_cross0.64886_n14.mtx","synthetic_4690002_4690002_18769585_avg4.00204_std1.18853_diagonal_bw4544.55293_skew323.33443_neigh0.41529_cross0.64886_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_diagonal_bw3184.88357_skew312.8356_neigh0.41551_cross0.64901_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_diagonal_bw3466.39633_skew312.8356_neigh0.41544_cross0.64895_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_diagonal_bw3795.79694_skew312.8356_neigh0.41543_cross0.64893_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_diagonal_bw4074.65853_skew312.8356_neigh0.41535_cross0.64891_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_diagonal_bw4200.67402_skew312.8356_neigh0.54031_cross0.6488_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_diagonal_bw4279.94263_skew312.8356_neigh0.51216_cross0.6488_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_diagonal_bw4358.28843_skew312.8356_neigh0.48436_cross0.64888_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_diagonal_bw4407.10934_skew312.8356_neigh0.41525_cross0.64888_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_diagonal_bw4433.9589_skew312.8356_neigh0.45655_cross0.64867_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_diagonal_bw4508.13803_skew312.8356_neigh0.42901_cross0.64876_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_diagonal_bw4542.4258_skew312.8356_neigh0.41512_cross0.58418_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_diagonal_bw4543.20029_skew312.8356_neigh0.41608_cross0.84265_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_diagonal_bw4543.24698_skew312.8356_neigh0.41538_cross0.62721_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_diagonal_bw4543.90921_skew312.8356_neigh0.41541_cross0.45424_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_diagonal_bw4544.3722_skew312.8356_neigh0.41496_cross0.67021_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_diagonal_bw4544.551_skew312.8356_neigh0.41603_cross0.79985_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_diagonal_bw4544.99823_skew312.8356_neigh0.415_cross0.54079_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_diagonal_bw4545.19427_skew312.8356_neigh0.4153_cross0.49762_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_diagonal_bw4546.90824_skew312.8356_neigh0.41469_cross0.71357_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_diagonal_bw4547.16626_skew312.8356_neigh0.41515_cross0.75665_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_diagonal_bw4581.4107_skew312.8356_neigh0.40122_cross0.64877_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_diagonal_bw4652.81349_skew312.8356_neigh0.37381_cross0.64869_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_diagonal_bw4685.0085_skew312.8356_neigh0.41522_cross0.64883_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_diagonal_bw4720.29248_skew312.8356_neigh0.3463_cross0.64858_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_diagonal_bw4787.73638_skew312.8356_neigh0.31846_cross0.64869_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_diagonal_bw4854.49698_skew312.8356_neigh0.29083_cross0.64876_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_diagonal_bw5013.47671_skew312.8356_neigh0.41526_cross0.64883_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_diagonal_bw5289.58717_skew312.8356_neigh0.41524_cross0.64878_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_diagonal_bw5622.90244_skew312.8356_neigh0.41522_cross0.64879_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_diagonal_bw5904.00292_skew312.8356_neigh0.41517_cross0.64878_n14.mtx","synthetic_4690002_4690002_18770121_avg4.00216_std1.17768_diagonal_bw4544.61405_skew302.33648_neigh0.41531_cross0.64886_n14.mtx","synthetic_4690002_4690002_18770136_avg4.00216_std1.17836_diagonal_bw4544.58465_skew302.5861_neigh0.41528_cross0.64886_n14.mtx","synthetic_4690002_4690002_18770757_avg4.00229_std1.16774_diagonal_bw4544.58986_skew281.58808_neigh0.41528_cross0.64886_n14.mtx","synthetic_4690002_4690002_18771502_avg4.00245_std1.15845_diagonal_bw4544.58937_skew260.58973_neigh0.41528_cross0.64886_n14.mtx","synthetic_4690002_4690002_18772478_avg4.00266_std1.15153_diagonal_bw4544.58274_skew240.08974_neigh0.41532_cross0.64886_n14.mtx","synthetic_4690002_4690002_18773573_avg4.00289_std1.14442_diagonal_bw4544.5891_skew219.34068_neigh0.4153_cross0.64886_n14.mtx","synthetic_4690002_4690002_18774622_avg4.00312_std1.42675_diagonal_bw4544.59601_skew312.75559_neigh0.41536_cross0.64886_n14.mtx","synthetic_4690002_4690002_18776169_avg4.00345_std1.497_diagonal_bw4544.58594_skew312.72973_neigh0.41537_cross0.64886_n14.mtx","synthetic_4690002_4690002_23458256_avg5.00176_std1.22307_diagonal_bw4355.44656_skew308.6911_neigh0.44325_cross0.64839_n14.mtx","synthetic_4690002_4690002_23458625_avg5.00184_std1.21051_diagonal_bw4355.4414_skew292.09232_neigh0.44326_cross0.64839_n14.mtx","synthetic_4690002_4690002_23459038_avg5.00192_std1.19839_diagonal_bw4355.43174_skew275.29363_neigh0.44326_cross0.64839_n14.mtx","synthetic_4690002_4690002_28147894_avg6.00168_std1.23536_diagonal_bw4776.66629_skew271.09045_neigh0.46196_cross0.64855_n14.mtx","synthetic_473242_473242_1892968_avg4.0_std0.0_diagonal_bw643.09685_skew0.0_neigh0.37802_cross0.99028_n14.mtx","synthetic_47342_47342_5640490_avg119.14347_std31.78033_diagonal_bw4706.12927_skew1.14028_neigh1.79368_cross0.80973_n14.mtx","synthetic_47513_47513_1853007_avg39.0_std0.0_diagonal_bw11035.73908_skew0.0_neigh1.40111_cross0.80716_n14.mtx","synthetic_47684_47684_3704607_avg77.69078_std983.84259_diagonal_bw1484.67635_skew612.76655_neigh1.33477_cross0.83815_n14.mtx","synthetic_47784_47784_2177960_avg45.57927_std31.38054_diagonal_bw1146.70877_skew2.68589_neigh1.21315_cross0.77186_n14.mtx","synthetic_47879_47879_3068257_avg64.08356_std14.01465_diagonal_bw346.42014_skew0.93497_neigh1.583_cross0.83014_n14.mtx","synthetic_48396_48396_2450061_avg50.62528_std27.78261_diagonal_bw7308.68938_skew3.14813_neigh1.63518_cross0.76979_n14.mtx","synthetic_4846335_4846335_19395478_avg4.00209_std1.18059_diagonal_bw4694.1316_skew312.83587_neigh0.41556_cross0.64854_n14.mtx","synthetic_49152_49152_1327104_avg27.0_std0.0_diagonal_bw11307.3231_skew0.0_neigh1.38197_cross0.80419_n14.mtx","synthetic_49152_49152_1474560_avg30.0_std0.0_diagonal_bw11595.80863_skew0.0_neigh1.39239_cross0.80817_n14.mtx","synthetic_49152_49152_1622016_avg33.0_std0.0_diagonal_bw11493.20247_skew0.0_neigh1.39477_cross0.80691_n14.mtx","synthetic_49152_49152_1720320_avg35.0_std0.0_diagonal_bw11394.44574_skew0.0_neigh1.39367_cross0.80647_n14.mtx","synthetic_49152_49152_1867776_avg38.0_std0.0_diagonal_bw11349.18608_skew0.0_neigh1.40287_cross0.8065_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_diagonal_bw10308.18809_skew0.0_neigh1.40356_cross0.80771_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_diagonal_bw11012.95563_skew0.0_neigh1.40324_cross0.80742_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_diagonal_bw11346.284_skew0.0_neigh1.1685_cross0.80302_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_diagonal_bw11390.72874_skew0.0_neigh0.98082_cross0.80701_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_diagonal_bw11391.50515_skew0.0_neigh1.07508_cross0.8087_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_diagonal_bw11399.78432_skew0.0_neigh1.25841_cross0.80863_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_diagonal_bw11414.96842_skew0.0_neigh1.40418_cross0.99013_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_diagonal_bw11418.68166_skew0.0_neigh1.40287_cross0.78307_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_diagonal_bw11422.21698_skew0.0_neigh1.40004_cross0.5727_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_diagonal_bw11424.32371_skew0.0_neigh1.35394_cross0.80634_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_diagonal_bw11425.76263_skew0.0_neigh1.40146_cross0.67683_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_diagonal_bw11434.58527_skew0.0_neigh1.40111_cross0.62362_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_diagonal_bw11435.72972_skew0.0_neigh1.40014_cross0.7295_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_diagonal_bw11437.37392_skew0.0_neigh1.40223_cross0.80454_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_diagonal_bw11439.83738_skew0.0_neigh1.40359_cross0.83214_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_diagonal_bw11448.35966_skew0.0_neigh1.4023_cross0.93723_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_diagonal_bw11458.2605_skew0.0_neigh1.39946_cross0.88672_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_diagonal_bw11460.26939_skew0.0_neigh1.44716_cross0.80489_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_diagonal_bw11518.20732_skew0.0_neigh1.54189_cross0.80766_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_diagonal_bw11741.60107_skew0.0_neigh1.6337_cross0.80807_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_diagonal_bw11774.41903_skew0.0_neigh1.40065_cross0.80951_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_diagonal_bw12457.04529_skew0.0_neigh1.40302_cross0.81002_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_diagonal_bw12492.34208_skew0.0_neigh1.72608_cross0.80667_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_diagonal_bw13195.13298_skew0.0_neigh1.39937_cross0.80881_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_diagonal_bw13899.08816_skew0.0_neigh1.39772_cross0.80832_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_diagonal_bw14638.79781_skew0.0_neigh1.39901_cross0.80937_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_diagonal_bw17395.89585_skew0.0_neigh1.81611_cross0.81026_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_diagonal_bw8111.08537_skew0.0_neigh1.40272_cross0.80591_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_diagonal_bw8847.37626_skew0.0_neigh1.40198_cross0.80857_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_diagonal_bw9576.764_skew0.0_neigh1.40409_cross0.80825_n14.mtx","synthetic_49152_49152_1966080_avg40.0_std0.0_diagonal_bw11481.6866_skew0.0_neigh1.4051_cross0.80332_n14.mtx","synthetic_49152_49152_2113536_avg43.0_std0.0_diagonal_bw11425.25747_skew0.0_neigh1.40405_cross0.80294_n14.mtx","synthetic_49152_49152_2260992_avg46.0_std0.0_diagonal_bw11375.92244_skew0.0_neigh1.407_cross0.80755_n14.mtx","synthetic_49152_49152_2359296_avg48.0_std0.0_diagonal_bw11455.144_skew0.0_neigh1.40962_cross0.8078_n14.mtx","synthetic_49152_49152_2506752_avg51.0_std0.0_diagonal_bw11427.36776_skew0.0_neigh1.41299_cross0.80878_n14.mtx","synthetic_49161_49161_5521770_avg112.32013_std88.82977_diagonal_bw19423.22797_skew3.82549_neigh1.89372_cross0.64143_n14.mtx","synthetic_50367_50367_2296192_avg45.58922_std31.37796_diagonal_bw1208.68088_skew2.68508_neigh1.2113_cross0.76966_n14.mtx","synthetic_50790_50790_1980810_avg39.0_std0.0_diagonal_bw11784.56686_skew0.0_neigh1.40283_cross0.80853_n14.mtx","synthetic_508297_508297_2033188_avg4.0_std0.0_diagonal_bw694.94548_skew0.0_neigh0.36737_cross0.99098_n14.mtx","synthetic_51518_51518_2608832_avg50.63923_std27.762_diagonal_bw7791.12322_skew3.14698_neigh1.63399_cross0.76903_n14.mtx","synthetic_5159002_5159002_20646759_avg4.00208_std1.17597_diagonal_bw4999.05211_skew312.8365_neigh0.41586_cross0.64852_n14.mtx","synthetic_51970_51970_5840112_avg112.37468_std88.844_diagonal_bw20596.85007_skew3.82315_neigh1.89334_cross0.64183_n14.mtx","synthetic_52042_52042_3335826_avg64.09873_std14.0_diagonal_bw375.19427_skew0.93452_neigh1.57868_cross0.8294_n14.mtx","synthetic_52226_52226_4093416_avg78.37889_std1019.1394_diagonal_bw1620.53483_skew665.32736_neigh1.34078_cross0.83814_n14.mtx","synthetic_525825_525825_1577449_avg2.99995_std0.00703_diagonal_bw858.65423_skew2e-05_neigh0.32898_cross0.99111_n14.mtx","synthetic_525825_525825_1577475_avg3.0_std0.0_diagonal_bw858.11898_skew0.0_neigh0.3297_cross0.99112_n14.mtx","synthetic_525825_525825_1583911_avg3.01224_std0.10995_diagonal_bw794.94425_skew0.32792_neigh0.26356_cross0.98852_n14.mtx","synthetic_525825_525825_2046881_avg3.8927_std0.30949_diagonal_bw644.63048_skew0.02756_neigh0.14022_cross0.96875_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_diagonal_bw505.65007_skew0.0_neigh0.37404_cross0.99123_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_diagonal_bw554.71674_skew0.0_neigh0.37192_cross0.99123_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_diagonal_bw606.72183_skew0.0_neigh0.37221_cross0.99122_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_diagonal_bw644.28962_skew0.0_neigh0.37132_cross0.99121_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_diagonal_bw670.65649_skew0.0_neigh0.48388_cross0.99123_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_diagonal_bw683.48363_skew0.0_neigh0.45899_cross0.99121_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_diagonal_bw695.19178_skew0.0_neigh0.43396_cross0.9912_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_diagonal_bw702.15381_skew0.0_neigh0.37156_cross0.99122_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_diagonal_bw705.67664_skew0.0_neigh0.40817_cross0.99123_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_diagonal_bw714.97854_skew0.0_neigh0.38388_cross0.99124_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_diagonal_bw717.19496_skew0.0_neigh0.37708_cross0.95839_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_diagonal_bw718.53995_skew0.0_neigh0.37405_cross0.89258_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_diagonal_bw719.73706_skew0.0_neigh0.37722_cross0.76298_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_diagonal_bw719.84723_skew0.0_neigh0.37147_cross0.99224_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_diagonal_bw720.36711_skew0.0_neigh0.37694_cross0.69678_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_diagonal_bw720.4043_skew0.0_neigh0.37129_cross0.99122_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_diagonal_bw720.62767_skew0.0_neigh0.3775_cross0.8279_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_diagonal_bw726.02105_skew0.0_neigh0.35715_cross0.99121_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_diagonal_bw737.10622_skew0.0_neigh0.37107_cross0.99122_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_diagonal_bw737.13597_skew0.0_neigh0.33298_cross0.9912_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_diagonal_bw744.1915_skew0.0_neigh0.30973_cross0.99122_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_diagonal_bw756.74982_skew0.0_neigh0.28172_cross0.99121_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_diagonal_bw766.27818_skew0.0_neigh0.25988_cross0.99121_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_diagonal_bw792.21338_skew0.0_neigh0.37115_cross0.99122_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_diagonal_bw832.70023_skew0.0_neigh0.371_cross0.9912_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_diagonal_bw890.5806_skew0.0_neigh0.37102_cross0.99121_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_diagonal_bw939.37529_skew0.0_neigh0.37252_cross0.9912_n14.mtx","synthetic_525825_525825_2146618_avg4.08238_std0.27494_diagonal_bw492.86623_skew0.22478_neigh0.18979_cross0.97725_n14.mtx","synthetic_525825_525825_2619767_avg4.9822_std0.13221_diagonal_bw661.36508_skew0.00357_neigh0.31619_cross0.98832_n14.mtx","synthetic_525825_525825_2629125_avg5.0_std0.0_diagonal_bw685.10273_skew0.0_neigh0.39925_cross0.99119_n14.mtx","synthetic_525825_525825_2629143_avg5.00003_std0.00585_diagonal_bw685.04151_skew0.19999_neigh0.39906_cross0.99119_n14.mtx","synthetic_54067_54067_2108613_avg39.0_std0.0_diagonal_bw12526.00168_skew0.0_neigh1.40239_cross0.80938_n14.mtx","synthetic_543352_543352_2173408_avg4.0_std0.0_diagonal_bw737.33837_skew0.0_neigh0.38232_cross0.99144_n14.mtx","synthetic_54640_54640_2767687_avg50.65313_std27.75459_diagonal_bw8285.22183_skew3.14584_neigh1.63356_cross0.77112_n14.mtx","synthetic_5471669_5471669_21898040_avg4.00208_std1.17221_diagonal_bw5301.34866_skew312.83705_neigh0.41566_cross0.6484_n14.mtx","synthetic_54779_54779_6163317_avg112.5124_std88.83086_diagonal_bw21771.99144_skew3.81725_neigh1.89359_cross0.64245_n14.mtx","synthetic_56205_56205_3603711_avg64.11727_std13.99803_diagonal_bw403.30752_skew0.93396_neigh1.57631_cross0.8294_n14.mtx","synthetic_56767_56767_4479663_avg78.91315_std1046.34011_diagonal_bw1754.34908_skew718.36043_neigh1.3421_cross0.83778_n14.mtx","synthetic_57344_57344_2236416_avg39.0_std0.0_diagonal_bw13320.73858_skew0.0_neigh1.39839_cross0.80795_n14.mtx","synthetic_57763_57763_2926741_avg50.66809_std27.74629_diagonal_bw8733.73693_skew3.16436_neigh1.63025_cross0.77273_n14.mtx","synthetic_578407_578407_2313628_avg4.0_std0.0_diagonal_bw786.09292_skew0.0_neigh0.37349_cross0.99188_n14.mtx","synthetic_5784335_5784335_23149312_avg4.00207_std1.16847_diagonal_bw5604.48836_skew312.83761_neigh0.4158_cross0.6484_n14.mtx","synthetic_58333_58333_4204101_avg72.07071_std19.00931_diagonal_bw3296.83076_skew1.13679_neigh1.65316_cross0.77161_n14.mtx","synthetic_60369_60369_3871327_avg64.12773_std13.99319_diagonal_bw432.23492_skew0.93364_neigh1.57393_cross0.82891_n14.mtx","synthetic_603864_603864_13450351_avg22.27381_std30.02405_diagonal_bw145119.2978_skew313.58473_neigh1.25727_cross0.75786_n14.mtx","synthetic_60620_60620_2364180_avg39.0_std0.0_diagonal_bw14073.89657_skew0.0_neigh1.40012_cross0.80617_n14.mtx","synthetic_60885_60885_3084540_avg50.66174_std27.74397_diagonal_bw9209.10073_skew3.16488_neigh1.63202_cross0.77149_n14.mtx","synthetic_6097002_6097002_24400590_avg4.00206_std1.16523_diagonal_bw5909.43124_skew312.83809_neigh0.41578_cross0.64825_n14.mtx","synthetic_61308_61308_4858098_avg79.24085_std1065.00271_diagonal_bw1887.76817_skew772.69186_neigh1.34799_cross0.83808_n14.mtx","synthetic_613462_613462_2453848_avg4.0_std0.0_diagonal_bw842.69066_skew0.0_neigh0.3823_cross0.9922_n14.mtx","synthetic_62451_62451_2802494_avg44.87509_std13.98221_diagonal_bw436.8282_skew1.33983_neigh1.53945_cross0.78907_n14.mtx","synthetic_62451_62451_3069539_avg49.15116_std13.98917_diagonal_bw444.92629_skew1.21765_neigh1.54779_cross0.80067_n14.mtx","synthetic_62451_62451_3336828_avg53.43114_std13.99111_diagonal_bw443.55866_skew1.11487_neigh1.55347_cross0.81011_n14.mtx","synthetic_62451_62451_3603802_avg57.70607_std13.99012_diagonal_bw443.04206_skew1.04485_neigh1.56426_cross0.8182_n14.mtx","synthetic_62451_62451_3870981_avg61.98429_std13.98816_diagonal_bw442.47014_skew0.96824_neigh1.56874_cross0.82542_n14.mtx","synthetic_62451_62451_4003815_avg64.1113_std18.18532_diagonal_bw439.05052_skew1.2149_neigh1.57718_cross0.80166_n14.mtx","synthetic_62451_62451_4004004_avg64.11433_std17.25509_diagonal_bw440.54065_skew1.1524_neigh1.57552_cross0.80816_n14.mtx","synthetic_62451_62451_4004101_avg64.11588_std16.32075_diagonal_bw441.99681_skew1.08997_neigh1.5736_cross0.81519_n14.mtx","synthetic_62451_62451_4004374_avg64.12025_std15.38744_diagonal_bw441.76455_skew1.02744_neigh1.57521_cross0.82106_n14.mtx","synthetic_62451_62451_4004540_avg64.12291_std14.45551_diagonal_bw446.48393_skew0.96498_neigh1.57187_cross0.8254_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_diagonal_bw318.44834_skew0.93375_neigh1.5857_cross0.8319_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_diagonal_bw347.21382_skew0.93375_neigh1.58282_cross0.83047_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_diagonal_bw375.5891_skew0.93375_neigh1.5788_cross0.82967_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_diagonal_bw403.32614_skew0.93375_neigh1.57688_cross0.82914_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_diagonal_bw428.19497_skew0.93375_neigh1.56047_cross0.89117_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_diagonal_bw431.64225_skew0.93375_neigh1.17959_cross0.83144_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_diagonal_bw432.20602_skew0.93375_neigh1.57402_cross0.82868_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_diagonal_bw432.28425_skew0.93375_neigh1.26448_cross0.83122_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_diagonal_bw434.25807_skew0.93375_neigh1.3496_cross0.82903_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_diagonal_bw435.77275_skew0.93375_neigh1.43957_cross0.82958_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_diagonal_bw436.2092_skew0.93375_neigh1.56891_cross0.85105_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_diagonal_bw441.89184_skew0.93375_neigh1.52756_cross0.82773_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_diagonal_bw446.0811_skew0.93375_neigh1.57318_cross0.82708_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_diagonal_bw448.47362_skew0.93375_neigh1.61679_cross0.82621_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_diagonal_bw455.82892_skew0.93375_neigh1.57514_cross0.8052_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_diagonal_bw460.20173_skew0.93375_neigh1.57143_cross0.82713_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_diagonal_bw471.01044_skew0.93375_neigh1.58044_cross0.75754_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_diagonal_bw474.11936_skew0.93375_neigh1.70896_cross0.82572_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_diagonal_bw482.92841_skew0.93375_neigh1.58602_cross0.71177_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_diagonal_bw488.82353_skew0.93375_neigh1.5698_cross0.82576_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_diagonal_bw492.24525_skew0.93375_neigh1.58863_cross0.66769_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_diagonal_bw501.27634_skew0.93375_neigh1.592_cross0.62297_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_diagonal_bw518.7649_skew0.93375_neigh1.56528_cross0.82577_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_diagonal_bw534.51368_skew0.93375_neigh1.80272_cross0.82363_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_diagonal_bw545.14101_skew0.93375_neigh1.56484_cross0.82581_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_diagonal_bw573.04011_skew0.93375_neigh1.56402_cross0.82536_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_diagonal_bw655.89393_skew0.93375_neigh1.90447_cross0.82019_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_diagonal_bw688.45913_skew0.93375_neigh1.89805_cross0.8202_n14.mtx","synthetic_62451_62451_4004768_avg64.12656_std12.59183_diagonal_bw449.42601_skew0.84011_neigh1.56907_cross0.83623_n14.mtx","synthetic_62451_62451_4004839_avg64.1277_std13.52372_diagonal_bw447.77336_skew0.90245_neigh1.56888_cross0.83178_n14.mtx","synthetic_62451_62451_4005045_avg64.131_std11.65916_diagonal_bw451.49143_skew0.77761_neigh1.56649_cross0.83952_n14.mtx","synthetic_62451_62451_4005321_avg64.13542_std10.72592_diagonal_bw453.45016_skew0.71512_neigh1.56305_cross0.84869_n14.mtx","synthetic_62451_62451_4005505_avg64.13836_std9.79513_diagonal_bw456.12735_skew0.65268_neigh1.56103_cross0.85395_n14.mtx","synthetic_62451_62451_4138184_avg66.26289_std13.99067_diagonal_bw443.34518_skew0.90152_neigh1.57487_cross0.83169_n14.mtx","synthetic_62451_62451_4405420_avg70.54202_std13.98985_diagonal_bw444.61993_skew0.85705_neigh1.57908_cross0.83695_n14.mtx","synthetic_62451_62451_4672484_avg74.8184_std13.9889_diagonal_bw448.7849_skew0.80437_neigh1.58559_cross0.8421_n14.mtx","synthetic_62451_62451_4939666_avg79.09667_std13.99063_diagonal_bw451.01291_skew0.75734_neigh1.58874_cross0.84589_n14.mtx","synthetic_62451_62451_5206981_avg83.37706_std13.99111_diagonal_bw451.49187_skew0.7151_neigh1.5926_cross0.8518_n14.mtx","synthetic_63889_63889_4604376_avg72.06837_std18.98718_diagonal_bw3617.55279_skew1.13686_neigh1.65392_cross0.76978_n14.mtx","synthetic_63897_63897_2491983_avg39.0_std0.0_diagonal_bw14829.98433_skew0.0_neigh1.40105_cross0.80703_n14.mtx","synthetic_64532_64532_4137790_avg64.11997_std13.98737_diagonal_bw460.49369_skew0.93387_neigh1.57096_cross0.82708_n14.mtx","synthetic_648517_648517_2594068_avg4.0_std0.0_diagonal_bw886.29581_skew0.0_neigh0.37472_cross0.99258_n14.mtx","synthetic_65850_65850_5226388_avg79.36808_std1075.21227_diagonal_bw2024.76418_skew828.67864_neigh1.34353_cross0.83818_n14.mtx","synthetic_661375_661375_14730847_avg22.27306_std29.96346_diagonal_bw159071.45445_skew313.59526_neigh1.25597_cross0.75815_n14.mtx","synthetic_666542_666542_32559308_avg48.8481_std11.96425_diagonal_bw114344.90623_skew1.12905_neigh1.69253_cross0.80431_n14.mtx","synthetic_68121_68121_3787555_avg55.6004_std1068.24938_diagonal_bw1718.51627_skew856.74559_neigh1.78059_cross0.84932_n14.mtx","synthetic_68121_68121_4147401_avg60.88286_std1069.9982_diagonal_bw1926.13858_skew857.05765_neigh1.62569_cross0.84303_n14.mtx","synthetic_68121_68121_4507272_avg66.16568_std1071.85693_diagonal_bw2010.64137_skew857.28489_neigh1.51921_cross0.84027_n14.mtx","synthetic_68121_68121_4867182_avg71.44907_std1073.8353_diagonal_bw2058.69186_skew857.47165_neigh1.43412_cross0.83891_n14.mtx","synthetic_68121_68121_5227217_avg76.7343_std1075.93952_diagonal_bw2086.64904_skew857.61216_neigh1.36941_cross0.83829_n14.mtx","synthetic_68121_68121_5356033_avg78.62528_std1024.59921_diagonal_bw2111.50808_skew865.40068_neigh1.23427_cross0.83732_n14.mtx","synthetic_68121_68121_5371179_avg78.84762_std1041.23556_diagonal_bw2110.12014_skew862.95755_neigh1.25133_cross0.8377_n14.mtx","synthetic_68121_68121_5385572_avg79.05891_std1056.36091_diagonal_bw2104.37146_skew860.64861_neigh1.27611_cross0.83748_n14.mtx","synthetic_68121_68121_5397801_avg79.23843_std1068.36979_diagonal_bw2101.31551_skew858.6965_neigh1.30166_cross0.83769_n14.mtx","synthetic_68121_68121_5398333_avg79.24624_std1068.28806_diagonal_bw2041.94523_skew601.36298_neigh1.52107_cross0.83991_n14.mtx","synthetic_68121_68121_5400082_avg79.27191_std1070.0369_diagonal_bw2059.09281_skew658.4265_neigh1.46808_cross0.83939_n14.mtx","synthetic_68121_68121_5402340_avg79.30506_std1071.88875_diagonal_bw2075.75188_skew715.38556_neigh1.42245_cross0.83852_n14.mtx","synthetic_68121_68121_5404162_avg79.33181_std1073.85709_diagonal_bw2085.38637_skew772.35942_neigh1.3883_cross0.8379_n14.mtx","synthetic_68121_68121_5405884_avg79.35709_std1075.92117_diagonal_bw2096.53019_skew857.41106_neigh1.32589_cross0.83788_n14.mtx","synthetic_68121_68121_5405960_avg79.3582_std834.61171_diagonal_bw2119.37251_skew857.399_neigh1.16523_cross0.8395_n14.mtx","synthetic_68121_68121_5406317_avg79.36344_std1075.95011_diagonal_bw2092.50296_skew829.23113_neigh1.35417_cross0.83799_n14.mtx","synthetic_68121_68121_5406423_avg79.365_std903.5954_diagonal_bw2114.95495_skew857.32548_neigh1.21021_cross0.83745_n14.mtx","synthetic_68121_68121_5406595_avg79.36752_std972.82101_diagonal_bw2111.91293_skew857.29818_neigh1.25842_cross0.83751_n14.mtx","synthetic_68121_68121_5406754_avg79.36986_std765.8736_diagonal_bw2124.58844_skew857.27294_neigh1.12605_cross0.86678_n14.mtx","synthetic_68121_68121_5407048_avg79.37417_std1042.23736_diagonal_bw2101.43831_skew857.22627_neigh1.31378_cross0.83792_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_diagonal_bw1491.43754_skew857.20278_neigh1.35156_cross0.83798_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_diagonal_bw1624.0197_skew857.20278_neigh1.34962_cross0.8382_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_diagonal_bw1758.02102_skew857.20278_neigh1.3463_cross0.83749_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_diagonal_bw1895.79908_skew857.20278_neigh1.34593_cross0.83806_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_diagonal_bw2025.50456_skew857.20278_neigh1.3433_cross0.83803_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_diagonal_bw2082.54704_skew857.20278_neigh1.34143_cross0.84309_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_diagonal_bw2086.37071_skew857.20278_neigh1.41767_cross0.83887_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_diagonal_bw2086.77817_skew857.20278_neigh1.47784_cross0.83958_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_diagonal_bw2091.20798_skew857.20278_neigh1.38952_cross0.83839_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_diagonal_bw2093.83269_skew857.20278_neigh1.44388_cross0.83897_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_diagonal_bw2093.9416_skew857.20278_neigh1.35558_cross0.83786_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_diagonal_bw2094.95959_skew857.20278_neigh1.29648_cross0.83766_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_diagonal_bw2094.9781_skew857.20278_neigh1.32585_cross0.83766_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_diagonal_bw2095.49089_skew857.20278_neigh1.23838_cross0.83723_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_diagonal_bw2096.11441_skew857.20278_neigh1.27082_cross0.83753_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_diagonal_bw2101.47777_skew857.20278_neigh1.21125_cross0.83652_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_diagonal_bw2157.03003_skew857.20278_neigh1.34041_cross0.83822_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_diagonal_bw2169.50548_skew857.20278_neigh1.34139_cross0.8107_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_diagonal_bw2293.06146_skew857.20278_neigh1.33571_cross0.83819_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_diagonal_bw2300.29781_skew857.20278_neigh1.33945_cross0.75919_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_diagonal_bw2404.30044_skew857.20278_neigh1.34229_cross0.7026_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_diagonal_bw2427.21701_skew857.20278_neigh1.33918_cross0.83764_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_diagonal_bw2485.72726_skew857.20278_neigh1.34463_cross0.6511_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_diagonal_bw2552.48722_skew857.20278_neigh1.34607_cross0.59496_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_diagonal_bw2557.58023_skew857.20278_neigh1.34055_cross0.83779_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_diagonal_bw2689.25582_skew857.20278_neigh1.33474_cross0.83821_n14.mtx","synthetic_68121_68121_5407447_avg79.38003_std1111.80732_diagonal_bw2088.98344_skew857.16294_neigh1.36649_cross0.83852_n14.mtx","synthetic_68121_68121_5407867_avg79.3862_std1181.49691_diagonal_bw2075.59383_skew857.0963_neigh1.43167_cross0.83884_n14.mtx","synthetic_68121_68121_5408409_avg79.39415_std1251.28152_diagonal_bw2053.19145_skew857.0103_neigh1.49371_cross0.83947_n14.mtx","synthetic_68121_68121_5409111_avg79.40446_std1321.14786_diagonal_bw2024.25072_skew856.89895_neigh1.56267_cross0.84086_n14.mtx","synthetic_68121_68121_5409514_avg79.41037_std1391.07361_diagonal_bw1990.12473_skew856.83504_neigh1.64055_cross0.84129_n14.mtx","synthetic_68121_68121_5584961_avg81.98589_std1075.92069_diagonal_bw2103.42705_skew829.88685_neigh1.31388_cross0.83785_n14.mtx","synthetic_68121_68121_5934842_avg87.12206_std1068.36291_diagonal_bw2115.43957_skew780.90298_neigh1.27178_cross0.83756_n14.mtx","synthetic_68121_68121_6280428_avg92.19518_std1056.3285_diagonal_bw2124.28281_skew737.87809_neigh1.23335_cross0.83743_n14.mtx","synthetic_68121_68121_6623921_avg97.23758_std1041.17147_diagonal_bw2129.92795_skew699.5625_neigh1.20508_cross0.83709_n14.mtx","synthetic_68121_68121_6966795_avg102.27089_std1024.4946_diagonal_bw2136.61749_skew665.08399_neigh1.18226_cross0.8378_n14.mtx","synthetic_683572_683572_2734288_avg4.0_std0.0_diagonal_bw924.57479_skew0.0_neigh0.37964_cross0.9929_n14.mtx","synthetic_68696_68696_4403097_avg64.09539_std13.99668_diagonal_bw488.31048_skew0.93462_neigh1.5701_cross0.82639_n14.mtx","synthetic_690692_690692_50143771_avg72.59932_std15.84383_diagonal_bw10332.89385_skew0.96972_neigh1.69604_cross0.81876_n14.mtx","synthetic_69445_69445_5001284_avg72.01791_std19.00309_diagonal_bw3910.75892_skew1.13836_neigh1.65289_cross0.77119_n14.mtx","synthetic_700003_700003_2220092_avg3.17155_std25.65674_diagonal_bw84360.35744_skew1481.87283_neigh0.14269_cross0.90247_n14.mtx","synthetic_70391_70391_5585080_avg79.34367_std1076.52341_diagonal_bw2157.78507_skew858.03517_neigh1.33819_cross0.83827_n14.mtx","synthetic_718886_718886_16011537_avg22.27271_std29.91357_diagonal_bw172575.43963_skew313.60029_neigh1.25733_cross0.76047_n14.mtx","synthetic_72859_72859_4668546_avg64.07645_std14.00516_diagonal_bw518.58045_skew0.93519_neigh1.56556_cross0.82561_n14.mtx","synthetic_730022_730022_35670999_avg48.86291_std11.95289_diagonal_bw125143.89165_skew1.14887_neigh1.69318_cross0.80372_n14.mtx","synthetic_74933_74933_5942603_avg79.30555_std1075.58283_diagonal_bw2295.08486_skew858.44801_neigh1.33914_cross0.83797_n14.mtx","synthetic_75000_75000_5399893_avg71.99857_std19.01052_diagonal_bw4226.28011_skew1.13893_neigh1.65498_cross0.76971_n14.mtx","synthetic_756472_756472_54943929_avg72.63181_std15.80449_diagonal_bw11306.62304_skew1.03767_neigh1.69544_cross0.81974_n14.mtx","synthetic_766670_766670_2431595_avg3.17163_std25.62988_diagonal_bw92517.8455_skew1481.83288_neigh0.14291_cross0.90344_n14.mtx","synthetic_77022_77022_4936326_avg64.08982_std14.01488_diagonal_bw546.71648_skew0.93478_neigh1.56521_cross0.82414_n14.mtx","synthetic_776397_776397_17292444_avg22.27268_std29.86978_diagonal_bw186487.71815_skew313.60063_neigh1.25606_cross0.75904_n14.mtx","synthetic_793502_793502_38791634_avg48.88662_std11.94806_diagonal_bw136182.3121_skew1.39329_neigh1.69181_cross0.80518_n14.mtx","synthetic_79474_79474_6302859_avg79.30718_std1074.75192_diagonal_bw2429.79547_skew858.43036_neigh1.33761_cross0.83764_n14.mtx","synthetic_80556_80556_5802157_avg72.02638_std19.02772_diagonal_bw4548.81679_skew1.13811_neigh1.65123_cross0.77098_n14.mtx","synthetic_81186_81186_5203811_avg64.09739_std14.01954_diagonal_bw573.41229_skew0.93456_neigh1.56394_cross0.82507_n14.mtx","synthetic_822252_822252_59710654_avg72.61844_std15.79674_diagonal_bw12289.27629_skew1.02428_neigh1.69505_cross0.82015_n14.mtx","synthetic_833337_833337_2642605_avg3.17111_std25.60679_diagonal_bw100178.12772_skew1482.07595_neigh0.14558_cross0.90327_n14.mtx","synthetic_83334_83334_4201657_avg50.41948_std18.95017_diagonal_bw4709.6572_skew1.61804_neigh1.63244_cross0.73261_n14.mtx","synthetic_83334_83334_4601152_avg55.21338_std18.98799_diagonal_bw4646.03415_skew1.48128_neigh1.63903_cross0.74386_n14.mtx","synthetic_83334_83334_5001407_avg60.0164_std19.00589_diagonal_bw4595.50832_skew1.36602_neigh1.64466_cross0.7532_n14.mtx","synthetic_83334_83334_5402033_avg64.82388_std19.01381_diagonal_bw4698.8203_skew1.25226_neigh1.64878_cross0.75926_n14.mtx","synthetic_83334_83334_5802662_avg69.63139_std19.01513_diagonal_bw4668.44682_skew1.16856_neigh1.65019_cross0.76698_n14.mtx","synthetic_83334_83334_6001725_avg72.02012_std24.68331_diagonal_bw4597.09978_skew1.47153_neigh1.65763_cross0.74295_n14.mtx","synthetic_83334_83334_6001767_avg72.02063_std23.43414_diagonal_bw4605.04571_skew1.40209_neigh1.65672_cross0.74782_n14.mtx","synthetic_83334_83334_6001894_avg72.02215_std22.17621_diagonal_bw4646.80877_skew1.31873_neigh1.65465_cross0.75393_n14.mtx","synthetic_83334_83334_6002259_avg72.02653_std20.9136_diagonal_bw4656.17762_skew1.24917_neigh1.65449_cross0.76067_n14.mtx","synthetic_83334_83334_6002747_avg72.03239_std19.64906_diagonal_bw4673.90798_skew1.16569_neigh1.65312_cross0.76886_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_diagonal_bw3305.20832_skew1.13786_neigh1.65315_cross0.77231_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_diagonal_bw3620.90018_skew1.13786_neigh1.65349_cross0.77196_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_diagonal_bw3937.58802_skew1.13786_neigh1.65156_cross0.77221_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_diagonal_bw4166.4266_skew1.13786_neigh1.61515_cross0.87118_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_diagonal_bw4233.14868_skew1.13786_neigh1.65482_cross0.76951_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_diagonal_bw4319.26886_skew1.13786_neigh1.63126_cross0.84629_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_diagonal_bw4526.66129_skew1.13786_neigh1.27337_cross0.77072_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_diagonal_bw4530.37209_skew1.13786_neigh1.16582_cross0.76816_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_diagonal_bw4534.10717_skew1.13786_neigh1.38187_cross0.76916_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_diagonal_bw4547.8539_skew1.13786_neigh1.65373_cross0.7709_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_diagonal_bw4579.65029_skew1.13786_neigh1.48964_cross0.77146_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_diagonal_bw4585.62608_skew1.13786_neigh1.65019_cross0.79589_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_diagonal_bw4637.47048_skew1.13786_neigh1.59841_cross0.77139_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_diagonal_bw4689.47064_skew1.13786_neigh1.65378_cross0.77101_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_diagonal_bw4780.79023_skew1.13786_neigh1.65879_cross0.74352_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_diagonal_bw4789.06709_skew1.13786_neigh1.70747_cross0.77176_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_diagonal_bw4857.24227_skew1.13786_neigh1.65353_cross0.76955_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_diagonal_bw4925.35613_skew1.13786_neigh1.6643_cross0.6948_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_diagonal_bw5058.46251_skew1.13786_neigh1.66965_cross0.64539_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_diagonal_bw5147.50469_skew1.13786_neigh1.67236_cross0.59361_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_diagonal_bw5154.62121_skew1.13786_neigh1.65272_cross0.77108_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_diagonal_bw5233.28398_skew1.13786_neigh1.67487_cross0.54727_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_diagonal_bw5476.98466_skew1.13786_neigh1.65336_cross0.76948_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_diagonal_bw5507.02461_skew1.13786_neigh1.81577_cross0.77147_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_diagonal_bw5678.54487_skew1.13786_neigh1.92809_cross0.77038_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_diagonal_bw5771.62843_skew1.13786_neigh1.65209_cross0.77094_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_diagonal_bw5838.52381_skew1.13786_neigh1.92627_cross0.77149_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_diagonal_bw6077.07144_skew1.13786_neigh1.65366_cross0.76991_n14.mtx","synthetic_83334_83334_6003209_avg72.03793_std18.38192_diagonal_bw4707.48488_skew1.09612_neigh1.65203_cross0.77498_n14.mtx","synthetic_83334_83334_6003696_avg72.04378_std17.11472_diagonal_bw4728.66163_skew1.01267_neigh1.65174_cross0.7812_n14.mtx","synthetic_83334_83334_6004305_avg72.05108_std15.84803_diagonal_bw4777.9355_skew0.94307_neigh1.65015_cross0.78619_n14.mtx","synthetic_83334_83334_6004738_avg72.05628_std14.57979_diagonal_bw4790.14851_skew0.87354_neigh1.64947_cross0.79306_n14.mtx","synthetic_83334_83334_6005175_avg72.06152_std13.31259_diagonal_bw4831.11382_skew0.79014_neigh1.64887_cross0.7998_n14.mtx","synthetic_83334_83334_6203256_avg74.43848_std19.01659_diagonal_bw4654.6223_skew1.09569_neigh1.65385_cross0.77339_n14.mtx","synthetic_83334_83334_6603858_avg79.24566_std19.01545_diagonal_bw4699.25025_skew1.03166_neigh1.65768_cross0.78094_n14.mtx","synthetic_83334_83334_7004670_avg84.05537_std19.01725_diagonal_bw4679.09844_skew0.97489_neigh1.65994_cross0.78535_n14.mtx","synthetic_83334_83334_7405486_avg88.86512_std19.01689_diagonal_bw4672.23001_skew0.91301_neigh1.66199_cross0.79152_n14.mtx","synthetic_83334_83334_7806125_avg93.67275_std19.01502_diagonal_bw4727.18715_skew0.86821_neigh1.66453_cross0.79468_n14.mtx","synthetic_833908_833908_18572841_avg22.27205_std29.833_diagonal_bw200221.84865_skew313.60956_neigh1.2563_cross0.76034_n14.mtx","synthetic_84015_84015_6661114_avg79.28482_std1074.00684_diagonal_bw2558.388_skew858.67278_neigh1.33638_cross0.83771_n14.mtx","synthetic_84834_84834_1835279_avg21.63377_std13.7861_diagonal_bw39708.14271_skew4.03842_neigh1.00596_cross0.57165_n14.mtx","synthetic_856982_856982_41861338_avg48.84739_std11.94057_diagonal_bw146875.5352_skew1.3338_neigh1.69178_cross0.8051_n14.mtx","synthetic_86111_86111_6203103_avg72.03613_std19.01524_diagonal_bw4859.13722_skew1.13782_neigh1.65351_cross0.76961_n14.mtx","synthetic_862664_862664_13443097_avg15.58324_std29.57178_diagonal_bw205914.8262_skew313.76132_neigh1.19467_cross0.75677_n14.mtx","synthetic_862664_862664_14783705_avg17.13727_std29.61764_diagonal_bw200544.4475_skew312.46885_neigh1.21062_cross0.75788_n14.mtx","synthetic_862664_862664_15984607_avg18.52935_std29.67134_diagonal_bw204031.16331_skew314.12161_neigh1.21772_cross0.75849_n14.mtx","synthetic_862664_862664_17373620_avg20.1395_std29.72434_diagonal_bw201184.47121_skew312.11605_neigh1.23462_cross0.75917_n14.mtx","synthetic_862664_862664_18524582_avg21.47369_std29.78706_diagonal_bw202931.99997_skew314.40918_neigh1.23555_cross0.75791_n14.mtx","synthetic_862664_862664_19089435_avg22.12847_std21.22646_diagonal_bw207932.34557_skew315.65089_neigh1.26294_cross0.76006_n14.mtx","synthetic_862664_862664_19114072_avg22.15703_std23.12053_diagonal_bw207837.85303_skew315.24275_neigh1.26195_cross0.76014_n14.mtx","synthetic_862664_862664_19142097_avg22.18952_std25.02501_diagonal_bw207559.16437_skew314.77975_neigh1.2614_cross0.76033_n14.mtx","synthetic_862664_862664_19162894_avg22.21362_std30.15004_diagonal_bw207436.8044_skew408.47843_neigh1.26104_cross0.76006_n14.mtx","synthetic_862664_862664_19171989_avg22.22417_std26.93718_diagonal_bw207418.24466_skew314.2874_neigh1.26045_cross0.75991_n14.mtx","synthetic_862664_862664_19173147_avg22.22551_std30.06905_diagonal_bw207404.3214_skew387.38255_neigh1.26037_cross0.75992_n14.mtx","synthetic_862664_862664_19184120_avg22.23823_std29.99101_diagonal_bw207352.46946_skew366.25046_neigh1.25968_cross0.76002_n14.mtx","synthetic_862664_862664_19195719_avg22.25168_std29.91773_diagonal_bw207364.02997_skew345.17619_neigh1.25896_cross0.7599_n14.mtx","synthetic_862664_862664_19200670_avg22.25741_std28.85544_diagonal_bw207358.48533_skew313.81644_neigh1.25804_cross0.75992_n14.mtx","synthetic_862664_862664_19207356_avg22.26516_std29.84926_diagonal_bw207298.49088_skew324.12672_neigh1.25745_cross0.75965_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_diagonal_bw147175.94882_skew313.61244_neigh1.25927_cross0.75819_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_diagonal_bw160722.70732_skew313.61244_neigh1.25886_cross0.75853_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_diagonal_bw174086.80059_skew313.61244_neigh1.2581_cross0.75845_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_diagonal_bw187595.73649_skew313.61244_neigh1.25764_cross0.75905_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_diagonal_bw197107.06742_skew313.61244_neigh1.15768_cross0.98285_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_diagonal_bw200512.30622_skew313.61244_neigh1.25755_cross0.75937_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_diagonal_bw201450.72793_skew313.61244_neigh0.87919_cross0.75907_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_diagonal_bw201648.55655_skew313.61244_neigh0.96351_cross0.75931_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_diagonal_bw202106.02353_skew313.61244_neigh1.04694_cross0.75957_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_diagonal_bw202474.16303_skew313.61244_neigh1.13071_cross0.75965_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_diagonal_bw202922.36711_skew313.61244_neigh1.21508_cross0.75942_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_diagonal_bw203998.92204_skew313.61244_neigh1.29803_cross0.759_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_diagonal_bw204574.41148_skew313.61244_neigh1.23281_cross0.9336_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_diagonal_bw205344.97895_skew313.61244_neigh1.38267_cross0.76047_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_diagonal_bw205907.14271_skew313.61244_neigh1.24839_cross0.88389_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_diagonal_bw206626.31443_skew313.61244_neigh1.2532_cross0.83425_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_diagonal_bw206974.26599_skew313.61244_neigh1.25565_cross0.78408_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_diagonal_bw207389.25109_skew313.61244_neigh1.25761_cross0.73493_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_diagonal_bw207439.23549_skew313.61244_neigh1.46644_cross0.76053_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_diagonal_bw207459.39564_skew313.61244_neigh1.25937_cross0.68571_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_diagonal_bw207488.93911_skew313.61244_neigh1.25981_cross0.63655_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_diagonal_bw207554.94279_skew313.61244_neigh1.26046_cross0.587_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_diagonal_bw207595.91908_skew313.61244_neigh1.2611_cross0.53735_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_diagonal_bw212590.02941_skew313.61244_neigh1.54974_cross0.76249_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_diagonal_bw213723.771_skew313.61244_neigh1.25664_cross0.75921_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_diagonal_bw225205.18654_skew313.61244_neigh1.63328_cross0.76333_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_diagonal_bw226779.75408_skew313.61244_neigh1.25626_cross0.75971_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_diagonal_bw239405.59411_skew313.61244_neigh1.25615_cross0.75997_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_diagonal_bw252280.20232_skew313.61244_neigh1.2553_cross0.76024_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_diagonal_bw264965.46748_skew313.61244_neigh1.25507_cross0.76101_n14.mtx","synthetic_862664_862664_19218664_avg22.27827_std29.78506_diagonal_bw207102.49027_skew303.10795_neigh1.25635_cross0.75926_n14.mtx","synthetic_862664_862664_19223712_avg22.28412_std30.77861_diagonal_bw207046.68591_skew313.4391_neigh1.25463_cross0.75893_n14.mtx","synthetic_862664_862664_19229090_avg22.29036_std29.72467_diagonal_bw206867.28901_skew282.12689_neigh1.25356_cross0.759_n14.mtx","synthetic_862664_862664_19237224_avg22.29979_std29.6692_diagonal_bw206653.68064_skew261.15496_neigh1.25095_cross0.7591_n14.mtx","synthetic_862664_862664_19238563_avg22.30134_std32.70575_diagonal_bw206629.43331_skew313.19637_neigh1.25007_cross0.75903_n14.mtx","synthetic_862664_862664_19242590_avg22.30601_std29.61737_diagonal_bw206307.07674_skew240.28029_neigh1.2466_cross0.75831_n14.mtx","synthetic_862664_862664_19243461_avg22.30702_std38.50483_diagonal_bw204657.66387_skew313.1164_neigh1.23352_cross0.75708_n14.mtx","synthetic_862664_862664_19244915_avg22.3087_std29.57096_diagonal_bw205734.95503_skew219.45208_neigh1.24265_cross0.75782_n14.mtx","synthetic_862664_862664_19245171_avg22.309_std34.6357_diagonal_bw205994.25147_skew313.08849_neigh1.24488_cross0.75815_n14.mtx","synthetic_862664_862664_19245832_avg22.30977_std36.57002_diagonal_bw205514.46748_skew313.0777_neigh1.23854_cross0.7575_n14.mtx","synthetic_862664_862664_19963054_avg23.14117_std29.84969_diagonal_bw201699.12835_skew311.90553_neigh1.2515_cross0.75854_n14.mtx","synthetic_862664_862664_21064571_avg24.41805_std29.92055_diagonal_bw202624.20371_skew314.66815_neigh1.24936_cross0.75798_n14.mtx","synthetic_862664_862664_22549917_avg26.13986_std29.9915_diagonal_bw201802.77435_skew311.74076_neigh1.26527_cross0.75886_n14.mtx","synthetic_862664_862664_23605495_avg27.36349_std30.07249_diagonal_bw202516.58382_skew314.82232_neigh1.2598_cross0.75787_n14.mtx","synthetic_862664_862664_25134553_avg29.13597_std30.15077_diagonal_bw202078.38456_skew311.6376_neigh1.27289_cross0.7586_n14.mtx","synthetic_88557_88557_7019496_avg79.26529_std1073.34158_diagonal_bw2695.50047_skew858.88461_neigh1.33448_cross0.83815_n14.mtx","synthetic_888032_888032_64506105_avg72.63939_std15.82947_diagonal_bw13252.2913_skew1.00993_neigh1.69563_cross0.81974_n14.mtx","synthetic_891419_891419_19853723_avg22.27204_std29.80105_diagonal_bw214143.5371_skew313.60965_neigh1.25649_cross0.76024_n14.mtx","synthetic_900004_900004_2853888_avg3.17097_std25.58712_diagonal_bw108539.4849_skew1482.14118_neigh0.14041_cross0.90359_n14.mtx","synthetic_91667_91667_6603697_avg72.04007_std19.01165_diagonal_bw5166.06684_skew1.19322_neigh1.65306_cross0.77077_n14.mtx","synthetic_920462_920462_44975685_avg48.86208_std11.94475_diagonal_bw157813.69929_skew1.2103_neigh1.69158_cross0.8057_n14.mtx","synthetic_92913_92913_2009995_avg21.63309_std13.78454_diagonal_bw43437.75323_skew4.03858_neigh1.00444_cross0.57612_n14.mtx","synthetic_948930_948930_21133875_avg22.27127_std29.77202_diagonal_bw227994.28681_skew313.6206_neigh1.25605_cross0.76054_n14.mtx","synthetic_952203_952203_32554296_avg34.1884_std11.93673_diagonal_bw195297.92349_skew1.69097_neigh1.66003_cross0.77069_n14.mtx","synthetic_952203_952203_35650826_avg37.44036_std11.95176_diagonal_bw168585.46646_skew1.53737_neigh1.6689_cross0.78029_n14.mtx","synthetic_952203_952203_38750463_avg40.69559_std11.9572_diagonal_bw172420.33981_skew1.4327_neigh1.67668_cross0.78884_n14.mtx","synthetic_952203_952203_41851500_avg43.95229_std11.95993_diagonal_bw175566.75547_skew1.3207_neigh1.68288_cross0.79655_n14.mtx","synthetic_952203_952203_44952640_avg47.20909_std11.96013_diagonal_bw161847.45785_skew1.22415_neigh1.68913_cross0.80243_n14.mtx","synthetic_952203_952203_46500626_avg48.83478_std13.95044_diagonal_bw161361.82335_skew1.37536_neigh1.696_cross0.7901_n14.mtx","synthetic_952203_952203_46500963_avg48.83514_std14.74428_diagonal_bw160687.29213_skew1.45725_neigh1.69741_cross0.78406_n14.mtx","synthetic_952203_952203_46501138_avg48.83532_std15.536_diagonal_bw160058.71086_skew1.53915_neigh1.6982_cross0.77843_n14.mtx","synthetic_952203_952203_46502122_avg48.83635_std13.15524_diagonal_bw162100.43586_skew1.31385_neigh1.69445_cross0.79612_n14.mtx","synthetic_952203_952203_46502884_avg48.83715_std12.35905_diagonal_bw162891.6186_skew1.23191_neigh1.69248_cross0.80224_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_diagonal_bw116106.75277_skew1.19093_neigh1.69117_cross0.80488_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_diagonal_bw126698.85453_skew1.19093_neigh1.69122_cross0.80482_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_diagonal_bw137306.6029_skew1.19093_neigh1.69135_cross0.80527_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_diagonal_bw143715.18864_skew1.19093_neigh1.18395_cross0.80203_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_diagonal_bw144653.44632_skew1.19093_neigh1.29656_cross0.80259_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_diagonal_bw146143.79431_skew1.19093_neigh1.40968_cross0.80337_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_diagonal_bw147733.42467_skew1.19093_neigh1.69116_cross0.8053_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_diagonal_bw149667.71742_skew1.19093_neigh1.52206_cross0.80376_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_diagonal_bw150291.80563_skew1.19093_neigh1.61648_cross0.88662_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_diagonal_bw150742.69167_skew1.19093_neigh1.62479_cross0.88374_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_diagonal_bw158141.44775_skew1.19093_neigh1.69161_cross0.80521_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_diagonal_bw158785.70138_skew1.19093_neigh1.63527_cross0.80486_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_diagonal_bw159871.29872_skew1.19093_neigh1.68017_cross0.83168_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_diagonal_bw163210.79241_skew1.19093_neigh1.69195_cross0.80513_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_diagonal_bw165912.04361_skew1.19093_neigh1.70043_cross0.77937_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_diagonal_bw168260.36222_skew1.19093_neigh1.69196_cross0.80515_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_diagonal_bw170143.42422_skew1.19093_neigh1.71296_cross0.72708_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_diagonal_bw170449.52862_skew1.19093_neigh1.74831_cross0.80609_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_diagonal_bw173244.29895_skew1.19093_neigh1.72242_cross0.67391_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_diagonal_bw175577.55656_skew1.19093_neigh1.72872_cross0.62205_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_diagonal_bw177436.06958_skew1.19093_neigh1.73407_cross0.56915_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_diagonal_bw178609.376_skew1.19093_neigh1.69178_cross0.80552_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_diagonal_bw188630.00924_skew1.19093_neigh1.69167_cross0.80566_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_diagonal_bw196253.24669_skew1.19093_neigh1.88306_cross0.80917_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_diagonal_bw198626.45229_skew1.19093_neigh1.6924_cross0.80549_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_diagonal_bw208660.18746_skew1.19093_neigh1.69223_cross0.80594_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_diagonal_bw224378.247_skew1.19093_neigh1.86131_cross0.80899_n14.mtx","synthetic_952203_952203_46504635_avg48.83899_std11.56217_diagonal_bw163741.45383_skew1.14992_neigh1.6908_cross0.80838_n14.mtx","synthetic_952203_952203_46504874_avg48.83924_std10.76526_diagonal_bw164534.32716_skew1.06801_neigh1.68826_cross0.81515_n14.mtx","synthetic_952203_952203_46506617_avg48.84107_std9.96939_diagonal_bw165495.48128_skew0.98603_neigh1.68708_cross0.82079_n14.mtx","synthetic_952203_952203_46507502_avg48.842_std9.17223_diagonal_bw166478.11101_skew0.9041_neigh1.68492_cross0.82769_n14.mtx","synthetic_952203_952203_46508902_avg48.84347_std8.37556_diagonal_bw167919.97397_skew0.82215_neigh1.68255_cross0.83397_n14.mtx","synthetic_952203_952203_48054277_avg50.46642_std11.96125_diagonal_bw164498.00225_skew1.14004_neigh1.69477_cross0.8085_n14.mtx","synthetic_952203_952203_51155719_avg53.72354_std11.96081_diagonal_bw166947.78914_skew1.08475_neigh1.69857_cross0.81333_n14.mtx","synthetic_952203_952203_54257059_avg56.98056_std11.96124_diagonal_bw158641.62561_skew1.01823_neigh1.70257_cross0.81769_n14.mtx","synthetic_952203_952203_57358241_avg60.23741_std11.96072_diagonal_bw160780.13898_skew0.95892_neigh1.70685_cross0.82181_n14.mtx","synthetic_952203_952203_60460102_avg63.49497_std11.9616_diagonal_bw162722.07884_skew0.90566_neigh1.71034_cross0.82524_n14.mtx","synthetic_953812_953812_69251992_avg72.60549_std15.82731_diagonal_bw14247.53739_skew1.05219_neigh1.69481_cross0.82031_n14.mtx","synthetic_966671_966671_3064966_avg3.17064_std25.57082_diagonal_bw116665.94473_skew1482.29662_neigh0.14335_cross0.9042_n14.mtx","synthetic_97223_97223_7006972_avg72.07114_std19.01384_diagonal_bw5487.10435_skew1.19228_neigh1.65371_cross0.7705_n14.mtx","synthetic_983943_983943_48052837_avg48.83701_std11.95054_diagonal_bw168639.17559_skew1.12953_neigh1.69189_cross0.80596_n14.mtx","synthetic_98611_98611_5466825_avg55.43829_std11.03861_diagonal_bw3819.4861_skew0.894_neigh1.63102_cross0.78874_n14.mtx","synthetic_986703_986703_50142919_avg50.81865_std15.80386_diagonal_bw14822.43122_skew1.44005_neigh1.67248_cross0.78706_n14.mtx","synthetic_986703_986703_54918080_avg55.65817_std15.81077_diagonal_bw15138.82145_skew1.31772_neigh1.67905_cross0.79644_n14.mtx","synthetic_986703_986703_59694812_avg60.49927_std15.81312_diagonal_bw14664.0448_skew1.2149_neigh1.68455_cross0.80453_n14.mtx","synthetic_986703_986703_64472911_avg65.34176_std15.81391_diagonal_bw14414.58333_skew1.11197_neigh1.68893_cross0.81154_n14.mtx","synthetic_986703_986703_69250817_avg70.18405_std15.81417_diagonal_bw14641.24259_skew1.0375_neigh1.69312_cross0.81698_n14.mtx","synthetic_986703_986703_71632861_avg72.5982_std20.55272_diagonal_bw14466.90673_skew1.31411_neigh1.70018_cross0.79544_n14.mtx","synthetic_986703_986703_71633918_avg72.59927_std19.50045_diagonal_bw14532.04367_skew1.2452_neigh1.69963_cross0.80057_n14.mtx","synthetic_986703_986703_71635339_avg72.60071_std18.448_diagonal_bw14581.49631_skew1.17629_neigh1.69833_cross0.80629_n14.mtx","synthetic_986703_986703_71637178_avg72.60257_std17.39427_diagonal_bw14634.97505_skew1.10736_neigh1.69727_cross0.81167_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_diagonal_bw10335.21324_skew1.0109_neigh1.69533_cross0.82034_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_diagonal_bw11318.36353_skew1.0109_neigh1.69541_cross0.82033_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_diagonal_bw12275.00965_skew1.0109_neigh1.69568_cross0.82032_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_diagonal_bw13203.20771_skew1.0109_neigh1.63897_cross0.89577_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_diagonal_bw13249.95843_skew1.0109_neigh1.69575_cross0.82075_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_diagonal_bw13805.02825_skew1.0109_neigh1.19003_cross0.81894_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_diagonal_bw13844.57814_skew1.0109_neigh1.30228_cross0.81902_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_diagonal_bw13882.13942_skew1.0109_neigh1.41423_cross0.81952_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_diagonal_bw13951.55398_skew1.0109_neigh1.52662_cross0.82014_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_diagonal_bw14238.03579_skew1.0109_neigh1.69538_cross0.82019_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_diagonal_bw14385.45717_skew1.0109_neigh1.63958_cross0.82034_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_diagonal_bw14735.70059_skew1.0109_neigh1.69514_cross0.82041_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_diagonal_bw15212.0391_skew1.0109_neigh1.6958_cross0.82068_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_diagonal_bw15394.12525_skew1.0109_neigh1.75112_cross0.82054_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_diagonal_bw15634.92651_skew1.0109_neigh1.71345_cross0.73894_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_diagonal_bw16040.9703_skew1.0109_neigh1.72027_cross0.68529_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_diagonal_bw16190.01108_skew1.0109_neigh1.69515_cross0.82012_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_diagonal_bw16364.90593_skew1.0109_neigh1.72531_cross0.63012_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_diagonal_bw16621.34957_skew1.0109_neigh1.72913_cross0.57601_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_diagonal_bw17168.96391_skew1.0109_neigh1.69549_cross0.82015_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_diagonal_bw18143.13335_skew1.0109_neigh1.69549_cross0.82027_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_diagonal_bw19086.21946_skew1.0109_neigh1.69561_cross0.82051_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_diagonal_bw20302.1187_skew1.0109_neigh1.91272_cross0.82094_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_diagonal_bw27082.55582_skew1.0109_neigh1.86325_cross0.82124_n14.mtx","synthetic_986703_986703_71640715_avg72.60616_std15.28673_diagonal_bw14762.27652_skew0.96953_neigh1.69488_cross0.82275_n14.mtx","synthetic_986703_986703_71642304_avg72.60777_std14.23366_diagonal_bw14828.49584_skew0.90062_neigh1.6931_cross0.829_n14.mtx","synthetic_986703_986703_71644194_avg72.60968_std13.17962_diagonal_bw14914.32275_skew0.84548_neigh1.6914_cross0.83495_n14.mtx","synthetic_986703_986703_71646204_avg72.61172_std12.12567_diagonal_bw15020.25928_skew0.77657_neigh1.68947_cross0.84053_n14.mtx","synthetic_986703_986703_71647231_avg72.61276_std11.07215_diagonal_bw15141.16106_skew0.70769_neigh1.68759_cross0.84656_n14.mtx","synthetic_986703_986703_74029078_avg75.02671_std15.81428_diagonal_bw14414.50158_skew0.97263_neigh1.69708_cross0.82306_n14.mtx","synthetic_986703_986703_78806613_avg79.86863_std15.81421_diagonal_bw14604.1321_skew0.91565_neigh1.70019_cross0.82763_n14.mtx","synthetic_986703_986703_83583783_avg84.71017_std15.81435_diagonal_bw14458.40041_skew0.86518_neigh1.70355_cross0.83196_n14.mtx","synthetic_986703_986703_88361224_avg89.552_std15.81425_diagonal_bw14614.23314_skew0.82017_neigh1.70575_cross0.83603_n14.mtx","synthetic_986703_986703_93139124_avg94.39428_std15.81414_diagonal_bw14500.66288_skew0.76917_neigh1.70841_cross0.8393_n14.mtx","synthetic_4690002_4690002_14082483_avg3.00266_std1.15153_diagonal_bw5391.75085_skew319.7156_neigh0.3689_cross0.64826_n14.mtx","synthetic_4690002_4690002_18770459_avg4.00223_std1.21745_diagonal_bw4544.58663_skew312.82517_neigh0.41529_cross0.64886_n14.mtx","synthetic_4690002_4690002_18770794_avg4.0023_std1.16864_diagonal_bw4544.54744_skew281.83738_neigh0.4153_cross0.64886_n14.mtx","synthetic_4690002_4690002_18771567_avg4.00246_std1.15985_diagonal_bw4544.56125_skew260.83867_neigh0.4153_cross0.64886_n14.mtx","synthetic_4690002_4690002_18771765_avg4.00251_std1.28647_diagonal_bw4544.58181_skew312.80334_neigh0.41533_cross0.64886_n14.mtx","synthetic_4690002_4690002_18773155_avg4.0028_std1.35648_diagonal_bw4544.56842_skew312.7801_neigh0.41535_cross0.64886_n14.mtx","synthetic_986703_986703_71638484_avg72.6039_std16.3415_diagonal_bw14690.20318_skew1.03846_neigh1.69598_cross0.81731_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_diagonal_bw14278.82315_skew1.0109_neigh1.68517_cross0.84736_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_diagonal_bw15076.26174_skew1.0109_neigh1.70327_cross0.79318_n14.mtx",]
mikri=["synthetic_1000005_1000005_2190550_avg2.19054_std25.45314_random_bw119819.1947_skew1501.82644_neigh0.0864_cross0.84356_n14.mtx","synthetic_1000005_1000005_2378888_avg2.37888_std25.47359_random_bw127104.78039_skew1514.4215_neigh0.103_cross0.88018_n14.mtx","synthetic_1000005_1000005_2551748_avg2.55174_std25.499_random_bw119896.54265_skew1534.81764_neigh0.1086_cross0.87541_n14.mtx","synthetic_1000005_1000005_2793946_avg2.79393_std25.52481_random_bw139709.49965_skew1513.71115_neigh0.10178_cross0.85786_n14.mtx","synthetic_1000005_1000005_3068666_avg3.06865_std25.54963_random_bw116338.41941_skew1480.43289_neigh0.11916_cross0.88158_n14.mtx","synthetic_1000005_1000005_3110917_avg3.1109_std33.11754_random_bw103532.18733_skew1510.459_neigh0.11814_cross0.85466_n14.mtx","synthetic_1000005_1000005_3116965_avg3.11695_std31.43703_random_bw107163.38446_skew1507.52625_neigh0.11804_cross0.86266_n14.mtx","synthetic_1000005_1000005_3121903_avg3.12189_std25.45302_random_bw110119.5356_skew1053.81063_neigh0.11769_cross0.86713_n14.mtx","synthetic_1000005_1000005_3127972_avg3.12796_std29.75772_random_bw112018.04327_skew1502.5376_neigh0.12161_cross0.87113_n14.mtx","synthetic_1000005_1000005_3132968_avg3.13295_std25.47429_random_bw113902.70335_skew1150.31021_neigh0.12143_cross0.87472_n14.mtx","synthetic_1000005_1000005_3136144_avg3.13613_std18.05255_random_bw135367.82089_skew1498.61976_neigh0.21685_cross0.93223_n14.mtx","synthetic_1000005_1000005_3144021_avg3.14401_std28.07823_random_bw116953.11281_skew1494.86263_neigh0.12731_cross0.88232_n14.mtx","synthetic_1000005_1000005_3144795_avg3.14478_std25.49712_random_bw117373.02426_skew1245.51038_neigh0.12684_cross0.88308_n14.mtx","synthetic_1000005_1000005_3156371_avg3.15636_std25.52223_random_bw120795.31056_skew1340.10381_neigh0.13144_cross0.89113_n14.mtx","synthetic_1000005_1000005_3160475_avg3.16046_std19.71515_random_bw135013.30133_skew1487.0749_neigh0.21383_cross0.93153_n14.mtx","synthetic_1000005_1000005_3162276_avg3.16226_std26.40103_random_bw122511.6205_skew1486.2274_neigh0.13472_cross0.89571_n14.mtx","synthetic_1000005_1000005_3166512_avg3.1665_std25.54915_random_bw123598.70118_skew1434.65625_neigh0.13874_cross0.8991_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_random_bw104073.79051_skew1482.18441_neigh0.14395_cross0.90267_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_random_bw112547.15199_skew1482.18441_neigh0.14364_cross0.90269_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_random_bw120556.18558_skew1482.18441_neigh0.14326_cross0.90269_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_random_bw121722.82145_skew1482.18441_neigh0.1869_cross0.90333_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_random_bw122150.26353_skew1482.18441_neigh0.05805_cross0.96156_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_random_bw122426.6617_skew1482.18441_neigh0.17652_cross0.90307_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_random_bw123138.58643_skew1482.18441_neigh0.16628_cross0.90292_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_random_bw123297.83473_skew1482.18441_neigh0.11813_cross0.93173_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_random_bw123697.2713_skew1482.18441_neigh0.1568_cross0.90295_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_random_bw124260.11141_skew1482.18441_neigh0.14853_cross0.90276_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_random_bw124950.43144_skew1482.18441_neigh0.13931_cross0.90257_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_random_bw125531.54795_skew1482.18441_neigh0.15898_cross0.87296_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_random_bw125571.10503_skew1482.18441_neigh0.12939_cross0.90242_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_random_bw126188.03769_skew1482.18441_neigh0.11966_cross0.90226_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_random_bw126648.45731_skew1482.18441_neigh0.11069_cross0.90206_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_random_bw127142.75789_skew1482.18441_neigh0.17733_cross0.81362_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_random_bw127147.10169_skew1482.18441_neigh0.10042_cross0.90174_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_random_bw128104.19842_skew1482.18441_neigh0.18795_cross0.75388_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_random_bw128465.7325_skew1482.18441_neigh0.19495_cross0.69438_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_random_bw128711.29975_skew1482.18441_neigh0.20001_cross0.63303_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_random_bw128912.58669_skew1482.18441_neigh0.14293_cross0.90268_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_random_bw137896.76343_skew1482.18441_neigh0.14246_cross0.90269_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_random_bw145843.60046_skew1482.18441_neigh0.14298_cross0.90268_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_random_bw154831.75347_skew1482.18441_neigh0.14358_cross0.90267_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_random_bw162157.8696_skew1482.18441_neigh0.14347_cross0.90267_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_random_bw87649.18931_skew1482.18441_neigh0.14287_cross0.90272_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_random_bw95510.6301_skew1482.18441_neigh0.14423_cross0.9027_n14.mtx","synthetic_1000005_1000005_3174621_avg3.17461_std25.57736_random_bw125956.24319_skew1529.58406_neigh0.14718_cross0.90575_n14.mtx","synthetic_1000005_1000005_3177914_avg3.1779_std24.72495_random_bw127016.47781_skew1478.909_neigh0.15361_cross0.90914_n14.mtx","synthetic_1000005_1000005_3179023_avg3.17901_std21.38191_random_bw134207.95085_skew1478.39273_neigh0.19999_cross0.92826_n14.mtx","synthetic_1000005_1000005_3180578_avg3.18056_std25.60874_random_bw128092.87982_skew1625.12766_neigh0.15702_cross0.91168_n14.mtx","synthetic_1000005_1000005_3184160_avg3.18414_std25.64113_random_bw130081.20538_skew1721.91199_neigh0.16859_cross0.9164_n14.mtx","synthetic_1000005_1000005_3185020_avg3.185_std23.052_random_bw131609.81549_skew1475.60722_neigh0.17602_cross0.92056_n14.mtx","synthetic_1000005_1000005_3185310_avg3.18529_std25.71304_random_bw132686.94329_skew1917.81812_neigh0.18507_cross0.92381_n14.mtx","synthetic_1000005_1000005_3185554_avg3.18554_std25.67645_random_bw131621.35039_skew1819.4146_neigh0.17691_cross0.9205_n14.mtx","synthetic_1000005_1000005_3238430_avg3.23841_std25.57761_random_bw128487.05961_skew1499.42591_neigh0.17003_cross0.91626_n14.mtx","synthetic_1000005_1000005_3338044_avg3.33803_std25.60993_random_bw120102.95798_skew1548.7177_neigh0.17012_cross0.91371_n14.mtx","synthetic_1000005_1000005_3554394_avg3.55438_std25.64567_random_bw113500.84038_skew1542.73078_neigh0.13151_cross0.88485_n14.mtx","synthetic_1000005_1000005_3911756_avg3.91174_std25.67945_random_bw133527.84178_skew1481.71748_neigh0.13986_cross0.88512_n14.mtx","synthetic_1000005_1000005_4154204_avg4.15418_std25.71363_random_bw158208.83055_skew1470.76946_neigh0.19591_cross0.92055_n14.mtx","synthetic_100993_100993_2185846_avg21.64354_std13.77891_random_bw54912.02009_skew4.03614_neigh1.01465_cross0.56058_n14.mtx","synthetic_1033338_1033338_3275830_avg3.17014_std25.55637_random_bw129053.64703_skew1482.52894_neigh0.14465_cross0.90234_n14.mtx","synthetic_109072_109072_2360564_avg21.64225_std13.77936_random_bw59445.13864_skew4.08265_neigh1.0157_cross0.55937_n14.mtx","synthetic_1100005_1100005_3487516_avg3.17045_std25.54298_random_bw137079.35531_skew1482.38345_neigh0.14227_cross0.90274_n14.mtx","synthetic_1166672_1166672_3699263_avg3.17078_std25.53274_random_bw145853.05163_skew1482.23015_neigh0.14123_cross0.90291_n14.mtx","synthetic_117152_117152_2535420_avg21.64214_std13.77711_random_bw63786.53108_skew4.08268_neigh1.01456_cross0.56071_n14.mtx","synthetic_119698_119698_670830_avg5.60435_std4.47718_random_bw38463.99053_skew62.8789_neigh0.60838_cross0.60967_n14.mtx","synthetic_121192_121192_1836052_avg15.14994_std12.36381_random_bw63604.68267_skew3.8185_neigh0.98085_cross0.56421_n14.mtx","synthetic_121192_121192_2010928_avg16.59291_std12.82631_random_bw63387.43514_skew3.94187_neigh0.99258_cross0.55956_n14.mtx","synthetic_121192_121192_2185768_avg18.03558_std13.23353_random_bw65140.22976_skew4.04558_neigh0.99835_cross0.55476_n14.mtx","synthetic_121192_121192_2360629_avg19.47842_std13.60481_random_bw64734.37292_skew4.13389_neigh1.00738_cross0.55096_n14.mtx","synthetic_121192_121192_2535413_avg20.92063_std13.77469_random_bw66438.45794_skew4.11457_neigh1.01319_cross0.55608_n14.mtx","synthetic_121192_121192_2622629_avg21.64028_std13.77138_random_bw65424.26549_skew3.85206_neigh1.01668_cross0.55626_n14.mtx","synthetic_121192_121192_2622663_avg21.64056_std13.63311_random_bw64870.55905_skew3.52853_neigh1.01573_cross0.55103_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_random_bw46185.35154_skew4.08299_neigh1.01504_cross0.5599_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_random_bw50573.38166_skew4.08299_neigh1.01509_cross0.56075_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_random_bw54972.82661_skew4.08299_neigh1.01519_cross0.5598_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_random_bw59411.92528_skew4.08299_neigh1.01561_cross0.56068_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_random_bw61692.98851_skew4.08299_neigh0.98464_cross0.72926_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_random_bw62953.41461_skew4.08299_neigh0.99544_cross0.69172_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_random_bw63724.66045_skew4.08299_neigh1.0157_cross0.55944_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_random_bw63863.58232_skew4.08299_neigh1.00127_cross0.65566_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_random_bw64806.59055_skew4.08299_neigh1.00789_cross0.61815_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_random_bw65523.57715_skew4.08299_neigh1.01329_cross0.58066_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_random_bw65666.88089_skew4.08299_neigh0.77995_cross0.56148_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_random_bw65781.74026_skew4.08299_neigh1.11633_cross0.56174_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_random_bw65912.90063_skew4.08299_neigh0.8475_cross0.55891_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_random_bw66094.34436_skew4.08299_neigh0.91339_cross0.55811_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_random_bw66297.82562_skew4.08299_neigh1.01673_cross0.5433_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_random_bw66303.01327_skew4.08299_neigh0.71059_cross0.55994_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_random_bw66344.93551_skew4.08299_neigh1.18402_cross0.56026_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_random_bw66632.5966_skew4.08299_neigh0.98135_cross0.55939_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_random_bw66915.08177_skew4.08299_neigh1.04961_cross0.5616_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_random_bw66999.72835_skew4.08299_neigh1.02006_cross0.50617_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_random_bw67029.04955_skew4.08299_neigh1.25119_cross0.56135_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_random_bw67482.8569_skew4.08299_neigh1.02304_cross0.46991_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_random_bw68050.39675_skew4.08299_neigh1.02635_cross0.4318_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_random_bw68172.92334_skew4.08299_neigh1.01556_cross0.56046_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_random_bw68534.78082_skew4.08299_neigh1.31693_cross0.56196_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_random_bw68594.96245_skew4.08299_neigh1.02897_cross0.39354_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_random_bw72555.87469_skew4.08299_neigh1.0152_cross0.56039_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_random_bw76889.20434_skew4.08299_neigh1.01471_cross0.5605_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_random_bw81259.52524_skew4.08299_neigh1.01612_cross0.56016_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_random_bw85714.50154_skew4.08299_neigh1.01527_cross0.56045_n14.mtx","synthetic_121192_121192_2622724_avg21.64107_std13.77599_random_bw65800.92968_skew3.99051_neigh1.01427_cross0.5606_n14.mtx","synthetic_121192_121192_2622736_avg21.64116_std14.23425_random_bw65388.53812_skew4.08291_neigh1.01644_cross0.55562_n14.mtx","synthetic_121192_121192_2622750_avg21.64128_std13.768_random_bw65046.54528_skew3.71321_neigh1.01557_cross0.55052_n14.mtx","synthetic_121192_121192_2622777_avg21.6415_std15.59481_random_bw64466.94691_skew4.08283_neigh1.01521_cross0.55446_n14.mtx","synthetic_121192_121192_2622792_avg21.64163_std13.77856_random_bw66101.8109_skew4.129_neigh1.01554_cross0.56191_n14.mtx","synthetic_121192_121192_2622812_avg21.64179_std16.1555_random_bw64390.16957_skew3.99034_neigh1.01546_cross0.55795_n14.mtx","synthetic_121192_121192_2622828_avg21.64192_std13.43338_random_bw65117.16008_skew3.29722_neigh1.01575_cross0.55268_n14.mtx","synthetic_121192_121192_2622837_avg21.642_std15.01875_random_bw64761.9232_skew4.12892_neigh1.01546_cross0.55076_n14.mtx","synthetic_121192_121192_2622876_avg21.64232_std16.69445_random_bw64326.40631_skew3.94402_neigh1.01578_cross0.55631_n14.mtx","synthetic_121192_121192_2622972_avg21.64311_std13.78148_random_bw66363.1673_skew4.31347_neigh1.01343_cross0.566_n14.mtx","synthetic_121192_121192_2623047_avg21.64373_std13.32114_random_bw66419.48364_skew4.0361_neigh1.01466_cross0.56615_n14.mtx","synthetic_121192_121192_2623109_avg21.64424_std13.78493_random_bw66631.25408_skew4.4518_neigh1.01539_cross0.56882_n14.mtx","synthetic_121192_121192_2623156_avg21.64463_std11.48856_random_bw68448.4577_skew3.89729_neigh1.01326_cross0.58356_n14.mtx","synthetic_121192_121192_2623189_avg21.6449_std13.78718_random_bw67331.69717_skew4.77503_neigh1.01343_cross0.57336_n14.mtx","synthetic_121192_121192_2623197_avg21.64497_std13.78635_random_bw66921.85212_skew4.59021_neigh1.01484_cross0.57212_n14.mtx","synthetic_121192_121192_2623249_avg21.6454_std12.40537_random_bw67456.81523_skew3.98951_neigh1.01486_cross0.57624_n14.mtx","synthetic_121192_121192_2623583_avg21.64815_std9.65791_random_bw70183.15791_skew3.8503_neigh1.01496_cross0.59679_n14.mtx","synthetic_121192_121192_2623613_avg21.6484_std10.57459_random_bw69356.8168_skew3.85024_neigh1.0146_cross0.5911_n14.mtx","synthetic_121192_121192_2710314_avg22.3638_std13.78076_random_bw66994.6287_skew4.05281_neigh1.01691_cross0.56487_n14.mtx","synthetic_121192_121192_2885490_avg23.80924_std13.78209_random_bw67434.57931_skew3.99806_neigh1.02152_cross0.5719_n14.mtx","synthetic_121192_121192_3060321_avg25.25184_std13.78548_random_bw68906.15443_skew3.95013_neigh1.02431_cross0.57782_n14.mtx","synthetic_121192_121192_3235599_avg26.69812_std13.78831_random_bw69042.29829_skew3.86926_neigh1.02849_cross0.58477_n14.mtx","synthetic_121192_121192_3410529_avg28.14154_std13.78985_random_bw69312.50343_skew3.86825_neigh1.03157_cross0.58839_n14.mtx","synthetic_1233339_1233339_3909668_avg3.16999_std25.52165_random_bw154195.41372_skew1482.60253_neigh0.14111_cross0.90327_n14.mtx","synthetic_125231_125231_2710355_avg21.64284_std13.77742_random_bw68141.31404_skew4.08251_neigh1.01464_cross0.56127_n14.mtx","synthetic_1300006_1300006_4120758_avg3.1698_std25.51264_random_bw162585.00942_skew1482.69019_neigh0.14075_cross0.90351_n14.mtx","synthetic_131098_131098_734679_avg5.60404_std4.47302_random_bw41879.66163_skew62.88244_neigh0.60932_cross0.60986_n14.mtx","synthetic_133311_133311_2884795_avg21.63959_std13.78017_random_bw72606.70986_skew4.08328_neigh1.01353_cross0.56105_n14.mtx","synthetic_141390_141390_3059797_avg21.64083_std13.77882_random_bw76927.21946_skew4.08298_neigh1.01482_cross0.56115_n14.mtx","synthetic_142498_142498_798541_avg5.60388_std4.46852_random_bw45521.18383_skew62.88436_neigh0.61099_cross0.60948_n14.mtx","synthetic_144550_144550_890999_avg6.16395_std4.44624_random_bw249.77523_skew7.27392_neigh0.18177_cross0.33337_n14.mtx","synthetic_14839_14839_1040332_avg70.10796_std6.31579_random_bw1146.69634_skew0.34079_neigh1.78621_cross0.91392_n14.mtx","synthetic_149470_149470_3234141_avg21.63739_std13.78086_random_bw81415.43761_skew4.08379_neigh1.01461_cross0.56019_n14.mtx","synthetic_153898_153898_862342_avg5.60333_std4.46589_random_bw49364.34344_skew62.89053_neigh0.61051_cross0.60952_n14.mtx","synthetic_157549_157549_3408821_avg21.63658_std13.78244_random_bw85829.82001_skew4.08398_neigh1.01388_cross0.56058_n14.mtx","synthetic_158316_158316_975794_avg6.16358_std4.44586_random_bw273.88663_skew7.27441_neigh0.17907_cross0.32957_n14.mtx","synthetic_16253_16253_1139653_avg70.11955_std6.30855_random_bw1286.71322_skew0.34057_neigh1.7819_cross0.91551_n14.mtx","synthetic_165298_165298_926272_avg5.60365_std4.4624_random_bw52893.04048_skew62.88694_neigh0.60862_cross0.6098_n14.mtx","synthetic_170998_170998_1008507_avg5.89777_std3.14995_random_bw47719.98025_skew59.87046_neigh0.63807_cross0.61706_n14.mtx","synthetic_170998_170998_1037773_avg6.06892_std4.43265_random_bw48509.54384_skew59.96638_neigh0.66301_cross0.62469_n14.mtx","synthetic_170998_170998_1043193_avg6.10062_std4.44271_random_bw48724.69743_skew63.58365_neigh0.67297_cross0.62698_n14.mtx","synthetic_170998_170998_1092151_avg6.38692_std4.47337_random_bw49002.39485_skew64.44623_neigh0.64099_cross0.61384_n14.mtx","synthetic_170998_170998_1211686_avg7.08597_std4.4649_random_bw52652.61671_skew64.76379_neigh0.68861_cross0.62669_n14.mtx","synthetic_170998_170998_1212281_avg7.08945_std4.45045_random_bw52659.56747_skew61.3462_neigh0.68857_cross0.62682_n14.mtx","synthetic_170998_170998_703429_avg4.11367_std4.4083_random_bw66949.59583_skew60.01611_neigh0.59084_cross0.62104_n14.mtx","synthetic_170998_170998_709969_avg4.15191_std4.42013_random_bw67041.59758_skew65.23451_neigh0.60621_cross0.62439_n14.mtx","synthetic_170998_170998_813246_avg4.75588_std4.44226_random_bw49547.73476_skew61.86954_neigh0.577_cross0.60721_n14.mtx","synthetic_170998_170998_875832_avg5.12188_std4.42226_random_bw52588.79732_skew62.06273_neigh0.64594_cross0.6236_n14.mtx","synthetic_170998_170998_882159_avg5.15888_std4.44207_random_bw52806.32402_skew66.06876_neigh0.63734_cross0.62272_n14.mtx","synthetic_170998_170998_944721_avg5.52475_std5.76053_random_bw53759.0296_skew63.79933_neigh0.61291_cross0.61003_n14.mtx","synthetic_170998_170998_945632_avg5.53008_std5.47116_random_bw53929.01913_skew63.7369_neigh0.61326_cross0.61142_n14.mtx","synthetic_170998_170998_946661_avg5.53609_std4.44002_random_bw54131.28407_skew44.88072_neigh0.61289_cross0.6094_n14.mtx","synthetic_170998_170998_947518_avg5.54111_std5.18255_random_bw54056.59899_skew63.60804_neigh0.61159_cross0.61005_n14.mtx","synthetic_170998_170998_948659_avg5.54778_std4.44165_random_bw54194.39357_skew48.9299_neigh0.61004_cross0.61069_n14.mtx","synthetic_170998_170998_950710_avg5.55977_std4.8941_random_bw54225.54433_skew63.39112_neigh0.60981_cross0.60991_n14.mtx","synthetic_170998_170998_951053_avg5.56178_std4.44852_random_bw54287.96161_skew53.11938_neigh0.6081_cross0.60983_n14.mtx","synthetic_170998_170998_953667_avg5.57707_std4.45309_random_bw54491.631_skew57.09507_neigh0.6098_cross0.60996_n14.mtx","synthetic_170998_170998_955297_avg5.5866_std4.60347_random_bw54609.35347_skew63.08194_neigh0.6098_cross0.61023_n14.mtx","synthetic_170998_170998_956707_avg5.59484_std4.45769_random_bw54693.36679_skew61.0214_neigh0.61022_cross0.60929_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_random_bw38321.91494_skew62.88573_neigh0.60926_cross0.60964_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_random_bw41929.48688_skew62.88573_neigh0.60983_cross0.6094_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_random_bw45628.56654_skew62.88573_neigh0.61026_cross0.60953_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_random_bw46903.96538_skew62.88573_neigh0.54722_cross0.60807_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_random_bw47843.37714_skew62.88573_neigh0.50741_cross0.60854_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_random_bw48523.58357_skew62.88573_neigh0.46873_cross0.60766_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_random_bw49181.26028_skew62.88573_neigh0.60917_cross0.60964_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_random_bw49272.78155_skew62.88573_neigh0.42658_cross0.60457_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_random_bw49297.41096_skew62.88573_neigh0.79067_cross0.61249_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_random_bw50512.55887_skew62.88573_neigh0.75263_cross0.61034_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_random_bw51880.4182_skew62.88573_neigh0.71336_cross0.6105_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_random_bw52905.44733_skew62.88573_neigh0.60897_cross0.60969_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_random_bw53028.52728_skew62.88573_neigh0.67336_cross0.60943_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_random_bw53125.78092_skew62.88573_neigh0.55319_cross0.78965_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_random_bw53660.70344_skew62.88573_neigh0.56909_cross0.75077_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_random_bw54099.133_skew62.88573_neigh0.58565_cross0.70928_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_random_bw54240.77009_skew62.88573_neigh0.63324_cross0.60935_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_random_bw54411.68621_skew62.88573_neigh0.59727_cross0.66902_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_random_bw54696.32419_skew62.88573_neigh0.60505_cross0.63095_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_random_bw54839.18485_skew62.88573_neigh0.61271_cross0.58999_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_random_bw54975.41519_skew62.88573_neigh0.61926_cross0.54811_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_random_bw55027.14865_skew62.88573_neigh0.62632_cross0.50615_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_random_bw55105.55545_skew62.88573_neigh0.63063_cross0.46746_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_random_bw55236.3212_skew62.88573_neigh0.63219_cross0.42616_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_random_bw55398.06761_skew62.88573_neigh0.58791_cross0.60932_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_random_bw56503.9465_skew62.88573_neigh0.60809_cross0.60967_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_random_bw60168.38237_skew62.88573_neigh0.60916_cross0.60962_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_random_bw64070.5493_skew62.88573_neigh0.60935_cross0.60965_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_random_bw67352.62944_skew62.88573_neigh0.60959_cross0.60963_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_random_bw71097.05963_skew62.88573_neigh0.60883_cross0.60961_n14.mtx","synthetic_170998_170998_959835_avg5.61314_std4.46266_random_bw54839.73947_skew64.91681_neigh0.60778_cross0.61012_n14.mtx","synthetic_170998_170998_961623_avg5.62359_std4.31537_random_bw54830.15481_skew62.66038_neigh0.60743_cross0.6101_n14.mtx","synthetic_170998_170998_963061_avg5.632_std4.4671_random_bw54958.67312_skew68.77981_neigh0.6102_cross0.6093_n14.mtx","synthetic_170998_170998_966597_avg5.65268_std4.47351_random_bw55278.98278_skew72.59341_neigh0.60946_cross0.60959_n14.mtx","synthetic_170998_170998_970106_avg5.6732_std4.48093_random_bw55352.2479_skew76.55763_neigh0.61119_cross0.61051_n14.mtx","synthetic_170998_170998_970315_avg5.67442_std4.02645_random_bw55373.82904_skew62.09011_neigh0.61062_cross0.61139_n14.mtx","synthetic_170998_170998_973557_avg5.69338_std4.48437_random_bw55499.03393_skew80.32248_neigh0.61487_cross0.61054_n14.mtx","synthetic_170998_170998_981215_avg5.73817_std3.73499_random_bw55852.32383_skew61.38927_neigh0.61949_cross0.61025_n14.mtx","synthetic_170998_170998_994346_avg5.81496_std3.44543_random_bw56542.54022_skew60.73734_neigh0.6252_cross0.61459_n14.mtx","synthetic_172083_172083_1060867_avg6.16486_std4.44424_random_bw297.78023_skew7.2727_neigh0.17596_cross0.32947_n14.mtx","synthetic_17666_17666_1238955_avg70.13217_std6.31199_random_bw1373.26344_skew0.34033_neigh1.78005_cross0.91685_n14.mtx","synthetic_176697_176697_990167_avg5.60376_std4.45819_random_bw56447.76777_skew62.88571_neigh0.60833_cross0.60971_n14.mtx","synthetic_185639_185639_15003294_avg80.81973_std87.6573_random_bw25664.94771_skew8.39127_neigh1.21561_cross0.82365_n14.mtx","synthetic_185639_185639_15003763_avg80.82226_std142.24862_random_bw20258.95107_skew7.19081_neigh1.26005_cross0.95405_n14.mtx","synthetic_185639_185639_15004156_avg80.82437_std94.60907_random_bw25577.96053_skew8.30412_neigh1.21738_cross0.82741_n14.mtx","synthetic_185639_185639_15004647_avg80.82702_std101.43013_random_bw25579.20672_skew8.20484_neigh1.21614_cross0.83107_n14.mtx","synthetic_185639_185639_15005689_avg80.83263_std108.09033_random_bw25508.21416_skew8.08049_neigh1.21648_cross0.83534_n14.mtx","synthetic_185639_185639_15006242_avg80.83561_std119.38413_random_bw25324.46226_skew8.66158_neigh1.21359_cross0.83963_n14.mtx","synthetic_185639_185639_15006431_avg80.83663_std114.55767_random_bw25341.18275_skew7.94397_neigh1.21465_cross0.84203_n14.mtx","synthetic_185639_185639_15006681_avg80.83798_std118.31728_random_bw25323.95677_skew8.14174_neigh1.2126_cross0.84303_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_random_bw17706.01026_skew7.8818_neigh1.2138_cross0.84521_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_random_bw19404.35313_skew7.8818_neigh1.21315_cross0.84474_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_random_bw21104.27872_skew7.8818_neigh1.21478_cross0.84522_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_random_bw22750.22312_skew7.8818_neigh1.21263_cross0.84532_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_random_bw24441.96582_skew7.8818_neigh1.21268_cross0.84507_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_random_bw26179.82365_skew7.8818_neigh1.21167_cross0.84498_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_random_bw27821.86705_skew7.8818_neigh1.21199_cross0.84555_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_random_bw29515.52781_skew7.8818_neigh1.21272_cross0.84497_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_random_bw31178.6333_skew7.8818_neigh1.21304_cross0.84523_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_random_bw32880.54075_skew7.8818_neigh1.21042_cross0.84511_n14.mtx","synthetic_185639_185639_15007305_avg80.84134_std120.8084_random_bw25218.21287_skew7.80738_neigh1.21325_cross0.84823_n14.mtx","synthetic_185639_185639_15008187_avg80.84609_std126.81395_random_bw25055.73053_skew7.64606_neigh1.20604_cross0.8579_n14.mtx","synthetic_185639_185639_15008702_avg80.84886_std137.983_random_bw24238.46969_skew7.32417_neigh1.20104_cross0.89187_n14.mtx","synthetic_185639_185639_15008859_avg80.84971_std132.54683_random_bw24711.02466_skew7.48488_neigh1.20312_cross0.87083_n14.mtx","synthetic_185639_185639_18505682_avg99.68639_std122.27517_random_bw25744.70036_skew8.23897_neigh1.22874_cross0.82863_n14.mtx","synthetic_185639_185639_19505424_avg105.0718_std123.33624_random_bw25716.49132_skew8.2984_neigh1.23114_cross0.82664_n14.mtx","synthetic_185850_185850_1145706_avg6.16468_std4.44467_random_bw321.67519_skew7.27293_neigh0.17292_cross0.32599_n14.mtx","synthetic_188097_188097_1054064_avg5.60383_std4.45656_random_bw60049.05112_skew62.88486_neigh0.60992_cross0.60918_n14.mtx","synthetic_19080_19080_1337993_avg70.12542_std6.31221_random_bw1489.05252_skew0.34046_neigh1.78444_cross0.91485_n14.mtx","synthetic_199497_199497_1118014_avg5.60416_std4.45398_random_bw64023.47114_skew62.88107_neigh0.60994_cross0.60927_n14.mtx","synthetic_199616_199616_1230682_avg6.16525_std4.44318_random_bw345.0451_skew7.27217_neigh0.1707_cross0.32355_n14.mtx","synthetic_20493_20493_1437302_avg70.13624_std6.31204_random_bw1605.41268_skew0.34025_neigh1.77707_cross0.91532_n14.mtx","synthetic_206500_206500_1060353_avg5.13488_std4.44292_random_bw335.27304_skew7.3741_neigh0.15993_cross0.3084_n14.mtx","synthetic_206500_206500_1145534_avg5.54738_std4.44341_random_bw356.84439_skew7.2922_neigh0.16358_cross0.31502_n14.mtx","synthetic_206500_206500_1230425_avg5.95847_std4.44436_random_bw374.92322_skew7.22358_neigh0.16711_cross0.31892_n14.mtx","synthetic_206500_206500_1272393_avg6.16171_std5.76962_random_bw331.0109_skew7.43922_neigh0.1768_cross0.30559_n14.mtx","synthetic_206500_206500_1272581_avg6.16262_std4.44309_random_bw352.77997_skew6.30209_neigh0.16944_cross0.31989_n14.mtx","synthetic_206500_206500_1272606_avg6.16274_std4.44382_random_bw350.20787_skew5.97741_neigh0.16939_cross0.31707_n14.mtx","synthetic_206500_206500_1272648_avg6.16294_std5.47591_random_bw338.0763_skew7.43753_neigh0.17479_cross0.31105_n14.mtx","synthetic_206500_206500_1272653_avg6.16297_std5.1813_random_bw344.41462_skew7.27523_neigh0.17252_cross0.31341_n14.mtx","synthetic_206500_206500_1272686_avg6.16313_std4.88791_random_bw350.39832_skew7.27502_neigh0.17198_cross0.31772_n14.mtx","synthetic_206500_206500_1272775_avg6.16356_std4.44367_random_bw347.43103_skew5.48976_neigh0.16995_cross0.31545_n14.mtx","synthetic_206500_206500_1272904_avg6.16418_std4.4462_random_bw361.82239_skew8.73365_neigh0.1696_cross0.32814_n14.mtx","synthetic_206500_206500_1272958_avg6.16445_std4.59126_random_bw354.96969_skew7.27325_neigh0.16996_cross0.32281_n14.mtx","synthetic_206500_206500_1272981_avg6.16456_std4.44526_random_bw362.54571_skew9.05749_neigh0.16989_cross0.32918_n14.mtx","synthetic_206500_206500_1272983_avg6.16457_std4.4441_random_bw354.47146_skew6.62422_neigh0.16961_cross0.32233_n14.mtx","synthetic_206500_206500_1272990_avg6.1646_std4.44515_random_bw357.90535_skew7.43526_neigh0.16959_cross0.32445_n14.mtx","synthetic_206500_206500_1272993_avg6.16462_std4.44589_random_bw359.27284_skew7.92189_neigh0.16953_cross0.32657_n14.mtx","synthetic_206500_206500_1272999_avg6.16464_std4.44572_random_bw360.70137_skew8.24628_neigh0.16983_cross0.32797_n14.mtx","synthetic_206500_206500_1273034_avg6.16481_std4.29748_random_bw359.14318_skew7.27276_neigh0.17002_cross0.32629_n14.mtx","synthetic_206500_206500_1273051_avg6.1649_std3.71025_random_bw366.49966_skew7.27265_neigh0.16733_cross0.33042_n14.mtx","synthetic_206500_206500_1273053_avg6.16491_std4.44447_random_bw356.03003_skew7.11042_neigh0.1699_cross0.32349_n14.mtx","synthetic_206500_206500_1273074_avg6.16501_std4.00298_random_bw362.98579_skew7.2725_neigh0.16799_cross0.32983_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_random_bw250.53829_skew7.27211_neigh0.18184_cross0.33208_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_random_bw272.80182_skew7.27211_neigh0.17887_cross0.32932_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_random_bw297.33833_skew7.27211_neigh0.17494_cross0.32838_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_random_bw321.61163_skew7.27211_neigh0.17327_cross0.32563_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_random_bw345.2926_skew7.27211_neigh0.17126_cross0.32436_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_random_bw353.07637_skew7.27211_neigh0.1683_cross0.41181_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_random_bw354.05036_skew7.27211_neigh0.1697_cross0.39234_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_random_bw355.04431_skew7.27211_neigh0.16961_cross0.37276_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_random_bw355.0493_skew7.27211_neigh0.193_cross0.3226_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_random_bw356.01345_skew7.27211_neigh0.16921_cross0.35329_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_random_bw356.02066_skew7.27211_neigh0.18376_cross0.32185_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_random_bw356.56768_skew7.27211_neigh0.17513_cross0.32339_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_random_bw357.15197_skew7.27211_neigh0.16977_cross0.3328_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_random_bw357.48545_skew7.27211_neigh0.16999_cross0.31374_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_random_bw357.59221_skew7.27211_neigh0.16543_cross0.32304_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_random_bw358.45388_skew7.27211_neigh0.17021_cross0.29537_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_random_bw358.52034_skew7.27211_neigh0.15599_cross0.32162_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_random_bw358.88324_skew7.27211_neigh0.17043_cross0.27574_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_random_bw359.59658_skew7.27211_neigh0.17102_cross0.25585_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_random_bw359.63777_skew7.27211_neigh0.14622_cross0.32225_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_random_bw360.3102_skew7.27211_neigh0.17173_cross0.23608_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_random_bw360.37211_skew7.27211_neigh0.13735_cross0.3228_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_random_bw361.09723_skew7.27211_neigh0.12774_cross0.32289_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_random_bw368.19459_skew7.27211_neigh0.16931_cross0.32373_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_random_bw378.37779_skew7.27211_neigh0.21109_cross0.3219_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_random_bw379.42115_skew7.27211_neigh0.20197_cross0.32156_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_random_bw392.56075_skew7.27211_neigh0.16763_cross0.32173_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_random_bw416.95627_skew7.27211_neigh0.16635_cross0.32052_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_random_bw440.8278_skew7.27211_neigh0.16504_cross0.3198_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_random_bw464.28694_skew7.27211_neigh0.16438_cross0.31804_n14.mtx","synthetic_206500_206500_1273351_avg6.16635_std3.41298_random_bw369.41645_skew7.2707_neigh0.16589_cross0.33282_n14.mtx","synthetic_206500_206500_1274354_avg6.17121_std3.11913_random_bw372.02783_skew7.10214_neigh0.16567_cross0.33662_n14.mtx","synthetic_206500_206500_1315280_avg6.36939_std4.44461_random_bw364.31629_skew7.32104_neigh0.1716_cross0.32663_n14.mtx","synthetic_206500_206500_1400260_avg6.78092_std4.44568_random_bw375.96645_skew7.25847_neigh0.17353_cross0.33102_n14.mtx","synthetic_206500_206500_1485240_avg7.19245_std4.44584_random_bw368.40782_skew7.20305_neigh0.17827_cross0.33719_n14.mtx","synthetic_206500_206500_1570037_avg7.60308_std4.44659_random_bw376.86972_skew7.15458_neigh0.18053_cross0.33804_n14.mtx","synthetic_206500_206500_1655223_avg8.01561_std4.44684_random_bw384.89388_skew7.23394_neigh0.18176_cross0.34264_n14.mtx","synthetic_206500_206500_890883_avg4.3142_std4.3742_random_bw314.87298_skew7.34453_neigh0.15043_cross0.29772_n14.mtx","synthetic_206500_206500_975718_avg4.72503_std4.44295_random_bw342.64695_skew7.46556_neigh0.15368_cross0.29915_n14.mtx","synthetic_210897_210897_1181860_avg5.60397_std4.45232_random_bw67447.98409_skew62.88331_neigh0.61013_cross0.60909_n14.mtx","synthetic_21200_21200_1040239_avg49.06788_std6.3116_random_bw2270.10316_skew0.48774_neigh1.73665_cross0.90005_n14.mtx","synthetic_21200_21200_1139550_avg53.75236_std6.31054_random_bw2327.97646_skew0.4325_neigh1.747_cross0.90406_n14.mtx","synthetic_21200_21200_1238795_avg58.43373_std6.30753_random_bw2370.53675_skew0.4033_neigh1.76263_cross0.90541_n14.mtx","synthetic_21200_21200_1338002_avg63.1133_std6.31089_random_bw1621.46392_skew0.37847_neigh1.76689_cross0.91021_n14.mtx","synthetic_21200_21200_1437342_avg67.79915_std6.31153_random_bw1625.12618_skew0.35695_neigh1.78043_cross0.91191_n14.mtx","synthetic_21200_21200_1486357_avg70.11118_std8.20133_random_bw1621.03675_skew0.44057_neigh1.79499_cross0.90156_n14.mtx","synthetic_21200_21200_1486477_avg70.11684_std7.77973_random_bw1623.75863_skew0.41193_neigh1.78416_cross0.90649_n14.mtx","synthetic_21200_21200_1486586_avg70.12198_std7.35959_random_bw1633.7316_skew0.39756_neigh1.78517_cross0.90911_n14.mtx","synthetic_21200_21200_1486691_avg70.12693_std6.94018_random_bw1638.06472_skew0.36895_neigh1.78923_cross0.90946_n14.mtx","synthetic_21200_21200_1486777_avg70.13099_std6.51934_random_bw1665.03075_skew0.35461_neigh1.77737_cross0.91541_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_random_bw1091.14792_skew0.3403_neigh1.27638_cross0.91529_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_random_bw1099.35217_skew0.3403_neigh1.38725_cross0.91539_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_random_bw1129.65231_skew0.3403_neigh1.4963_cross0.91405_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_random_bw1150.42368_skew0.3403_neigh1.6111_cross0.91717_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_random_bw1155.31632_skew0.3403_neigh1.7853_cross0.91405_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_random_bw1291.37047_skew0.3403_neigh1.78149_cross0.91544_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_random_bw1309.76505_skew0.3403_neigh1.7269_cross0.91625_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_random_bw1369.88212_skew0.3403_neigh1.7807_cross0.91642_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_random_bw1489.33288_skew0.3403_neigh1.78357_cross0.91451_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_random_bw1563.73939_skew0.3403_neigh1.71868_cross0.94884_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_random_bw1590.50175_skew0.3403_neigh1.72943_cross0.94422_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_random_bw1604.78255_skew0.3403_neigh1.77801_cross0.91536_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_random_bw1650.62693_skew0.3403_neigh1.86722_cross0.64927_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_random_bw1663.23962_skew0.3403_neigh1.86073_cross0.70503_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_random_bw1668.34939_skew0.3403_neigh1.77532_cross0.91565_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_random_bw1670.25646_skew0.3403_neigh1.85028_cross0.77171_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_random_bw1674.10208_skew0.3403_neigh1.83639_cross0.82849_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_random_bw1677.25429_skew0.3403_neigh1.80579_cross0.88625_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_random_bw1710.35316_skew0.3403_neigh1.78264_cross0.91384_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_random_bw1844.95325_skew0.3403_neigh1.77183_cross0.91655_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_random_bw1925.18679_skew0.3403_neigh1.77933_cross0.91379_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_random_bw2023.01226_skew0.3403_neigh1.85164_cross0.91467_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_random_bw2027.65887_skew0.3403_neigh1.77965_cross0.91342_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_random_bw2126.54439_skew0.3403_neigh1.7794_cross0.91462_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_random_bw2183.93132_skew0.3403_neigh1.83524_cross0.91315_n14.mtx","synthetic_21200_21200_1487000_avg70.14151_std6.10324_random_bw1671.56_skew0.32589_neigh1.77648_cross0.91706_n14.mtx","synthetic_21200_21200_1487077_avg70.14514_std5.67801_random_bw1678.95703_skew0.29731_neigh1.77486_cross0.91968_n14.mtx","synthetic_21200_21200_1487192_avg70.15057_std5.25576_random_bw1669.85401_skew0.28295_neigh1.78043_cross0.92058_n14.mtx","synthetic_21200_21200_1487315_avg70.15637_std4.84276_random_bw1691.18377_skew0.25434_neigh1.76784_cross0.925_n14.mtx","synthetic_21200_21200_1487478_avg70.16406_std4.4186_random_bw1723.39901_skew0.23995_neigh1.76461_cross0.92734_n14.mtx","synthetic_21200_21200_1536543_avg72.47844_std6.30853_random_bw1661.22915_skew0.32453_neigh1.78331_cross0.91588_n14.mtx","synthetic_21200_21200_1635721_avg77.15665_std6.31046_random_bw1691.41788_skew0.30903_neigh1.78778_cross0.91802_n14.mtx","synthetic_21200_21200_1735090_avg81.84387_std6.31393_random_bw1717.89467_skew0.29515_neigh1.79144_cross0.92217_n14.mtx","synthetic_21200_21200_1834315_avg86.52429_std6.30851_random_bw1449.33458_skew0.27132_neigh1.8011_cross0.92178_n14.mtx","synthetic_21200_21200_1933465_avg91.20118_std6.31043_random_bw1477.8358_skew0.26095_neigh1.80472_cross0.92273_n14.mtx","synthetic_213383_213383_1315556_avg6.16523_std4.44392_random_bw368.70315_skew7.27219_neigh0.16899_cross0.32206_n14.mtx","synthetic_21906_21906_1536320_avg70.13238_std6.31756_random_bw1709.96585_skew0.34032_neigh1.77856_cross0.91434_n14.mtx","synthetic_222297_222297_1245675_avg5.60365_std4.45068_random_bw71187.12274_skew62.88691_neigh0.61086_cross0.60946_n14.mtx","synthetic_227150_227150_1400378_avg6.16499_std4.4444_random_bw392.60485_skew7.27252_neigh0.16823_cross0.32207_n14.mtx","synthetic_23320_23320_1635398_avg70.12856_std6.31946_random_bw1840.50772_skew0.3404_neigh1.7719_cross0.91648_n14.mtx","synthetic_240916_240916_1485276_avg6.16512_std4.4437_random_bw416.93905_skew7.27235_neigh0.16633_cross0.32069_n14.mtx","synthetic_24733_24733_1734543_avg70.13072_std6.31636_random_bw1933.712_skew0.34035_neigh1.7778_cross0.91451_n14.mtx","synthetic_254683_254683_1570192_avg6.16528_std4.4433_random_bw441.04844_skew7.27213_neigh0.16638_cross0.31989_n14.mtx","synthetic_26146_26146_1833536_avg70.12683_std6.31243_random_bw2034.00092_skew0.34043_neigh1.7785_cross0.91319_n14.mtx","synthetic_268450_268450_1655008_avg6.16505_std4.44456_random_bw465.05871_skew7.27244_neigh0.16485_cross0.31978_n14.mtx","synthetic_27120_27120_1234696_avg45.52714_std31.4396_random_bw657.74856_skew2.64618_neigh1.22988_cross0.77937_n14.mtx","synthetic_27560_27560_1933075_avg70.1406_std6.31297_random_bw2125.47631_skew0.34017_neigh1.77961_cross0.91464_n14.mtx","synthetic_29703_29703_1352183_avg45.52345_std31.43457_random_bw718.17419_skew2.64647_neigh1.22769_cross0.77551_n14.mtx","synthetic_32286_32286_1469958_avg45.52927_std31.42835_random_bw783.22802_skew2.66797_neigh1.22513_cross0.77351_n14.mtx","synthetic_32784_32784_1657863_avg50.56927_std27.82469_random_bw5242.21303_skew3.09339_neigh1.63697_cross0.76668_n14.mtx","synthetic_34406_34406_1341834_avg39.0_std0.0_random_bw8401.7213_skew0.0_neigh1.4059_cross0.80649_n14.mtx","synthetic_34869_34869_1587838_avg45.53724_std31.4194_random_bw844.81465_skew2.66733_neigh1.22431_cross0.77337_n14.mtx","synthetic_35906_35906_1816148_avg50.58063_std27.80623_random_bw5716.92046_skew3.09248_neigh1.63607_cross0.7699_n14.mtx","synthetic_368077_368077_1472308_avg4.0_std0.0_random_bw504.70969_skew0.0_neigh0.37372_cross0.9891_n14.mtx","synthetic_37452_37452_1706036_avg45.5526_std31.4087_random_bw906.67014_skew2.66609_neigh1.22181_cross0.77055_n14.mtx","synthetic_37683_37683_1469637_avg39.0_std0.0_random_bw9214.2749_skew0.0_neigh1.40447_cross0.80565_n14.mtx","synthetic_38120_38120_14004492_avg367.37912_std206.50003_random_bw17131.47558_skew1.32185_neigh1.97443_cross0.91972_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_random_bw12163.96299_skew1.57045_neigh1.97477_cross0.91968_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_random_bw13487.94121_skew1.57045_neigh1.97466_cross0.91853_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_random_bw14593.25708_skew1.57045_neigh1.97453_cross0.91966_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_random_bw15458.63701_skew1.57045_neigh1.97488_cross0.91833_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_random_bw16697.59053_skew1.57045_neigh1.97466_cross0.91971_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_random_bw17311.49276_skew1.57045_neigh1.97473_cross0.91875_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_random_bw17930.28565_skew1.57045_neigh1.97458_cross0.91885_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_random_bw18890.56044_skew1.57045_neigh1.97464_cross0.91976_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_random_bw19998.78673_skew1.57045_neigh1.97496_cross0.91885_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_random_bw21254.44919_skew1.57045_neigh1.9747_cross0.9186_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_random_bw22981.65732_skew1.57045_neigh1.97438_cross0.91938_n14.mtx","synthetic_38120_38120_15009192_avg393.73536_std249.23315_random_bw20164.63922_skew1.65915_neigh1.95556_cross0.89934_n14.mtx","synthetic_38120_38120_15509350_avg406.85598_std254.10081_random_bw18952.83022_skew1.83638_neigh1.9601_cross0.87267_n14.mtx","synthetic_38120_38120_15831412_avg415.30462_std256.81933_random_bw18361.46692_skew2.01225_neigh1.9625_cross0.85222_n14.mtx","synthetic_38120_38120_16013257_avg420.07495_std256.51996_random_bw18132.49037_skew2.18991_neigh1.96341_cross0.83427_n14.mtx","synthetic_38120_38120_17911818_avg469.8798_std294.64729_random_bw17768.78305_skew1.77305_neigh1.96177_cross0.87832_n14.mtx","synthetic_38120_38120_17943763_avg470.71781_std404.16901_random_bw13028.28431_skew3.74169_neigh1.974_cross0.55914_n14.mtx","synthetic_38120_38120_19284934_avg505.90068_std314.8817_random_bw17404.61952_skew1.89583_neigh1.96456_cross0.86292_n14.mtx","synthetic_38120_38120_20568467_avg539.57154_std333.79776_random_bw17206.90546_skew2.00609_neigh1.96636_cross0.85062_n14.mtx","synthetic_38744_38744_1161488_avg29.97853_std22.5272_random_bw1034.84256_skew1.93543_neigh1.22772_cross0.85244_n14.mtx","synthetic_38744_38744_1307882_avg33.75702_std24.88267_random_bw998.77878_skew2.08084_neigh1.17487_cross0.82838_n14.mtx","synthetic_38744_38744_1449216_avg37.40491_std27.07152_random_bw962.94693_skew2.28834_neigh1.19126_cross0.80394_n14.mtx","synthetic_38744_38744_1579684_avg40.77235_std28.98643_random_bw952.64469_skew2.45823_neigh1.20264_cross0.78378_n14.mtx","synthetic_38744_38744_1703850_avg43.97713_std30.65815_random_bw943.0718_skew2.61552_neigh1.21338_cross0.77875_n14.mtx","synthetic_38744_38744_1707520_avg44.07186_std32.46486_random_bw995.81409_skew2.10856_neigh1.20504_cross0.82864_n14.mtx","synthetic_38744_38744_1730367_avg44.66155_std32.51286_random_bw976.76174_skew2.22425_neigh1.21085_cross0.80896_n14.mtx","synthetic_38744_38744_1736329_avg44.81543_std26.74959_random_bw958.24236_skew1.96773_neigh1.21246_cross0.79103_n14.mtx","synthetic_38744_38744_1746236_avg45.07113_std32.41542_random_bw952.99461_skew2.35026_neigh1.21614_cross0.79836_n14.mtx","synthetic_38744_38744_1747768_avg45.11068_std28.0118_random_bw950.83561_skew2.12565_neigh1.21055_cross0.78825_n14.mtx","synthetic_38744_38744_1755242_avg45.30358_std29.13201_random_bw947.62389_skew2.28892_neigh1.21485_cross0.78295_n14.mtx","synthetic_38744_38744_1756611_avg45.33892_std32.15335_random_bw944.97597_skew2.48486_neigh1.21552_cross0.78836_n14.mtx","synthetic_38744_38744_1760377_avg45.43612_std30.12952_random_bw943.2046_skew2.43339_neigh1.21615_cross0.78132_n14.mtx","synthetic_38744_38744_1762627_avg45.49419_std31.70473_random_bw945.51417_skew2.60486_neigh1.21499_cross0.77366_n14.mtx","synthetic_38744_38744_1763457_avg45.51562_std31.01391_random_bw938.21335_skew2.60316_neigh1.21504_cross0.77558_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_random_bw1012.91606_skew2.6669_neigh1.23666_cross0.60626_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_random_bw1028.13896_skew2.6669_neigh1.24007_cross0.5571_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_random_bw1030.11568_skew2.6669_neigh1.2189_cross0.77356_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_random_bw1090.87397_skew2.6669_neigh1.21727_cross0.76922_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_random_bw1152.64756_skew2.6669_neigh1.2162_cross0.77082_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_random_bw1218.42621_skew2.6669_neigh1.21339_cross0.77016_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_random_bw659.5718_skew2.6669_neigh1.23127_cross0.77639_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_random_bw720.8265_skew2.6669_neigh1.23049_cross0.7745_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_random_bw783.97236_skew2.6669_neigh1.22458_cross0.77252_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_random_bw790.44606_skew2.6669_neigh1.18687_cross0.89462_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_random_bw799.34238_skew2.6669_neigh1.18845_cross0.8908_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_random_bw843.39606_skew2.6669_neigh1.22114_cross0.77348_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_random_bw875.30678_skew2.6669_neigh1.20367_cross0.84315_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_random_bw905.63736_skew2.6669_neigh1.21901_cross0.77413_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_random_bw922.58182_skew2.6669_neigh1.21558_cross0.79594_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_random_bw932.78459_skew2.6669_neigh1.03232_cross0.77348_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_random_bw935.45003_skew2.6669_neigh1.17793_cross0.77396_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_random_bw936.96549_skew2.6669_neigh1.25866_cross0.77239_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_random_bw939.4777_skew2.6669_neigh0.88501_cross0.77405_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_random_bw940.09284_skew2.6669_neigh0.96014_cross0.7732_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_random_bw940.09785_skew2.6669_neigh1.10807_cross0.77234_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_random_bw943.56437_skew2.6669_neigh1.32906_cross0.77341_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_random_bw946.71836_skew2.6669_neigh1.40773_cross0.77338_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_random_bw952.76706_skew2.6669_neigh1.22251_cross0.74756_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_random_bw956.49716_skew2.6669_neigh1.48195_cross0.77307_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_random_bw963.6331_skew2.6669_neigh1.22154_cross0.77193_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_random_bw974.85482_skew2.6669_neigh1.23062_cross0.69785_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_random_bw975.63003_skew2.6669_neigh1.56256_cross0.77213_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_random_bw995.95674_skew2.6669_neigh1.23589_cross0.65094_n14.mtx","synthetic_38744_38744_1765368_avg45.56494_std31.78636_random_bw934.44368_skew2.75289_neigh1.21894_cross0.77198_n14.mtx","synthetic_38744_38744_1765877_avg45.57808_std31.07073_random_bw933.8017_skew2.72986_neigh1.22255_cross0.76872_n14.mtx","synthetic_38744_38744_1766496_avg45.59405_std32.481_random_bw931.2818_skew2.88209_neigh1.21975_cross0.76797_n14.mtx","synthetic_38744_38744_1767073_avg45.60895_std26.86577_random_bw926.3303_skew3.122_neigh1.22395_cross0.74655_n14.mtx","synthetic_38744_38744_1767160_avg45.61119_std30.25452_random_bw933.41929_skew2.83678_neigh1.21906_cross0.76087_n14.mtx","synthetic_38744_38744_1767330_avg45.61558_std33.08901_random_bw930.75617_skew3.03371_neigh1.22002_cross0.7649_n14.mtx","synthetic_38744_38744_1767431_avg45.61819_std29.2665_random_bw931.34116_skew2.94579_neigh1.22054_cross0.7542_n14.mtx","synthetic_38744_38744_1767460_avg45.61893_std28.1289_random_bw924.86287_skew3.03341_neigh1.22219_cross0.75561_n14.mtx","synthetic_38744_38744_1767651_avg45.62386_std33.63482_random_bw928.32175_skew3.16449_neigh1.22329_cross0.76327_n14.mtx","synthetic_38744_38744_1767760_avg45.62668_std34.12282_random_bw928.16813_skew3.31765_neigh1.22071_cross0.76095_n14.mtx","synthetic_38744_38744_1824565_avg47.09284_std32.11946_random_bw938.10585_skew2.7373_neigh1.22202_cross0.76943_n14.mtx","synthetic_38744_38744_1943737_avg50.16872_std33.41096_random_bw937.69409_skew2.82709_neigh1.22997_cross0.76198_n14.mtx","synthetic_38744_38744_2062005_avg53.22127_std34.5782_random_bw928.18411_skew2.90821_neigh1.23817_cross0.75999_n14.mtx","synthetic_38744_38744_2179882_avg56.26373_std35.63951_random_bw931.14637_skew2.98125_neigh1.24047_cross0.75443_n14.mtx","synthetic_38744_38744_2297582_avg59.30162_std36.62621_random_bw928.90731_skew3.03024_neigh1.24558_cross0.75336_n14.mtx","synthetic_39029_39029_1974236_avg50.58382_std27.80163_random_bw6199.88865_skew3.11199_neigh1.63568_cross0.76891_n14.mtx","synthetic_40035_40035_1824084_avg45.56223_std31.3971_random_bw969.35456_skew2.66532_neigh1.21894_cross0.77386_n14.mtx","synthetic_403132_403132_1612528_avg4.0_std0.0_random_bw553.55446_skew0.0_neigh0.37461_cross0.98908_n14.mtx","synthetic_40960_40960_1597440_avg39.0_std0.0_random_bw9995.91001_skew0.0_neigh1.40666_cross0.80631_n14.mtx","synthetic_42151_42151_2133812_avg50.62305_std27.78297_random_bw6697.26877_skew3.12855_neigh1.63324_cross0.77066_n14.mtx","synthetic_42618_42618_1942443_avg45.578_std31.38541_random_bw1030.63919_skew2.66405_neigh1.21482_cross0.7709_n14.mtx","synthetic_43715_43715_2801362_avg64.0824_std14.01594_random_bw314.71989_skew0.93501_neigh1.5866_cross0.83319_n14.mtx","synthetic_438187_438187_1752748_avg4.0_std0.0_random_bw605.4017_skew0.0_neigh0.36958_cross0.98958_n14.mtx","synthetic_44236_44236_1725204_avg39.0_std0.0_random_bw10814.82767_skew0.0_neigh1.40668_cross0.80597_n14.mtx","synthetic_45201_45201_2060190_avg45.57842_std31.38204_random_bw1097.18524_skew2.68596_neigh1.21466_cross0.77308_n14.mtx","synthetic_45273_45273_2292008_avg50.62638_std27.77577_random_bw7230.17452_skew3.12828_neigh1.63423_cross0.77066_n14.mtx","synthetic_46835_46835_1659331_avg35.42929_std24.19453_random_bw7661.97799_skew2.92331_neigh1.59887_cross0.77081_n14.mtx","synthetic_46835_46835_1817944_avg38.81593_std25.43637_random_bw7818.83557_skew3.04473_neigh1.61137_cross0.76248_n14.mtx","synthetic_46835_46835_1976036_avg42.19144_std26.52871_random_bw7433.66969_skew3.14776_neigh1.62142_cross0.75764_n14.mtx","synthetic_46835_46835_2134025_avg45.56475_std27.52318_random_bw7560.21004_skew3.21378_neigh1.62711_cross0.75172_n14.mtx","synthetic_46835_46835_2292218_avg48.94241_std27.77072_random_bw7378.23094_skew3.16816_neigh1.63162_cross0.7647_n14.mtx","synthetic_46835_46835_2370905_avg50.6225_std27.74621_random_bw7334.7539_skew2.81253_neigh1.63507_cross0.75275_n14.mtx","synthetic_46835_46835_2371001_avg50.62455_std27.39541_random_bw7354.04117_skew2.69386_neigh1.63657_cross0.75406_n14.mtx","synthetic_46835_46835_2371113_avg50.62695_std32.27909_random_bw7361.07123_skew3.10848_neigh1.63754_cross0.75502_n14.mtx","synthetic_46835_46835_2371156_avg50.62786_std27.75732_random_bw7391.41134_skew2.91089_neigh1.63514_cross0.76051_n14.mtx","synthetic_46835_46835_2371164_avg50.62803_std30.41679_random_bw7309.21574_skew3.22691_neigh1.63831_cross0.75152_n14.mtx","synthetic_46835_46835_2371201_avg50.62882_std33.12436_random_bw7335.62733_skew3.04908_neigh1.63667_cross0.75847_n14.mtx","synthetic_46835_46835_2371202_avg50.62885_std27.76569_random_bw7416.01217_skew3.00957_neigh1.63418_cross0.76445_n14.mtx","synthetic_46835_46835_2371250_avg50.62987_std31.37317_random_bw7354.67208_skew3.1675_neigh1.63654_cross0.75207_n14.mtx","synthetic_46835_46835_2371308_avg50.63111_std28.69655_random_bw7409.30012_skew3.1674_neigh1.638_cross0.76266_n14.mtx","synthetic_46835_46835_2371384_avg50.63273_std27.77434_random_bw7430.84482_skew3.08826_neigh1.63371_cross0.76567_n14.mtx","synthetic_46835_46835_2371440_avg50.63393_std27.7783_random_bw7484.63769_skew3.18692_neigh1.63258_cross0.77158_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_random_bw5235.2982_skew3.12766_neigh1.63485_cross0.7661_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_random_bw5745.43256_skew3.12766_neigh1.63461_cross0.76806_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_random_bw6229.28771_skew3.12766_neigh1.63338_cross0.76748_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_random_bw6661.50364_skew3.12766_neigh1.555_cross0.88602_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_random_bw6744.12087_skew3.12766_neigh1.63266_cross0.76858_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_random_bw7030.71878_skew3.12766_neigh1.60145_cross0.84428_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_random_bw7051.92288_skew3.12766_neigh1.1472_cross0.76824_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_random_bw7076.9514_skew3.12766_neigh1.25341_cross0.77005_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_random_bw7108.27014_skew3.12766_neigh1.36391_cross0.76831_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_random_bw7209.39063_skew3.12766_neigh1.57945_cross0.76788_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_random_bw7217.16063_skew3.12766_neigh1.47014_cross0.76969_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_random_bw7237.79114_skew3.12766_neigh1.63478_cross0.7678_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_random_bw7362.32181_skew3.12766_neigh1.62253_cross0.79466_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_random_bw7533.90896_skew3.12766_neigh1.68673_cross0.77234_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_random_bw7581.89267_skew3.12766_neigh1.64059_cross0.74279_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_random_bw7694.42259_skew3.12766_neigh1.63441_cross0.76753_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_random_bw7725.2259_skew3.12766_neigh1.64727_cross0.69381_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_random_bw7852.92027_skew3.12766_neigh1.65613_cross0.64131_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_random_bw7964.00444_skew3.12766_neigh1.66283_cross0.58936_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_random_bw8050.7121_skew3.12766_neigh1.66655_cross0.53913_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_random_bw8187.29021_skew3.12766_neigh1.63352_cross0.76751_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_random_bw8518.3466_skew3.12766_neigh1.89786_cross0.7699_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_random_bw8759.33535_skew3.12766_neigh1.63533_cross0.76743_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_random_bw9196.04473_skew3.12766_neigh1.63338_cross0.76885_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_random_bw9256.19861_skew3.12766_neigh1.79462_cross0.76946_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_random_bw9716.25818_skew3.12766_neigh1.63456_cross0.76651_n14.mtx","synthetic_46835_46835_2371578_avg50.63687_std27.78811_random_bw7532.72126_skew3.36441_neigh1.63252_cross0.77731_n14.mtx","synthetic_46835_46835_2371669_avg50.63882_std26.85583_random_bw7496.12448_skew3.10752_neigh1.63286_cross0.77698_n14.mtx","synthetic_46835_46835_2371693_avg50.63933_std27.78141_random_bw7520.67403_skew3.26546_neigh1.63151_cross0.77728_n14.mtx","synthetic_46835_46835_2371771_avg50.64099_std27.79721_random_bw7596.83937_skew3.60102_neigh1.63043_cross0.7826_n14.mtx","synthetic_46835_46835_2371874_avg50.64319_std27.79128_random_bw7573.47232_skew3.48234_neigh1.63324_cross0.78289_n14.mtx","synthetic_46835_46835_2371944_avg50.64469_std25.00834_random_bw7661.37113_skew3.02806_neigh1.62991_cross0.78443_n14.mtx","synthetic_46835_46835_2372249_avg50.6512_std23.16377_random_bw7721.62859_skew3.0078_neigh1.63_cross0.7957_n14.mtx","synthetic_46835_46835_2372391_avg50.65423_std21.31556_random_bw7865.19097_skew2.98782_neigh1.6257_cross0.80882_n14.mtx","synthetic_46835_46835_2372900_avg50.6651_std19.46618_random_bw7985.40892_skew2.96723_neigh1.62267_cross0.81739_n14.mtx","synthetic_46835_46835_2450659_avg52.32538_std27.78086_random_bw7558.44102_skew3.1089_neigh1.63528_cross0.77453_n14.mtx","synthetic_46835_46835_2609003_avg55.70627_std27.78687_random_bw7460.64719_skew3.03904_neigh1.63822_cross0.78391_n14.mtx","synthetic_46835_46835_2767329_avg59.08677_std27.79319_random_bw7641.74269_skew3.01105_neigh1.64146_cross0.79361_n14.mtx","synthetic_46835_46835_2925939_avg62.47334_std27.79699_random_bw7575.97229_skew3.00171_neigh1.64383_cross0.80098_n14.mtx","synthetic_46835_46835_3084303_avg65.85466_std27.80201_random_bw7666.96814_skew2.97846_neigh1.64609_cross0.80651_n14.mtx","synthetic_473242_473242_1892968_avg4.0_std0.0_random_bw643.34994_skew0.0_neigh0.37806_cross0.99029_n14.mtx","synthetic_47513_47513_1853007_avg39.0_std0.0_random_bw11630.5911_skew0.0_neigh1.40553_cross0.80427_n14.mtx","synthetic_47784_47784_2177960_avg45.57927_std31.38054_random_bw1153.05592_skew2.68589_neigh1.21393_cross0.77217_n14.mtx","synthetic_47879_47879_3068257_avg64.08356_std14.01465_random_bw343.15784_skew0.93497_neigh1.58587_cross0.83121_n14.mtx","synthetic_48396_48396_2450061_avg50.62528_std27.78261_random_bw7683.6283_skew3.14813_neigh1.6334_cross0.76917_n14.mtx","synthetic_49152_49152_1327104_avg27.0_std0.0_random_bw11888.55758_skew0.0_neigh1.39065_cross0.80136_n14.mtx","synthetic_49152_49152_1474560_avg30.0_std0.0_random_bw12201.67973_skew0.0_neigh1.39972_cross0.80562_n14.mtx","synthetic_49152_49152_1622016_avg33.0_std0.0_random_bw12114.60683_skew0.0_neigh1.3977_cross0.80501_n14.mtx","synthetic_49152_49152_1720320_avg35.0_std0.0_random_bw11976.30068_skew0.0_neigh1.40142_cross0.80302_n14.mtx","synthetic_49152_49152_1867776_avg38.0_std0.0_random_bw11962.14925_skew0.0_neigh1.40582_cross0.80314_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_random_bw10814.92206_skew0.0_neigh1.40656_cross0.80518_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_random_bw11602.43638_skew0.0_neigh1.40627_cross0.80459_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_random_bw11997.39925_skew0.0_neigh1.17226_cross0.80303_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_random_bw11999.34074_skew0.0_neigh1.41023_cross0.99003_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_random_bw12019.23385_skew0.0_neigh1.26333_cross0.80653_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_random_bw12021.09155_skew0.0_neigh1.35943_cross0.80353_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_random_bw12028.01392_skew0.0_neigh1.40369_cross0.56602_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_random_bw12033.44261_skew0.0_neigh1.40424_cross0.72534_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_random_bw12034.67322_skew0.0_neigh1.40695_cross0.78004_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_random_bw12036.00016_skew0.0_neigh1.45087_cross0.8025_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_random_bw12044.37571_skew0.0_neigh1.40455_cross0.62007_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_random_bw12045.43215_skew0.0_neigh1.40443_cross0.67088_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_random_bw12050.9789_skew0.0_neigh1.07727_cross0.806_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_random_bw12058.91376_skew0.0_neigh1.40774_cross0.83098_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_random_bw12073.32371_skew0.0_neigh1.40561_cross0.80444_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_random_bw12085.13057_skew0.0_neigh0.9811_cross0.8051_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_random_bw12096.1748_skew0.0_neigh1.40517_cross0.88682_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_random_bw12100.19297_skew0.0_neigh1.40732_cross0.93736_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_random_bw12100.23661_skew0.0_neigh1.54428_cross0.80491_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_random_bw12254.45363_skew0.0_neigh1.63837_cross0.80419_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_random_bw12401.00248_skew0.0_neigh1.40613_cross0.80523_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_random_bw12966.94114_skew0.0_neigh1.73293_cross0.80316_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_random_bw13193.13125_skew0.0_neigh1.40663_cross0.80616_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_random_bw14038.84108_skew0.0_neigh1.40505_cross0.80485_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_random_bw14826.38826_skew0.0_neigh1.4046_cross0.80484_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_random_bw15659.63725_skew0.0_neigh1.40594_cross0.80484_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_random_bw17726.56396_skew0.0_neigh1.82624_cross0.8031_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_random_bw8412.1531_skew0.0_neigh1.40648_cross0.80556_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_random_bw9217.67674_skew0.0_neigh1.40509_cross0.80535_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_random_bw9999.7901_skew0.0_neigh1.40701_cross0.8066_n14.mtx","synthetic_49152_49152_1966080_avg40.0_std0.0_random_bw12103.00734_skew0.0_neigh1.40894_cross0.80074_n14.mtx","synthetic_49152_49152_2113536_avg43.0_std0.0_random_bw12029.22158_skew0.0_neigh1.40822_cross0.80163_n14.mtx","synthetic_49152_49152_2260992_avg46.0_std0.0_random_bw11993.41239_skew0.0_neigh1.41259_cross0.8045_n14.mtx","synthetic_49152_49152_2359296_avg48.0_std0.0_random_bw12095.14152_skew0.0_neigh1.41171_cross0.80651_n14.mtx","synthetic_49152_49152_2506752_avg51.0_std0.0_random_bw12095.6193_skew0.0_neigh1.41564_cross0.80645_n14.mtx","synthetic_50367_50367_2296192_avg45.58922_std31.37796_random_bw1217.66939_skew2.68508_neigh1.21323_cross0.76936_n14.mtx","synthetic_50790_50790_1980810_avg39.0_std0.0_random_bw12402.57342_skew0.0_neigh1.40626_cross0.80547_n14.mtx","synthetic_508297_508297_2033188_avg4.0_std0.0_random_bw695.22929_skew0.0_neigh0.36764_cross0.99098_n14.mtx","synthetic_51518_51518_2608832_avg50.63923_std27.762_random_bw8203.55775_skew3.14698_neigh1.63544_cross0.76613_n14.mtx","synthetic_52042_52042_3335826_avg64.09873_std14.0_random_bw372.49024_skew0.93452_neigh1.58046_cross0.82963_n14.mtx","synthetic_525825_525825_1577449_avg2.99995_std0.00703_random_bw859.08869_skew2e-05_neigh0.3293_cross0.99111_n14.mtx","synthetic_525825_525825_1577475_avg3.0_std0.0_random_bw858.55443_skew0.0_neigh0.33003_cross0.99112_n14.mtx","synthetic_525825_525825_1583911_avg3.01224_std0.10995_random_bw797.16394_skew0.32792_neigh0.26282_cross0.98853_n14.mtx","synthetic_525825_525825_2046881_avg3.8927_std0.30949_random_bw622.52163_skew0.02756_neigh0.1362_cross0.96883_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_random_bw505.76605_skew0.0_neigh0.37416_cross0.99125_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_random_bw554.92392_skew0.0_neigh0.37192_cross0.99125_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_random_bw606.93324_skew0.0_neigh0.37258_cross0.99124_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_random_bw644.53081_skew0.0_neigh0.37119_cross0.99121_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_random_bw670.66979_skew0.0_neigh0.48412_cross0.99123_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_random_bw683.49694_skew0.0_neigh0.45923_cross0.9912_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_random_bw695.20509_skew0.0_neigh0.4342_cross0.9912_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_random_bw702.53172_skew0.0_neigh0.37156_cross0.99124_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_random_bw705.68994_skew0.0_neigh0.40842_cross0.99122_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_random_bw714.99184_skew0.0_neigh0.38412_cross0.99123_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_random_bw717.31533_skew0.0_neigh0.3775_cross0.95838_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_random_bw719.01499_skew0.0_neigh0.37393_cross0.8926_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_random_bw719.86053_skew0.0_neigh0.37171_cross0.99223_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_random_bw719.98069_skew0.0_neigh0.37726_cross0.76297_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_random_bw720.4176_skew0.0_neigh0.37153_cross0.99122_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_random_bw720.57144_skew0.0_neigh0.37696_cross0.69668_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_random_bw720.84278_skew0.0_neigh0.37761_cross0.82791_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_random_bw726.03436_skew0.0_neigh0.35739_cross0.9912_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_random_bw737.14928_skew0.0_neigh0.33322_cross0.99119_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_random_bw737.4945_skew0.0_neigh0.3712_cross0.99123_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_random_bw744.2048_skew0.0_neigh0.30997_cross0.99121_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_random_bw756.76312_skew0.0_neigh0.28196_cross0.99121_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_random_bw766.29148_skew0.0_neigh0.26012_cross0.9912_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_random_bw792.51217_skew0.0_neigh0.37127_cross0.99121_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_random_bw833.10523_skew0.0_neigh0.37123_cross0.99121_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_random_bw890.92849_skew0.0_neigh0.37114_cross0.9912_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_random_bw939.84754_skew0.0_neigh0.3724_cross0.99121_n14.mtx","synthetic_525825_525825_2146618_avg4.08238_std0.27494_random_bw490.88949_skew0.22478_neigh0.18414_cross0.97729_n14.mtx","synthetic_525825_525825_2619767_avg4.9822_std0.13221_random_bw654.36998_skew0.00357_neigh0.31537_cross0.98835_n14.mtx","synthetic_525825_525825_2629125_avg5.0_std0.0_random_bw685.43441_skew0.0_neigh0.39914_cross0.9912_n14.mtx","synthetic_525825_525825_2629143_avg5.00003_std0.00585_random_bw685.37638_skew0.19999_neigh0.3989_cross0.9912_n14.mtx","synthetic_54067_54067_2108613_avg39.0_std0.0_random_bw13195.94642_skew0.0_neigh1.40596_cross0.80602_n14.mtx","synthetic_543352_543352_2173408_avg4.0_std0.0_random_bw737.48539_skew0.0_neigh0.3826_cross0.99144_n14.mtx","synthetic_54640_54640_2767687_avg50.65313_std27.75459_random_bw8703.93389_skew3.14584_neigh1.6342_cross0.76941_n14.mtx","synthetic_56205_56205_3603711_avg64.11727_std13.99803_random_bw400.50616_skew0.93396_neigh1.57644_cross0.82872_n14.mtx","synthetic_57344_57344_2236416_avg39.0_std0.0_random_bw14033.40515_skew0.0_neigh1.40406_cross0.80608_n14.mtx","synthetic_57763_57763_2926741_avg50.66809_std27.74629_random_bw9179.53278_skew3.16436_neigh1.63113_cross0.77117_n14.mtx","synthetic_578407_578407_2313628_avg4.0_std0.0_random_bw786.14351_skew0.0_neigh0.37361_cross0.99189_n14.mtx","synthetic_60369_60369_3871327_avg64.12773_std13.99319_random_bw430.33024_skew0.93364_neigh1.57281_cross0.82791_n14.mtx","synthetic_60620_60620_2364180_avg39.0_std0.0_random_bw14835.70082_skew0.0_neigh1.4037_cross0.80546_n14.mtx","synthetic_60885_60885_3084540_avg50.66174_std27.74397_random_bw9701.3275_skew3.16488_neigh1.63253_cross0.76879_n14.mtx","synthetic_613462_613462_2453848_avg4.0_std0.0_random_bw842.88409_skew0.0_neigh0.38222_cross0.9922_n14.mtx","synthetic_62451_62451_2802494_avg44.87509_std13.98221_random_bw433.69895_skew1.33983_neigh1.53816_cross0.79072_n14.mtx","synthetic_62451_62451_3069539_avg49.15116_std13.98917_random_bw442.35894_skew1.21765_neigh1.54755_cross0.80178_n14.mtx","synthetic_62451_62451_3336828_avg53.43114_std13.99111_random_bw442.79608_skew1.11487_neigh1.55465_cross0.80814_n14.mtx","synthetic_62451_62451_3603802_avg57.70607_std13.99012_random_bw440.31634_skew1.04485_neigh1.56479_cross0.81778_n14.mtx","synthetic_62451_62451_3870981_avg61.98429_std13.98816_random_bw440.14062_skew0.96824_neigh1.57158_cross0.82571_n14.mtx","synthetic_62451_62451_4003815_avg64.1113_std18.18532_random_bw433.90471_skew1.2149_neigh1.57849_cross0.80426_n14.mtx","synthetic_62451_62451_4004004_avg64.11433_std17.25509_random_bw437.19148_skew1.1524_neigh1.57787_cross0.80809_n14.mtx","synthetic_62451_62451_4004101_avg64.11588_std16.32075_random_bw439.33923_skew1.08997_neigh1.57568_cross0.81415_n14.mtx","synthetic_62451_62451_4004374_avg64.12025_std15.38744_random_bw440.62742_skew1.02744_neigh1.5754_cross0.81871_n14.mtx","synthetic_62451_62451_4004540_avg64.12291_std14.45551_random_bw443.23719_skew0.96498_neigh1.573_cross0.82479_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_random_bw314.5764_skew0.93375_neigh1.58679_cross0.83355_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_random_bw343.48287_skew0.93375_neigh1.58569_cross0.83066_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_random_bw372.86596_skew0.93375_neigh1.57974_cross0.82953_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_random_bw400.84485_skew0.93375_neigh1.57624_cross0.82892_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_random_bw430.23839_skew0.93375_neigh1.57299_cross0.82774_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_random_bw458.26891_skew0.93375_neigh1.57086_cross0.82777_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_random_bw485.32971_skew0.93375_neigh1.56912_cross0.82716_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_random_bw514.6847_skew0.93375_neigh1.56848_cross0.82621_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_random_bw541.61408_skew0.93375_neigh1.56544_cross0.82639_n14.mtx","synthetic_62451_62451_4004768_avg64.12656_std12.59183_random_bw445.64555_skew0.84011_neigh1.57031_cross0.83683_n14.mtx","synthetic_62451_62451_4004839_avg64.1277_std13.52372_random_bw443.69469_skew0.90245_neigh1.57245_cross0.83204_n14.mtx","synthetic_62451_62451_4005045_avg64.131_std11.65916_random_bw450.22189_skew0.77761_neigh1.56813_cross0.8418_n14.mtx","synthetic_62451_62451_4005321_avg64.13542_std10.72592_random_bw451.83634_skew0.71512_neigh1.56389_cross0.84902_n14.mtx","synthetic_62451_62451_4005505_avg64.13836_std9.79513_random_bw454.37802_skew0.65268_neigh1.56244_cross0.85409_n14.mtx","synthetic_62451_62451_4138184_avg66.26289_std13.99067_random_bw440.8832_skew0.90152_neigh1.57532_cross0.83038_n14.mtx","synthetic_62451_62451_4405420_avg70.54202_std13.98985_random_bw441.29735_skew0.85705_neigh1.58148_cross0.83621_n14.mtx","synthetic_62451_62451_4672484_avg74.8184_std13.9889_random_bw446.77627_skew0.80437_neigh1.58533_cross0.84209_n14.mtx","synthetic_62451_62451_4939666_avg79.09667_std13.99063_random_bw446.66801_skew0.75734_neigh1.59109_cross0.84662_n14.mtx","synthetic_62451_62451_5206981_avg83.37706_std13.99111_random_bw448.16022_skew0.7151_neigh1.59447_cross0.85123_n14.mtx","synthetic_63897_63897_2491983_avg39.0_std0.0_random_bw15621.79835_skew0.0_neigh1.40542_cross0.80575_n14.mtx","synthetic_64532_64532_4137790_avg64.11997_std13.98737_random_bw458.31169_skew0.93387_neigh1.5704_cross0.82793_n14.mtx","synthetic_648517_648517_2594068_avg4.0_std0.0_random_bw886.62276_skew0.0_neigh0.37472_cross0.99258_n14.mtx","synthetic_683572_683572_2734288_avg4.0_std0.0_random_bw924.48268_skew0.0_neigh0.37975_cross0.9929_n14.mtx","synthetic_68696_68696_4403097_avg64.09539_std13.99668_random_bw485.33335_skew0.93462_neigh1.56937_cross0.82685_n14.mtx","synthetic_700003_700003_2220092_avg3.17155_std25.65674_random_bw87391.5121_skew1481.87283_neigh0.14428_cross0.90055_n14.mtx","synthetic_72859_72859_4668546_avg64.07645_std14.00516_random_bw513.63554_skew0.93519_neigh1.56865_cross0.82618_n14.mtx","synthetic_766670_766670_2431595_avg3.17163_std25.62988_random_bw95810.1159_skew1481.83288_neigh0.14367_cross0.90161_n14.mtx","synthetic_77022_77022_4936326_avg64.08982_std14.01488_random_bw542.85377_skew0.93478_neigh1.56641_cross0.8258_n14.mtx","synthetic_81186_81186_5203811_avg64.09739_std14.01954_random_bw572.64595_skew0.93456_neigh1.56212_cross0.82312_n14.mtx","synthetic_833337_833337_2642605_avg3.17111_std25.60679_random_bw103837.45496_skew1482.07595_neigh0.14411_cross0.90125_n14.mtx","synthetic_84834_84834_1835279_avg21.63377_std13.7861_random_bw46227.07554_skew4.03842_neigh1.01517_cross0.55965_n14.mtx","synthetic_900004_900004_2853888_avg3.17097_std25.58712_random_bw112429.88878_skew1482.14118_neigh0.14011_cross0.90178_n14.mtx","synthetic_92913_92913_2009995_avg21.63309_std13.78454_random_bw50425.51531_skew4.03858_neigh1.01427_cross0.56119_n14.mtx","synthetic_966671_966671_3064966_avg3.17064_std25.57082_random_bw120972.93224_skew1482.29662_neigh0.14283_cross0.90232_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_random_bw12314.98414_skew1.0109_neigh1.69561_cross0.82051_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_random_bw13218.95443_skew1.0109_neigh1.63894_cross0.89577_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_random_bw13302.83667_skew1.0109_neigh1.69559_cross0.82053_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_random_bw13936.21876_skew1.0109_neigh1.41417_cross0.8198_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_random_bw14004.39261_skew1.0109_neigh1.52687_cross0.8201_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_random_bw14289.87368_skew1.0109_neigh1.69536_cross0.81999_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_random_bw14335.05046_skew1.0109_neigh1.68453_cross0.84715_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_random_bw14787.04212_skew1.0109_neigh1.69513_cross0.82047_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_random_bw15130.18363_skew1.0109_neigh1.7031_cross0.79329_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_random_bw16264.19116_skew1.0109_neigh1.69516_cross0.82009_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_random_bw16429.30637_skew1.0109_neigh1.72541_cross0.63001_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_random_bw16689.96273_skew1.0109_neigh1.72925_cross0.57549_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_random_bw17252.2932_skew1.0109_neigh1.69476_cross0.8205_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_random_bw18227.45253_skew1.0109_neigh1.69515_cross0.82059_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_random_bw20444.65782_skew1.0109_neigh1.91237_cross0.8206_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_random_bw27298.86084_skew1.0109_neigh1.86289_cross0.82084_n14.mtx","synthetic_1006441_1006441_22414673_avg22.27122_std29.74727_random_bw254733.54551_skew313.62123_neigh1.26289_cross0.75654_n14.mtx","synthetic_1019593_1019593_74066178_avg72.64289_std15.80935_random_bw15286.94394_skew1.03736_neigh1.69506_cross0.8204_n14.mtx","synthetic_102778_102778_7408141_avg72.07905_std19.01282_random_bw5890.51486_skew1.19204_neigh1.65342_cross0.76938_n14.mtx","synthetic_1047423_1047423_51169710_avg48.85296_std11.94333_random_bw189507.21493_skew1.10837_neigh1.69026_cross0.8037_n14.mtx","synthetic_1060229_1060229_12992547_avg12.25447_std37.61142_random_bw24980.56284_skew632.64621_neigh1.32193_cross0.78513_n14.mtx","synthetic_1063952_1063952_23695040_avg22.27078_std29.72463_random_bw269168.5156_skew313.62752_neigh1.26394_cross0.75677_n14.mtx","synthetic_108003_108003_5987303_avg55.43645_std11.04356_random_bw4243.01828_skew0.89406_neigh1.62997_cross0.78726_n14.mtx","synthetic_108334_108334_7808284_avg72.07602_std19.0204_random_bw6217.09331_skew1.19213_neigh1.65302_cross0.76997_n14.mtx","synthetic_1085373_1085373_78800672_avg72.60239_std15.80629_random_bw16260.36911_skew1.02473_neigh1.69518_cross0.82006_n14.mtx","synthetic_1110903_1110903_54291613_avg48.87161_std11.93583_random_bw200760.47456_skew1.10756_neigh1.68972_cross0.80443_n14.mtx","synthetic_1121463_1121463_24976074_avg22.27097_std29.7046_random_bw283780.07078_skew313.62476_neigh1.26318_cross0.75684_n14.mtx","synthetic_112749_112749_5727652_avg50.80002_std19.54637_random_bw3458.33754_skew1.7559_neigh1.22842_cross0.71885_n14.mtx","synthetic_1151153_1151153_83612943_avg72.63408_std15.81129_random_bw17250.83068_skew1.01008_neigh1.69488_cross0.82066_n14.mtx","synthetic_1152423_1152423_14123090_avg12.25513_std37.58092_random_bw27186.1267_skew632.61237_neigh1.32195_cross0.78618_n14.mtx","synthetic_117394_117394_6507908_avg55.43646_std11.04171_random_bw4592.46353_skew0.89406_neigh1.63085_cross0.78812_n14.mtx","synthetic_1174383_1174383_57377479_avg48.85755_std11.94355_random_bw212160.93638_skew1.10817_neigh1.68973_cross0.80444_n14.mtx","synthetic_1216933_1216933_88363297_avg72.61147_std15.79644_random_bw18231.88915_skew1.03825_neigh1.69444_cross0.82088_n14.mtx","synthetic_123486_123486_6272129_avg50.79223_std19.54882_random_bw3787.53287_skew1.75633_neigh1.22949_cross0.71628_n14.mtx","synthetic_1237863_1237863_60496296_avg48.87156_std11.94811_random_bw223383.86547_skew1.20987_neigh1.69022_cross0.80452_n14.mtx","synthetic_1244617_1244617_15251343_avg12.25384_std37.55512_random_bw29349.06445_skew632.67869_neigh1.32246_cross0.78617_n14.mtx","synthetic_126786_126786_7028939_avg55.43939_std11.04404_random_bw4958.06756_skew0.89396_neigh1.63099_cross0.78932_n14.mtx","synthetic_1282713_1282713_93191153_avg72.6516_std15.81312_random_bw19218.72077_skew1.10594_neigh1.69435_cross0.82082_n14.mtx","synthetic_129947_129947_10504776_avg80.83893_std117.70867_random_bw17726.53455_skew7.85712_neigh1.21404_cross0.84553_n14.mtx","synthetic_1336811_1336811_16381252_avg12.25398_std37.5327_random_bw31479.27429_skew632.6718_neigh1.32292_cross0.78634_n14.mtx","synthetic_134225_134225_6816930_avg50.78733_std19.56874_random_bw4113.48627_skew1.77628_neigh1.22748_cross0.71779_n14.mtx","synthetic_136178_136178_7548659_avg55.43229_std11.05483_random_bw5329.09974_skew0.91224_neigh1.63079_cross0.78836_n14.mtx","synthetic_1382908_1382908_11844497_avg8.56492_std37.37396_random_bw40487.08926_skew633.56515_neigh1.20543_cross0.77916_n14.mtx","synthetic_1382908_1382908_12976596_avg9.38356_std37.40231_random_bw42638.77576_skew633.40762_neigh1.23946_cross0.78166_n14.mtx","synthetic_1382908_1382908_14113272_avg10.2055_std37.43376_random_bw29869.67508_skew632.97168_neigh1.26037_cross0.782_n14.mtx","synthetic_1382908_1382908_15242471_avg11.02204_std37.4678_random_bw31130.67806_skew633.00226_neigh1.26986_cross0.78321_n14.mtx","synthetic_1382908_1382908_16349573_avg11.8226_std37.50438_random_bw32032.95325_skew633.88554_neigh1.28645_cross0.78343_n14.mtx","synthetic_1382908_1382908_16848093_avg12.18309_std26.47929_random_bw31499.6008_skew636.35882_neigh1.38178_cross0.79033_n14.mtx","synthetic_1382908_1382908_16896659_avg12.21821_std28.92386_random_bw31578.67454_skew634.52686_neigh1.37766_cross0.7904_n14.mtx","synthetic_1382908_1382908_16922141_avg12.23664_std48.62398_random_bw32402.87716_skew633.48814_neigh1.28006_cross0.77973_n14.mtx","synthetic_1382908_1382908_16923406_avg12.23755_std46.1541_random_bw32550.44114_skew633.44071_neigh1.28373_cross0.78136_n14.mtx","synthetic_1382908_1382908_16923784_avg12.23782_std37.37393_random_bw32578.59823_skew443.68692_neigh1.28762_cross0.78233_n14.mtx","synthetic_1382908_1382908_16926342_avg12.23967_std43.68598_random_bw32633.88821_skew633.41236_neigh1.29181_cross0.78314_n14.mtx","synthetic_1382908_1382908_16927023_avg12.24017_std37.40259_random_bw32650.87537_skew485.75812_neigh1.29364_cross0.78327_n14.mtx","synthetic_1382908_1382908_16931896_avg12.24369_std37.43372_random_bw32726.41259_skew527.76219_neigh1.30038_cross0.78452_n14.mtx","synthetic_1382908_1382908_16931967_avg12.24374_std41.21881_random_bw32683.4221_skew633.2016_neigh1.29995_cross0.78417_n14.mtx","synthetic_1382908_1382908_16933417_avg12.24479_std31.37528_random_bw31839.54587_skew633.1473_neigh1.36655_cross0.78964_n14.mtx","synthetic_1382908_1382908_16938002_avg12.2481_std37.46714_random_bw32660.61927_skew569.78219_neigh1.30898_cross0.7854_n14.mtx","synthetic_1382908_1382908_16941719_avg12.25079_std38.75385_random_bw32663.46745_skew632.83654_neigh1.31388_cross0.78566_n14.mtx","synthetic_1382908_1382908_16943948_avg12.2524_std37.50354_random_bw32608.3439_skew611.69607_neigh1.31792_cross0.78598_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_random_bw16831.66287_skew632.66322_neigh1.72456_cross0.78841_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_random_bw22849.17172_skew632.66322_neigh1.32203_cross0.78644_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_random_bw25001.14321_skew632.66322_neigh1.32215_cross0.78651_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_random_bw26972.08395_skew632.66322_neigh1.63487_cross0.7871_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_random_bw27152.96238_skew632.66322_neigh1.32223_cross0.78644_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_random_bw28588.16229_skew632.66322_neigh0.52059_cross0.98566_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_random_bw28608.91048_skew632.66322_neigh1.10181_cross0.78592_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_random_bw28827.3735_skew632.66322_neigh0.92504_cross0.78425_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_random_bw29311.03639_skew632.66322_neigh1.3223_cross0.7864_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_random_bw29812.11226_skew632.66322_neigh1.01255_cross0.78502_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_random_bw30178.2994_skew632.66322_neigh1.18993_cross0.78639_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_random_bw31055.2687_skew632.66322_neigh1.36651_cross0.78696_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_random_bw31485.89914_skew632.66322_neigh1.32204_cross0.78641_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_random_bw31866.08413_skew632.66322_neigh1.3625_cross0.55123_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_random_bw31960.20546_skew632.66322_neigh1.35743_cross0.60367_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_random_bw32103.51377_skew632.66322_neigh1.3514_cross0.65603_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_random_bw32298.61541_skew632.66322_neigh1.342_cross0.70831_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_random_bw32527.47818_skew632.66322_neigh1.32971_cross0.7607_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_random_bw32649.42139_skew632.66322_neigh1.31221_cross0.81225_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_random_bw33004.14191_skew632.66322_neigh0.99375_cross0.96683_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_random_bw33018.43863_skew632.66322_neigh1.28269_cross0.8644_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_random_bw33385.39612_skew632.66322_neigh1.21925_cross0.91646_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_random_bw33686.12799_skew632.66322_neigh1.32166_cross0.7864_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_random_bw33939.34987_skew632.66322_neigh1.27774_cross0.78619_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_random_bw35035.98789_skew632.66322_neigh1.545_cross0.78785_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_random_bw41464.2511_skew632.66322_neigh1.45537_cross0.78695_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_random_bw42380.53696_skew632.66322_neigh1.32182_cross0.78634_n14.mtx","synthetic_1382908_1382908_16948048_avg12.25537_std37.72372_random_bw32047.73507_skew822.06782_neigh1.35662_cross0.78905_n14.mtx","synthetic_1382908_1382908_16948709_avg12.25585_std37.54211_random_bw32537.49715_skew653.62631_neigh1.32642_cross0.78675_n14.mtx","synthetic_1382908_1382908_16950422_avg12.25709_std36.29175_random_bw32502.49439_skew632.51111_neigh1.33052_cross0.78757_n14.mtx","synthetic_1382908_1382908_16950660_avg12.25726_std33.83118_random_bw32159.50342_skew632.50221_neigh1.34985_cross0.78899_n14.mtx","synthetic_1382908_1382908_16951601_avg12.25794_std37.67423_random_bw32164.80648_skew779.80014_neigh1.34967_cross0.78884_n14.mtx","synthetic_1382908_1382908_16951652_avg12.25798_std37.58358_random_bw32449.74613_skew695.60771_neigh1.33402_cross0.7879_n14.mtx","synthetic_1382908_1382908_16952713_avg12.25874_std37.62729_random_bw32268.27374_skew737.65652_neigh1.34279_cross0.78851_n14.mtx","synthetic_1382908_1382908_17445616_avg12.61517_std37.54329_random_bw32804.62199_skew635.05973_neigh1.31534_cross0.78529_n14.mtx","synthetic_1382908_1382908_18572951_avg13.43036_std37.58404_random_bw33577.21407_skew634.94726_neigh1.34983_cross0.78702_n14.mtx","synthetic_1382908_1382908_19756490_avg14.28619_std37.62781_random_bw28948.7818_skew633.10877_neigh1.36875_cross0.78902_n14.mtx","synthetic_1382908_1382908_20949793_avg15.14909_std37.6751_random_bw30001.28368_skew631.18333_neigh1.36337_cross0.78796_n14.mtx","synthetic_1382908_1382908_22070012_avg15.95913_std37.72535_random_bw30789.33457_skew631.49052_neigh1.34689_cross0.78668_n14.mtx","synthetic_140874_140874_10153211_avg72.07299_std11.05163_random_bw5566.17624_skew0.7066_neigh1.64681_cross0.80416_n14.mtx","synthetic_140874_140874_5465261_avg38.79538_std11.0515_random_bw5472.25499_skew1.31986_neigh1.60427_cross0.75763_n14.mtx","synthetic_140874_140874_5986164_avg42.49304_std11.0536_random_bw5615.53216_skew1.18859_neigh1.61127_cross0.76558_n14.mtx","synthetic_140874_140874_6506839_avg46.18907_std11.05195_random_bw5467.46501_skew1.10006_neigh1.61774_cross0.77319_n14.mtx","synthetic_140874_140874_7027827_avg49.88732_std11.05353_random_bw5578.52051_skew1.02456_neigh1.62272_cross0.78142_n14.mtx","synthetic_140874_140874_7548897_avg53.58616_std11.05321_random_bw5458.59888_skew0.9408_neigh1.6305_cross0.78364_n14.mtx","synthetic_140874_140874_7807971_avg55.42521_std14.36638_random_bw5391.79174_skew1.18312_neigh1.63652_cross0.76654_n14.mtx","synthetic_140874_140874_7808175_avg55.42666_std13.63005_random_bw5409.69514_skew1.12894_neigh1.63453_cross0.77116_n14.mtx","synthetic_140874_140874_7808374_avg55.42807_std12.89549_random_bw5451.61362_skew1.07476_neigh1.63304_cross0.77494_n14.mtx","synthetic_140874_140874_7808713_avg55.43048_std12.15675_random_bw5470.09146_skew1.00251_neigh1.63286_cross0.77934_n14.mtx","synthetic_140874_140874_7809044_avg55.43283_std11.42187_random_bw5501.94588_skew0.9483_neigh1.63246_cross0.78383_n14.mtx","synthetic_140874_140874_7809186_avg55.43383_std10.68354_random_bw5528.77301_skew0.89415_neigh1.63008_cross0.79051_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_random_bw3868.73909_skew0.91217_neigh1.62925_cross0.78945_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_random_bw4235.02181_skew0.91217_neigh1.62951_cross0.78894_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_random_bw4605.71603_skew0.91217_neigh1.63013_cross0.78855_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_random_bw4770.72113_skew0.91217_neigh1.56132_cross0.89987_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_random_bw4956.26364_skew0.91217_neigh1.63116_cross0.78903_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_random_bw5095.18835_skew0.91217_neigh1.59792_cross0.86514_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_random_bw5267.47496_skew0.91217_neigh1.14546_cross0.78537_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_random_bw5270.93334_skew0.91217_neigh1.256_cross0.78678_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_random_bw5285.20601_skew0.91217_neigh1.3622_cross0.78648_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_random_bw5319.89205_skew0.91217_neigh1.46878_cross0.78637_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_random_bw5329.16386_skew0.91217_neigh1.63054_cross0.78839_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_random_bw5371.77114_skew0.91217_neigh1.57638_cross0.78738_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_random_bw5396.20773_skew0.91217_neigh1.62321_cross0.81439_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_random_bw5522.95748_skew0.91217_neigh1.6306_cross0.7874_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_random_bw5613.49824_skew0.91217_neigh1.63776_cross0.76192_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_random_bw5693.34228_skew0.91217_neigh1.63074_cross0.78739_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_random_bw5768.02808_skew0.91217_neigh1.64697_cross0.7083_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_random_bw5802.87171_skew0.91217_neigh1.68497_cross0.78689_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_random_bw5877.42657_skew0.91217_neigh1.65421_cross0.65718_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_random_bw5969.91225_skew0.91217_neigh1.65949_cross0.60613_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_random_bw6034.73118_skew0.91217_neigh1.66318_cross0.55387_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_random_bw6073.33464_skew0.91217_neigh1.63029_cross0.78753_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_random_bw6436.17949_skew0.91217_neigh1.62906_cross0.78757_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_random_bw6525.25347_skew0.91217_neigh1.90151_cross0.7872_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_random_bw6689.33888_skew0.91217_neigh1.89964_cross0.78632_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_random_bw6800.23242_skew0.91217_neigh1.62946_cross0.78737_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_random_bw7164.09343_skew0.91217_neigh1.62998_cross0.78797_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_random_bw7287.18556_skew0.91217_neigh1.79126_cross0.78857_n14.mtx","synthetic_140874_140874_7809480_avg55.43592_std9.94668_random_bw5548.2026_skew0.82192_neigh1.62952_cross0.79532_n14.mtx","synthetic_140874_140874_7809970_avg55.4394_std9.2113_random_bw5619.82513_skew0.7677_neigh1.62745_cross0.7992_n14.mtx","synthetic_140874_140874_7810161_avg55.44076_std8.47606_random_bw5628.17949_skew0.6955_neigh1.62718_cross0.80558_n14.mtx","synthetic_140874_140874_7810358_avg55.44215_std7.74028_random_bw5680.4538_skew0.64135_neigh1.62737_cross0.8106_n14.mtx","synthetic_140874_140874_7812605_avg55.4581_std11.08015_random_bw6222.03617_skew2.11947_neigh1.61513_cross0.85242_n14.mtx","synthetic_140874_140874_7812630_avg55.45828_std11.08_random_bw6217.1087_skew2.06537_neigh1.61471_cross0.85345_n14.mtx","synthetic_140874_140874_8069568_avg57.28217_std11.05207_random_bw5567.22519_skew0.8854_neigh1.63271_cross0.78944_n14.mtx","synthetic_140874_140874_8590481_avg60.97989_std11.05315_random_bw5493.64159_skew0.83667_neigh1.63609_cross0.79354_n14.mtx","synthetic_140874_140874_9111507_avg64.67841_std11.05222_random_bw5571.17536_skew0.77803_neigh1.64018_cross0.79828_n14.mtx","synthetic_140874_140874_9632262_avg68.37502_std11.0533_random_bw5500.1344_skew0.7404_neigh1.64395_cross0.80172_n14.mtx","synthetic_142323_142323_11504673_avg80.83495_std117.71151_random_bw19450.52367_skew7.85755_neigh1.21346_cross0.84523_n14.mtx","synthetic_1429004_1429004_17510920_avg12.25393_std37.51291_random_bw33714.51176_skew632.67408_neigh1.3215_cross0.7862_n14.mtx","synthetic_144963_144963_7361596_avg50.78259_std19.57819_random_bw4444.28282_skew1.77654_neigh1.22789_cross0.7157_n14.mtx","synthetic_145569_145569_8068734_avg55.42893_std11.05658_random_bw5696.50499_skew0.91236_neigh1.63059_cross0.78723_n14.mtx","synthetic_1521198_1521198_18638978_avg12.25283_std37.49579_random_bw35839.82327_skew632.73123_neigh1.3217_cross0.78709_n14.mtx","synthetic_152542_152542_8133417_avg53.3192_std4.76067_random_bw10831.6453_skew3.36991_neigh1.81197_cross0.94119_n14.mtx","synthetic_154699_154699_12503850_avg80.82696_std117.71492_random_bw21128.24826_skew7.85843_neigh1.21484_cross0.84433_n14.mtx","synthetic_154961_154961_8588351_avg55.42266_std11.06218_random_bw6071.63273_skew0.91258_neigh1.62979_cross0.78766_n14.mtx","synthetic_155701_155701_7905533_avg50.77381_std19.58895_random_bw4775.17387_skew1.77702_neigh1.22646_cross0.71616_n14.mtx","synthetic_161070_161070_10085122_avg62.61329_std19.67482_random_bw5009.75586_skew1.44357_neigh1.23526_cross0.7413_n14.mtx","synthetic_161070_161070_10630527_avg65.99942_std19.67949_random_bw5044.97444_skew1.36366_neigh1.23861_cross0.74622_n14.mtx","synthetic_161070_161070_5763734_avg35.78403_std19.07947_random_bw4731.12239_skew2.52112_neigh1.21212_cross0.66116_n14.mtx","synthetic_161070_161070_6293192_avg39.07116_std19.27381_random_bw4796.72539_skew2.30167_neigh1.2143_cross0.67765_n14.mtx","synthetic_161070_161070_6828184_avg42.39265_std19.41502_random_bw4843.00839_skew2.13734_neigh1.21781_cross0.691_n14.mtx","synthetic_161070_161070_7366852_avg45.73696_std19.51164_random_bw4891.21219_skew1.97353_neigh1.22313_cross0.70301_n14.mtx","synthetic_161070_161070_7908198_avg49.0979_std19.57809_random_bw4902.19808_skew1.83108_neigh1.2252_cross0.71126_n14.mtx","synthetic_161070_161070_8177031_avg50.76694_std16.39176_random_bw5026.50779_skew1.48193_neigh1.22306_cross0.73804_n14.mtx","synthetic_161070_161070_8177317_avg50.76872_std15.08847_random_bw5069.23063_skew1.36366_neigh1.22345_cross0.74511_n14.mtx","synthetic_161070_161070_8177462_avg50.76962_std17.68784_random_bw4990.5517_skew1.59998_neigh1.2267_cross0.72883_n14.mtx","synthetic_161070_161070_8177930_avg50.77252_std13.78024_random_bw5097.10939_skew1.24531_neigh1.22217_cross0.75643_n14.mtx","synthetic_161070_161070_8178443_avg50.77571_std18.96773_random_bw4956.1697_skew1.71784_neigh1.22663_cross0.71981_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_random_bw3455.35798_skew1.77657_neigh1.22972_cross0.71662_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_random_bw3785.72739_skew1.77657_neigh1.22674_cross0.71723_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_random_bw4111.1518_skew1.77657_neigh1.22748_cross0.71758_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_random_bw4422.81445_skew1.77657_neigh1.19628_cross0.82221_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_random_bw4441.89847_skew1.77657_neigh1.22792_cross0.71561_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_random_bw4627.28018_skew1.77657_neigh1.21189_cross0.78656_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_random_bw4774.35335_skew1.77657_neigh1.22637_cross0.71614_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_random_bw4843.79053_skew1.77657_neigh1.22333_cross0.73927_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_random_bw4898.56338_skew1.77657_neigh1.10619_cross0.71643_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_random_bw4900.19763_skew1.77657_neigh0.94384_cross0.71595_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_random_bw4901.06241_skew1.77657_neigh0.86553_cross0.71558_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_random_bw4902.46508_skew1.77657_neigh1.02504_cross0.71613_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_random_bw4934.87943_skew1.77657_neigh1.18606_cross0.71564_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_random_bw4941.28083_skew1.77657_neigh1.22782_cross0.71491_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_random_bw4945.13241_skew1.77657_neigh1.26686_cross0.71552_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_random_bw4964.76983_skew1.77657_neigh1.34692_cross0.71497_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_random_bw4977.92274_skew1.77657_neigh1.42811_cross0.71687_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_random_bw5011.34549_skew1.77657_neigh1.50982_cross0.7154_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_random_bw5016.74181_skew1.77657_neigh1.22985_cross0.69227_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_random_bw5049.43708_skew1.77657_neigh1.58911_cross0.7181_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_random_bw5104.02865_skew1.77657_neigh1.22549_cross0.71514_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_random_bw5170.8755_skew1.77657_neigh1.23483_cross0.64645_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_random_bw5295.40467_skew1.77657_neigh1.2381_cross0.60018_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_random_bw5409.67965_skew1.77657_neigh1.24201_cross0.55228_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_random_bw5421.6343_skew1.77657_neigh1.22563_cross0.71654_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_random_bw5516.25936_skew1.77657_neigh1.24385_cross0.50438_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_random_bw5765.30102_skew1.77657_neigh1.22547_cross0.71569_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_random_bw6093.91118_skew1.77657_neigh1.22469_cross0.7147_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_random_bw6414.53412_skew1.77657_neigh1.22329_cross0.7164_n14.mtx","synthetic_161070_161070_8181013_avg50.79166_std20.23119_random_bw4921.85183_skew1.83511_neigh1.22714_cross0.71122_n14.mtx","synthetic_161070_161070_8181297_avg50.79343_std19.67479_random_bw5300.87224_skew2.30751_neigh1.21351_cross0.79956_n14.mtx","synthetic_161070_161070_8184643_avg50.8142_std21.47457_random_bw4878.01938_skew1.95193_neigh1.22869_cross0.70478_n14.mtx","synthetic_161070_161070_8190209_avg50.84876_std22.69412_random_bw4850.04792_skew2.06792_neigh1.22945_cross0.69379_n14.mtx","synthetic_161070_161070_8197860_avg50.89626_std23.8911_random_bw4817.59002_skew2.18295_neigh1.23133_cross0.68541_n14.mtx","synthetic_161070_161070_8207913_avg50.95867_std25.06294_random_bw4778.56517_skew2.29679_neigh1.23219_cross0.67699_n14.mtx","synthetic_161070_161070_8451173_avg52.46895_std19.62109_random_bw4948.49578_skew1.72542_neigh1.22786_cross0.71962_n14.mtx","synthetic_161070_161070_8995114_avg55.84599_std19.64654_random_bw4972.88478_skew1.61433_neigh1.23174_cross0.72925_n14.mtx","synthetic_161070_161070_9539943_avg59.22855_std19.66505_random_bw5000.60859_skew1.53256_neigh1.23372_cross0.73505_n14.mtx","synthetic_1613392_1613392_19770221_avg12.25382_std37.48086_random_bw38048.92289_skew632.67976_neigh1.3224_cross0.78669_n14.mtx","synthetic_164353_164353_9109408_avg55.42587_std11.0688_random_bw6425.46749_skew0.91246_neigh1.62988_cross0.78753_n14.mtx","synthetic_166438_166438_8452047_avg50.78195_std19.60687_random_bw5103.62439_skew1.77658_neigh1.22548_cross0.7151_n14.mtx","synthetic_167070_167070_8908050_avg53.31927_std4.75919_random_bw11844.75739_skew3.3699_neigh1.81234_cross0.94129_n14.mtx","synthetic_167075_167075_13504757_avg80.83051_std117.71597_random_bw22746.915_skew7.87041_neigh1.21275_cross0.84541_n14.mtx","synthetic_1705586_1705586_20897972_avg12.25266_std37.46728_random_bw40228.04846_skew632.73974_neigh1.32222_cross0.78649_n14.mtx","synthetic_173744_173744_9631043_avg55.43238_std11.0756_random_bw6798.57169_skew0.91224_neigh1.62874_cross0.78706_n14.mtx","synthetic_177177_177177_8998963_avg50.79081_std19.6143_random_bw5426.967_skew1.77609_neigh1.22568_cross0.7169_n14.mtx","synthetic_179451_179451_14506771_avg80.83973_std117.71232_random_bw24453.77012_skew7.8694_neigh1.21214_cross0.84477_n14.mtx","synthetic_1797780_1797780_22027651_avg12.2527_std37.45466_random_bw42402.70848_skew632.7381_neigh1.32199_cross0.78712_n14.mtx","synthetic_181598_181598_9682693_avg53.31938_std4.75813_random_bw12869.05691_skew3.36989_neigh1.81177_cross0.94114_n14.mtx","synthetic_183136_183136_10152501_avg55.43695_std11.07295_random_bw7165.08293_skew0.91208_neigh1.63012_cross0.78692_n14.mtx","synthetic_185639_185639_10500260_avg56.56279_std99.5812_random_bw19106.22939_skew7.18559_neigh1.25551_cross0.95442_n14.mtx","synthetic_185639_185639_11501046_avg61.95382_std109.06293_random_bw19420.97121_skew7.19966_neigh1.25396_cross0.95351_n14.mtx","synthetic_185639_185639_12507698_avg67.37646_std112.75287_random_bw24196.24969_skew7.4154_neigh1.18657_cross0.88107_n14.mtx","synthetic_185639_185639_13507262_avg72.76091_std115.00904_random_bw24859.95962_skew7.61726_neigh1.19956_cross0.86027_n14.mtx","synthetic_185639_185639_14507140_avg78.14705_std116.8824_random_bw25219.52172_skew7.80392_neigh1.20963_cross0.84859_n14.mtx","synthetic_185639_185639_15001505_avg80.81009_std108.51788_random_bw25511.99254_skew5.36059_neigh1.20064_cross0.8813_n14.mtx","synthetic_185639_185639_15005658_avg80.83247_std121.76167_random_bw25355.77027_skew10.22074_neigh1.21411_cross0.83223_n14.mtx","synthetic_185639_185639_15005807_avg80.83327_std121.07944_random_bw25384.15564_skew9.70104_neigh1.21342_cross0.83416_n14.mtx","synthetic_185639_185639_15006000_avg80.83431_std120.29592_random_bw25293.65767_skew9.18132_neigh1.21471_cross0.83668_n14.mtx","synthetic_185639_185639_15006279_avg80.83581_std111.39593_random_bw25316.2149_skew5.93999_neigh1.20276_cross0.86868_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_random_bw23672.1534_skew7.8818_neigh1.18863_cross0.87685_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_random_bw23954.47017_skew7.8818_neigh1.19585_cross0.87257_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_random_bw25265.00391_skew7.8818_neigh1.49293_cross0.84614_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_random_bw25293.93346_skew7.8818_neigh1.41168_cross0.84605_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_random_bw25297.50533_skew7.8818_neigh1.33097_cross0.84586_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_random_bw25337.47945_skew7.8818_neigh1.17285_cross0.84478_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_random_bw25357.35315_skew7.8818_neigh1.25202_cross0.84469_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_random_bw25422.38964_skew7.8818_neigh1.09355_cross0.84331_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_random_bw25459.31441_skew7.8818_neigh1.01249_cross0.84366_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_random_bw25490.88255_skew7.8818_neigh0.93357_cross0.84298_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_random_bw25498.90443_skew7.8818_neigh1.57331_cross0.84717_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_random_bw25568.41993_skew7.8818_neigh0.85133_cross0.84267_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_random_bw26253.68657_skew7.8818_neigh1.22152_cross0.81618_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_random_bw27626.6459_skew7.8818_neigh1.23351_cross0.76024_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_random_bw28571.00316_skew7.8818_neigh1.2369_cross0.70583_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_random_bw29320.71745_skew7.8818_neigh1.24185_cross0.64948_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_random_bw29911.00532_skew7.8818_neigh1.24556_cross0.59443_n14.mtx","synthetic_185639_185639_15007163_avg80.84057_std117.05079_random_bw25318.13414_skew7.60954_neigh1.20941_cross0.84686_n14.mtx","synthetic_185639_185639_15007417_avg80.84194_std113.68148_random_bw25329.82537_skew6.50848_neigh1.20564_cross0.85898_n14.mtx","synthetic_185639_185639_15007427_avg80.84199_std115.53272_random_bw25278.26792_skew7.06512_neigh1.20929_cross0.85246_n14.mtx","synthetic_185639_185639_15506741_avg83.5317_std118.48272_random_bw25422.1247_skew7.94271_neigh1.21538_cross0.84172_n14.mtx","synthetic_185639_185639_16506181_avg88.91548_std119.88133_random_bw25572.96292_skew8.06479_neigh1.21966_cross0.83617_n14.mtx","synthetic_185639_185639_17505764_avg94.30003_std121.13382_random_bw25625.10673_skew8.16224_neigh1.22371_cross0.83275_n14.mtx","synthetic_185639_185639_18505682_avg99.68639_std122.27517_random_bw25744.70036_skew8.23897_neigh1.22874_cross0.82863_n14.mtx","synthetic_187667_187667_12936634_avg68.93398_std101.26087_random_bw22637.37919_skew10.08307_neigh1.12453_cross0.83741_n14.mtx","synthetic_187915_187915_9545196_avg50.79529_std19.60651_random_bw5758.22991_skew1.77585_neigh1.22523_cross0.71562_n14.mtx","synthetic_191826_191826_15508397_avg80.84617_std117.70962_random_bw26139.99935_skew7.88106_neigh1.21134_cross0.84514_n14.mtx","synthetic_196126_196126_10457351_avg53.31955_std4.75831_random_bw14006.14808_skew3.36988_neigh1.81257_cross0.94078_n14.mtx","synthetic_198653_198653_10092676_avg50.80556_std19.60715_random_bw6093.46107_skew1.77529_neigh1.22471_cross0.71497_n14.mtx","synthetic_204202_204202_16508082_avg80.84192_std117.71063_random_bw27859.72659_skew7.88153_neigh1.21321_cross0.84455_n14.mtx","synthetic_205540_205540_14169500_avg68.93792_std101.25978_random_bw24815.66416_skew10.08243_neigh1.12421_cross0.83711_n14.mtx","synthetic_209391_209391_10637567_avg50.80241_std19.60148_random_bw6420.80853_skew1.77546_neigh1.22408_cross0.716_n14.mtx","synthetic_210654_210654_11231930_avg53.31933_std4.75723_random_bw14914.26662_skew3.3699_neigh1.81313_cross0.94073_n14.mtx","synthetic_216578_216578_17508080_avg80.83961_std117.71328_random_bw29528.4278_skew7.89416_neigh1.2132_cross0.8447_n14.mtx","synthetic_217918_217918_10485592_avg48.11714_std4.75335_random_bw15844.7796_skew3.36435_neigh1.77209_cross0.93961_n14.mtx","synthetic_217918_217918_11231492_avg51.53999_std4.76893_random_bw15995.38461_skew3.36554_neigh1.75717_cross0.93963_n14.mtx","synthetic_217918_217918_11582841_avg53.15229_std3.34176_random_bw14914.85908_skew3.38363_neigh1.83128_cross0.94011_n14.mtx","synthetic_217918_217918_11590559_avg53.18771_std3.65776_random_bw14955.00461_skew3.38071_neigh1.82908_cross0.93983_n14.mtx","synthetic_217918_217918_11598747_avg53.22528_std3.97324_random_bw15044.77643_skew3.37762_neigh1.82449_cross0.9406_n14.mtx","synthetic_217918_217918_11607157_avg53.26387_std4.28638_random_bw15200.0146_skew3.37445_neigh1.82082_cross0.9401_n14.mtx","synthetic_217918_217918_11608217_avg53.26874_std4.76281_random_bw15269.0692_skew4.08741_neigh1.81967_cross0.94009_n14.mtx","synthetic_217918_217918_11610489_avg53.27916_std4.76243_random_bw15240.4697_skew3.93626_neigh1.81896_cross0.94023_n14.mtx","synthetic_217918_217918_11612905_avg53.29025_std4.75996_random_bw15310.45462_skew3.76635_neigh1.81736_cross0.94035_n14.mtx","synthetic_217918_217918_11615389_avg53.30165_std4.60014_random_bw15350.22672_skew3.37135_neigh1.81586_cross0.94062_n14.mtx","synthetic_217918_217918_11615400_avg53.3017_std4.75902_random_bw15342.03914_skew3.61524_neigh1.81594_cross0.94062_n14.mtx","synthetic_217918_217918_11617960_avg53.31345_std4.75796_random_bw15405.69215_skew3.44541_neigh1.81369_cross0.94068_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_random_bw10737.64739_skew3.36989_neigh1.81318_cross0.94085_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_random_bw11891.67531_skew3.36989_neigh1.81288_cross0.94086_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_random_bw12724.60274_skew3.36989_neigh1.50678_cross0.94071_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_random_bw12730.20473_skew3.36989_neigh1.26613_cross0.9391_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_random_bw12738.48211_skew3.36989_neigh1.38666_cross0.94034_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_random_bw12837.98779_skew3.36989_neigh1.81264_cross0.9409_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_random_bw13058.23403_skew3.36989_neigh1.62674_cross0.94024_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_random_bw13376.58862_skew3.36989_neigh1.74964_cross0.94006_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_random_bw13808.95321_skew3.36989_neigh1.81302_cross0.94102_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_random_bw14783.21932_skew3.36989_neigh1.83882_cross0.66043_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_random_bw14834.61294_skew3.36989_neigh1.83738_cross0.72231_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_random_bw14844.13273_skew3.36989_neigh1.83615_cross0.78285_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_random_bw14910.1263_skew3.36989_neigh1.81269_cross0.94076_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_random_bw15016.8092_skew3.36989_neigh1.83172_cross0.84569_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_random_bw15289.18189_skew3.36989_neigh1.8215_cross0.90858_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_random_bw15805.89794_skew3.36989_neigh1.78683_cross0.97162_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_random_bw15930.0478_skew3.36989_neigh1.81219_cross0.94078_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_random_bw16309.5886_skew3.36989_neigh1.7473_cross0.98961_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_random_bw16886.23663_skew3.36989_neigh1.87576_cross0.93935_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_random_bw16921.52031_skew3.36989_neigh1.81297_cross0.94073_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_random_bw17934.76325_skew3.36989_neigh1.81325_cross0.94068_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_random_bw19000.93916_skew3.36989_neigh1.81255_cross0.94072_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_random_bw19930.43914_skew3.36989_neigh1.81307_cross0.94069_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_random_bw5768.22535_skew3.36989_neigh1.93911_cross0.93966_n14.mtx","synthetic_217918_217918_11620586_avg53.3255_std4.75691_random_bw15468.18448_skew3.29438_neigh1.81163_cross0.94092_n14.mtx","synthetic_217918_217918_11622834_avg53.33581_std4.91358_random_bw15511.53295_skew3.36855_neigh1.80929_cross0.94061_n14.mtx","synthetic_217918_217918_11623241_avg53.33768_std4.75547_random_bw15548.19386_skew3.12466_neigh1.8088_cross0.94062_n14.mtx","synthetic_217918_217918_11625780_avg53.34933_std4.75397_random_bw15624.44027_skew2.97381_neigh1.80583_cross0.94018_n14.mtx","synthetic_217918_217918_11628184_avg53.36036_std4.75308_random_bw15742.20805_skew2.80432_neigh1.80167_cross0.93967_n14.mtx","synthetic_217918_217918_11628620_avg53.36237_std5.22805_random_bw15759.47133_skew3.36637_neigh1.80061_cross0.93977_n14.mtx","synthetic_217918_217918_11630299_avg53.37007_std4.7531_random_bw15847.73759_skew2.65373_neigh1.79719_cross0.93963_n14.mtx","synthetic_217918_217918_11632309_avg53.37929_std5.54522_random_bw16023.97314_skew3.36499_neigh1.78977_cross0.9395_n14.mtx","synthetic_217918_217918_11634099_avg53.38751_std5.86172_random_bw16271.97895_skew3.36432_neigh1.77745_cross0.93875_n14.mtx","synthetic_217918_217918_11634741_avg53.39045_std6.17863_random_bw16478.31237_skew3.36408_neigh1.76344_cross0.93829_n14.mtx","synthetic_217918_217918_12031582_avg55.21151_std4.74931_random_bw15618.56314_skew3.36503_neigh1.82311_cross0.94108_n14.mtx","synthetic_217918_217918_12797820_avg58.72769_std4.7736_random_bw14173.92179_skew3.3591_neigh1.75513_cross0.93936_n14.mtx","synthetic_217918_217918_13561979_avg62.23432_std4.75526_random_bw13352.00925_skew3.37058_neigh1.8388_cross0.94017_n14.mtx","synthetic_217918_217918_14374351_avg65.9622_std4.76726_random_bw14674.102_skew3.36614_neigh1.78599_cross0.9393_n14.mtx","synthetic_217918_217918_15093236_avg69.26108_std4.76589_random_bw14165.84201_skew3.37475_neigh1.82997_cross0.94033_n14.mtx","synthetic_217918_217918_8143542_avg37.36975_std4.75333_random_bw21449.04695_skew3.36182_neigh1.7462_cross0.93898_n14.mtx","synthetic_217918_217918_8924884_avg40.95524_std4.76049_random_bw22967.50218_skew3.34621_neigh1.71344_cross0.93831_n14.mtx","synthetic_217918_217918_9684687_avg44.44189_std4.75752_random_bw14709.39543_skew3.36525_neigh1.77216_cross0.93925_n14.mtx","synthetic_223413_223413_15400297_avg68.93196_std101.25897_random_bw27005.84088_skew10.0979_neigh1.12422_cross0.83714_n14.mtx","synthetic_225181_225181_12006505_avg53.31935_std4.75652_random_bw15784.15411_skew3.3699_neigh1.8126_cross0.94063_n14.mtx","synthetic_228954_228954_18508358_avg80.83876_std117.71434_random_bw31230.9613_skew7.89425_neigh1.21214_cross0.84502_n14.mtx","synthetic_239709_239709_12781051_avg53.31903_std4.75618_random_bw17005.93316_skew3.36992_neigh1.81258_cross0.9406_n14.mtx","synthetic_241286_241286_16633118_avg68.93528_std101.25723_random_bw29154.84234_skew10.09737_neigh1.12198_cross0.83731_n14.mtx","synthetic_241330_241330_19509069_avg80.8398_std117.71435_random_bw32920.56295_skew7.89413_neigh1.21032_cross0.845_n14.mtx","synthetic_254237_254237_13555788_avg53.31949_std4.75562_random_bw17988.99306_skew3.36988_neigh1.81207_cross0.9405_n14.mtx","synthetic_25491_25491_3028356_avg118.80099_std31.7722_random_bw2627.38543_skew1.00335_neigh1.79456_cross0.81089_n14.mtx","synthetic_259159_259159_17866974_avg68.94213_std101.25583_random_bw31249.92925_skew10.11077_neigh1.12114_cross0.83763_n14.mtx","synthetic_26684_26684_10204693_avg382.42741_std245.44826_random_bw12208.0425_skew1.57042_neigh1.97487_cross0.9207_n14.mtx","synthetic_268096_268096_12941160_avg48.27062_std95.20699_random_bw27594.27638_skew9.2754_neigh1.08407_cross0.8943_n14.mtx","synthetic_268096_268096_14172610_avg52.86394_std97.03631_random_bw30537.79348_skew9.5554_neigh1.09138_cross0.86621_n14.mtx","synthetic_268096_268096_15403716_avg57.45597_std98.50696_random_bw31584.47846_skew9.75606_neigh1.10235_cross0.85247_n14.mtx","synthetic_268096_268096_16634948_avg62.04848_std99.72897_random_bw31984.02507_skew9.92694_neigh1.11286_cross0.84502_n14.mtx","synthetic_268096_268096_17866195_avg66.64104_std100.78017_random_bw32220.36555_skew10.05925_neigh1.12121_cross0.83959_n14.mtx","synthetic_268096_268096_18481037_avg68.9344_std103.42196_random_bw32441.44161_skew12.99882_neigh1.12449_cross0.83_n14.mtx","synthetic_268096_268096_18481246_avg68.93518_std102.1488_random_bw32365.8807_skew11.06931_neigh1.12496_cross0.83401_n14.mtx","synthetic_268096_268096_18481318_avg68.93545_std102.6344_random_bw32417.76157_skew11.72205_neigh1.12377_cross0.83241_n14.mtx","synthetic_268096_268096_18481428_avg68.93586_std103.0557_random_bw32473.21371_skew12.36025_neigh1.12371_cross0.83096_n14.mtx","synthetic_268096_268096_18481631_avg68.93662_std101.58036_random_bw32372.11142_skew10.43079_neigh1.12404_cross0.8362_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_random_bw30754.23082_skew10.11157_neigh1.10438_cross0.86329_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_random_bw31287.63614_skew10.11157_neigh1.12125_cross0.83758_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_random_bw32306.79739_skew10.11157_neigh1.15983_cross0.83743_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_random_bw32321.14848_skew10.11157_neigh1.23407_cross0.83798_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_random_bw32329.4436_skew10.11157_neigh1.01031_cross0.83738_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_random_bw32349.06004_skew10.11157_neigh1.38185_cross0.83852_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_random_bw32367.35543_skew10.11157_neigh1.08557_cross0.83678_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_random_bw32375.57571_skew10.11157_neigh0.93684_cross0.8365_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_random_bw32420.97615_skew10.11157_neigh1.30741_cross0.8385_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_random_bw32430.76354_skew10.11157_neigh0.86471_cross0.83616_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_random_bw32452.85556_skew10.11157_neigh0.79_cross0.83549_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_random_bw32557.34156_skew10.11157_neigh1.45611_cross0.83956_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_random_bw33506.09611_skew10.11157_neigh1.12211_cross0.83722_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_random_bw33650.79702_skew10.11157_neigh1.13371_cross0.80829_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_random_bw35478.95576_skew10.11157_neigh1.14236_cross0.75351_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_random_bw35561.27802_skew10.11157_neigh1.12238_cross0.83753_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_random_bw36758.75994_skew10.11157_neigh1.14871_cross0.69834_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_random_bw37755.70411_skew10.11157_neigh1.12209_cross0.83735_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_random_bw37857.24747_skew10.11157_neigh1.15306_cross0.64222_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_random_bw38714.99432_skew10.11157_neigh1.15592_cross0.5881_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_random_bw39930.72019_skew10.11157_neigh1.12081_cross0.83752_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_random_bw42070.66722_skew10.11157_neigh1.12184_cross0.83718_n14.mtx","synthetic_268096_268096_18482060_avg68.93822_std100.90606_random_bw32385.89976_skew9.79227_neigh1.12123_cross0.83833_n14.mtx","synthetic_268096_268096_18482363_avg68.93935_std100.10125_random_bw32276.82294_skew9.13935_neigh1.12203_cross0.84217_n14.mtx","synthetic_268096_268096_18482725_avg68.9407_std99.12285_random_bw32363.10438_skew8.47191_neigh1.12037_cross0.84459_n14.mtx","synthetic_268096_268096_18483147_avg68.94227_std97.91705_random_bw32207.13852_skew7.80447_neigh1.12058_cross0.85009_n14.mtx","synthetic_268096_268096_18483826_avg68.9448_std96.40124_random_bw32262.35735_skew7.10794_neigh1.11413_cross0.85517_n14.mtx","synthetic_268765_268765_14330411_avg53.31948_std4.75541_random_bw19027.02115_skew3.36988_neigh1.81207_cross0.94046_n14.mtx","synthetic_277032_277032_19096094_avg68.931_std101.25944_random_bw33505.32851_skew10.12707_neigh1.12297_cross0.83731_n14.mtx","synthetic_27919_27919_3319237_avg118.8881_std31.78107_random_bw2882.12425_skew1.00188_neigh1.79322_cross0.80796_n14.mtx","synthetic_283293_283293_15104954_avg53.31919_std4.75508_random_bw20060.27324_skew3.36991_neigh1.81213_cross0.94052_n14.mtx","synthetic_29225_29225_11176419_avg382.42666_std245.44724_random_bw13101.62433_skew1.57043_neigh1.97449_cross0.92145_n14.mtx","synthetic_294905_294905_20328825_avg68.93347_std101.25836_random_bw35580.71368_skew10.12667_neigh1.12383_cross0.83765_n14.mtx","synthetic_29496_29496_3296630_avg111.76532_std88.66285_random_bw13978.43416_skew3.35734_neigh1.89349_cross0.6274_n14.mtx","synthetic_2998_2998_6123474_avg2042.51968_std621.91091_random_bw2814.28252_skew0.46779_neigh1.82839_cross0.81924_n14.mtx","synthetic_30347_30347_3608195_avg118.89791_std31.76143_random_bw3114.986_skew1.00172_neigh1.79361_cross0.81079_n14.mtx","synthetic_312778_312778_21563736_avg68.94262_std101.25558_random_bw37752.7149_skew10.1397_neigh1.12125_cross0.83735_n14.mtx","synthetic_31766_31766_12148131_avg382.42558_std245.44713_random_bw14374.34849_skew1.57043_neigh1.97472_cross0.91791_n14.mtx","synthetic_32305_32305_3613009_avg111.84055_std88.75337_random_bw15339.91778_skew3.35441_neigh1.8933_cross0.62712_n14.mtx","synthetic_32775_32775_3898242_avg118.9395_std31.78937_random_bw3355.91933_skew1.00102_neigh1.79257_cross0.81174_n14.mtx","synthetic_3283001_3283001_13139084_avg4.00216_std1.21615_random_bw3180.83136_skew312.8308_neigh0.41641_cross0.6482_n14.mtx","synthetic_3284_3284_6742726_avg2053.20524_std666.12092_random_bw3090.98173_skew0.59945_neigh1.79209_cross0.78602_n14.mtx","synthetic_330651_330651_22801392_avg68.95909_std101.25183_random_bw39914.18466_skew10.13704_neigh1.12248_cross0.83693_n14.mtx","synthetic_34308_34308_13120227_avg382.42471_std245.44609_random_bw15373.72135_skew1.57044_neigh1.97444_cross0.91794_n14.mtx","synthetic_348524_348524_24034194_avg68.95994_std101.25128_random_bw42061.27886_skew10.1369_neigh1.12113_cross0.83729_n14.mtx","synthetic_35115_35115_3931279_avg111.95441_std88.67056_random_bw16758.071_skew3.34999_neigh1.89338_cross0.62737_n14.mtx","synthetic_35203_35203_4188100_avg118.96997_std31.72558_random_bw3622.57756_skew1.0005_neigh1.79296_cross0.80889_n14.mtx","synthetic_3569_3569_7446679_avg2086.48893_std694.14116_random_bw3307.12945_skew0.71053_neigh1.77466_cross0.76435_n14.mtx","synthetic_3595668_3595668_14390363_avg4.00214_std1.20637_random_bw3483.23396_skew312.83218_neigh0.41589_cross0.64799_n14.mtx","synthetic_36417_36417_3031503_avg83.24417_std31.62901_random_bw3778.50658_skew1.4266_neigh1.77891_cross0.76976_n14.mtx","synthetic_36417_36417_3320284_avg91.17401_std31.69422_random_bw3853.70876_skew1.30329_neigh1.78298_cross0.77984_n14.mtx","synthetic_36417_36417_3609556_avg99.11734_std31.72484_random_bw3797.36502_skew1.19941_neigh1.78594_cross0.79053_n14.mtx","synthetic_36417_36417_3899087_avg107.06777_std31.73599_random_bw3752.97905_skew1.11081_neigh1.78782_cross0.80067_n14.mtx","synthetic_36417_36417_4188724_avg115.02112_std31.74031_random_bw3797.6023_skew1.03441_neigh1.79206_cross0.80583_n14.mtx","synthetic_36417_36417_4330804_avg118.92259_std41.20033_random_bw3675.94291_skew1.30402_neigh1.79795_cross0.77953_n14.mtx","synthetic_36417_36417_4331206_avg118.93363_std39.11615_random_bw3697.36082_skew1.23654_neigh1.79608_cross0.7852_n14.mtx","synthetic_36417_36417_4331707_avg118.94739_std37.01905_random_bw3704.02521_skew1.16903_neigh1.79589_cross0.79365_n14.mtx","synthetic_36417_36417_4332360_avg118.96532_std34.9075_random_bw3710.06437_skew1.10145_neigh1.79431_cross0.8013_n14.mtx","synthetic_36417_36417_4333163_avg118.98737_std32.79928_random_bw3723.49809_skew1.03383_neigh1.79539_cross0.80667_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_random_bw2623.52629_skew1.00003_neigh1.79451_cross0.81215_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_random_bw2877.98784_skew1.00003_neigh1.79304_cross0.80927_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_random_bw3116.1024_skew1.00003_neigh1.79446_cross0.81024_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_random_bw3362.22435_skew1.00003_neigh1.79146_cross0.81185_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_random_bw3467.27641_skew1.00003_neigh1.77473_cross0.86839_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_random_bw3597.87423_skew1.00003_neigh1.61979_cross0.81098_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_random_bw3598.07988_skew1.00003_neigh1.50455_cross0.81028_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_random_bw3599.29363_skew1.00003_neigh1.27022_cross0.8107_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_random_bw3601.93981_skew1.00003_neigh1.38778_cross0.81049_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_random_bw3621.76799_skew1.00003_neigh1.79294_cross0.80868_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_random_bw3628.53802_skew1.00003_neigh1.78888_cross0.83631_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_random_bw3688.34179_skew1.00003_neigh1.73475_cross0.80937_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_random_bw3736.64409_skew1.00003_neigh1.79285_cross0.80866_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_random_bw3824.14642_skew1.00003_neigh1.79702_cross0.78418_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_random_bw3846.82772_skew1.00003_neigh1.79307_cross0.81062_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_random_bw3974.32482_skew1.00003_neigh1.80409_cross0.73066_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_random_bw4041.25675_skew1.00003_neigh1.85155_cross0.80883_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_random_bw4082.51341_skew1.00003_neigh1.80817_cross0.68078_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_random_bw4114.70692_skew1.00003_neigh1.79169_cross0.81094_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_random_bw4164.96568_skew1.00003_neigh1.81125_cross0.63039_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_random_bw4243.70544_skew1.00003_neigh1.81309_cross0.57648_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_random_bw4381.77057_skew1.00003_neigh1.79261_cross0.80821_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_random_bw4600.41277_skew1.00003_neigh1.79347_cross0.80785_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_random_bw4827.23959_skew1.00003_neigh1.79176_cross0.81018_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_random_bw5452.08949_skew1.00003_neigh1.94835_cross0.80771_n14.mtx","synthetic_36417_36417_4333866_avg119.00667_std30.68416_random_bw3732.46783_skew0.96628_neigh1.79286_cross0.81383_n14.mtx","synthetic_36417_36417_4334685_avg119.02916_std28.56617_random_bw3758.80797_skew0.89869_neigh1.79181_cross0.82202_n14.mtx","synthetic_36417_36417_4335436_avg119.04978_std26.44817_random_bw3785.04915_skew0.83117_neigh1.78962_cross0.82614_n14.mtx","synthetic_36417_36417_4336151_avg119.06942_std24.33643_random_bw3809.09534_skew0.76368_neigh1.78823_cross0.83433_n14.mtx","synthetic_36417_36417_4336875_avg119.0893_std22.21912_random_bw3824.95716_skew0.7046_neigh1.78872_cross0.84268_n14.mtx","synthetic_36417_36417_4478356_avg122.97433_std31.74172_random_bw3753.51149_skew0.96789_neigh1.79439_cross0.81446_n14.mtx","synthetic_36417_36417_4767999_avg130.92784_std31.7426_random_bw3736.97166_skew0.90945_neigh1.79577_cross0.81963_n14.mtx","synthetic_36417_36417_5057685_avg138.88253_std31.74387_random_bw3770.7999_skew0.85769_neigh1.79863_cross0.82586_n14.mtx","synthetic_36417_36417_5347290_avg146.83499_std31.74228_random_bw3749.94288_skew0.81156_neigh1.8002_cross0.83033_n14.mtx","synthetic_36417_36417_5636923_avg154.78823_std31.74188_random_bw3744.12662_skew0.77016_neigh1.80391_cross0.8341_n14.mtx","synthetic_36849_36849_14091918_avg382.42335_std245.44508_random_bw16368.21697_skew1.57045_neigh1.97457_cross0.91965_n14.mtx","synthetic_37630_37630_4477645_avg118.99136_std31.72824_random_bw3844.70563_skew1.00015_neigh1.793_cross0.81096_n14.mtx","synthetic_37924_37924_4245700_avg111.95285_std88.65794_random_bw18034.65215_skew3.35005_neigh1.89352_cross0.62758_n14.mtx","synthetic_38120_38120_10204584_avg267.69633_std171.81206_random_bw17167.82778_skew1.57008_neigh1.97267_cross0.91545_n14.mtx","synthetic_38120_38120_11176434_avg293.19082_std188.17435_random_bw18077.60719_skew1.5717_neigh1.97255_cross0.91793_n14.mtx","synthetic_38120_38120_12148318_avg318.6862_std204.53731_random_bw19160.42267_skew1.56993_neigh1.97325_cross0.91894_n14.mtx","synthetic_38120_38120_13120182_avg344.18106_std220.90077_random_bw19517.48116_skew1.57132_neigh1.97392_cross0.91784_n14.mtx","synthetic_38120_38120_14092040_avg369.67576_std237.2634_random_bw16796.84077_skew1.56982_neigh1.9745_cross0.91796_n14.mtx","synthetic_38120_38120_14186695_avg372.15884_std217.74813_random_bw17520.62065_skew1.39145_neigh1.97428_cross0.91992_n14.mtx","synthetic_38120_38120_14353565_avg376.53633_std228.90355_random_bw17193.09321_skew1.46191_neigh1.97454_cross0.91856_n14.mtx","synthetic_38120_38120_14506403_avg380.54572_std239.95763_random_bw16970.09357_skew1.5332_neigh1.97424_cross0.92361_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_random_bw16928.18211_skew1.57045_neigh1.97488_cross0.70725_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_random_bw16978.22802_skew1.57045_neigh1.97475_cross0.6468_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_random_bw16980.42762_skew1.57045_neigh1.97471_cross0.88897_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_random_bw17047.29614_skew1.57045_neigh1.9748_cross0.76884_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_random_bw17164.09806_skew1.57045_neigh1.9747_cross0.82515_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_random_bw17288.93704_skew1.57045_neigh1.97422_cross0.94898_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_random_bw17337.86668_skew1.57045_neigh1.9736_cross0.99006_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_random_bw17436.03269_skew1.57045_neigh1.90981_cross0.91796_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_random_bw17997.22335_skew1.57045_neigh1.77798_cross0.91958_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_random_bw18145.65771_skew1.57045_neigh1.65034_cross0.92138_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_random_bw18235.12718_skew1.57045_neigh1.51713_cross0.921_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_random_bw18240.43122_skew1.57045_neigh1.3892_cross0.9198_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_random_bw5058.0712_skew1.57045_neigh1.99412_cross0.92723_n14.mtx","synthetic_38120_38120_14646455_avg384.2197_std250.9036_random_bw17379.7798_skew1.60788_neigh1.97452_cross0.92139_n14.mtx","synthetic_38120_38120_14774683_avg387.5835_std261.73289_random_bw17213.87985_skew1.68071_neigh1.97461_cross0.91994_n14.mtx","synthetic_38120_38120_14892124_avg390.66432_std272.44037_random_bw17256.11215_skew1.75428_neigh1.97475_cross0.9194_n14.mtx","synthetic_38120_38120_15029193_avg394.26005_std283.24138_random_bw20307.43822_skew1.83569_neigh1.96002_cross0.918_n14.mtx","synthetic_38120_38120_15063906_avg395.17067_std253.62681_random_bw17391.16758_skew1.57104_neigh1.97506_cross0.9205_n14.mtx","synthetic_38120_38120_15246247_avg399.95401_std294.48908_random_bw20475.55488_skew1.94534_neigh1.95593_cross0.90935_n14.mtx","synthetic_38120_38120_16417883_avg430.68948_std273.30431_random_bw20513.51482_skew1.63763_neigh1.95832_cross0.8988_n14.mtx","synthetic_38120_38120_17911818_avg469.8798_std294.64729_random_bw17768.78305_skew1.77305_neigh1.96177_cross0.87832_n14.mtx","synthetic_3855_3855_8084357_avg2097.10947_std723.99546_random_bw3581.91751_skew0.83824_neigh1.7534_cross0.73513_n14.mtx","synthetic_3908334_3908334_15641636_avg4.00212_std1.19836_random_bw3788.22744_skew312.83338_neigh0.41514_cross0.64836_n14.mtx","synthetic_39390_39390_15063634_avg382.4228_std245.44522_random_bw17726.0756_skew1.57045_neigh1.97466_cross0.91891_n14.mtx","synthetic_40058_40058_4768887_avg119.04955_std31.78281_random_bw4114.1045_skew0.99917_neigh1.79221_cross0.81063_n14.mtx","synthetic_40733_40733_4573744_avg112.28596_std88.81893_random_bw19403.9372_skew3.33714_neigh1.89379_cross0.62734_n14.mtx","synthetic_4141_4141_8723864_avg2106.70466_std753.89371_random_bw3856.85245_skew0.96563_neigh1.73618_cross0.70892_n14.mtx","synthetic_41932_41932_16035737_avg382.42242_std245.44481_random_bw18664.87153_skew1.57046_neigh1.9748_cross0.9208_n14.mtx","synthetic_42138_42138_3666754_avg87.01775_std81.65352_random_bw19853.31741_skew4.24031_neigh1.89219_cross0.57666_n14.mtx","synthetic_42138_42138_3893764_avg92.40505_std83.38574_random_bw17974.59227_skew4.01055_neigh1.89331_cross0.57355_n14.mtx","synthetic_42138_42138_3928503_avg93.22946_std52.30654_random_bw27312.13121_skew1.38122_neigh1.71236_cross0.97097_n14.mtx","synthetic_42138_42138_4006220_avg95.07381_std54.75201_random_bw26439.50325_skew1.47176_neigh1.72394_cross0.96381_n14.mtx","synthetic_42138_42138_4072421_avg96.64486_std57.11798_random_bw25980.40367_skew1.5661_neigh1.74415_cross0.95788_n14.mtx","synthetic_42138_42138_4126809_avg97.93557_std85.02631_random_bw18557.95631_skew3.79907_neigh1.8936_cross0.5899_n14.mtx","synthetic_42138_42138_4128960_avg97.98662_std59.40057_random_bw25487.21109_skew1.65342_neigh1.7633_cross0.95074_n14.mtx","synthetic_42138_42138_4176699_avg99.11954_std61.5954_random_bw24959.75837_skew1.74416_neigh1.77059_cross0.94519_n14.mtx","synthetic_42138_42138_4365370_avg103.59699_std86.57531_random_bw19147.75084_skew3.60438_neigh1.89374_cross0.60541_n14.mtx","synthetic_42138_42138_4491637_avg106.5935_std66.96099_random_bw21515.96234_skew2.48989_neigh1.88186_cross0.70673_n14.mtx","synthetic_42138_42138_4533678_avg107.5912_std72.07832_random_bw21100.98446_skew2.69919_neigh1.88564_cross0.68688_n14.mtx","synthetic_42138_42138_4582731_avg108.7553_std77.01103_random_bw20786.03004_skew2.88947_neigh1.88766_cross0.66816_n14.mtx","synthetic_42138_42138_4608940_avg109.37728_std88.04095_random_bw19683.90524_skew3.42505_neigh1.89317_cross0.62053_n14.mtx","synthetic_42138_42138_4638390_avg110.07618_std81.79532_random_bw20354.8565_skew3.07899_neigh1.89046_cross0.65066_n14.mtx","synthetic_42138_42138_4699816_avg111.53391_std86.4582_random_bw20073.78231_skew3.24983_neigh1.89286_cross0.635_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_random_bw14002.42427_skew3.33605_neigh1.89425_cross0.62763_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_random_bw14320.37603_skew3.33605_neigh1.97024_cross0.65977_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_random_bw15315.12146_skew3.33605_neigh1.89405_cross0.62765_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_random_bw15931.56866_skew3.33605_neigh1.32686_cross0.62684_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_random_bw15959.80735_skew3.33605_neigh1.45483_cross0.62671_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_random_bw16095.01927_skew3.33605_neigh1.57976_cross0.62768_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_random_bw16296.61787_skew3.33605_neigh1.7053_cross0.62795_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_random_bw16555.88737_skew3.33605_neigh1.89387_cross0.6282_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_random_bw16953.95005_skew3.33605_neigh1.83072_cross0.62763_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_random_bw17957.18508_skew3.33605_neigh1.89403_cross0.62819_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_random_bw18224.68475_skew3.33605_neigh1.95842_cross0.66307_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_random_bw19284.41321_skew3.33605_neigh1.89382_cross0.62757_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_random_bw19954.55513_skew3.33605_neigh1.89279_cross0.63258_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_random_bw19982.02784_skew3.33605_neigh1.89327_cross0.62802_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_random_bw20091.36831_skew3.33605_neigh1.89497_cross0.60786_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_random_bw20234.19666_skew3.33605_neigh1.89725_cross0.563_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_random_bw20448.61299_skew3.33605_neigh1.89904_cross0.52249_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_random_bw20616.53341_skew3.33605_neigh1.90059_cross0.4835_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_random_bw20714.32773_skew3.33605_neigh1.90175_cross0.44229_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_random_bw20750.59585_skew3.33605_neigh1.8937_cross0.62968_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_random_bw21583.20827_skew3.33605_neigh1.89368_cross0.63603_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_random_bw22018.06151_skew3.33605_neigh1.89495_cross0.63936_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_random_bw22492.64431_skew3.33605_neigh1.89475_cross0.64482_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_random_bw22731.73506_skew3.33605_neigh1.89483_cross0.6471_n14.mtx","synthetic_42138_42138_4766804_avg113.12364_std90.99737_random_bw19857.30628_skew3.41994_neigh1.89446_cross0.62081_n14.mtx","synthetic_42138_42138_4838257_avg114.81933_std95.4594_random_bw19620.37349_skew3.5724_neigh1.89654_cross0.6073_n14.mtx","synthetic_42138_42138_4857446_avg115.27472_std89.41484_random_bw20263.25834_skew3.25939_neigh1.89365_cross0.63484_n14.mtx","synthetic_42138_42138_4913422_avg116.60311_std99.85384_random_bw19444.31781_skew3.72543_neigh1.89784_cross0.59642_n14.mtx","synthetic_42138_42138_4991872_avg118.46485_std104.19584_random_bw17244.71835_skew3.8622_neigh1.89896_cross0.58472_n14.mtx","synthetic_42138_42138_5073389_avg120.39938_std108.48726_random_bw17138.19232_skew4.00003_neigh1.90049_cross0.57502_n14.mtx","synthetic_42138_42138_5111019_avg121.2924_std90.68096_random_bw18636.57063_skew3.10578_neigh1.89429_cross0.64795_n14.mtx","synthetic_42138_42138_5368820_avg127.41041_std91.85872_random_bw19117.17587_skew2.96357_neigh1.89434_cross0.66066_n14.mtx","synthetic_42138_42138_5630518_avg133.62091_std92.94843_random_bw19407.61094_skew2.83174_neigh1.89425_cross0.67317_n14.mtx","synthetic_42138_42138_5895869_avg139.9181_std93.95081_random_bw19763.9757_skew2.70931_neigh1.8945_cross0.68576_n14.mtx","synthetic_4221001_4221001_16892920_avg4.00211_std1.19165_random_bw4086.05171_skew312.83427_neigh0.41605_cross0.64812_n14.mtx","synthetic_42486_42486_5059907_avg119.09587_std31.75092_random_bw4380.455_skew1.14113_neigh1.79204_cross0.80881_n14.mtx","synthetic_4284_4284_10161733_avg2372.01984_std415.11429_random_bw4053.5726_skew0.80606_neigh1.72262_cross0.77626_n14.mtx","synthetic_4284_4284_10296320_avg2403.43604_std664.88425_random_bw4000.62278_skew0.78245_neigh1.74482_cross0.76139_n14.mtx","synthetic_4284_4284_11128593_avg2597.71078_std612.25995_random_bw4020.83217_skew0.64914_neigh1.75863_cross0.79849_n14.mtx","synthetic_4284_4284_11955257_avg2790.67624_std565.77456_random_bw4036.47432_skew0.53511_neigh1.7748_cross0.83525_n14.mtx","synthetic_4284_4284_12776299_avg2982.32937_std524.4261_random_bw4061.7514_skew0.43646_neigh1.79331_cross0.868_n14.mtx","synthetic_4284_4284_5176835_avg1208.41153_std875.52905_random_bw3946.70121_skew2.54515_neigh1.71142_cross0.50146_n14.mtx","synthetic_4284_4284_6045664_avg1411.21942_std813.05724_random_bw3941.44468_skew2.03567_neigh1.70361_cross0.54989_n14.mtx","synthetic_4284_4284_6908449_avg1612.61648_std777.77715_random_bw3940.60668_skew1.65655_neigh1.70614_cross0.59717_n14.mtx","synthetic_4284_4284_7606767_avg1775.62255_std826.63023_random_bw3957.25163_skew1.41267_neigh1.72072_cross0.62851_n14.mtx","synthetic_4284_4284_7764407_avg1812.41993_std766.39496_random_bw3956.24113_skew1.36369_neigh1.71331_cross0.63304_n14.mtx","synthetic_4284_4284_7948514_avg1855.39542_std808.46009_random_bw3960.53291_skew1.30894_neigh1.72141_cross0.64445_n14.mtx","synthetic_4284_4284_8278029_avg1932.31303_std796.54977_random_bw3961.24043_skew1.21703_neigh1.72325_cross0.6619_n14.mtx","synthetic_4284_4284_8399652_avg1960.70308_std840.91619_random_bw3956.79902_skew1.18493_neigh1.7318_cross0.66546_n14.mtx","synthetic_4284_4284_8563923_avg1999.04832_std817.99013_random_bw3958.27801_skew1.14302_neigh1.73026_cross0.67074_n14.mtx","synthetic_4284_4284_8591948_avg2005.5901_std788.67453_random_bw3972.91223_skew1.13603_neigh1.72562_cross0.67423_n14.mtx","synthetic_4284_4284_8613657_avg2010.65756_std774.7613_random_bw3958.64706_skew1.13065_neigh1.72447_cross0.67632_n14.mtx","synthetic_4284_4284_8716487_avg2034.66083_std804.2751_random_bw3962.64192_skew1.10551_neigh1.73048_cross0.68001_n14.mtx","synthetic_4284_4284_8853517_avg2066.64729_std794.27725_random_bw3951.22479_skew1.07292_neigh1.7316_cross0.68495_n14.mtx","synthetic_4284_4284_8891530_avg2075.52054_std785.22497_random_bw3958.54809_skew1.06406_neigh1.7304_cross0.68853_n14.mtx","synthetic_4284_4284_8977954_avg2095.69421_std780.20568_random_bw3962.95075_skew1.04419_neigh1.73103_cross0.68744_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_random_bw2869.04809_skew1.03117_neigh1.83364_cross0.82024_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_random_bw3085.78268_skew1.03117_neigh1.80212_cross0.7937_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_random_bw3110.41713_skew1.03117_neigh1.99854_cross0.71407_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_random_bw3327.52498_skew1.03117_neigh1.77515_cross0.76336_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_random_bw3588.97456_skew1.03117_neigh1.75307_cross0.73669_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_random_bw3823.53688_skew1.03117_neigh1.73708_cross0.7132_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_random_bw3910.22712_skew1.03117_neigh1.7316_cross0.71867_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_random_bw3938.6781_skew1.03117_neigh1.73122_cross0.71141_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_random_bw3943.74907_skew1.03117_neigh1.73054_cross0.70283_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_random_bw3949.98413_skew1.03117_neigh1.7303_cross0.69957_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_random_bw3951.74276_skew1.03117_neigh1.70211_cross0.70719_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_random_bw3960.64356_skew1.03117_neigh1.82061_cross0.66726_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_random_bw3963.83333_skew1.03117_neigh1.75897_cross0.68514_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_random_bw3966.8373_skew1.03117_neigh1.72932_cross0.69638_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_random_bw3969.17367_skew1.03117_neigh1.88668_cross0.64196_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_random_bw3972.03175_skew1.03117_neigh1.64659_cross0.72312_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_random_bw3972.66503_skew1.03117_neigh1.72911_cross0.69369_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_random_bw3985.19958_skew1.03117_neigh1.5959_cross0.73688_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_random_bw3987.60574_skew1.03117_neigh1.7287_cross0.69077_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_random_bw4000.61111_skew1.03117_neigh1.72832_cross0.6858_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_random_bw4001.37395_skew1.03117_neigh1.54602_cross0.75029_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_random_bw4004.8268_skew1.03117_neigh1.9578_cross0.61085_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_random_bw4005.42204_skew1.03117_neigh1.72805_cross0.6856_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_random_bw4006.26377_skew1.03117_neigh1.72822_cross0.67994_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_random_bw4022.77614_skew1.03117_neigh1.49943_cross0.76079_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_random_bw4107.28268_skew1.03117_neigh1.72254_cross0.68493_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_random_bw4115.8254_skew1.03117_neigh1.72168_cross0.68593_n14.mtx","synthetic_4284_4284_9091186_avg2122.12558_std733.44441_random_bw3969.63002_skew1.01873_neigh1.72798_cross0.70101_n14.mtx","synthetic_4284_4284_9176988_avg2142.15406_std713.01815_random_bw3972.35014_skew0.99986_neigh1.72769_cross0.70804_n14.mtx","synthetic_4284_4284_9196527_avg2146.71499_std692.64997_random_bw3992.58567_skew0.99561_neigh1.72588_cross0.71021_n14.mtx","synthetic_4284_4284_9292087_avg2169.02124_std655.40775_random_bw3988.24556_skew0.97508_neigh1.72422_cross0.71568_n14.mtx","synthetic_4284_4284_9379803_avg2189.4965_std621.87428_random_bw4005.78431_skew0.95661_neigh1.72224_cross0.72544_n14.mtx","synthetic_4284_4284_9448827_avg2205.60854_std630.77463_random_bw4007.62162_skew0.94232_neigh1.72532_cross0.7253_n14.mtx","synthetic_4284_4284_9456745_avg2207.45682_std723.4564_random_bw3975.21195_skew0.94069_neigh1.73405_cross0.72004_n14.mtx","synthetic_4284_4284_9460554_avg2208.34594_std591.75846_random_bw3997.86905_skew0.93991_neigh1.72216_cross0.73111_n14.mtx","synthetic_4284_4284_9703417_avg2265.03665_std553.19719_random_bw4014.98599_skew0.89136_neigh1.72415_cross0.74686_n14.mtx","synthetic_4284_4284_9941769_avg2320.67437_std481.87027_random_bw4030.17951_skew0.84602_neigh1.72296_cross0.75864_n14.mtx","synthetic_43542_43542_4892832_avg112.3704_std88.7642_random_bw20697.51918_skew3.82333_neigh1.89327_cross0.62809_n14.mtx","synthetic_4426_4426_9354099_avg2113.44306_std776.70231_random_bw4087.97854_skew1.09421_neigh1.72575_cross0.6866_n14.mtx","synthetic_44473_44473_17007427_avg382.4214_std245.44406_random_bw19571.46993_skew1.57046_neigh1.97497_cross0.92019_n14.mtx","synthetic_44686_44686_9904566_avg221.64808_std96.54092_random_bw36393.77102_skew15.40889_neigh1.70401_cross0.85291_n14.mtx","synthetic_44914_44914_5350806_avg119.13448_std31.75488_random_bw4611.11074_skew1.14044_neigh1.79301_cross0.80788_n14.mtx","synthetic_4533668_4533668_18144199_avg4.0021_std1.18568_random_bw4391.88464_skew312.83513_neigh0.41656_cross0.6482_n14.mtx","synthetic_46351_46351_5209319_avg112.38849_std88.86396_random_bw22009.55751_skew3.82256_neigh1.89342_cross0.62869_n14.mtx","synthetic_46524_46524_7261288_avg156.07618_std357.60767_random_bw27277.78536_skew297.08521_neigh1.84057_cross0.94343_n14.mtx","synthetic_4690002_4690002_14082483_avg3.00266_std1.15153_random_bw5393.19648_skew319.7156_neigh0.36894_cross0.64814_n14.mtx","synthetic_4690002_4690002_14083544_avg3.00289_std1.14359_random_bw5393.22819_skew291.71835_neigh0.36895_cross0.64814_n14.mtx","synthetic_4690002_4690002_18765165_avg4.0011_std0.88465_random_bw4545.45148_skew312.91371_neigh0.41524_cross0.6488_n14.mtx","synthetic_4690002_4690002_18766060_avg4.00129_std0.94875_random_bw4545.42947_skew312.89874_neigh0.41525_cross0.6488_n14.mtx","synthetic_4690002_4690002_18767037_avg4.0015_std1.01413_random_bw4545.42698_skew312.88239_neigh0.41527_cross0.6488_n14.mtx","synthetic_4690002_4690002_18767925_avg4.00169_std1.23629_random_bw4545.43862_skew406.578_neigh0.4153_cross0.6488_n14.mtx","synthetic_4690002_4690002_18768094_avg4.00172_std1.08092_random_bw4545.42342_skew312.86472_neigh0.41527_cross0.6488_n14.mtx","synthetic_4690002_4690002_18768255_avg4.00176_std1.22278_random_bw4545.43101_skew385.58006_neigh0.41529_cross0.6488_n14.mtx","synthetic_4690002_4690002_18768639_avg4.00184_std1.21074_random_bw4545.44234_skew364.83169_neigh0.41529_cross0.6488_n14.mtx","synthetic_4690002_4690002_18769069_avg4.00193_std1.19917_random_bw4545.4511_skew344.08333_neigh0.4153_cross0.6488_n14.mtx","synthetic_4690002_4690002_18769236_avg4.00197_std1.1487_random_bw4545.4197_skew312.84562_neigh0.41529_cross0.6488_n14.mtx","synthetic_4690002_4690002_18769566_avg4.00204_std1.18832_random_bw4545.4388_skew323.33476_neigh0.41528_cross0.6488_n14.mtx","synthetic_4690002_4690002_18769585_avg4.00204_std1.18853_random_bw4545.45064_skew323.33443_neigh0.41531_cross0.6488_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_random_bw3185.35558_skew312.8356_neigh0.41546_cross0.64898_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_random_bw3466.9726_skew312.8356_neigh0.41544_cross0.64891_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_random_bw3796.51273_skew312.8356_neigh0.41538_cross0.64888_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_random_bw4075.34022_skew312.8356_neigh0.41535_cross0.64886_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_random_bw4201.23754_skew312.8356_neigh0.54031_cross0.64874_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_random_bw4280.60386_skew312.8356_neigh0.51221_cross0.64875_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_random_bw4358.8812_skew312.8356_neigh0.48441_cross0.64883_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_random_bw4407.96081_skew312.8356_neigh0.41528_cross0.64881_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_random_bw4434.70451_skew312.8356_neigh0.45658_cross0.6486_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_random_bw4508.95932_skew312.8356_neigh0.429_cross0.64869_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_random_bw4543.29273_skew312.8356_neigh0.41514_cross0.58411_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_random_bw4544.09158_skew312.8356_neigh0.4154_cross0.62714_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_random_bw4544.19255_skew312.8356_neigh0.41607_cross0.84262_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_random_bw4544.8615_skew312.8356_neigh0.41542_cross0.45413_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_random_bw4545.23971_skew312.8356_neigh0.41497_cross0.67016_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_random_bw4545.54195_skew312.8356_neigh0.41601_cross0.79982_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_random_bw4545.90204_skew312.8356_neigh0.41502_cross0.54071_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_random_bw4546.18642_skew312.8356_neigh0.4153_cross0.49753_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_random_bw4547.86071_skew312.8356_neigh0.41468_cross0.71352_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_random_bw4548.10776_skew312.8356_neigh0.41515_cross0.7566_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_random_bw4582.29087_skew312.8356_neigh0.40122_cross0.6487_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_random_bw4653.77434_skew312.8356_neigh0.37381_cross0.64862_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_random_bw4685.95172_skew312.8356_neigh0.41524_cross0.64878_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_random_bw4721.25686_skew312.8356_neigh0.34632_cross0.64851_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_random_bw4788.70054_skew312.8356_neigh0.31846_cross0.64863_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_random_bw4855.60684_skew312.8356_neigh0.29083_cross0.64871_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_random_bw5014.52865_skew312.8356_neigh0.41525_cross0.64877_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_random_bw5290.58011_skew312.8356_neigh0.41527_cross0.64873_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_random_bw5624.25737_skew312.8356_neigh0.41525_cross0.64872_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_random_bw5905.56921_skew312.8356_neigh0.4152_cross0.64868_n14.mtx","synthetic_4690002_4690002_18770121_avg4.00216_std1.17768_random_bw4545.45419_skew302.33648_neigh0.4153_cross0.6488_n14.mtx","synthetic_4690002_4690002_18770136_avg4.00216_std1.17836_random_bw4545.46735_skew302.5861_neigh0.4153_cross0.6488_n14.mtx","synthetic_4690002_4690002_18770459_avg4.00223_std1.21745_random_bw4545.43962_skew312.82517_neigh0.4153_cross0.6488_n14.mtx","synthetic_4690002_4690002_18770757_avg4.00229_std1.16774_random_bw4545.45494_skew281.58808_neigh0.4153_cross0.6488_n14.mtx","synthetic_4690002_4690002_18770794_avg4.0023_std1.16864_random_bw4545.45869_skew281.83738_neigh0.4153_cross0.6488_n14.mtx","synthetic_4690002_4690002_18771502_avg4.00245_std1.15845_random_bw4545.49654_skew260.58973_neigh0.41529_cross0.6488_n14.mtx","synthetic_4690002_4690002_18771567_avg4.00246_std1.15985_random_bw4545.48401_skew260.83867_neigh0.41529_cross0.6488_n14.mtx","synthetic_4690002_4690002_18771765_avg4.00251_std1.28647_random_bw4545.47248_skew312.80334_neigh0.41532_cross0.6488_n14.mtx","synthetic_4690002_4690002_18773155_avg4.0028_std1.35648_random_bw4545.50339_skew312.7801_neigh0.41532_cross0.6488_n14.mtx","synthetic_4690002_4690002_18773573_avg4.00289_std1.14442_random_bw4545.51523_skew219.34068_neigh0.4153_cross0.6488_n14.mtx","synthetic_4690002_4690002_18774622_avg4.00312_std1.42675_random_bw4545.47999_skew312.75559_neigh0.41534_cross0.6488_n14.mtx","synthetic_4690002_4690002_18776169_avg4.00345_std1.497_random_bw4545.44654_skew312.72973_neigh0.41535_cross0.6488_n14.mtx","synthetic_4690002_4690002_23458256_avg5.00176_std1.22307_random_bw4356.33337_skew308.6911_neigh0.44326_cross0.64833_n14.mtx","synthetic_4690002_4690002_23458625_avg5.00184_std1.21051_random_bw4356.3482_skew292.09232_neigh0.44326_cross0.64833_n14.mtx","synthetic_4690002_4690002_23459038_avg5.00192_std1.19839_random_bw4356.34599_skew275.29363_neigh0.44327_cross0.64833_n14.mtx","synthetic_4690002_4690002_28147894_avg6.00168_std1.23536_random_bw4777.70775_skew271.09045_neigh0.46196_cross0.6485_n14.mtx","synthetic_47014_47014_17979151_avg382.42121_std245.44364_random_bw21213.06834_skew1.57046_neigh1.97471_cross0.91776_n14.mtx","synthetic_4712_4712_10000031_avg2122.24767_std807.79622_random_bw4350.07343_skew1.22029_neigh1.71541_cross0.66543_n14.mtx","synthetic_47342_47342_5640490_avg119.14347_std31.78033_random_bw4829.64226_skew1.14028_neigh1.79166_cross0.81115_n14.mtx","synthetic_47684_47684_3704607_avg77.69078_std983.84259_random_bw1475.52561_skew612.76655_neigh1.33846_cross0.83857_n14.mtx","synthetic_4846335_4846335_19395478_avg4.00209_std1.18059_random_bw4694.95912_skew312.83587_neigh0.41556_cross0.64846_n14.mtx","synthetic_48942_48942_10847397_avg221.6378_std96.4853_random_bw40035.34643_skew15.40966_neigh1.70398_cross0.84816_n14.mtx","synthetic_49161_49161_5521770_avg112.32013_std88.82977_random_bw23368.76097_skew3.82549_neigh1.89374_cross0.62848_n14.mtx","synthetic_49556_49556_18951229_avg382.42047_std245.44344_random_bw21913.94158_skew1.57047_neigh1.97494_cross0.91632_n14.mtx","synthetic_4998_4998_10667079_avg2134.26951_std834.36333_random_bw4642.41957_skew1.34178_neigh1.70696_cross0.64169_n14.mtx","synthetic_50954_50954_7955612_avg156.13322_std368.01601_random_bw29951.25091_skew325.34951_neigh1.84107_cross0.94282_n14.mtx","synthetic_5159002_5159002_20646759_avg4.00208_std1.17597_random_bw5000.15374_skew312.8365_neigh0.41584_cross0.64845_n14.mtx","synthetic_51970_51970_5840112_avg112.37468_std88.844_random_bw24618.90708_skew3.82315_neigh1.89384_cross0.62918_n14.mtx","synthetic_52226_52226_4093416_avg78.37889_std1019.1394_random_bw1610.58794_skew665.32736_neigh1.33958_cross0.83862_n14.mtx","synthetic_5283_5283_11298374_avg2138.62843_std877.77445_random_bw4852.64547_skew1.47027_neigh1.70262_cross0.62729_n14.mtx","synthetic_53198_53198_11791218_avg221.64777_std96.43769_random_bw43503.74766_skew15.40892_neigh1.70456_cross0.84719_n14.mtx","synthetic_5471669_5471669_21898040_avg4.00208_std1.17221_random_bw5302.30865_skew312.83705_neigh0.41568_cross0.64833_n14.mtx","synthetic_54779_54779_6163317_avg112.5124_std88.83086_random_bw26024.3978_skew3.81725_neigh1.89353_cross0.62937_n14.mtx","synthetic_55385_55385_8650205_avg156.18317_std377.56925_random_bw32639.28862_skew353.61567_neigh1.83893_cross0.94232_n14.mtx","synthetic_5569_5569_11949345_avg2145.68953_std911.81863_random_bw5136.07919_skew1.59544_neigh1.69647_cross0.60886_n14.mtx","synthetic_56767_56767_4479663_avg78.91315_std1046.34011_random_bw1746.70014_skew718.36043_neigh1.34074_cross0.83789_n14.mtx","synthetic_57454_57454_12734908_avg221.65398_std96.39664_random_bw46842.79518_skew15.40846_neigh1.70371_cross0.85054_n14.mtx","synthetic_5784335_5784335_23149312_avg4.00207_std1.16847_random_bw5605.51869_skew312.83761_neigh0.41582_cross0.64832_n14.mtx","synthetic_58333_58333_4204101_avg72.07071_std19.00931_random_bw3352.53952_skew1.13679_neigh1.65318_cross0.7705_n14.mtx","synthetic_59816_59816_9344868_avg156.2269_std386.43932_random_bw35412.98975_skew381.87902_neigh1.83858_cross0.94305_n14.mtx","synthetic_603864_603864_13450351_avg22.27381_std30.02405_random_bw152862.83998_skew313.58473_neigh1.26422_cross0.75345_n14.mtx","synthetic_6097002_6097002_24400590_avg4.00206_std1.16523_random_bw5910.6367_skew312.83809_neigh0.41581_cross0.64818_n14.mtx","synthetic_61308_61308_4858098_avg79.24085_std1065.00271_random_bw1888.99212_skew772.69186_neigh1.34152_cross0.83808_n14.mtx","synthetic_61710_61710_13678136_avg221.65186_std96.36424_random_bw50412.03223_skew15.40862_neigh1.70378_cross0.84921_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_random_bw399.28715_skew0.93375_neigh1.57754_cross0.89197_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_random_bw427.54729_skew0.93375_neigh1.17987_cross0.83144_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_random_bw430.40577_skew0.93375_neigh1.2674_cross0.82956_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_random_bw430.53472_skew0.93375_neigh1.35298_cross0.83078_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_random_bw431.15425_skew0.93375_neigh1.56885_cross0.85081_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_random_bw432.49407_skew0.93375_neigh1.44041_cross0.82902_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_random_bw437.05305_skew0.93375_neigh1.52957_cross0.82898_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_random_bw443.57612_skew0.93375_neigh1.61781_cross0.82827_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_random_bw444.27433_skew0.93375_neigh1.57185_cross0.82753_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_random_bw453.26434_skew0.93375_neigh1.57391_cross0.80665_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_random_bw471.54689_skew0.93375_neigh1.58217_cross0.75806_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_random_bw472.59519_skew0.93375_neigh1.70735_cross0.82682_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_random_bw482.56044_skew0.93375_neigh1.58512_cross0.71559_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_random_bw493.41782_skew0.93375_neigh1.58831_cross0.66574_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_random_bw501.92205_skew0.93375_neigh1.59187_cross0.621_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_random_bw533.47514_skew0.93375_neigh1.8037_cross0.82219_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_random_bw573.2297_skew0.93375_neigh1.56189_cross0.82326_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_random_bw653.65655_skew0.93375_neigh1.90485_cross0.81986_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_random_bw683.09787_skew0.93375_neigh1.8984_cross0.81849_n14.mtx","synthetic_63838_63838_10847423_avg169.9211_std96.15794_random_bw50815.19321_skew15.38408_neigh1.69783_cross0.83843_n14.mtx","synthetic_63838_63838_11790856_avg184.69965_std96.20654_random_bw51110.24539_skew15.38877_neigh1.69913_cross0.84602_n14.mtx","synthetic_63838_63838_12734460_avg199.48087_std96.25772_random_bw51593.46717_skew15.39756_neigh1.70252_cross0.84507_n14.mtx","synthetic_63838_63838_13677970_avg214.26063_std96.31751_random_bw51973.46416_skew15.40525_neigh1.70373_cross0.84684_n14.mtx","synthetic_63838_63838_14148710_avg221.63461_std96.12924_random_bw51283.67085_skew11.06039_neigh1.70527_cross0.84272_n14.mtx","synthetic_63838_63838_14148875_avg221.63719_std96.16987_random_bw51535.13301_skew12.02579_neigh1.70395_cross0.8451_n14.mtx","synthetic_63838_63838_14148974_avg221.63874_std125.01008_random_bw50694.22474_skew15.38251_neigh1.706_cross0.83858_n14.mtx","synthetic_63838_63838_14149076_avg221.64034_std96.21451_random_bw51658.38851_skew12.99114_neigh1.70423_cross0.84718_n14.mtx","synthetic_63838_63838_14149188_avg221.64209_std118.63327_random_bw50889.44055_skew15.38678_neigh1.70517_cross0.84295_n14.mtx","synthetic_63838_63838_14149208_avg221.64241_std96.26387_random_bw51855.74965_skew13.96104_neigh1.70421_cross0.84669_n14.mtx","synthetic_63838_63838_14149296_avg221.64379_std112.26231_random_bw51274.57478_skew15.39568_neigh1.70495_cross0.84257_n14.mtx","synthetic_63838_63838_14149329_avg221.6443_std105.89153_random_bw51549.08313_skew15.40015_neigh1.70303_cross0.84806_n14.mtx","synthetic_63838_63838_14149335_avg221.6444_std96.31796_random_bw52140.13924_skew14.92641_neigh1.70443_cross0.84646_n14.mtx","synthetic_63838_63838_14149338_avg221.64444_std99.52693_random_bw52057.82066_skew15.40465_neigh1.70386_cross0.84887_n14.mtx","synthetic_63838_63838_14149538_avg221.64758_std93.16614_random_bw52341.45014_skew15.40893_neigh1.70422_cross0.84685_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_random_bw31553.90656_skew15.40883_neigh1.97927_cross0.90185_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_random_bw36435.81718_skew15.40883_neigh1.70415_cross0.85204_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_random_bw40001.91804_skew15.40883_neigh1.70412_cross0.84856_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_random_bw41994.67554_skew15.40883_neigh1.62327_cross0.97823_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_random_bw43525.69963_skew15.40883_neigh1.70435_cross0.8478_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_random_bw46863.79909_skew15.40883_neigh1.70408_cross0.85043_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_random_bw48556.48783_skew15.40883_neigh1.67822_cross0.93499_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_random_bw50400.40091_skew15.40883_neigh1.70389_cross0.84914_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_random_bw51440.8804_skew15.40883_neigh1.69826_cross0.87894_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_random_bw51860.61629_skew15.40883_neigh1.19383_cross0.8502_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_random_bw51860.91715_skew15.40883_neigh1.30912_cross0.84912_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_random_bw51934.71139_skew15.40883_neigh1.53425_cross0.84942_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_random_bw51974.98208_skew15.40883_neigh1.42134_cross0.8471_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_random_bw52133.12723_skew15.40883_neigh1.64881_cross0.84612_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_random_bw52424.50489_skew15.40883_neigh1.76191_cross0.85027_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_random_bw52642.58633_skew15.40883_neigh1.70867_cross0.81986_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_random_bw53211.08631_skew15.40883_neigh1.87632_cross0.84853_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_random_bw53321.33316_skew15.40883_neigh1.71205_cross0.7625_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_random_bw53759.27571_skew15.40883_neigh1.71483_cross0.70856_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_random_bw53829.21378_skew15.40883_neigh1.70392_cross0.84808_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_random_bw54127.70973_skew15.40883_neigh1.71701_cross0.64906_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_random_bw54345.60945_skew15.40883_neigh1.71839_cross0.59238_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_random_bw57263.41878_skew15.40883_neigh1.70375_cross0.85062_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_random_bw58199.76162_skew15.40883_neigh1.70478_cross0.85336_n14.mtx","synthetic_63838_63838_14149682_avg221.64983_std96.37657_random_bw52178.12439_skew15.887_neigh1.70295_cross0.84932_n14.mtx","synthetic_63838_63838_14149757_avg221.65101_std96.43499_random_bw52389.17814_skew16.85239_neigh1.70325_cross0.85077_n14.mtx","synthetic_63838_63838_14149836_avg221.65224_std86.81097_random_bw52747.35418_skew15.4131_neigh1.7044_cross0.85031_n14.mtx","synthetic_63838_63838_14149837_avg221.65226_std96.50454_random_bw52563.14853_skew17.81776_neigh1.70366_cross0.84972_n14.mtx","synthetic_63838_63838_14149882_avg221.65297_std80.46101_random_bw53023.30211_skew15.41756_neigh1.70467_cross0.84957_n14.mtx","synthetic_63838_63838_14149914_avg221.65347_std74.11814_random_bw53380.64075_skew15.42203_neigh1.70407_cross0.85405_n14.mtx","synthetic_63838_63838_14150121_avg221.65671_std67.78709_random_bw53636.29238_skew15.4263_neigh1.7049_cross0.85444_n14.mtx","synthetic_63838_63838_14150132_avg221.65688_std96.57304_random_bw52625.73423_skew18.78283_neigh1.70324_cross0.85136_n14.mtx","synthetic_63838_63838_14150259_avg221.65887_std96.64741_random_bw52705.54984_skew19.7481_neigh1.705_cross0.85168_n14.mtx","synthetic_63838_63838_14621335_avg229.03811_std96.37884_random_bw52333.38261_skew15.40775_neigh1.70591_cross0.8492_n14.mtx","synthetic_63838_63838_15564722_avg243.81594_std96.44368_random_bw52583.81193_skew15.41402_neigh1.70736_cross0.85221_n14.mtx","synthetic_63838_63838_16508065_avg258.59308_std96.5168_random_bw52719.01902_skew15.41575_neigh1.70764_cross0.8531_n14.mtx","synthetic_63838_63838_17451479_avg273.37133_std96.59121_random_bw53072.43958_skew15.41723_neigh1.70777_cross0.85229_n14.mtx","synthetic_63838_63838_18395041_avg288.1519_std96.67009_random_bw53224.04721_skew15.42189_neigh1.71047_cross0.85161_n14.mtx","synthetic_63838_63838_9903973_avg155.14228_std96.11613_random_bw50204.46286_skew15.37207_neigh1.69502_cross0.83641_n14.mtx","synthetic_63889_63889_4604376_avg72.06837_std18.98718_random_bw3672.48262_skew1.13686_neigh1.6535_cross0.76942_n14.mtx","synthetic_64247_64247_10039260_avg156.26037_std394.7431_random_bw37749.51279_skew410.15351_neigh1.83858_cross0.94311_n14.mtx","synthetic_65850_65850_5226388_avg79.36808_std1075.21227_random_bw2021.34421_skew828.67864_neigh1.34471_cross0.83794_n14.mtx","synthetic_65965_65965_14620660_avg221.64269_std96.33127_random_bw53946.07292_skew15.40929_neigh1.7042_cross0.84826_n14.mtx","synthetic_661375_661375_14730847_avg22.27306_std29.96346_random_bw167527.31499_skew313.59526_neigh1.26295_cross0.75385_n14.mtx","synthetic_66463_66463_10040261_avg151.06542_std395.7133_random_bw39089.18279_skew424.82214_neigh1.83705_cross0.94111_n14.mtx","synthetic_66463_66463_10375238_avg156.10547_std374.16577_random_bw37885.35229_skew297.97094_neigh1.84271_cross0.9369_n14.mtx","synthetic_66463_66463_10377678_avg156.14218_std378.88433_random_bw38300.96201_skew326.18897_neigh1.84277_cross0.93802_n14.mtx","synthetic_66463_66463_10378609_avg156.15619_std376.2538_random_bw40147.56841_skew424.61873_neigh1.83726_cross0.94556_n14.mtx","synthetic_66463_66463_10380253_avg156.18093_std381.55088_random_bw40132.11695_skew424.55132_neigh1.83814_cross0.94385_n14.mtx","synthetic_66463_66463_10380322_avg156.18197_std384.04911_random_bw38766.29347_skew354.38034_neigh1.84508_cross0.93857_n14.mtx","synthetic_66463_66463_10381996_avg156.20715_std386.69205_random_bw39470.08879_skew424.47988_neigh1.83953_cross0.9449_n14.mtx","synthetic_66463_66463_10382686_avg156.21753_std389.6586_random_bw38980.4995_skew382.5741_neigh1.84251_cross0.94018_n14.mtx","synthetic_66463_66463_10383864_avg156.23526_std391.65048_random_bw39602.28441_skew424.40333_neigh1.84213_cross0.94243_n14.mtx","synthetic_66463_66463_10384998_avg156.25232_std492.58539_random_bw37240.82965_skew424.35688_neigh1.85054_cross0.93288_n14.mtx","synthetic_66463_66463_10385329_avg156.2573_std395.72064_random_bw38976.54612_skew410.73756_neigh1.84231_cross0.94152_n14.mtx","synthetic_66463_66463_10385409_avg156.2585_std471.2363_random_bw37706.06005_skew424.34005_neigh1.84635_cross0.93626_n14.mtx","synthetic_66463_66463_10385572_avg156.26096_std396.41268_random_bw39282.12082_skew424.33337_neigh1.84007_cross0.94251_n14.mtx","synthetic_66463_66463_10385645_avg156.26206_std450.123_random_bw38391.32296_skew424.33038_neigh1.84642_cross0.93565_n14.mtx","synthetic_66463_66463_10386025_avg156.26777_std429.29508_random_bw38743.10672_skew424.31482_neigh1.84579_cross0.93782_n14.mtx","synthetic_66463_66463_10386363_avg156.27286_std408.80831_random_bw38921.95154_skew424.30098_neigh1.84281_cross0.94099_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_random_bw27380.53228_skew424.29959_neigh1.83971_cross0.94311_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_random_bw30115.66172_skew424.29959_neigh1.83841_cross0.94227_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_random_bw32802.33897_skew424.29959_neigh1.8391_cross0.94285_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_random_bw33891.3234_skew424.29959_neigh1.30566_cross0.9399_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_random_bw33914.40305_skew424.29959_neigh1.42021_cross0.9414_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_random_bw34216.51382_skew424.29959_neigh1.54427_cross0.94023_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_random_bw34386.70958_skew424.29959_neigh1.66268_cross0.94107_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_random_bw35355.46844_skew424.29959_neigh1.83904_cross0.94283_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_random_bw35880.75824_skew424.29959_neigh1.78051_cross0.94121_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_random_bw36358.1962_skew424.29959_neigh1.75925_cross0.97917_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_random_bw36828.67158_skew424.29959_neigh1.94766_cross0.95084_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_random_bw37162.55769_skew424.29959_neigh1.78345_cross0.9726_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_random_bw37752.50297_skew424.29959_neigh1.83874_cross0.94308_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_random_bw39749.89256_skew424.29959_neigh1.86406_cross0.91226_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_random_bw39793.17787_skew424.29959_neigh1.90076_cross0.72217_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_random_bw39841.02385_skew424.29959_neigh1.90364_cross0.65888_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_random_bw40073.12002_skew424.29959_neigh1.88547_cross0.84863_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_random_bw40078.15401_skew424.29959_neigh1.89472_cross0.78617_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_random_bw40874.04356_skew424.29959_neigh1.838_cross0.94255_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_random_bw43013.40304_skew424.29959_neigh1.92851_cross0.95044_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_random_bw43275.46909_skew424.29959_neigh1.8389_cross0.94246_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_random_bw45759.36917_skew424.29959_neigh1.83956_cross0.94233_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_random_bw48163.28926_skew424.29959_neigh1.84119_cross0.9421_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_random_bw50080.08679_skew424.29959_neigh1.84023_cross0.94336_n14.mtx","synthetic_66463_66463_10386663_avg156.27737_std388.73672_random_bw39544.24019_skew424.2887_neigh1.83892_cross0.9432_n14.mtx","synthetic_66463_66463_10387014_avg156.28265_std369.1724_random_bw40157.88734_skew424.27433_neigh1.8384_cross0.94372_n14.mtx","synthetic_66463_66463_10387408_avg156.28858_std350.24164_random_bw40130.74235_skew424.25819_neigh1.83783_cross0.94626_n14.mtx","synthetic_66463_66463_10387855_avg156.29531_std332.11193_random_bw40571.57874_skew424.23989_neigh1.8371_cross0.9482_n14.mtx","synthetic_66463_66463_10388533_avg156.30551_std315.01147_random_bw41077.64824_skew424.21214_neigh1.83814_cross0.94745_n14.mtx","synthetic_66463_66463_10730769_avg161.45478_std396.39605_random_bw39721.11865_skew410.65087_neigh1.8442_cross0.94219_n14.mtx","synthetic_66463_66463_11418942_avg171.80901_std391.59435_random_bw40372.93939_skew385.84235_neigh1.84536_cross0.94343_n14.mtx","synthetic_66463_66463_12107419_avg182.16781_std386.59381_random_bw38834.28252_skew363.84492_neigh1.84927_cross0.94558_n14.mtx","synthetic_66463_66463_12795815_avg192.52539_std381.413_random_bw39460.49056_skew344.2168_neigh1.85447_cross0.94527_n14.mtx","synthetic_66463_66463_13484307_avg202.88442_std376.07335_random_bw38668.20735_skew326.59046_neigh1.85481_cross0.94695_n14.mtx","synthetic_66463_66463_7269367_avg109.37464_std374.11902_random_bw41648.04419_skew424.85738_neigh1.81971_cross0.92957_n14.mtx","synthetic_66463_66463_7962072_avg119.79706_std378.84315_random_bw43046.45655_skew424.84518_neigh1.82641_cross0.93386_n14.mtx","synthetic_66463_66463_8654839_avg130.22041_std384.01598_random_bw39759.86055_skew424.83187_neigh1.83083_cross0.93703_n14.mtx","synthetic_66463_66463_9347522_avg140.64249_std389.6368_random_bw40907.57737_skew424.82436_neigh1.83448_cross0.93954_n14.mtx","synthetic_666542_666542_32559308_avg48.8481_std11.96425_random_bw120493.81988_skew1.12905_neigh1.69092_cross0.80241_n14.mtx","synthetic_68121_68121_3787555_avg55.6004_std1068.24938_random_bw1710.80564_skew856.74559_neigh1.75999_cross0.84954_n14.mtx","synthetic_68121_68121_4147401_avg60.88286_std1069.9982_random_bw1924.33426_skew857.05765_neigh1.62192_cross0.84311_n14.mtx","synthetic_68121_68121_4507272_avg66.16568_std1071.85693_random_bw2009.01365_skew857.28489_neigh1.51372_cross0.84049_n14.mtx","synthetic_68121_68121_4867182_avg71.44907_std1073.8353_random_bw2057.30139_skew857.47165_neigh1.43283_cross0.83917_n14.mtx","synthetic_68121_68121_5227217_avg76.7343_std1075.93952_random_bw2082.79121_skew857.61216_neigh1.37242_cross0.83802_n14.mtx","synthetic_68121_68121_5356033_avg78.62528_std1024.59921_random_bw2109.44585_skew865.40068_neigh1.23716_cross0.83754_n14.mtx","synthetic_68121_68121_5371179_avg78.84762_std1041.23556_random_bw2105.22274_skew862.95755_neigh1.25608_cross0.8375_n14.mtx","synthetic_68121_68121_5385572_avg79.05891_std1056.36091_random_bw2101.63207_skew860.64861_neigh1.27562_cross0.83773_n14.mtx","synthetic_68121_68121_5397801_avg79.23843_std1068.36979_random_bw2095.97678_skew858.6965_neigh1.29918_cross0.83772_n14.mtx","synthetic_68121_68121_5398333_avg79.24624_std1068.28806_random_bw2035.98597_skew601.36298_neigh1.51198_cross0.84041_n14.mtx","synthetic_68121_68121_5400082_avg79.27191_std1070.0369_random_bw2057.79974_skew658.4265_neigh1.46277_cross0.83918_n14.mtx","synthetic_68121_68121_5402340_avg79.30506_std1071.88875_random_bw2069.56856_skew715.38556_neigh1.42374_cross0.83882_n14.mtx","synthetic_68121_68121_5404162_avg79.33181_std1073.85709_random_bw2080.78824_skew772.35942_neigh1.38955_cross0.83863_n14.mtx","synthetic_68121_68121_5405884_avg79.35709_std1075.92117_random_bw2092.30701_skew857.41106_neigh1.32607_cross0.83815_n14.mtx","synthetic_68121_68121_5405960_avg79.3582_std834.61171_random_bw2118.46771_skew857.399_neigh1.17219_cross0.83969_n14.mtx","synthetic_68121_68121_5406317_avg79.36344_std1075.95011_random_bw2086.76499_skew829.23113_neigh1.35604_cross0.83848_n14.mtx","synthetic_68121_68121_5406423_avg79.365_std903.5954_random_bw2113.96985_skew857.32548_neigh1.20794_cross0.83738_n14.mtx","synthetic_68121_68121_5406595_avg79.36752_std972.82101_random_bw2105.47931_skew857.29818_neigh1.25674_cross0.83732_n14.mtx","synthetic_68121_68121_5406754_avg79.36986_std765.8736_random_bw2126.17506_skew857.27294_neigh1.12346_cross0.86663_n14.mtx","synthetic_68121_68121_5407048_avg79.37417_std1042.23736_random_bw2098.32533_skew857.22627_neigh1.3103_cross0.83794_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_random_bw1475.2481_skew857.20278_neigh1.35089_cross0.83825_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_random_bw1616.29047_skew857.20278_neigh1.3466_cross0.83808_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_random_bw1747.72823_skew857.20278_neigh1.34503_cross0.83777_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_random_bw1888.70535_skew857.20278_neigh1.34185_cross0.83764_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_random_bw2022.47778_skew857.20278_neigh1.34097_cross0.83789_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_random_bw2074.43904_skew857.20278_neigh1.34668_cross0.84315_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_random_bw2084.69378_skew857.20278_neigh1.47356_cross0.83912_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_random_bw2084.8877_skew857.20278_neigh1.41898_cross0.83878_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_random_bw2087.46808_skew857.20278_neigh1.44672_cross0.83882_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_random_bw2087.55843_skew857.20278_neigh1.38435_cross0.83848_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_random_bw2091.70347_skew857.20278_neigh1.24164_cross0.83704_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_random_bw2092.93644_skew857.20278_neigh1.27201_cross0.83748_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_random_bw2093.47124_skew857.20278_neigh1.29801_cross0.83744_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_random_bw2093.49917_skew857.20278_neigh1.35469_cross0.83806_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_random_bw2093.88641_skew857.20278_neigh1.32253_cross0.83718_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_random_bw2096.20556_skew857.20278_neigh1.21315_cross0.83699_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_random_bw2157.43003_skew857.20278_neigh1.33753_cross0.83793_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_random_bw2184.16415_skew857.20278_neigh1.33864_cross0.81059_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_random_bw2297.56153_skew857.20278_neigh1.34152_cross0.83789_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_random_bw2316.85899_skew857.20278_neigh1.33597_cross0.75893_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_random_bw2425.34644_skew857.20278_neigh1.33787_cross0.70358_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_random_bw2429.8828_skew857.20278_neigh1.33672_cross0.83792_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_random_bw2506.6977_skew857.20278_neigh1.33929_cross0.65052_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_random_bw2565.36768_skew857.20278_neigh1.33531_cross0.83829_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_random_bw2580.54001_skew857.20278_neigh1.34006_cross0.59378_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_random_bw2698.36137_skew857.20278_neigh1.33462_cross0.83799_n14.mtx","synthetic_68121_68121_5407447_avg79.38003_std1111.80732_random_bw2086.89475_skew857.16294_neigh1.369_cross0.83803_n14.mtx","synthetic_68121_68121_5407867_avg79.3862_std1181.49691_random_bw2070.29451_skew857.0963_neigh1.42499_cross0.83866_n14.mtx","synthetic_68121_68121_5408409_avg79.39415_std1251.28152_random_bw2052.9943_skew857.0103_neigh1.49396_cross0.83934_n14.mtx","synthetic_68121_68121_5409111_avg79.40446_std1321.14786_random_bw2021.97833_skew856.89895_neigh1.56017_cross0.84082_n14.mtx","synthetic_68121_68121_5409514_avg79.41037_std1391.07361_random_bw1983.1191_skew856.83504_neigh1.63329_cross0.84144_n14.mtx","synthetic_68121_68121_5584961_avg81.98589_std1075.92069_random_bw2099.22202_skew829.88685_neigh1.31053_cross0.83776_n14.mtx","synthetic_68121_68121_5934842_avg87.12206_std1068.36291_random_bw2110.99739_skew780.90298_neigh1.27284_cross0.83727_n14.mtx","synthetic_68121_68121_6280428_avg92.19518_std1056.3285_random_bw2123.14184_skew737.87809_neigh1.23532_cross0.83721_n14.mtx","synthetic_68121_68121_6623921_avg97.23758_std1041.17147_random_bw2125.39362_skew699.5625_neigh1.21008_cross0.83733_n14.mtx","synthetic_68121_68121_6966795_avg102.27089_std1024.4946_random_bw2134.44271_skew665.08399_neigh1.1864_cross0.83805_n14.mtx","synthetic_68678_68678_10731088_avg156.25219_std397.34109_random_bw40858.96562_skew424.88203_neigh1.83835_cross0.94238_n14.mtx","synthetic_690692_690692_50143771_avg72.59932_std15.84383_random_bw10372.13539_skew0.96972_neigh1.69572_cross0.81872_n14.mtx","synthetic_69445_69445_5001284_avg72.01791_std19.00309_random_bw3971.49729_skew1.13836_neigh1.65217_cross0.77151_n14.mtx","synthetic_70221_70221_15563153_avg221.63104_std96.30657_random_bw57348.75106_skew15.41016_neigh1.70385_cross0.85141_n14.mtx","synthetic_70391_70391_5585080_avg79.34367_std1076.52341_random_bw2156.59175_skew858.03517_neigh1.33809_cross0.83781_n14.mtx","synthetic_718886_718886_16011537_avg22.27271_std29.91357_random_bw181769.97437_skew313.60029_neigh1.26418_cross0.75622_n14.mtx","synthetic_730022_730022_35670999_avg48.86291_std11.95289_random_bw131807.11807_skew1.14887_neigh1.69169_cross0.8019_n14.mtx","synthetic_73109_73109_11420815_avg156.21627_std394.47456_random_bw43325.79208_skew424.97997_neigh1.83945_cross0.94217_n14.mtx","synthetic_74477_74477_16505933_avg221.62457_std96.28344_random_bw60807.87753_skew15.41064_neigh1.70429_cross0.84959_n14.mtx","synthetic_74933_74933_5942603_avg79.30555_std1075.58283_random_bw2296.59318_skew858.44801_neigh1.34268_cross0.83766_n14.mtx","synthetic_75000_75000_5399893_avg71.99857_std19.01052_random_bw4301.01067_skew1.13893_neigh1.6539_cross0.76935_n14.mtx","synthetic_756472_756472_54943929_avg72.63181_std15.80449_random_bw11344.89478_skew1.03767_neigh1.69516_cross0.81986_n14.mtx","synthetic_77540_77540_12111057_avg156.19109_std391.93861_random_bw45639.49656_skew425.04863_neigh1.84051_cross0.94184_n14.mtx","synthetic_776397_776397_17292444_avg22.27268_std29.86978_random_bw196406.45729_skew313.60063_neigh1.26218_cross0.75486_n14.mtx","synthetic_78733_78733_17449768_avg221.6322_std96.26215_random_bw64358.73616_skew15.41007_neigh1.70377_cross0.84821_n14.mtx","synthetic_793502_793502_38791634_avg48.88662_std11.94806_random_bw143383.51376_skew1.39329_neigh1.69098_cross0.8029_n14.mtx","synthetic_79474_79474_6302859_avg79.30718_std1074.75192_random_bw2429.36754_skew858.43036_neigh1.33896_cross0.83795_n14.mtx","synthetic_80556_80556_5802157_avg72.02638_std19.02772_random_bw4624.77567_skew1.13811_neigh1.6521_cross0.76889_n14.mtx","synthetic_81971_81971_12801291_avg156.16854_std389.68003_random_bw48091.50462_skew425.11016_neigh1.8392_cross0.9423_n14.mtx","synthetic_822252_822252_59710654_avg72.61844_std15.79674_random_bw12337.39137_skew1.02428_neigh1.69509_cross0.82013_n14.mtx","synthetic_82989_82989_18393238_avg221.63465_std96.24293_random_bw67768.63417_skew15.40989_neigh1.70426_cross0.84856_n14.mtx","synthetic_83334_83334_4201657_avg50.41948_std18.95017_random_bw4791.17217_skew1.61804_neigh1.63203_cross0.73132_n14.mtx","synthetic_83334_83334_4601152_avg55.21338_std18.98799_random_bw4722.67207_skew1.48128_neigh1.63833_cross0.74354_n14.mtx","synthetic_83334_83334_5001407_avg60.0164_std19.00589_random_bw4672.15651_skew1.36602_neigh1.64462_cross0.75232_n14.mtx","synthetic_83334_83334_5402033_avg64.82388_std19.01381_random_bw4770.34201_skew1.25226_neigh1.64891_cross0.75864_n14.mtx","synthetic_83334_83334_5802662_avg69.63139_std19.01513_random_bw4732.86769_skew1.16856_neigh1.65263_cross0.76629_n14.mtx","synthetic_83334_83334_6001725_avg72.02012_std24.68331_random_bw4665.54866_skew1.47153_neigh1.65687_cross0.7425_n14.mtx","synthetic_83334_83334_6001767_avg72.02063_std23.43414_random_bw4680.6887_skew1.40209_neigh1.65691_cross0.74737_n14.mtx","synthetic_83334_83334_6001894_avg72.02215_std22.17621_random_bw4719.37063_skew1.31873_neigh1.65546_cross0.7542_n14.mtx","synthetic_83334_83334_6002259_avg72.02653_std20.9136_random_bw4737.64665_skew1.24917_neigh1.65459_cross0.76108_n14.mtx","synthetic_83334_83334_6002747_avg72.03239_std19.64906_random_bw4762.0232_skew1.16569_neigh1.65259_cross0.76576_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_random_bw3350.26591_skew1.13786_neigh1.65428_cross0.77109_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_random_bw3665.49009_skew1.13786_neigh1.65236_cross0.77134_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_random_bw3982.36841_skew1.13786_neigh1.65216_cross0.77018_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_random_bw4220.40105_skew1.13786_neigh1.61436_cross0.87115_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_random_bw4298.73964_skew1.13786_neigh1.65361_cross0.7697_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_random_bw4395.07819_skew1.13786_neigh1.63026_cross0.84649_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_random_bw4592.28661_skew1.13786_neigh1.16508_cross0.77088_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_random_bw4606.03839_skew1.13786_neigh1.27327_cross0.76874_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_random_bw4608.30407_skew1.13786_neigh1.38168_cross0.77039_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_random_bw4614.10663_skew1.13786_neigh1.653_cross0.77104_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_random_bw4654.25229_skew1.13786_neigh1.4901_cross0.76949_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_random_bw4659.61285_skew1.13786_neigh1.6488_cross0.79591_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_random_bw4701.35313_skew1.13786_neigh1.59891_cross0.77081_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_random_bw4776.47572_skew1.13786_neigh1.65401_cross0.76988_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_random_bw4850.09408_skew1.13786_neigh1.65707_cross0.74566_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_random_bw4869.2196_skew1.13786_neigh1.70817_cross0.76996_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_random_bw4931.43106_skew1.13786_neigh1.65277_cross0.76964_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_random_bw4997.26823_skew1.13786_neigh1.66492_cross0.69556_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_random_bw5131.45013_skew1.13786_neigh1.66917_cross0.64549_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_random_bw5220.44701_skew1.13786_neigh1.67321_cross0.59335_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_random_bw5240.64593_skew1.13786_neigh1.65263_cross0.76939_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_random_bw5314.98523_skew1.13786_neigh1.67563_cross0.54557_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_random_bw5580.33847_skew1.13786_neigh1.65308_cross0.76857_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_random_bw5607.76149_skew1.13786_neigh1.81609_cross0.77014_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_random_bw5792.30158_skew1.13786_neigh1.92751_cross0.7686_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_random_bw5891.31062_skew1.13786_neigh1.65284_cross0.76938_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_random_bw5944.6548_skew1.13786_neigh1.92575_cross0.76959_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_random_bw6200.06394_skew1.13786_neigh1.65377_cross0.76946_n14.mtx","synthetic_83334_83334_6003209_avg72.03793_std18.38192_random_bw4782.87446_skew1.09612_neigh1.6521_cross0.77432_n14.mtx","synthetic_83334_83334_6003696_avg72.04378_std17.11472_random_bw4810.91187_skew1.01267_neigh1.65178_cross0.77955_n14.mtx","synthetic_83334_83334_6004305_avg72.05108_std15.84803_random_bw4849.17645_skew0.94307_neigh1.6506_cross0.7874_n14.mtx","synthetic_83334_83334_6004738_avg72.05628_std14.57979_random_bw4873.18033_skew0.87354_neigh1.6472_cross0.79352_n14.mtx","synthetic_83334_83334_6005175_avg72.06152_std13.31259_random_bw4900.61932_skew0.79014_neigh1.6502_cross0.79935_n14.mtx","synthetic_83334_83334_6203256_avg74.43848_std19.01659_random_bw4717.39456_skew1.09569_neigh1.65429_cross0.77391_n14.mtx","synthetic_83334_83334_6603858_avg79.24566_std19.01545_random_bw4770.86497_skew1.03166_neigh1.65742_cross0.7795_n14.mtx","synthetic_83334_83334_7004670_avg84.05537_std19.01725_random_bw4746.86476_skew0.97489_neigh1.66015_cross0.78394_n14.mtx","synthetic_83334_83334_7405486_avg88.86512_std19.01689_random_bw4748.30992_skew0.91301_neigh1.66314_cross0.78781_n14.mtx","synthetic_83334_83334_7806125_avg93.67275_std19.01502_random_bw4799.57248_skew0.86821_neigh1.66483_cross0.79526_n14.mtx","synthetic_833908_833908_18572841_avg22.27205_std29.833_random_bw210844.85277_skew313.60956_neigh1.26298_cross0.75563_n14.mtx","synthetic_84015_84015_6661114_avg79.28482_std1074.00684_random_bw2567.05804_skew858.67278_neigh1.33657_cross0.83743_n14.mtx","synthetic_856982_856982_41861338_avg48.84739_std11.94057_random_bw154675.82077_skew1.3338_neigh1.69011_cross0.80335_n14.mtx","synthetic_86111_86111_6203103_avg72.03613_std19.01524_random_bw4930.68174_skew1.13782_neigh1.65292_cross0.76969_n14.mtx","synthetic_862664_862664_13443097_avg15.58324_std29.57178_random_bw217449.07728_skew313.76132_neigh1.20059_cross0.75196_n14.mtx","synthetic_862664_862664_14783705_avg17.13727_std29.61764_random_bw211187.81291_skew312.46885_neigh1.21655_cross0.75346_n14.mtx","synthetic_862664_862664_15984607_avg18.52935_std29.67134_random_bw215438.01831_skew314.12161_neigh1.22333_cross0.75444_n14.mtx","synthetic_862664_862664_17373620_avg20.1395_std29.72434_random_bw211752.41659_skew312.11605_neigh1.23946_cross0.75537_n14.mtx","synthetic_862664_862664_18524582_avg21.47369_std29.78706_random_bw214344.29615_skew314.40918_neigh1.24035_cross0.75391_n14.mtx","synthetic_862664_862664_19089435_avg22.12847_std21.22646_random_bw218646.30801_skew315.65089_neigh1.27062_cross0.75619_n14.mtx","synthetic_862664_862664_19142097_avg22.18952_std25.02501_random_bw218419.55757_skew314.77975_neigh1.26903_cross0.75599_n14.mtx","synthetic_862664_862664_19162894_avg22.21362_std30.15004_random_bw218402.40102_skew408.47843_neigh1.26817_cross0.75575_n14.mtx","synthetic_862664_862664_19171989_avg22.22417_std26.93718_random_bw218359.93899_skew314.2874_neigh1.26727_cross0.7556_n14.mtx","synthetic_862664_862664_19173147_avg22.22551_std30.06905_random_bw218342.75064_skew387.38255_neigh1.26736_cross0.75562_n14.mtx","synthetic_862664_862664_19184120_avg22.23823_std29.99101_random_bw218334.93381_skew366.25046_neigh1.26648_cross0.75604_n14.mtx","synthetic_862664_862664_19195719_avg22.25168_std29.91773_random_bw218360.52038_skew345.17619_neigh1.26538_cross0.75561_n14.mtx","synthetic_862664_862664_19200670_avg22.25741_std28.85544_random_bw218339.10627_skew313.81644_neigh1.26454_cross0.75565_n14.mtx","synthetic_862664_862664_19207356_avg22.26516_std29.84926_random_bw218277.96985_skew324.12672_neigh1.26399_cross0.75549_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_random_bw152638.10671_skew313.61244_neigh1.26367_cross0.75505_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_random_bw167333.37581_skew313.61244_neigh1.26365_cross0.75511_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_random_bw181720.63529_skew313.61244_neigh1.26353_cross0.75502_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_random_bw196516.2028_skew313.61244_neigh1.26346_cross0.75516_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_random_bw211000.2943_skew313.61244_neigh1.26342_cross0.75507_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_random_bw213175.84954_skew313.61244_neigh0.96905_cross0.75596_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_random_bw213223.70119_skew313.61244_neigh0.88372_cross0.75578_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_random_bw213418.94648_skew313.61244_neigh1.05278_cross0.75588_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_random_bw213434.42462_skew313.61244_neigh1.13672_cross0.75617_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_random_bw213774.00756_skew313.61244_neigh1.22102_cross0.755_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_random_bw214411.79173_skew313.61244_neigh1.30492_cross0.75491_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_random_bw215370.12458_skew313.61244_neigh1.38935_cross0.75586_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_random_bw216786.80808_skew313.61244_neigh1.47408_cross0.75519_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_random_bw218507.74866_skew313.61244_neigh1.26845_cross0.52792_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_random_bw221443.99058_skew313.61244_neigh1.55823_cross0.75678_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_random_bw225527.89524_skew313.61244_neigh1.26339_cross0.75523_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_random_bw233014.20901_skew313.61244_neigh1.6426_cross0.75641_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_random_bw240175.84299_skew313.61244_neigh1.26362_cross0.75505_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_random_bw254512.86048_skew313.61244_neigh1.26353_cross0.7551_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_random_bw269234.4357_skew313.61244_neigh1.26372_cross0.75516_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_random_bw283733.51022_skew313.61244_neigh1.26362_cross0.75506_n14.mtx","synthetic_862664_862664_19218664_avg22.27827_std29.78506_random_bw218120.72798_skew303.10795_neigh1.26259_cross0.75534_n14.mtx","synthetic_862664_862664_19223712_avg22.28412_std30.77861_random_bw218094.95048_skew313.4391_neigh1.2615_cross0.75525_n14.mtx","synthetic_862664_862664_19229090_avg22.29036_std29.72467_random_bw217946.1315_skew282.12689_neigh1.26031_cross0.75517_n14.mtx","synthetic_862664_862664_19237224_avg22.29979_std29.6692_random_bw217791.40434_skew261.15496_neigh1.25666_cross0.75502_n14.mtx","synthetic_862664_862664_19238563_avg22.30134_std32.70575_random_bw217839.24923_skew313.19637_neigh1.25574_cross0.75505_n14.mtx","synthetic_862664_862664_19242590_avg22.30601_std29.61737_random_bw217546.12267_skew240.28029_neigh1.252_cross0.75446_n14.mtx","synthetic_862664_862664_19243461_avg22.30702_std38.50483_random_bw216241.00134_skew313.1164_neigh1.23809_cross0.7533_n14.mtx","synthetic_862664_862664_19244915_avg22.3087_std29.57096_random_bw217130.97238_skew219.45208_neigh1.24755_cross0.75425_n14.mtx","synthetic_862664_862664_19963054_avg23.14117_std29.84969_random_bw212519.73122_skew311.90553_neigh1.25719_cross0.75481_n14.mtx","synthetic_862664_862664_21064571_avg24.41805_std29.92055_random_bw214229.26362_skew314.66815_neigh1.25303_cross0.75497_n14.mtx","synthetic_862664_862664_22549917_avg26.13986_std29.9915_random_bw212791.27921_skew311.74076_neigh1.27038_cross0.75528_n14.mtx","synthetic_862664_862664_23605495_avg27.36349_std30.07249_random_bw214076.51411_skew314.82232_neigh1.26413_cross0.75466_n14.mtx","synthetic_862664_862664_25134553_avg29.13597_std30.15077_random_bw213148.94325_skew311.6376_neigh1.2775_cross0.75586_n14.mtx","synthetic_86401_86401_13491346_avg156.14803_std387.65694_random_bw50727.43053_skew425.16612_neigh1.83819_cross0.94244_n14.mtx","synthetic_88557_88557_7019496_avg79.26529_std1073.34158_random_bw2696.06683_skew858.88461_neigh1.33763_cross0.83799_n14.mtx","synthetic_888032_888032_64506105_avg72.63939_std15.82947_random_bw13315.63085_skew1.00993_neigh1.69523_cross0.81974_n14.mtx","synthetic_891419_891419_19853723_avg22.27204_std29.80105_random_bw225604.82234_skew313.60965_neigh1.26302_cross0.75611_n14.mtx","synthetic_91667_91667_6603697_avg72.04007_std19.01165_random_bw5241.08871_skew1.19322_neigh1.65278_cross0.76932_n14.mtx","synthetic_920462_920462_44975685_avg48.86208_std11.94475_random_bw166256.93189_skew1.2103_neigh1.69039_cross0.80408_n14.mtx","synthetic_948930_948930_21133875_avg22.27127_std29.77202_random_bw239983.61722_skew313.6206_neigh1.26346_cross0.75671_n14.mtx","synthetic_952203_952203_32554296_avg34.1884_std11.93673_random_bw208499.30602_skew1.69097_neigh1.6586_cross0.76805_n14.mtx","synthetic_952203_952203_35650826_avg37.44036_std11.95176_random_bw178182.3534_skew1.53737_neigh1.66758_cross0.77791_n14.mtx","synthetic_952203_952203_38750463_avg40.69559_std11.9572_random_bw182355.14926_skew1.4327_neigh1.67525_cross0.7865_n14.mtx","synthetic_952203_952203_41851500_avg43.95229_std11.95993_random_bw185907.68003_skew1.3207_neigh1.68123_cross0.79456_n14.mtx","synthetic_952203_952203_44952640_avg47.20909_std11.96013_random_bw170436.18015_skew1.22415_neigh1.68791_cross0.80043_n14.mtx","synthetic_952203_952203_46500626_avg48.83478_std13.95044_random_bw170085.21621_skew1.37536_neigh1.69447_cross0.78846_n14.mtx","synthetic_952203_952203_46500963_avg48.83514_std14.74428_random_bw169353.05265_skew1.45725_neigh1.69595_cross0.78191_n14.mtx","synthetic_952203_952203_46501138_avg48.83532_std15.536_random_bw168758.23927_skew1.53915_neigh1.69704_cross0.77638_n14.mtx","synthetic_952203_952203_46502122_avg48.83635_std13.15524_random_bw170806.64163_skew1.31385_neigh1.69248_cross0.79425_n14.mtx","synthetic_952203_952203_46502884_avg48.83715_std12.35905_random_bw171544.88603_skew1.23191_neigh1.69063_cross0.80082_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_random_bw120375.53352_skew1.19093_neigh1.69025_cross0.8034_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_random_bw131783.961_skew1.19093_neigh1.6902_cross0.80347_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_random_bw143412.56495_skew1.19093_neigh1.69053_cross0.8034_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_random_bw154853.41935_skew1.19093_neigh1.69028_cross0.80351_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_random_bw166308.11219_skew1.19093_neigh1.69031_cross0.80348_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_random_bw177753.75801_skew1.19093_neigh1.69007_cross0.80347_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_random_bw189177.26804_skew1.19093_neigh1.69023_cross0.80345_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_random_bw200570.66789_skew1.19093_neigh1.69027_cross0.80345_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_random_bw212041.87573_skew1.19093_neigh1.69043_cross0.80333_n14.mtx","synthetic_952203_952203_46504635_avg48.83899_std11.56217_random_bw172412.673_skew1.14992_neigh1.6892_cross0.80663_n14.mtx","synthetic_952203_952203_46504874_avg48.83924_std10.76526_random_bw173220.71164_skew1.06801_neigh1.68664_cross0.81356_n14.mtx","synthetic_952203_952203_46506617_avg48.84107_std9.96939_random_bw174360.33325_skew0.98603_neigh1.68579_cross0.81888_n14.mtx","synthetic_952203_952203_46507502_avg48.842_std9.17223_random_bw175367.02321_skew0.9041_neigh1.68327_cross0.82582_n14.mtx","synthetic_952203_952203_46508902_avg48.84347_std8.37556_random_bw177002.71627_skew0.82215_neigh1.68094_cross0.8324_n14.mtx","synthetic_952203_952203_48054277_avg50.46642_std11.96125_random_bw173372.13525_skew1.14004_neigh1.69321_cross0.80658_n14.mtx","synthetic_952203_952203_51155719_avg53.72354_std11.96081_random_bw176138.00989_skew1.08475_neigh1.69709_cross0.81142_n14.mtx","synthetic_952203_952203_54257059_avg56.98056_std11.96124_random_bw166937.37724_skew1.01823_neigh1.70097_cross0.81597_n14.mtx","synthetic_952203_952203_57358241_avg60.23741_std11.96072_random_bw169273.03323_skew0.95892_neigh1.70515_cross0.82049_n14.mtx","synthetic_952203_952203_60460102_avg63.49497_std11.9616_random_bw171406.13604_skew0.90566_neigh1.70892_cross0.82386_n14.mtx","synthetic_953812_953812_69251992_avg72.60549_std15.82731_random_bw14300.01986_skew1.05219_neigh1.69474_cross0.82044_n14.mtx","synthetic_968035_968035_11863180_avg12.25491_std37.64729_random_bw22809.30094_skew632.62368_neigh1.3222_cross0.78548_n14.mtx","synthetic_97223_97223_7006972_avg72.07114_std19.01384_random_bw5580.63884_skew1.19228_neigh1.65246_cross0.76899_n14.mtx","synthetic_983943_983943_48052837_avg48.83701_std11.95054_random_bw177774.01613_skew1.12953_neigh1.69013_cross0.804_n14.mtx","synthetic_98611_98611_5466825_avg55.43829_std11.03861_random_bw3858.831_skew0.894_neigh1.63088_cross0.78813_n14.mtx","synthetic_986703_986703_50142919_avg50.81865_std15.80386_random_bw14886.34812_skew1.44005_neigh1.67248_cross0.78656_n14.mtx","synthetic_986703_986703_54918080_avg55.65817_std15.81077_random_bw15208.31576_skew1.31772_neigh1.6788_cross0.79621_n14.mtx","synthetic_986703_986703_59694812_avg60.49927_std15.81312_random_bw14727.02983_skew1.2149_neigh1.68487_cross0.80451_n14.mtx","synthetic_986703_986703_64472911_avg65.34176_std15.81391_random_bw14462.32703_skew1.11197_neigh1.68893_cross0.81154_n14.mtx","synthetic_986703_986703_69250817_avg70.18405_std15.81417_random_bw14701.88268_skew1.0375_neigh1.6931_cross0.81692_n14.mtx","synthetic_986703_986703_71632861_avg72.5982_std20.55272_random_bw14534.71352_skew1.31411_neigh1.69992_cross0.79562_n14.mtx","synthetic_986703_986703_71633918_avg72.59927_std19.50045_random_bw14580.21846_skew1.2452_neigh1.69942_cross0.8008_n14.mtx","synthetic_986703_986703_71635339_avg72.60071_std18.448_random_bw14638.21943_skew1.17629_neigh1.6985_cross0.8063_n14.mtx","synthetic_986703_986703_71637178_avg72.60257_std17.39427_random_bw14693.79283_skew1.10736_neigh1.69733_cross0.81127_n14.mtx","synthetic_986703_986703_71638484_avg72.6039_std16.3415_random_bw14738.44751_skew1.03846_neigh1.69562_cross0.81775_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_random_bw10351.48927_skew1.0109_neigh1.69556_cross0.82059_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_random_bw11350.18676_skew1.0109_neigh1.69576_cross0.82027_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_random_bw13218.95443_skew1.0109_neigh1.63894_cross0.89577_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_random_bw13857.1301_skew1.0109_neigh1.1897_cross0.81891_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_random_bw13898.10027_skew1.0109_neigh1.30212_cross0.81916_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_random_bw14441.63744_skew1.0109_neigh1.63911_cross0.82048_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_random_bw14787.04212_skew1.0109_neigh1.69513_cross0.82047_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_random_bw15272.02993_skew1.0109_neigh1.69545_cross0.82033_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_random_bw15463.25016_skew1.0109_neigh1.75112_cross0.82045_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_random_bw15698.85861_skew1.0109_neigh1.71329_cross0.73895_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_random_bw16106.42649_skew1.0109_neigh1.72022_cross0.68496_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_random_bw19207.41674_skew1.0109_neigh1.69559_cross0.8201_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_random_bw20444.65782_skew1.0109_neigh1.91237_cross0.8206_n14.mtx","synthetic_986703_986703_71640715_avg72.60616_std15.28673_random_bw14825.68345_skew0.96953_neigh1.69453_cross0.82276_n14.mtx","synthetic_986703_986703_71642304_avg72.60777_std14.23366_random_bw14897.2269_skew0.90062_neigh1.69343_cross0.8287_n14.mtx","synthetic_986703_986703_71644194_avg72.60968_std13.17962_random_bw14980.02571_skew0.84548_neigh1.69135_cross0.83479_n14.mtx","synthetic_986703_986703_71646204_avg72.61172_std12.12567_random_bw15073.83895_skew0.77657_neigh1.68951_cross0.84052_n14.mtx","synthetic_986703_986703_71647231_avg72.61276_std11.07215_random_bw15188.71315_skew0.70769_neigh1.68786_cross0.84661_n14.mtx","synthetic_986703_986703_74029078_avg75.02671_std15.81428_random_bw14477.08036_skew0.97263_neigh1.69683_cross0.82287_n14.mtx","synthetic_986703_986703_78806613_avg79.86863_std15.81421_random_bw14655.966_skew0.91565_neigh1.69988_cross0.82778_n14.mtx","synthetic_986703_986703_83583783_avg84.71017_std15.81435_random_bw14520.34653_skew0.86518_neigh1.70339_cross0.83169_n14.mtx","synthetic_986703_986703_88361224_avg89.552_std15.81425_random_bw14683.12554_skew0.82017_neigh1.7057_cross0.83562_n14.mtx","synthetic_986703_986703_93139124_avg94.39428_std15.81414_random_bw14552.95978_skew0.76917_neigh1.70838_cross0.83944_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_random_bw35853.81968_skew632.66322_neigh1.32203_cross0.78644_n14.mtx","synthetic_268096_268096_19097540_avg71.23396_std101.70964_random_bw32428.45227_skew10.16041_neigh1.12478_cross0.8356_n14.mtx","synthetic_268096_268096_20328893_avg75.82692_std102.55303_random_bw32567.72158_skew10.2493_neigh1.12931_cross0.83183_n14.mtx","synthetic_4690002_4690002_18772478_avg4.00266_std1.15153_random_bw4545.51747_skew240.08974_neigh0.41531_cross0.6488_n14.mtx","synthetic_862664_862664_19114072_avg22.15703_std23.12053_random_bw218587.40518_skew315.24275_neigh1.26934_cross0.75615_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_random_bw218387.82176_skew313.61244_neigh1.26748_cross0.57934_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_random_bw223475.36954_skew1.19093_neigh1.69052_cross0.80348_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_random_bw37993.81919_skew632.66322_neigh1.32219_cross0.78642_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_random_bw40249.088_skew632.66322_neigh1.32209_cross0.7864_n14.mtx","synthetic_268096_268096_18479164_avg68.92741_std73.74951_random_bw32856.73895_skew10.4033_neigh1.1224_cross0.84096_n14.mtx","synthetic_268096_268096_18479645_avg68.92921_std80.36452_random_bw32647.3807_skew10.403_neigh1.12506_cross0.82761_n14.mtx","synthetic_268096_268096_18480123_avg68.93099_std86.42344_random_bw32612.95544_skew10.33017_neigh1.12468_cross0.82902_n14.mtx","synthetic_268096_268096_18480967_avg68.93414_std92.41882_random_bw32501.5288_skew10.25712_neigh1.12262_cross0.83284_n14.mtx","synthetic_268096_268096_18481691_avg68.93684_std98.3339_random_bw32411.34374_skew10.16964_neigh1.12316_cross0.83523_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_random_bw22631.95884_skew10.11157_neigh1.1236_cross0.83759_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_random_bw24810.92538_skew10.11157_neigh1.12535_cross0.83717_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_random_bw26946.62422_skew10.11157_neigh1.12451_cross0.83736_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_random_bw29104.95345_skew10.11157_neigh1.1245_cross0.83739_n14.mtx","synthetic_268096_268096_18482362_avg68.93934_std104.15593_random_bw32241.45427_skew10.0677_neigh1.12209_cross0.83937_n14.mtx","synthetic_268096_268096_18483075_avg68.942_std109.86851_random_bw32121.44382_skew9.95123_neigh1.1203_cross0.84327_n14.mtx","synthetic_268096_268096_18483963_avg68.94531_std115.46168_random_bw31912.0398_skew9.82017_neigh1.11692_cross0.84863_n14.mtx","synthetic_268096_268096_18484698_avg68.94806_std120.91939_random_bw31591.06942_skew9.68921_neigh1.11467_cross0.85538_n14.mtx","synthetic_268096_268096_18485774_avg68.95207_std126.22922_random_bw31186.72806_skew9.55806_neigh1.11278_cross0.86343_n14.mtx","synthetic_268096_268096_21560408_avg80.42048_std103.33442_random_bw32634.53398_skew10.30309_neigh1.13401_cross0.8296_n14.mtx","synthetic_268096_268096_22792018_avg85.01439_std104.0723_random_bw32673.47377_skew10.36278_neigh1.13741_cross0.82812_n14.mtx","synthetic_268096_268096_24023179_avg89.60663_std104.77794_random_bw32722.96392_skew10.40541_neigh1.1392_cross0.8264_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_random_bw209088.73821_skew313.61244_neigh1.15533_cross0.98278_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_random_bw216180.85999_skew313.61244_neigh1.23792_cross0.93242_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_random_bw217290.8255_skew313.61244_neigh1.2532_cross0.88202_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_random_bw217679.16493_skew313.61244_neigh1.25924_cross0.83135_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_random_bw218002.31666_skew313.61244_neigh1.26186_cross0.78113_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_random_bw218342.9789_skew313.61244_neigh1.26682_cross0.62984_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_random_bw218343.96868_skew313.61244_neigh1.26388_cross0.73052_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_random_bw218353.64903_skew313.61244_neigh1.26627_cross0.6806_n14.mtx","synthetic_862664_862664_19245171_avg22.309_std34.6357_random_bw217268.52825_skew313.08849_neigh1.25013_cross0.75439_n14.mtx","synthetic_862664_862664_19245832_avg22.30977_std36.57002_random_bw216933.33678_skew313.0777_neigh1.24283_cross0.75358_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_random_bw150854.23009_skew1.19093_neigh1.18385_cross0.80149_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_random_bw151895.95754_skew1.19093_neigh1.29667_cross0.80161_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_random_bw153481.11228_skew1.19093_neigh1.40932_cross0.80268_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_random_bw157146.27492_skew1.19093_neigh1.52169_cross0.80287_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_random_bw161158.68364_skew1.19093_neigh1.61052_cross0.88656_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_random_bw161480.87896_skew1.19093_neigh1.61762_cross0.88363_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_random_bw167259.59789_skew1.19093_neigh1.63354_cross0.80363_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_random_bw169055.00469_skew1.19093_neigh1.67727_cross0.83029_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_random_bw172056.91803_skew1.19093_neigh1.69026_cross0.80348_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_random_bw174466.89467_skew1.19093_neigh1.70003_cross0.77698_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_random_bw178531.20153_skew1.19093_neigh1.71349_cross0.72328_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_random_bw179918.61642_skew1.19093_neigh1.7464_cross0.80393_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_random_bw181221.45031_skew1.19093_neigh1.72367_cross0.66928_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_random_bw183413.76762_skew1.19093_neigh1.73057_cross0.61618_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_random_bw185134.26789_skew1.19093_neigh1.73599_cross0.56267_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_random_bw209833.19969_skew1.19093_neigh1.87978_cross0.80421_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_random_bw241430.72348_skew1.19093_neigh1.85806_cross0.80383_n14.mtx","synthetic_1060229_1060229_12992547_avg12.25447_std37.61142_diagonal_bw24877.30989_skew632.64621_neigh1.32175_cross0.78562_n14.mtx","synthetic_112749_112749_5727652_avg50.80002_std19.54637_diagonal_bw3425.86088_skew1.7559_neigh1.22834_cross0.71806_n14.mtx","synthetic_1152423_1152423_14123090_avg12.25513_std37.58092_diagonal_bw27066.48438_skew632.61237_neigh1.32182_cross0.78662_n14.mtx","synthetic_123486_123486_6272129_avg50.79223_std19.54882_diagonal_bw3753.58169_skew1.75633_neigh1.22934_cross0.71722_n14.mtx","synthetic_1244617_1244617_15251343_avg12.25384_std37.55512_diagonal_bw29219.49964_skew632.67869_neigh1.3223_cross0.78671_n14.mtx","synthetic_1336811_1336811_16381252_avg12.25398_std37.5327_diagonal_bw31339.89957_skew632.6718_neigh1.32267_cross0.78669_n14.mtx","synthetic_134225_134225_6816930_avg50.78733_std19.56874_diagonal_bw4082.86505_skew1.77628_neigh1.22794_cross0.71641_n14.mtx","synthetic_1382908_1382908_11844497_avg8.56492_std37.37396_diagonal_bw40250.63741_skew633.56515_neigh1.20442_cross0.7799_n14.mtx","synthetic_1382908_1382908_12976596_avg9.38356_std37.40231_diagonal_bw42376.82952_skew633.40762_neigh1.23873_cross0.78254_n14.mtx","synthetic_1382908_1382908_14113272_avg10.2055_std37.43376_diagonal_bw29741.04005_skew632.97168_neigh1.26021_cross0.78249_n14.mtx","synthetic_1382908_1382908_15242471_avg11.02204_std37.4678_diagonal_bw30995.34918_skew633.00226_neigh1.2701_cross0.78366_n14.mtx","synthetic_1382908_1382908_16349573_avg11.8226_std37.50438_diagonal_bw31873.17246_skew633.88554_neigh1.28656_cross0.78385_n14.mtx","synthetic_1382908_1382908_16848093_avg12.18309_std26.47929_diagonal_bw31429.05804_skew636.35882_neigh1.38049_cross0.79092_n14.mtx","synthetic_1382908_1382908_16896659_avg12.21821_std28.92386_diagonal_bw31501.87862_skew634.52686_neigh1.37656_cross0.79099_n14.mtx","synthetic_1382908_1382908_16922141_avg12.23664_std48.62398_diagonal_bw32239.82905_skew633.48814_neigh1.28004_cross0.7802_n14.mtx","synthetic_1382908_1382908_16923406_avg12.23755_std46.1541_diagonal_bw32382.62648_skew633.44071_neigh1.28391_cross0.78182_n14.mtx","synthetic_1382908_1382908_16926342_avg12.23967_std43.68598_diagonal_bw32452.39208_skew633.41236_neigh1.2921_cross0.78362_n14.mtx","synthetic_1382908_1382908_16931967_avg12.24374_std41.21881_diagonal_bw32524.36102_skew633.2016_neigh1.30009_cross0.7846_n14.mtx","synthetic_1382908_1382908_16933417_avg12.24479_std31.37528_diagonal_bw31740.52473_skew633.1473_neigh1.36561_cross0.7903_n14.mtx","synthetic_1382908_1382908_16941719_avg12.25079_std38.75385_diagonal_bw32512.7569_skew632.83654_neigh1.31404_cross0.78613_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_diagonal_bw22778.5047_skew632.66322_neigh1.32228_cross0.78682_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_diagonal_bw24917.9724_skew632.66322_neigh1.32217_cross0.78679_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_diagonal_bw27035.12761_skew632.66322_neigh1.32227_cross0.7868_n14.mtx","synthetic_1382908_1382908_16950422_avg12.25709_std36.29175_diagonal_bw32375.96458_skew632.51111_neigh1.33014_cross0.78815_n14.mtx","synthetic_1382908_1382908_16950660_avg12.25726_std33.83118_diagonal_bw32049.12593_skew632.50221_neigh1.34897_cross0.78969_n14.mtx","synthetic_1382908_1382908_17445616_avg12.61517_std37.54329_diagonal_bw32660.91296_skew635.05973_neigh1.31546_cross0.78564_n14.mtx","synthetic_1382908_1382908_18572951_avg13.43036_std37.58404_diagonal_bw33424.56483_skew634.94726_neigh1.34978_cross0.78747_n14.mtx","synthetic_1382908_1382908_19756490_avg14.28619_std37.62781_diagonal_bw28853.64125_skew633.10877_neigh1.36846_cross0.78954_n14.mtx","synthetic_1382908_1382908_20949793_avg15.14909_std37.6751_diagonal_bw29885.40906_skew631.18333_neigh1.36304_cross0.78832_n14.mtx","synthetic_1382908_1382908_22070012_avg15.95913_std37.72535_diagonal_bw30643.05341_skew631.49052_neigh1.34712_cross0.78696_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_diagonal_bw4732.21902_skew0.91217_neigh1.56188_cross0.89987_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_diagonal_bw5037.41918_skew0.91217_neigh1.59723_cross0.86565_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_diagonal_bw5204.83825_skew0.91217_neigh1.14624_cross0.78728_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_diagonal_bw5224.88197_skew0.91217_neigh1.25499_cross0.78723_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_diagonal_bw5232.21638_skew0.91217_neigh1.36051_cross0.78676_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_diagonal_bw5264.87446_skew0.91217_neigh1.46866_cross0.78769_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_diagonal_bw5313.69498_skew0.91217_neigh1.57687_cross0.78795_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_diagonal_bw5335.36823_skew0.91217_neigh1.6227_cross0.815_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_diagonal_bw5457.86719_skew0.91217_neigh1.63077_cross0.78851_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_diagonal_bw5549.51317_skew0.91217_neigh1.63866_cross0.76193_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_diagonal_bw5706.21311_skew0.91217_neigh1.64636_cross0.71043_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_diagonal_bw5738.5913_skew0.91217_neigh1.68499_cross0.78752_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_diagonal_bw5820.74673_skew0.91217_neigh1.65384_cross0.65812_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_diagonal_bw5912.46692_skew0.91217_neigh1.65889_cross0.60755_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_diagonal_bw5978.71195_skew0.91217_neigh1.66285_cross0.5548_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_diagonal_bw6442.07975_skew0.91217_neigh1.902_cross0.78882_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_diagonal_bw6597.81502_skew0.91217_neigh1.90014_cross0.78819_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_diagonal_bw7193.09661_skew0.91217_neigh1.79177_cross0.78885_n14.mtx","synthetic_140874_140874_7812605_avg55.4581_std11.08015_diagonal_bw6152.40279_skew2.11947_neigh1.6145_cross0.85299_n14.mtx","synthetic_140874_140874_7812630_avg55.45828_std11.08_diagonal_bw6149.91056_skew2.06537_neigh1.61493_cross0.85302_n14.mtx","synthetic_1429004_1429004_17510920_avg12.25393_std37.51291_diagonal_bw33558.90432_skew632.67408_neigh1.32147_cross0.78663_n14.mtx","synthetic_144963_144963_7361596_avg50.78259_std19.57819_diagonal_bw4399.88447_skew1.77654_neigh1.22836_cross0.71716_n14.mtx","synthetic_1521198_1521198_18638978_avg12.25283_std37.49579_diagonal_bw35682.83773_skew632.73123_neigh1.32136_cross0.78772_n14.mtx","synthetic_152542_152542_8133417_avg53.3192_std4.76067_diagonal_bw10659.33841_skew3.36991_neigh1.81127_cross0.94184_n14.mtx","synthetic_155701_155701_7905533_avg50.77381_std19.58895_diagonal_bw4730.89037_skew1.77702_neigh1.22735_cross0.71638_n14.mtx","synthetic_161070_161070_10085122_avg62.61329_std19.67482_diagonal_bw4970.48093_skew1.44357_neigh1.23583_cross0.74036_n14.mtx","synthetic_161070_161070_10630527_avg65.99942_std19.67949_diagonal_bw4997.9576_skew1.36366_neigh1.23787_cross0.74712_n14.mtx","synthetic_161070_161070_5763734_avg35.78403_std19.07947_diagonal_bw4691.45329_skew2.52112_neigh1.21157_cross0.6613_n14.mtx","synthetic_161070_161070_6293192_avg39.07116_std19.27381_diagonal_bw4754.31728_skew2.30167_neigh1.21361_cross0.6789_n14.mtx","synthetic_161070_161070_6828184_avg42.39265_std19.41502_diagonal_bw4805.07083_skew2.13734_neigh1.21786_cross0.69146_n14.mtx","synthetic_161070_161070_7366852_avg45.73696_std19.51164_diagonal_bw4846.14217_skew1.97353_neigh1.22207_cross0.70264_n14.mtx","synthetic_161070_161070_7908198_avg49.0979_std19.57809_diagonal_bw4854.49827_skew1.83108_neigh1.2254_cross0.71257_n14.mtx","synthetic_161070_161070_8177031_avg50.76694_std16.39176_diagonal_bw4982.53222_skew1.48193_neigh1.22352_cross0.73821_n14.mtx","synthetic_161070_161070_8177317_avg50.76872_std15.08847_diagonal_bw5015.21029_skew1.36366_neigh1.22314_cross0.74727_n14.mtx","synthetic_161070_161070_8177462_avg50.76962_std17.68784_diagonal_bw4942.64516_skew1.59998_neigh1.22734_cross0.72915_n14.mtx","synthetic_161070_161070_8177930_avg50.77252_std13.78024_diagonal_bw5053.39532_skew1.24531_neigh1.22227_cross0.75674_n14.mtx","synthetic_161070_161070_8178443_avg50.77571_std18.96773_diagonal_bw4913.07711_skew1.71784_neigh1.22757_cross0.72014_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_diagonal_bw3436.43137_skew1.77657_neigh1.22839_cross0.71742_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_diagonal_bw3766.73128_skew1.77657_neigh1.22745_cross0.71673_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_diagonal_bw4085.27859_skew1.77657_neigh1.22766_cross0.7165_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_diagonal_bw4397.09225_skew1.77657_neigh1.19684_cross0.82225_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_diagonal_bw4402.87749_skew1.77657_neigh1.22824_cross0.71675_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_diagonal_bw4582.03666_skew1.77657_neigh1.21427_cross0.78739_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_diagonal_bw4732.34129_skew1.77657_neigh1.22757_cross0.71639_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_diagonal_bw4806.31347_skew1.77657_neigh1.22333_cross0.7394_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_diagonal_bw4848.33162_skew1.77657_neigh0.945_cross0.71714_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_diagonal_bw4854.14888_skew1.77657_neigh0.86533_cross0.7155_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_diagonal_bw4858.05593_skew1.77657_neigh1.10606_cross0.71633_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_diagonal_bw4859.13659_skew1.77657_neigh1.02495_cross0.71552_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_diagonal_bw4887.50244_skew1.77657_neigh1.22794_cross0.7157_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_diagonal_bw4889.05284_skew1.77657_neigh1.18606_cross0.71625_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_diagonal_bw4905.36219_skew1.77657_neigh1.26702_cross0.71513_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_diagonal_bw4920.32821_skew1.77657_neigh1.34684_cross0.71421_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_diagonal_bw4938.19408_skew1.77657_neigh1.42871_cross0.71564_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_diagonal_bw4963.01172_skew1.77657_neigh1.50995_cross0.71659_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_diagonal_bw4974.48644_skew1.77657_neigh1.2296_cross0.6917_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_diagonal_bw5006.7359_skew1.77657_neigh1.58991_cross0.71725_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_diagonal_bw5047.52778_skew1.77657_neigh1.22518_cross0.71711_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_diagonal_bw5128.09611_skew1.77657_neigh1.23448_cross0.64524_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_diagonal_bw5252.15061_skew1.77657_neigh1.23809_cross0.60038_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_diagonal_bw5358.15437_skew1.77657_neigh1.24126_cross0.55385_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_diagonal_bw5368.754_skew1.77657_neigh1.22547_cross0.71707_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_diagonal_bw5466.81631_skew1.77657_neigh1.24378_cross0.50496_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_diagonal_bw5702.6109_skew1.77657_neigh1.22489_cross0.7154_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_diagonal_bw6025.13256_skew1.77657_neigh1.22428_cross0.71487_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_diagonal_bw6352.66382_skew1.77657_neigh1.22335_cross0.71563_n14.mtx","synthetic_161070_161070_8181013_avg50.79166_std20.23119_diagonal_bw4876.39439_skew1.83511_neigh1.22713_cross0.71077_n14.mtx","synthetic_161070_161070_8181297_avg50.79343_std19.67479_diagonal_bw5249.20404_skew2.30751_neigh1.21368_cross0.79983_n14.mtx","synthetic_161070_161070_8184643_avg50.8142_std21.47457_diagonal_bw4835.93918_skew1.95193_neigh1.22927_cross0.70445_n14.mtx","synthetic_161070_161070_8190209_avg50.84876_std22.69412_diagonal_bw4805.6814_skew2.06792_neigh1.23025_cross0.69419_n14.mtx","synthetic_161070_161070_8197860_avg50.89626_std23.8911_diagonal_bw4766.49097_skew2.18295_neigh1.23139_cross0.68671_n14.mtx","synthetic_161070_161070_8207913_avg50.95867_std25.06294_diagonal_bw4739.27473_skew2.29679_neigh1.2307_cross0.67748_n14.mtx","synthetic_161070_161070_8451173_avg52.46895_std19.62109_diagonal_bw4901.00951_skew1.72542_neigh1.22729_cross0.72045_n14.mtx","synthetic_161070_161070_8995114_avg55.84599_std19.64654_diagonal_bw4934.95017_skew1.61433_neigh1.23106_cross0.72911_n14.mtx","synthetic_161070_161070_9539943_avg59.22855_std19.66505_diagonal_bw4956.28317_skew1.53256_neigh1.23358_cross0.73578_n14.mtx","synthetic_1613392_1613392_19770221_avg12.25382_std37.48086_diagonal_bw37889.56893_skew632.67976_neigh1.3217_cross0.78728_n14.mtx","synthetic_166438_166438_8452047_avg50.78195_std19.60687_diagonal_bw5049.85631_skew1.77658_neigh1.22495_cross0.71681_n14.mtx","synthetic_167070_167070_8908050_avg53.31927_std4.75919_diagonal_bw11705.28266_skew3.3699_neigh1.8115_cross0.9416_n14.mtx","synthetic_1705586_1705586_20897972_avg12.25266_std37.46728_diagonal_bw40049.59136_skew632.73974_neigh1.32185_cross0.78691_n14.mtx","synthetic_177177_177177_8998963_avg50.79081_std19.6143_diagonal_bw5379.20051_skew1.77609_neigh1.22643_cross0.71605_n14.mtx","synthetic_1797780_1797780_22027651_avg12.2527_std37.45466_diagonal_bw42214.60549_skew632.7381_neigh1.32143_cross0.78759_n14.mtx","synthetic_181598_181598_9682693_avg53.31938_std4.75813_diagonal_bw12669.60339_skew3.36989_neigh1.81184_cross0.9412_n14.mtx","synthetic_185639_185639_15001505_avg80.81009_std108.51788_diagonal_bw24304.53698_skew5.36059_neigh1.20249_cross0.88135_n14.mtx","synthetic_185639_185639_15005658_avg80.83247_std121.76167_diagonal_bw24068.62026_skew10.22074_neigh1.21908_cross0.83189_n14.mtx","synthetic_185639_185639_15005807_avg80.83327_std121.07944_diagonal_bw24098.67153_skew9.70104_neigh1.21713_cross0.83417_n14.mtx","synthetic_185639_185639_15006000_avg80.83431_std120.29592_diagonal_bw24022.966_skew9.18132_neigh1.21687_cross0.83669_n14.mtx","synthetic_185639_185639_15006242_avg80.83561_std119.38413_diagonal_bw24043.49472_skew8.66158_neigh1.21482_cross0.83974_n14.mtx","synthetic_185639_185639_15006279_avg80.83581_std111.39593_diagonal_bw24110.85286_skew5.93999_neigh1.20322_cross0.86853_n14.mtx","synthetic_185639_185639_15006681_avg80.83798_std118.31728_diagonal_bw24061.44702_skew8.14174_neigh1.21351_cross0.84318_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_diagonal_bw18670.03953_skew7.8818_neigh1.21456_cross0.84468_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_diagonal_bw20186.89682_skew7.8818_neigh1.2138_cross0.8455_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_diagonal_bw21725.13769_skew7.8818_neigh1.21452_cross0.84491_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_diagonal_bw22498.75533_skew7.8818_neigh1.19263_cross0.87688_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_diagonal_bw22743.18636_skew7.8818_neigh1.19716_cross0.87274_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_diagonal_bw23239.31109_skew7.8818_neigh1.21284_cross0.84509_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_diagonal_bw23967.12665_skew7.8818_neigh1.49579_cross0.84608_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_diagonal_bw24008.68622_skew7.8818_neigh1.33536_cross0.84607_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_diagonal_bw24018.14304_skew7.8818_neigh1.41334_cross0.84578_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_diagonal_bw24034.31555_skew7.8818_neigh1.25453_cross0.84477_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_diagonal_bw24106.64346_skew7.8818_neigh1.1756_cross0.84461_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_diagonal_bw24114.69933_skew7.8818_neigh1.57356_cross0.8475_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_diagonal_bw24131.56928_skew7.8818_neigh1.09712_cross0.84415_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_diagonal_bw24182.54428_skew7.8818_neigh1.01304_cross0.84375_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_diagonal_bw24230.56658_skew7.8818_neigh0.93581_cross0.84308_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_diagonal_bw24274.8686_skew7.8818_neigh0.85549_cross0.84272_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_diagonal_bw24811.92989_skew7.8818_neigh1.21348_cross0.84512_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_diagonal_bw24955.57252_skew7.8818_neigh1.2234_cross0.81646_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_diagonal_bw26276.50771_skew7.8818_neigh1.23325_cross0.75987_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_diagonal_bw26290.01987_skew7.8818_neigh1.21444_cross0.84532_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_diagonal_bw27182.66132_skew7.8818_neigh1.23712_cross0.7061_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_diagonal_bw27777.91443_skew7.8818_neigh1.2129_cross0.84488_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_diagonal_bw27910.49869_skew7.8818_neigh1.24252_cross0.65023_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_diagonal_bw28492.91056_skew7.8818_neigh1.24563_cross0.59619_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_diagonal_bw29291.3444_skew7.8818_neigh1.21419_cross0.84494_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_diagonal_bw30735.2641_skew7.8818_neigh1.21293_cross0.84528_n14.mtx","synthetic_185639_185639_15007163_avg80.84057_std117.05079_diagonal_bw24046.68677_skew7.60954_neigh1.21368_cross0.84709_n14.mtx","synthetic_185639_185639_15007417_avg80.84194_std113.68148_diagonal_bw24066.61476_skew6.50848_neigh1.20615_cross0.85925_n14.mtx","synthetic_185639_185639_15007427_avg80.84199_std115.53272_diagonal_bw24030.31379_skew7.06512_neigh1.21132_cross0.85201_n14.mtx","synthetic_187915_187915_9545196_avg50.79529_std19.60651_diagonal_bw5703.99496_skew1.77585_neigh1.22534_cross0.71637_n14.mtx","synthetic_196126_196126_10457351_avg53.31955_std4.75831_diagonal_bw13779.54019_skew3.36988_neigh1.81248_cross0.94112_n14.mtx","synthetic_198653_198653_10092676_avg50.80556_std19.60715_diagonal_bw6040.72041_skew1.77529_neigh1.2246_cross0.71507_n14.mtx","synthetic_205540_205540_14169500_avg68.93792_std101.25978_diagonal_bw23682.46024_skew10.08243_neigh1.12517_cross0.83739_n14.mtx","synthetic_209391_209391_10637567_avg50.80241_std19.60148_diagonal_bw6367.17197_skew1.77546_neigh1.22433_cross0.71571_n14.mtx","synthetic_210654_210654_11231930_avg53.31933_std4.75723_diagonal_bw14684.9664_skew3.3699_neigh1.81334_cross0.94103_n14.mtx","synthetic_217918_217918_10485592_avg48.11714_std4.75335_diagonal_bw15572.84676_skew3.36435_neigh1.77256_cross0.93985_n14.mtx","synthetic_217918_217918_11231492_avg51.53999_std4.76893_diagonal_bw15690.56769_skew3.36554_neigh1.75769_cross0.93992_n14.mtx","synthetic_217918_217918_11582841_avg53.15229_std3.34176_diagonal_bw14752.54676_skew3.38363_neigh1.83064_cross0.94044_n14.mtx","synthetic_217918_217918_11590559_avg53.18771_std3.65776_diagonal_bw14759.15174_skew3.38071_neigh1.82883_cross0.94023_n14.mtx","synthetic_217918_217918_11598747_avg53.22528_std3.97324_diagonal_bw14804.58821_skew3.37762_neigh1.82457_cross0.94082_n14.mtx","synthetic_217918_217918_11607157_avg53.26387_std4.28638_diagonal_bw14926.5797_skew3.37445_neigh1.82058_cross0.94051_n14.mtx","synthetic_217918_217918_11608217_avg53.26874_std4.76281_diagonal_bw14978.94192_skew4.08741_neigh1.8196_cross0.94057_n14.mtx","synthetic_217918_217918_11610489_avg53.27916_std4.76243_diagonal_bw14965.80924_skew3.93626_neigh1.8186_cross0.94068_n14.mtx","synthetic_217918_217918_11612905_avg53.29025_std4.75996_diagonal_bw15022.46198_skew3.76635_neigh1.8171_cross0.94082_n14.mtx","synthetic_217918_217918_11615389_avg53.30165_std4.60014_diagonal_bw15069.01749_skew3.37135_neigh1.81567_cross0.941_n14.mtx","synthetic_217918_217918_11615400_avg53.3017_std4.75902_diagonal_bw15068.40239_skew3.61524_neigh1.81564_cross0.94109_n14.mtx","synthetic_217918_217918_11617960_avg53.31345_std4.75796_diagonal_bw15125.67497_skew3.44541_neigh1.81377_cross0.94083_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_diagonal_bw10615.26121_skew3.36989_neigh1.81293_cross0.941_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_diagonal_bw11723.96417_skew3.36989_neigh1.81304_cross0.94098_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_diagonal_bw12548.56527_skew3.36989_neigh1.50637_cross0.94082_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_diagonal_bw12555.41407_skew3.36989_neigh1.26528_cross0.93926_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_diagonal_bw12561.18815_skew3.36989_neigh1.38503_cross0.94038_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_diagonal_bw12662.63745_skew3.36989_neigh1.81249_cross0.94104_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_diagonal_bw12875.08402_skew3.36989_neigh1.6263_cross0.94024_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_diagonal_bw13192.62211_skew3.36989_neigh1.74925_cross0.94027_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_diagonal_bw13589.06211_skew3.36989_neigh1.8131_cross0.94142_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_diagonal_bw14637.1457_skew3.36989_neigh1.81291_cross0.94097_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_diagonal_bw14639.64249_skew3.36989_neigh1.83771_cross0.66265_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_diagonal_bw14678.79878_skew3.36989_neigh1.83641_cross0.72459_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_diagonal_bw14692.70001_skew3.36989_neigh1.83507_cross0.78486_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_diagonal_bw14839.20089_skew3.36989_neigh1.8307_cross0.84682_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_diagonal_bw15050.52926_skew3.36989_neigh1.82143_cross0.90888_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_diagonal_bw15508.12602_skew3.36989_neigh1.7873_cross0.97163_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_diagonal_bw15626.79599_skew3.36989_neigh1.81205_cross0.94094_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_diagonal_bw15937.73698_skew3.36989_neigh1.74855_cross0.98962_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_diagonal_bw16599.98545_skew3.36989_neigh1.81308_cross0.9411_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_diagonal_bw16602.40328_skew3.36989_neigh1.87566_cross0.93981_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_diagonal_bw17572.60761_skew3.36989_neigh1.81311_cross0.94094_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_diagonal_bw18584.92687_skew3.36989_neigh1.81265_cross0.94092_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_diagonal_bw19575.60228_skew3.36989_neigh1.81285_cross0.94097_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_diagonal_bw5775.00642_skew3.36989_neigh1.93946_cross0.94054_n14.mtx","synthetic_217918_217918_11620586_avg53.3255_std4.75691_diagonal_bw15174.3723_skew3.29438_neigh1.81188_cross0.94105_n14.mtx","synthetic_217918_217918_11622834_avg53.33581_std4.91358_diagonal_bw15211.80314_skew3.36855_neigh1.80941_cross0.94098_n14.mtx","synthetic_217918_217918_11623241_avg53.33768_std4.75547_diagonal_bw15225.00254_skew3.12466_neigh1.80948_cross0.9409_n14.mtx","synthetic_217918_217918_11625780_avg53.34933_std4.75397_diagonal_bw15317.68741_skew2.97381_neigh1.806_cross0.94062_n14.mtx","synthetic_217918_217918_11628184_avg53.36036_std4.75308_diagonal_bw15462.19266_skew2.80432_neigh1.8014_cross0.94016_n14.mtx","synthetic_217918_217918_11628620_avg53.36237_std5.22805_diagonal_bw15467.4387_skew3.36637_neigh1.80121_cross0.93991_n14.mtx","synthetic_217918_217918_11630299_avg53.37007_std4.7531_diagonal_bw15559.9952_skew2.65373_neigh1.79743_cross0.9399_n14.mtx","synthetic_217918_217918_11632309_avg53.37929_std5.54522_diagonal_bw15720.40304_skew3.36499_neigh1.78973_cross0.93981_n14.mtx","synthetic_217918_217918_11634099_avg53.38751_std5.86172_diagonal_bw15948.7068_skew3.36432_neigh1.77763_cross0.93909_n14.mtx","synthetic_217918_217918_11634741_avg53.39045_std6.17863_diagonal_bw16156.60265_skew3.36408_neigh1.76393_cross0.93894_n14.mtx","synthetic_217918_217918_12031582_avg55.21151_std4.74931_diagonal_bw15391.15638_skew3.36503_neigh1.82291_cross0.9415_n14.mtx","synthetic_217918_217918_12797820_avg58.72769_std4.7736_diagonal_bw13915.978_skew3.3591_neigh1.75599_cross0.93955_n14.mtx","synthetic_217918_217918_13561979_avg62.23432_std4.75526_diagonal_bw13183.43978_skew3.37058_neigh1.83856_cross0.94033_n14.mtx","synthetic_217918_217918_14374351_avg65.9622_std4.76726_diagonal_bw14447.91926_skew3.36614_neigh1.78633_cross0.93943_n14.mtx","synthetic_217918_217918_15093236_avg69.26108_std4.76589_diagonal_bw13990.39745_skew3.37475_neigh1.82922_cross0.94054_n14.mtx","synthetic_217918_217918_8143542_avg37.36975_std4.75333_diagonal_bw20866.53868_skew3.36182_neigh1.74733_cross0.9391_n14.mtx","synthetic_217918_217918_8924884_avg40.95524_std4.76049_diagonal_bw22294.09855_skew3.34621_neigh1.71612_cross0.93867_n14.mtx","synthetic_217918_217918_9684687_avg44.44189_std4.75752_diagonal_bw14468.16582_skew3.36525_neigh1.77178_cross0.93964_n14.mtx","synthetic_223413_223413_15400297_avg68.93196_std101.25897_diagonal_bw25758.67821_skew10.0979_neigh1.12654_cross0.83736_n14.mtx","synthetic_225181_225181_12006505_avg53.31935_std4.75652_diagonal_bw15512.77201_skew3.3699_neigh1.81265_cross0.94113_n14.mtx","synthetic_239709_239709_12781051_avg53.31903_std4.75618_diagonal_bw16764.10618_skew3.36992_neigh1.81204_cross0.94086_n14.mtx","synthetic_241286_241286_16633118_avg68.93528_std101.25723_diagonal_bw27818.08052_skew10.09737_neigh1.12625_cross0.83765_n14.mtx","synthetic_254237_254237_13555788_avg53.31949_std4.75562_diagonal_bw17672.99219_skew3.36988_neigh1.81182_cross0.94118_n14.mtx","synthetic_259159_259159_17866974_avg68.94213_std101.25583_diagonal_bw29844.7971_skew10.11077_neigh1.1228_cross0.83747_n14.mtx","synthetic_26684_26684_10204693_avg382.42741_std245.44826_diagonal_bw11448.26195_skew1.57042_neigh1.97454_cross0.92536_n14.mtx","synthetic_268096_268096_12941160_avg48.27062_std95.20699_diagonal_bw26357.31682_skew9.2754_neigh1.08518_cross0.89421_n14.mtx","synthetic_268096_268096_14172610_avg52.86394_std97.03631_diagonal_bw29123.12309_skew9.5554_neigh1.0936_cross0.86612_n14.mtx","synthetic_268096_268096_15403716_avg57.45597_std98.50696_diagonal_bw30125.15948_skew9.75606_neigh1.10358_cross0.85268_n14.mtx","synthetic_268096_268096_16634948_avg62.04848_std99.72897_diagonal_bw30499.11718_skew9.92694_neigh1.11411_cross0.84513_n14.mtx","synthetic_268096_268096_17866195_avg66.64104_std100.78017_diagonal_bw30751.27234_skew10.05925_neigh1.12044_cross0.83953_n14.mtx","synthetic_268096_268096_18479164_avg68.92741_std73.74951_diagonal_bw31377.65125_skew10.4033_neigh1.1237_cross0.84107_n14.mtx","synthetic_268096_268096_18479645_avg68.92921_std80.36452_diagonal_bw31191.46774_skew10.403_neigh1.12752_cross0.82739_n14.mtx","synthetic_268096_268096_18480123_avg68.93099_std86.42344_diagonal_bw31125.07403_skew10.33017_neigh1.12545_cross0.82944_n14.mtx","synthetic_268096_268096_18480967_avg68.93414_std92.41882_diagonal_bw31046.67467_skew10.25712_neigh1.12466_cross0.8325_n14.mtx","synthetic_268096_268096_18481691_avg68.93684_std98.3339_diagonal_bw30926.5338_skew10.16964_neigh1.12377_cross0.83538_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_diagonal_bw21923.44482_skew10.11157_neigh1.12703_cross0.83756_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_diagonal_bw23935.8137_skew10.11157_neigh1.12612_cross0.8371_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_diagonal_bw25895.422_skew10.11157_neigh1.12483_cross0.83763_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_diagonal_bw27899.12455_skew10.11157_neigh1.12567_cross0.83764_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_diagonal_bw29902.52638_skew10.11157_neigh1.12497_cross0.83752_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_diagonal_bw31924.98227_skew10.11157_neigh1.12445_cross0.83711_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_diagonal_bw33817.07914_skew10.11157_neigh1.12399_cross0.83739_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_diagonal_bw35779.99142_skew10.11157_neigh1.12322_cross0.83708_n14.mtx","synthetic_268096_268096_18482362_avg68.93934_std104.15593_diagonal_bw30765.9396_skew10.0677_neigh1.12333_cross0.83908_n14.mtx","synthetic_268096_268096_18483075_avg68.942_std109.86851_diagonal_bw30647.70205_skew9.95123_neigh1.12162_cross0.84333_n14.mtx","synthetic_268096_268096_18483963_avg68.94531_std115.46168_diagonal_bw30446.51037_skew9.82017_neigh1.11909_cross0.84887_n14.mtx","synthetic_268096_268096_18484698_avg68.94806_std120.91939_diagonal_bw30186.45871_skew9.68921_neigh1.11754_cross0.85523_n14.mtx","synthetic_268096_268096_18485774_avg68.95207_std126.22922_diagonal_bw29770.97031_skew9.55806_neigh1.11488_cross0.86335_n14.mtx","synthetic_268096_268096_19097540_avg71.23396_std101.70964_diagonal_bw30933.99411_skew10.16041_neigh1.12541_cross0.83599_n14.mtx","synthetic_268096_268096_20328893_avg75.82692_std102.55303_diagonal_bw31081.7585_skew10.2493_neigh1.1336_cross0.83222_n14.mtx","synthetic_268096_268096_21560408_avg80.42048_std103.33442_diagonal_bw31148.48253_skew10.30309_neigh1.13451_cross0.82948_n14.mtx","synthetic_268096_268096_22792018_avg85.01439_std104.0723_diagonal_bw31199.25223_skew10.36278_neigh1.13885_cross0.8282_n14.mtx","synthetic_268096_268096_24023179_avg89.60663_std104.77794_diagonal_bw31239.70547_skew10.40541_neigh1.14366_cross0.82667_n14.mtx","synthetic_268765_268765_14330411_avg53.31948_std4.75541_diagonal_bw18731.37211_skew3.36988_neigh1.81156_cross0.94066_n14.mtx","synthetic_277032_277032_19096094_avg68.931_std101.25944_diagonal_bw31953.72601_skew10.12707_neigh1.12352_cross0.83736_n14.mtx","synthetic_283293_283293_15104954_avg53.31919_std4.75508_diagonal_bw19749.01419_skew3.36991_neigh1.81186_cross0.9407_n14.mtx","synthetic_29225_29225_11176419_avg382.42666_std245.44724_diagonal_bw12425.97666_skew1.57043_neigh1.97381_cross0.92588_n14.mtx","synthetic_294905_294905_20328825_avg68.93347_std101.25836_diagonal_bw33931.21622_skew10.12667_neigh1.12413_cross0.83784_n14.mtx","synthetic_2998_2998_6123474_avg2042.51968_std621.91091_diagonal_bw2392.08572_skew0.46779_neigh1.915_cross0.85498_n14.mtx","synthetic_312778_312778_21563736_avg68.94262_std101.25558_diagonal_bw36065.10736_skew10.1397_neigh1.12116_cross0.83732_n14.mtx","synthetic_31766_31766_12148131_avg382.42558_std245.44713_diagonal_bw13817.64257_skew1.57043_neigh1.97384_cross0.91941_n14.mtx","synthetic_3284_3284_6742726_avg2053.20524_std666.12092_diagonal_bw2537.96711_skew0.59945_neigh1.8895_cross0.83825_n14.mtx","synthetic_330651_330651_22801392_avg68.95909_std101.25183_diagonal_bw38055.27574_skew10.13704_neigh1.1241_cross0.83714_n14.mtx","synthetic_34308_34308_13120227_avg382.42471_std245.44609_diagonal_bw14650.04739_skew1.57044_neigh1.97406_cross0.9232_n14.mtx","synthetic_348524_348524_24034194_avg68.95994_std101.25128_diagonal_bw40157.18736_skew10.1369_neigh1.12259_cross0.83707_n14.mtx","synthetic_3569_3569_7446679_avg2086.48893_std694.14116_diagonal_bw2698.16419_skew0.71053_neigh1.86781_cross0.82629_n14.mtx","synthetic_36849_36849_14091918_avg382.42335_std245.44508_diagonal_bw15781.99004_skew1.57045_neigh1.97399_cross0.92492_n14.mtx","synthetic_38120_38120_10204584_avg267.69633_std171.81206_diagonal_bw16895.89722_skew1.57008_neigh1.97152_cross0.92219_n14.mtx","synthetic_38120_38120_11176434_avg293.19082_std188.17435_diagonal_bw17618.12893_skew1.5717_neigh1.97171_cross0.9225_n14.mtx","synthetic_38120_38120_12148318_avg318.6862_std204.53731_diagonal_bw18097.15551_skew1.56993_neigh1.97265_cross0.92312_n14.mtx","synthetic_38120_38120_13120182_avg344.18106_std220.90077_diagonal_bw18722.69885_skew1.57132_neigh1.97302_cross0.92199_n14.mtx","synthetic_38120_38120_14004492_avg367.37912_std206.50003_diagonal_bw16517.99428_skew1.32185_neigh1.97296_cross0.92309_n14.mtx","synthetic_38120_38120_14092040_avg369.67576_std237.2634_diagonal_bw15900.27104_skew1.56982_neigh1.97398_cross0.92174_n14.mtx","synthetic_38120_38120_14186695_avg372.15884_std217.74813_diagonal_bw16610.97285_skew1.39145_neigh1.97336_cross0.92298_n14.mtx","synthetic_38120_38120_14353565_avg376.53633_std228.90355_diagonal_bw16362.02206_skew1.46191_neigh1.97392_cross0.9229_n14.mtx","synthetic_38120_38120_14506403_avg380.54572_std239.95763_diagonal_bw16221.47907_skew1.5332_neigh1.97371_cross0.92545_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_diagonal_bw11840.96207_skew1.57045_neigh1.97389_cross0.92172_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_diagonal_bw13051.93368_skew1.57045_neigh1.97364_cross0.9215_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_diagonal_bw14054.5128_skew1.57045_neigh1.97418_cross0.92287_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_diagonal_bw14901.13825_skew1.57045_neigh1.9738_cross0.92245_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_diagonal_bw15702.30456_skew1.57045_neigh1.97328_cross0.9903_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_diagonal_bw16005.2249_skew1.57045_neigh1.97376_cross0.92394_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_diagonal_bw16107.56653_skew1.57045_neigh1.90891_cross0.91992_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_diagonal_bw16165.12815_skew1.57045_neigh1.3922_cross0.92174_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_diagonal_bw16173.10614_skew1.57045_neigh1.64922_cross0.92071_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_diagonal_bw16181.2495_skew1.57045_neigh1.51874_cross0.92011_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_diagonal_bw16189.91522_skew1.57045_neigh1.77707_cross0.91956_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_diagonal_bw16251.80551_skew1.57045_neigh1.97384_cross0.89306_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_diagonal_bw16403.35273_skew1.57045_neigh1.97397_cross0.72233_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_diagonal_bw16412.02442_skew1.57045_neigh1.97389_cross0.66503_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_diagonal_bw16450.53809_skew1.57045_neigh1.9739_cross0.77998_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_diagonal_bw16454.6271_skew1.57045_neigh1.97336_cross0.95155_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_diagonal_bw16504.44638_skew1.57045_neigh1.97349_cross0.83444_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_diagonal_bw16601.00073_skew1.57045_neigh1.97376_cross0.92272_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_diagonal_bw16971.93463_skew1.57045_neigh1.9734_cross0.92126_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_diagonal_bw17802.86905_skew1.57045_neigh1.97345_cross0.9247_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_diagonal_bw18943.77943_skew1.57045_neigh1.97338_cross0.92328_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_diagonal_bw20020.58657_skew1.57045_neigh1.97348_cross0.92229_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_diagonal_bw21202.83549_skew1.57045_neigh1.97368_cross0.92139_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_diagonal_bw5783.47169_skew1.57045_neigh1.99394_cross0.9293_n14.mtx","synthetic_38120_38120_14646455_avg384.2197_std250.9036_diagonal_bw16369.04491_skew1.60788_neigh1.97362_cross0.92456_n14.mtx","synthetic_38120_38120_14774683_avg387.5835_std261.73289_diagonal_bw16271.8489_skew1.68071_neigh1.97337_cross0.92347_n14.mtx","synthetic_38120_38120_14892124_avg390.66432_std272.44037_diagonal_bw16642.15774_skew1.75428_neigh1.9735_cross0.92246_n14.mtx","synthetic_38120_38120_15009192_avg393.73536_std249.23315_diagonal_bw17334.85677_skew1.65915_neigh1.95771_cross0.90213_n14.mtx","synthetic_38120_38120_15029193_avg394.26005_std283.24138_diagonal_bw17593.26878_skew1.83569_neigh1.95985_cross0.92052_n14.mtx","synthetic_38120_38120_15063906_avg395.17067_std253.62681_diagonal_bw16677.30215_skew1.57104_neigh1.97408_cross0.92294_n14.mtx","synthetic_38120_38120_15246247_avg399.95401_std294.48908_diagonal_bw17717.6793_skew1.94534_neigh1.95613_cross0.91225_n14.mtx","synthetic_38120_38120_15509350_avg406.85598_std254.10081_diagonal_bw16470.84627_skew1.83638_neigh1.96076_cross0.87593_n14.mtx","synthetic_38120_38120_15831412_avg415.30462_std256.81933_diagonal_bw15755.80622_skew2.01225_neigh1.96337_cross0.85513_n14.mtx","synthetic_38120_38120_16013257_avg420.07495_std256.51996_diagonal_bw15554.95262_skew2.18991_neigh1.96477_cross0.83814_n14.mtx","synthetic_38120_38120_16417883_avg430.68948_std273.30431_diagonal_bw17866.36671_skew1.63763_neigh1.95894_cross0.90329_n14.mtx","synthetic_38120_38120_17911818_avg469.8798_std294.64729_diagonal_bw15437.19732_skew1.77305_neigh1.96314_cross0.88403_n14.mtx","synthetic_38120_38120_17943763_avg470.71781_std404.16901_diagonal_bw11533.34024_skew3.74169_neigh1.97374_cross0.57584_n14.mtx","synthetic_38120_38120_19284934_avg505.90068_std314.8817_diagonal_bw15240.20312_skew1.89583_neigh1.96535_cross0.86661_n14.mtx","synthetic_38120_38120_20568467_avg539.57154_std333.79776_diagonal_bw14935.54641_skew2.00609_neigh1.96694_cross0.85709_n14.mtx","synthetic_3855_3855_8084357_avg2097.10947_std723.99546_diagonal_bw2875.22931_skew0.83824_neigh1.84384_cross0.81071_n14.mtx","synthetic_39390_39390_15063634_avg382.4228_std245.44522_diagonal_bw17150.32615_skew1.57045_neigh1.97365_cross0.9222_n14.mtx","synthetic_4141_4141_8723864_avg2106.70466_std753.89371_diagonal_bw3055.31055_skew0.96563_neigh1.82243_cross0.79514_n14.mtx","synthetic_41932_41932_16035737_avg382.42242_std245.44481_diagonal_bw18021.62279_skew1.57046_neigh1.97387_cross0.92405_n14.mtx","synthetic_4284_4284_10161733_avg2372.01984_std415.11429_diagonal_bw3157.98039_skew0.80606_neigh1.8259_cross0.87786_n14.mtx","synthetic_4284_4284_10296320_avg2403.43604_std664.88425_diagonal_bw3183.64402_skew0.78245_neigh1.84578_cross0.84547_n14.mtx","synthetic_4284_4284_11128593_avg2597.71078_std612.25995_diagonal_bw3227.32843_skew0.64914_neigh1.86927_cross0.87428_n14.mtx","synthetic_4284_4284_11955257_avg2790.67624_std565.77456_diagonal_bw3299.09944_skew0.53511_neigh1.89073_cross0.89611_n14.mtx","synthetic_4284_4284_12776299_avg2982.32937_std524.4261_diagonal_bw3366.44678_skew0.43646_neigh1.91312_cross0.91453_n14.mtx","synthetic_4284_4284_5176835_avg1208.41153_std875.52905_diagonal_bw3144.46919_skew2.54515_neigh1.74013_cross0.56551_n14.mtx","synthetic_4284_4284_6045664_avg1411.21942_std813.05724_diagonal_bw3116.32236_skew2.03567_neigh1.73991_cross0.63076_n14.mtx","synthetic_4284_4284_6908449_avg1612.61648_std777.77715_diagonal_bw3104.0042_skew1.65655_neigh1.75254_cross0.68691_n14.mtx","synthetic_4284_4284_7606767_avg1775.62255_std826.63023_diagonal_bw3138.01167_skew1.41267_neigh1.77542_cross0.71893_n14.mtx","synthetic_4284_4284_7764407_avg1812.41993_std766.39496_diagonal_bw3107.61204_skew1.36369_neigh1.77486_cross0.73027_n14.mtx","synthetic_4284_4284_7948514_avg1855.39542_std808.46009_diagonal_bw3131.42437_skew1.30894_neigh1.78337_cross0.73476_n14.mtx","synthetic_4284_4284_8278029_avg1932.31303_std796.54977_diagonal_bw3133.04062_skew1.21703_neigh1.79149_cross0.75251_n14.mtx","synthetic_4284_4284_8399652_avg1960.70308_std840.91619_diagonal_bw3170.12582_skew1.18493_neigh1.79818_cross0.75117_n14.mtx","synthetic_4284_4284_8563923_avg1999.04832_std817.99013_diagonal_bw3151.49883_skew1.14302_neigh1.802_cross0.75986_n14.mtx","synthetic_4284_4284_8591948_avg2005.5901_std788.67453_diagonal_bw3136.71662_skew1.13603_neigh1.80156_cross0.76701_n14.mtx","synthetic_4284_4284_8613657_avg2010.65756_std774.7613_diagonal_bw3127.03525_skew1.13065_neigh1.8022_cross0.76595_n14.mtx","synthetic_4284_4284_8716487_avg2034.66083_std804.2751_diagonal_bw3151.36345_skew1.10551_neigh1.80589_cross0.76962_n14.mtx","synthetic_4284_4284_8853517_avg2066.64729_std794.27725_diagonal_bw3141.82166_skew1.07292_neigh1.80997_cross0.77415_n14.mtx","synthetic_4284_4284_8891530_avg2075.52054_std785.22497_diagonal_bw3124.03922_skew1.06406_neigh1.81154_cross0.77892_n14.mtx","synthetic_4284_4284_8977954_avg2095.69421_std780.20568_diagonal_bw3133.23389_skew1.04419_neigh1.81378_cross0.78051_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_diagonal_bw2587.0796_skew1.03117_neigh1.8919_cross0.84164_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_diagonal_bw2710.43301_skew1.03117_neigh1.87148_cross0.83111_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_diagonal_bw2823.22969_skew1.03117_neigh1.85445_cross0.8163_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_diagonal_bw2954.99697_skew1.03117_neigh1.83634_cross0.8091_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_diagonal_bw3067.13329_skew1.03117_neigh1.82242_cross0.79404_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_diagonal_bw3102.39846_skew1.03117_neigh1.81661_cross0.79393_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_diagonal_bw3107.22993_skew1.03117_neigh1.8162_cross0.79153_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_diagonal_bw3110.80742_skew1.03117_neigh1.81584_cross0.79256_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_diagonal_bw3114.21218_skew1.03117_neigh1.99853_cross0.70901_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_diagonal_bw3121.18417_skew1.03117_neigh1.81524_cross0.79023_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_diagonal_bw3126.92857_skew1.03117_neigh1.87269_cross0.76602_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_diagonal_bw3127.45448_skew1.03117_neigh1.91846_cross0.74602_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_diagonal_bw3130.74066_skew1.03117_neigh1.8144_cross0.78625_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_diagonal_bw3132.41083_skew1.03117_neigh1.8143_cross0.78794_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_diagonal_bw3133.53035_skew1.03117_neigh1.83285_cross0.77848_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_diagonal_bw3135.26284_skew1.03117_neigh1.79588_cross0.7941_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_diagonal_bw3135.64356_skew1.03117_neigh1.76094_cross0.80472_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_diagonal_bw3142.22782_skew1.03117_neigh1.81328_cross0.78516_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_diagonal_bw3144.31513_skew1.03117_neigh1.72858_cross0.81697_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_diagonal_bw3149.68371_skew1.03117_neigh1.69844_cross0.82642_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_diagonal_bw3156.4239_skew1.03117_neigh1.81211_cross0.78361_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_diagonal_bw3156.47082_skew1.03117_neigh1.96828_cross0.71812_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_diagonal_bw3162.04692_skew1.03117_neigh1.81086_cross0.78323_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_diagonal_bw3171.51494_skew1.03117_neigh1.66807_cross0.8333_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_diagonal_bw3178.75864_skew1.03117_neigh1.80961_cross0.78012_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_diagonal_bw3198.155_skew1.03117_neigh1.80553_cross0.78061_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_diagonal_bw3209.27381_skew1.03117_neigh1.80497_cross0.77778_n14.mtx","synthetic_4284_4284_9091186_avg2122.12558_std733.44441_diagonal_bw3133.95915_skew1.01873_neigh1.81407_cross0.79058_n14.mtx","synthetic_4284_4284_9176988_avg2142.15406_std713.01815_diagonal_bw3139.19981_skew0.99986_neigh1.8134_cross0.80089_n14.mtx","synthetic_4284_4284_9196527_avg2146.71499_std692.64997_diagonal_bw3137.96078_skew0.99561_neigh1.81239_cross0.80355_n14.mtx","synthetic_4284_4284_9292087_avg2169.02124_std655.40775_diagonal_bw3136.66106_skew0.97508_neigh1.81305_cross0.81195_n14.mtx","synthetic_4284_4284_9379803_avg2189.4965_std621.87428_diagonal_bw3140.22502_skew0.95661_neigh1.81248_cross0.82177_n14.mtx","synthetic_4284_4284_9448827_avg2205.60854_std630.77463_diagonal_bw3151.63002_skew0.94232_neigh1.81415_cross0.82352_n14.mtx","synthetic_4284_4284_9456745_avg2207.45682_std723.4564_diagonal_bw3150.4225_skew0.94069_neigh1.8231_cross0.80811_n14.mtx","synthetic_4284_4284_9460554_avg2208.34594_std591.75846_diagonal_bw3131.92063_skew0.93991_neigh1.81387_cross0.82831_n14.mtx","synthetic_4284_4284_9703417_avg2265.03665_std553.19719_diagonal_bw3154.62862_skew0.89136_neigh1.8169_cross0.84303_n14.mtx","synthetic_4284_4284_9941769_avg2320.67437_std481.87027_diagonal_bw3156.55415_skew0.84602_neigh1.82089_cross0.86146_n14.mtx","synthetic_4426_4426_9354099_avg2113.44306_std776.70231_diagonal_bw3225.23949_skew1.09421_neigh1.80628_cross0.77807_n14.mtx","synthetic_44473_44473_17007427_avg382.4214_std245.44406_diagonal_bw18588.49515_skew1.57046_neigh1.9742_cross0.92197_n14.mtx","synthetic_44686_44686_9904566_avg221.64808_std96.54092_diagonal_bw28996.12507_skew15.40889_neigh1.70385_cross0.85407_n14.mtx","synthetic_46524_46524_7261288_avg156.07618_std357.60767_diagonal_bw23117.62254_skew297.08521_neigh1.84312_cross0.94487_n14.mtx","synthetic_47014_47014_17979151_avg382.42121_std245.44364_diagonal_bw20028.40735_skew1.57046_neigh1.97384_cross0.92279_n14.mtx","synthetic_4712_4712_10000031_avg2122.24767_std807.79622_diagonal_bw3425.88222_skew1.22029_neigh1.78887_cross0.75824_n14.mtx","synthetic_48942_48942_10847397_avg221.6378_std96.4853_diagonal_bw31784.2128_skew15.40966_neigh1.7043_cross0.85189_n14.mtx","synthetic_49556_49556_18951229_avg382.42047_std245.44344_diagonal_bw21059.9188_skew1.57047_neigh1.97406_cross0.92093_n14.mtx","synthetic_4998_4998_10667079_avg2134.26951_std834.36333_diagonal_bw3626.9994_skew1.34178_neigh1.7719_cross0.74002_n14.mtx","synthetic_50954_50954_7955612_avg156.13322_std368.01601_diagonal_bw25406.9614_skew325.34951_neigh1.84494_cross0.94426_n14.mtx","synthetic_5283_5283_11298374_avg2138.62843_std877.77445_diagonal_bw3805.81677_skew1.47027_neigh1.7624_cross0.71965_n14.mtx","synthetic_53198_53198_11791218_avg221.64777_std96.43769_diagonal_bw34602.16726_skew15.40892_neigh1.70332_cross0.85286_n14.mtx","synthetic_55385_55385_8650205_avg156.18317_std377.56925_diagonal_bw27666.03716_skew353.61567_neigh1.84432_cross0.94383_n14.mtx","synthetic_5569_5569_11949345_avg2145.68953_std911.81863_diagonal_bw4008.83264_skew1.59544_neigh1.75126_cross0.70162_n14.mtx","synthetic_57454_57454_12734908_avg221.65398_std96.39664_diagonal_bw37281.76299_skew15.40846_neigh1.70414_cross0.85028_n14.mtx","synthetic_59816_59816_9344868_avg156.2269_std386.43932_diagonal_bw29761.58428_skew381.87902_neigh1.84288_cross0.94411_n14.mtx","synthetic_61710_61710_13678136_avg221.65186_std96.36424_diagonal_bw39991.18778_skew15.40862_neigh1.70436_cross0.85228_n14.mtx","synthetic_63838_63838_10847423_avg169.9211_std96.15794_diagonal_bw40401.27927_skew15.38408_neigh1.69606_cross0.84555_n14.mtx","synthetic_63838_63838_11790856_avg184.69965_std96.20654_diagonal_bw40776.4429_skew15.38877_neigh1.69867_cross0.84918_n14.mtx","synthetic_63838_63838_12734460_avg199.48087_std96.25772_diagonal_bw41109.50902_skew15.39756_neigh1.70139_cross0.84605_n14.mtx","synthetic_63838_63838_13677970_avg214.26063_std96.31751_diagonal_bw41355.45747_skew15.40525_neigh1.70246_cross0.84998_n14.mtx","synthetic_63838_63838_14148710_avg221.63461_std96.12924_diagonal_bw40820.71309_skew11.06039_neigh1.70496_cross0.84517_n14.mtx","synthetic_63838_63838_14148875_avg221.63719_std96.16987_diagonal_bw40978.10773_skew12.02579_neigh1.7049_cross0.84703_n14.mtx","synthetic_63838_63838_14148974_avg221.63874_std125.01008_diagonal_bw40403.16755_skew15.38251_neigh1.70576_cross0.84119_n14.mtx","synthetic_63838_63838_14149076_avg221.64034_std96.21451_diagonal_bw41078.1748_skew12.99114_neigh1.70372_cross0.8494_n14.mtx","synthetic_63838_63838_14149188_avg221.64209_std118.63327_diagonal_bw40496.69658_skew15.38678_neigh1.70525_cross0.84551_n14.mtx","synthetic_63838_63838_14149208_avg221.64241_std96.26387_diagonal_bw41309.92768_skew13.96104_neigh1.7036_cross0.84889_n14.mtx","synthetic_63838_63838_14149296_avg221.64379_std112.26231_diagonal_bw40884.59971_skew15.39568_neigh1.70481_cross0.84573_n14.mtx","synthetic_63838_63838_14149329_avg221.6443_std105.89153_diagonal_bw41036.28118_skew15.40015_neigh1.70463_cross0.8482_n14.mtx","synthetic_63838_63838_14149335_avg221.6444_std96.31796_diagonal_bw41384.37215_skew14.92641_neigh1.70468_cross0.85201_n14.mtx","synthetic_63838_63838_14149338_avg221.64444_std99.52693_diagonal_bw41338.26943_skew15.40465_neigh1.70463_cross0.85088_n14.mtx","synthetic_63838_63838_14149538_avg221.64758_std93.16614_diagonal_bw41616.56296_skew15.40893_neigh1.70332_cross0.8503_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_diagonal_bw31209.32828_skew15.40883_neigh1.70379_cross0.85318_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_diagonal_bw31263.79578_skew15.40883_neigh1.97907_cross0.90114_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_diagonal_bw33310.08077_skew15.40883_neigh1.62891_cross0.97834_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_diagonal_bw33741.33778_skew15.40883_neigh1.70407_cross0.84949_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_diagonal_bw36088.99159_skew15.40883_neigh1.70329_cross0.85235_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_diagonal_bw38238.34086_skew15.40883_neigh1.70401_cross0.8519_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_diagonal_bw38844.09753_skew15.40883_neigh1.68246_cross0.9336_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_diagonal_bw40356.3582_skew15.40883_neigh1.70389_cross0.85225_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_diagonal_bw40916.26257_skew15.40883_neigh1.69941_cross0.87831_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_diagonal_bw41070.66258_skew15.40883_neigh1.19535_cross0.84968_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_diagonal_bw41074.56236_skew15.40883_neigh1.30766_cross0.85115_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_diagonal_bw41141.45558_skew15.40883_neigh1.42075_cross0.85031_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_diagonal_bw41227.36727_skew15.40883_neigh1.53352_cross0.85087_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_diagonal_bw41364.42425_skew15.40883_neigh1.64744_cross0.85051_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_diagonal_bw41755.58307_skew15.40883_neigh1.76114_cross0.85034_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_diagonal_bw41906.82374_skew15.40883_neigh1.70747_cross0.82402_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_diagonal_bw42415.19562_skew15.40883_neigh1.71119_cross0.76651_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_diagonal_bw42495.96779_skew15.40883_neigh1.7039_cross0.85098_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_diagonal_bw42773.9898_skew15.40883_neigh1.8751_cross0.85514_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_diagonal_bw42815.11517_skew15.40883_neigh1.71375_cross0.7124_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_diagonal_bw43078.55498_skew15.40883_neigh1.71506_cross0.65509_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_diagonal_bw43268.40659_skew15.40883_neigh1.71684_cross0.60215_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_diagonal_bw44501.11614_skew15.40883_neigh1.70412_cross0.85016_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_diagonal_bw46019.35751_skew15.40883_neigh1.70471_cross0.84937_n14.mtx","synthetic_63838_63838_14149682_avg221.64983_std96.37657_diagonal_bw41560.95078_skew15.887_neigh1.70358_cross0.84917_n14.mtx","synthetic_63838_63838_14149757_avg221.65101_std96.43499_diagonal_bw41589.99978_skew16.85239_neigh1.70335_cross0.8529_n14.mtx","synthetic_63838_63838_14149836_avg221.65224_std86.81097_diagonal_bw41852.38988_skew15.4131_neigh1.70445_cross0.8526_n14.mtx","synthetic_63838_63838_14149837_avg221.65226_std96.50454_diagonal_bw41795.62602_skew17.81776_neigh1.7041_cross0.85102_n14.mtx","synthetic_63838_63838_14149882_avg221.65297_std80.46101_diagonal_bw42155.13125_skew15.41756_neigh1.70314_cross0.85556_n14.mtx","synthetic_63838_63838_14149914_avg221.65347_std74.11814_diagonal_bw42395.55904_skew15.42203_neigh1.70343_cross0.855_n14.mtx","synthetic_63838_63838_14150121_avg221.65671_std67.78709_diagonal_bw42629.81854_skew15.4263_neigh1.70372_cross0.85703_n14.mtx","synthetic_63838_63838_14150132_avg221.65688_std96.57304_diagonal_bw41766.83688_skew18.78283_neigh1.70227_cross0.85422_n14.mtx","synthetic_63838_63838_14150259_avg221.65887_std96.64741_diagonal_bw41910.74145_skew19.7481_neigh1.70367_cross0.85371_n14.mtx","synthetic_63838_63838_14621335_avg229.03811_std96.37884_diagonal_bw41539.90302_skew15.40775_neigh1.70411_cross0.85184_n14.mtx","synthetic_63838_63838_15564722_avg243.81594_std96.44368_diagonal_bw41752.45562_skew15.41402_neigh1.70623_cross0.85371_n14.mtx","synthetic_63838_63838_16508065_avg258.59308_std96.5168_diagonal_bw41932.57262_skew15.41575_neigh1.70761_cross0.85252_n14.mtx","synthetic_63838_63838_17451479_avg273.37133_std96.59121_diagonal_bw42040.04983_skew15.41723_neigh1.70991_cross0.85482_n14.mtx","synthetic_63838_63838_18395041_avg288.1519_std96.67009_diagonal_bw42202.0742_skew15.42189_neigh1.71077_cross0.85457_n14.mtx","synthetic_63838_63838_9903973_avg155.14228_std96.11613_diagonal_bw40026.20256_skew15.37207_neigh1.69445_cross0.84169_n14.mtx","synthetic_64247_64247_10039260_avg156.26037_std394.7431_diagonal_bw31668.91797_skew410.15351_neigh1.84448_cross0.94465_n14.mtx","synthetic_65965_65965_14620660_avg221.64269_std96.33127_diagonal_bw42883.51398_skew15.40929_neigh1.70416_cross0.8507_n14.mtx","synthetic_66463_66463_10040261_avg151.06542_std395.7133_diagonal_bw32962.39217_skew424.82214_neigh1.844_cross0.94181_n14.mtx","synthetic_66463_66463_10375238_avg156.10547_std374.16577_diagonal_bw32112.74231_skew297.97094_neigh1.84818_cross0.93876_n14.mtx","synthetic_66463_66463_10377678_avg156.14218_std378.88433_diagonal_bw32368.34001_skew326.18897_neigh1.84976_cross0.93964_n14.mtx","synthetic_66463_66463_10378609_avg156.15619_std376.2538_diagonal_bw33885.40087_skew424.61873_neigh1.84267_cross0.94663_n14.mtx","synthetic_66463_66463_10380253_avg156.18093_std381.55088_diagonal_bw33648.55795_skew424.55132_neigh1.84118_cross0.94627_n14.mtx","synthetic_66463_66463_10380322_avg156.18197_std384.04911_diagonal_bw32719.54181_skew354.38034_neigh1.847_cross0.94116_n14.mtx","synthetic_66463_66463_10381996_avg156.20715_std386.69205_diagonal_bw33367.73913_skew424.47988_neigh1.84_cross0.94697_n14.mtx","synthetic_66463_66463_10382686_avg156.21753_std389.6586_diagonal_bw32926.14501_skew382.5741_neigh1.84812_cross0.94145_n14.mtx","synthetic_66463_66463_10383864_avg156.23526_std391.65048_diagonal_bw33368.29388_skew424.40333_neigh1.84441_cross0.94475_n14.mtx","synthetic_66463_66463_10384998_avg156.25232_std492.58539_diagonal_bw31613.53189_skew424.35688_neigh1.85286_cross0.93515_n14.mtx","synthetic_66463_66463_10385329_avg156.2573_std395.72064_diagonal_bw32889.72072_skew410.73756_neigh1.8454_cross0.94344_n14.mtx","synthetic_66463_66463_10385409_avg156.2585_std471.2363_diagonal_bw32083.54764_skew424.34005_neigh1.85336_cross0.93664_n14.mtx","synthetic_66463_66463_10385572_avg156.26096_std396.41268_diagonal_bw33072.52193_skew424.33337_neigh1.84327_cross0.94513_n14.mtx","synthetic_66463_66463_10385645_avg156.26206_std450.123_diagonal_bw32496.96564_skew424.33038_neigh1.85048_cross0.93744_n14.mtx","synthetic_66463_66463_10386025_avg156.26777_std429.29508_diagonal_bw32700.40045_skew424.31482_neigh1.84744_cross0.94073_n14.mtx","synthetic_66463_66463_10386363_avg156.27286_std408.80831_diagonal_bw32967.75383_skew424.30098_neigh1.84617_cross0.94247_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_diagonal_bw24478.41774_skew424.29959_neigh1.8439_cross0.94323_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_diagonal_bw26438.0019_skew424.29959_neigh1.84659_cross0.94266_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_diagonal_bw28604.35611_skew424.29959_neigh1.8441_cross0.94365_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_diagonal_bw29170.51087_skew424.29959_neigh1.30538_cross0.94141_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_diagonal_bw29263.20915_skew424.29959_neigh1.42161_cross0.9415_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_diagonal_bw29455.90226_skew424.29959_neigh1.54434_cross0.94126_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_diagonal_bw29562.86522_skew424.29959_neigh1.66172_cross0.94313_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_diagonal_bw30146.88541_skew424.29959_neigh1.76709_cross0.97938_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_diagonal_bw30341.85309_skew424.29959_neigh1.8421_cross0.94456_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_diagonal_bw30743.21286_skew424.29959_neigh1.7836_cross0.94242_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_diagonal_bw30957.89412_skew424.29959_neigh1.78881_cross0.97358_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_diagonal_bw32012.39481_skew424.29959_neigh1.84485_cross0.94356_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_diagonal_bw33940.03652_skew424.29959_neigh1.8657_cross0.91479_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_diagonal_bw34285.62295_skew424.29959_neigh1.84502_cross0.94387_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_diagonal_bw34484.39386_skew424.29959_neigh1.88647_cross0.85463_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_diagonal_bw34739.67447_skew424.29959_neigh1.89449_cross0.79398_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_diagonal_bw34819.1538_skew424.29959_neigh1.89891_cross0.73344_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_diagonal_bw34936.4192_skew424.29959_neigh1.90192_cross0.6744_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_diagonal_bw35702.1289_skew424.29959_neigh1.84209_cross0.94568_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_diagonal_bw36237.71714_skew424.29959_neigh1.94624_cross0.95035_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_diagonal_bw37525.80485_skew424.29959_neigh1.84482_cross0.94433_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_diagonal_bw39170.09238_skew424.29959_neigh1.84527_cross0.94403_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_diagonal_bw40665.0048_skew424.29959_neigh1.84538_cross0.94433_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_diagonal_bw41811.7269_skew424.29959_neigh1.92884_cross0.94905_n14.mtx","synthetic_66463_66463_10386663_avg156.27737_std388.73672_diagonal_bw33293.45057_skew424.2887_neigh1.84329_cross0.94546_n14.mtx","synthetic_66463_66463_10387014_avg156.28265_std369.1724_diagonal_bw33710.6116_skew424.27433_neigh1.84189_cross0.94604_n14.mtx","synthetic_66463_66463_10387408_avg156.28858_std350.24164_diagonal_bw33963.66553_skew424.25819_neigh1.84411_cross0.94677_n14.mtx","synthetic_66463_66463_10387855_avg156.29531_std332.11193_diagonal_bw34351.45063_skew424.23989_neigh1.8405_cross0.94886_n14.mtx","synthetic_66463_66463_10388533_avg156.30551_std315.01147_diagonal_bw34714.95685_skew424.21214_neigh1.83978_cross0.94968_n14.mtx","synthetic_66463_66463_10730769_avg161.45478_std396.39605_diagonal_bw33389.55323_skew410.65087_neigh1.84718_cross0.94432_n14.mtx","synthetic_66463_66463_11418942_avg171.80901_std391.59435_diagonal_bw33904.10221_skew385.84235_neigh1.8474_cross0.94666_n14.mtx","synthetic_66463_66463_12107419_avg182.16781_std386.59381_diagonal_bw33078.68366_skew363.84492_neigh1.85114_cross0.94649_n14.mtx","synthetic_66463_66463_12795815_avg192.52539_std381.413_diagonal_bw33483.01449_skew344.2168_neigh1.85524_cross0.94648_n14.mtx","synthetic_66463_66463_13484307_avg202.88442_std376.07335_diagonal_bw32893.08663_skew326.59046_neigh1.85894_cross0.94691_n14.mtx","synthetic_66463_66463_7269367_avg109.37464_std374.11902_diagonal_bw34146.31739_skew424.85738_neigh1.83213_cross0.93119_n14.mtx","synthetic_66463_66463_7962072_avg119.79706_std378.84315_diagonal_bw35198.97952_skew424.84518_neigh1.835_cross0.93592_n14.mtx","synthetic_66463_66463_8654839_avg130.22041_std384.01598_diagonal_bw33135.07615_skew424.83187_neigh1.83541_cross0.94038_n14.mtx","synthetic_66463_66463_9347522_avg140.64249_std389.6368_diagonal_bw33861.58836_skew424.82436_neigh1.84023_cross0.94139_n14.mtx","synthetic_68678_68678_10731088_avg156.25219_std397.34109_diagonal_bw34504.32861_skew424.88203_neigh1.844_cross0.94379_n14.mtx","synthetic_70221_70221_15563153_avg221.63104_std96.30657_diagonal_bw45552.93294_skew15.41016_neigh1.70514_cross0.85192_n14.mtx","synthetic_73109_73109_11420815_avg156.21627_std394.47456_diagonal_bw36296.88892_skew424.97997_neigh1.84425_cross0.94399_n14.mtx","synthetic_74477_74477_16505933_avg221.62457_std96.28344_diagonal_bw48331.68664_skew15.41064_neigh1.70415_cross0.85307_n14.mtx","synthetic_77540_77540_12111057_avg156.19109_std391.93861_diagonal_bw38852.76655_skew425.04863_neigh1.84276_cross0.94411_n14.mtx","synthetic_78733_78733_17449768_avg221.6322_std96.26215_diagonal_bw51104.86271_skew15.41007_neigh1.7037_cross0.8514_n14.mtx","synthetic_81971_81971_12801291_avg156.16854_std389.68003_diagonal_bw40877.20774_skew425.11016_neigh1.84172_cross0.94425_n14.mtx","synthetic_82989_82989_18393238_avg221.63465_std96.24293_diagonal_bw53916.13093_skew15.40989_neigh1.70419_cross0.85143_n14.mtx","synthetic_86401_86401_13491346_avg156.14803_std387.65694_diagonal_bw43065.81676_skew425.16612_neigh1.845_cross0.94312_n14.mtx","synthetic_968035_968035_11863180_avg12.25491_std37.64729_diagonal_bw22702.75661_skew632.62368_neigh1.32267_cross0.78592_n14.mtx","synthetic_1382908_1382908_16923784_avg12.23782_std37.37393_diagonal_bw32417.85806_skew443.68692_neigh1.28774_cross0.78284_n14.mtx","synthetic_1382908_1382908_16927023_avg12.24017_std37.40259_diagonal_bw32489.12672_skew485.75812_neigh1.2937_cross0.78368_n14.mtx","synthetic_1382908_1382908_16931896_avg12.24369_std37.43372_diagonal_bw32563.1309_skew527.76219_neigh1.30041_cross0.78501_n14.mtx","synthetic_1382908_1382908_16938002_avg12.2481_std37.46714_diagonal_bw32505.8507_skew569.78219_neigh1.30908_cross0.78582_n14.mtx","synthetic_1382908_1382908_16943948_avg12.2524_std37.50354_diagonal_bw32473.5113_skew611.69607_neigh1.31734_cross0.7865_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_diagonal_bw16934.48122_skew632.66322_neigh1.72397_cross0.78921_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_diagonal_bw26955.89151_skew632.66322_neigh1.63424_cross0.788_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_diagonal_bw28433.29645_skew632.66322_neigh0.52815_cross0.98565_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_diagonal_bw28482.64659_skew632.66322_neigh1.10244_cross0.78637_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_diagonal_bw28699.51453_skew632.66322_neigh0.92649_cross0.7846_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_diagonal_bw29202.3537_skew632.66322_neigh1.32237_cross0.78681_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_diagonal_bw29676.91177_skew632.66322_neigh1.01331_cross0.78531_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_diagonal_bw30053.31884_skew632.66322_neigh1.19029_cross0.78701_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_diagonal_bw30933.71381_skew632.66322_neigh1.36619_cross0.78741_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_diagonal_bw31363.81879_skew632.66322_neigh1.32155_cross0.78695_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_diagonal_bw31759.41838_skew632.66322_neigh1.36147_cross0.55271_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_diagonal_bw31850.6332_skew632.66322_neigh1.35679_cross0.60476_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_diagonal_bw31986.10955_skew632.66322_neigh1.35083_cross0.65686_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_diagonal_bw32172.68164_skew632.66322_neigh1.34169_cross0.70926_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_diagonal_bw32389.12793_skew632.66322_neigh1.32961_cross0.76125_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_diagonal_bw32518.92977_skew632.66322_neigh1.31159_cross0.81268_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_diagonal_bw32792.31039_skew632.66322_neigh0.99726_cross0.96686_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_diagonal_bw32847.57488_skew632.66322_neigh1.28282_cross0.8647_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_diagonal_bw33206.98578_skew632.66322_neigh1.21973_cross0.91658_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_diagonal_bw33535.67693_skew632.66322_neigh1.32147_cross0.78686_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_diagonal_bw33796.51893_skew632.66322_neigh1.27722_cross0.78689_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_diagonal_bw34964.23666_skew632.66322_neigh1.54388_cross0.78867_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_diagonal_bw35676.67346_skew632.66322_neigh1.32177_cross0.78687_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_diagonal_bw37797.05653_skew632.66322_neigh1.32178_cross0.78697_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_diagonal_bw40050.68177_skew632.66322_neigh1.32136_cross0.78693_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_diagonal_bw41281.79686_skew632.66322_neigh1.45465_cross0.78781_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_diagonal_bw42127.99755_skew632.66322_neigh1.32137_cross0.787_n14.mtx","synthetic_1382908_1382908_16948048_avg12.25537_std37.72372_diagonal_bw31935.00008_skew822.06782_neigh1.35574_cross0.78975_n14.mtx","synthetic_1382908_1382908_16948709_avg12.25585_std37.54211_diagonal_bw32403.26495_skew653.62631_neigh1.32622_cross0.78731_n14.mtx","synthetic_1382908_1382908_16951601_avg12.25794_std37.67423_diagonal_bw32052.58183_skew779.80014_neigh1.34903_cross0.78953_n14.mtx","synthetic_1382908_1382908_16951652_avg12.25798_std37.58358_diagonal_bw32324.44939_skew695.60771_neigh1.33364_cross0.78844_n14.mtx","synthetic_1382908_1382908_16952713_avg12.25874_std37.62729_diagonal_bw32145.64971_skew737.65652_neigh1.34221_cross0.78918_n14.mtx","synthetic_187667_187667_12936634_avg68.93398_std101.26087_diagonal_bw21604.74821_skew10.08307_neigh1.12835_cross0.83707_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_diagonal_bw37687.1288_skew10.11157_neigh1.12382_cross0.83773_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_diagonal_bw39604.98722_skew10.11157_neigh1.12032_cross0.83705_n14.mtx","synthetic_268096_268096_18483826_avg68.9448_std96.40124_diagonal_bw30839.36433_skew7.10794_neigh1.1166_cross0.85528_n14.mtx","synthetic_1000005_1000005_2190550_avg2.19054_std25.45314_diagonal_bw114108.9876_skew1501.82644_neigh0.08498_cross0.84657_n14.mtx","synthetic_1000005_1000005_2378888_avg2.37888_std25.47359_diagonal_bw121468.2977_skew1514.4215_neigh0.10196_cross0.883_n14.mtx","synthetic_1000005_1000005_2551748_avg2.55174_std25.499_diagonal_bw114016.12502_skew1534.81764_neigh0.10873_cross0.87763_n14.mtx","synthetic_1000005_1000005_2793946_avg2.79393_std25.52481_diagonal_bw131905.46734_skew1513.71115_neigh0.10444_cross0.86052_n14.mtx","synthetic_1000005_1000005_3068666_avg3.06865_std25.54963_diagonal_bw112236.356_skew1480.43289_neigh0.12104_cross0.88326_n14.mtx","synthetic_1000005_1000005_3110917_avg3.1109_std33.11754_diagonal_bw99822.95915_skew1510.459_neigh0.11796_cross0.85632_n14.mtx","synthetic_1000005_1000005_3116965_avg3.11695_std31.43703_diagonal_bw103495.94606_skew1507.52625_neigh0.11943_cross0.86424_n14.mtx","synthetic_1000005_1000005_3121903_avg3.12189_std25.45302_diagonal_bw106120.25943_skew1053.81063_neigh0.11898_cross0.86904_n14.mtx","synthetic_1000005_1000005_3127972_avg3.12796_std29.75772_diagonal_bw108092.68293_skew1502.5376_neigh0.12362_cross0.87298_n14.mtx","synthetic_1000005_1000005_3132968_avg3.13295_std25.47429_diagonal_bw109696.38688_skew1150.31021_neigh0.12322_cross0.87652_n14.mtx","synthetic_1000005_1000005_3136144_avg3.13613_std18.05255_diagonal_bw131504.88603_skew1498.61976_neigh0.20987_cross0.9344_n14.mtx","synthetic_1000005_1000005_3144021_avg3.14401_std28.07823_diagonal_bw112779.06701_skew1494.86263_neigh0.12564_cross0.88411_n14.mtx","synthetic_1000005_1000005_3144795_avg3.14478_std25.49712_diagonal_bw113241.01438_skew1245.51038_neigh0.12721_cross0.885_n14.mtx","synthetic_1000005_1000005_3156371_avg3.15636_std25.52223_diagonal_bw116391.07562_skew1340.10381_neigh0.13204_cross0.89292_n14.mtx","synthetic_1000005_1000005_3160475_avg3.16046_std19.71515_diagonal_bw131174.03454_skew1487.0749_neigh0.20681_cross0.93372_n14.mtx","synthetic_1000005_1000005_3162276_avg3.16226_std26.40103_diagonal_bw118193.42178_skew1486.2274_neigh0.13377_cross0.89745_n14.mtx","synthetic_1000005_1000005_3166512_avg3.1665_std25.54915_diagonal_bw119317.10886_skew1434.65625_neigh0.13844_cross0.90081_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_diagonal_bw100875.1064_skew1482.18441_neigh0.14415_cross0.90421_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_diagonal_bw108834.1731_skew1482.18441_neigh0.14336_cross0.90436_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_diagonal_bw116472.78808_skew1482.18441_neigh0.14374_cross0.90463_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_diagonal_bw117096.97406_skew1482.18441_neigh0.06224_cross0.96168_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_diagonal_bw117677.40758_skew1482.18441_neigh0.18675_cross0.90502_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_diagonal_bw118297.28426_skew1482.18441_neigh0.17603_cross0.90478_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_diagonal_bw118732.2225_skew1482.18441_neigh0.11799_cross0.93285_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_diagonal_bw118876.87938_skew1482.18441_neigh0.164_cross0.9047_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_diagonal_bw119416.85487_skew1482.18441_neigh0.1575_cross0.90469_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_diagonal_bw119981.2095_skew1482.18441_neigh0.14756_cross0.90455_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_diagonal_bw120655.04241_skew1482.18441_neigh0.13912_cross0.90438_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_diagonal_bw121157.45525_skew1482.18441_neigh0.12935_cross0.90421_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_diagonal_bw121505.44901_skew1482.18441_neigh0.15658_cross0.87562_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_diagonal_bw121800.16162_skew1482.18441_neigh0.1198_cross0.90401_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_diagonal_bw122167.3375_skew1482.18441_neigh0.10989_cross0.90394_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_diagonal_bw122547.34699_skew1482.18441_neigh0.1007_cross0.90373_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_diagonal_bw122970.89533_skew1482.18441_neigh0.17404_cross0.8183_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_diagonal_bw123852.15593_skew1482.18441_neigh0.18429_cross0.76047_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_diagonal_bw124316.39867_skew1482.18441_neigh0.14315_cross0.90464_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_diagonal_bw124333.68929_skew1482.18441_neigh0.19183_cross0.70272_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_diagonal_bw124629.28556_skew1482.18441_neigh0.19573_cross0.64408_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_diagonal_bw132643.35795_skew1482.18441_neigh0.14112_cross0.90484_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_diagonal_bw139742.14787_skew1482.18441_neigh0.14113_cross0.9048_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_diagonal_bw147748.47168_skew1482.18441_neigh0.14113_cross0.90477_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_diagonal_bw154767.97818_skew1482.18441_neigh0.14355_cross0.90514_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_diagonal_bw85508.89004_skew1482.18441_neigh0.14435_cross0.90404_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_diagonal_bw92882.01052_skew1482.18441_neigh0.14352_cross0.90404_n14.mtx","synthetic_1000005_1000005_3174621_avg3.17461_std25.57736_diagonal_bw121783.7238_skew1529.58406_neigh0.14525_cross0.90761_n14.mtx","synthetic_1000005_1000005_3177914_avg3.1779_std24.72495_diagonal_bw122735.04959_skew1478.909_neigh0.15049_cross0.91097_n14.mtx","synthetic_1000005_1000005_3179023_avg3.17901_std21.38191_diagonal_bw130097.76418_skew1478.39273_neigh0.19486_cross0.93033_n14.mtx","synthetic_1000005_1000005_3180578_avg3.18056_std25.60874_diagonal_bw123725.69417_skew1625.12766_neigh0.15445_cross0.91356_n14.mtx","synthetic_1000005_1000005_3184160_avg3.18414_std25.64113_diagonal_bw125738.34862_skew1721.91199_neigh0.16506_cross0.91841_n14.mtx","synthetic_1000005_1000005_3185020_avg3.185_std23.052_diagonal_bw127168.47608_skew1475.60722_neigh0.17295_cross0.92261_n14.mtx","synthetic_1000005_1000005_3185310_avg3.18529_std25.71304_diagonal_bw128318.0595_skew1917.81812_neigh0.18146_cross0.92577_n14.mtx","synthetic_1000005_1000005_3185554_avg3.18554_std25.67645_diagonal_bw127162.91658_skew1819.4146_neigh0.17343_cross0.92255_n14.mtx","synthetic_1000005_1000005_3238430_avg3.23841_std25.57761_diagonal_bw124163.77369_skew1499.42591_neigh0.16758_cross0.91813_n14.mtx","synthetic_1000005_1000005_3338044_avg3.33803_std25.60993_diagonal_bw115730.26217_skew1548.7177_neigh0.16947_cross0.91526_n14.mtx","synthetic_1000005_1000005_3554394_avg3.55438_std25.64567_diagonal_bw109077.20175_skew1542.73078_neigh0.13309_cross0.88625_n14.mtx","synthetic_1000005_1000005_3911756_avg3.91174_std25.67945_diagonal_bw128127.29214_skew1481.71748_neigh0.14126_cross0.8869_n14.mtx","synthetic_1000005_1000005_4154204_avg4.15418_std25.71363_diagonal_bw152102.90185_skew1470.76946_neigh0.19404_cross0.9225_n14.mtx","synthetic_1006441_1006441_22414673_avg22.27122_std29.74727_diagonal_bw241757.44752_skew313.62123_neigh1.25654_cross0.76073_n14.mtx","synthetic_100993_100993_2185846_avg21.64354_std13.77891_diagonal_bw47261.74273_skew4.03614_neigh1.00741_cross0.57221_n14.mtx","synthetic_1019593_1019593_74066178_avg72.64289_std15.80935_diagonal_bw15214.38873_skew1.03736_neigh1.69545_cross0.82041_n14.mtx","synthetic_102778_102778_7408141_avg72.07905_std19.01282_diagonal_bw5790.43171_skew1.19204_neigh1.65271_cross0.77107_n14.mtx","synthetic_1033338_1033338_3275830_avg3.17014_std25.55637_diagonal_bw124512.44793_skew1482.52894_neigh0.14371_cross0.90431_n14.mtx","synthetic_1047423_1047423_51169710_avg48.85296_std11.94333_diagonal_bw179790.95101_skew1.10837_neigh1.69181_cross0.80579_n14.mtx","synthetic_1063952_1063952_23695040_avg22.27078_std29.72463_diagonal_bw255536.35333_skew313.62752_neigh1.25753_cross0.76084_n14.mtx","synthetic_108003_108003_5987303_avg55.43645_std11.04356_diagonal_bw4191.65781_skew0.89406_neigh1.63079_cross0.7877_n14.mtx","synthetic_108334_108334_7808284_avg72.07602_std19.0204_diagonal_bw6122.43647_skew1.19213_neigh1.65216_cross0.76954_n14.mtx","synthetic_1085373_1085373_78800672_avg72.60239_std15.80629_diagonal_bw16200.45996_skew1.02473_neigh1.69539_cross0.82034_n14.mtx","synthetic_109072_109072_2360564_avg21.64225_std13.77936_diagonal_bw51027.24323_skew4.08265_neigh1.00811_cross0.57094_n14.mtx","synthetic_1100005_1100005_3487516_avg3.17045_std25.54298_diagonal_bw132402.99662_skew1482.38345_neigh0.14166_cross0.90462_n14.mtx","synthetic_1110903_1110903_54291613_avg48.87161_std11.93583_diagonal_bw190510.69059_skew1.10756_neigh1.69155_cross0.80595_n14.mtx","synthetic_1121463_1121463_24976074_avg22.27097_std29.7046_diagonal_bw269393.05866_skew313.62476_neigh1.2569_cross0.76099_n14.mtx","synthetic_1151153_1151153_83612943_avg72.63408_std15.81129_diagonal_bw17174.81048_skew1.01008_neigh1.69486_cross0.82103_n14.mtx","synthetic_1166672_1166672_3699263_avg3.17078_std25.53274_diagonal_bw140872.67348_skew1482.23015_neigh0.13992_cross0.90484_n14.mtx","synthetic_117152_117152_2535420_avg21.64214_std13.77711_diagonal_bw54858.54236_skew4.08268_neigh1.00527_cross0.57159_n14.mtx","synthetic_117394_117394_6507908_avg55.43646_std11.04171_diagonal_bw4548.03296_skew0.89406_neigh1.63141_cross0.78802_n14.mtx","synthetic_1174383_1174383_57377479_avg48.85755_std11.94355_diagonal_bw201337.3527_skew1.10817_neigh1.69149_cross0.80605_n14.mtx","synthetic_119698_119698_670830_avg5.60435_std4.47718_diagonal_bw35851.2119_skew62.8789_neigh0.59319_cross0.62674_n14.mtx","synthetic_121192_121192_1836052_avg15.14994_std12.36381_diagonal_bw54781.70924_skew3.8185_neigh0.97072_cross0.58007_n14.mtx","synthetic_121192_121192_2010928_avg16.59291_std12.82631_diagonal_bw54606.41729_skew3.94187_neigh0.98248_cross0.57348_n14.mtx","synthetic_121192_121192_2185768_avg18.03558_std13.23353_diagonal_bw55828.6145_skew4.04558_neigh0.98987_cross0.56777_n14.mtx","synthetic_121192_121192_2360629_avg19.47842_std13.60481_diagonal_bw55574.55169_skew4.13389_neigh0.99709_cross0.56533_n14.mtx","synthetic_121192_121192_2535413_avg20.92063_std13.77469_diagonal_bw56884.2582_skew4.11457_neigh1.00373_cross0.57077_n14.mtx","synthetic_121192_121192_2622629_avg21.64028_std13.77138_diagonal_bw56242.87207_skew3.85206_neigh1.00809_cross0.56863_n14.mtx","synthetic_121192_121192_2622663_avg21.64056_std13.63311_diagonal_bw55697.66942_skew3.52853_neigh1.00738_cross0.56431_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_diagonal_bw41449.60518_skew4.08299_neigh1.00709_cross0.56924_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_diagonal_bw44969.04646_skew4.08299_neigh1.00918_cross0.5684_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_diagonal_bw48375.95076_skew4.08299_neigh1.0066_cross0.57186_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_diagonal_bw51808.58859_skew4.08299_neigh1.00555_cross0.57153_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_diagonal_bw52786.48945_skew4.08299_neigh0.98178_cross0.73314_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_diagonal_bw53882.38648_skew4.08299_neigh0.98833_cross0.69862_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_diagonal_bw54816.63871_skew4.08299_neigh0.99503_cross0.66136_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_diagonal_bw55046.60667_skew4.08299_neigh1.00618_cross0.57223_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_diagonal_bw55588.08012_skew4.08299_neigh1.00197_cross0.62866_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_diagonal_bw56289.99224_skew4.08299_neigh0.77156_cross0.57111_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_diagonal_bw56335.34997_skew4.08299_neigh1.00541_cross0.59144_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_diagonal_bw56445.73291_skew4.08299_neigh0.83909_cross0.57135_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_diagonal_bw56550.54337_skew4.08299_neigh0.70497_cross0.57228_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_diagonal_bw56640.70898_skew4.08299_neigh1.10568_cross0.57621_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_diagonal_bw56646.25647_skew4.08299_neigh0.90696_cross0.57222_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_diagonal_bw56981.50968_skew4.08299_neigh1.00923_cross0.55476_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_diagonal_bw57143.44331_skew4.08299_neigh1.1732_cross0.57642_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_diagonal_bw57206.83955_skew4.08299_neigh0.97383_cross0.57171_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_diagonal_bw57392.3198_skew4.08299_neigh1.03836_cross0.57342_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_diagonal_bw57680.27683_skew4.08299_neigh1.01286_cross0.52046_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_diagonal_bw57736.54601_skew4.08299_neigh1.2401_cross0.57851_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_diagonal_bw58118.42279_skew4.08299_neigh1.01366_cross0.48629_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_diagonal_bw58305.84042_skew4.08299_neigh1.00621_cross0.5718_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_diagonal_bw58608.29432_skew4.08299_neigh1.01533_cross0.452_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_diagonal_bw59091.75387_skew4.08299_neigh1.01816_cross0.41458_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_diagonal_bw59170.98197_skew4.08299_neigh1.30576_cross0.57913_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_diagonal_bw61351.56958_skew4.08299_neigh1.00459_cross0.57351_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_diagonal_bw64498.77718_skew4.08299_neigh1.00458_cross0.57593_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_diagonal_bw67553.05734_skew4.08299_neigh1.00396_cross0.57603_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_diagonal_bw70322.56861_skew4.08299_neigh1.00409_cross0.57935_n14.mtx","synthetic_121192_121192_2622724_avg21.64107_std13.77599_diagonal_bw56586.74825_skew3.99051_neigh1.00718_cross0.57135_n14.mtx","synthetic_121192_121192_2622736_avg21.64116_std14.23425_diagonal_bw56154.62111_skew4.08291_neigh1.00762_cross0.56767_n14.mtx","synthetic_121192_121192_2622750_avg21.64128_std13.768_diagonal_bw55889.48449_skew3.71321_neigh1.00743_cross0.56293_n14.mtx","synthetic_121192_121192_2622777_avg21.6415_std15.59481_diagonal_bw55368.29813_skew4.08283_neigh1.00631_cross0.56573_n14.mtx","synthetic_121192_121192_2622792_avg21.64163_std13.77856_diagonal_bw56873.57123_skew4.129_neigh1.0086_cross0.57384_n14.mtx","synthetic_121192_121192_2622812_avg21.64179_std16.1555_diagonal_bw55349.84176_skew3.99034_neigh1.00627_cross0.56748_n14.mtx","synthetic_121192_121192_2622828_avg21.64192_std13.43338_diagonal_bw55856.38154_skew3.29722_neigh1.00759_cross0.56535_n14.mtx","synthetic_121192_121192_2622837_avg21.642_std15.01875_diagonal_bw55585.0516_skew4.12892_neigh1.00673_cross0.56331_n14.mtx","synthetic_121192_121192_2622876_avg21.64232_std16.69445_diagonal_bw55240.42113_skew3.94402_neigh1.00765_cross0.5674_n14.mtx","synthetic_121192_121192_2622972_avg21.64311_std13.78148_diagonal_bw57067.76991_skew4.31347_neigh1.00707_cross0.57783_n14.mtx","synthetic_121192_121192_2623047_avg21.64373_std13.32114_diagonal_bw57121.20154_skew4.0361_neigh1.00729_cross0.57944_n14.mtx","synthetic_121192_121192_2623109_avg21.64424_std13.78493_diagonal_bw57338.33986_skew4.4518_neigh1.00545_cross0.58243_n14.mtx","synthetic_121192_121192_2623156_avg21.64463_std11.48856_diagonal_bw58934.13194_skew3.89729_neigh1.00538_cross0.59657_n14.mtx","synthetic_121192_121192_2623189_avg21.6449_std13.78718_diagonal_bw57991.06353_skew4.77503_neigh1.00635_cross0.58424_n14.mtx","synthetic_121192_121192_2623197_avg21.64497_std13.78635_diagonal_bw57626.49167_skew4.59021_neigh1.00568_cross0.58329_n14.mtx","synthetic_121192_121192_2623249_avg21.6454_std12.40537_diagonal_bw57996.48538_skew3.98951_neigh1.0059_cross0.58819_n14.mtx","synthetic_121192_121192_2623583_avg21.64815_std9.65791_diagonal_bw60405.36362_skew3.8503_neigh1.00451_cross0.61091_n14.mtx","synthetic_121192_121192_2623613_avg21.6484_std10.57459_diagonal_bw59660.52937_skew3.85024_neigh1.00452_cross0.60326_n14.mtx","synthetic_121192_121192_2710314_avg22.3638_std13.78076_diagonal_bw57516.41338_skew4.05281_neigh1.00876_cross0.57867_n14.mtx","synthetic_121192_121192_2885490_avg23.80924_std13.78209_diagonal_bw57922.69442_skew3.99806_neigh1.01487_cross0.58481_n14.mtx","synthetic_121192_121192_3060321_avg25.25184_std13.78548_diagonal_bw58954.02529_skew3.95013_neigh1.0179_cross0.59132_n14.mtx","synthetic_121192_121192_3235599_avg26.69812_std13.78831_diagonal_bw59260.45423_skew3.86926_neigh1.02285_cross0.59628_n14.mtx","synthetic_121192_121192_3410529_avg28.14154_std13.78985_diagonal_bw59475.58642_skew3.86825_neigh1.02443_cross0.59913_n14.mtx","synthetic_1216933_1216933_88363297_avg72.61147_std15.79644_diagonal_bw18153.04703_skew1.03825_neigh1.69455_cross0.82106_n14.mtx","synthetic_1233339_1233339_3909668_avg3.16999_std25.52165_diagonal_bw148870.18881_skew1482.60253_neigh0.14197_cross0.90505_n14.mtx","synthetic_1237863_1237863_60496296_avg48.87156_std11.94811_diagonal_bw211983.92253_skew1.20987_neigh1.69135_cross0.80663_n14.mtx","synthetic_125231_125231_2710355_avg21.64284_std13.77742_diagonal_bw58546.37263_skew4.08251_neigh1.00665_cross0.57313_n14.mtx","synthetic_126786_126786_7028939_avg55.43939_std11.04404_diagonal_bw4906.49056_skew0.89396_neigh1.63134_cross0.78947_n14.mtx","synthetic_1282713_1282713_93191153_avg72.6516_std15.81312_diagonal_bw19135.30286_skew1.10594_neigh1.69446_cross0.821_n14.mtx","synthetic_129947_129947_10504776_avg80.83893_std117.70867_diagonal_bw16837.76377_skew7.85712_neigh1.21519_cross0.8459_n14.mtx","synthetic_1300006_1300006_4120758_avg3.1698_std25.51264_diagonal_bw157091.52358_skew1482.69019_neigh0.13981_cross0.90518_n14.mtx","synthetic_131098_131098_734679_avg5.60404_std4.47302_diagonal_bw39029.11606_skew62.88244_neigh0.59297_cross0.62541_n14.mtx","synthetic_133311_133311_2884795_avg21.63959_std13.78017_diagonal_bw62354.80178_skew4.08328_neigh1.00679_cross0.57368_n14.mtx","synthetic_136178_136178_7548659_avg55.43229_std11.05483_diagonal_bw5281.69958_skew0.91224_neigh1.63102_cross0.78738_n14.mtx","synthetic_140874_140874_10153211_avg72.07299_std11.05163_diagonal_bw5512.27589_skew0.7066_neigh1.64608_cross0.80473_n14.mtx","synthetic_140874_140874_5465261_avg38.79538_std11.0515_diagonal_bw5412.9335_skew1.31986_neigh1.60389_cross0.75869_n14.mtx","synthetic_140874_140874_5986164_avg42.49304_std11.0536_diagonal_bw5548.31213_skew1.18859_neigh1.61095_cross0.76635_n14.mtx","synthetic_140874_140874_6506839_avg46.18907_std11.05195_diagonal_bw5412.39753_skew1.10006_neigh1.61828_cross0.77328_n14.mtx","synthetic_140874_140874_7027827_avg49.88732_std11.05353_diagonal_bw5512.6976_skew1.02456_neigh1.62363_cross0.78108_n14.mtx","synthetic_140874_140874_7548897_avg53.58616_std11.05321_diagonal_bw5415.30582_skew0.9408_neigh1.62912_cross0.7833_n14.mtx","synthetic_140874_140874_7807971_avg55.42521_std14.36638_diagonal_bw5335.16197_skew1.18312_neigh1.63649_cross0.76697_n14.mtx","synthetic_140874_140874_7808175_avg55.42666_std13.63005_diagonal_bw5359.29341_skew1.12894_neigh1.63441_cross0.77184_n14.mtx","synthetic_140874_140874_7808374_avg55.42807_std12.89549_diagonal_bw5396.31618_skew1.07476_neigh1.63344_cross0.77504_n14.mtx","synthetic_140874_140874_7808713_avg55.43048_std12.15675_diagonal_bw5408.74805_skew1.00251_neigh1.63324_cross0.78138_n14.mtx","synthetic_140874_140874_7809044_avg55.43283_std11.42187_diagonal_bw5439.57739_skew0.9483_neigh1.6324_cross0.78435_n14.mtx","synthetic_140874_140874_7809186_avg55.43383_std10.68354_diagonal_bw5480.44304_skew0.89415_neigh1.62983_cross0.78974_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_diagonal_bw3838.69228_skew0.91217_neigh1.63011_cross0.78907_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_diagonal_bw4204.41167_skew0.91217_neigh1.62943_cross0.78923_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_diagonal_bw4558.67931_skew0.91217_neigh1.63021_cross0.78941_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_diagonal_bw4913.35727_skew0.91217_neigh1.63146_cross0.78927_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_diagonal_bw5282.63594_skew0.91217_neigh1.63072_cross0.78718_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_diagonal_bw5457.86719_skew0.91217_neigh1.63077_cross0.78851_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_diagonal_bw5648.31754_skew0.91217_neigh1.62967_cross0.78702_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_diagonal_bw6002.23969_skew0.91217_neigh1.63068_cross0.78728_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_diagonal_bw6356.58993_skew0.91217_neigh1.62903_cross0.78853_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_diagonal_bw6710.26111_skew0.91217_neigh1.62896_cross0.78828_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_diagonal_bw7054.0657_skew0.91217_neigh1.63058_cross0.78783_n14.mtx","synthetic_140874_140874_7809480_avg55.43592_std9.94668_diagonal_bw5490.06378_skew0.82192_neigh1.63009_cross0.79615_n14.mtx","synthetic_140874_140874_7809970_avg55.4394_std9.2113_diagonal_bw5566.8335_skew0.7677_neigh1.62757_cross0.7999_n14.mtx","synthetic_140874_140874_7810161_avg55.44076_std8.47606_diagonal_bw5572.74971_skew0.6955_neigh1.62616_cross0.80687_n14.mtx","synthetic_140874_140874_7810358_avg55.44215_std7.74028_diagonal_bw5620.76373_skew0.64135_neigh1.62766_cross0.81104_n14.mtx","synthetic_140874_140874_8069568_avg57.28217_std11.05207_diagonal_bw5516.71932_skew0.8854_neigh1.63387_cross0.78936_n14.mtx","synthetic_140874_140874_8590481_avg60.97989_std11.05315_diagonal_bw5437.17752_skew0.83667_neigh1.63703_cross0.79384_n14.mtx","synthetic_140874_140874_9111507_avg64.67841_std11.05222_diagonal_bw5508.99952_skew0.77803_neigh1.6398_cross0.79875_n14.mtx","synthetic_140874_140874_9632262_avg68.37502_std11.0533_diagonal_bw5447.32524_skew0.7404_neigh1.64497_cross0.80063_n14.mtx","synthetic_141390_141390_3059797_avg21.64083_std13.77882_diagonal_bw66168.99875_skew4.08298_neigh1.00521_cross0.57194_n14.mtx","synthetic_142323_142323_11504673_avg80.83495_std117.71151_diagonal_bw18462.18834_skew7.85755_neigh1.21247_cross0.84559_n14.mtx","synthetic_142498_142498_798541_avg5.60388_std4.46852_diagonal_bw42535.5183_skew62.88436_neigh0.59531_cross0.62546_n14.mtx","synthetic_144550_144550_890999_avg6.16395_std4.44624_diagonal_bw249.77002_skew7.27392_neigh0.18253_cross0.33392_n14.mtx","synthetic_145569_145569_8068734_avg55.42893_std11.05658_diagonal_bw5649.76329_skew0.91236_neigh1.62963_cross0.7868_n14.mtx","synthetic_14839_14839_1040332_avg70.10796_std6.31579_diagonal_bw1132.91226_skew0.34079_neigh1.79083_cross0.91451_n14.mtx","synthetic_149470_149470_3234141_avg21.63739_std13.78086_diagonal_bw69927.19965_skew4.08379_neigh1.00664_cross0.57176_n14.mtx","synthetic_153898_153898_862342_avg5.60333_std4.46589_diagonal_bw45848.56757_skew62.89053_neigh0.59749_cross0.62567_n14.mtx","synthetic_154699_154699_12503850_avg80.82696_std117.71492_diagonal_bw20044.28137_skew7.85843_neigh1.21606_cross0.84532_n14.mtx","synthetic_154961_154961_8588351_avg55.42266_std11.06218_diagonal_bw6007.51607_skew0.91258_neigh1.63019_cross0.78673_n14.mtx","synthetic_157549_157549_3408821_avg21.63658_std13.78244_diagonal_bw73653.17583_skew4.08398_neigh1.00489_cross0.5734_n14.mtx","synthetic_158316_158316_975794_avg6.16358_std4.44586_diagonal_bw273.64732_skew7.27441_neigh0.1798_cross0.33007_n14.mtx","synthetic_16253_16253_1139653_avg70.11955_std6.30855_diagonal_bw1264.2658_skew0.34057_neigh1.78351_cross0.91717_n14.mtx","synthetic_164353_164353_9109408_avg55.42587_std11.0688_diagonal_bw6365.94759_skew0.91246_neigh1.6294_cross0.78797_n14.mtx","synthetic_165298_165298_926272_avg5.60365_std4.4624_diagonal_bw49243.13259_skew62.88694_neigh0.59497_cross0.62573_n14.mtx","synthetic_167075_167075_13504757_avg80.83051_std117.71597_diagonal_bw21622.74045_skew7.87041_neigh1.21592_cross0.84505_n14.mtx","synthetic_170998_170998_1008507_avg5.89777_std3.14995_diagonal_bw45171.22177_skew59.87046_neigh0.62353_cross0.6314_n14.mtx","synthetic_170998_170998_1037773_avg6.06892_std4.43265_diagonal_bw45901.38609_skew59.96638_neigh0.64574_cross0.63979_n14.mtx","synthetic_170998_170998_1043193_avg6.10062_std4.44271_diagonal_bw46214.72752_skew63.58365_neigh0.6538_cross0.64274_n14.mtx","synthetic_170998_170998_1092151_avg6.38692_std4.47337_diagonal_bw46053.98994_skew64.44623_neigh0.62763_cross0.62819_n14.mtx","synthetic_170998_170998_1211686_avg7.08597_std4.4649_diagonal_bw49555.83123_skew64.76379_neigh0.67173_cross0.6434_n14.mtx","synthetic_170998_170998_1212281_avg7.08945_std4.45045_diagonal_bw49559.75803_skew61.3462_neigh0.67177_cross0.64357_n14.mtx","synthetic_170998_170998_703429_avg4.11367_std4.4083_diagonal_bw62525.67356_skew60.01611_neigh0.55916_cross0.6516_n14.mtx","synthetic_170998_170998_709969_avg4.15191_std4.42013_diagonal_bw62829.82587_skew65.23451_neigh0.57062_cross0.65616_n14.mtx","synthetic_170998_170998_813246_avg4.75588_std4.44226_diagonal_bw46482.54362_skew61.86954_neigh0.56085_cross0.62458_n14.mtx","synthetic_170998_170998_875832_avg5.12188_std4.42226_diagonal_bw49819.5485_skew62.06273_neigh0.62237_cross0.64457_n14.mtx","synthetic_170998_170998_882159_avg5.15888_std4.44207_diagonal_bw49851.31179_skew66.06876_neigh0.61393_cross0.64338_n14.mtx","synthetic_170998_170998_944721_avg5.52475_std5.76053_diagonal_bw50240.49201_skew63.79933_neigh0.59642_cross0.62742_n14.mtx","synthetic_170998_170998_945632_avg5.53008_std5.47116_diagonal_bw50240.27614_skew63.7369_neigh0.5981_cross0.62739_n14.mtx","synthetic_170998_170998_946661_avg5.53609_std4.44002_diagonal_bw50440.90686_skew44.88072_neigh0.59713_cross0.62583_n14.mtx","synthetic_170998_170998_947518_avg5.54111_std5.18255_diagonal_bw50420.37847_skew63.60804_neigh0.59685_cross0.62681_n14.mtx","synthetic_170998_170998_948659_avg5.54778_std4.44165_diagonal_bw50506.82008_skew48.9299_neigh0.59655_cross0.62663_n14.mtx","synthetic_170998_170998_950710_avg5.55977_std4.8941_diagonal_bw50483.86054_skew63.39112_neigh0.59518_cross0.62663_n14.mtx","synthetic_170998_170998_951053_avg5.56178_std4.44852_diagonal_bw50556.26173_skew53.11938_neigh0.59346_cross0.62678_n14.mtx","synthetic_170998_170998_953667_avg5.57707_std4.45309_diagonal_bw50785.42323_skew57.09507_neigh0.59646_cross0.62658_n14.mtx","synthetic_170998_170998_955297_avg5.5866_std4.60347_diagonal_bw50921.85081_skew63.08194_neigh0.596_cross0.6275_n14.mtx","synthetic_170998_170998_956707_avg5.59484_std4.45769_diagonal_bw51000.98471_skew61.0214_neigh0.59632_cross0.62507_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_diagonal_bw36461.87693_skew62.88573_neigh0.59695_cross0.61997_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_diagonal_bw39610.08149_skew62.88573_neigh0.59906_cross0.62096_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_diagonal_bw43105.47571_skew62.88573_neigh0.59816_cross0.62237_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_diagonal_bw44031.98084_skew62.88573_neigh0.5372_cross0.62197_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_diagonal_bw44847.35264_skew62.88573_neigh0.49711_cross0.62325_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_diagonal_bw45384.2646_skew62.88573_neigh0.4588_cross0.62138_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_diagonal_bw45971.97334_skew62.88573_neigh0.41681_cross0.61967_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_diagonal_bw46159.87112_skew62.88573_neigh0.59674_cross0.62414_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_diagonal_bw46483.06345_skew62.88573_neigh0.77274_cross0.62785_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_diagonal_bw47421.7408_skew62.88573_neigh0.73666_cross0.62676_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_diagonal_bw48584.83352_skew62.88573_neigh0.69732_cross0.62642_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_diagonal_bw49003.35613_skew62.88573_neigh0.54697_cross0.79644_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_diagonal_bw49449.25941_skew62.88573_neigh0.59289_cross0.62664_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_diagonal_bw49562.93434_skew62.88573_neigh0.65726_cross0.6251_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_diagonal_bw49598.09636_skew62.88573_neigh0.56075_cross0.75943_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_diagonal_bw50162.74852_skew62.88573_neigh0.57537_cross0.72046_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_diagonal_bw50491.14134_skew62.88573_neigh0.58432_cross0.6826_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_diagonal_bw50571.15461_skew62.88573_neigh0.61583_cross0.62382_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_diagonal_bw50893.01638_skew62.88573_neigh0.5929_cross0.64621_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_diagonal_bw51129.15176_skew62.88573_neigh0.59642_cross0.60766_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_diagonal_bw51234.36906_skew62.88573_neigh0.60413_cross0.5687_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_diagonal_bw51395.69692_skew62.88573_neigh0.61075_cross0.52993_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_diagonal_bw51420.69302_skew62.88573_neigh0.61218_cross0.49298_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_diagonal_bw51535.92532_skew62.88573_neigh0.57482_cross0.62495_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_diagonal_bw51655.22982_skew62.88573_neigh0.61451_cross0.45451_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_diagonal_bw52368.46746_skew62.88573_neigh0.59522_cross0.62658_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_diagonal_bw55725.08564_skew62.88573_neigh0.59126_cross0.62765_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_diagonal_bw58848.50641_skew62.88573_neigh0.5924_cross0.62814_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_diagonal_bw61752.83746_skew62.88573_neigh0.59197_cross0.62973_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_diagonal_bw64753.49829_skew62.88573_neigh0.59013_cross0.63179_n14.mtx","synthetic_170998_170998_959835_avg5.61314_std4.46266_diagonal_bw51077.39694_skew64.91681_neigh0.59508_cross0.62613_n14.mtx","synthetic_170998_170998_961623_avg5.62359_std4.31537_diagonal_bw51156.06217_skew62.66038_neigh0.59268_cross0.62607_n14.mtx","synthetic_170998_170998_963061_avg5.632_std4.4671_diagonal_bw51143.10828_skew68.77981_neigh0.59568_cross0.62558_n14.mtx","synthetic_170998_170998_966597_avg5.65268_std4.47351_diagonal_bw51344.14215_skew72.59341_neigh0.59618_cross0.6253_n14.mtx","synthetic_170998_170998_970106_avg5.6732_std4.48093_diagonal_bw51465.29233_skew76.55763_neigh0.59896_cross0.62591_n14.mtx","synthetic_170998_170998_970315_avg5.67442_std4.02645_diagonal_bw51483.87979_skew62.09011_neigh0.59847_cross0.62615_n14.mtx","synthetic_170998_170998_973557_avg5.69338_std4.48437_diagonal_bw51658.71521_skew80.32248_neigh0.59938_cross0.6276_n14.mtx","synthetic_170998_170998_981215_avg5.73817_std3.73499_diagonal_bw52085.53194_skew61.38927_neigh0.60333_cross0.62711_n14.mtx","synthetic_170998_170998_994346_avg5.81496_std3.44543_diagonal_bw52702.41379_skew60.73734_neigh0.6117_cross0.63171_n14.mtx","synthetic_172083_172083_1060867_avg6.16486_std4.44424_diagonal_bw297.53188_skew7.2727_neigh0.17686_cross0.32932_n14.mtx","synthetic_173744_173744_9631043_avg55.43238_std11.0756_diagonal_bw6722.25909_skew0.91224_neigh1.62924_cross0.7879_n14.mtx","synthetic_17666_17666_1238955_avg70.13217_std6.31199_diagonal_bw1367.70486_skew0.34033_neigh1.78108_cross0.91647_n14.mtx","synthetic_176697_176697_990167_avg5.60376_std4.45819_diagonal_bw52568.63684_skew62.88571_neigh0.59492_cross0.62595_n14.mtx","synthetic_179451_179451_14506771_avg80.83973_std117.71232_diagonal_bw23249.97453_skew7.8694_neigh1.21265_cross0.84464_n14.mtx","synthetic_183136_183136_10152501_avg55.43695_std11.07295_diagonal_bw7090.3351_skew0.91208_neigh1.631_cross0.78675_n14.mtx","synthetic_185639_185639_10500260_avg56.56279_std99.5812_diagonal_bw18098.88818_skew7.18559_neigh1.25585_cross0.95445_n14.mtx","synthetic_185639_185639_11501046_avg61.95382_std109.06293_diagonal_bw18425.33593_skew7.19966_neigh1.25674_cross0.95406_n14.mtx","synthetic_185639_185639_12507698_avg67.37646_std112.75287_diagonal_bw23036.91435_skew7.4154_neigh1.18654_cross0.88126_n14.mtx","synthetic_185639_185639_13507262_avg72.76091_std115.00904_diagonal_bw23636.41108_skew7.61726_neigh1.19914_cross0.86016_n14.mtx","synthetic_185639_185639_14507140_avg78.14705_std116.8824_diagonal_bw23970.8135_skew7.80392_neigh1.21053_cross0.84821_n14.mtx","synthetic_185639_185639_15003294_avg80.81973_std87.6573_diagonal_bw24360.81189_skew8.39127_neigh1.21918_cross0.82425_n14.mtx","synthetic_185639_185639_15003763_avg80.82226_std142.24862_diagonal_bw19300.06598_skew7.19081_neigh1.26239_cross0.95424_n14.mtx","synthetic_185639_185639_15004156_avg80.82437_std94.60907_diagonal_bw24270.69581_skew8.30412_neigh1.21969_cross0.82786_n14.mtx","synthetic_185639_185639_15004647_avg80.82702_std101.43013_diagonal_bw24276.21957_skew8.20484_neigh1.21772_cross0.83109_n14.mtx","synthetic_185639_185639_15005689_avg80.83263_std108.09033_diagonal_bw24209.38904_skew8.08049_neigh1.21736_cross0.83603_n14.mtx","synthetic_185639_185639_15006431_avg80.83663_std114.55767_diagonal_bw24067.75636_skew7.94397_neigh1.21737_cross0.84232_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_diagonal_bw17089.64436_skew7.8818_neigh1.21539_cross0.84521_n14.mtx","synthetic_185639_185639_15007305_avg80.84134_std120.8084_diagonal_bw23932.70372_skew7.80738_neigh1.21348_cross0.84884_n14.mtx","synthetic_185639_185639_15008187_avg80.84609_std126.81395_diagonal_bw23820.75271_skew7.64606_neigh1.20923_cross0.85805_n14.mtx","synthetic_185639_185639_15008702_avg80.84886_std137.983_diagonal_bw23110.0529_skew7.32417_neigh1.20249_cross0.89205_n14.mtx","synthetic_185639_185639_15008859_avg80.84971_std132.54683_diagonal_bw23516.89351_skew7.48488_neigh1.20404_cross0.87118_n14.mtx","synthetic_185639_185639_15506741_avg83.5317_std118.48272_diagonal_bw24157.25407_skew7.94271_neigh1.21788_cross0.84141_n14.mtx","synthetic_185639_185639_16506181_avg88.91548_std119.88133_diagonal_bw24248.66297_skew8.06479_neigh1.22306_cross0.83613_n14.mtx","synthetic_185639_185639_17505764_avg94.30003_std121.13382_diagonal_bw24308.77245_skew8.16224_neigh1.22615_cross0.83329_n14.mtx","synthetic_185639_185639_18505682_avg99.68639_std122.27517_diagonal_bw24387.0841_skew8.23897_neigh1.22956_cross0.82911_n14.mtx","synthetic_185639_185639_19505424_avg105.0718_std123.33624_diagonal_bw24435.45274_skew8.2984_neigh1.23248_cross0.82657_n14.mtx","synthetic_185850_185850_1145706_avg6.16468_std4.44467_diagonal_bw321.4122_skew7.27293_neigh0.17353_cross0.32676_n14.mtx","synthetic_188097_188097_1054064_avg5.60383_std4.45656_diagonal_bw55856.29168_skew62.88486_neigh0.59686_cross0.62548_n14.mtx","synthetic_19080_19080_1337993_avg70.12542_std6.31221_diagonal_bw1466.79911_skew0.34046_neigh1.78524_cross0.91492_n14.mtx","synthetic_191826_191826_15508397_avg80.84617_std117.70962_diagonal_bw24854.19831_skew7.88106_neigh1.21284_cross0.84477_n14.mtx","synthetic_199497_199497_1118014_avg5.60416_std4.45398_diagonal_bw59520.93639_skew62.88107_neigh0.59561_cross0.62565_n14.mtx","synthetic_199616_199616_1230682_avg6.16525_std4.44318_diagonal_bw345.00688_skew7.27217_neigh0.17135_cross0.32384_n14.mtx","synthetic_204202_204202_16508082_avg80.84192_std117.71063_diagonal_bw26469.08637_skew7.88153_neigh1.21323_cross0.84417_n14.mtx","synthetic_20493_20493_1437302_avg70.13624_std6.31204_diagonal_bw1570.36622_skew0.34025_neigh1.78301_cross0.915_n14.mtx","synthetic_206500_206500_1060353_avg5.13488_std4.44292_diagonal_bw335.07862_skew7.3741_neigh0.16054_cross0.3088_n14.mtx","synthetic_206500_206500_1145534_avg5.54738_std4.44341_diagonal_bw356.74644_skew7.2922_neigh0.16406_cross0.31454_n14.mtx","synthetic_206500_206500_1230425_avg5.95847_std4.44436_diagonal_bw374.57129_skew7.22358_neigh0.16743_cross0.31868_n14.mtx","synthetic_206500_206500_1272393_avg6.16171_std5.76962_diagonal_bw330.69652_skew7.43922_neigh0.17725_cross0.30587_n14.mtx","synthetic_206500_206500_1272581_avg6.16262_std4.44309_diagonal_bw352.66015_skew6.30209_neigh0.16953_cross0.32032_n14.mtx","synthetic_206500_206500_1272606_avg6.16274_std4.44382_diagonal_bw349.99388_skew5.97741_neigh0.16985_cross0.31715_n14.mtx","synthetic_206500_206500_1272648_avg6.16294_std5.47591_diagonal_bw338.10897_skew7.43753_neigh0.17526_cross0.31073_n14.mtx","synthetic_206500_206500_1272653_avg6.16297_std5.1813_diagonal_bw344.20462_skew7.27523_neigh0.17301_cross0.31335_n14.mtx","synthetic_206500_206500_1272686_avg6.16313_std4.88791_diagonal_bw350.18601_skew7.27502_neigh0.1719_cross0.3183_n14.mtx","synthetic_206500_206500_1272775_avg6.16356_std4.44367_diagonal_bw347.38147_skew5.48976_neigh0.17032_cross0.31578_n14.mtx","synthetic_206500_206500_1272904_avg6.16418_std4.4462_diagonal_bw361.59791_skew8.73365_neigh0.17074_cross0.32864_n14.mtx","synthetic_206500_206500_1272958_avg6.16445_std4.59126_diagonal_bw354.70395_skew7.27325_neigh0.171_cross0.32279_n14.mtx","synthetic_206500_206500_1272981_avg6.16456_std4.44526_diagonal_bw362.3522_skew9.05749_neigh0.17062_cross0.32883_n14.mtx","synthetic_206500_206500_1272983_avg6.16457_std4.4441_diagonal_bw354.25261_skew6.62422_neigh0.17027_cross0.32275_n14.mtx","synthetic_206500_206500_1272990_avg6.1646_std4.44515_diagonal_bw357.75317_skew7.43526_neigh0.17014_cross0.3244_n14.mtx","synthetic_206500_206500_1272993_avg6.16462_std4.44589_diagonal_bw358.92702_skew7.92189_neigh0.17019_cross0.32662_n14.mtx","synthetic_206500_206500_1272999_avg6.16464_std4.44572_diagonal_bw360.42425_skew8.24628_neigh0.17053_cross0.3283_n14.mtx","synthetic_206500_206500_1273034_avg6.16481_std4.29748_diagonal_bw358.86067_skew7.27276_neigh0.17064_cross0.32624_n14.mtx","synthetic_206500_206500_1273051_avg6.1649_std3.71025_diagonal_bw366.16904_skew7.27265_neigh0.16791_cross0.33052_n14.mtx","synthetic_206500_206500_1273053_avg6.16491_std4.44447_diagonal_bw355.79495_skew7.11042_neigh0.17039_cross0.3241_n14.mtx","synthetic_206500_206500_1273074_avg6.16501_std4.00298_diagonal_bw362.83666_skew7.2725_neigh0.16845_cross0.33021_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_diagonal_bw250.50201_skew7.27211_neigh0.1828_cross0.332_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_diagonal_bw272.67229_skew7.27211_neigh0.18005_cross0.32953_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_diagonal_bw297.25621_skew7.27211_neigh0.17513_cross0.32858_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_diagonal_bw321.39573_skew7.27211_neigh0.17392_cross0.32618_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_diagonal_bw345.22322_skew7.27211_neigh0.17126_cross0.32461_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_diagonal_bw352.99268_skew7.27211_neigh0.16832_cross0.41146_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_diagonal_bw353.69447_skew7.27211_neigh0.17026_cross0.39266_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_diagonal_bw354.69502_skew7.27211_neigh0.19369_cross0.32315_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_diagonal_bw354.75358_skew7.27211_neigh0.16996_cross0.37247_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_diagonal_bw355.71822_skew7.27211_neigh0.18424_cross0.32217_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_diagonal_bw355.76259_skew7.27211_neigh0.16942_cross0.35332_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_diagonal_bw356.3296_skew7.27211_neigh0.17524_cross0.3238_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_diagonal_bw356.8881_skew7.27211_neigh0.16993_cross0.33263_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_diagonal_bw357.32531_skew7.27211_neigh0.17048_cross0.3136_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_diagonal_bw357.60403_skew7.27211_neigh0.16581_cross0.32313_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_diagonal_bw358.15523_skew7.27211_neigh0.17117_cross0.29493_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_diagonal_bw358.35721_skew7.27211_neigh0.15654_cross0.322_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_diagonal_bw358.64557_skew7.27211_neigh0.17114_cross0.27625_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_diagonal_bw359.47191_skew7.27211_neigh0.17162_cross0.25643_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_diagonal_bw359.53662_skew7.27211_neigh0.14621_cross0.32227_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_diagonal_bw360.09546_skew7.27211_neigh0.17221_cross0.23667_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_diagonal_bw360.26387_skew7.27211_neigh0.13732_cross0.32285_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_diagonal_bw360.84579_skew7.27211_neigh0.12827_cross0.32319_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_diagonal_bw367.88503_skew7.27211_neigh0.17018_cross0.32346_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_diagonal_bw378.11669_skew7.27211_neigh0.21111_cross0.3229_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_diagonal_bw379.08464_skew7.27211_neigh0.20234_cross0.32283_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_diagonal_bw392.32879_skew7.27211_neigh0.16826_cross0.32093_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_diagonal_bw416.65222_skew7.27211_neigh0.16705_cross0.32007_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_diagonal_bw440.35505_skew7.27211_neigh0.16563_cross0.32035_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_diagonal_bw463.79818_skew7.27211_neigh0.16463_cross0.31842_n14.mtx","synthetic_206500_206500_1273351_avg6.16635_std3.41298_diagonal_bw369.12411_skew7.2707_neigh0.16658_cross0.33288_n14.mtx","synthetic_206500_206500_1274354_avg6.17121_std3.11913_diagonal_bw371.78899_skew7.10214_neigh0.1662_cross0.33678_n14.mtx","synthetic_206500_206500_1315280_avg6.36939_std4.44461_diagonal_bw364.33156_skew7.32104_neigh0.17162_cross0.32684_n14.mtx","synthetic_206500_206500_1400260_avg6.78092_std4.44568_diagonal_bw375.85394_skew7.25847_neigh0.17409_cross0.33062_n14.mtx","synthetic_206500_206500_1485240_avg7.19245_std4.44584_diagonal_bw368.30232_skew7.20305_neigh0.17875_cross0.33711_n14.mtx","synthetic_206500_206500_1570037_avg7.60308_std4.44659_diagonal_bw376.69061_skew7.15458_neigh0.18122_cross0.33823_n14.mtx","synthetic_206500_206500_1655223_avg8.01561_std4.44684_diagonal_bw384.76944_skew7.23394_neigh0.18229_cross0.34249_n14.mtx","synthetic_206500_206500_890883_avg4.3142_std4.3742_diagonal_bw314.74509_skew7.34453_neigh0.14985_cross0.29741_n14.mtx","synthetic_206500_206500_975718_avg4.72503_std4.44295_diagonal_bw342.29814_skew7.46556_neigh0.15394_cross0.29927_n14.mtx","synthetic_210897_210897_1181860_avg5.60397_std4.45232_diagonal_bw62903.73817_skew62.88331_neigh0.59456_cross0.6267_n14.mtx","synthetic_21200_21200_1040239_avg49.06788_std6.3116_diagonal_bw2199.9967_skew0.48774_neigh1.7444_cross0.90083_n14.mtx","synthetic_21200_21200_1139550_avg53.75236_std6.31054_diagonal_bw2242.21934_skew0.4325_neigh1.75417_cross0.90404_n14.mtx","synthetic_21200_21200_1238795_avg58.43373_std6.30753_diagonal_bw2270.23382_skew0.4033_neigh1.7655_cross0.90641_n14.mtx","synthetic_21200_21200_1338002_avg63.1133_std6.31089_diagonal_bw1589.57203_skew0.37847_neigh1.76861_cross0.91301_n14.mtx","synthetic_21200_21200_1437342_avg67.79915_std6.31153_diagonal_bw1595.57099_skew0.35695_neigh1.77813_cross0.91446_n14.mtx","synthetic_21200_21200_1486357_avg70.11118_std8.20133_diagonal_bw1585.25472_skew0.44057_neigh1.79532_cross0.90224_n14.mtx","synthetic_21200_21200_1486477_avg70.11684_std7.77973_diagonal_bw1583.02868_skew0.41193_neigh1.79079_cross0.90683_n14.mtx","synthetic_21200_21200_1486586_avg70.12198_std7.35959_diagonal_bw1593.45448_skew0.39756_neigh1.78432_cross0.91026_n14.mtx","synthetic_21200_21200_1486691_avg70.12693_std6.94018_diagonal_bw1598.67321_skew0.36895_neigh1.79102_cross0.91039_n14.mtx","synthetic_21200_21200_1486777_avg70.13099_std6.51934_diagonal_bw1631.45156_skew0.35461_neigh1.78321_cross0.91501_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_diagonal_bw1081.97071_skew0.3403_neigh1.27549_cross0.91799_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_diagonal_bw1084.54524_skew0.3403_neigh1.38938_cross0.91731_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_diagonal_bw1117.74425_skew0.3403_neigh1.49561_cross0.91247_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_diagonal_bw1145.98717_skew0.3403_neigh1.61232_cross0.9156_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_diagonal_bw1149.7934_skew0.3403_neigh1.7861_cross0.91524_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_diagonal_bw1271.02222_skew0.3403_neigh1.78132_cross0.91674_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_diagonal_bw1300.1284_skew0.3403_neigh1.73006_cross0.9152_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_diagonal_bw1368.9767_skew0.3403_neigh1.77995_cross0.91561_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_diagonal_bw1468.32679_skew0.3403_neigh1.78426_cross0.91509_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_diagonal_bw1537.10948_skew0.3403_neigh1.72718_cross0.94892_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_diagonal_bw1564.65613_skew0.3403_neigh1.73429_cross0.94452_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_diagonal_bw1571.45608_skew0.3403_neigh1.78136_cross0.91469_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_diagonal_bw1625.49264_skew0.3403_neigh1.77718_cross0.91652_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_diagonal_bw1634.23774_skew0.3403_neigh1.86126_cross0.70933_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_diagonal_bw1635.55807_skew0.3403_neigh1.83717_cross0.82731_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_diagonal_bw1642.04132_skew0.3403_neigh1.85064_cross0.77163_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_diagonal_bw1646.38675_skew0.3403_neigh1.86599_cross0.6495_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_diagonal_bw1648.95406_skew0.3403_neigh1.80659_cross0.88791_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_diagonal_bw1682.76604_skew0.3403_neigh1.77909_cross0.91593_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_diagonal_bw1801.4084_skew0.3403_neigh1.77944_cross0.91386_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_diagonal_bw1870.38679_skew0.3403_neigh1.77964_cross0.91599_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_diagonal_bw1950.42142_skew0.3403_neigh1.85569_cross0.91574_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_diagonal_bw1986.16745_skew0.3403_neigh1.78558_cross0.9125_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_diagonal_bw2072.52415_skew0.3403_neigh1.78223_cross0.91562_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_diagonal_bw2091.85245_skew0.3403_neigh1.83963_cross0.91531_n14.mtx","synthetic_21200_21200_1487000_avg70.14151_std6.10324_diagonal_bw1634.66854_skew0.32589_neigh1.77365_cross0.91984_n14.mtx","synthetic_21200_21200_1487077_avg70.14514_std5.67801_diagonal_bw1633.00311_skew0.29731_neigh1.77437_cross0.91951_n14.mtx","synthetic_21200_21200_1487192_avg70.15057_std5.25576_diagonal_bw1652.73608_skew0.28295_neigh1.78041_cross0.92082_n14.mtx","synthetic_21200_21200_1487315_avg70.15637_std4.84276_diagonal_bw1652.45113_skew0.25434_neigh1.77351_cross0.92458_n14.mtx","synthetic_21200_21200_1487478_avg70.16406_std4.4186_diagonal_bw1687.16137_skew0.23995_neigh1.76694_cross0.92745_n14.mtx","synthetic_21200_21200_1536543_avg72.47844_std6.30853_diagonal_bw1645.73835_skew0.32453_neigh1.78455_cross0.91665_n14.mtx","synthetic_21200_21200_1635721_avg77.15665_std6.31046_diagonal_bw1656.96292_skew0.30903_neigh1.79294_cross0.9192_n14.mtx","synthetic_21200_21200_1735090_avg81.84387_std6.31393_diagonal_bw1680.13108_skew0.29515_neigh1.79626_cross0.9206_n14.mtx","synthetic_21200_21200_1834315_avg86.52429_std6.30851_diagonal_bw1434.02858_skew0.27132_neigh1.79882_cross0.92217_n14.mtx","synthetic_21200_21200_1933465_avg91.20118_std6.31043_diagonal_bw1436.35245_skew0.26095_neigh1.80915_cross0.92347_n14.mtx","synthetic_213383_213383_1315556_avg6.16523_std4.44392_diagonal_bw368.56408_skew7.27219_neigh0.16933_cross0.32183_n14.mtx","synthetic_216578_216578_17508080_avg80.83961_std117.71328_diagonal_bw28056.02979_skew7.89416_neigh1.2146_cross0.84473_n14.mtx","synthetic_21906_21906_1536320_avg70.13238_std6.31756_diagonal_bw1682.27098_skew0.34032_neigh1.78153_cross0.91497_n14.mtx","synthetic_222297_222297_1245675_avg5.60365_std4.45068_diagonal_bw66275.4602_skew62.88691_neigh0.59474_cross0.62554_n14.mtx","synthetic_227150_227150_1400378_avg6.16499_std4.4444_diagonal_bw392.40004_skew7.27252_neigh0.16863_cross0.32173_n14.mtx","synthetic_228954_228954_18508358_avg80.83876_std117.71434_diagonal_bw29655.86157_skew7.89425_neigh1.21497_cross0.8448_n14.mtx","synthetic_23320_23320_1635398_avg70.12856_std6.31946_diagonal_bw1799.99889_skew0.3404_neigh1.78021_cross0.91361_n14.mtx","synthetic_240916_240916_1485276_avg6.16512_std4.4437_diagonal_bw416.70601_skew7.27235_neigh0.16679_cross0.32047_n14.mtx","synthetic_241330_241330_19509069_avg80.8398_std117.71435_diagonal_bw31247.32631_skew7.89413_neigh1.21166_cross0.84524_n14.mtx","synthetic_24733_24733_1734543_avg70.13072_std6.31636_diagonal_bw1883.50532_skew0.34035_neigh1.77994_cross0.91558_n14.mtx","synthetic_254683_254683_1570192_avg6.16528_std4.4433_diagonal_bw440.76115_skew7.27213_neigh0.1672_cross0.32015_n14.mtx","synthetic_25491_25491_3028356_avg118.80099_std31.7722_diagonal_bw2553.87309_skew1.00335_neigh1.79544_cross0.81183_n14.mtx","synthetic_26146_26146_1833536_avg70.12683_std6.31243_diagonal_bw2003.83967_skew0.34043_neigh1.78323_cross0.91235_n14.mtx","synthetic_268096_268096_18481037_avg68.9344_std103.42196_diagonal_bw30970.77096_skew12.99882_neigh1.12598_cross0.8301_n14.mtx","synthetic_268096_268096_18481246_avg68.93518_std102.1488_diagonal_bw30884.51728_skew11.06931_neigh1.12482_cross0.83411_n14.mtx","synthetic_268096_268096_18481318_avg68.93545_std102.6344_diagonal_bw30907.47443_skew11.72205_neigh1.12565_cross0.83282_n14.mtx","synthetic_268096_268096_18481428_avg68.93586_std103.0557_diagonal_bw30967.38712_skew12.36025_neigh1.12626_cross0.83123_n14.mtx","synthetic_268096_268096_18481631_avg68.93662_std101.58036_diagonal_bw30873.57231_skew10.43079_neigh1.12458_cross0.83658_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_diagonal_bw29370.46931_skew10.11157_neigh1.10661_cross0.8633_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_diagonal_bw30799.97611_skew10.11157_neigh1.16245_cross0.83769_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_diagonal_bw30825.7074_skew10.11157_neigh1.23495_cross0.83815_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_diagonal_bw30839.54861_skew10.11157_neigh1.38307_cross0.83895_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_diagonal_bw30893.42836_skew10.11157_neigh1.01276_cross0.83715_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_diagonal_bw30904.3262_skew10.11157_neigh1.31024_cross0.83886_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_diagonal_bw30928.04662_skew10.11157_neigh1.08835_cross0.83649_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_diagonal_bw30929.83701_skew10.11157_neigh0.93859_cross0.83647_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_diagonal_bw30967.23835_skew10.11157_neigh0.86484_cross0.83608_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_diagonal_bw30999.29783_skew10.11157_neigh1.45859_cross0.83966_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_diagonal_bw31009.21338_skew10.11157_neigh0.79088_cross0.83554_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_diagonal_bw32084.18859_skew10.11157_neigh1.13434_cross0.80865_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_diagonal_bw33853.27533_skew10.11157_neigh1.14358_cross0.7543_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_diagonal_bw35112.29857_skew10.11157_neigh1.14998_cross0.69937_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_diagonal_bw36191.33218_skew10.11157_neigh1.15329_cross0.64297_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_diagonal_bw37019.2485_skew10.11157_neigh1.15704_cross0.58827_n14.mtx","synthetic_268096_268096_18482060_avg68.93822_std100.90606_diagonal_bw30904.29893_skew9.79227_neigh1.12389_cross0.83844_n14.mtx","synthetic_268096_268096_18482363_avg68.93935_std100.10125_diagonal_bw30812.69345_skew9.13935_neigh1.12221_cross0.84194_n14.mtx","synthetic_268096_268096_18482725_avg68.9407_std99.12285_diagonal_bw30864.65616_skew8.47191_neigh1.12038_cross0.84492_n14.mtx","synthetic_268096_268096_18483147_avg68.94227_std97.91705_diagonal_bw30780.25248_skew7.80447_neigh1.11844_cross0.84993_n14.mtx","synthetic_268450_268450_1655008_avg6.16505_std4.44456_diagonal_bw464.64015_skew7.27244_neigh0.16504_cross0.31934_n14.mtx","synthetic_27120_27120_1234696_avg45.52714_std31.4396_diagonal_bw652.95889_skew2.64618_neigh1.23287_cross0.77761_n14.mtx","synthetic_27560_27560_1933075_avg70.1406_std6.31297_diagonal_bw2085.91422_skew0.34017_neigh1.78453_cross0.915_n14.mtx","synthetic_27919_27919_3319237_avg118.8881_std31.78107_diagonal_bw2786.27282_skew1.00188_neigh1.79352_cross0.80999_n14.mtx","synthetic_29496_29496_3296630_avg111.76532_std88.66285_diagonal_bw11717.2995_skew3.35734_neigh1.89319_cross0.64015_n14.mtx","synthetic_29703_29703_1352183_avg45.52345_std31.43457_diagonal_bw714.4414_skew2.64647_neigh1.23245_cross0.77394_n14.mtx","synthetic_30347_30347_3608195_avg118.89791_std31.76143_diagonal_bw3042.07187_skew1.00172_neigh1.79272_cross0.81233_n14.mtx","synthetic_32286_32286_1469958_avg45.52927_std31.42835_diagonal_bw776.30704_skew2.66797_neigh1.22078_cross0.77601_n14.mtx","synthetic_32305_32305_3613009_avg111.84055_std88.75337_diagonal_bw12837.01436_skew3.35441_neigh1.89289_cross0.64054_n14.mtx","synthetic_32775_32775_3898242_avg118.9395_std31.78937_diagonal_bw3266.08442_skew1.00102_neigh1.79344_cross0.81117_n14.mtx","synthetic_32784_32784_1657863_avg50.56927_std27.82469_diagonal_bw4979.98978_skew3.09339_neigh1.63543_cross0.76823_n14.mtx","synthetic_3283001_3283001_13139084_avg4.00216_std1.21615_diagonal_bw3180.2682_skew312.8308_neigh0.41642_cross0.64828_n14.mtx","synthetic_34406_34406_1341834_avg39.0_std0.0_diagonal_bw7973.05903_skew0.0_neigh1.39882_cross0.80525_n14.mtx","synthetic_34869_34869_1587838_avg45.53724_std31.4194_diagonal_bw841.85663_skew2.66733_neigh1.22513_cross0.77194_n14.mtx","synthetic_35115_35115_3931279_avg111.95441_std88.67056_diagonal_bw13934.66499_skew3.34999_neigh1.89327_cross0.64172_n14.mtx","synthetic_35203_35203_4188100_avg118.96997_std31.72558_diagonal_bw3517.51751_skew1.0005_neigh1.79419_cross0.8104_n14.mtx","synthetic_35906_35906_1816148_avg50.58063_std27.80623_diagonal_bw5428.79923_skew3.09248_neigh1.63383_cross0.77123_n14.mtx","synthetic_3595668_3595668_14390363_avg4.00214_std1.20637_diagonal_bw3482.69612_skew312.83218_neigh0.41594_cross0.64805_n14.mtx","synthetic_36417_36417_3031503_avg83.24417_std31.62901_diagonal_bw3659.90647_skew1.4266_neigh1.77779_cross0.7712_n14.mtx","synthetic_36417_36417_3320284_avg91.17401_std31.69422_diagonal_bw3723.96647_skew1.30329_neigh1.78294_cross0.78319_n14.mtx","synthetic_36417_36417_3609556_avg99.11734_std31.72484_diagonal_bw3679.72145_skew1.19941_neigh1.78657_cross0.7906_n14.mtx","synthetic_36417_36417_3899087_avg107.06777_std31.73599_diagonal_bw3640.28097_skew1.11081_neigh1.79005_cross0.79883_n14.mtx","synthetic_36417_36417_4188724_avg115.02112_std31.74031_diagonal_bw3682.55345_skew1.03441_neigh1.79122_cross0.80743_n14.mtx","synthetic_36417_36417_4330804_avg118.92259_std41.20033_diagonal_bw3566.11415_skew1.30402_neigh1.79774_cross0.77959_n14.mtx","synthetic_36417_36417_4331206_avg118.93363_std39.11615_diagonal_bw3582.20153_skew1.23654_neigh1.79553_cross0.78805_n14.mtx","synthetic_36417_36417_4331707_avg118.94739_std37.01905_diagonal_bw3596.97108_skew1.16903_neigh1.79583_cross0.79348_n14.mtx","synthetic_36417_36417_4332360_avg118.96532_std34.9075_diagonal_bw3586.20139_skew1.10145_neigh1.79703_cross0.80194_n14.mtx","synthetic_36417_36417_4333163_avg118.98737_std32.79928_diagonal_bw3623.78502_skew1.03383_neigh1.79496_cross0.80499_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_diagonal_bw2579.24335_skew1.00003_neigh1.79431_cross0.81183_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_diagonal_bw2808.21114_skew1.00003_neigh1.79322_cross0.81044_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_diagonal_bw3060.87028_skew1.00003_neigh1.7932_cross0.81204_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_diagonal_bw3277.14828_skew1.00003_neigh1.79298_cross0.81293_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_diagonal_bw3375.93338_skew1.00003_neigh1.77805_cross0.86843_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_diagonal_bw3480.48944_skew1.00003_neigh1.38997_cross0.81257_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_diagonal_bw3491.53049_skew1.00003_neigh1.50396_cross0.81194_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_diagonal_bw3492.88901_skew1.00003_neigh1.27007_cross0.81135_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_diagonal_bw3503.97943_skew1.00003_neigh1.62041_cross0.81058_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_diagonal_bw3512.30892_skew1.00003_neigh1.7876_cross0.83745_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_diagonal_bw3516.78812_skew1.00003_neigh1.79411_cross0.811_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_diagonal_bw3588.56273_skew1.00003_neigh1.73452_cross0.81056_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_diagonal_bw3629.41058_skew1.00003_neigh1.79306_cross0.81098_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_diagonal_bw3710.52736_skew1.00003_neigh1.79803_cross0.78467_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_diagonal_bw3750.91707_skew1.00003_neigh1.79431_cross0.80913_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_diagonal_bw3858.63267_skew1.00003_neigh1.80498_cross0.73111_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_diagonal_bw3893.78263_skew1.00003_neigh1.85196_cross0.8121_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_diagonal_bw3967.32032_skew1.00003_neigh1.80868_cross0.68014_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_diagonal_bw3977.20534_skew1.00003_neigh1.79315_cross0.81223_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_diagonal_bw4055.86613_skew1.00003_neigh1.81065_cross0.62863_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_diagonal_bw4135.85089_skew1.00003_neigh1.81242_cross0.57712_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_diagonal_bw4215.83134_skew1.00003_neigh1.79293_cross0.81131_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_diagonal_bw4432.89683_skew1.00003_neigh1.79479_cross0.81053_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_diagonal_bw4655.25722_skew1.00003_neigh1.79359_cross0.80909_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_diagonal_bw5224.60291_skew1.00003_neigh1.94938_cross0.81229_n14.mtx","synthetic_36417_36417_4333866_avg119.00667_std30.68416_diagonal_bw3624.47093_skew0.96628_neigh1.79345_cross0.81402_n14.mtx","synthetic_36417_36417_4334685_avg119.02916_std28.56617_diagonal_bw3659.71673_skew0.89869_neigh1.79021_cross0.8221_n14.mtx","synthetic_36417_36417_4335436_avg119.04978_std26.44817_diagonal_bw3656.97312_skew0.83117_neigh1.79127_cross0.82819_n14.mtx","synthetic_36417_36417_4336151_avg119.06942_std24.33643_diagonal_bw3688.3458_skew0.76368_neigh1.79072_cross0.83561_n14.mtx","synthetic_36417_36417_4336875_avg119.0893_std22.21912_diagonal_bw3701.03968_skew0.7046_neigh1.78927_cross0.84338_n14.mtx","synthetic_36417_36417_4478356_avg122.97433_std31.74172_diagonal_bw3655.42475_skew0.96789_neigh1.79462_cross0.81529_n14.mtx","synthetic_36417_36417_4767999_avg130.92784_std31.7426_diagonal_bw3634.76154_skew0.90945_neigh1.79683_cross0.8202_n14.mtx","synthetic_36417_36417_5057685_avg138.88253_std31.74387_diagonal_bw3658.71557_skew0.85769_neigh1.80018_cross0.82673_n14.mtx","synthetic_36417_36417_5347290_avg146.83499_std31.74228_diagonal_bw3634.01996_skew0.81156_neigh1.80021_cross0.83143_n14.mtx","synthetic_36417_36417_5636923_avg154.78823_std31.74188_diagonal_bw3643.12294_skew0.77016_neigh1.80217_cross0.8324_n14.mtx","synthetic_368077_368077_1472308_avg4.0_std0.0_diagonal_bw504.58307_skew0.0_neigh0.37345_cross0.98911_n14.mtx","synthetic_37452_37452_1706036_avg45.5526_std31.4087_diagonal_bw902.84756_skew2.66609_neigh1.22018_cross0.77042_n14.mtx","synthetic_37630_37630_4477645_avg118.99136_std31.72824_diagonal_bw3752.49753_skew1.00015_neigh1.79415_cross0.80957_n14.mtx","synthetic_37683_37683_1469637_avg39.0_std0.0_diagonal_bw8731.54935_skew0.0_neigh1.39924_cross0.81006_n14.mtx","synthetic_37924_37924_4245700_avg111.95285_std88.65794_diagonal_bw15018.56603_skew3.35005_neigh1.89355_cross0.6412_n14.mtx","synthetic_38120_38120_14186695_avg372.15884_std217.74813_diagonal_bw16610.97285_skew1.39145_neigh1.97336_cross0.92298_n14.mtx","synthetic_38120_38120_14353565_avg376.53633_std228.90355_diagonal_bw16362.02206_skew1.46191_neigh1.97392_cross0.9229_n14.mtx","synthetic_38120_38120_14506403_avg380.54572_std239.95763_diagonal_bw16221.47907_skew1.5332_neigh1.97371_cross0.92545_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_diagonal_bw16107.56653_skew1.57045_neigh1.90891_cross0.91992_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_diagonal_bw16165.12815_skew1.57045_neigh1.3922_cross0.92174_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_diagonal_bw16173.10614_skew1.57045_neigh1.64922_cross0.92071_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_diagonal_bw16181.2495_skew1.57045_neigh1.51874_cross0.92011_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_diagonal_bw16189.91522_skew1.57045_neigh1.77707_cross0.91956_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_diagonal_bw16403.35273_skew1.57045_neigh1.97397_cross0.72233_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_diagonal_bw16412.02442_skew1.57045_neigh1.97389_cross0.66503_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_diagonal_bw16450.53809_skew1.57045_neigh1.9739_cross0.77998_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_diagonal_bw16504.44638_skew1.57045_neigh1.97349_cross0.83444_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_diagonal_bw5783.47169_skew1.57045_neigh1.99394_cross0.9293_n14.mtx","synthetic_38120_38120_14646455_avg384.2197_std250.9036_diagonal_bw16369.04491_skew1.60788_neigh1.97362_cross0.92456_n14.mtx","synthetic_38120_38120_14774683_avg387.5835_std261.73289_diagonal_bw16271.8489_skew1.68071_neigh1.97337_cross0.92347_n14.mtx","synthetic_38120_38120_14892124_avg390.66432_std272.44037_diagonal_bw16642.15774_skew1.75428_neigh1.9735_cross0.92246_n14.mtx","synthetic_38120_38120_15029193_avg394.26005_std283.24138_diagonal_bw17593.26878_skew1.83569_neigh1.95985_cross0.92052_n14.mtx","synthetic_38120_38120_15246247_avg399.95401_std294.48908_diagonal_bw17717.6793_skew1.94534_neigh1.95613_cross0.91225_n14.mtx","synthetic_38744_38744_1161488_avg29.97853_std22.5272_diagonal_bw1029.10133_skew1.93543_neigh1.22753_cross0.85246_n14.mtx","synthetic_38744_38744_1307882_avg33.75702_std24.88267_diagonal_bw994.82547_skew2.08084_neigh1.17474_cross0.82692_n14.mtx","synthetic_38744_38744_1449216_avg37.40491_std27.07152_diagonal_bw955.52689_skew2.28834_neigh1.19116_cross0.80482_n14.mtx","synthetic_38744_38744_1579684_avg40.77235_std28.98643_diagonal_bw944.42949_skew2.45823_neigh1.20967_cross0.78737_n14.mtx","synthetic_38744_38744_1703850_avg43.97713_std30.65815_diagonal_bw937.00284_skew2.61552_neigh1.2153_cross0.77679_n14.mtx","synthetic_38744_38744_1707520_avg44.07186_std32.46486_diagonal_bw992.99381_skew2.10856_neigh1.20651_cross0.82436_n14.mtx","synthetic_38744_38744_1730367_avg44.66155_std32.51286_diagonal_bw967.85456_skew2.22425_neigh1.21385_cross0.81258_n14.mtx","synthetic_38744_38744_1736329_avg44.81543_std26.74959_diagonal_bw952.63904_skew1.96773_neigh1.21351_cross0.79326_n14.mtx","synthetic_38744_38744_1746236_avg45.07113_std32.41542_diagonal_bw950.25712_skew2.35026_neigh1.21262_cross0.79957_n14.mtx","synthetic_38744_38744_1747768_avg45.11068_std28.0118_diagonal_bw947.08319_skew2.12565_neigh1.21504_cross0.78804_n14.mtx","synthetic_38744_38744_1755242_avg45.30358_std29.13201_diagonal_bw940.57346_skew2.28892_neigh1.21712_cross0.78411_n14.mtx","synthetic_38744_38744_1756611_avg45.33892_std32.15335_diagonal_bw942.33254_skew2.48486_neigh1.21428_cross0.78987_n14.mtx","synthetic_38744_38744_1760377_avg45.43612_std30.12952_diagonal_bw941.83703_skew2.43339_neigh1.22146_cross0.77671_n14.mtx","synthetic_38744_38744_1762627_avg45.49419_std31.70473_diagonal_bw939.03988_skew2.60486_neigh1.21755_cross0.77579_n14.mtx","synthetic_38744_38744_1763457_avg45.51562_std31.01391_diagonal_bw935.46596_skew2.60316_neigh1.22021_cross0.77524_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_diagonal_bw1007.61049_skew2.6669_neigh1.23901_cross0.60466_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_diagonal_bw1020.39872_skew2.6669_neigh1.23921_cross0.56066_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_diagonal_bw1025.26425_skew2.6669_neigh1.21375_cross0.77219_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_diagonal_bw1083.3129_skew2.6669_neigh1.21379_cross0.77025_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_diagonal_bw1146.61932_skew2.6669_neigh1.21114_cross0.77133_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_diagonal_bw1206.53601_skew2.6669_neigh1.21863_cross0.77168_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_diagonal_bw658.37758_skew2.6669_neigh1.22947_cross0.77507_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_diagonal_bw718.69366_skew2.6669_neigh1.22855_cross0.77459_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_diagonal_bw780.82743_skew2.6669_neigh1.22346_cross0.77387_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_diagonal_bw812.95816_skew2.6669_neigh1.18464_cross0.89437_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_diagonal_bw816.16013_skew2.6669_neigh1.18813_cross0.89061_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_diagonal_bw837.69265_skew2.6669_neigh1.22334_cross0.77418_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_diagonal_bw873.34813_skew2.6669_neigh1.20844_cross0.84314_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_diagonal_bw897.51727_skew2.6669_neigh1.2208_cross0.77498_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_diagonal_bw915.8694_skew2.6669_neigh1.21725_cross0.79709_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_diagonal_bw930.70419_skew2.6669_neigh1.10614_cross0.77727_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_diagonal_bw930.71709_skew2.6669_neigh1.18489_cross0.77237_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_diagonal_bw931.48136_skew2.6669_neigh1.03297_cross0.77123_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_diagonal_bw932.33357_skew2.6669_neigh0.96264_cross0.77189_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_diagonal_bw932.7993_skew2.6669_neigh1.25769_cross0.77371_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_diagonal_bw934.61429_skew2.6669_neigh0.88819_cross0.7721_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_diagonal_bw937.26642_skew2.6669_neigh1.33096_cross0.77342_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_diagonal_bw940.71841_skew2.6669_neigh1.40988_cross0.77173_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_diagonal_bw950.09139_skew2.6669_neigh1.48389_cross0.77368_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_diagonal_bw950.26476_skew2.6669_neigh1.2235_cross0.74756_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_diagonal_bw961.91957_skew2.6669_neigh1.22023_cross0.77204_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_diagonal_bw968.81104_skew2.6669_neigh1.56264_cross0.77172_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_diagonal_bw970.44453_skew2.6669_neigh1.2313_cross0.70063_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_diagonal_bw991.09919_skew2.6669_neigh1.23319_cross0.6528_n14.mtx","synthetic_38744_38744_1765368_avg45.56494_std31.78636_diagonal_bw930.97393_skew2.75289_neigh1.22265_cross0.77205_n14.mtx","synthetic_38744_38744_1765877_avg45.57808_std31.07073_diagonal_bw931.07658_skew2.72986_neigh1.22085_cross0.76806_n14.mtx","synthetic_38744_38744_1766496_avg45.59405_std32.481_diagonal_bw927.24148_skew2.88209_neigh1.22172_cross0.76806_n14.mtx","synthetic_38744_38744_1767073_avg45.60895_std26.86577_diagonal_bw920.15365_skew3.122_neigh1.22525_cross0.74801_n14.mtx","synthetic_38744_38744_1767160_avg45.61119_std30.25452_diagonal_bw926.58649_skew2.83678_neigh1.22133_cross0.7642_n14.mtx","synthetic_38744_38744_1767330_avg45.61558_std33.08901_diagonal_bw927.56388_skew3.03371_neigh1.22246_cross0.76288_n14.mtx","synthetic_38744_38744_1767431_avg45.61819_std29.2665_diagonal_bw927.4427_skew2.94579_neigh1.22246_cross0.75424_n14.mtx","synthetic_38744_38744_1767460_avg45.61893_std28.1289_diagonal_bw919.38078_skew3.03341_neigh1.22097_cross0.75479_n14.mtx","synthetic_38744_38744_1767651_avg45.62386_std33.63482_diagonal_bw924.81207_skew3.16449_neigh1.22491_cross0.76276_n14.mtx","synthetic_38744_38744_1767760_avg45.62668_std34.12282_diagonal_bw922.18836_skew3.31765_neigh1.22536_cross0.76056_n14.mtx","synthetic_38744_38744_1824565_avg47.09284_std32.11946_diagonal_bw936.71239_skew2.7373_neigh1.22296_cross0.76775_n14.mtx","synthetic_38744_38744_1943737_avg50.16872_std33.41096_diagonal_bw932.86026_skew2.82709_neigh1.22921_cross0.76384_n14.mtx","synthetic_38744_38744_2062005_avg53.22127_std34.5782_diagonal_bw922.50232_skew2.90821_neigh1.23925_cross0.76092_n14.mtx","synthetic_38744_38744_2179882_avg56.26373_std35.63951_diagonal_bw923.68405_skew2.98125_neigh1.24163_cross0.75814_n14.mtx","synthetic_38744_38744_2297582_avg59.30162_std36.62621_diagonal_bw923.20411_skew3.03024_neigh1.25019_cross0.75495_n14.mtx","synthetic_39029_39029_1974236_avg50.58382_std27.80163_diagonal_bw5889.07769_skew3.11199_neigh1.63432_cross0.77123_n14.mtx","synthetic_3908334_3908334_15641636_avg4.00212_std1.19836_diagonal_bw3787.4883_skew312.83338_neigh0.41515_cross0.64842_n14.mtx","synthetic_40035_40035_1824084_avg45.56223_std31.3971_diagonal_bw964.72262_skew2.66532_neigh1.22009_cross0.77447_n14.mtx","synthetic_40058_40058_4768887_avg119.04955_std31.78281_diagonal_bw3989.12307_skew0.99917_neigh1.79379_cross0.8114_n14.mtx","synthetic_403132_403132_1612528_avg4.0_std0.0_diagonal_bw553.42608_skew0.0_neigh0.37448_cross0.98906_n14.mtx","synthetic_40733_40733_4573744_avg112.28596_std88.81893_diagonal_bw16070.31044_skew3.33714_neigh1.8936_cross0.64178_n14.mtx","synthetic_40960_40960_1597440_avg39.0_std0.0_diagonal_bw9495.98555_skew0.0_neigh1.40202_cross0.80537_n14.mtx","synthetic_42138_42138_3666754_avg87.01775_std81.65352_diagonal_bw17018.78651_skew4.24031_neigh1.89191_cross0.58167_n14.mtx","synthetic_42138_42138_3893764_avg92.40505_std83.38574_diagonal_bw15269.74446_skew4.01055_neigh1.89198_cross0.59237_n14.mtx","synthetic_42138_42138_3928503_avg93.22946_std52.30654_diagonal_bw21754.9879_skew1.38122_neigh1.71883_cross0.97165_n14.mtx","synthetic_42138_42138_4006220_avg95.07381_std54.75201_diagonal_bw21199.48559_skew1.47176_neigh1.73848_cross0.96424_n14.mtx","synthetic_42138_42138_4072421_avg96.64486_std57.11798_diagonal_bw20552.35225_skew1.5661_neigh1.75712_cross0.95786_n14.mtx","synthetic_42138_42138_4126809_avg97.93557_std85.02631_diagonal_bw15674.99532_skew3.79907_neigh1.89239_cross0.60629_n14.mtx","synthetic_42138_42138_4128960_avg97.98662_std59.40057_diagonal_bw20131.7263_skew1.65342_neigh1.77031_cross0.9522_n14.mtx","synthetic_42138_42138_4176699_avg99.11954_std61.5954_diagonal_bw19886.75485_skew1.74416_neigh1.78051_cross0.94591_n14.mtx","synthetic_42138_42138_4365370_avg103.59699_std86.57531_diagonal_bw16089.94143_skew3.60438_neigh1.89274_cross0.62091_n14.mtx","synthetic_42138_42138_4491637_avg106.5935_std66.96099_diagonal_bw17746.25056_skew2.48989_neigh1.88275_cross0.71376_n14.mtx","synthetic_42138_42138_4533678_avg107.5912_std72.07832_diagonal_bw17488.78198_skew2.69919_neigh1.88644_cross0.69523_n14.mtx","synthetic_42138_42138_4582731_avg108.7553_std77.01103_diagonal_bw17151.27085_skew2.88947_neigh1.88851_cross0.67805_n14.mtx","synthetic_42138_42138_4608940_avg109.37728_std88.04095_diagonal_bw16405.47544_skew3.42505_neigh1.89321_cross0.63399_n14.mtx","synthetic_42138_42138_4638390_avg110.07618_std81.79532_diagonal_bw16937.41371_skew3.07899_neigh1.88999_cross0.6614_n14.mtx","synthetic_42138_42138_4699816_avg111.53391_std86.4582_diagonal_bw16776.08987_skew3.24983_neigh1.89212_cross0.64722_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_diagonal_bw12401.69942_skew3.33605_neigh1.89352_cross0.6363_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_diagonal_bw13394.34783_skew3.33605_neigh1.89357_cross0.63833_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_diagonal_bw13613.90223_skew3.33605_neigh1.32672_cross0.62991_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_diagonal_bw13637.14168_skew3.33605_neigh1.45441_cross0.63031_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_diagonal_bw13744.74961_skew3.33605_neigh1.57876_cross0.6321_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_diagonal_bw13939.96504_skew3.33605_neigh1.70489_cross0.63265_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_diagonal_bw14040.32778_skew3.33605_neigh1.96986_cross0.65808_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_diagonal_bw14295.7245_skew3.33605_neigh1.89407_cross0.63944_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_diagonal_bw14505.18821_skew3.33605_neigh1.83046_cross0.63541_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_diagonal_bw15310.55024_skew3.33605_neigh1.89386_cross0.6408_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_diagonal_bw16210.1421_skew3.33605_neigh1.89362_cross0.64161_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_diagonal_bw16611.63949_skew3.33605_neigh1.89329_cross0.64481_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_diagonal_bw16636.23077_skew3.33605_neigh1.89346_cross0.6403_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_diagonal_bw16806.17125_skew3.33605_neigh1.89434_cross0.62136_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_diagonal_bw17086.3858_skew3.33605_neigh1.89638_cross0.5859_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_diagonal_bw17169.41374_skew3.33605_neigh1.89337_cross0.64205_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_diagonal_bw17388.06735_skew3.33605_neigh1.89731_cross0.5472_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_diagonal_bw17574.48287_skew3.33605_neigh1.89818_cross0.5104_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_diagonal_bw17859.63285_skew3.33605_neigh1.89909_cross0.46925_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_diagonal_bw17951.80365_skew3.33605_neigh1.9579_cross0.66188_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_diagonal_bw17999.08176_skew3.33605_neigh1.89332_cross0.64342_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_diagonal_bw18723.11078_skew3.33605_neigh1.89352_cross0.64341_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_diagonal_bw19493.96504_skew3.33605_neigh1.89374_cross0.64434_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_diagonal_bw20097.98809_skew3.33605_neigh1.89382_cross0.64549_n14.mtx","synthetic_42138_42138_4766804_avg113.12364_std90.99737_diagonal_bw16569.85666_skew3.41994_neigh1.8942_cross0.63355_n14.mtx","synthetic_42138_42138_4838257_avg114.81933_std95.4594_diagonal_bw16396.72595_skew3.5724_neigh1.89578_cross0.62195_n14.mtx","synthetic_42138_42138_4857446_avg115.27472_std89.41484_diagonal_bw16809.13779_skew3.25939_neigh1.89318_cross0.64791_n14.mtx","synthetic_42138_42138_4913422_avg116.60311_std99.85384_diagonal_bw16243.33808_skew3.72543_neigh1.89696_cross0.61156_n14.mtx","synthetic_42138_42138_4991872_avg118.46485_std104.19584_diagonal_bw14758.12336_skew3.8622_neigh1.89775_cross0.60004_n14.mtx","synthetic_42138_42138_5073389_avg120.39938_std108.48726_diagonal_bw14653.196_skew4.00003_neigh1.89942_cross0.59075_n14.mtx","synthetic_42138_42138_5111019_avg121.2924_std90.68096_diagonal_bw15825.42845_skew3.10578_neigh1.89371_cross0.65868_n14.mtx","synthetic_42138_42138_5368820_avg127.41041_std91.85872_diagonal_bw16050.79954_skew2.96357_neigh1.89482_cross0.67034_n14.mtx","synthetic_42138_42138_5630518_avg133.62091_std92.94843_diagonal_bw16358.79403_skew2.83174_neigh1.89447_cross0.682_n14.mtx","synthetic_42138_42138_5895869_avg139.9181_std93.95081_diagonal_bw16631.46988_skew2.70931_neigh1.89472_cross0.69266_n14.mtx","synthetic_42151_42151_2133812_avg50.62305_std27.78297_diagonal_bw6371.02387_skew3.12855_neigh1.63402_cross0.77144_n14.mtx","synthetic_4221001_4221001_16892920_avg4.00211_std1.19165_diagonal_bw4085.33261_skew312.83427_neigh0.41605_cross0.64818_n14.mtx","synthetic_42486_42486_5059907_avg119.09587_std31.75092_diagonal_bw4233.6327_skew1.14113_neigh1.79264_cross0.81065_n14.mtx","synthetic_42618_42618_1942443_avg45.578_std31.38541_diagonal_bw1023.09081_skew2.66405_neigh1.21726_cross0.77332_n14.mtx","synthetic_43542_43542_4892832_avg112.3704_std88.7642_diagonal_bw17276.74331_skew3.82333_neigh1.89356_cross0.64204_n14.mtx","synthetic_43715_43715_2801362_avg64.0824_std14.01594_diagonal_bw317.77484_skew0.93501_neigh1.58611_cross0.83222_n14.mtx","synthetic_438187_438187_1752748_avg4.0_std0.0_diagonal_bw605.21006_skew0.0_neigh0.36936_cross0.98955_n14.mtx","synthetic_44236_44236_1725204_avg39.0_std0.0_diagonal_bw10260.92608_skew0.0_neigh1.40155_cross0.80735_n14.mtx","synthetic_44914_44914_5350806_avg119.13448_std31.75488_diagonal_bw4462.1276_skew1.14044_neigh1.7944_cross0.81059_n14.mtx","synthetic_45201_45201_2060190_avg45.57842_std31.38204_diagonal_bw1086.85529_skew2.68596_neigh1.21635_cross0.77336_n14.mtx","synthetic_45273_45273_2292008_avg50.62638_std27.77577_diagonal_bw6880.17573_skew3.12828_neigh1.63398_cross0.77235_n14.mtx","synthetic_4533668_4533668_18144199_avg4.0021_std1.18568_diagonal_bw4390.98196_skew312.83513_neigh0.41656_cross0.64825_n14.mtx","synthetic_46351_46351_5209319_avg112.38849_std88.86396_diagonal_bw18374.76208_skew3.82256_neigh1.89341_cross0.64137_n14.mtx","synthetic_46835_46835_1659331_avg35.42929_std24.19453_diagonal_bw7302.21917_skew2.92331_neigh1.5963_cross0.7705_n14.mtx","synthetic_46835_46835_1817944_avg38.81593_std25.43637_diagonal_bw7388.90979_skew3.04473_neigh1.61055_cross0.76366_n14.mtx","synthetic_46835_46835_1976036_avg42.19144_std26.52871_diagonal_bw7052.0227_skew3.14776_neigh1.62131_cross0.75772_n14.mtx","synthetic_46835_46835_2134025_avg45.56475_std27.52318_diagonal_bw7179.05959_skew3.21378_neigh1.62818_cross0.75329_n14.mtx","synthetic_46835_46835_2292218_avg48.94241_std27.77072_diagonal_bw6998.59428_skew3.16816_neigh1.62975_cross0.76816_n14.mtx","synthetic_46835_46835_2370905_avg50.6225_std27.74621_diagonal_bw6973.90507_skew2.81253_neigh1.63594_cross0.75611_n14.mtx","synthetic_46835_46835_2371001_avg50.62455_std27.39541_diagonal_bw6984.37081_skew2.69386_neigh1.6379_cross0.75534_n14.mtx","synthetic_46835_46835_2371113_avg50.62695_std32.27909_diagonal_bw6994.42105_skew3.10848_neigh1.63552_cross0.75717_n14.mtx","synthetic_46835_46835_2371156_avg50.62786_std27.75732_diagonal_bw7042.80683_skew2.91089_neigh1.63466_cross0.76079_n14.mtx","synthetic_46835_46835_2371164_avg50.62803_std30.41679_diagonal_bw6939.09181_skew3.22691_neigh1.63687_cross0.75372_n14.mtx","synthetic_46835_46835_2371201_avg50.62882_std33.12436_diagonal_bw6969.84853_skew3.04908_neigh1.63376_cross0.76209_n14.mtx","synthetic_46835_46835_2371202_avg50.62885_std27.76569_diagonal_bw7070.97963_skew3.00957_neigh1.6345_cross0.76617_n14.mtx","synthetic_46835_46835_2371250_avg50.62987_std31.37317_diagonal_bw6975.62833_skew3.1675_neigh1.6373_cross0.75463_n14.mtx","synthetic_46835_46835_2371308_avg50.63111_std28.69655_diagonal_bw7047.95307_skew3.1674_neigh1.63521_cross0.76523_n14.mtx","synthetic_46835_46835_2371384_avg50.63273_std27.77434_diagonal_bw7064.33473_skew3.08826_neigh1.63349_cross0.76857_n14.mtx","synthetic_46835_46835_2371440_avg50.63393_std27.7783_diagonal_bw7114.30981_skew3.18692_neigh1.63074_cross0.77345_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_diagonal_bw5050.44975_skew3.12766_neigh1.63384_cross0.7683_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_diagonal_bw5519.37801_skew3.12766_neigh1.63425_cross0.76948_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_diagonal_bw5974.71957_skew3.12766_neigh1.63482_cross0.76832_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_diagonal_bw6289.96966_skew3.12766_neigh1.56424_cross0.88608_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_diagonal_bw6432.35471_skew3.12766_neigh1.63211_cross0.77136_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_diagonal_bw6665.6247_skew3.12766_neigh1.60147_cross0.846_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_diagonal_bw6698.93127_skew3.12766_neigh1.14856_cross0.76971_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_diagonal_bw6757.40436_skew3.12766_neigh1.25689_cross0.76811_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_diagonal_bw6780.66794_skew3.12766_neigh1.36547_cross0.76752_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_diagonal_bw6866.76471_skew3.12766_neigh1.57918_cross0.77036_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_diagonal_bw6870.58687_skew3.12766_neigh1.47124_cross0.76996_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_diagonal_bw6888.78317_skew3.12766_neigh1.63511_cross0.76873_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_diagonal_bw6998.11442_skew3.12766_neigh1.62573_cross0.79459_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_diagonal_bw7163.16152_skew3.12766_neigh1.68835_cross0.77345_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_diagonal_bw7218.82703_skew3.12766_neigh1.6394_cross0.74525_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_diagonal_bw7311.94737_skew3.12766_neigh1.63543_cross0.76939_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_diagonal_bw7364.02791_skew3.12766_neigh1.64601_cross0.69592_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_diagonal_bw7473.56761_skew3.12766_neigh1.65519_cross0.64484_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_diagonal_bw7584.44582_skew3.12766_neigh1.66065_cross0.59521_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_diagonal_bw7668.03327_skew3.12766_neigh1.66479_cross0.54423_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_diagonal_bw7752.28955_skew3.12766_neigh1.63462_cross0.76859_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_diagonal_bw8144.77457_skew3.12766_neigh1.8993_cross0.77798_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_diagonal_bw8228.79031_skew3.12766_neigh1.63299_cross0.77184_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_diagonal_bw8649.96035_skew3.12766_neigh1.63342_cross0.77078_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_diagonal_bw8734.98268_skew3.12766_neigh1.79476_cross0.77224_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_diagonal_bw9093.75454_skew3.12766_neigh1.63511_cross0.76942_n14.mtx","synthetic_46835_46835_2371578_avg50.63687_std27.78811_diagonal_bw7158.8859_skew3.36441_neigh1.63233_cross0.77814_n14.mtx","synthetic_46835_46835_2371669_avg50.63882_std26.85583_diagonal_bw7120.5289_skew3.10752_neigh1.63199_cross0.77811_n14.mtx","synthetic_46835_46835_2371693_avg50.63933_std27.78141_diagonal_bw7136.54039_skew3.26546_neigh1.63148_cross0.77945_n14.mtx","synthetic_46835_46835_2371771_avg50.64099_std27.79721_diagonal_bw7216.51397_skew3.60102_neigh1.63057_cross0.78376_n14.mtx","synthetic_46835_46835_2371874_avg50.64319_std27.79128_diagonal_bw7207.36039_skew3.48234_neigh1.63236_cross0.78304_n14.mtx","synthetic_46835_46835_2371944_avg50.64469_std25.00834_diagonal_bw7255.44428_skew3.02806_neigh1.63083_cross0.78597_n14.mtx","synthetic_46835_46835_2372249_avg50.6512_std23.16377_diagonal_bw7332.95584_skew3.0078_neigh1.63031_cross0.79742_n14.mtx","synthetic_46835_46835_2372391_avg50.65423_std21.31556_diagonal_bw7497.52574_skew2.98782_neigh1.6247_cross0.80892_n14.mtx","synthetic_46835_46835_2372900_avg50.6651_std19.46618_diagonal_bw7594.71372_skew2.96723_neigh1.62451_cross0.818_n14.mtx","synthetic_46835_46835_2450659_avg52.32538_std27.78086_diagonal_bw7165.93817_skew3.1089_neigh1.6353_cross0.77626_n14.mtx","synthetic_46835_46835_2609003_avg55.70627_std27.78687_diagonal_bw7091.41887_skew3.03904_neigh1.63996_cross0.78555_n14.mtx","synthetic_46835_46835_2767329_avg59.08677_std27.79319_diagonal_bw7277.89561_skew3.01105_neigh1.64136_cross0.79423_n14.mtx","synthetic_46835_46835_2925939_avg62.47334_std27.79699_diagonal_bw7235.67689_skew3.00171_neigh1.64415_cross0.80009_n14.mtx","synthetic_46835_46835_3084303_avg65.85466_std27.80201_diagonal_bw7327.42293_skew2.97846_neigh1.64707_cross0.80559_n14.mtx","synthetic_4690002_4690002_14083544_avg3.00289_std1.14359_diagonal_bw5391.75836_skew291.71835_neigh0.36891_cross0.64826_n14.mtx","synthetic_4690002_4690002_18765165_avg4.0011_std0.88465_diagonal_bw4544.61296_skew312.91371_neigh0.41523_cross0.64886_n14.mtx","synthetic_4690002_4690002_18766060_avg4.00129_std0.94875_diagonal_bw4544.61433_skew312.89874_neigh0.41523_cross0.64886_n14.mtx","synthetic_4690002_4690002_18767037_avg4.0015_std1.01413_diagonal_bw4544.62177_skew312.88239_neigh0.41524_cross0.64886_n14.mtx","synthetic_4690002_4690002_18767925_avg4.00169_std1.23629_diagonal_bw4544.6141_skew406.578_neigh0.4153_cross0.64886_n14.mtx","synthetic_4690002_4690002_18768094_avg4.00172_std1.08092_diagonal_bw4544.60917_skew312.86472_neigh0.41525_cross0.64886_n14.mtx","synthetic_4690002_4690002_18768255_avg4.00176_std1.22278_diagonal_bw4544.60003_skew385.58006_neigh0.41528_cross0.64886_n14.mtx","synthetic_4690002_4690002_18768639_avg4.00184_std1.21074_diagonal_bw4544.61468_skew364.83169_neigh0.41531_cross0.64886_n14.mtx","synthetic_4690002_4690002_18769069_avg4.00193_std1.19917_diagonal_bw4544.58175_skew344.08333_neigh0.41529_cross0.64886_n14.mtx","synthetic_4690002_4690002_18769236_avg4.00197_std1.1487_diagonal_bw4544.58612_skew312.84562_neigh0.41528_cross0.64886_n14.mtx","synthetic_4690002_4690002_18769566_avg4.00204_std1.18832_diagonal_bw4544.53443_skew323.33476_neigh0.41529_cross0.64886_n14.mtx","synthetic_4690002_4690002_18769585_avg4.00204_std1.18853_diagonal_bw4544.55293_skew323.33443_neigh0.41529_cross0.64886_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_diagonal_bw3184.88357_skew312.8356_neigh0.41551_cross0.64901_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_diagonal_bw3466.39633_skew312.8356_neigh0.41544_cross0.64895_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_diagonal_bw3795.79694_skew312.8356_neigh0.41543_cross0.64893_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_diagonal_bw4074.65853_skew312.8356_neigh0.41535_cross0.64891_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_diagonal_bw4200.67402_skew312.8356_neigh0.54031_cross0.6488_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_diagonal_bw4279.94263_skew312.8356_neigh0.51216_cross0.6488_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_diagonal_bw4358.28843_skew312.8356_neigh0.48436_cross0.64888_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_diagonal_bw4407.10934_skew312.8356_neigh0.41525_cross0.64888_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_diagonal_bw4433.9589_skew312.8356_neigh0.45655_cross0.64867_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_diagonal_bw4508.13803_skew312.8356_neigh0.42901_cross0.64876_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_diagonal_bw4542.4258_skew312.8356_neigh0.41512_cross0.58418_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_diagonal_bw4543.20029_skew312.8356_neigh0.41608_cross0.84265_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_diagonal_bw4543.24698_skew312.8356_neigh0.41538_cross0.62721_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_diagonal_bw4543.90921_skew312.8356_neigh0.41541_cross0.45424_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_diagonal_bw4544.3722_skew312.8356_neigh0.41496_cross0.67021_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_diagonal_bw4544.551_skew312.8356_neigh0.41603_cross0.79985_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_diagonal_bw4544.99823_skew312.8356_neigh0.415_cross0.54079_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_diagonal_bw4545.19427_skew312.8356_neigh0.4153_cross0.49762_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_diagonal_bw4546.90824_skew312.8356_neigh0.41469_cross0.71357_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_diagonal_bw4547.16626_skew312.8356_neigh0.41515_cross0.75665_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_diagonal_bw4581.4107_skew312.8356_neigh0.40122_cross0.64877_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_diagonal_bw4652.81349_skew312.8356_neigh0.37381_cross0.64869_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_diagonal_bw4685.0085_skew312.8356_neigh0.41522_cross0.64883_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_diagonal_bw4720.29248_skew312.8356_neigh0.3463_cross0.64858_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_diagonal_bw4787.73638_skew312.8356_neigh0.31846_cross0.64869_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_diagonal_bw4854.49698_skew312.8356_neigh0.29083_cross0.64876_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_diagonal_bw5013.47671_skew312.8356_neigh0.41526_cross0.64883_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_diagonal_bw5289.58717_skew312.8356_neigh0.41524_cross0.64878_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_diagonal_bw5622.90244_skew312.8356_neigh0.41522_cross0.64879_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_diagonal_bw5904.00292_skew312.8356_neigh0.41517_cross0.64878_n14.mtx","synthetic_4690002_4690002_18770121_avg4.00216_std1.17768_diagonal_bw4544.61405_skew302.33648_neigh0.41531_cross0.64886_n14.mtx","synthetic_4690002_4690002_18770136_avg4.00216_std1.17836_diagonal_bw4544.58465_skew302.5861_neigh0.41528_cross0.64886_n14.mtx","synthetic_4690002_4690002_18770757_avg4.00229_std1.16774_diagonal_bw4544.58986_skew281.58808_neigh0.41528_cross0.64886_n14.mtx","synthetic_4690002_4690002_18771502_avg4.00245_std1.15845_diagonal_bw4544.58937_skew260.58973_neigh0.41528_cross0.64886_n14.mtx","synthetic_4690002_4690002_18772478_avg4.00266_std1.15153_diagonal_bw4544.58274_skew240.08974_neigh0.41532_cross0.64886_n14.mtx","synthetic_4690002_4690002_18773573_avg4.00289_std1.14442_diagonal_bw4544.5891_skew219.34068_neigh0.4153_cross0.64886_n14.mtx","synthetic_4690002_4690002_18774622_avg4.00312_std1.42675_diagonal_bw4544.59601_skew312.75559_neigh0.41536_cross0.64886_n14.mtx","synthetic_4690002_4690002_18776169_avg4.00345_std1.497_diagonal_bw4544.58594_skew312.72973_neigh0.41537_cross0.64886_n14.mtx","synthetic_4690002_4690002_23458256_avg5.00176_std1.22307_diagonal_bw4355.44656_skew308.6911_neigh0.44325_cross0.64839_n14.mtx","synthetic_4690002_4690002_23458625_avg5.00184_std1.21051_diagonal_bw4355.4414_skew292.09232_neigh0.44326_cross0.64839_n14.mtx","synthetic_4690002_4690002_23459038_avg5.00192_std1.19839_diagonal_bw4355.43174_skew275.29363_neigh0.44326_cross0.64839_n14.mtx","synthetic_4690002_4690002_28147894_avg6.00168_std1.23536_diagonal_bw4776.66629_skew271.09045_neigh0.46196_cross0.64855_n14.mtx","synthetic_473242_473242_1892968_avg4.0_std0.0_diagonal_bw643.09685_skew0.0_neigh0.37802_cross0.99028_n14.mtx","synthetic_47342_47342_5640490_avg119.14347_std31.78033_diagonal_bw4706.12927_skew1.14028_neigh1.79368_cross0.80973_n14.mtx","synthetic_47513_47513_1853007_avg39.0_std0.0_diagonal_bw11035.73908_skew0.0_neigh1.40111_cross0.80716_n14.mtx","synthetic_47684_47684_3704607_avg77.69078_std983.84259_diagonal_bw1484.67635_skew612.76655_neigh1.33477_cross0.83815_n14.mtx","synthetic_47784_47784_2177960_avg45.57927_std31.38054_diagonal_bw1146.70877_skew2.68589_neigh1.21315_cross0.77186_n14.mtx","synthetic_47879_47879_3068257_avg64.08356_std14.01465_diagonal_bw346.42014_skew0.93497_neigh1.583_cross0.83014_n14.mtx","synthetic_48396_48396_2450061_avg50.62528_std27.78261_diagonal_bw7308.68938_skew3.14813_neigh1.63518_cross0.76979_n14.mtx","synthetic_4846335_4846335_19395478_avg4.00209_std1.18059_diagonal_bw4694.1316_skew312.83587_neigh0.41556_cross0.64854_n14.mtx","synthetic_49152_49152_1327104_avg27.0_std0.0_diagonal_bw11307.3231_skew0.0_neigh1.38197_cross0.80419_n14.mtx","synthetic_49152_49152_1474560_avg30.0_std0.0_diagonal_bw11595.80863_skew0.0_neigh1.39239_cross0.80817_n14.mtx","synthetic_49152_49152_1622016_avg33.0_std0.0_diagonal_bw11493.20247_skew0.0_neigh1.39477_cross0.80691_n14.mtx","synthetic_49152_49152_1720320_avg35.0_std0.0_diagonal_bw11394.44574_skew0.0_neigh1.39367_cross0.80647_n14.mtx","synthetic_49152_49152_1867776_avg38.0_std0.0_diagonal_bw11349.18608_skew0.0_neigh1.40287_cross0.8065_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_diagonal_bw10308.18809_skew0.0_neigh1.40356_cross0.80771_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_diagonal_bw11012.95563_skew0.0_neigh1.40324_cross0.80742_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_diagonal_bw11346.284_skew0.0_neigh1.1685_cross0.80302_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_diagonal_bw11390.72874_skew0.0_neigh0.98082_cross0.80701_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_diagonal_bw11391.50515_skew0.0_neigh1.07508_cross0.8087_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_diagonal_bw11399.78432_skew0.0_neigh1.25841_cross0.80863_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_diagonal_bw11414.96842_skew0.0_neigh1.40418_cross0.99013_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_diagonal_bw11418.68166_skew0.0_neigh1.40287_cross0.78307_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_diagonal_bw11422.21698_skew0.0_neigh1.40004_cross0.5727_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_diagonal_bw11424.32371_skew0.0_neigh1.35394_cross0.80634_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_diagonal_bw11425.76263_skew0.0_neigh1.40146_cross0.67683_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_diagonal_bw11434.58527_skew0.0_neigh1.40111_cross0.62362_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_diagonal_bw11435.72972_skew0.0_neigh1.40014_cross0.7295_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_diagonal_bw11437.37392_skew0.0_neigh1.40223_cross0.80454_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_diagonal_bw11439.83738_skew0.0_neigh1.40359_cross0.83214_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_diagonal_bw11448.35966_skew0.0_neigh1.4023_cross0.93723_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_diagonal_bw11458.2605_skew0.0_neigh1.39946_cross0.88672_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_diagonal_bw11460.26939_skew0.0_neigh1.44716_cross0.80489_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_diagonal_bw11518.20732_skew0.0_neigh1.54189_cross0.80766_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_diagonal_bw11741.60107_skew0.0_neigh1.6337_cross0.80807_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_diagonal_bw11774.41903_skew0.0_neigh1.40065_cross0.80951_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_diagonal_bw12457.04529_skew0.0_neigh1.40302_cross0.81002_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_diagonal_bw12492.34208_skew0.0_neigh1.72608_cross0.80667_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_diagonal_bw13195.13298_skew0.0_neigh1.39937_cross0.80881_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_diagonal_bw13899.08816_skew0.0_neigh1.39772_cross0.80832_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_diagonal_bw14638.79781_skew0.0_neigh1.39901_cross0.80937_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_diagonal_bw17395.89585_skew0.0_neigh1.81611_cross0.81026_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_diagonal_bw8111.08537_skew0.0_neigh1.40272_cross0.80591_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_diagonal_bw8847.37626_skew0.0_neigh1.40198_cross0.80857_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_diagonal_bw9576.764_skew0.0_neigh1.40409_cross0.80825_n14.mtx","synthetic_49152_49152_1966080_avg40.0_std0.0_diagonal_bw11481.6866_skew0.0_neigh1.4051_cross0.80332_n14.mtx","synthetic_49152_49152_2113536_avg43.0_std0.0_diagonal_bw11425.25747_skew0.0_neigh1.40405_cross0.80294_n14.mtx","synthetic_49152_49152_2260992_avg46.0_std0.0_diagonal_bw11375.92244_skew0.0_neigh1.407_cross0.80755_n14.mtx","synthetic_49152_49152_2359296_avg48.0_std0.0_diagonal_bw11455.144_skew0.0_neigh1.40962_cross0.8078_n14.mtx","synthetic_49152_49152_2506752_avg51.0_std0.0_diagonal_bw11427.36776_skew0.0_neigh1.41299_cross0.80878_n14.mtx","synthetic_49161_49161_5521770_avg112.32013_std88.82977_diagonal_bw19423.22797_skew3.82549_neigh1.89372_cross0.64143_n14.mtx","synthetic_50367_50367_2296192_avg45.58922_std31.37796_diagonal_bw1208.68088_skew2.68508_neigh1.2113_cross0.76966_n14.mtx","synthetic_50790_50790_1980810_avg39.0_std0.0_diagonal_bw11784.56686_skew0.0_neigh1.40283_cross0.80853_n14.mtx","synthetic_508297_508297_2033188_avg4.0_std0.0_diagonal_bw694.94548_skew0.0_neigh0.36737_cross0.99098_n14.mtx","synthetic_51518_51518_2608832_avg50.63923_std27.762_diagonal_bw7791.12322_skew3.14698_neigh1.63399_cross0.76903_n14.mtx","synthetic_5159002_5159002_20646759_avg4.00208_std1.17597_diagonal_bw4999.05211_skew312.8365_neigh0.41586_cross0.64852_n14.mtx","synthetic_51970_51970_5840112_avg112.37468_std88.844_diagonal_bw20596.85007_skew3.82315_neigh1.89334_cross0.64183_n14.mtx","synthetic_52042_52042_3335826_avg64.09873_std14.0_diagonal_bw375.19427_skew0.93452_neigh1.57868_cross0.8294_n14.mtx","synthetic_52226_52226_4093416_avg78.37889_std1019.1394_diagonal_bw1620.53483_skew665.32736_neigh1.34078_cross0.83814_n14.mtx","synthetic_525825_525825_1577449_avg2.99995_std0.00703_diagonal_bw858.65423_skew2e-05_neigh0.32898_cross0.99111_n14.mtx","synthetic_525825_525825_1577475_avg3.0_std0.0_diagonal_bw858.11898_skew0.0_neigh0.3297_cross0.99112_n14.mtx","synthetic_525825_525825_1583911_avg3.01224_std0.10995_diagonal_bw794.94425_skew0.32792_neigh0.26356_cross0.98852_n14.mtx","synthetic_525825_525825_2046881_avg3.8927_std0.30949_diagonal_bw644.63048_skew0.02756_neigh0.14022_cross0.96875_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_diagonal_bw505.65007_skew0.0_neigh0.37404_cross0.99123_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_diagonal_bw554.71674_skew0.0_neigh0.37192_cross0.99123_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_diagonal_bw606.72183_skew0.0_neigh0.37221_cross0.99122_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_diagonal_bw644.28962_skew0.0_neigh0.37132_cross0.99121_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_diagonal_bw670.65649_skew0.0_neigh0.48388_cross0.99123_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_diagonal_bw683.48363_skew0.0_neigh0.45899_cross0.99121_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_diagonal_bw695.19178_skew0.0_neigh0.43396_cross0.9912_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_diagonal_bw702.15381_skew0.0_neigh0.37156_cross0.99122_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_diagonal_bw705.67664_skew0.0_neigh0.40817_cross0.99123_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_diagonal_bw714.97854_skew0.0_neigh0.38388_cross0.99124_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_diagonal_bw717.19496_skew0.0_neigh0.37708_cross0.95839_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_diagonal_bw718.53995_skew0.0_neigh0.37405_cross0.89258_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_diagonal_bw719.73706_skew0.0_neigh0.37722_cross0.76298_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_diagonal_bw719.84723_skew0.0_neigh0.37147_cross0.99224_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_diagonal_bw720.36711_skew0.0_neigh0.37694_cross0.69678_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_diagonal_bw720.4043_skew0.0_neigh0.37129_cross0.99122_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_diagonal_bw720.62767_skew0.0_neigh0.3775_cross0.8279_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_diagonal_bw726.02105_skew0.0_neigh0.35715_cross0.99121_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_diagonal_bw737.10622_skew0.0_neigh0.37107_cross0.99122_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_diagonal_bw737.13597_skew0.0_neigh0.33298_cross0.9912_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_diagonal_bw744.1915_skew0.0_neigh0.30973_cross0.99122_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_diagonal_bw756.74982_skew0.0_neigh0.28172_cross0.99121_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_diagonal_bw766.27818_skew0.0_neigh0.25988_cross0.99121_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_diagonal_bw792.21338_skew0.0_neigh0.37115_cross0.99122_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_diagonal_bw832.70023_skew0.0_neigh0.371_cross0.9912_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_diagonal_bw890.5806_skew0.0_neigh0.37102_cross0.99121_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_diagonal_bw939.37529_skew0.0_neigh0.37252_cross0.9912_n14.mtx","synthetic_525825_525825_2146618_avg4.08238_std0.27494_diagonal_bw492.86623_skew0.22478_neigh0.18979_cross0.97725_n14.mtx","synthetic_525825_525825_2619767_avg4.9822_std0.13221_diagonal_bw661.36508_skew0.00357_neigh0.31619_cross0.98832_n14.mtx","synthetic_525825_525825_2629125_avg5.0_std0.0_diagonal_bw685.10273_skew0.0_neigh0.39925_cross0.99119_n14.mtx","synthetic_525825_525825_2629143_avg5.00003_std0.00585_diagonal_bw685.04151_skew0.19999_neigh0.39906_cross0.99119_n14.mtx","synthetic_54067_54067_2108613_avg39.0_std0.0_diagonal_bw12526.00168_skew0.0_neigh1.40239_cross0.80938_n14.mtx","synthetic_543352_543352_2173408_avg4.0_std0.0_diagonal_bw737.33837_skew0.0_neigh0.38232_cross0.99144_n14.mtx","synthetic_54640_54640_2767687_avg50.65313_std27.75459_diagonal_bw8285.22183_skew3.14584_neigh1.63356_cross0.77112_n14.mtx","synthetic_5471669_5471669_21898040_avg4.00208_std1.17221_diagonal_bw5301.34866_skew312.83705_neigh0.41566_cross0.6484_n14.mtx","synthetic_54779_54779_6163317_avg112.5124_std88.83086_diagonal_bw21771.99144_skew3.81725_neigh1.89359_cross0.64245_n14.mtx","synthetic_56205_56205_3603711_avg64.11727_std13.99803_diagonal_bw403.30752_skew0.93396_neigh1.57631_cross0.8294_n14.mtx","synthetic_56767_56767_4479663_avg78.91315_std1046.34011_diagonal_bw1754.34908_skew718.36043_neigh1.3421_cross0.83778_n14.mtx","synthetic_57344_57344_2236416_avg39.0_std0.0_diagonal_bw13320.73858_skew0.0_neigh1.39839_cross0.80795_n14.mtx","synthetic_57763_57763_2926741_avg50.66809_std27.74629_diagonal_bw8733.73693_skew3.16436_neigh1.63025_cross0.77273_n14.mtx","synthetic_578407_578407_2313628_avg4.0_std0.0_diagonal_bw786.09292_skew0.0_neigh0.37349_cross0.99188_n14.mtx","synthetic_5784335_5784335_23149312_avg4.00207_std1.16847_diagonal_bw5604.48836_skew312.83761_neigh0.4158_cross0.6484_n14.mtx","synthetic_58333_58333_4204101_avg72.07071_std19.00931_diagonal_bw3296.83076_skew1.13679_neigh1.65316_cross0.77161_n14.mtx","synthetic_60369_60369_3871327_avg64.12773_std13.99319_diagonal_bw432.23492_skew0.93364_neigh1.57393_cross0.82891_n14.mtx","synthetic_603864_603864_13450351_avg22.27381_std30.02405_diagonal_bw145119.2978_skew313.58473_neigh1.25727_cross0.75786_n14.mtx","synthetic_60620_60620_2364180_avg39.0_std0.0_diagonal_bw14073.89657_skew0.0_neigh1.40012_cross0.80617_n14.mtx","synthetic_60885_60885_3084540_avg50.66174_std27.74397_diagonal_bw9209.10073_skew3.16488_neigh1.63202_cross0.77149_n14.mtx","synthetic_6097002_6097002_24400590_avg4.00206_std1.16523_diagonal_bw5909.43124_skew312.83809_neigh0.41578_cross0.64825_n14.mtx","synthetic_61308_61308_4858098_avg79.24085_std1065.00271_diagonal_bw1887.76817_skew772.69186_neigh1.34799_cross0.83808_n14.mtx","synthetic_613462_613462_2453848_avg4.0_std0.0_diagonal_bw842.69066_skew0.0_neigh0.3823_cross0.9922_n14.mtx","synthetic_62451_62451_2802494_avg44.87509_std13.98221_diagonal_bw436.8282_skew1.33983_neigh1.53945_cross0.78907_n14.mtx","synthetic_62451_62451_3069539_avg49.15116_std13.98917_diagonal_bw444.92629_skew1.21765_neigh1.54779_cross0.80067_n14.mtx","synthetic_62451_62451_3336828_avg53.43114_std13.99111_diagonal_bw443.55866_skew1.11487_neigh1.55347_cross0.81011_n14.mtx","synthetic_62451_62451_3603802_avg57.70607_std13.99012_diagonal_bw443.04206_skew1.04485_neigh1.56426_cross0.8182_n14.mtx","synthetic_62451_62451_3870981_avg61.98429_std13.98816_diagonal_bw442.47014_skew0.96824_neigh1.56874_cross0.82542_n14.mtx","synthetic_62451_62451_4003815_avg64.1113_std18.18532_diagonal_bw439.05052_skew1.2149_neigh1.57718_cross0.80166_n14.mtx","synthetic_62451_62451_4004004_avg64.11433_std17.25509_diagonal_bw440.54065_skew1.1524_neigh1.57552_cross0.80816_n14.mtx","synthetic_62451_62451_4004101_avg64.11588_std16.32075_diagonal_bw441.99681_skew1.08997_neigh1.5736_cross0.81519_n14.mtx","synthetic_62451_62451_4004374_avg64.12025_std15.38744_diagonal_bw441.76455_skew1.02744_neigh1.57521_cross0.82106_n14.mtx","synthetic_62451_62451_4004540_avg64.12291_std14.45551_diagonal_bw446.48393_skew0.96498_neigh1.57187_cross0.8254_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_diagonal_bw318.44834_skew0.93375_neigh1.5857_cross0.8319_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_diagonal_bw347.21382_skew0.93375_neigh1.58282_cross0.83047_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_diagonal_bw375.5891_skew0.93375_neigh1.5788_cross0.82967_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_diagonal_bw403.32614_skew0.93375_neigh1.57688_cross0.82914_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_diagonal_bw428.19497_skew0.93375_neigh1.56047_cross0.89117_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_diagonal_bw431.64225_skew0.93375_neigh1.17959_cross0.83144_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_diagonal_bw432.20602_skew0.93375_neigh1.57402_cross0.82868_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_diagonal_bw432.28425_skew0.93375_neigh1.26448_cross0.83122_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_diagonal_bw434.25807_skew0.93375_neigh1.3496_cross0.82903_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_diagonal_bw435.77275_skew0.93375_neigh1.43957_cross0.82958_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_diagonal_bw436.2092_skew0.93375_neigh1.56891_cross0.85105_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_diagonal_bw441.89184_skew0.93375_neigh1.52756_cross0.82773_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_diagonal_bw446.0811_skew0.93375_neigh1.57318_cross0.82708_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_diagonal_bw448.47362_skew0.93375_neigh1.61679_cross0.82621_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_diagonal_bw455.82892_skew0.93375_neigh1.57514_cross0.8052_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_diagonal_bw460.20173_skew0.93375_neigh1.57143_cross0.82713_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_diagonal_bw471.01044_skew0.93375_neigh1.58044_cross0.75754_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_diagonal_bw474.11936_skew0.93375_neigh1.70896_cross0.82572_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_diagonal_bw482.92841_skew0.93375_neigh1.58602_cross0.71177_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_diagonal_bw488.82353_skew0.93375_neigh1.5698_cross0.82576_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_diagonal_bw492.24525_skew0.93375_neigh1.58863_cross0.66769_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_diagonal_bw501.27634_skew0.93375_neigh1.592_cross0.62297_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_diagonal_bw518.7649_skew0.93375_neigh1.56528_cross0.82577_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_diagonal_bw534.51368_skew0.93375_neigh1.80272_cross0.82363_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_diagonal_bw545.14101_skew0.93375_neigh1.56484_cross0.82581_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_diagonal_bw573.04011_skew0.93375_neigh1.56402_cross0.82536_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_diagonal_bw655.89393_skew0.93375_neigh1.90447_cross0.82019_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_diagonal_bw688.45913_skew0.93375_neigh1.89805_cross0.8202_n14.mtx","synthetic_62451_62451_4004768_avg64.12656_std12.59183_diagonal_bw449.42601_skew0.84011_neigh1.56907_cross0.83623_n14.mtx","synthetic_62451_62451_4004839_avg64.1277_std13.52372_diagonal_bw447.77336_skew0.90245_neigh1.56888_cross0.83178_n14.mtx","synthetic_62451_62451_4005045_avg64.131_std11.65916_diagonal_bw451.49143_skew0.77761_neigh1.56649_cross0.83952_n14.mtx","synthetic_62451_62451_4005321_avg64.13542_std10.72592_diagonal_bw453.45016_skew0.71512_neigh1.56305_cross0.84869_n14.mtx","synthetic_62451_62451_4005505_avg64.13836_std9.79513_diagonal_bw456.12735_skew0.65268_neigh1.56103_cross0.85395_n14.mtx","synthetic_62451_62451_4138184_avg66.26289_std13.99067_diagonal_bw443.34518_skew0.90152_neigh1.57487_cross0.83169_n14.mtx","synthetic_62451_62451_4405420_avg70.54202_std13.98985_diagonal_bw444.61993_skew0.85705_neigh1.57908_cross0.83695_n14.mtx","synthetic_62451_62451_4672484_avg74.8184_std13.9889_diagonal_bw448.7849_skew0.80437_neigh1.58559_cross0.8421_n14.mtx","synthetic_62451_62451_4939666_avg79.09667_std13.99063_diagonal_bw451.01291_skew0.75734_neigh1.58874_cross0.84589_n14.mtx","synthetic_62451_62451_5206981_avg83.37706_std13.99111_diagonal_bw451.49187_skew0.7151_neigh1.5926_cross0.8518_n14.mtx","synthetic_63889_63889_4604376_avg72.06837_std18.98718_diagonal_bw3617.55279_skew1.13686_neigh1.65392_cross0.76978_n14.mtx","synthetic_63897_63897_2491983_avg39.0_std0.0_diagonal_bw14829.98433_skew0.0_neigh1.40105_cross0.80703_n14.mtx","synthetic_64532_64532_4137790_avg64.11997_std13.98737_diagonal_bw460.49369_skew0.93387_neigh1.57096_cross0.82708_n14.mtx","synthetic_648517_648517_2594068_avg4.0_std0.0_diagonal_bw886.29581_skew0.0_neigh0.37472_cross0.99258_n14.mtx","synthetic_65850_65850_5226388_avg79.36808_std1075.21227_diagonal_bw2024.76418_skew828.67864_neigh1.34353_cross0.83818_n14.mtx","synthetic_661375_661375_14730847_avg22.27306_std29.96346_diagonal_bw159071.45445_skew313.59526_neigh1.25597_cross0.75815_n14.mtx","synthetic_666542_666542_32559308_avg48.8481_std11.96425_diagonal_bw114344.90623_skew1.12905_neigh1.69253_cross0.80431_n14.mtx","synthetic_68121_68121_3787555_avg55.6004_std1068.24938_diagonal_bw1718.51627_skew856.74559_neigh1.78059_cross0.84932_n14.mtx","synthetic_68121_68121_4147401_avg60.88286_std1069.9982_diagonal_bw1926.13858_skew857.05765_neigh1.62569_cross0.84303_n14.mtx","synthetic_68121_68121_4507272_avg66.16568_std1071.85693_diagonal_bw2010.64137_skew857.28489_neigh1.51921_cross0.84027_n14.mtx","synthetic_68121_68121_4867182_avg71.44907_std1073.8353_diagonal_bw2058.69186_skew857.47165_neigh1.43412_cross0.83891_n14.mtx","synthetic_68121_68121_5227217_avg76.7343_std1075.93952_diagonal_bw2086.64904_skew857.61216_neigh1.36941_cross0.83829_n14.mtx","synthetic_68121_68121_5356033_avg78.62528_std1024.59921_diagonal_bw2111.50808_skew865.40068_neigh1.23427_cross0.83732_n14.mtx","synthetic_68121_68121_5371179_avg78.84762_std1041.23556_diagonal_bw2110.12014_skew862.95755_neigh1.25133_cross0.8377_n14.mtx","synthetic_68121_68121_5385572_avg79.05891_std1056.36091_diagonal_bw2104.37146_skew860.64861_neigh1.27611_cross0.83748_n14.mtx","synthetic_68121_68121_5397801_avg79.23843_std1068.36979_diagonal_bw2101.31551_skew858.6965_neigh1.30166_cross0.83769_n14.mtx","synthetic_68121_68121_5398333_avg79.24624_std1068.28806_diagonal_bw2041.94523_skew601.36298_neigh1.52107_cross0.83991_n14.mtx","synthetic_68121_68121_5400082_avg79.27191_std1070.0369_diagonal_bw2059.09281_skew658.4265_neigh1.46808_cross0.83939_n14.mtx","synthetic_68121_68121_5402340_avg79.30506_std1071.88875_diagonal_bw2075.75188_skew715.38556_neigh1.42245_cross0.83852_n14.mtx","synthetic_68121_68121_5404162_avg79.33181_std1073.85709_diagonal_bw2085.38637_skew772.35942_neigh1.3883_cross0.8379_n14.mtx","synthetic_68121_68121_5405884_avg79.35709_std1075.92117_diagonal_bw2096.53019_skew857.41106_neigh1.32589_cross0.83788_n14.mtx","synthetic_68121_68121_5405960_avg79.3582_std834.61171_diagonal_bw2119.37251_skew857.399_neigh1.16523_cross0.8395_n14.mtx","synthetic_68121_68121_5406317_avg79.36344_std1075.95011_diagonal_bw2092.50296_skew829.23113_neigh1.35417_cross0.83799_n14.mtx","synthetic_68121_68121_5406423_avg79.365_std903.5954_diagonal_bw2114.95495_skew857.32548_neigh1.21021_cross0.83745_n14.mtx","synthetic_68121_68121_5406595_avg79.36752_std972.82101_diagonal_bw2111.91293_skew857.29818_neigh1.25842_cross0.83751_n14.mtx","synthetic_68121_68121_5406754_avg79.36986_std765.8736_diagonal_bw2124.58844_skew857.27294_neigh1.12605_cross0.86678_n14.mtx","synthetic_68121_68121_5407048_avg79.37417_std1042.23736_diagonal_bw2101.43831_skew857.22627_neigh1.31378_cross0.83792_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_diagonal_bw1491.43754_skew857.20278_neigh1.35156_cross0.83798_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_diagonal_bw1624.0197_skew857.20278_neigh1.34962_cross0.8382_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_diagonal_bw1758.02102_skew857.20278_neigh1.3463_cross0.83749_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_diagonal_bw1895.79908_skew857.20278_neigh1.34593_cross0.83806_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_diagonal_bw2025.50456_skew857.20278_neigh1.3433_cross0.83803_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_diagonal_bw2082.54704_skew857.20278_neigh1.34143_cross0.84309_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_diagonal_bw2086.37071_skew857.20278_neigh1.41767_cross0.83887_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_diagonal_bw2086.77817_skew857.20278_neigh1.47784_cross0.83958_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_diagonal_bw2091.20798_skew857.20278_neigh1.38952_cross0.83839_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_diagonal_bw2093.83269_skew857.20278_neigh1.44388_cross0.83897_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_diagonal_bw2093.9416_skew857.20278_neigh1.35558_cross0.83786_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_diagonal_bw2094.95959_skew857.20278_neigh1.29648_cross0.83766_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_diagonal_bw2094.9781_skew857.20278_neigh1.32585_cross0.83766_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_diagonal_bw2095.49089_skew857.20278_neigh1.23838_cross0.83723_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_diagonal_bw2096.11441_skew857.20278_neigh1.27082_cross0.83753_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_diagonal_bw2101.47777_skew857.20278_neigh1.21125_cross0.83652_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_diagonal_bw2157.03003_skew857.20278_neigh1.34041_cross0.83822_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_diagonal_bw2169.50548_skew857.20278_neigh1.34139_cross0.8107_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_diagonal_bw2293.06146_skew857.20278_neigh1.33571_cross0.83819_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_diagonal_bw2300.29781_skew857.20278_neigh1.33945_cross0.75919_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_diagonal_bw2404.30044_skew857.20278_neigh1.34229_cross0.7026_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_diagonal_bw2427.21701_skew857.20278_neigh1.33918_cross0.83764_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_diagonal_bw2485.72726_skew857.20278_neigh1.34463_cross0.6511_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_diagonal_bw2552.48722_skew857.20278_neigh1.34607_cross0.59496_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_diagonal_bw2557.58023_skew857.20278_neigh1.34055_cross0.83779_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_diagonal_bw2689.25582_skew857.20278_neigh1.33474_cross0.83821_n14.mtx","synthetic_68121_68121_5407447_avg79.38003_std1111.80732_diagonal_bw2088.98344_skew857.16294_neigh1.36649_cross0.83852_n14.mtx","synthetic_68121_68121_5407867_avg79.3862_std1181.49691_diagonal_bw2075.59383_skew857.0963_neigh1.43167_cross0.83884_n14.mtx","synthetic_68121_68121_5408409_avg79.39415_std1251.28152_diagonal_bw2053.19145_skew857.0103_neigh1.49371_cross0.83947_n14.mtx","synthetic_68121_68121_5409111_avg79.40446_std1321.14786_diagonal_bw2024.25072_skew856.89895_neigh1.56267_cross0.84086_n14.mtx","synthetic_68121_68121_5409514_avg79.41037_std1391.07361_diagonal_bw1990.12473_skew856.83504_neigh1.64055_cross0.84129_n14.mtx","synthetic_68121_68121_5584961_avg81.98589_std1075.92069_diagonal_bw2103.42705_skew829.88685_neigh1.31388_cross0.83785_n14.mtx","synthetic_68121_68121_5934842_avg87.12206_std1068.36291_diagonal_bw2115.43957_skew780.90298_neigh1.27178_cross0.83756_n14.mtx","synthetic_68121_68121_6280428_avg92.19518_std1056.3285_diagonal_bw2124.28281_skew737.87809_neigh1.23335_cross0.83743_n14.mtx","synthetic_68121_68121_6623921_avg97.23758_std1041.17147_diagonal_bw2129.92795_skew699.5625_neigh1.20508_cross0.83709_n14.mtx","synthetic_68121_68121_6966795_avg102.27089_std1024.4946_diagonal_bw2136.61749_skew665.08399_neigh1.18226_cross0.8378_n14.mtx","synthetic_683572_683572_2734288_avg4.0_std0.0_diagonal_bw924.57479_skew0.0_neigh0.37964_cross0.9929_n14.mtx","synthetic_68696_68696_4403097_avg64.09539_std13.99668_diagonal_bw488.31048_skew0.93462_neigh1.5701_cross0.82639_n14.mtx","synthetic_690692_690692_50143771_avg72.59932_std15.84383_diagonal_bw10332.89385_skew0.96972_neigh1.69604_cross0.81876_n14.mtx","synthetic_69445_69445_5001284_avg72.01791_std19.00309_diagonal_bw3910.75892_skew1.13836_neigh1.65289_cross0.77119_n14.mtx","synthetic_700003_700003_2220092_avg3.17155_std25.65674_diagonal_bw84360.35744_skew1481.87283_neigh0.14269_cross0.90247_n14.mtx","synthetic_70391_70391_5585080_avg79.34367_std1076.52341_diagonal_bw2157.78507_skew858.03517_neigh1.33819_cross0.83827_n14.mtx","synthetic_718886_718886_16011537_avg22.27271_std29.91357_diagonal_bw172575.43963_skew313.60029_neigh1.25733_cross0.76047_n14.mtx","synthetic_72859_72859_4668546_avg64.07645_std14.00516_diagonal_bw518.58045_skew0.93519_neigh1.56556_cross0.82561_n14.mtx","synthetic_730022_730022_35670999_avg48.86291_std11.95289_diagonal_bw125143.89165_skew1.14887_neigh1.69318_cross0.80372_n14.mtx","synthetic_74933_74933_5942603_avg79.30555_std1075.58283_diagonal_bw2295.08486_skew858.44801_neigh1.33914_cross0.83797_n14.mtx","synthetic_75000_75000_5399893_avg71.99857_std19.01052_diagonal_bw4226.28011_skew1.13893_neigh1.65498_cross0.76971_n14.mtx","synthetic_756472_756472_54943929_avg72.63181_std15.80449_diagonal_bw11306.62304_skew1.03767_neigh1.69544_cross0.81974_n14.mtx","synthetic_766670_766670_2431595_avg3.17163_std25.62988_diagonal_bw92517.8455_skew1481.83288_neigh0.14291_cross0.90344_n14.mtx","synthetic_77022_77022_4936326_avg64.08982_std14.01488_diagonal_bw546.71648_skew0.93478_neigh1.56521_cross0.82414_n14.mtx","synthetic_776397_776397_17292444_avg22.27268_std29.86978_diagonal_bw186487.71815_skew313.60063_neigh1.25606_cross0.75904_n14.mtx","synthetic_793502_793502_38791634_avg48.88662_std11.94806_diagonal_bw136182.3121_skew1.39329_neigh1.69181_cross0.80518_n14.mtx","synthetic_79474_79474_6302859_avg79.30718_std1074.75192_diagonal_bw2429.79547_skew858.43036_neigh1.33761_cross0.83764_n14.mtx","synthetic_80556_80556_5802157_avg72.02638_std19.02772_diagonal_bw4548.81679_skew1.13811_neigh1.65123_cross0.77098_n14.mtx","synthetic_81186_81186_5203811_avg64.09739_std14.01954_diagonal_bw573.41229_skew0.93456_neigh1.56394_cross0.82507_n14.mtx","synthetic_822252_822252_59710654_avg72.61844_std15.79674_diagonal_bw12289.27629_skew1.02428_neigh1.69505_cross0.82015_n14.mtx","synthetic_833337_833337_2642605_avg3.17111_std25.60679_diagonal_bw100178.12772_skew1482.07595_neigh0.14558_cross0.90327_n14.mtx","synthetic_83334_83334_4201657_avg50.41948_std18.95017_diagonal_bw4709.6572_skew1.61804_neigh1.63244_cross0.73261_n14.mtx","synthetic_83334_83334_4601152_avg55.21338_std18.98799_diagonal_bw4646.03415_skew1.48128_neigh1.63903_cross0.74386_n14.mtx","synthetic_83334_83334_5001407_avg60.0164_std19.00589_diagonal_bw4595.50832_skew1.36602_neigh1.64466_cross0.7532_n14.mtx","synthetic_83334_83334_5402033_avg64.82388_std19.01381_diagonal_bw4698.8203_skew1.25226_neigh1.64878_cross0.75926_n14.mtx","synthetic_83334_83334_5802662_avg69.63139_std19.01513_diagonal_bw4668.44682_skew1.16856_neigh1.65019_cross0.76698_n14.mtx","synthetic_83334_83334_6001725_avg72.02012_std24.68331_diagonal_bw4597.09978_skew1.47153_neigh1.65763_cross0.74295_n14.mtx","synthetic_83334_83334_6001767_avg72.02063_std23.43414_diagonal_bw4605.04571_skew1.40209_neigh1.65672_cross0.74782_n14.mtx","synthetic_83334_83334_6001894_avg72.02215_std22.17621_diagonal_bw4646.80877_skew1.31873_neigh1.65465_cross0.75393_n14.mtx","synthetic_83334_83334_6002259_avg72.02653_std20.9136_diagonal_bw4656.17762_skew1.24917_neigh1.65449_cross0.76067_n14.mtx","synthetic_83334_83334_6002747_avg72.03239_std19.64906_diagonal_bw4673.90798_skew1.16569_neigh1.65312_cross0.76886_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_diagonal_bw3305.20832_skew1.13786_neigh1.65315_cross0.77231_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_diagonal_bw3620.90018_skew1.13786_neigh1.65349_cross0.77196_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_diagonal_bw3937.58802_skew1.13786_neigh1.65156_cross0.77221_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_diagonal_bw4166.4266_skew1.13786_neigh1.61515_cross0.87118_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_diagonal_bw4233.14868_skew1.13786_neigh1.65482_cross0.76951_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_diagonal_bw4319.26886_skew1.13786_neigh1.63126_cross0.84629_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_diagonal_bw4526.66129_skew1.13786_neigh1.27337_cross0.77072_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_diagonal_bw4530.37209_skew1.13786_neigh1.16582_cross0.76816_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_diagonal_bw4534.10717_skew1.13786_neigh1.38187_cross0.76916_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_diagonal_bw4547.8539_skew1.13786_neigh1.65373_cross0.7709_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_diagonal_bw4579.65029_skew1.13786_neigh1.48964_cross0.77146_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_diagonal_bw4585.62608_skew1.13786_neigh1.65019_cross0.79589_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_diagonal_bw4637.47048_skew1.13786_neigh1.59841_cross0.77139_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_diagonal_bw4689.47064_skew1.13786_neigh1.65378_cross0.77101_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_diagonal_bw4780.79023_skew1.13786_neigh1.65879_cross0.74352_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_diagonal_bw4789.06709_skew1.13786_neigh1.70747_cross0.77176_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_diagonal_bw4857.24227_skew1.13786_neigh1.65353_cross0.76955_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_diagonal_bw4925.35613_skew1.13786_neigh1.6643_cross0.6948_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_diagonal_bw5058.46251_skew1.13786_neigh1.66965_cross0.64539_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_diagonal_bw5147.50469_skew1.13786_neigh1.67236_cross0.59361_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_diagonal_bw5154.62121_skew1.13786_neigh1.65272_cross0.77108_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_diagonal_bw5233.28398_skew1.13786_neigh1.67487_cross0.54727_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_diagonal_bw5476.98466_skew1.13786_neigh1.65336_cross0.76948_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_diagonal_bw5507.02461_skew1.13786_neigh1.81577_cross0.77147_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_diagonal_bw5678.54487_skew1.13786_neigh1.92809_cross0.77038_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_diagonal_bw5771.62843_skew1.13786_neigh1.65209_cross0.77094_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_diagonal_bw5838.52381_skew1.13786_neigh1.92627_cross0.77149_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_diagonal_bw6077.07144_skew1.13786_neigh1.65366_cross0.76991_n14.mtx","synthetic_83334_83334_6003209_avg72.03793_std18.38192_diagonal_bw4707.48488_skew1.09612_neigh1.65203_cross0.77498_n14.mtx","synthetic_83334_83334_6003696_avg72.04378_std17.11472_diagonal_bw4728.66163_skew1.01267_neigh1.65174_cross0.7812_n14.mtx","synthetic_83334_83334_6004305_avg72.05108_std15.84803_diagonal_bw4777.9355_skew0.94307_neigh1.65015_cross0.78619_n14.mtx","synthetic_83334_83334_6004738_avg72.05628_std14.57979_diagonal_bw4790.14851_skew0.87354_neigh1.64947_cross0.79306_n14.mtx","synthetic_83334_83334_6005175_avg72.06152_std13.31259_diagonal_bw4831.11382_skew0.79014_neigh1.64887_cross0.7998_n14.mtx","synthetic_83334_83334_6203256_avg74.43848_std19.01659_diagonal_bw4654.6223_skew1.09569_neigh1.65385_cross0.77339_n14.mtx","synthetic_83334_83334_6603858_avg79.24566_std19.01545_diagonal_bw4699.25025_skew1.03166_neigh1.65768_cross0.78094_n14.mtx","synthetic_83334_83334_7004670_avg84.05537_std19.01725_diagonal_bw4679.09844_skew0.97489_neigh1.65994_cross0.78535_n14.mtx","synthetic_83334_83334_7405486_avg88.86512_std19.01689_diagonal_bw4672.23001_skew0.91301_neigh1.66199_cross0.79152_n14.mtx","synthetic_83334_83334_7806125_avg93.67275_std19.01502_diagonal_bw4727.18715_skew0.86821_neigh1.66453_cross0.79468_n14.mtx","synthetic_833908_833908_18572841_avg22.27205_std29.833_diagonal_bw200221.84865_skew313.60956_neigh1.2563_cross0.76034_n14.mtx","synthetic_84015_84015_6661114_avg79.28482_std1074.00684_diagonal_bw2558.388_skew858.67278_neigh1.33638_cross0.83771_n14.mtx","synthetic_84834_84834_1835279_avg21.63377_std13.7861_diagonal_bw39708.14271_skew4.03842_neigh1.00596_cross0.57165_n14.mtx","synthetic_856982_856982_41861338_avg48.84739_std11.94057_diagonal_bw146875.5352_skew1.3338_neigh1.69178_cross0.8051_n14.mtx","synthetic_86111_86111_6203103_avg72.03613_std19.01524_diagonal_bw4859.13722_skew1.13782_neigh1.65351_cross0.76961_n14.mtx","synthetic_862664_862664_13443097_avg15.58324_std29.57178_diagonal_bw205914.8262_skew313.76132_neigh1.19467_cross0.75677_n14.mtx","synthetic_862664_862664_14783705_avg17.13727_std29.61764_diagonal_bw200544.4475_skew312.46885_neigh1.21062_cross0.75788_n14.mtx","synthetic_862664_862664_15984607_avg18.52935_std29.67134_diagonal_bw204031.16331_skew314.12161_neigh1.21772_cross0.75849_n14.mtx","synthetic_862664_862664_17373620_avg20.1395_std29.72434_diagonal_bw201184.47121_skew312.11605_neigh1.23462_cross0.75917_n14.mtx","synthetic_862664_862664_18524582_avg21.47369_std29.78706_diagonal_bw202931.99997_skew314.40918_neigh1.23555_cross0.75791_n14.mtx","synthetic_862664_862664_19089435_avg22.12847_std21.22646_diagonal_bw207932.34557_skew315.65089_neigh1.26294_cross0.76006_n14.mtx","synthetic_862664_862664_19114072_avg22.15703_std23.12053_diagonal_bw207837.85303_skew315.24275_neigh1.26195_cross0.76014_n14.mtx","synthetic_862664_862664_19142097_avg22.18952_std25.02501_diagonal_bw207559.16437_skew314.77975_neigh1.2614_cross0.76033_n14.mtx","synthetic_862664_862664_19162894_avg22.21362_std30.15004_diagonal_bw207436.8044_skew408.47843_neigh1.26104_cross0.76006_n14.mtx","synthetic_862664_862664_19171989_avg22.22417_std26.93718_diagonal_bw207418.24466_skew314.2874_neigh1.26045_cross0.75991_n14.mtx","synthetic_862664_862664_19173147_avg22.22551_std30.06905_diagonal_bw207404.3214_skew387.38255_neigh1.26037_cross0.75992_n14.mtx","synthetic_862664_862664_19184120_avg22.23823_std29.99101_diagonal_bw207352.46946_skew366.25046_neigh1.25968_cross0.76002_n14.mtx","synthetic_862664_862664_19195719_avg22.25168_std29.91773_diagonal_bw207364.02997_skew345.17619_neigh1.25896_cross0.7599_n14.mtx","synthetic_862664_862664_19200670_avg22.25741_std28.85544_diagonal_bw207358.48533_skew313.81644_neigh1.25804_cross0.75992_n14.mtx","synthetic_862664_862664_19207356_avg22.26516_std29.84926_diagonal_bw207298.49088_skew324.12672_neigh1.25745_cross0.75965_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_diagonal_bw147175.94882_skew313.61244_neigh1.25927_cross0.75819_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_diagonal_bw160722.70732_skew313.61244_neigh1.25886_cross0.75853_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_diagonal_bw174086.80059_skew313.61244_neigh1.2581_cross0.75845_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_diagonal_bw187595.73649_skew313.61244_neigh1.25764_cross0.75905_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_diagonal_bw197107.06742_skew313.61244_neigh1.15768_cross0.98285_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_diagonal_bw200512.30622_skew313.61244_neigh1.25755_cross0.75937_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_diagonal_bw201450.72793_skew313.61244_neigh0.87919_cross0.75907_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_diagonal_bw201648.55655_skew313.61244_neigh0.96351_cross0.75931_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_diagonal_bw202106.02353_skew313.61244_neigh1.04694_cross0.75957_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_diagonal_bw202474.16303_skew313.61244_neigh1.13071_cross0.75965_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_diagonal_bw202922.36711_skew313.61244_neigh1.21508_cross0.75942_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_diagonal_bw203998.92204_skew313.61244_neigh1.29803_cross0.759_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_diagonal_bw204574.41148_skew313.61244_neigh1.23281_cross0.9336_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_diagonal_bw205344.97895_skew313.61244_neigh1.38267_cross0.76047_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_diagonal_bw205907.14271_skew313.61244_neigh1.24839_cross0.88389_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_diagonal_bw206626.31443_skew313.61244_neigh1.2532_cross0.83425_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_diagonal_bw206974.26599_skew313.61244_neigh1.25565_cross0.78408_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_diagonal_bw207389.25109_skew313.61244_neigh1.25761_cross0.73493_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_diagonal_bw207439.23549_skew313.61244_neigh1.46644_cross0.76053_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_diagonal_bw207459.39564_skew313.61244_neigh1.25937_cross0.68571_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_diagonal_bw207488.93911_skew313.61244_neigh1.25981_cross0.63655_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_diagonal_bw207554.94279_skew313.61244_neigh1.26046_cross0.587_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_diagonal_bw207595.91908_skew313.61244_neigh1.2611_cross0.53735_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_diagonal_bw212590.02941_skew313.61244_neigh1.54974_cross0.76249_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_diagonal_bw213723.771_skew313.61244_neigh1.25664_cross0.75921_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_diagonal_bw225205.18654_skew313.61244_neigh1.63328_cross0.76333_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_diagonal_bw226779.75408_skew313.61244_neigh1.25626_cross0.75971_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_diagonal_bw239405.59411_skew313.61244_neigh1.25615_cross0.75997_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_diagonal_bw252280.20232_skew313.61244_neigh1.2553_cross0.76024_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_diagonal_bw264965.46748_skew313.61244_neigh1.25507_cross0.76101_n14.mtx","synthetic_862664_862664_19218664_avg22.27827_std29.78506_diagonal_bw207102.49027_skew303.10795_neigh1.25635_cross0.75926_n14.mtx","synthetic_862664_862664_19223712_avg22.28412_std30.77861_diagonal_bw207046.68591_skew313.4391_neigh1.25463_cross0.75893_n14.mtx","synthetic_862664_862664_19229090_avg22.29036_std29.72467_diagonal_bw206867.28901_skew282.12689_neigh1.25356_cross0.759_n14.mtx","synthetic_862664_862664_19237224_avg22.29979_std29.6692_diagonal_bw206653.68064_skew261.15496_neigh1.25095_cross0.7591_n14.mtx","synthetic_862664_862664_19238563_avg22.30134_std32.70575_diagonal_bw206629.43331_skew313.19637_neigh1.25007_cross0.75903_n14.mtx","synthetic_862664_862664_19242590_avg22.30601_std29.61737_diagonal_bw206307.07674_skew240.28029_neigh1.2466_cross0.75831_n14.mtx","synthetic_862664_862664_19243461_avg22.30702_std38.50483_diagonal_bw204657.66387_skew313.1164_neigh1.23352_cross0.75708_n14.mtx","synthetic_862664_862664_19244915_avg22.3087_std29.57096_diagonal_bw205734.95503_skew219.45208_neigh1.24265_cross0.75782_n14.mtx","synthetic_862664_862664_19245171_avg22.309_std34.6357_diagonal_bw205994.25147_skew313.08849_neigh1.24488_cross0.75815_n14.mtx","synthetic_862664_862664_19245832_avg22.30977_std36.57002_diagonal_bw205514.46748_skew313.0777_neigh1.23854_cross0.7575_n14.mtx","synthetic_862664_862664_19963054_avg23.14117_std29.84969_diagonal_bw201699.12835_skew311.90553_neigh1.2515_cross0.75854_n14.mtx","synthetic_862664_862664_21064571_avg24.41805_std29.92055_diagonal_bw202624.20371_skew314.66815_neigh1.24936_cross0.75798_n14.mtx","synthetic_862664_862664_22549917_avg26.13986_std29.9915_diagonal_bw201802.77435_skew311.74076_neigh1.26527_cross0.75886_n14.mtx","synthetic_862664_862664_23605495_avg27.36349_std30.07249_diagonal_bw202516.58382_skew314.82232_neigh1.2598_cross0.75787_n14.mtx","synthetic_862664_862664_25134553_avg29.13597_std30.15077_diagonal_bw202078.38456_skew311.6376_neigh1.27289_cross0.7586_n14.mtx","synthetic_88557_88557_7019496_avg79.26529_std1073.34158_diagonal_bw2695.50047_skew858.88461_neigh1.33448_cross0.83815_n14.mtx","synthetic_888032_888032_64506105_avg72.63939_std15.82947_diagonal_bw13252.2913_skew1.00993_neigh1.69563_cross0.81974_n14.mtx","synthetic_891419_891419_19853723_avg22.27204_std29.80105_diagonal_bw214143.5371_skew313.60965_neigh1.25649_cross0.76024_n14.mtx","synthetic_900004_900004_2853888_avg3.17097_std25.58712_diagonal_bw108539.4849_skew1482.14118_neigh0.14041_cross0.90359_n14.mtx","synthetic_91667_91667_6603697_avg72.04007_std19.01165_diagonal_bw5166.06684_skew1.19322_neigh1.65306_cross0.77077_n14.mtx","synthetic_920462_920462_44975685_avg48.86208_std11.94475_diagonal_bw157813.69929_skew1.2103_neigh1.69158_cross0.8057_n14.mtx","synthetic_92913_92913_2009995_avg21.63309_std13.78454_diagonal_bw43437.75323_skew4.03858_neigh1.00444_cross0.57612_n14.mtx","synthetic_948930_948930_21133875_avg22.27127_std29.77202_diagonal_bw227994.28681_skew313.6206_neigh1.25605_cross0.76054_n14.mtx","synthetic_952203_952203_32554296_avg34.1884_std11.93673_diagonal_bw195297.92349_skew1.69097_neigh1.66003_cross0.77069_n14.mtx","synthetic_952203_952203_35650826_avg37.44036_std11.95176_diagonal_bw168585.46646_skew1.53737_neigh1.6689_cross0.78029_n14.mtx","synthetic_952203_952203_38750463_avg40.69559_std11.9572_diagonal_bw172420.33981_skew1.4327_neigh1.67668_cross0.78884_n14.mtx","synthetic_952203_952203_41851500_avg43.95229_std11.95993_diagonal_bw175566.75547_skew1.3207_neigh1.68288_cross0.79655_n14.mtx","synthetic_952203_952203_44952640_avg47.20909_std11.96013_diagonal_bw161847.45785_skew1.22415_neigh1.68913_cross0.80243_n14.mtx","synthetic_952203_952203_46500626_avg48.83478_std13.95044_diagonal_bw161361.82335_skew1.37536_neigh1.696_cross0.7901_n14.mtx","synthetic_952203_952203_46500963_avg48.83514_std14.74428_diagonal_bw160687.29213_skew1.45725_neigh1.69741_cross0.78406_n14.mtx","synthetic_952203_952203_46501138_avg48.83532_std15.536_diagonal_bw160058.71086_skew1.53915_neigh1.6982_cross0.77843_n14.mtx","synthetic_4690002_4690002_14082483_avg3.00266_std1.15153_diagonal_bw5391.75085_skew319.7156_neigh0.3689_cross0.64826_n14.mtx","synthetic_4690002_4690002_18770459_avg4.00223_std1.21745_diagonal_bw4544.58663_skew312.82517_neigh0.41529_cross0.64886_n14.mtx","synthetic_4690002_4690002_18770794_avg4.0023_std1.16864_diagonal_bw4544.54744_skew281.83738_neigh0.4153_cross0.64886_n14.mtx","synthetic_4690002_4690002_18771567_avg4.00246_std1.15985_diagonal_bw4544.56125_skew260.83867_neigh0.4153_cross0.64886_n14.mtx","synthetic_4690002_4690002_18771765_avg4.00251_std1.28647_diagonal_bw4544.58181_skew312.80334_neigh0.41533_cross0.64886_n14.mtx","synthetic_4690002_4690002_18773155_avg4.0028_std1.35648_diagonal_bw4544.56842_skew312.7801_neigh0.41535_cross0.64886_n14.mtx","synthetic_952203_952203_46501138_avg48.83532_std15.536_diagonal_bw160058.71086_skew1.53915_neigh1.6982_cross0.77843_n14.mtx","synthetic_952203_952203_46502122_avg48.83635_std13.15524_diagonal_bw162100.43586_skew1.31385_neigh1.69445_cross0.79612_n14.mtx","synthetic_952203_952203_46502884_avg48.83715_std12.35905_diagonal_bw162891.6186_skew1.23191_neigh1.69248_cross0.80224_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_diagonal_bw116106.75277_skew1.19093_neigh1.69117_cross0.80488_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_diagonal_bw126698.85453_skew1.19093_neigh1.69122_cross0.80482_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_diagonal_bw137306.6029_skew1.19093_neigh1.69135_cross0.80527_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_diagonal_bw143715.18864_skew1.19093_neigh1.18395_cross0.80203_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_diagonal_bw144653.44632_skew1.19093_neigh1.29656_cross0.80259_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_diagonal_bw146143.79431_skew1.19093_neigh1.40968_cross0.80337_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_diagonal_bw147733.42467_skew1.19093_neigh1.69116_cross0.8053_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_diagonal_bw149667.71742_skew1.19093_neigh1.52206_cross0.80376_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_diagonal_bw150291.80563_skew1.19093_neigh1.61648_cross0.88662_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_diagonal_bw150742.69167_skew1.19093_neigh1.62479_cross0.88374_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_diagonal_bw158141.44775_skew1.19093_neigh1.69161_cross0.80521_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_diagonal_bw158785.70138_skew1.19093_neigh1.63527_cross0.80486_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_diagonal_bw159871.29872_skew1.19093_neigh1.68017_cross0.83168_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_diagonal_bw163210.79241_skew1.19093_neigh1.69195_cross0.80513_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_diagonal_bw165912.04361_skew1.19093_neigh1.70043_cross0.77937_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_diagonal_bw168260.36222_skew1.19093_neigh1.69196_cross0.80515_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_diagonal_bw170143.42422_skew1.19093_neigh1.71296_cross0.72708_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_diagonal_bw170449.52862_skew1.19093_neigh1.74831_cross0.80609_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_diagonal_bw173244.29895_skew1.19093_neigh1.72242_cross0.67391_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_diagonal_bw175577.55656_skew1.19093_neigh1.72872_cross0.62205_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_diagonal_bw177436.06958_skew1.19093_neigh1.73407_cross0.56915_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_diagonal_bw178609.376_skew1.19093_neigh1.69178_cross0.80552_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_diagonal_bw188630.00924_skew1.19093_neigh1.69167_cross0.80566_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_diagonal_bw196253.24669_skew1.19093_neigh1.88306_cross0.80917_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_diagonal_bw198626.45229_skew1.19093_neigh1.6924_cross0.80549_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_diagonal_bw208660.18746_skew1.19093_neigh1.69223_cross0.80594_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_diagonal_bw224378.247_skew1.19093_neigh1.86131_cross0.80899_n14.mtx","synthetic_952203_952203_46504635_avg48.83899_std11.56217_diagonal_bw163741.45383_skew1.14992_neigh1.6908_cross0.80838_n14.mtx","synthetic_952203_952203_46504874_avg48.83924_std10.76526_diagonal_bw164534.32716_skew1.06801_neigh1.68826_cross0.81515_n14.mtx","synthetic_952203_952203_46506617_avg48.84107_std9.96939_diagonal_bw165495.48128_skew0.98603_neigh1.68708_cross0.82079_n14.mtx","synthetic_952203_952203_46507502_avg48.842_std9.17223_diagonal_bw166478.11101_skew0.9041_neigh1.68492_cross0.82769_n14.mtx","synthetic_952203_952203_46508902_avg48.84347_std8.37556_diagonal_bw167919.97397_skew0.82215_neigh1.68255_cross0.83397_n14.mtx","synthetic_952203_952203_48054277_avg50.46642_std11.96125_diagonal_bw164498.00225_skew1.14004_neigh1.69477_cross0.8085_n14.mtx","synthetic_952203_952203_51155719_avg53.72354_std11.96081_diagonal_bw166947.78914_skew1.08475_neigh1.69857_cross0.81333_n14.mtx","synthetic_952203_952203_54257059_avg56.98056_std11.96124_diagonal_bw158641.62561_skew1.01823_neigh1.70257_cross0.81769_n14.mtx","synthetic_952203_952203_57358241_avg60.23741_std11.96072_diagonal_bw160780.13898_skew0.95892_neigh1.70685_cross0.82181_n14.mtx","synthetic_952203_952203_60460102_avg63.49497_std11.9616_diagonal_bw162722.07884_skew0.90566_neigh1.71034_cross0.82524_n14.mtx","synthetic_953812_953812_69251992_avg72.60549_std15.82731_diagonal_bw14247.53739_skew1.05219_neigh1.69481_cross0.82031_n14.mtx","synthetic_966671_966671_3064966_avg3.17064_std25.57082_diagonal_bw116665.94473_skew1482.29662_neigh0.14335_cross0.9042_n14.mtx","synthetic_97223_97223_7006972_avg72.07114_std19.01384_diagonal_bw5487.10435_skew1.19228_neigh1.65371_cross0.7705_n14.mtx","synthetic_983943_983943_48052837_avg48.83701_std11.95054_diagonal_bw168639.17559_skew1.12953_neigh1.69189_cross0.80596_n14.mtx","synthetic_98611_98611_5466825_avg55.43829_std11.03861_diagonal_bw3819.4861_skew0.894_neigh1.63102_cross0.78874_n14.mtx","synthetic_986703_986703_50142919_avg50.81865_std15.80386_diagonal_bw14822.43122_skew1.44005_neigh1.67248_cross0.78706_n14.mtx","synthetic_986703_986703_54918080_avg55.65817_std15.81077_diagonal_bw15138.82145_skew1.31772_neigh1.67905_cross0.79644_n14.mtx","synthetic_986703_986703_59694812_avg60.49927_std15.81312_diagonal_bw14664.0448_skew1.2149_neigh1.68455_cross0.80453_n14.mtx","synthetic_986703_986703_64472911_avg65.34176_std15.81391_diagonal_bw14414.58333_skew1.11197_neigh1.68893_cross0.81154_n14.mtx","synthetic_986703_986703_69250817_avg70.18405_std15.81417_diagonal_bw14641.24259_skew1.0375_neigh1.69312_cross0.81698_n14.mtx","synthetic_986703_986703_71632861_avg72.5982_std20.55272_diagonal_bw14466.90673_skew1.31411_neigh1.70018_cross0.79544_n14.mtx","synthetic_986703_986703_71633918_avg72.59927_std19.50045_diagonal_bw14532.04367_skew1.2452_neigh1.69963_cross0.80057_n14.mtx","synthetic_986703_986703_71635339_avg72.60071_std18.448_diagonal_bw14581.49631_skew1.17629_neigh1.69833_cross0.80629_n14.mtx","synthetic_986703_986703_71637178_avg72.60257_std17.39427_diagonal_bw14634.97505_skew1.10736_neigh1.69727_cross0.81167_n14.mtx","synthetic_986703_986703_71638484_avg72.6039_std16.3415_diagonal_bw14690.20318_skew1.03846_neigh1.69598_cross0.81731_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_diagonal_bw10335.21324_skew1.0109_neigh1.69533_cross0.82034_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_diagonal_bw11318.36353_skew1.0109_neigh1.69541_cross0.82033_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_diagonal_bw12275.00965_skew1.0109_neigh1.69568_cross0.82032_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_diagonal_bw13203.20771_skew1.0109_neigh1.63897_cross0.89577_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_diagonal_bw13249.95843_skew1.0109_neigh1.69575_cross0.82075_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_diagonal_bw13805.02825_skew1.0109_neigh1.19003_cross0.81894_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_diagonal_bw13844.57814_skew1.0109_neigh1.30228_cross0.81902_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_diagonal_bw13882.13942_skew1.0109_neigh1.41423_cross0.81952_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_diagonal_bw13951.55398_skew1.0109_neigh1.52662_cross0.82014_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_diagonal_bw14238.03579_skew1.0109_neigh1.69538_cross0.82019_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_diagonal_bw14278.82315_skew1.0109_neigh1.68517_cross0.84736_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_diagonal_bw14385.45717_skew1.0109_neigh1.63958_cross0.82034_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_diagonal_bw14735.70059_skew1.0109_neigh1.69514_cross0.82041_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_diagonal_bw15076.26174_skew1.0109_neigh1.70327_cross0.79318_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_diagonal_bw15212.0391_skew1.0109_neigh1.6958_cross0.82068_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_diagonal_bw15394.12525_skew1.0109_neigh1.75112_cross0.82054_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_diagonal_bw15634.92651_skew1.0109_neigh1.71345_cross0.73894_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_diagonal_bw16040.9703_skew1.0109_neigh1.72027_cross0.68529_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_diagonal_bw16190.01108_skew1.0109_neigh1.69515_cross0.82012_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_diagonal_bw16364.90593_skew1.0109_neigh1.72531_cross0.63012_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_diagonal_bw16621.34957_skew1.0109_neigh1.72913_cross0.57601_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_diagonal_bw17168.96391_skew1.0109_neigh1.69549_cross0.82015_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_diagonal_bw18143.13335_skew1.0109_neigh1.69549_cross0.82027_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_diagonal_bw19086.21946_skew1.0109_neigh1.69561_cross0.82051_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_diagonal_bw20302.1187_skew1.0109_neigh1.91272_cross0.82094_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_diagonal_bw27082.55582_skew1.0109_neigh1.86325_cross0.82124_n14.mtx","synthetic_986703_986703_71640715_avg72.60616_std15.28673_diagonal_bw14762.27652_skew0.96953_neigh1.69488_cross0.82275_n14.mtx","synthetic_986703_986703_71642304_avg72.60777_std14.23366_diagonal_bw14828.49584_skew0.90062_neigh1.6931_cross0.829_n14.mtx","synthetic_986703_986703_71644194_avg72.60968_std13.17962_diagonal_bw14914.32275_skew0.84548_neigh1.6914_cross0.83495_n14.mtx","synthetic_986703_986703_71646204_avg72.61172_std12.12567_diagonal_bw15020.25928_skew0.77657_neigh1.68947_cross0.84053_n14.mtx","synthetic_986703_986703_71647231_avg72.61276_std11.07215_diagonal_bw15141.16106_skew0.70769_neigh1.68759_cross0.84656_n14.mtx","synthetic_986703_986703_74029078_avg75.02671_std15.81428_diagonal_bw14414.50158_skew0.97263_neigh1.69708_cross0.82306_n14.mtx","synthetic_986703_986703_78806613_avg79.86863_std15.81421_diagonal_bw14604.1321_skew0.91565_neigh1.70019_cross0.82763_n14.mtx","synthetic_986703_986703_83583783_avg84.71017_std15.81435_diagonal_bw14458.40041_skew0.86518_neigh1.70355_cross0.83196_n14.mtx","synthetic_986703_986703_88361224_avg89.552_std15.81425_diagonal_bw14614.23314_skew0.82017_neigh1.70575_cross0.83603_n14.mtx","synthetic_986703_986703_93139124_avg94.39428_std15.81414_diagonal_bw14500.66288_skew0.76917_neigh1.70841_cross0.8393_n14.mtx"]
for m in mikri:
    if m not in megali:
        print(m)







pprint(Counter(["synthetic_1000005_1000005_2190550_avg2.19054_std25.45314_random_bw119819.1947_skew1501.82644_neigh0.0864_cross0.84356_n14.mtx","synthetic_1000005_1000005_2378888_avg2.37888_std25.47359_random_bw127104.78039_skew1514.4215_neigh0.103_cross0.88018_n14.mtx","synthetic_1000005_1000005_2551748_avg2.55174_std25.499_random_bw119896.54265_skew1534.81764_neigh0.1086_cross0.87541_n14.mtx","synthetic_1000005_1000005_2793946_avg2.79393_std25.52481_random_bw139709.49965_skew1513.71115_neigh0.10178_cross0.85786_n14.mtx","synthetic_1000005_1000005_3068666_avg3.06865_std25.54963_random_bw116338.41941_skew1480.43289_neigh0.11916_cross0.88158_n14.mtx","synthetic_1000005_1000005_3110917_avg3.1109_std33.11754_random_bw103532.18733_skew1510.459_neigh0.11814_cross0.85466_n14.mtx","synthetic_1000005_1000005_3116965_avg3.11695_std31.43703_random_bw107163.38446_skew1507.52625_neigh0.11804_cross0.86266_n14.mtx","synthetic_1000005_1000005_3121903_avg3.12189_std25.45302_random_bw110119.5356_skew1053.81063_neigh0.11769_cross0.86713_n14.mtx","synthetic_1000005_1000005_3127972_avg3.12796_std29.75772_random_bw112018.04327_skew1502.5376_neigh0.12161_cross0.87113_n14.mtx","synthetic_1000005_1000005_3132968_avg3.13295_std25.47429_random_bw113902.70335_skew1150.31021_neigh0.12143_cross0.87472_n14.mtx","synthetic_1000005_1000005_3136144_avg3.13613_std18.05255_random_bw135367.82089_skew1498.61976_neigh0.21685_cross0.93223_n14.mtx","synthetic_1000005_1000005_3144021_avg3.14401_std28.07823_random_bw116953.11281_skew1494.86263_neigh0.12731_cross0.88232_n14.mtx","synthetic_1000005_1000005_3144795_avg3.14478_std25.49712_random_bw117373.02426_skew1245.51038_neigh0.12684_cross0.88308_n14.mtx","synthetic_1000005_1000005_3156371_avg3.15636_std25.52223_random_bw120795.31056_skew1340.10381_neigh0.13144_cross0.89113_n14.mtx","synthetic_1000005_1000005_3160475_avg3.16046_std19.71515_random_bw135013.30133_skew1487.0749_neigh0.21383_cross0.93153_n14.mtx","synthetic_1000005_1000005_3162276_avg3.16226_std26.40103_random_bw122511.6205_skew1486.2274_neigh0.13472_cross0.89571_n14.mtx","synthetic_1000005_1000005_3166512_avg3.1665_std25.54915_random_bw123598.70118_skew1434.65625_neigh0.13874_cross0.8991_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_random_bw104073.79051_skew1482.18441_neigh0.14395_cross0.90267_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_random_bw112547.15199_skew1482.18441_neigh0.14364_cross0.90269_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_random_bw120556.18558_skew1482.18441_neigh0.14326_cross0.90269_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_random_bw121722.82145_skew1482.18441_neigh0.1869_cross0.90333_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_random_bw122150.26353_skew1482.18441_neigh0.05805_cross0.96156_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_random_bw122426.6617_skew1482.18441_neigh0.17652_cross0.90307_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_random_bw123138.58643_skew1482.18441_neigh0.16628_cross0.90292_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_random_bw123297.83473_skew1482.18441_neigh0.11813_cross0.93173_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_random_bw123697.2713_skew1482.18441_neigh0.1568_cross0.90295_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_random_bw124260.11141_skew1482.18441_neigh0.14853_cross0.90276_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_random_bw124950.43144_skew1482.18441_neigh0.13931_cross0.90257_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_random_bw125531.54795_skew1482.18441_neigh0.15898_cross0.87296_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_random_bw125571.10503_skew1482.18441_neigh0.12939_cross0.90242_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_random_bw126188.03769_skew1482.18441_neigh0.11966_cross0.90226_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_random_bw126648.45731_skew1482.18441_neigh0.11069_cross0.90206_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_random_bw127142.75789_skew1482.18441_neigh0.17733_cross0.81362_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_random_bw127147.10169_skew1482.18441_neigh0.10042_cross0.90174_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_random_bw128104.19842_skew1482.18441_neigh0.18795_cross0.75388_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_random_bw128465.7325_skew1482.18441_neigh0.19495_cross0.69438_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_random_bw128711.29975_skew1482.18441_neigh0.20001_cross0.63303_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_random_bw128912.58669_skew1482.18441_neigh0.14293_cross0.90268_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_random_bw137896.76343_skew1482.18441_neigh0.14246_cross0.90269_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_random_bw145843.60046_skew1482.18441_neigh0.14298_cross0.90268_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_random_bw154831.75347_skew1482.18441_neigh0.14358_cross0.90267_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_random_bw162157.8696_skew1482.18441_neigh0.14347_cross0.90267_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_random_bw87649.18931_skew1482.18441_neigh0.14287_cross0.90272_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_random_bw95510.6301_skew1482.18441_neigh0.14423_cross0.9027_n14.mtx","synthetic_1000005_1000005_3174621_avg3.17461_std25.57736_random_bw125956.24319_skew1529.58406_neigh0.14718_cross0.90575_n14.mtx","synthetic_1000005_1000005_3177914_avg3.1779_std24.72495_random_bw127016.47781_skew1478.909_neigh0.15361_cross0.90914_n14.mtx","synthetic_1000005_1000005_3179023_avg3.17901_std21.38191_random_bw134207.95085_skew1478.39273_neigh0.19999_cross0.92826_n14.mtx","synthetic_1000005_1000005_3180578_avg3.18056_std25.60874_random_bw128092.87982_skew1625.12766_neigh0.15702_cross0.91168_n14.mtx","synthetic_1000005_1000005_3184160_avg3.18414_std25.64113_random_bw130081.20538_skew1721.91199_neigh0.16859_cross0.9164_n14.mtx","synthetic_1000005_1000005_3185020_avg3.185_std23.052_random_bw131609.81549_skew1475.60722_neigh0.17602_cross0.92056_n14.mtx","synthetic_1000005_1000005_3185310_avg3.18529_std25.71304_random_bw132686.94329_skew1917.81812_neigh0.18507_cross0.92381_n14.mtx","synthetic_1000005_1000005_3185554_avg3.18554_std25.67645_random_bw131621.35039_skew1819.4146_neigh0.17691_cross0.9205_n14.mtx","synthetic_1000005_1000005_3238430_avg3.23841_std25.57761_random_bw128487.05961_skew1499.42591_neigh0.17003_cross0.91626_n14.mtx","synthetic_1000005_1000005_3338044_avg3.33803_std25.60993_random_bw120102.95798_skew1548.7177_neigh0.17012_cross0.91371_n14.mtx","synthetic_1000005_1000005_3554394_avg3.55438_std25.64567_random_bw113500.84038_skew1542.73078_neigh0.13151_cross0.88485_n14.mtx","synthetic_1000005_1000005_3911756_avg3.91174_std25.67945_random_bw133527.84178_skew1481.71748_neigh0.13986_cross0.88512_n14.mtx","synthetic_1000005_1000005_4154204_avg4.15418_std25.71363_random_bw158208.83055_skew1470.76946_neigh0.19591_cross0.92055_n14.mtx","synthetic_100993_100993_2185846_avg21.64354_std13.77891_random_bw54912.02009_skew4.03614_neigh1.01465_cross0.56058_n14.mtx","synthetic_1033338_1033338_3275830_avg3.17014_std25.55637_random_bw129053.64703_skew1482.52894_neigh0.14465_cross0.90234_n14.mtx","synthetic_109072_109072_2360564_avg21.64225_std13.77936_random_bw59445.13864_skew4.08265_neigh1.0157_cross0.55937_n14.mtx","synthetic_1100005_1100005_3487516_avg3.17045_std25.54298_random_bw137079.35531_skew1482.38345_neigh0.14227_cross0.90274_n14.mtx","synthetic_1166672_1166672_3699263_avg3.17078_std25.53274_random_bw145853.05163_skew1482.23015_neigh0.14123_cross0.90291_n14.mtx","synthetic_117152_117152_2535420_avg21.64214_std13.77711_random_bw63786.53108_skew4.08268_neigh1.01456_cross0.56071_n14.mtx","synthetic_119698_119698_670830_avg5.60435_std4.47718_random_bw38463.99053_skew62.8789_neigh0.60838_cross0.60967_n14.mtx","synthetic_121192_121192_1836052_avg15.14994_std12.36381_random_bw63604.68267_skew3.8185_neigh0.98085_cross0.56421_n14.mtx","synthetic_121192_121192_2010928_avg16.59291_std12.82631_random_bw63387.43514_skew3.94187_neigh0.99258_cross0.55956_n14.mtx","synthetic_121192_121192_2185768_avg18.03558_std13.23353_random_bw65140.22976_skew4.04558_neigh0.99835_cross0.55476_n14.mtx","synthetic_121192_121192_2360629_avg19.47842_std13.60481_random_bw64734.37292_skew4.13389_neigh1.00738_cross0.55096_n14.mtx","synthetic_121192_121192_2535413_avg20.92063_std13.77469_random_bw66438.45794_skew4.11457_neigh1.01319_cross0.55608_n14.mtx","synthetic_121192_121192_2622629_avg21.64028_std13.77138_random_bw65424.26549_skew3.85206_neigh1.01668_cross0.55626_n14.mtx","synthetic_121192_121192_2622663_avg21.64056_std13.63311_random_bw64870.55905_skew3.52853_neigh1.01573_cross0.55103_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_random_bw46185.35154_skew4.08299_neigh1.01504_cross0.5599_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_random_bw50573.38166_skew4.08299_neigh1.01509_cross0.56075_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_random_bw54972.82661_skew4.08299_neigh1.01519_cross0.5598_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_random_bw59411.92528_skew4.08299_neigh1.01561_cross0.56068_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_random_bw61692.98851_skew4.08299_neigh0.98464_cross0.72926_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_random_bw62953.41461_skew4.08299_neigh0.99544_cross0.69172_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_random_bw63724.66045_skew4.08299_neigh1.0157_cross0.55944_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_random_bw63863.58232_skew4.08299_neigh1.00127_cross0.65566_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_random_bw64806.59055_skew4.08299_neigh1.00789_cross0.61815_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_random_bw65523.57715_skew4.08299_neigh1.01329_cross0.58066_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_random_bw65666.88089_skew4.08299_neigh0.77995_cross0.56148_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_random_bw65781.74026_skew4.08299_neigh1.11633_cross0.56174_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_random_bw65912.90063_skew4.08299_neigh0.8475_cross0.55891_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_random_bw66094.34436_skew4.08299_neigh0.91339_cross0.55811_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_random_bw66297.82562_skew4.08299_neigh1.01673_cross0.5433_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_random_bw66303.01327_skew4.08299_neigh0.71059_cross0.55994_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_random_bw66344.93551_skew4.08299_neigh1.18402_cross0.56026_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_random_bw66632.5966_skew4.08299_neigh0.98135_cross0.55939_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_random_bw66915.08177_skew4.08299_neigh1.04961_cross0.5616_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_random_bw66999.72835_skew4.08299_neigh1.02006_cross0.50617_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_random_bw67029.04955_skew4.08299_neigh1.25119_cross0.56135_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_random_bw67482.8569_skew4.08299_neigh1.02304_cross0.46991_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_random_bw68050.39675_skew4.08299_neigh1.02635_cross0.4318_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_random_bw68172.92334_skew4.08299_neigh1.01556_cross0.56046_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_random_bw68534.78082_skew4.08299_neigh1.31693_cross0.56196_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_random_bw68594.96245_skew4.08299_neigh1.02897_cross0.39354_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_random_bw72555.87469_skew4.08299_neigh1.0152_cross0.56039_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_random_bw76889.20434_skew4.08299_neigh1.01471_cross0.5605_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_random_bw81259.52524_skew4.08299_neigh1.01612_cross0.56016_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_random_bw85714.50154_skew4.08299_neigh1.01527_cross0.56045_n14.mtx","synthetic_121192_121192_2622724_avg21.64107_std13.77599_random_bw65800.92968_skew3.99051_neigh1.01427_cross0.5606_n14.mtx","synthetic_121192_121192_2622736_avg21.64116_std14.23425_random_bw65388.53812_skew4.08291_neigh1.01644_cross0.55562_n14.mtx","synthetic_121192_121192_2622750_avg21.64128_std13.768_random_bw65046.54528_skew3.71321_neigh1.01557_cross0.55052_n14.mtx","synthetic_121192_121192_2622777_avg21.6415_std15.59481_random_bw64466.94691_skew4.08283_neigh1.01521_cross0.55446_n14.mtx","synthetic_121192_121192_2622792_avg21.64163_std13.77856_random_bw66101.8109_skew4.129_neigh1.01554_cross0.56191_n14.mtx","synthetic_121192_121192_2622812_avg21.64179_std16.1555_random_bw64390.16957_skew3.99034_neigh1.01546_cross0.55795_n14.mtx","synthetic_121192_121192_2622828_avg21.64192_std13.43338_random_bw65117.16008_skew3.29722_neigh1.01575_cross0.55268_n14.mtx","synthetic_121192_121192_2622837_avg21.642_std15.01875_random_bw64761.9232_skew4.12892_neigh1.01546_cross0.55076_n14.mtx","synthetic_121192_121192_2622876_avg21.64232_std16.69445_random_bw64326.40631_skew3.94402_neigh1.01578_cross0.55631_n14.mtx","synthetic_121192_121192_2622972_avg21.64311_std13.78148_random_bw66363.1673_skew4.31347_neigh1.01343_cross0.566_n14.mtx","synthetic_121192_121192_2623047_avg21.64373_std13.32114_random_bw66419.48364_skew4.0361_neigh1.01466_cross0.56615_n14.mtx","synthetic_121192_121192_2623109_avg21.64424_std13.78493_random_bw66631.25408_skew4.4518_neigh1.01539_cross0.56882_n14.mtx","synthetic_121192_121192_2623156_avg21.64463_std11.48856_random_bw68448.4577_skew3.89729_neigh1.01326_cross0.58356_n14.mtx","synthetic_121192_121192_2623189_avg21.6449_std13.78718_random_bw67331.69717_skew4.77503_neigh1.01343_cross0.57336_n14.mtx","synthetic_121192_121192_2623197_avg21.64497_std13.78635_random_bw66921.85212_skew4.59021_neigh1.01484_cross0.57212_n14.mtx","synthetic_121192_121192_2623249_avg21.6454_std12.40537_random_bw67456.81523_skew3.98951_neigh1.01486_cross0.57624_n14.mtx","synthetic_121192_121192_2623583_avg21.64815_std9.65791_random_bw70183.15791_skew3.8503_neigh1.01496_cross0.59679_n14.mtx","synthetic_121192_121192_2623613_avg21.6484_std10.57459_random_bw69356.8168_skew3.85024_neigh1.0146_cross0.5911_n14.mtx","synthetic_121192_121192_2710314_avg22.3638_std13.78076_random_bw66994.6287_skew4.05281_neigh1.01691_cross0.56487_n14.mtx","synthetic_121192_121192_2885490_avg23.80924_std13.78209_random_bw67434.57931_skew3.99806_neigh1.02152_cross0.5719_n14.mtx","synthetic_121192_121192_3060321_avg25.25184_std13.78548_random_bw68906.15443_skew3.95013_neigh1.02431_cross0.57782_n14.mtx","synthetic_121192_121192_3235599_avg26.69812_std13.78831_random_bw69042.29829_skew3.86926_neigh1.02849_cross0.58477_n14.mtx","synthetic_121192_121192_3410529_avg28.14154_std13.78985_random_bw69312.50343_skew3.86825_neigh1.03157_cross0.58839_n14.mtx","synthetic_1233339_1233339_3909668_avg3.16999_std25.52165_random_bw154195.41372_skew1482.60253_neigh0.14111_cross0.90327_n14.mtx","synthetic_125231_125231_2710355_avg21.64284_std13.77742_random_bw68141.31404_skew4.08251_neigh1.01464_cross0.56127_n14.mtx","synthetic_1300006_1300006_4120758_avg3.1698_std25.51264_random_bw162585.00942_skew1482.69019_neigh0.14075_cross0.90351_n14.mtx","synthetic_131098_131098_734679_avg5.60404_std4.47302_random_bw41879.66163_skew62.88244_neigh0.60932_cross0.60986_n14.mtx","synthetic_133311_133311_2884795_avg21.63959_std13.78017_random_bw72606.70986_skew4.08328_neigh1.01353_cross0.56105_n14.mtx","synthetic_141390_141390_3059797_avg21.64083_std13.77882_random_bw76927.21946_skew4.08298_neigh1.01482_cross0.56115_n14.mtx","synthetic_142498_142498_798541_avg5.60388_std4.46852_random_bw45521.18383_skew62.88436_neigh0.61099_cross0.60948_n14.mtx","synthetic_144550_144550_890999_avg6.16395_std4.44624_random_bw249.77523_skew7.27392_neigh0.18177_cross0.33337_n14.mtx","synthetic_14839_14839_1040332_avg70.10796_std6.31579_random_bw1146.69634_skew0.34079_neigh1.78621_cross0.91392_n14.mtx","synthetic_149470_149470_3234141_avg21.63739_std13.78086_random_bw81415.43761_skew4.08379_neigh1.01461_cross0.56019_n14.mtx","synthetic_153898_153898_862342_avg5.60333_std4.46589_random_bw49364.34344_skew62.89053_neigh0.61051_cross0.60952_n14.mtx","synthetic_157549_157549_3408821_avg21.63658_std13.78244_random_bw85829.82001_skew4.08398_neigh1.01388_cross0.56058_n14.mtx","synthetic_158316_158316_975794_avg6.16358_std4.44586_random_bw273.88663_skew7.27441_neigh0.17907_cross0.32957_n14.mtx","synthetic_16253_16253_1139653_avg70.11955_std6.30855_random_bw1286.71322_skew0.34057_neigh1.7819_cross0.91551_n14.mtx","synthetic_165298_165298_926272_avg5.60365_std4.4624_random_bw52893.04048_skew62.88694_neigh0.60862_cross0.6098_n14.mtx","synthetic_170998_170998_1008507_avg5.89777_std3.14995_random_bw47719.98025_skew59.87046_neigh0.63807_cross0.61706_n14.mtx","synthetic_170998_170998_1037773_avg6.06892_std4.43265_random_bw48509.54384_skew59.96638_neigh0.66301_cross0.62469_n14.mtx","synthetic_170998_170998_1043193_avg6.10062_std4.44271_random_bw48724.69743_skew63.58365_neigh0.67297_cross0.62698_n14.mtx","synthetic_170998_170998_1092151_avg6.38692_std4.47337_random_bw49002.39485_skew64.44623_neigh0.64099_cross0.61384_n14.mtx","synthetic_170998_170998_1211686_avg7.08597_std4.4649_random_bw52652.61671_skew64.76379_neigh0.68861_cross0.62669_n14.mtx","synthetic_170998_170998_1212281_avg7.08945_std4.45045_random_bw52659.56747_skew61.3462_neigh0.68857_cross0.62682_n14.mtx","synthetic_170998_170998_703429_avg4.11367_std4.4083_random_bw66949.59583_skew60.01611_neigh0.59084_cross0.62104_n14.mtx","synthetic_170998_170998_709969_avg4.15191_std4.42013_random_bw67041.59758_skew65.23451_neigh0.60621_cross0.62439_n14.mtx","synthetic_170998_170998_813246_avg4.75588_std4.44226_random_bw49547.73476_skew61.86954_neigh0.577_cross0.60721_n14.mtx","synthetic_170998_170998_875832_avg5.12188_std4.42226_random_bw52588.79732_skew62.06273_neigh0.64594_cross0.6236_n14.mtx","synthetic_170998_170998_882159_avg5.15888_std4.44207_random_bw52806.32402_skew66.06876_neigh0.63734_cross0.62272_n14.mtx","synthetic_170998_170998_944721_avg5.52475_std5.76053_random_bw53759.0296_skew63.79933_neigh0.61291_cross0.61003_n14.mtx","synthetic_170998_170998_945632_avg5.53008_std5.47116_random_bw53929.01913_skew63.7369_neigh0.61326_cross0.61142_n14.mtx","synthetic_170998_170998_946661_avg5.53609_std4.44002_random_bw54131.28407_skew44.88072_neigh0.61289_cross0.6094_n14.mtx","synthetic_170998_170998_947518_avg5.54111_std5.18255_random_bw54056.59899_skew63.60804_neigh0.61159_cross0.61005_n14.mtx","synthetic_170998_170998_948659_avg5.54778_std4.44165_random_bw54194.39357_skew48.9299_neigh0.61004_cross0.61069_n14.mtx","synthetic_170998_170998_950710_avg5.55977_std4.8941_random_bw54225.54433_skew63.39112_neigh0.60981_cross0.60991_n14.mtx","synthetic_170998_170998_951053_avg5.56178_std4.44852_random_bw54287.96161_skew53.11938_neigh0.6081_cross0.60983_n14.mtx","synthetic_170998_170998_953667_avg5.57707_std4.45309_random_bw54491.631_skew57.09507_neigh0.6098_cross0.60996_n14.mtx","synthetic_170998_170998_955297_avg5.5866_std4.60347_random_bw54609.35347_skew63.08194_neigh0.6098_cross0.61023_n14.mtx","synthetic_170998_170998_956707_avg5.59484_std4.45769_random_bw54693.36679_skew61.0214_neigh0.61022_cross0.60929_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_random_bw38321.91494_skew62.88573_neigh0.60926_cross0.60964_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_random_bw41929.48688_skew62.88573_neigh0.60983_cross0.6094_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_random_bw45628.56654_skew62.88573_neigh0.61026_cross0.60953_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_random_bw46903.96538_skew62.88573_neigh0.54722_cross0.60807_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_random_bw47843.37714_skew62.88573_neigh0.50741_cross0.60854_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_random_bw48523.58357_skew62.88573_neigh0.46873_cross0.60766_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_random_bw49181.26028_skew62.88573_neigh0.60917_cross0.60964_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_random_bw49272.78155_skew62.88573_neigh0.42658_cross0.60457_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_random_bw49297.41096_skew62.88573_neigh0.79067_cross0.61249_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_random_bw50512.55887_skew62.88573_neigh0.75263_cross0.61034_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_random_bw51880.4182_skew62.88573_neigh0.71336_cross0.6105_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_random_bw52905.44733_skew62.88573_neigh0.60897_cross0.60969_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_random_bw53028.52728_skew62.88573_neigh0.67336_cross0.60943_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_random_bw53125.78092_skew62.88573_neigh0.55319_cross0.78965_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_random_bw53660.70344_skew62.88573_neigh0.56909_cross0.75077_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_random_bw54099.133_skew62.88573_neigh0.58565_cross0.70928_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_random_bw54240.77009_skew62.88573_neigh0.63324_cross0.60935_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_random_bw54411.68621_skew62.88573_neigh0.59727_cross0.66902_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_random_bw54696.32419_skew62.88573_neigh0.60505_cross0.63095_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_random_bw54839.18485_skew62.88573_neigh0.61271_cross0.58999_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_random_bw54975.41519_skew62.88573_neigh0.61926_cross0.54811_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_random_bw55027.14865_skew62.88573_neigh0.62632_cross0.50615_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_random_bw55105.55545_skew62.88573_neigh0.63063_cross0.46746_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_random_bw55236.3212_skew62.88573_neigh0.63219_cross0.42616_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_random_bw55398.06761_skew62.88573_neigh0.58791_cross0.60932_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_random_bw56503.9465_skew62.88573_neigh0.60809_cross0.60967_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_random_bw60168.38237_skew62.88573_neigh0.60916_cross0.60962_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_random_bw64070.5493_skew62.88573_neigh0.60935_cross0.60965_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_random_bw67352.62944_skew62.88573_neigh0.60959_cross0.60963_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_random_bw71097.05963_skew62.88573_neigh0.60883_cross0.60961_n14.mtx","synthetic_170998_170998_959835_avg5.61314_std4.46266_random_bw54839.73947_skew64.91681_neigh0.60778_cross0.61012_n14.mtx","synthetic_170998_170998_961623_avg5.62359_std4.31537_random_bw54830.15481_skew62.66038_neigh0.60743_cross0.6101_n14.mtx","synthetic_170998_170998_963061_avg5.632_std4.4671_random_bw54958.67312_skew68.77981_neigh0.6102_cross0.6093_n14.mtx","synthetic_170998_170998_966597_avg5.65268_std4.47351_random_bw55278.98278_skew72.59341_neigh0.60946_cross0.60959_n14.mtx","synthetic_170998_170998_970106_avg5.6732_std4.48093_random_bw55352.2479_skew76.55763_neigh0.61119_cross0.61051_n14.mtx","synthetic_170998_170998_970315_avg5.67442_std4.02645_random_bw55373.82904_skew62.09011_neigh0.61062_cross0.61139_n14.mtx","synthetic_170998_170998_973557_avg5.69338_std4.48437_random_bw55499.03393_skew80.32248_neigh0.61487_cross0.61054_n14.mtx","synthetic_170998_170998_981215_avg5.73817_std3.73499_random_bw55852.32383_skew61.38927_neigh0.61949_cross0.61025_n14.mtx","synthetic_170998_170998_994346_avg5.81496_std3.44543_random_bw56542.54022_skew60.73734_neigh0.6252_cross0.61459_n14.mtx","synthetic_172083_172083_1060867_avg6.16486_std4.44424_random_bw297.78023_skew7.2727_neigh0.17596_cross0.32947_n14.mtx","synthetic_17666_17666_1238955_avg70.13217_std6.31199_random_bw1373.26344_skew0.34033_neigh1.78005_cross0.91685_n14.mtx","synthetic_176697_176697_990167_avg5.60376_std4.45819_random_bw56447.76777_skew62.88571_neigh0.60833_cross0.60971_n14.mtx","synthetic_185639_185639_15003294_avg80.81973_std87.6573_random_bw25664.94771_skew8.39127_neigh1.21561_cross0.82365_n14.mtx","synthetic_185639_185639_15003763_avg80.82226_std142.24862_random_bw20258.95107_skew7.19081_neigh1.26005_cross0.95405_n14.mtx","synthetic_185639_185639_15004156_avg80.82437_std94.60907_random_bw25577.96053_skew8.30412_neigh1.21738_cross0.82741_n14.mtx","synthetic_185639_185639_15004647_avg80.82702_std101.43013_random_bw25579.20672_skew8.20484_neigh1.21614_cross0.83107_n14.mtx","synthetic_185639_185639_15005689_avg80.83263_std108.09033_random_bw25508.21416_skew8.08049_neigh1.21648_cross0.83534_n14.mtx","synthetic_185639_185639_15006242_avg80.83561_std119.38413_random_bw25324.46226_skew8.66158_neigh1.21359_cross0.83963_n14.mtx","synthetic_185639_185639_15006431_avg80.83663_std114.55767_random_bw25341.18275_skew7.94397_neigh1.21465_cross0.84203_n14.mtx","synthetic_185639_185639_15006681_avg80.83798_std118.31728_random_bw25323.95677_skew8.14174_neigh1.2126_cross0.84303_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_random_bw17706.01026_skew7.8818_neigh1.2138_cross0.84521_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_random_bw19404.35313_skew7.8818_neigh1.21315_cross0.84474_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_random_bw21104.27872_skew7.8818_neigh1.21478_cross0.84522_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_random_bw22750.22312_skew7.8818_neigh1.21263_cross0.84532_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_random_bw24441.96582_skew7.8818_neigh1.21268_cross0.84507_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_random_bw26179.82365_skew7.8818_neigh1.21167_cross0.84498_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_random_bw27821.86705_skew7.8818_neigh1.21199_cross0.84555_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_random_bw29515.52781_skew7.8818_neigh1.21272_cross0.84497_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_random_bw31178.6333_skew7.8818_neigh1.21304_cross0.84523_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_random_bw32880.54075_skew7.8818_neigh1.21042_cross0.84511_n14.mtx","synthetic_185639_185639_15007305_avg80.84134_std120.8084_random_bw25218.21287_skew7.80738_neigh1.21325_cross0.84823_n14.mtx","synthetic_185639_185639_15008187_avg80.84609_std126.81395_random_bw25055.73053_skew7.64606_neigh1.20604_cross0.8579_n14.mtx","synthetic_185639_185639_15008702_avg80.84886_std137.983_random_bw24238.46969_skew7.32417_neigh1.20104_cross0.89187_n14.mtx","synthetic_185639_185639_15008859_avg80.84971_std132.54683_random_bw24711.02466_skew7.48488_neigh1.20312_cross0.87083_n14.mtx","synthetic_185639_185639_18505682_avg99.68639_std122.27517_random_bw25744.70036_skew8.23897_neigh1.22874_cross0.82863_n14.mtx","synthetic_185639_185639_19505424_avg105.0718_std123.33624_random_bw25716.49132_skew8.2984_neigh1.23114_cross0.82664_n14.mtx","synthetic_185850_185850_1145706_avg6.16468_std4.44467_random_bw321.67519_skew7.27293_neigh0.17292_cross0.32599_n14.mtx","synthetic_188097_188097_1054064_avg5.60383_std4.45656_random_bw60049.05112_skew62.88486_neigh0.60992_cross0.60918_n14.mtx","synthetic_19080_19080_1337993_avg70.12542_std6.31221_random_bw1489.05252_skew0.34046_neigh1.78444_cross0.91485_n14.mtx","synthetic_199497_199497_1118014_avg5.60416_std4.45398_random_bw64023.47114_skew62.88107_neigh0.60994_cross0.60927_n14.mtx","synthetic_199616_199616_1230682_avg6.16525_std4.44318_random_bw345.0451_skew7.27217_neigh0.1707_cross0.32355_n14.mtx","synthetic_20493_20493_1437302_avg70.13624_std6.31204_random_bw1605.41268_skew0.34025_neigh1.77707_cross0.91532_n14.mtx","synthetic_206500_206500_1060353_avg5.13488_std4.44292_random_bw335.27304_skew7.3741_neigh0.15993_cross0.3084_n14.mtx","synthetic_206500_206500_1145534_avg5.54738_std4.44341_random_bw356.84439_skew7.2922_neigh0.16358_cross0.31502_n14.mtx","synthetic_206500_206500_1230425_avg5.95847_std4.44436_random_bw374.92322_skew7.22358_neigh0.16711_cross0.31892_n14.mtx","synthetic_206500_206500_1272393_avg6.16171_std5.76962_random_bw331.0109_skew7.43922_neigh0.1768_cross0.30559_n14.mtx","synthetic_206500_206500_1272581_avg6.16262_std4.44309_random_bw352.77997_skew6.30209_neigh0.16944_cross0.31989_n14.mtx","synthetic_206500_206500_1272606_avg6.16274_std4.44382_random_bw350.20787_skew5.97741_neigh0.16939_cross0.31707_n14.mtx","synthetic_206500_206500_1272648_avg6.16294_std5.47591_random_bw338.0763_skew7.43753_neigh0.17479_cross0.31105_n14.mtx","synthetic_206500_206500_1272653_avg6.16297_std5.1813_random_bw344.41462_skew7.27523_neigh0.17252_cross0.31341_n14.mtx","synthetic_206500_206500_1272686_avg6.16313_std4.88791_random_bw350.39832_skew7.27502_neigh0.17198_cross0.31772_n14.mtx","synthetic_206500_206500_1272775_avg6.16356_std4.44367_random_bw347.43103_skew5.48976_neigh0.16995_cross0.31545_n14.mtx","synthetic_206500_206500_1272904_avg6.16418_std4.4462_random_bw361.82239_skew8.73365_neigh0.1696_cross0.32814_n14.mtx","synthetic_206500_206500_1272958_avg6.16445_std4.59126_random_bw354.96969_skew7.27325_neigh0.16996_cross0.32281_n14.mtx","synthetic_206500_206500_1272981_avg6.16456_std4.44526_random_bw362.54571_skew9.05749_neigh0.16989_cross0.32918_n14.mtx","synthetic_206500_206500_1272983_avg6.16457_std4.4441_random_bw354.47146_skew6.62422_neigh0.16961_cross0.32233_n14.mtx","synthetic_206500_206500_1272990_avg6.1646_std4.44515_random_bw357.90535_skew7.43526_neigh0.16959_cross0.32445_n14.mtx","synthetic_206500_206500_1272993_avg6.16462_std4.44589_random_bw359.27284_skew7.92189_neigh0.16953_cross0.32657_n14.mtx","synthetic_206500_206500_1272999_avg6.16464_std4.44572_random_bw360.70137_skew8.24628_neigh0.16983_cross0.32797_n14.mtx","synthetic_206500_206500_1273034_avg6.16481_std4.29748_random_bw359.14318_skew7.27276_neigh0.17002_cross0.32629_n14.mtx","synthetic_206500_206500_1273051_avg6.1649_std3.71025_random_bw366.49966_skew7.27265_neigh0.16733_cross0.33042_n14.mtx","synthetic_206500_206500_1273053_avg6.16491_std4.44447_random_bw356.03003_skew7.11042_neigh0.1699_cross0.32349_n14.mtx","synthetic_206500_206500_1273074_avg6.16501_std4.00298_random_bw362.98579_skew7.2725_neigh0.16799_cross0.32983_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_random_bw250.53829_skew7.27211_neigh0.18184_cross0.33208_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_random_bw272.80182_skew7.27211_neigh0.17887_cross0.32932_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_random_bw297.33833_skew7.27211_neigh0.17494_cross0.32838_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_random_bw321.61163_skew7.27211_neigh0.17327_cross0.32563_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_random_bw345.2926_skew7.27211_neigh0.17126_cross0.32436_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_random_bw353.07637_skew7.27211_neigh0.1683_cross0.41181_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_random_bw354.05036_skew7.27211_neigh0.1697_cross0.39234_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_random_bw355.04431_skew7.27211_neigh0.16961_cross0.37276_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_random_bw355.0493_skew7.27211_neigh0.193_cross0.3226_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_random_bw356.01345_skew7.27211_neigh0.16921_cross0.35329_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_random_bw356.02066_skew7.27211_neigh0.18376_cross0.32185_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_random_bw356.56768_skew7.27211_neigh0.17513_cross0.32339_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_random_bw357.15197_skew7.27211_neigh0.16977_cross0.3328_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_random_bw357.48545_skew7.27211_neigh0.16999_cross0.31374_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_random_bw357.59221_skew7.27211_neigh0.16543_cross0.32304_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_random_bw358.45388_skew7.27211_neigh0.17021_cross0.29537_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_random_bw358.52034_skew7.27211_neigh0.15599_cross0.32162_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_random_bw358.88324_skew7.27211_neigh0.17043_cross0.27574_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_random_bw359.59658_skew7.27211_neigh0.17102_cross0.25585_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_random_bw359.63777_skew7.27211_neigh0.14622_cross0.32225_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_random_bw360.3102_skew7.27211_neigh0.17173_cross0.23608_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_random_bw360.37211_skew7.27211_neigh0.13735_cross0.3228_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_random_bw361.09723_skew7.27211_neigh0.12774_cross0.32289_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_random_bw368.19459_skew7.27211_neigh0.16931_cross0.32373_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_random_bw378.37779_skew7.27211_neigh0.21109_cross0.3219_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_random_bw379.42115_skew7.27211_neigh0.20197_cross0.32156_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_random_bw392.56075_skew7.27211_neigh0.16763_cross0.32173_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_random_bw416.95627_skew7.27211_neigh0.16635_cross0.32052_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_random_bw440.8278_skew7.27211_neigh0.16504_cross0.3198_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_random_bw464.28694_skew7.27211_neigh0.16438_cross0.31804_n14.mtx","synthetic_206500_206500_1273351_avg6.16635_std3.41298_random_bw369.41645_skew7.2707_neigh0.16589_cross0.33282_n14.mtx","synthetic_206500_206500_1274354_avg6.17121_std3.11913_random_bw372.02783_skew7.10214_neigh0.16567_cross0.33662_n14.mtx","synthetic_206500_206500_1315280_avg6.36939_std4.44461_random_bw364.31629_skew7.32104_neigh0.1716_cross0.32663_n14.mtx","synthetic_206500_206500_1400260_avg6.78092_std4.44568_random_bw375.96645_skew7.25847_neigh0.17353_cross0.33102_n14.mtx","synthetic_206500_206500_1485240_avg7.19245_std4.44584_random_bw368.40782_skew7.20305_neigh0.17827_cross0.33719_n14.mtx","synthetic_206500_206500_1570037_avg7.60308_std4.44659_random_bw376.86972_skew7.15458_neigh0.18053_cross0.33804_n14.mtx","synthetic_206500_206500_1655223_avg8.01561_std4.44684_random_bw384.89388_skew7.23394_neigh0.18176_cross0.34264_n14.mtx","synthetic_206500_206500_890883_avg4.3142_std4.3742_random_bw314.87298_skew7.34453_neigh0.15043_cross0.29772_n14.mtx","synthetic_206500_206500_975718_avg4.72503_std4.44295_random_bw342.64695_skew7.46556_neigh0.15368_cross0.29915_n14.mtx","synthetic_210897_210897_1181860_avg5.60397_std4.45232_random_bw67447.98409_skew62.88331_neigh0.61013_cross0.60909_n14.mtx","synthetic_21200_21200_1040239_avg49.06788_std6.3116_random_bw2270.10316_skew0.48774_neigh1.73665_cross0.90005_n14.mtx","synthetic_21200_21200_1139550_avg53.75236_std6.31054_random_bw2327.97646_skew0.4325_neigh1.747_cross0.90406_n14.mtx","synthetic_21200_21200_1238795_avg58.43373_std6.30753_random_bw2370.53675_skew0.4033_neigh1.76263_cross0.90541_n14.mtx","synthetic_21200_21200_1338002_avg63.1133_std6.31089_random_bw1621.46392_skew0.37847_neigh1.76689_cross0.91021_n14.mtx","synthetic_21200_21200_1437342_avg67.79915_std6.31153_random_bw1625.12618_skew0.35695_neigh1.78043_cross0.91191_n14.mtx","synthetic_21200_21200_1486357_avg70.11118_std8.20133_random_bw1621.03675_skew0.44057_neigh1.79499_cross0.90156_n14.mtx","synthetic_21200_21200_1486477_avg70.11684_std7.77973_random_bw1623.75863_skew0.41193_neigh1.78416_cross0.90649_n14.mtx","synthetic_21200_21200_1486586_avg70.12198_std7.35959_random_bw1633.7316_skew0.39756_neigh1.78517_cross0.90911_n14.mtx","synthetic_21200_21200_1486691_avg70.12693_std6.94018_random_bw1638.06472_skew0.36895_neigh1.78923_cross0.90946_n14.mtx","synthetic_21200_21200_1486777_avg70.13099_std6.51934_random_bw1665.03075_skew0.35461_neigh1.77737_cross0.91541_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_random_bw1091.14792_skew0.3403_neigh1.27638_cross0.91529_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_random_bw1099.35217_skew0.3403_neigh1.38725_cross0.91539_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_random_bw1129.65231_skew0.3403_neigh1.4963_cross0.91405_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_random_bw1150.42368_skew0.3403_neigh1.6111_cross0.91717_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_random_bw1155.31632_skew0.3403_neigh1.7853_cross0.91405_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_random_bw1291.37047_skew0.3403_neigh1.78149_cross0.91544_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_random_bw1309.76505_skew0.3403_neigh1.7269_cross0.91625_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_random_bw1369.88212_skew0.3403_neigh1.7807_cross0.91642_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_random_bw1489.33288_skew0.3403_neigh1.78357_cross0.91451_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_random_bw1563.73939_skew0.3403_neigh1.71868_cross0.94884_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_random_bw1590.50175_skew0.3403_neigh1.72943_cross0.94422_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_random_bw1604.78255_skew0.3403_neigh1.77801_cross0.91536_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_random_bw1650.62693_skew0.3403_neigh1.86722_cross0.64927_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_random_bw1663.23962_skew0.3403_neigh1.86073_cross0.70503_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_random_bw1668.34939_skew0.3403_neigh1.77532_cross0.91565_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_random_bw1670.25646_skew0.3403_neigh1.85028_cross0.77171_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_random_bw1674.10208_skew0.3403_neigh1.83639_cross0.82849_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_random_bw1677.25429_skew0.3403_neigh1.80579_cross0.88625_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_random_bw1710.35316_skew0.3403_neigh1.78264_cross0.91384_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_random_bw1844.95325_skew0.3403_neigh1.77183_cross0.91655_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_random_bw1925.18679_skew0.3403_neigh1.77933_cross0.91379_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_random_bw2023.01226_skew0.3403_neigh1.85164_cross0.91467_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_random_bw2027.65887_skew0.3403_neigh1.77965_cross0.91342_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_random_bw2126.54439_skew0.3403_neigh1.7794_cross0.91462_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_random_bw2183.93132_skew0.3403_neigh1.83524_cross0.91315_n14.mtx","synthetic_21200_21200_1487000_avg70.14151_std6.10324_random_bw1671.56_skew0.32589_neigh1.77648_cross0.91706_n14.mtx","synthetic_21200_21200_1487077_avg70.14514_std5.67801_random_bw1678.95703_skew0.29731_neigh1.77486_cross0.91968_n14.mtx","synthetic_21200_21200_1487192_avg70.15057_std5.25576_random_bw1669.85401_skew0.28295_neigh1.78043_cross0.92058_n14.mtx","synthetic_21200_21200_1487315_avg70.15637_std4.84276_random_bw1691.18377_skew0.25434_neigh1.76784_cross0.925_n14.mtx","synthetic_21200_21200_1487478_avg70.16406_std4.4186_random_bw1723.39901_skew0.23995_neigh1.76461_cross0.92734_n14.mtx","synthetic_21200_21200_1536543_avg72.47844_std6.30853_random_bw1661.22915_skew0.32453_neigh1.78331_cross0.91588_n14.mtx","synthetic_21200_21200_1635721_avg77.15665_std6.31046_random_bw1691.41788_skew0.30903_neigh1.78778_cross0.91802_n14.mtx","synthetic_21200_21200_1735090_avg81.84387_std6.31393_random_bw1717.89467_skew0.29515_neigh1.79144_cross0.92217_n14.mtx","synthetic_21200_21200_1834315_avg86.52429_std6.30851_random_bw1449.33458_skew0.27132_neigh1.8011_cross0.92178_n14.mtx","synthetic_21200_21200_1933465_avg91.20118_std6.31043_random_bw1477.8358_skew0.26095_neigh1.80472_cross0.92273_n14.mtx","synthetic_213383_213383_1315556_avg6.16523_std4.44392_random_bw368.70315_skew7.27219_neigh0.16899_cross0.32206_n14.mtx","synthetic_21906_21906_1536320_avg70.13238_std6.31756_random_bw1709.96585_skew0.34032_neigh1.77856_cross0.91434_n14.mtx","synthetic_222297_222297_1245675_avg5.60365_std4.45068_random_bw71187.12274_skew62.88691_neigh0.61086_cross0.60946_n14.mtx","synthetic_227150_227150_1400378_avg6.16499_std4.4444_random_bw392.60485_skew7.27252_neigh0.16823_cross0.32207_n14.mtx","synthetic_23320_23320_1635398_avg70.12856_std6.31946_random_bw1840.50772_skew0.3404_neigh1.7719_cross0.91648_n14.mtx","synthetic_240916_240916_1485276_avg6.16512_std4.4437_random_bw416.93905_skew7.27235_neigh0.16633_cross0.32069_n14.mtx","synthetic_24733_24733_1734543_avg70.13072_std6.31636_random_bw1933.712_skew0.34035_neigh1.7778_cross0.91451_n14.mtx","synthetic_254683_254683_1570192_avg6.16528_std4.4433_random_bw441.04844_skew7.27213_neigh0.16638_cross0.31989_n14.mtx","synthetic_26146_26146_1833536_avg70.12683_std6.31243_random_bw2034.00092_skew0.34043_neigh1.7785_cross0.91319_n14.mtx","synthetic_268450_268450_1655008_avg6.16505_std4.44456_random_bw465.05871_skew7.27244_neigh0.16485_cross0.31978_n14.mtx","synthetic_27120_27120_1234696_avg45.52714_std31.4396_random_bw657.74856_skew2.64618_neigh1.22988_cross0.77937_n14.mtx","synthetic_27560_27560_1933075_avg70.1406_std6.31297_random_bw2125.47631_skew0.34017_neigh1.77961_cross0.91464_n14.mtx","synthetic_29703_29703_1352183_avg45.52345_std31.43457_random_bw718.17419_skew2.64647_neigh1.22769_cross0.77551_n14.mtx","synthetic_32286_32286_1469958_avg45.52927_std31.42835_random_bw783.22802_skew2.66797_neigh1.22513_cross0.77351_n14.mtx","synthetic_32784_32784_1657863_avg50.56927_std27.82469_random_bw5242.21303_skew3.09339_neigh1.63697_cross0.76668_n14.mtx","synthetic_34406_34406_1341834_avg39.0_std0.0_random_bw8401.7213_skew0.0_neigh1.4059_cross0.80649_n14.mtx","synthetic_34869_34869_1587838_avg45.53724_std31.4194_random_bw844.81465_skew2.66733_neigh1.22431_cross0.77337_n14.mtx","synthetic_35906_35906_1816148_avg50.58063_std27.80623_random_bw5716.92046_skew3.09248_neigh1.63607_cross0.7699_n14.mtx","synthetic_368077_368077_1472308_avg4.0_std0.0_random_bw504.70969_skew0.0_neigh0.37372_cross0.9891_n14.mtx","synthetic_37452_37452_1706036_avg45.5526_std31.4087_random_bw906.67014_skew2.66609_neigh1.22181_cross0.77055_n14.mtx","synthetic_37683_37683_1469637_avg39.0_std0.0_random_bw9214.2749_skew0.0_neigh1.40447_cross0.80565_n14.mtx","synthetic_38120_38120_14004492_avg367.37912_std206.50003_random_bw17131.47558_skew1.32185_neigh1.97443_cross0.91972_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_random_bw12163.96299_skew1.57045_neigh1.97477_cross0.91968_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_random_bw13487.94121_skew1.57045_neigh1.97466_cross0.91853_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_random_bw14593.25708_skew1.57045_neigh1.97453_cross0.91966_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_random_bw15458.63701_skew1.57045_neigh1.97488_cross0.91833_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_random_bw16697.59053_skew1.57045_neigh1.97466_cross0.91971_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_random_bw17311.49276_skew1.57045_neigh1.97473_cross0.91875_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_random_bw17930.28565_skew1.57045_neigh1.97458_cross0.91885_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_random_bw18890.56044_skew1.57045_neigh1.97464_cross0.91976_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_random_bw19998.78673_skew1.57045_neigh1.97496_cross0.91885_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_random_bw21254.44919_skew1.57045_neigh1.9747_cross0.9186_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_random_bw22981.65732_skew1.57045_neigh1.97438_cross0.91938_n14.mtx","synthetic_38120_38120_15009192_avg393.73536_std249.23315_random_bw20164.63922_skew1.65915_neigh1.95556_cross0.89934_n14.mtx","synthetic_38120_38120_15509350_avg406.85598_std254.10081_random_bw18952.83022_skew1.83638_neigh1.9601_cross0.87267_n14.mtx","synthetic_38120_38120_15831412_avg415.30462_std256.81933_random_bw18361.46692_skew2.01225_neigh1.9625_cross0.85222_n14.mtx","synthetic_38120_38120_16013257_avg420.07495_std256.51996_random_bw18132.49037_skew2.18991_neigh1.96341_cross0.83427_n14.mtx","synthetic_38120_38120_17911818_avg469.8798_std294.64729_random_bw17768.78305_skew1.77305_neigh1.96177_cross0.87832_n14.mtx","synthetic_38120_38120_17943763_avg470.71781_std404.16901_random_bw13028.28431_skew3.74169_neigh1.974_cross0.55914_n14.mtx","synthetic_38120_38120_19284934_avg505.90068_std314.8817_random_bw17404.61952_skew1.89583_neigh1.96456_cross0.86292_n14.mtx","synthetic_38120_38120_20568467_avg539.57154_std333.79776_random_bw17206.90546_skew2.00609_neigh1.96636_cross0.85062_n14.mtx","synthetic_38744_38744_1161488_avg29.97853_std22.5272_random_bw1034.84256_skew1.93543_neigh1.22772_cross0.85244_n14.mtx","synthetic_38744_38744_1307882_avg33.75702_std24.88267_random_bw998.77878_skew2.08084_neigh1.17487_cross0.82838_n14.mtx","synthetic_38744_38744_1449216_avg37.40491_std27.07152_random_bw962.94693_skew2.28834_neigh1.19126_cross0.80394_n14.mtx","synthetic_38744_38744_1579684_avg40.77235_std28.98643_random_bw952.64469_skew2.45823_neigh1.20264_cross0.78378_n14.mtx","synthetic_38744_38744_1703850_avg43.97713_std30.65815_random_bw943.0718_skew2.61552_neigh1.21338_cross0.77875_n14.mtx","synthetic_38744_38744_1707520_avg44.07186_std32.46486_random_bw995.81409_skew2.10856_neigh1.20504_cross0.82864_n14.mtx","synthetic_38744_38744_1730367_avg44.66155_std32.51286_random_bw976.76174_skew2.22425_neigh1.21085_cross0.80896_n14.mtx","synthetic_38744_38744_1736329_avg44.81543_std26.74959_random_bw958.24236_skew1.96773_neigh1.21246_cross0.79103_n14.mtx","synthetic_38744_38744_1746236_avg45.07113_std32.41542_random_bw952.99461_skew2.35026_neigh1.21614_cross0.79836_n14.mtx","synthetic_38744_38744_1747768_avg45.11068_std28.0118_random_bw950.83561_skew2.12565_neigh1.21055_cross0.78825_n14.mtx","synthetic_38744_38744_1755242_avg45.30358_std29.13201_random_bw947.62389_skew2.28892_neigh1.21485_cross0.78295_n14.mtx","synthetic_38744_38744_1756611_avg45.33892_std32.15335_random_bw944.97597_skew2.48486_neigh1.21552_cross0.78836_n14.mtx","synthetic_38744_38744_1760377_avg45.43612_std30.12952_random_bw943.2046_skew2.43339_neigh1.21615_cross0.78132_n14.mtx","synthetic_38744_38744_1762627_avg45.49419_std31.70473_random_bw945.51417_skew2.60486_neigh1.21499_cross0.77366_n14.mtx","synthetic_38744_38744_1763457_avg45.51562_std31.01391_random_bw938.21335_skew2.60316_neigh1.21504_cross0.77558_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_random_bw1012.91606_skew2.6669_neigh1.23666_cross0.60626_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_random_bw1028.13896_skew2.6669_neigh1.24007_cross0.5571_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_random_bw1030.11568_skew2.6669_neigh1.2189_cross0.77356_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_random_bw1090.87397_skew2.6669_neigh1.21727_cross0.76922_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_random_bw1152.64756_skew2.6669_neigh1.2162_cross0.77082_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_random_bw1218.42621_skew2.6669_neigh1.21339_cross0.77016_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_random_bw659.5718_skew2.6669_neigh1.23127_cross0.77639_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_random_bw720.8265_skew2.6669_neigh1.23049_cross0.7745_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_random_bw783.97236_skew2.6669_neigh1.22458_cross0.77252_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_random_bw790.44606_skew2.6669_neigh1.18687_cross0.89462_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_random_bw799.34238_skew2.6669_neigh1.18845_cross0.8908_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_random_bw843.39606_skew2.6669_neigh1.22114_cross0.77348_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_random_bw875.30678_skew2.6669_neigh1.20367_cross0.84315_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_random_bw905.63736_skew2.6669_neigh1.21901_cross0.77413_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_random_bw922.58182_skew2.6669_neigh1.21558_cross0.79594_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_random_bw932.78459_skew2.6669_neigh1.03232_cross0.77348_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_random_bw935.45003_skew2.6669_neigh1.17793_cross0.77396_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_random_bw936.96549_skew2.6669_neigh1.25866_cross0.77239_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_random_bw939.4777_skew2.6669_neigh0.88501_cross0.77405_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_random_bw940.09284_skew2.6669_neigh0.96014_cross0.7732_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_random_bw940.09785_skew2.6669_neigh1.10807_cross0.77234_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_random_bw943.56437_skew2.6669_neigh1.32906_cross0.77341_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_random_bw946.71836_skew2.6669_neigh1.40773_cross0.77338_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_random_bw952.76706_skew2.6669_neigh1.22251_cross0.74756_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_random_bw956.49716_skew2.6669_neigh1.48195_cross0.77307_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_random_bw963.6331_skew2.6669_neigh1.22154_cross0.77193_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_random_bw974.85482_skew2.6669_neigh1.23062_cross0.69785_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_random_bw975.63003_skew2.6669_neigh1.56256_cross0.77213_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_random_bw995.95674_skew2.6669_neigh1.23589_cross0.65094_n14.mtx","synthetic_38744_38744_1765368_avg45.56494_std31.78636_random_bw934.44368_skew2.75289_neigh1.21894_cross0.77198_n14.mtx","synthetic_38744_38744_1765877_avg45.57808_std31.07073_random_bw933.8017_skew2.72986_neigh1.22255_cross0.76872_n14.mtx","synthetic_38744_38744_1766496_avg45.59405_std32.481_random_bw931.2818_skew2.88209_neigh1.21975_cross0.76797_n14.mtx","synthetic_38744_38744_1767073_avg45.60895_std26.86577_random_bw926.3303_skew3.122_neigh1.22395_cross0.74655_n14.mtx","synthetic_38744_38744_1767160_avg45.61119_std30.25452_random_bw933.41929_skew2.83678_neigh1.21906_cross0.76087_n14.mtx","synthetic_38744_38744_1767330_avg45.61558_std33.08901_random_bw930.75617_skew3.03371_neigh1.22002_cross0.7649_n14.mtx","synthetic_38744_38744_1767431_avg45.61819_std29.2665_random_bw931.34116_skew2.94579_neigh1.22054_cross0.7542_n14.mtx","synthetic_38744_38744_1767460_avg45.61893_std28.1289_random_bw924.86287_skew3.03341_neigh1.22219_cross0.75561_n14.mtx","synthetic_38744_38744_1767651_avg45.62386_std33.63482_random_bw928.32175_skew3.16449_neigh1.22329_cross0.76327_n14.mtx","synthetic_38744_38744_1767760_avg45.62668_std34.12282_random_bw928.16813_skew3.31765_neigh1.22071_cross0.76095_n14.mtx","synthetic_38744_38744_1824565_avg47.09284_std32.11946_random_bw938.10585_skew2.7373_neigh1.22202_cross0.76943_n14.mtx","synthetic_38744_38744_1943737_avg50.16872_std33.41096_random_bw937.69409_skew2.82709_neigh1.22997_cross0.76198_n14.mtx","synthetic_38744_38744_2062005_avg53.22127_std34.5782_random_bw928.18411_skew2.90821_neigh1.23817_cross0.75999_n14.mtx","synthetic_38744_38744_2179882_avg56.26373_std35.63951_random_bw931.14637_skew2.98125_neigh1.24047_cross0.75443_n14.mtx","synthetic_38744_38744_2297582_avg59.30162_std36.62621_random_bw928.90731_skew3.03024_neigh1.24558_cross0.75336_n14.mtx","synthetic_39029_39029_1974236_avg50.58382_std27.80163_random_bw6199.88865_skew3.11199_neigh1.63568_cross0.76891_n14.mtx","synthetic_40035_40035_1824084_avg45.56223_std31.3971_random_bw969.35456_skew2.66532_neigh1.21894_cross0.77386_n14.mtx","synthetic_403132_403132_1612528_avg4.0_std0.0_random_bw553.55446_skew0.0_neigh0.37461_cross0.98908_n14.mtx","synthetic_40960_40960_1597440_avg39.0_std0.0_random_bw9995.91001_skew0.0_neigh1.40666_cross0.80631_n14.mtx","synthetic_42151_42151_2133812_avg50.62305_std27.78297_random_bw6697.26877_skew3.12855_neigh1.63324_cross0.77066_n14.mtx","synthetic_42618_42618_1942443_avg45.578_std31.38541_random_bw1030.63919_skew2.66405_neigh1.21482_cross0.7709_n14.mtx","synthetic_43715_43715_2801362_avg64.0824_std14.01594_random_bw314.71989_skew0.93501_neigh1.5866_cross0.83319_n14.mtx","synthetic_438187_438187_1752748_avg4.0_std0.0_random_bw605.4017_skew0.0_neigh0.36958_cross0.98958_n14.mtx","synthetic_44236_44236_1725204_avg39.0_std0.0_random_bw10814.82767_skew0.0_neigh1.40668_cross0.80597_n14.mtx","synthetic_45201_45201_2060190_avg45.57842_std31.38204_random_bw1097.18524_skew2.68596_neigh1.21466_cross0.77308_n14.mtx","synthetic_45273_45273_2292008_avg50.62638_std27.77577_random_bw7230.17452_skew3.12828_neigh1.63423_cross0.77066_n14.mtx","synthetic_46835_46835_1659331_avg35.42929_std24.19453_random_bw7661.97799_skew2.92331_neigh1.59887_cross0.77081_n14.mtx","synthetic_46835_46835_1817944_avg38.81593_std25.43637_random_bw7818.83557_skew3.04473_neigh1.61137_cross0.76248_n14.mtx","synthetic_46835_46835_1976036_avg42.19144_std26.52871_random_bw7433.66969_skew3.14776_neigh1.62142_cross0.75764_n14.mtx","synthetic_46835_46835_2134025_avg45.56475_std27.52318_random_bw7560.21004_skew3.21378_neigh1.62711_cross0.75172_n14.mtx","synthetic_46835_46835_2292218_avg48.94241_std27.77072_random_bw7378.23094_skew3.16816_neigh1.63162_cross0.7647_n14.mtx","synthetic_46835_46835_2370905_avg50.6225_std27.74621_random_bw7334.7539_skew2.81253_neigh1.63507_cross0.75275_n14.mtx","synthetic_46835_46835_2371001_avg50.62455_std27.39541_random_bw7354.04117_skew2.69386_neigh1.63657_cross0.75406_n14.mtx","synthetic_46835_46835_2371113_avg50.62695_std32.27909_random_bw7361.07123_skew3.10848_neigh1.63754_cross0.75502_n14.mtx","synthetic_46835_46835_2371156_avg50.62786_std27.75732_random_bw7391.41134_skew2.91089_neigh1.63514_cross0.76051_n14.mtx","synthetic_46835_46835_2371164_avg50.62803_std30.41679_random_bw7309.21574_skew3.22691_neigh1.63831_cross0.75152_n14.mtx","synthetic_46835_46835_2371201_avg50.62882_std33.12436_random_bw7335.62733_skew3.04908_neigh1.63667_cross0.75847_n14.mtx","synthetic_46835_46835_2371202_avg50.62885_std27.76569_random_bw7416.01217_skew3.00957_neigh1.63418_cross0.76445_n14.mtx","synthetic_46835_46835_2371250_avg50.62987_std31.37317_random_bw7354.67208_skew3.1675_neigh1.63654_cross0.75207_n14.mtx","synthetic_46835_46835_2371308_avg50.63111_std28.69655_random_bw7409.30012_skew3.1674_neigh1.638_cross0.76266_n14.mtx","synthetic_46835_46835_2371384_avg50.63273_std27.77434_random_bw7430.84482_skew3.08826_neigh1.63371_cross0.76567_n14.mtx","synthetic_46835_46835_2371440_avg50.63393_std27.7783_random_bw7484.63769_skew3.18692_neigh1.63258_cross0.77158_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_random_bw5235.2982_skew3.12766_neigh1.63485_cross0.7661_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_random_bw5745.43256_skew3.12766_neigh1.63461_cross0.76806_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_random_bw6229.28771_skew3.12766_neigh1.63338_cross0.76748_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_random_bw6661.50364_skew3.12766_neigh1.555_cross0.88602_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_random_bw6744.12087_skew3.12766_neigh1.63266_cross0.76858_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_random_bw7030.71878_skew3.12766_neigh1.60145_cross0.84428_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_random_bw7051.92288_skew3.12766_neigh1.1472_cross0.76824_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_random_bw7076.9514_skew3.12766_neigh1.25341_cross0.77005_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_random_bw7108.27014_skew3.12766_neigh1.36391_cross0.76831_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_random_bw7209.39063_skew3.12766_neigh1.57945_cross0.76788_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_random_bw7217.16063_skew3.12766_neigh1.47014_cross0.76969_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_random_bw7237.79114_skew3.12766_neigh1.63478_cross0.7678_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_random_bw7362.32181_skew3.12766_neigh1.62253_cross0.79466_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_random_bw7533.90896_skew3.12766_neigh1.68673_cross0.77234_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_random_bw7581.89267_skew3.12766_neigh1.64059_cross0.74279_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_random_bw7694.42259_skew3.12766_neigh1.63441_cross0.76753_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_random_bw7725.2259_skew3.12766_neigh1.64727_cross0.69381_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_random_bw7852.92027_skew3.12766_neigh1.65613_cross0.64131_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_random_bw7964.00444_skew3.12766_neigh1.66283_cross0.58936_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_random_bw8050.7121_skew3.12766_neigh1.66655_cross0.53913_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_random_bw8187.29021_skew3.12766_neigh1.63352_cross0.76751_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_random_bw8518.3466_skew3.12766_neigh1.89786_cross0.7699_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_random_bw8759.33535_skew3.12766_neigh1.63533_cross0.76743_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_random_bw9196.04473_skew3.12766_neigh1.63338_cross0.76885_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_random_bw9256.19861_skew3.12766_neigh1.79462_cross0.76946_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_random_bw9716.25818_skew3.12766_neigh1.63456_cross0.76651_n14.mtx","synthetic_46835_46835_2371578_avg50.63687_std27.78811_random_bw7532.72126_skew3.36441_neigh1.63252_cross0.77731_n14.mtx","synthetic_46835_46835_2371669_avg50.63882_std26.85583_random_bw7496.12448_skew3.10752_neigh1.63286_cross0.77698_n14.mtx","synthetic_46835_46835_2371693_avg50.63933_std27.78141_random_bw7520.67403_skew3.26546_neigh1.63151_cross0.77728_n14.mtx","synthetic_46835_46835_2371771_avg50.64099_std27.79721_random_bw7596.83937_skew3.60102_neigh1.63043_cross0.7826_n14.mtx","synthetic_46835_46835_2371874_avg50.64319_std27.79128_random_bw7573.47232_skew3.48234_neigh1.63324_cross0.78289_n14.mtx","synthetic_46835_46835_2371944_avg50.64469_std25.00834_random_bw7661.37113_skew3.02806_neigh1.62991_cross0.78443_n14.mtx","synthetic_46835_46835_2372249_avg50.6512_std23.16377_random_bw7721.62859_skew3.0078_neigh1.63_cross0.7957_n14.mtx","synthetic_46835_46835_2372391_avg50.65423_std21.31556_random_bw7865.19097_skew2.98782_neigh1.6257_cross0.80882_n14.mtx","synthetic_46835_46835_2372900_avg50.6651_std19.46618_random_bw7985.40892_skew2.96723_neigh1.62267_cross0.81739_n14.mtx","synthetic_46835_46835_2450659_avg52.32538_std27.78086_random_bw7558.44102_skew3.1089_neigh1.63528_cross0.77453_n14.mtx","synthetic_46835_46835_2609003_avg55.70627_std27.78687_random_bw7460.64719_skew3.03904_neigh1.63822_cross0.78391_n14.mtx","synthetic_46835_46835_2767329_avg59.08677_std27.79319_random_bw7641.74269_skew3.01105_neigh1.64146_cross0.79361_n14.mtx","synthetic_46835_46835_2925939_avg62.47334_std27.79699_random_bw7575.97229_skew3.00171_neigh1.64383_cross0.80098_n14.mtx","synthetic_46835_46835_3084303_avg65.85466_std27.80201_random_bw7666.96814_skew2.97846_neigh1.64609_cross0.80651_n14.mtx","synthetic_473242_473242_1892968_avg4.0_std0.0_random_bw643.34994_skew0.0_neigh0.37806_cross0.99029_n14.mtx","synthetic_47513_47513_1853007_avg39.0_std0.0_random_bw11630.5911_skew0.0_neigh1.40553_cross0.80427_n14.mtx","synthetic_47784_47784_2177960_avg45.57927_std31.38054_random_bw1153.05592_skew2.68589_neigh1.21393_cross0.77217_n14.mtx","synthetic_47879_47879_3068257_avg64.08356_std14.01465_random_bw343.15784_skew0.93497_neigh1.58587_cross0.83121_n14.mtx","synthetic_48396_48396_2450061_avg50.62528_std27.78261_random_bw7683.6283_skew3.14813_neigh1.6334_cross0.76917_n14.mtx","synthetic_49152_49152_1327104_avg27.0_std0.0_random_bw11888.55758_skew0.0_neigh1.39065_cross0.80136_n14.mtx","synthetic_49152_49152_1474560_avg30.0_std0.0_random_bw12201.67973_skew0.0_neigh1.39972_cross0.80562_n14.mtx","synthetic_49152_49152_1622016_avg33.0_std0.0_random_bw12114.60683_skew0.0_neigh1.3977_cross0.80501_n14.mtx","synthetic_49152_49152_1720320_avg35.0_std0.0_random_bw11976.30068_skew0.0_neigh1.40142_cross0.80302_n14.mtx","synthetic_49152_49152_1867776_avg38.0_std0.0_random_bw11962.14925_skew0.0_neigh1.40582_cross0.80314_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_random_bw10814.92206_skew0.0_neigh1.40656_cross0.80518_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_random_bw11602.43638_skew0.0_neigh1.40627_cross0.80459_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_random_bw11997.39925_skew0.0_neigh1.17226_cross0.80303_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_random_bw11999.34074_skew0.0_neigh1.41023_cross0.99003_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_random_bw12019.23385_skew0.0_neigh1.26333_cross0.80653_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_random_bw12021.09155_skew0.0_neigh1.35943_cross0.80353_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_random_bw12028.01392_skew0.0_neigh1.40369_cross0.56602_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_random_bw12033.44261_skew0.0_neigh1.40424_cross0.72534_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_random_bw12034.67322_skew0.0_neigh1.40695_cross0.78004_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_random_bw12036.00016_skew0.0_neigh1.45087_cross0.8025_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_random_bw12044.37571_skew0.0_neigh1.40455_cross0.62007_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_random_bw12045.43215_skew0.0_neigh1.40443_cross0.67088_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_random_bw12050.9789_skew0.0_neigh1.07727_cross0.806_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_random_bw12058.91376_skew0.0_neigh1.40774_cross0.83098_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_random_bw12073.32371_skew0.0_neigh1.40561_cross0.80444_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_random_bw12085.13057_skew0.0_neigh0.9811_cross0.8051_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_random_bw12096.1748_skew0.0_neigh1.40517_cross0.88682_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_random_bw12100.19297_skew0.0_neigh1.40732_cross0.93736_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_random_bw12100.23661_skew0.0_neigh1.54428_cross0.80491_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_random_bw12254.45363_skew0.0_neigh1.63837_cross0.80419_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_random_bw12401.00248_skew0.0_neigh1.40613_cross0.80523_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_random_bw12966.94114_skew0.0_neigh1.73293_cross0.80316_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_random_bw13193.13125_skew0.0_neigh1.40663_cross0.80616_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_random_bw14038.84108_skew0.0_neigh1.40505_cross0.80485_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_random_bw14826.38826_skew0.0_neigh1.4046_cross0.80484_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_random_bw15659.63725_skew0.0_neigh1.40594_cross0.80484_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_random_bw17726.56396_skew0.0_neigh1.82624_cross0.8031_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_random_bw8412.1531_skew0.0_neigh1.40648_cross0.80556_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_random_bw9217.67674_skew0.0_neigh1.40509_cross0.80535_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_random_bw9999.7901_skew0.0_neigh1.40701_cross0.8066_n14.mtx","synthetic_49152_49152_1966080_avg40.0_std0.0_random_bw12103.00734_skew0.0_neigh1.40894_cross0.80074_n14.mtx","synthetic_49152_49152_2113536_avg43.0_std0.0_random_bw12029.22158_skew0.0_neigh1.40822_cross0.80163_n14.mtx","synthetic_49152_49152_2260992_avg46.0_std0.0_random_bw11993.41239_skew0.0_neigh1.41259_cross0.8045_n14.mtx","synthetic_49152_49152_2359296_avg48.0_std0.0_random_bw12095.14152_skew0.0_neigh1.41171_cross0.80651_n14.mtx","synthetic_49152_49152_2506752_avg51.0_std0.0_random_bw12095.6193_skew0.0_neigh1.41564_cross0.80645_n14.mtx","synthetic_50367_50367_2296192_avg45.58922_std31.37796_random_bw1217.66939_skew2.68508_neigh1.21323_cross0.76936_n14.mtx","synthetic_50790_50790_1980810_avg39.0_std0.0_random_bw12402.57342_skew0.0_neigh1.40626_cross0.80547_n14.mtx","synthetic_508297_508297_2033188_avg4.0_std0.0_random_bw695.22929_skew0.0_neigh0.36764_cross0.99098_n14.mtx","synthetic_51518_51518_2608832_avg50.63923_std27.762_random_bw8203.55775_skew3.14698_neigh1.63544_cross0.76613_n14.mtx","synthetic_52042_52042_3335826_avg64.09873_std14.0_random_bw372.49024_skew0.93452_neigh1.58046_cross0.82963_n14.mtx","synthetic_525825_525825_1577449_avg2.99995_std0.00703_random_bw859.08869_skew2e-05_neigh0.3293_cross0.99111_n14.mtx","synthetic_525825_525825_1577475_avg3.0_std0.0_random_bw858.55443_skew0.0_neigh0.33003_cross0.99112_n14.mtx","synthetic_525825_525825_1583911_avg3.01224_std0.10995_random_bw797.16394_skew0.32792_neigh0.26282_cross0.98853_n14.mtx","synthetic_525825_525825_2046881_avg3.8927_std0.30949_random_bw622.52163_skew0.02756_neigh0.1362_cross0.96883_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_random_bw505.76605_skew0.0_neigh0.37416_cross0.99125_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_random_bw554.92392_skew0.0_neigh0.37192_cross0.99125_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_random_bw606.93324_skew0.0_neigh0.37258_cross0.99124_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_random_bw644.53081_skew0.0_neigh0.37119_cross0.99121_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_random_bw670.66979_skew0.0_neigh0.48412_cross0.99123_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_random_bw683.49694_skew0.0_neigh0.45923_cross0.9912_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_random_bw695.20509_skew0.0_neigh0.4342_cross0.9912_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_random_bw702.53172_skew0.0_neigh0.37156_cross0.99124_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_random_bw705.68994_skew0.0_neigh0.40842_cross0.99122_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_random_bw714.99184_skew0.0_neigh0.38412_cross0.99123_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_random_bw717.31533_skew0.0_neigh0.3775_cross0.95838_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_random_bw719.01499_skew0.0_neigh0.37393_cross0.8926_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_random_bw719.86053_skew0.0_neigh0.37171_cross0.99223_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_random_bw719.98069_skew0.0_neigh0.37726_cross0.76297_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_random_bw720.4176_skew0.0_neigh0.37153_cross0.99122_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_random_bw720.57144_skew0.0_neigh0.37696_cross0.69668_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_random_bw720.84278_skew0.0_neigh0.37761_cross0.82791_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_random_bw726.03436_skew0.0_neigh0.35739_cross0.9912_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_random_bw737.14928_skew0.0_neigh0.33322_cross0.99119_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_random_bw737.4945_skew0.0_neigh0.3712_cross0.99123_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_random_bw744.2048_skew0.0_neigh0.30997_cross0.99121_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_random_bw756.76312_skew0.0_neigh0.28196_cross0.99121_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_random_bw766.29148_skew0.0_neigh0.26012_cross0.9912_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_random_bw792.51217_skew0.0_neigh0.37127_cross0.99121_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_random_bw833.10523_skew0.0_neigh0.37123_cross0.99121_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_random_bw890.92849_skew0.0_neigh0.37114_cross0.9912_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_random_bw939.84754_skew0.0_neigh0.3724_cross0.99121_n14.mtx","synthetic_525825_525825_2146618_avg4.08238_std0.27494_random_bw490.88949_skew0.22478_neigh0.18414_cross0.97729_n14.mtx","synthetic_525825_525825_2619767_avg4.9822_std0.13221_random_bw654.36998_skew0.00357_neigh0.31537_cross0.98835_n14.mtx","synthetic_525825_525825_2629125_avg5.0_std0.0_random_bw685.43441_skew0.0_neigh0.39914_cross0.9912_n14.mtx","synthetic_525825_525825_2629143_avg5.00003_std0.00585_random_bw685.37638_skew0.19999_neigh0.3989_cross0.9912_n14.mtx","synthetic_54067_54067_2108613_avg39.0_std0.0_random_bw13195.94642_skew0.0_neigh1.40596_cross0.80602_n14.mtx","synthetic_543352_543352_2173408_avg4.0_std0.0_random_bw737.48539_skew0.0_neigh0.3826_cross0.99144_n14.mtx","synthetic_54640_54640_2767687_avg50.65313_std27.75459_random_bw8703.93389_skew3.14584_neigh1.6342_cross0.76941_n14.mtx","synthetic_56205_56205_3603711_avg64.11727_std13.99803_random_bw400.50616_skew0.93396_neigh1.57644_cross0.82872_n14.mtx","synthetic_57344_57344_2236416_avg39.0_std0.0_random_bw14033.40515_skew0.0_neigh1.40406_cross0.80608_n14.mtx","synthetic_57763_57763_2926741_avg50.66809_std27.74629_random_bw9179.53278_skew3.16436_neigh1.63113_cross0.77117_n14.mtx","synthetic_578407_578407_2313628_avg4.0_std0.0_random_bw786.14351_skew0.0_neigh0.37361_cross0.99189_n14.mtx","synthetic_60369_60369_3871327_avg64.12773_std13.99319_random_bw430.33024_skew0.93364_neigh1.57281_cross0.82791_n14.mtx","synthetic_60620_60620_2364180_avg39.0_std0.0_random_bw14835.70082_skew0.0_neigh1.4037_cross0.80546_n14.mtx","synthetic_60885_60885_3084540_avg50.66174_std27.74397_random_bw9701.3275_skew3.16488_neigh1.63253_cross0.76879_n14.mtx","synthetic_613462_613462_2453848_avg4.0_std0.0_random_bw842.88409_skew0.0_neigh0.38222_cross0.9922_n14.mtx","synthetic_62451_62451_2802494_avg44.87509_std13.98221_random_bw433.69895_skew1.33983_neigh1.53816_cross0.79072_n14.mtx","synthetic_62451_62451_3069539_avg49.15116_std13.98917_random_bw442.35894_skew1.21765_neigh1.54755_cross0.80178_n14.mtx","synthetic_62451_62451_3336828_avg53.43114_std13.99111_random_bw442.79608_skew1.11487_neigh1.55465_cross0.80814_n14.mtx","synthetic_62451_62451_3603802_avg57.70607_std13.99012_random_bw440.31634_skew1.04485_neigh1.56479_cross0.81778_n14.mtx","synthetic_62451_62451_3870981_avg61.98429_std13.98816_random_bw440.14062_skew0.96824_neigh1.57158_cross0.82571_n14.mtx","synthetic_62451_62451_4003815_avg64.1113_std18.18532_random_bw433.90471_skew1.2149_neigh1.57849_cross0.80426_n14.mtx","synthetic_62451_62451_4004004_avg64.11433_std17.25509_random_bw437.19148_skew1.1524_neigh1.57787_cross0.80809_n14.mtx","synthetic_62451_62451_4004101_avg64.11588_std16.32075_random_bw439.33923_skew1.08997_neigh1.57568_cross0.81415_n14.mtx","synthetic_62451_62451_4004374_avg64.12025_std15.38744_random_bw440.62742_skew1.02744_neigh1.5754_cross0.81871_n14.mtx","synthetic_62451_62451_4004540_avg64.12291_std14.45551_random_bw443.23719_skew0.96498_neigh1.573_cross0.82479_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_random_bw314.5764_skew0.93375_neigh1.58679_cross0.83355_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_random_bw343.48287_skew0.93375_neigh1.58569_cross0.83066_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_random_bw372.86596_skew0.93375_neigh1.57974_cross0.82953_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_random_bw400.84485_skew0.93375_neigh1.57624_cross0.82892_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_random_bw430.23839_skew0.93375_neigh1.57299_cross0.82774_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_random_bw458.26891_skew0.93375_neigh1.57086_cross0.82777_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_random_bw485.32971_skew0.93375_neigh1.56912_cross0.82716_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_random_bw514.6847_skew0.93375_neigh1.56848_cross0.82621_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_random_bw541.61408_skew0.93375_neigh1.56544_cross0.82639_n14.mtx","synthetic_62451_62451_4004768_avg64.12656_std12.59183_random_bw445.64555_skew0.84011_neigh1.57031_cross0.83683_n14.mtx","synthetic_62451_62451_4004839_avg64.1277_std13.52372_random_bw443.69469_skew0.90245_neigh1.57245_cross0.83204_n14.mtx","synthetic_62451_62451_4005045_avg64.131_std11.65916_random_bw450.22189_skew0.77761_neigh1.56813_cross0.8418_n14.mtx","synthetic_62451_62451_4005321_avg64.13542_std10.72592_random_bw451.83634_skew0.71512_neigh1.56389_cross0.84902_n14.mtx","synthetic_62451_62451_4005505_avg64.13836_std9.79513_random_bw454.37802_skew0.65268_neigh1.56244_cross0.85409_n14.mtx","synthetic_62451_62451_4138184_avg66.26289_std13.99067_random_bw440.8832_skew0.90152_neigh1.57532_cross0.83038_n14.mtx","synthetic_62451_62451_4405420_avg70.54202_std13.98985_random_bw441.29735_skew0.85705_neigh1.58148_cross0.83621_n14.mtx","synthetic_62451_62451_4672484_avg74.8184_std13.9889_random_bw446.77627_skew0.80437_neigh1.58533_cross0.84209_n14.mtx","synthetic_62451_62451_4939666_avg79.09667_std13.99063_random_bw446.66801_skew0.75734_neigh1.59109_cross0.84662_n14.mtx","synthetic_62451_62451_5206981_avg83.37706_std13.99111_random_bw448.16022_skew0.7151_neigh1.59447_cross0.85123_n14.mtx","synthetic_63897_63897_2491983_avg39.0_std0.0_random_bw15621.79835_skew0.0_neigh1.40542_cross0.80575_n14.mtx","synthetic_64532_64532_4137790_avg64.11997_std13.98737_random_bw458.31169_skew0.93387_neigh1.5704_cross0.82793_n14.mtx","synthetic_648517_648517_2594068_avg4.0_std0.0_random_bw886.62276_skew0.0_neigh0.37472_cross0.99258_n14.mtx","synthetic_683572_683572_2734288_avg4.0_std0.0_random_bw924.48268_skew0.0_neigh0.37975_cross0.9929_n14.mtx","synthetic_68696_68696_4403097_avg64.09539_std13.99668_random_bw485.33335_skew0.93462_neigh1.56937_cross0.82685_n14.mtx","synthetic_700003_700003_2220092_avg3.17155_std25.65674_random_bw87391.5121_skew1481.87283_neigh0.14428_cross0.90055_n14.mtx","synthetic_72859_72859_4668546_avg64.07645_std14.00516_random_bw513.63554_skew0.93519_neigh1.56865_cross0.82618_n14.mtx","synthetic_766670_766670_2431595_avg3.17163_std25.62988_random_bw95810.1159_skew1481.83288_neigh0.14367_cross0.90161_n14.mtx","synthetic_77022_77022_4936326_avg64.08982_std14.01488_random_bw542.85377_skew0.93478_neigh1.56641_cross0.8258_n14.mtx","synthetic_81186_81186_5203811_avg64.09739_std14.01954_random_bw572.64595_skew0.93456_neigh1.56212_cross0.82312_n14.mtx","synthetic_833337_833337_2642605_avg3.17111_std25.60679_random_bw103837.45496_skew1482.07595_neigh0.14411_cross0.90125_n14.mtx","synthetic_84834_84834_1835279_avg21.63377_std13.7861_random_bw46227.07554_skew4.03842_neigh1.01517_cross0.55965_n14.mtx","synthetic_900004_900004_2853888_avg3.17097_std25.58712_random_bw112429.88878_skew1482.14118_neigh0.14011_cross0.90178_n14.mtx","synthetic_92913_92913_2009995_avg21.63309_std13.78454_random_bw50425.51531_skew4.03858_neigh1.01427_cross0.56119_n14.mtx","synthetic_966671_966671_3064966_avg3.17064_std25.57082_random_bw120972.93224_skew1482.29662_neigh0.14283_cross0.90232_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_random_bw12314.98414_skew1.0109_neigh1.69561_cross0.82051_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_random_bw13218.95443_skew1.0109_neigh1.63894_cross0.89577_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_random_bw13302.83667_skew1.0109_neigh1.69559_cross0.82053_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_random_bw13936.21876_skew1.0109_neigh1.41417_cross0.8198_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_random_bw14004.39261_skew1.0109_neigh1.52687_cross0.8201_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_random_bw14289.87368_skew1.0109_neigh1.69536_cross0.81999_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_random_bw14335.05046_skew1.0109_neigh1.68453_cross0.84715_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_random_bw14787.04212_skew1.0109_neigh1.69513_cross0.82047_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_random_bw15130.18363_skew1.0109_neigh1.7031_cross0.79329_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_random_bw16264.19116_skew1.0109_neigh1.69516_cross0.82009_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_random_bw16429.30637_skew1.0109_neigh1.72541_cross0.63001_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_random_bw16689.96273_skew1.0109_neigh1.72925_cross0.57549_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_random_bw17252.2932_skew1.0109_neigh1.69476_cross0.8205_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_random_bw18227.45253_skew1.0109_neigh1.69515_cross0.82059_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_random_bw20444.65782_skew1.0109_neigh1.91237_cross0.8206_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_random_bw27298.86084_skew1.0109_neigh1.86289_cross0.82084_n14.mtx","synthetic_1006441_1006441_22414673_avg22.27122_std29.74727_random_bw254733.54551_skew313.62123_neigh1.26289_cross0.75654_n14.mtx","synthetic_1019593_1019593_74066178_avg72.64289_std15.80935_random_bw15286.94394_skew1.03736_neigh1.69506_cross0.8204_n14.mtx","synthetic_102778_102778_7408141_avg72.07905_std19.01282_random_bw5890.51486_skew1.19204_neigh1.65342_cross0.76938_n14.mtx","synthetic_1047423_1047423_51169710_avg48.85296_std11.94333_random_bw189507.21493_skew1.10837_neigh1.69026_cross0.8037_n14.mtx","synthetic_1060229_1060229_12992547_avg12.25447_std37.61142_random_bw24980.56284_skew632.64621_neigh1.32193_cross0.78513_n14.mtx","synthetic_1063952_1063952_23695040_avg22.27078_std29.72463_random_bw269168.5156_skew313.62752_neigh1.26394_cross0.75677_n14.mtx","synthetic_108003_108003_5987303_avg55.43645_std11.04356_random_bw4243.01828_skew0.89406_neigh1.62997_cross0.78726_n14.mtx","synthetic_108334_108334_7808284_avg72.07602_std19.0204_random_bw6217.09331_skew1.19213_neigh1.65302_cross0.76997_n14.mtx","synthetic_1085373_1085373_78800672_avg72.60239_std15.80629_random_bw16260.36911_skew1.02473_neigh1.69518_cross0.82006_n14.mtx","synthetic_1110903_1110903_54291613_avg48.87161_std11.93583_random_bw200760.47456_skew1.10756_neigh1.68972_cross0.80443_n14.mtx","synthetic_1121463_1121463_24976074_avg22.27097_std29.7046_random_bw283780.07078_skew313.62476_neigh1.26318_cross0.75684_n14.mtx","synthetic_112749_112749_5727652_avg50.80002_std19.54637_random_bw3458.33754_skew1.7559_neigh1.22842_cross0.71885_n14.mtx","synthetic_1151153_1151153_83612943_avg72.63408_std15.81129_random_bw17250.83068_skew1.01008_neigh1.69488_cross0.82066_n14.mtx","synthetic_1152423_1152423_14123090_avg12.25513_std37.58092_random_bw27186.1267_skew632.61237_neigh1.32195_cross0.78618_n14.mtx","synthetic_117394_117394_6507908_avg55.43646_std11.04171_random_bw4592.46353_skew0.89406_neigh1.63085_cross0.78812_n14.mtx","synthetic_1174383_1174383_57377479_avg48.85755_std11.94355_random_bw212160.93638_skew1.10817_neigh1.68973_cross0.80444_n14.mtx","synthetic_1216933_1216933_88363297_avg72.61147_std15.79644_random_bw18231.88915_skew1.03825_neigh1.69444_cross0.82088_n14.mtx","synthetic_123486_123486_6272129_avg50.79223_std19.54882_random_bw3787.53287_skew1.75633_neigh1.22949_cross0.71628_n14.mtx","synthetic_1237863_1237863_60496296_avg48.87156_std11.94811_random_bw223383.86547_skew1.20987_neigh1.69022_cross0.80452_n14.mtx","synthetic_1244617_1244617_15251343_avg12.25384_std37.55512_random_bw29349.06445_skew632.67869_neigh1.32246_cross0.78617_n14.mtx","synthetic_126786_126786_7028939_avg55.43939_std11.04404_random_bw4958.06756_skew0.89396_neigh1.63099_cross0.78932_n14.mtx","synthetic_1282713_1282713_93191153_avg72.6516_std15.81312_random_bw19218.72077_skew1.10594_neigh1.69435_cross0.82082_n14.mtx","synthetic_129947_129947_10504776_avg80.83893_std117.70867_random_bw17726.53455_skew7.85712_neigh1.21404_cross0.84553_n14.mtx","synthetic_1336811_1336811_16381252_avg12.25398_std37.5327_random_bw31479.27429_skew632.6718_neigh1.32292_cross0.78634_n14.mtx","synthetic_134225_134225_6816930_avg50.78733_std19.56874_random_bw4113.48627_skew1.77628_neigh1.22748_cross0.71779_n14.mtx","synthetic_136178_136178_7548659_avg55.43229_std11.05483_random_bw5329.09974_skew0.91224_neigh1.63079_cross0.78836_n14.mtx","synthetic_1382908_1382908_11844497_avg8.56492_std37.37396_random_bw40487.08926_skew633.56515_neigh1.20543_cross0.77916_n14.mtx","synthetic_1382908_1382908_12976596_avg9.38356_std37.40231_random_bw42638.77576_skew633.40762_neigh1.23946_cross0.78166_n14.mtx","synthetic_1382908_1382908_14113272_avg10.2055_std37.43376_random_bw29869.67508_skew632.97168_neigh1.26037_cross0.782_n14.mtx","synthetic_1382908_1382908_15242471_avg11.02204_std37.4678_random_bw31130.67806_skew633.00226_neigh1.26986_cross0.78321_n14.mtx","synthetic_1382908_1382908_16349573_avg11.8226_std37.50438_random_bw32032.95325_skew633.88554_neigh1.28645_cross0.78343_n14.mtx","synthetic_1382908_1382908_16848093_avg12.18309_std26.47929_random_bw31499.6008_skew636.35882_neigh1.38178_cross0.79033_n14.mtx","synthetic_1382908_1382908_16896659_avg12.21821_std28.92386_random_bw31578.67454_skew634.52686_neigh1.37766_cross0.7904_n14.mtx","synthetic_1382908_1382908_16922141_avg12.23664_std48.62398_random_bw32402.87716_skew633.48814_neigh1.28006_cross0.77973_n14.mtx","synthetic_1382908_1382908_16923406_avg12.23755_std46.1541_random_bw32550.44114_skew633.44071_neigh1.28373_cross0.78136_n14.mtx","synthetic_1382908_1382908_16923784_avg12.23782_std37.37393_random_bw32578.59823_skew443.68692_neigh1.28762_cross0.78233_n14.mtx","synthetic_1382908_1382908_16926342_avg12.23967_std43.68598_random_bw32633.88821_skew633.41236_neigh1.29181_cross0.78314_n14.mtx","synthetic_1382908_1382908_16927023_avg12.24017_std37.40259_random_bw32650.87537_skew485.75812_neigh1.29364_cross0.78327_n14.mtx","synthetic_1382908_1382908_16931896_avg12.24369_std37.43372_random_bw32726.41259_skew527.76219_neigh1.30038_cross0.78452_n14.mtx","synthetic_1382908_1382908_16931967_avg12.24374_std41.21881_random_bw32683.4221_skew633.2016_neigh1.29995_cross0.78417_n14.mtx","synthetic_1382908_1382908_16933417_avg12.24479_std31.37528_random_bw31839.54587_skew633.1473_neigh1.36655_cross0.78964_n14.mtx","synthetic_1382908_1382908_16938002_avg12.2481_std37.46714_random_bw32660.61927_skew569.78219_neigh1.30898_cross0.7854_n14.mtx","synthetic_1382908_1382908_16941719_avg12.25079_std38.75385_random_bw32663.46745_skew632.83654_neigh1.31388_cross0.78566_n14.mtx","synthetic_1382908_1382908_16943948_avg12.2524_std37.50354_random_bw32608.3439_skew611.69607_neigh1.31792_cross0.78598_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_random_bw16831.66287_skew632.66322_neigh1.72456_cross0.78841_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_random_bw22849.17172_skew632.66322_neigh1.32203_cross0.78644_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_random_bw25001.14321_skew632.66322_neigh1.32215_cross0.78651_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_random_bw26972.08395_skew632.66322_neigh1.63487_cross0.7871_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_random_bw27152.96238_skew632.66322_neigh1.32223_cross0.78644_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_random_bw28588.16229_skew632.66322_neigh0.52059_cross0.98566_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_random_bw28608.91048_skew632.66322_neigh1.10181_cross0.78592_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_random_bw28827.3735_skew632.66322_neigh0.92504_cross0.78425_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_random_bw29311.03639_skew632.66322_neigh1.3223_cross0.7864_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_random_bw29812.11226_skew632.66322_neigh1.01255_cross0.78502_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_random_bw30178.2994_skew632.66322_neigh1.18993_cross0.78639_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_random_bw31055.2687_skew632.66322_neigh1.36651_cross0.78696_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_random_bw31485.89914_skew632.66322_neigh1.32204_cross0.78641_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_random_bw31866.08413_skew632.66322_neigh1.3625_cross0.55123_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_random_bw31960.20546_skew632.66322_neigh1.35743_cross0.60367_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_random_bw32103.51377_skew632.66322_neigh1.3514_cross0.65603_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_random_bw32298.61541_skew632.66322_neigh1.342_cross0.70831_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_random_bw32527.47818_skew632.66322_neigh1.32971_cross0.7607_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_random_bw32649.42139_skew632.66322_neigh1.31221_cross0.81225_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_random_bw33004.14191_skew632.66322_neigh0.99375_cross0.96683_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_random_bw33018.43863_skew632.66322_neigh1.28269_cross0.8644_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_random_bw33385.39612_skew632.66322_neigh1.21925_cross0.91646_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_random_bw33686.12799_skew632.66322_neigh1.32166_cross0.7864_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_random_bw33939.34987_skew632.66322_neigh1.27774_cross0.78619_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_random_bw35035.98789_skew632.66322_neigh1.545_cross0.78785_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_random_bw41464.2511_skew632.66322_neigh1.45537_cross0.78695_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_random_bw42380.53696_skew632.66322_neigh1.32182_cross0.78634_n14.mtx","synthetic_1382908_1382908_16948048_avg12.25537_std37.72372_random_bw32047.73507_skew822.06782_neigh1.35662_cross0.78905_n14.mtx","synthetic_1382908_1382908_16948709_avg12.25585_std37.54211_random_bw32537.49715_skew653.62631_neigh1.32642_cross0.78675_n14.mtx","synthetic_1382908_1382908_16950422_avg12.25709_std36.29175_random_bw32502.49439_skew632.51111_neigh1.33052_cross0.78757_n14.mtx","synthetic_1382908_1382908_16950660_avg12.25726_std33.83118_random_bw32159.50342_skew632.50221_neigh1.34985_cross0.78899_n14.mtx","synthetic_1382908_1382908_16951601_avg12.25794_std37.67423_random_bw32164.80648_skew779.80014_neigh1.34967_cross0.78884_n14.mtx","synthetic_1382908_1382908_16951652_avg12.25798_std37.58358_random_bw32449.74613_skew695.60771_neigh1.33402_cross0.7879_n14.mtx","synthetic_1382908_1382908_16952713_avg12.25874_std37.62729_random_bw32268.27374_skew737.65652_neigh1.34279_cross0.78851_n14.mtx","synthetic_1382908_1382908_17445616_avg12.61517_std37.54329_random_bw32804.62199_skew635.05973_neigh1.31534_cross0.78529_n14.mtx","synthetic_1382908_1382908_18572951_avg13.43036_std37.58404_random_bw33577.21407_skew634.94726_neigh1.34983_cross0.78702_n14.mtx","synthetic_1382908_1382908_19756490_avg14.28619_std37.62781_random_bw28948.7818_skew633.10877_neigh1.36875_cross0.78902_n14.mtx","synthetic_1382908_1382908_20949793_avg15.14909_std37.6751_random_bw30001.28368_skew631.18333_neigh1.36337_cross0.78796_n14.mtx","synthetic_1382908_1382908_22070012_avg15.95913_std37.72535_random_bw30789.33457_skew631.49052_neigh1.34689_cross0.78668_n14.mtx","synthetic_140874_140874_10153211_avg72.07299_std11.05163_random_bw5566.17624_skew0.7066_neigh1.64681_cross0.80416_n14.mtx","synthetic_140874_140874_5465261_avg38.79538_std11.0515_random_bw5472.25499_skew1.31986_neigh1.60427_cross0.75763_n14.mtx","synthetic_140874_140874_5986164_avg42.49304_std11.0536_random_bw5615.53216_skew1.18859_neigh1.61127_cross0.76558_n14.mtx","synthetic_140874_140874_6506839_avg46.18907_std11.05195_random_bw5467.46501_skew1.10006_neigh1.61774_cross0.77319_n14.mtx","synthetic_140874_140874_7027827_avg49.88732_std11.05353_random_bw5578.52051_skew1.02456_neigh1.62272_cross0.78142_n14.mtx","synthetic_140874_140874_7548897_avg53.58616_std11.05321_random_bw5458.59888_skew0.9408_neigh1.6305_cross0.78364_n14.mtx","synthetic_140874_140874_7807971_avg55.42521_std14.36638_random_bw5391.79174_skew1.18312_neigh1.63652_cross0.76654_n14.mtx","synthetic_140874_140874_7808175_avg55.42666_std13.63005_random_bw5409.69514_skew1.12894_neigh1.63453_cross0.77116_n14.mtx","synthetic_140874_140874_7808374_avg55.42807_std12.89549_random_bw5451.61362_skew1.07476_neigh1.63304_cross0.77494_n14.mtx","synthetic_140874_140874_7808713_avg55.43048_std12.15675_random_bw5470.09146_skew1.00251_neigh1.63286_cross0.77934_n14.mtx","synthetic_140874_140874_7809044_avg55.43283_std11.42187_random_bw5501.94588_skew0.9483_neigh1.63246_cross0.78383_n14.mtx","synthetic_140874_140874_7809186_avg55.43383_std10.68354_random_bw5528.77301_skew0.89415_neigh1.63008_cross0.79051_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_random_bw3868.73909_skew0.91217_neigh1.62925_cross0.78945_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_random_bw4235.02181_skew0.91217_neigh1.62951_cross0.78894_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_random_bw4605.71603_skew0.91217_neigh1.63013_cross0.78855_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_random_bw4770.72113_skew0.91217_neigh1.56132_cross0.89987_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_random_bw4956.26364_skew0.91217_neigh1.63116_cross0.78903_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_random_bw5095.18835_skew0.91217_neigh1.59792_cross0.86514_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_random_bw5267.47496_skew0.91217_neigh1.14546_cross0.78537_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_random_bw5270.93334_skew0.91217_neigh1.256_cross0.78678_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_random_bw5285.20601_skew0.91217_neigh1.3622_cross0.78648_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_random_bw5319.89205_skew0.91217_neigh1.46878_cross0.78637_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_random_bw5329.16386_skew0.91217_neigh1.63054_cross0.78839_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_random_bw5371.77114_skew0.91217_neigh1.57638_cross0.78738_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_random_bw5396.20773_skew0.91217_neigh1.62321_cross0.81439_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_random_bw5522.95748_skew0.91217_neigh1.6306_cross0.7874_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_random_bw5613.49824_skew0.91217_neigh1.63776_cross0.76192_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_random_bw5693.34228_skew0.91217_neigh1.63074_cross0.78739_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_random_bw5768.02808_skew0.91217_neigh1.64697_cross0.7083_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_random_bw5802.87171_skew0.91217_neigh1.68497_cross0.78689_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_random_bw5877.42657_skew0.91217_neigh1.65421_cross0.65718_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_random_bw5969.91225_skew0.91217_neigh1.65949_cross0.60613_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_random_bw6034.73118_skew0.91217_neigh1.66318_cross0.55387_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_random_bw6073.33464_skew0.91217_neigh1.63029_cross0.78753_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_random_bw6436.17949_skew0.91217_neigh1.62906_cross0.78757_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_random_bw6525.25347_skew0.91217_neigh1.90151_cross0.7872_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_random_bw6689.33888_skew0.91217_neigh1.89964_cross0.78632_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_random_bw6800.23242_skew0.91217_neigh1.62946_cross0.78737_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_random_bw7164.09343_skew0.91217_neigh1.62998_cross0.78797_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_random_bw7287.18556_skew0.91217_neigh1.79126_cross0.78857_n14.mtx","synthetic_140874_140874_7809480_avg55.43592_std9.94668_random_bw5548.2026_skew0.82192_neigh1.62952_cross0.79532_n14.mtx","synthetic_140874_140874_7809970_avg55.4394_std9.2113_random_bw5619.82513_skew0.7677_neigh1.62745_cross0.7992_n14.mtx","synthetic_140874_140874_7810161_avg55.44076_std8.47606_random_bw5628.17949_skew0.6955_neigh1.62718_cross0.80558_n14.mtx","synthetic_140874_140874_7810358_avg55.44215_std7.74028_random_bw5680.4538_skew0.64135_neigh1.62737_cross0.8106_n14.mtx","synthetic_140874_140874_7812605_avg55.4581_std11.08015_random_bw6222.03617_skew2.11947_neigh1.61513_cross0.85242_n14.mtx","synthetic_140874_140874_7812630_avg55.45828_std11.08_random_bw6217.1087_skew2.06537_neigh1.61471_cross0.85345_n14.mtx","synthetic_140874_140874_8069568_avg57.28217_std11.05207_random_bw5567.22519_skew0.8854_neigh1.63271_cross0.78944_n14.mtx","synthetic_140874_140874_8590481_avg60.97989_std11.05315_random_bw5493.64159_skew0.83667_neigh1.63609_cross0.79354_n14.mtx","synthetic_140874_140874_9111507_avg64.67841_std11.05222_random_bw5571.17536_skew0.77803_neigh1.64018_cross0.79828_n14.mtx","synthetic_140874_140874_9632262_avg68.37502_std11.0533_random_bw5500.1344_skew0.7404_neigh1.64395_cross0.80172_n14.mtx","synthetic_142323_142323_11504673_avg80.83495_std117.71151_random_bw19450.52367_skew7.85755_neigh1.21346_cross0.84523_n14.mtx","synthetic_1429004_1429004_17510920_avg12.25393_std37.51291_random_bw33714.51176_skew632.67408_neigh1.3215_cross0.7862_n14.mtx","synthetic_144963_144963_7361596_avg50.78259_std19.57819_random_bw4444.28282_skew1.77654_neigh1.22789_cross0.7157_n14.mtx","synthetic_145569_145569_8068734_avg55.42893_std11.05658_random_bw5696.50499_skew0.91236_neigh1.63059_cross0.78723_n14.mtx","synthetic_1521198_1521198_18638978_avg12.25283_std37.49579_random_bw35839.82327_skew632.73123_neigh1.3217_cross0.78709_n14.mtx","synthetic_152542_152542_8133417_avg53.3192_std4.76067_random_bw10831.6453_skew3.36991_neigh1.81197_cross0.94119_n14.mtx","synthetic_154699_154699_12503850_avg80.82696_std117.71492_random_bw21128.24826_skew7.85843_neigh1.21484_cross0.84433_n14.mtx","synthetic_154961_154961_8588351_avg55.42266_std11.06218_random_bw6071.63273_skew0.91258_neigh1.62979_cross0.78766_n14.mtx","synthetic_155701_155701_7905533_avg50.77381_std19.58895_random_bw4775.17387_skew1.77702_neigh1.22646_cross0.71616_n14.mtx","synthetic_161070_161070_10085122_avg62.61329_std19.67482_random_bw5009.75586_skew1.44357_neigh1.23526_cross0.7413_n14.mtx","synthetic_161070_161070_10630527_avg65.99942_std19.67949_random_bw5044.97444_skew1.36366_neigh1.23861_cross0.74622_n14.mtx","synthetic_161070_161070_5763734_avg35.78403_std19.07947_random_bw4731.12239_skew2.52112_neigh1.21212_cross0.66116_n14.mtx","synthetic_161070_161070_6293192_avg39.07116_std19.27381_random_bw4796.72539_skew2.30167_neigh1.2143_cross0.67765_n14.mtx","synthetic_161070_161070_6828184_avg42.39265_std19.41502_random_bw4843.00839_skew2.13734_neigh1.21781_cross0.691_n14.mtx","synthetic_161070_161070_7366852_avg45.73696_std19.51164_random_bw4891.21219_skew1.97353_neigh1.22313_cross0.70301_n14.mtx","synthetic_161070_161070_7908198_avg49.0979_std19.57809_random_bw4902.19808_skew1.83108_neigh1.2252_cross0.71126_n14.mtx","synthetic_161070_161070_8177031_avg50.76694_std16.39176_random_bw5026.50779_skew1.48193_neigh1.22306_cross0.73804_n14.mtx","synthetic_161070_161070_8177317_avg50.76872_std15.08847_random_bw5069.23063_skew1.36366_neigh1.22345_cross0.74511_n14.mtx","synthetic_161070_161070_8177462_avg50.76962_std17.68784_random_bw4990.5517_skew1.59998_neigh1.2267_cross0.72883_n14.mtx","synthetic_161070_161070_8177930_avg50.77252_std13.78024_random_bw5097.10939_skew1.24531_neigh1.22217_cross0.75643_n14.mtx","synthetic_161070_161070_8178443_avg50.77571_std18.96773_random_bw4956.1697_skew1.71784_neigh1.22663_cross0.71981_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_random_bw3455.35798_skew1.77657_neigh1.22972_cross0.71662_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_random_bw3785.72739_skew1.77657_neigh1.22674_cross0.71723_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_random_bw4111.1518_skew1.77657_neigh1.22748_cross0.71758_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_random_bw4422.81445_skew1.77657_neigh1.19628_cross0.82221_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_random_bw4441.89847_skew1.77657_neigh1.22792_cross0.71561_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_random_bw4627.28018_skew1.77657_neigh1.21189_cross0.78656_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_random_bw4774.35335_skew1.77657_neigh1.22637_cross0.71614_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_random_bw4843.79053_skew1.77657_neigh1.22333_cross0.73927_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_random_bw4898.56338_skew1.77657_neigh1.10619_cross0.71643_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_random_bw4900.19763_skew1.77657_neigh0.94384_cross0.71595_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_random_bw4901.06241_skew1.77657_neigh0.86553_cross0.71558_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_random_bw4902.46508_skew1.77657_neigh1.02504_cross0.71613_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_random_bw4934.87943_skew1.77657_neigh1.18606_cross0.71564_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_random_bw4941.28083_skew1.77657_neigh1.22782_cross0.71491_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_random_bw4945.13241_skew1.77657_neigh1.26686_cross0.71552_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_random_bw4964.76983_skew1.77657_neigh1.34692_cross0.71497_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_random_bw4977.92274_skew1.77657_neigh1.42811_cross0.71687_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_random_bw5011.34549_skew1.77657_neigh1.50982_cross0.7154_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_random_bw5016.74181_skew1.77657_neigh1.22985_cross0.69227_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_random_bw5049.43708_skew1.77657_neigh1.58911_cross0.7181_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_random_bw5104.02865_skew1.77657_neigh1.22549_cross0.71514_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_random_bw5170.8755_skew1.77657_neigh1.23483_cross0.64645_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_random_bw5295.40467_skew1.77657_neigh1.2381_cross0.60018_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_random_bw5409.67965_skew1.77657_neigh1.24201_cross0.55228_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_random_bw5421.6343_skew1.77657_neigh1.22563_cross0.71654_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_random_bw5516.25936_skew1.77657_neigh1.24385_cross0.50438_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_random_bw5765.30102_skew1.77657_neigh1.22547_cross0.71569_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_random_bw6093.91118_skew1.77657_neigh1.22469_cross0.7147_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_random_bw6414.53412_skew1.77657_neigh1.22329_cross0.7164_n14.mtx","synthetic_161070_161070_8181013_avg50.79166_std20.23119_random_bw4921.85183_skew1.83511_neigh1.22714_cross0.71122_n14.mtx","synthetic_161070_161070_8181297_avg50.79343_std19.67479_random_bw5300.87224_skew2.30751_neigh1.21351_cross0.79956_n14.mtx","synthetic_161070_161070_8184643_avg50.8142_std21.47457_random_bw4878.01938_skew1.95193_neigh1.22869_cross0.70478_n14.mtx","synthetic_161070_161070_8190209_avg50.84876_std22.69412_random_bw4850.04792_skew2.06792_neigh1.22945_cross0.69379_n14.mtx","synthetic_161070_161070_8197860_avg50.89626_std23.8911_random_bw4817.59002_skew2.18295_neigh1.23133_cross0.68541_n14.mtx","synthetic_161070_161070_8207913_avg50.95867_std25.06294_random_bw4778.56517_skew2.29679_neigh1.23219_cross0.67699_n14.mtx","synthetic_161070_161070_8451173_avg52.46895_std19.62109_random_bw4948.49578_skew1.72542_neigh1.22786_cross0.71962_n14.mtx","synthetic_161070_161070_8995114_avg55.84599_std19.64654_random_bw4972.88478_skew1.61433_neigh1.23174_cross0.72925_n14.mtx","synthetic_161070_161070_9539943_avg59.22855_std19.66505_random_bw5000.60859_skew1.53256_neigh1.23372_cross0.73505_n14.mtx","synthetic_1613392_1613392_19770221_avg12.25382_std37.48086_random_bw38048.92289_skew632.67976_neigh1.3224_cross0.78669_n14.mtx","synthetic_164353_164353_9109408_avg55.42587_std11.0688_random_bw6425.46749_skew0.91246_neigh1.62988_cross0.78753_n14.mtx","synthetic_166438_166438_8452047_avg50.78195_std19.60687_random_bw5103.62439_skew1.77658_neigh1.22548_cross0.7151_n14.mtx","synthetic_167070_167070_8908050_avg53.31927_std4.75919_random_bw11844.75739_skew3.3699_neigh1.81234_cross0.94129_n14.mtx","synthetic_167075_167075_13504757_avg80.83051_std117.71597_random_bw22746.915_skew7.87041_neigh1.21275_cross0.84541_n14.mtx","synthetic_1705586_1705586_20897972_avg12.25266_std37.46728_random_bw40228.04846_skew632.73974_neigh1.32222_cross0.78649_n14.mtx","synthetic_173744_173744_9631043_avg55.43238_std11.0756_random_bw6798.57169_skew0.91224_neigh1.62874_cross0.78706_n14.mtx","synthetic_177177_177177_8998963_avg50.79081_std19.6143_random_bw5426.967_skew1.77609_neigh1.22568_cross0.7169_n14.mtx","synthetic_179451_179451_14506771_avg80.83973_std117.71232_random_bw24453.77012_skew7.8694_neigh1.21214_cross0.84477_n14.mtx","synthetic_1797780_1797780_22027651_avg12.2527_std37.45466_random_bw42402.70848_skew632.7381_neigh1.32199_cross0.78712_n14.mtx","synthetic_181598_181598_9682693_avg53.31938_std4.75813_random_bw12869.05691_skew3.36989_neigh1.81177_cross0.94114_n14.mtx","synthetic_183136_183136_10152501_avg55.43695_std11.07295_random_bw7165.08293_skew0.91208_neigh1.63012_cross0.78692_n14.mtx","synthetic_185639_185639_10500260_avg56.56279_std99.5812_random_bw19106.22939_skew7.18559_neigh1.25551_cross0.95442_n14.mtx","synthetic_185639_185639_11501046_avg61.95382_std109.06293_random_bw19420.97121_skew7.19966_neigh1.25396_cross0.95351_n14.mtx","synthetic_185639_185639_12507698_avg67.37646_std112.75287_random_bw24196.24969_skew7.4154_neigh1.18657_cross0.88107_n14.mtx","synthetic_185639_185639_13507262_avg72.76091_std115.00904_random_bw24859.95962_skew7.61726_neigh1.19956_cross0.86027_n14.mtx","synthetic_185639_185639_14507140_avg78.14705_std116.8824_random_bw25219.52172_skew7.80392_neigh1.20963_cross0.84859_n14.mtx","synthetic_185639_185639_15001505_avg80.81009_std108.51788_random_bw25511.99254_skew5.36059_neigh1.20064_cross0.8813_n14.mtx","synthetic_185639_185639_15005658_avg80.83247_std121.76167_random_bw25355.77027_skew10.22074_neigh1.21411_cross0.83223_n14.mtx","synthetic_185639_185639_15005807_avg80.83327_std121.07944_random_bw25384.15564_skew9.70104_neigh1.21342_cross0.83416_n14.mtx","synthetic_185639_185639_15006000_avg80.83431_std120.29592_random_bw25293.65767_skew9.18132_neigh1.21471_cross0.83668_n14.mtx","synthetic_185639_185639_15006279_avg80.83581_std111.39593_random_bw25316.2149_skew5.93999_neigh1.20276_cross0.86868_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_random_bw23672.1534_skew7.8818_neigh1.18863_cross0.87685_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_random_bw23954.47017_skew7.8818_neigh1.19585_cross0.87257_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_random_bw25265.00391_skew7.8818_neigh1.49293_cross0.84614_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_random_bw25293.93346_skew7.8818_neigh1.41168_cross0.84605_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_random_bw25297.50533_skew7.8818_neigh1.33097_cross0.84586_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_random_bw25337.47945_skew7.8818_neigh1.17285_cross0.84478_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_random_bw25357.35315_skew7.8818_neigh1.25202_cross0.84469_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_random_bw25422.38964_skew7.8818_neigh1.09355_cross0.84331_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_random_bw25459.31441_skew7.8818_neigh1.01249_cross0.84366_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_random_bw25490.88255_skew7.8818_neigh0.93357_cross0.84298_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_random_bw25498.90443_skew7.8818_neigh1.57331_cross0.84717_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_random_bw25568.41993_skew7.8818_neigh0.85133_cross0.84267_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_random_bw26253.68657_skew7.8818_neigh1.22152_cross0.81618_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_random_bw27626.6459_skew7.8818_neigh1.23351_cross0.76024_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_random_bw28571.00316_skew7.8818_neigh1.2369_cross0.70583_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_random_bw29320.71745_skew7.8818_neigh1.24185_cross0.64948_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_random_bw29911.00532_skew7.8818_neigh1.24556_cross0.59443_n14.mtx","synthetic_185639_185639_15007163_avg80.84057_std117.05079_random_bw25318.13414_skew7.60954_neigh1.20941_cross0.84686_n14.mtx","synthetic_185639_185639_15007417_avg80.84194_std113.68148_random_bw25329.82537_skew6.50848_neigh1.20564_cross0.85898_n14.mtx","synthetic_185639_185639_15007427_avg80.84199_std115.53272_random_bw25278.26792_skew7.06512_neigh1.20929_cross0.85246_n14.mtx","synthetic_185639_185639_15506741_avg83.5317_std118.48272_random_bw25422.1247_skew7.94271_neigh1.21538_cross0.84172_n14.mtx","synthetic_185639_185639_16506181_avg88.91548_std119.88133_random_bw25572.96292_skew8.06479_neigh1.21966_cross0.83617_n14.mtx","synthetic_185639_185639_17505764_avg94.30003_std121.13382_random_bw25625.10673_skew8.16224_neigh1.22371_cross0.83275_n14.mtx","synthetic_185639_185639_18505682_avg99.68639_std122.27517_random_bw25744.70036_skew8.23897_neigh1.22874_cross0.82863_n14.mtx","synthetic_187667_187667_12936634_avg68.93398_std101.26087_random_bw22637.37919_skew10.08307_neigh1.12453_cross0.83741_n14.mtx","synthetic_187915_187915_9545196_avg50.79529_std19.60651_random_bw5758.22991_skew1.77585_neigh1.22523_cross0.71562_n14.mtx","synthetic_191826_191826_15508397_avg80.84617_std117.70962_random_bw26139.99935_skew7.88106_neigh1.21134_cross0.84514_n14.mtx","synthetic_196126_196126_10457351_avg53.31955_std4.75831_random_bw14006.14808_skew3.36988_neigh1.81257_cross0.94078_n14.mtx","synthetic_198653_198653_10092676_avg50.80556_std19.60715_random_bw6093.46107_skew1.77529_neigh1.22471_cross0.71497_n14.mtx","synthetic_204202_204202_16508082_avg80.84192_std117.71063_random_bw27859.72659_skew7.88153_neigh1.21321_cross0.84455_n14.mtx","synthetic_205540_205540_14169500_avg68.93792_std101.25978_random_bw24815.66416_skew10.08243_neigh1.12421_cross0.83711_n14.mtx","synthetic_209391_209391_10637567_avg50.80241_std19.60148_random_bw6420.80853_skew1.77546_neigh1.22408_cross0.716_n14.mtx","synthetic_210654_210654_11231930_avg53.31933_std4.75723_random_bw14914.26662_skew3.3699_neigh1.81313_cross0.94073_n14.mtx","synthetic_216578_216578_17508080_avg80.83961_std117.71328_random_bw29528.4278_skew7.89416_neigh1.2132_cross0.8447_n14.mtx","synthetic_217918_217918_10485592_avg48.11714_std4.75335_random_bw15844.7796_skew3.36435_neigh1.77209_cross0.93961_n14.mtx","synthetic_217918_217918_11231492_avg51.53999_std4.76893_random_bw15995.38461_skew3.36554_neigh1.75717_cross0.93963_n14.mtx","synthetic_217918_217918_11582841_avg53.15229_std3.34176_random_bw14914.85908_skew3.38363_neigh1.83128_cross0.94011_n14.mtx","synthetic_217918_217918_11590559_avg53.18771_std3.65776_random_bw14955.00461_skew3.38071_neigh1.82908_cross0.93983_n14.mtx","synthetic_217918_217918_11598747_avg53.22528_std3.97324_random_bw15044.77643_skew3.37762_neigh1.82449_cross0.9406_n14.mtx","synthetic_217918_217918_11607157_avg53.26387_std4.28638_random_bw15200.0146_skew3.37445_neigh1.82082_cross0.9401_n14.mtx","synthetic_217918_217918_11608217_avg53.26874_std4.76281_random_bw15269.0692_skew4.08741_neigh1.81967_cross0.94009_n14.mtx","synthetic_217918_217918_11610489_avg53.27916_std4.76243_random_bw15240.4697_skew3.93626_neigh1.81896_cross0.94023_n14.mtx","synthetic_217918_217918_11612905_avg53.29025_std4.75996_random_bw15310.45462_skew3.76635_neigh1.81736_cross0.94035_n14.mtx","synthetic_217918_217918_11615389_avg53.30165_std4.60014_random_bw15350.22672_skew3.37135_neigh1.81586_cross0.94062_n14.mtx","synthetic_217918_217918_11615400_avg53.3017_std4.75902_random_bw15342.03914_skew3.61524_neigh1.81594_cross0.94062_n14.mtx","synthetic_217918_217918_11617960_avg53.31345_std4.75796_random_bw15405.69215_skew3.44541_neigh1.81369_cross0.94068_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_random_bw10737.64739_skew3.36989_neigh1.81318_cross0.94085_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_random_bw11891.67531_skew3.36989_neigh1.81288_cross0.94086_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_random_bw12724.60274_skew3.36989_neigh1.50678_cross0.94071_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_random_bw12730.20473_skew3.36989_neigh1.26613_cross0.9391_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_random_bw12738.48211_skew3.36989_neigh1.38666_cross0.94034_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_random_bw12837.98779_skew3.36989_neigh1.81264_cross0.9409_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_random_bw13058.23403_skew3.36989_neigh1.62674_cross0.94024_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_random_bw13376.58862_skew3.36989_neigh1.74964_cross0.94006_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_random_bw13808.95321_skew3.36989_neigh1.81302_cross0.94102_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_random_bw14783.21932_skew3.36989_neigh1.83882_cross0.66043_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_random_bw14834.61294_skew3.36989_neigh1.83738_cross0.72231_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_random_bw14844.13273_skew3.36989_neigh1.83615_cross0.78285_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_random_bw14910.1263_skew3.36989_neigh1.81269_cross0.94076_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_random_bw15016.8092_skew3.36989_neigh1.83172_cross0.84569_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_random_bw15289.18189_skew3.36989_neigh1.8215_cross0.90858_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_random_bw15805.89794_skew3.36989_neigh1.78683_cross0.97162_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_random_bw15930.0478_skew3.36989_neigh1.81219_cross0.94078_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_random_bw16309.5886_skew3.36989_neigh1.7473_cross0.98961_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_random_bw16886.23663_skew3.36989_neigh1.87576_cross0.93935_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_random_bw16921.52031_skew3.36989_neigh1.81297_cross0.94073_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_random_bw17934.76325_skew3.36989_neigh1.81325_cross0.94068_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_random_bw19000.93916_skew3.36989_neigh1.81255_cross0.94072_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_random_bw19930.43914_skew3.36989_neigh1.81307_cross0.94069_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_random_bw5768.22535_skew3.36989_neigh1.93911_cross0.93966_n14.mtx","synthetic_217918_217918_11620586_avg53.3255_std4.75691_random_bw15468.18448_skew3.29438_neigh1.81163_cross0.94092_n14.mtx","synthetic_217918_217918_11622834_avg53.33581_std4.91358_random_bw15511.53295_skew3.36855_neigh1.80929_cross0.94061_n14.mtx","synthetic_217918_217918_11623241_avg53.33768_std4.75547_random_bw15548.19386_skew3.12466_neigh1.8088_cross0.94062_n14.mtx","synthetic_217918_217918_11625780_avg53.34933_std4.75397_random_bw15624.44027_skew2.97381_neigh1.80583_cross0.94018_n14.mtx","synthetic_217918_217918_11628184_avg53.36036_std4.75308_random_bw15742.20805_skew2.80432_neigh1.80167_cross0.93967_n14.mtx","synthetic_217918_217918_11628620_avg53.36237_std5.22805_random_bw15759.47133_skew3.36637_neigh1.80061_cross0.93977_n14.mtx","synthetic_217918_217918_11630299_avg53.37007_std4.7531_random_bw15847.73759_skew2.65373_neigh1.79719_cross0.93963_n14.mtx","synthetic_217918_217918_11632309_avg53.37929_std5.54522_random_bw16023.97314_skew3.36499_neigh1.78977_cross0.9395_n14.mtx","synthetic_217918_217918_11634099_avg53.38751_std5.86172_random_bw16271.97895_skew3.36432_neigh1.77745_cross0.93875_n14.mtx","synthetic_217918_217918_11634741_avg53.39045_std6.17863_random_bw16478.31237_skew3.36408_neigh1.76344_cross0.93829_n14.mtx","synthetic_217918_217918_12031582_avg55.21151_std4.74931_random_bw15618.56314_skew3.36503_neigh1.82311_cross0.94108_n14.mtx","synthetic_217918_217918_12797820_avg58.72769_std4.7736_random_bw14173.92179_skew3.3591_neigh1.75513_cross0.93936_n14.mtx","synthetic_217918_217918_13561979_avg62.23432_std4.75526_random_bw13352.00925_skew3.37058_neigh1.8388_cross0.94017_n14.mtx","synthetic_217918_217918_14374351_avg65.9622_std4.76726_random_bw14674.102_skew3.36614_neigh1.78599_cross0.9393_n14.mtx","synthetic_217918_217918_15093236_avg69.26108_std4.76589_random_bw14165.84201_skew3.37475_neigh1.82997_cross0.94033_n14.mtx","synthetic_217918_217918_8143542_avg37.36975_std4.75333_random_bw21449.04695_skew3.36182_neigh1.7462_cross0.93898_n14.mtx","synthetic_217918_217918_8924884_avg40.95524_std4.76049_random_bw22967.50218_skew3.34621_neigh1.71344_cross0.93831_n14.mtx","synthetic_217918_217918_9684687_avg44.44189_std4.75752_random_bw14709.39543_skew3.36525_neigh1.77216_cross0.93925_n14.mtx","synthetic_223413_223413_15400297_avg68.93196_std101.25897_random_bw27005.84088_skew10.0979_neigh1.12422_cross0.83714_n14.mtx","synthetic_225181_225181_12006505_avg53.31935_std4.75652_random_bw15784.15411_skew3.3699_neigh1.8126_cross0.94063_n14.mtx","synthetic_228954_228954_18508358_avg80.83876_std117.71434_random_bw31230.9613_skew7.89425_neigh1.21214_cross0.84502_n14.mtx","synthetic_239709_239709_12781051_avg53.31903_std4.75618_random_bw17005.93316_skew3.36992_neigh1.81258_cross0.9406_n14.mtx","synthetic_241286_241286_16633118_avg68.93528_std101.25723_random_bw29154.84234_skew10.09737_neigh1.12198_cross0.83731_n14.mtx","synthetic_241330_241330_19509069_avg80.8398_std117.71435_random_bw32920.56295_skew7.89413_neigh1.21032_cross0.845_n14.mtx","synthetic_254237_254237_13555788_avg53.31949_std4.75562_random_bw17988.99306_skew3.36988_neigh1.81207_cross0.9405_n14.mtx","synthetic_25491_25491_3028356_avg118.80099_std31.7722_random_bw2627.38543_skew1.00335_neigh1.79456_cross0.81089_n14.mtx","synthetic_259159_259159_17866974_avg68.94213_std101.25583_random_bw31249.92925_skew10.11077_neigh1.12114_cross0.83763_n14.mtx","synthetic_26684_26684_10204693_avg382.42741_std245.44826_random_bw12208.0425_skew1.57042_neigh1.97487_cross0.9207_n14.mtx","synthetic_268096_268096_12941160_avg48.27062_std95.20699_random_bw27594.27638_skew9.2754_neigh1.08407_cross0.8943_n14.mtx","synthetic_268096_268096_14172610_avg52.86394_std97.03631_random_bw30537.79348_skew9.5554_neigh1.09138_cross0.86621_n14.mtx","synthetic_268096_268096_15403716_avg57.45597_std98.50696_random_bw31584.47846_skew9.75606_neigh1.10235_cross0.85247_n14.mtx","synthetic_268096_268096_16634948_avg62.04848_std99.72897_random_bw31984.02507_skew9.92694_neigh1.11286_cross0.84502_n14.mtx","synthetic_268096_268096_17866195_avg66.64104_std100.78017_random_bw32220.36555_skew10.05925_neigh1.12121_cross0.83959_n14.mtx","synthetic_268096_268096_18481037_avg68.9344_std103.42196_random_bw32441.44161_skew12.99882_neigh1.12449_cross0.83_n14.mtx","synthetic_268096_268096_18481246_avg68.93518_std102.1488_random_bw32365.8807_skew11.06931_neigh1.12496_cross0.83401_n14.mtx","synthetic_268096_268096_18481318_avg68.93545_std102.6344_random_bw32417.76157_skew11.72205_neigh1.12377_cross0.83241_n14.mtx","synthetic_268096_268096_18481428_avg68.93586_std103.0557_random_bw32473.21371_skew12.36025_neigh1.12371_cross0.83096_n14.mtx","synthetic_268096_268096_18481631_avg68.93662_std101.58036_random_bw32372.11142_skew10.43079_neigh1.12404_cross0.8362_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_random_bw30754.23082_skew10.11157_neigh1.10438_cross0.86329_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_random_bw31287.63614_skew10.11157_neigh1.12125_cross0.83758_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_random_bw32306.79739_skew10.11157_neigh1.15983_cross0.83743_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_random_bw32321.14848_skew10.11157_neigh1.23407_cross0.83798_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_random_bw32329.4436_skew10.11157_neigh1.01031_cross0.83738_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_random_bw32349.06004_skew10.11157_neigh1.38185_cross0.83852_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_random_bw32367.35543_skew10.11157_neigh1.08557_cross0.83678_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_random_bw32375.57571_skew10.11157_neigh0.93684_cross0.8365_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_random_bw32420.97615_skew10.11157_neigh1.30741_cross0.8385_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_random_bw32430.76354_skew10.11157_neigh0.86471_cross0.83616_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_random_bw32452.85556_skew10.11157_neigh0.79_cross0.83549_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_random_bw32557.34156_skew10.11157_neigh1.45611_cross0.83956_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_random_bw33506.09611_skew10.11157_neigh1.12211_cross0.83722_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_random_bw33650.79702_skew10.11157_neigh1.13371_cross0.80829_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_random_bw35478.95576_skew10.11157_neigh1.14236_cross0.75351_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_random_bw35561.27802_skew10.11157_neigh1.12238_cross0.83753_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_random_bw36758.75994_skew10.11157_neigh1.14871_cross0.69834_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_random_bw37755.70411_skew10.11157_neigh1.12209_cross0.83735_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_random_bw37857.24747_skew10.11157_neigh1.15306_cross0.64222_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_random_bw38714.99432_skew10.11157_neigh1.15592_cross0.5881_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_random_bw39930.72019_skew10.11157_neigh1.12081_cross0.83752_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_random_bw42070.66722_skew10.11157_neigh1.12184_cross0.83718_n14.mtx","synthetic_268096_268096_18482060_avg68.93822_std100.90606_random_bw32385.89976_skew9.79227_neigh1.12123_cross0.83833_n14.mtx","synthetic_268096_268096_18482363_avg68.93935_std100.10125_random_bw32276.82294_skew9.13935_neigh1.12203_cross0.84217_n14.mtx","synthetic_268096_268096_18482725_avg68.9407_std99.12285_random_bw32363.10438_skew8.47191_neigh1.12037_cross0.84459_n14.mtx","synthetic_268096_268096_18483147_avg68.94227_std97.91705_random_bw32207.13852_skew7.80447_neigh1.12058_cross0.85009_n14.mtx","synthetic_268096_268096_18483826_avg68.9448_std96.40124_random_bw32262.35735_skew7.10794_neigh1.11413_cross0.85517_n14.mtx","synthetic_268765_268765_14330411_avg53.31948_std4.75541_random_bw19027.02115_skew3.36988_neigh1.81207_cross0.94046_n14.mtx","synthetic_277032_277032_19096094_avg68.931_std101.25944_random_bw33505.32851_skew10.12707_neigh1.12297_cross0.83731_n14.mtx","synthetic_27919_27919_3319237_avg118.8881_std31.78107_random_bw2882.12425_skew1.00188_neigh1.79322_cross0.80796_n14.mtx","synthetic_283293_283293_15104954_avg53.31919_std4.75508_random_bw20060.27324_skew3.36991_neigh1.81213_cross0.94052_n14.mtx","synthetic_29225_29225_11176419_avg382.42666_std245.44724_random_bw13101.62433_skew1.57043_neigh1.97449_cross0.92145_n14.mtx","synthetic_294905_294905_20328825_avg68.93347_std101.25836_random_bw35580.71368_skew10.12667_neigh1.12383_cross0.83765_n14.mtx","synthetic_29496_29496_3296630_avg111.76532_std88.66285_random_bw13978.43416_skew3.35734_neigh1.89349_cross0.6274_n14.mtx","synthetic_2998_2998_6123474_avg2042.51968_std621.91091_random_bw2814.28252_skew0.46779_neigh1.82839_cross0.81924_n14.mtx","synthetic_30347_30347_3608195_avg118.89791_std31.76143_random_bw3114.986_skew1.00172_neigh1.79361_cross0.81079_n14.mtx","synthetic_312778_312778_21563736_avg68.94262_std101.25558_random_bw37752.7149_skew10.1397_neigh1.12125_cross0.83735_n14.mtx","synthetic_31766_31766_12148131_avg382.42558_std245.44713_random_bw14374.34849_skew1.57043_neigh1.97472_cross0.91791_n14.mtx","synthetic_32305_32305_3613009_avg111.84055_std88.75337_random_bw15339.91778_skew3.35441_neigh1.8933_cross0.62712_n14.mtx","synthetic_32775_32775_3898242_avg118.9395_std31.78937_random_bw3355.91933_skew1.00102_neigh1.79257_cross0.81174_n14.mtx","synthetic_3283001_3283001_13139084_avg4.00216_std1.21615_random_bw3180.83136_skew312.8308_neigh0.41641_cross0.6482_n14.mtx","synthetic_3284_3284_6742726_avg2053.20524_std666.12092_random_bw3090.98173_skew0.59945_neigh1.79209_cross0.78602_n14.mtx","synthetic_330651_330651_22801392_avg68.95909_std101.25183_random_bw39914.18466_skew10.13704_neigh1.12248_cross0.83693_n14.mtx","synthetic_34308_34308_13120227_avg382.42471_std245.44609_random_bw15373.72135_skew1.57044_neigh1.97444_cross0.91794_n14.mtx","synthetic_348524_348524_24034194_avg68.95994_std101.25128_random_bw42061.27886_skew10.1369_neigh1.12113_cross0.83729_n14.mtx","synthetic_35115_35115_3931279_avg111.95441_std88.67056_random_bw16758.071_skew3.34999_neigh1.89338_cross0.62737_n14.mtx","synthetic_35203_35203_4188100_avg118.96997_std31.72558_random_bw3622.57756_skew1.0005_neigh1.79296_cross0.80889_n14.mtx","synthetic_3569_3569_7446679_avg2086.48893_std694.14116_random_bw3307.12945_skew0.71053_neigh1.77466_cross0.76435_n14.mtx","synthetic_3595668_3595668_14390363_avg4.00214_std1.20637_random_bw3483.23396_skew312.83218_neigh0.41589_cross0.64799_n14.mtx","synthetic_36417_36417_3031503_avg83.24417_std31.62901_random_bw3778.50658_skew1.4266_neigh1.77891_cross0.76976_n14.mtx","synthetic_36417_36417_3320284_avg91.17401_std31.69422_random_bw3853.70876_skew1.30329_neigh1.78298_cross0.77984_n14.mtx","synthetic_36417_36417_3609556_avg99.11734_std31.72484_random_bw3797.36502_skew1.19941_neigh1.78594_cross0.79053_n14.mtx","synthetic_36417_36417_3899087_avg107.06777_std31.73599_random_bw3752.97905_skew1.11081_neigh1.78782_cross0.80067_n14.mtx","synthetic_36417_36417_4188724_avg115.02112_std31.74031_random_bw3797.6023_skew1.03441_neigh1.79206_cross0.80583_n14.mtx","synthetic_36417_36417_4330804_avg118.92259_std41.20033_random_bw3675.94291_skew1.30402_neigh1.79795_cross0.77953_n14.mtx","synthetic_36417_36417_4331206_avg118.93363_std39.11615_random_bw3697.36082_skew1.23654_neigh1.79608_cross0.7852_n14.mtx","synthetic_36417_36417_4331707_avg118.94739_std37.01905_random_bw3704.02521_skew1.16903_neigh1.79589_cross0.79365_n14.mtx","synthetic_36417_36417_4332360_avg118.96532_std34.9075_random_bw3710.06437_skew1.10145_neigh1.79431_cross0.8013_n14.mtx","synthetic_36417_36417_4333163_avg118.98737_std32.79928_random_bw3723.49809_skew1.03383_neigh1.79539_cross0.80667_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_random_bw2623.52629_skew1.00003_neigh1.79451_cross0.81215_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_random_bw2877.98784_skew1.00003_neigh1.79304_cross0.80927_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_random_bw3116.1024_skew1.00003_neigh1.79446_cross0.81024_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_random_bw3362.22435_skew1.00003_neigh1.79146_cross0.81185_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_random_bw3467.27641_skew1.00003_neigh1.77473_cross0.86839_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_random_bw3597.87423_skew1.00003_neigh1.61979_cross0.81098_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_random_bw3598.07988_skew1.00003_neigh1.50455_cross0.81028_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_random_bw3599.29363_skew1.00003_neigh1.27022_cross0.8107_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_random_bw3601.93981_skew1.00003_neigh1.38778_cross0.81049_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_random_bw3621.76799_skew1.00003_neigh1.79294_cross0.80868_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_random_bw3628.53802_skew1.00003_neigh1.78888_cross0.83631_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_random_bw3688.34179_skew1.00003_neigh1.73475_cross0.80937_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_random_bw3736.64409_skew1.00003_neigh1.79285_cross0.80866_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_random_bw3824.14642_skew1.00003_neigh1.79702_cross0.78418_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_random_bw3846.82772_skew1.00003_neigh1.79307_cross0.81062_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_random_bw3974.32482_skew1.00003_neigh1.80409_cross0.73066_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_random_bw4041.25675_skew1.00003_neigh1.85155_cross0.80883_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_random_bw4082.51341_skew1.00003_neigh1.80817_cross0.68078_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_random_bw4114.70692_skew1.00003_neigh1.79169_cross0.81094_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_random_bw4164.96568_skew1.00003_neigh1.81125_cross0.63039_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_random_bw4243.70544_skew1.00003_neigh1.81309_cross0.57648_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_random_bw4381.77057_skew1.00003_neigh1.79261_cross0.80821_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_random_bw4600.41277_skew1.00003_neigh1.79347_cross0.80785_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_random_bw4827.23959_skew1.00003_neigh1.79176_cross0.81018_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_random_bw5452.08949_skew1.00003_neigh1.94835_cross0.80771_n14.mtx","synthetic_36417_36417_4333866_avg119.00667_std30.68416_random_bw3732.46783_skew0.96628_neigh1.79286_cross0.81383_n14.mtx","synthetic_36417_36417_4334685_avg119.02916_std28.56617_random_bw3758.80797_skew0.89869_neigh1.79181_cross0.82202_n14.mtx","synthetic_36417_36417_4335436_avg119.04978_std26.44817_random_bw3785.04915_skew0.83117_neigh1.78962_cross0.82614_n14.mtx","synthetic_36417_36417_4336151_avg119.06942_std24.33643_random_bw3809.09534_skew0.76368_neigh1.78823_cross0.83433_n14.mtx","synthetic_36417_36417_4336875_avg119.0893_std22.21912_random_bw3824.95716_skew0.7046_neigh1.78872_cross0.84268_n14.mtx","synthetic_36417_36417_4478356_avg122.97433_std31.74172_random_bw3753.51149_skew0.96789_neigh1.79439_cross0.81446_n14.mtx","synthetic_36417_36417_4767999_avg130.92784_std31.7426_random_bw3736.97166_skew0.90945_neigh1.79577_cross0.81963_n14.mtx","synthetic_36417_36417_5057685_avg138.88253_std31.74387_random_bw3770.7999_skew0.85769_neigh1.79863_cross0.82586_n14.mtx","synthetic_36417_36417_5347290_avg146.83499_std31.74228_random_bw3749.94288_skew0.81156_neigh1.8002_cross0.83033_n14.mtx","synthetic_36417_36417_5636923_avg154.78823_std31.74188_random_bw3744.12662_skew0.77016_neigh1.80391_cross0.8341_n14.mtx","synthetic_36849_36849_14091918_avg382.42335_std245.44508_random_bw16368.21697_skew1.57045_neigh1.97457_cross0.91965_n14.mtx","synthetic_37630_37630_4477645_avg118.99136_std31.72824_random_bw3844.70563_skew1.00015_neigh1.793_cross0.81096_n14.mtx","synthetic_37924_37924_4245700_avg111.95285_std88.65794_random_bw18034.65215_skew3.35005_neigh1.89352_cross0.62758_n14.mtx","synthetic_38120_38120_10204584_avg267.69633_std171.81206_random_bw17167.82778_skew1.57008_neigh1.97267_cross0.91545_n14.mtx","synthetic_38120_38120_11176434_avg293.19082_std188.17435_random_bw18077.60719_skew1.5717_neigh1.97255_cross0.91793_n14.mtx","synthetic_38120_38120_12148318_avg318.6862_std204.53731_random_bw19160.42267_skew1.56993_neigh1.97325_cross0.91894_n14.mtx","synthetic_38120_38120_13120182_avg344.18106_std220.90077_random_bw19517.48116_skew1.57132_neigh1.97392_cross0.91784_n14.mtx","synthetic_38120_38120_14092040_avg369.67576_std237.2634_random_bw16796.84077_skew1.56982_neigh1.9745_cross0.91796_n14.mtx","synthetic_38120_38120_14186695_avg372.15884_std217.74813_random_bw17520.62065_skew1.39145_neigh1.97428_cross0.91992_n14.mtx","synthetic_38120_38120_14353565_avg376.53633_std228.90355_random_bw17193.09321_skew1.46191_neigh1.97454_cross0.91856_n14.mtx","synthetic_38120_38120_14506403_avg380.54572_std239.95763_random_bw16970.09357_skew1.5332_neigh1.97424_cross0.92361_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_random_bw16928.18211_skew1.57045_neigh1.97488_cross0.70725_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_random_bw16978.22802_skew1.57045_neigh1.97475_cross0.6468_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_random_bw16980.42762_skew1.57045_neigh1.97471_cross0.88897_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_random_bw17047.29614_skew1.57045_neigh1.9748_cross0.76884_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_random_bw17164.09806_skew1.57045_neigh1.9747_cross0.82515_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_random_bw17288.93704_skew1.57045_neigh1.97422_cross0.94898_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_random_bw17337.86668_skew1.57045_neigh1.9736_cross0.99006_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_random_bw17436.03269_skew1.57045_neigh1.90981_cross0.91796_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_random_bw17997.22335_skew1.57045_neigh1.77798_cross0.91958_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_random_bw18145.65771_skew1.57045_neigh1.65034_cross0.92138_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_random_bw18235.12718_skew1.57045_neigh1.51713_cross0.921_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_random_bw18240.43122_skew1.57045_neigh1.3892_cross0.9198_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_random_bw5058.0712_skew1.57045_neigh1.99412_cross0.92723_n14.mtx","synthetic_38120_38120_14646455_avg384.2197_std250.9036_random_bw17379.7798_skew1.60788_neigh1.97452_cross0.92139_n14.mtx","synthetic_38120_38120_14774683_avg387.5835_std261.73289_random_bw17213.87985_skew1.68071_neigh1.97461_cross0.91994_n14.mtx","synthetic_38120_38120_14892124_avg390.66432_std272.44037_random_bw17256.11215_skew1.75428_neigh1.97475_cross0.9194_n14.mtx","synthetic_38120_38120_15029193_avg394.26005_std283.24138_random_bw20307.43822_skew1.83569_neigh1.96002_cross0.918_n14.mtx","synthetic_38120_38120_15063906_avg395.17067_std253.62681_random_bw17391.16758_skew1.57104_neigh1.97506_cross0.9205_n14.mtx","synthetic_38120_38120_15246247_avg399.95401_std294.48908_random_bw20475.55488_skew1.94534_neigh1.95593_cross0.90935_n14.mtx","synthetic_38120_38120_16417883_avg430.68948_std273.30431_random_bw20513.51482_skew1.63763_neigh1.95832_cross0.8988_n14.mtx","synthetic_38120_38120_17911818_avg469.8798_std294.64729_random_bw17768.78305_skew1.77305_neigh1.96177_cross0.87832_n14.mtx","synthetic_3855_3855_8084357_avg2097.10947_std723.99546_random_bw3581.91751_skew0.83824_neigh1.7534_cross0.73513_n14.mtx","synthetic_3908334_3908334_15641636_avg4.00212_std1.19836_random_bw3788.22744_skew312.83338_neigh0.41514_cross0.64836_n14.mtx","synthetic_39390_39390_15063634_avg382.4228_std245.44522_random_bw17726.0756_skew1.57045_neigh1.97466_cross0.91891_n14.mtx","synthetic_40058_40058_4768887_avg119.04955_std31.78281_random_bw4114.1045_skew0.99917_neigh1.79221_cross0.81063_n14.mtx","synthetic_40733_40733_4573744_avg112.28596_std88.81893_random_bw19403.9372_skew3.33714_neigh1.89379_cross0.62734_n14.mtx","synthetic_4141_4141_8723864_avg2106.70466_std753.89371_random_bw3856.85245_skew0.96563_neigh1.73618_cross0.70892_n14.mtx","synthetic_41932_41932_16035737_avg382.42242_std245.44481_random_bw18664.87153_skew1.57046_neigh1.9748_cross0.9208_n14.mtx","synthetic_42138_42138_3666754_avg87.01775_std81.65352_random_bw19853.31741_skew4.24031_neigh1.89219_cross0.57666_n14.mtx","synthetic_42138_42138_3893764_avg92.40505_std83.38574_random_bw17974.59227_skew4.01055_neigh1.89331_cross0.57355_n14.mtx","synthetic_42138_42138_3928503_avg93.22946_std52.30654_random_bw27312.13121_skew1.38122_neigh1.71236_cross0.97097_n14.mtx","synthetic_42138_42138_4006220_avg95.07381_std54.75201_random_bw26439.50325_skew1.47176_neigh1.72394_cross0.96381_n14.mtx","synthetic_42138_42138_4072421_avg96.64486_std57.11798_random_bw25980.40367_skew1.5661_neigh1.74415_cross0.95788_n14.mtx","synthetic_42138_42138_4126809_avg97.93557_std85.02631_random_bw18557.95631_skew3.79907_neigh1.8936_cross0.5899_n14.mtx","synthetic_42138_42138_4128960_avg97.98662_std59.40057_random_bw25487.21109_skew1.65342_neigh1.7633_cross0.95074_n14.mtx","synthetic_42138_42138_4176699_avg99.11954_std61.5954_random_bw24959.75837_skew1.74416_neigh1.77059_cross0.94519_n14.mtx","synthetic_42138_42138_4365370_avg103.59699_std86.57531_random_bw19147.75084_skew3.60438_neigh1.89374_cross0.60541_n14.mtx","synthetic_42138_42138_4491637_avg106.5935_std66.96099_random_bw21515.96234_skew2.48989_neigh1.88186_cross0.70673_n14.mtx","synthetic_42138_42138_4533678_avg107.5912_std72.07832_random_bw21100.98446_skew2.69919_neigh1.88564_cross0.68688_n14.mtx","synthetic_42138_42138_4582731_avg108.7553_std77.01103_random_bw20786.03004_skew2.88947_neigh1.88766_cross0.66816_n14.mtx","synthetic_42138_42138_4608940_avg109.37728_std88.04095_random_bw19683.90524_skew3.42505_neigh1.89317_cross0.62053_n14.mtx","synthetic_42138_42138_4638390_avg110.07618_std81.79532_random_bw20354.8565_skew3.07899_neigh1.89046_cross0.65066_n14.mtx","synthetic_42138_42138_4699816_avg111.53391_std86.4582_random_bw20073.78231_skew3.24983_neigh1.89286_cross0.635_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_random_bw14002.42427_skew3.33605_neigh1.89425_cross0.62763_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_random_bw14320.37603_skew3.33605_neigh1.97024_cross0.65977_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_random_bw15315.12146_skew3.33605_neigh1.89405_cross0.62765_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_random_bw15931.56866_skew3.33605_neigh1.32686_cross0.62684_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_random_bw15959.80735_skew3.33605_neigh1.45483_cross0.62671_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_random_bw16095.01927_skew3.33605_neigh1.57976_cross0.62768_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_random_bw16296.61787_skew3.33605_neigh1.7053_cross0.62795_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_random_bw16555.88737_skew3.33605_neigh1.89387_cross0.6282_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_random_bw16953.95005_skew3.33605_neigh1.83072_cross0.62763_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_random_bw17957.18508_skew3.33605_neigh1.89403_cross0.62819_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_random_bw18224.68475_skew3.33605_neigh1.95842_cross0.66307_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_random_bw19284.41321_skew3.33605_neigh1.89382_cross0.62757_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_random_bw19954.55513_skew3.33605_neigh1.89279_cross0.63258_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_random_bw19982.02784_skew3.33605_neigh1.89327_cross0.62802_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_random_bw20091.36831_skew3.33605_neigh1.89497_cross0.60786_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_random_bw20234.19666_skew3.33605_neigh1.89725_cross0.563_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_random_bw20448.61299_skew3.33605_neigh1.89904_cross0.52249_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_random_bw20616.53341_skew3.33605_neigh1.90059_cross0.4835_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_random_bw20714.32773_skew3.33605_neigh1.90175_cross0.44229_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_random_bw20750.59585_skew3.33605_neigh1.8937_cross0.62968_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_random_bw21583.20827_skew3.33605_neigh1.89368_cross0.63603_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_random_bw22018.06151_skew3.33605_neigh1.89495_cross0.63936_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_random_bw22492.64431_skew3.33605_neigh1.89475_cross0.64482_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_random_bw22731.73506_skew3.33605_neigh1.89483_cross0.6471_n14.mtx","synthetic_42138_42138_4766804_avg113.12364_std90.99737_random_bw19857.30628_skew3.41994_neigh1.89446_cross0.62081_n14.mtx","synthetic_42138_42138_4838257_avg114.81933_std95.4594_random_bw19620.37349_skew3.5724_neigh1.89654_cross0.6073_n14.mtx","synthetic_42138_42138_4857446_avg115.27472_std89.41484_random_bw20263.25834_skew3.25939_neigh1.89365_cross0.63484_n14.mtx","synthetic_42138_42138_4913422_avg116.60311_std99.85384_random_bw19444.31781_skew3.72543_neigh1.89784_cross0.59642_n14.mtx","synthetic_42138_42138_4991872_avg118.46485_std104.19584_random_bw17244.71835_skew3.8622_neigh1.89896_cross0.58472_n14.mtx","synthetic_42138_42138_5073389_avg120.39938_std108.48726_random_bw17138.19232_skew4.00003_neigh1.90049_cross0.57502_n14.mtx","synthetic_42138_42138_5111019_avg121.2924_std90.68096_random_bw18636.57063_skew3.10578_neigh1.89429_cross0.64795_n14.mtx","synthetic_42138_42138_5368820_avg127.41041_std91.85872_random_bw19117.17587_skew2.96357_neigh1.89434_cross0.66066_n14.mtx","synthetic_42138_42138_5630518_avg133.62091_std92.94843_random_bw19407.61094_skew2.83174_neigh1.89425_cross0.67317_n14.mtx","synthetic_42138_42138_5895869_avg139.9181_std93.95081_random_bw19763.9757_skew2.70931_neigh1.8945_cross0.68576_n14.mtx","synthetic_4221001_4221001_16892920_avg4.00211_std1.19165_random_bw4086.05171_skew312.83427_neigh0.41605_cross0.64812_n14.mtx","synthetic_42486_42486_5059907_avg119.09587_std31.75092_random_bw4380.455_skew1.14113_neigh1.79204_cross0.80881_n14.mtx","synthetic_4284_4284_10161733_avg2372.01984_std415.11429_random_bw4053.5726_skew0.80606_neigh1.72262_cross0.77626_n14.mtx","synthetic_4284_4284_10296320_avg2403.43604_std664.88425_random_bw4000.62278_skew0.78245_neigh1.74482_cross0.76139_n14.mtx","synthetic_4284_4284_11128593_avg2597.71078_std612.25995_random_bw4020.83217_skew0.64914_neigh1.75863_cross0.79849_n14.mtx","synthetic_4284_4284_11955257_avg2790.67624_std565.77456_random_bw4036.47432_skew0.53511_neigh1.7748_cross0.83525_n14.mtx","synthetic_4284_4284_12776299_avg2982.32937_std524.4261_random_bw4061.7514_skew0.43646_neigh1.79331_cross0.868_n14.mtx","synthetic_4284_4284_5176835_avg1208.41153_std875.52905_random_bw3946.70121_skew2.54515_neigh1.71142_cross0.50146_n14.mtx","synthetic_4284_4284_6045664_avg1411.21942_std813.05724_random_bw3941.44468_skew2.03567_neigh1.70361_cross0.54989_n14.mtx","synthetic_4284_4284_6908449_avg1612.61648_std777.77715_random_bw3940.60668_skew1.65655_neigh1.70614_cross0.59717_n14.mtx","synthetic_4284_4284_7606767_avg1775.62255_std826.63023_random_bw3957.25163_skew1.41267_neigh1.72072_cross0.62851_n14.mtx","synthetic_4284_4284_7764407_avg1812.41993_std766.39496_random_bw3956.24113_skew1.36369_neigh1.71331_cross0.63304_n14.mtx","synthetic_4284_4284_7948514_avg1855.39542_std808.46009_random_bw3960.53291_skew1.30894_neigh1.72141_cross0.64445_n14.mtx","synthetic_4284_4284_8278029_avg1932.31303_std796.54977_random_bw3961.24043_skew1.21703_neigh1.72325_cross0.6619_n14.mtx","synthetic_4284_4284_8399652_avg1960.70308_std840.91619_random_bw3956.79902_skew1.18493_neigh1.7318_cross0.66546_n14.mtx","synthetic_4284_4284_8563923_avg1999.04832_std817.99013_random_bw3958.27801_skew1.14302_neigh1.73026_cross0.67074_n14.mtx","synthetic_4284_4284_8591948_avg2005.5901_std788.67453_random_bw3972.91223_skew1.13603_neigh1.72562_cross0.67423_n14.mtx","synthetic_4284_4284_8613657_avg2010.65756_std774.7613_random_bw3958.64706_skew1.13065_neigh1.72447_cross0.67632_n14.mtx","synthetic_4284_4284_8716487_avg2034.66083_std804.2751_random_bw3962.64192_skew1.10551_neigh1.73048_cross0.68001_n14.mtx","synthetic_4284_4284_8853517_avg2066.64729_std794.27725_random_bw3951.22479_skew1.07292_neigh1.7316_cross0.68495_n14.mtx","synthetic_4284_4284_8891530_avg2075.52054_std785.22497_random_bw3958.54809_skew1.06406_neigh1.7304_cross0.68853_n14.mtx","synthetic_4284_4284_8977954_avg2095.69421_std780.20568_random_bw3962.95075_skew1.04419_neigh1.73103_cross0.68744_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_random_bw2869.04809_skew1.03117_neigh1.83364_cross0.82024_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_random_bw3085.78268_skew1.03117_neigh1.80212_cross0.7937_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_random_bw3110.41713_skew1.03117_neigh1.99854_cross0.71407_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_random_bw3327.52498_skew1.03117_neigh1.77515_cross0.76336_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_random_bw3588.97456_skew1.03117_neigh1.75307_cross0.73669_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_random_bw3823.53688_skew1.03117_neigh1.73708_cross0.7132_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_random_bw3910.22712_skew1.03117_neigh1.7316_cross0.71867_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_random_bw3938.6781_skew1.03117_neigh1.73122_cross0.71141_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_random_bw3943.74907_skew1.03117_neigh1.73054_cross0.70283_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_random_bw3949.98413_skew1.03117_neigh1.7303_cross0.69957_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_random_bw3951.74276_skew1.03117_neigh1.70211_cross0.70719_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_random_bw3960.64356_skew1.03117_neigh1.82061_cross0.66726_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_random_bw3963.83333_skew1.03117_neigh1.75897_cross0.68514_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_random_bw3966.8373_skew1.03117_neigh1.72932_cross0.69638_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_random_bw3969.17367_skew1.03117_neigh1.88668_cross0.64196_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_random_bw3972.03175_skew1.03117_neigh1.64659_cross0.72312_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_random_bw3972.66503_skew1.03117_neigh1.72911_cross0.69369_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_random_bw3985.19958_skew1.03117_neigh1.5959_cross0.73688_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_random_bw3987.60574_skew1.03117_neigh1.7287_cross0.69077_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_random_bw4000.61111_skew1.03117_neigh1.72832_cross0.6858_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_random_bw4001.37395_skew1.03117_neigh1.54602_cross0.75029_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_random_bw4004.8268_skew1.03117_neigh1.9578_cross0.61085_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_random_bw4005.42204_skew1.03117_neigh1.72805_cross0.6856_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_random_bw4006.26377_skew1.03117_neigh1.72822_cross0.67994_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_random_bw4022.77614_skew1.03117_neigh1.49943_cross0.76079_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_random_bw4107.28268_skew1.03117_neigh1.72254_cross0.68493_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_random_bw4115.8254_skew1.03117_neigh1.72168_cross0.68593_n14.mtx","synthetic_4284_4284_9091186_avg2122.12558_std733.44441_random_bw3969.63002_skew1.01873_neigh1.72798_cross0.70101_n14.mtx","synthetic_4284_4284_9176988_avg2142.15406_std713.01815_random_bw3972.35014_skew0.99986_neigh1.72769_cross0.70804_n14.mtx","synthetic_4284_4284_9196527_avg2146.71499_std692.64997_random_bw3992.58567_skew0.99561_neigh1.72588_cross0.71021_n14.mtx","synthetic_4284_4284_9292087_avg2169.02124_std655.40775_random_bw3988.24556_skew0.97508_neigh1.72422_cross0.71568_n14.mtx","synthetic_4284_4284_9379803_avg2189.4965_std621.87428_random_bw4005.78431_skew0.95661_neigh1.72224_cross0.72544_n14.mtx","synthetic_4284_4284_9448827_avg2205.60854_std630.77463_random_bw4007.62162_skew0.94232_neigh1.72532_cross0.7253_n14.mtx","synthetic_4284_4284_9456745_avg2207.45682_std723.4564_random_bw3975.21195_skew0.94069_neigh1.73405_cross0.72004_n14.mtx","synthetic_4284_4284_9460554_avg2208.34594_std591.75846_random_bw3997.86905_skew0.93991_neigh1.72216_cross0.73111_n14.mtx","synthetic_4284_4284_9703417_avg2265.03665_std553.19719_random_bw4014.98599_skew0.89136_neigh1.72415_cross0.74686_n14.mtx","synthetic_4284_4284_9941769_avg2320.67437_std481.87027_random_bw4030.17951_skew0.84602_neigh1.72296_cross0.75864_n14.mtx","synthetic_43542_43542_4892832_avg112.3704_std88.7642_random_bw20697.51918_skew3.82333_neigh1.89327_cross0.62809_n14.mtx","synthetic_4426_4426_9354099_avg2113.44306_std776.70231_random_bw4087.97854_skew1.09421_neigh1.72575_cross0.6866_n14.mtx","synthetic_44473_44473_17007427_avg382.4214_std245.44406_random_bw19571.46993_skew1.57046_neigh1.97497_cross0.92019_n14.mtx","synthetic_44686_44686_9904566_avg221.64808_std96.54092_random_bw36393.77102_skew15.40889_neigh1.70401_cross0.85291_n14.mtx","synthetic_44914_44914_5350806_avg119.13448_std31.75488_random_bw4611.11074_skew1.14044_neigh1.79301_cross0.80788_n14.mtx","synthetic_4533668_4533668_18144199_avg4.0021_std1.18568_random_bw4391.88464_skew312.83513_neigh0.41656_cross0.6482_n14.mtx","synthetic_46351_46351_5209319_avg112.38849_std88.86396_random_bw22009.55751_skew3.82256_neigh1.89342_cross0.62869_n14.mtx","synthetic_46524_46524_7261288_avg156.07618_std357.60767_random_bw27277.78536_skew297.08521_neigh1.84057_cross0.94343_n14.mtx","synthetic_4690002_4690002_14082483_avg3.00266_std1.15153_random_bw5393.19648_skew319.7156_neigh0.36894_cross0.64814_n14.mtx","synthetic_4690002_4690002_14083544_avg3.00289_std1.14359_random_bw5393.22819_skew291.71835_neigh0.36895_cross0.64814_n14.mtx","synthetic_4690002_4690002_18765165_avg4.0011_std0.88465_random_bw4545.45148_skew312.91371_neigh0.41524_cross0.6488_n14.mtx","synthetic_4690002_4690002_18766060_avg4.00129_std0.94875_random_bw4545.42947_skew312.89874_neigh0.41525_cross0.6488_n14.mtx","synthetic_4690002_4690002_18767037_avg4.0015_std1.01413_random_bw4545.42698_skew312.88239_neigh0.41527_cross0.6488_n14.mtx","synthetic_4690002_4690002_18767925_avg4.00169_std1.23629_random_bw4545.43862_skew406.578_neigh0.4153_cross0.6488_n14.mtx","synthetic_4690002_4690002_18768094_avg4.00172_std1.08092_random_bw4545.42342_skew312.86472_neigh0.41527_cross0.6488_n14.mtx","synthetic_4690002_4690002_18768255_avg4.00176_std1.22278_random_bw4545.43101_skew385.58006_neigh0.41529_cross0.6488_n14.mtx","synthetic_4690002_4690002_18768639_avg4.00184_std1.21074_random_bw4545.44234_skew364.83169_neigh0.41529_cross0.6488_n14.mtx","synthetic_4690002_4690002_18769069_avg4.00193_std1.19917_random_bw4545.4511_skew344.08333_neigh0.4153_cross0.6488_n14.mtx","synthetic_4690002_4690002_18769236_avg4.00197_std1.1487_random_bw4545.4197_skew312.84562_neigh0.41529_cross0.6488_n14.mtx","synthetic_4690002_4690002_18769566_avg4.00204_std1.18832_random_bw4545.4388_skew323.33476_neigh0.41528_cross0.6488_n14.mtx","synthetic_4690002_4690002_18769585_avg4.00204_std1.18853_random_bw4545.45064_skew323.33443_neigh0.41531_cross0.6488_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_random_bw3185.35558_skew312.8356_neigh0.41546_cross0.64898_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_random_bw3466.9726_skew312.8356_neigh0.41544_cross0.64891_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_random_bw3796.51273_skew312.8356_neigh0.41538_cross0.64888_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_random_bw4075.34022_skew312.8356_neigh0.41535_cross0.64886_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_random_bw4201.23754_skew312.8356_neigh0.54031_cross0.64874_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_random_bw4280.60386_skew312.8356_neigh0.51221_cross0.64875_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_random_bw4358.8812_skew312.8356_neigh0.48441_cross0.64883_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_random_bw4407.96081_skew312.8356_neigh0.41528_cross0.64881_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_random_bw4434.70451_skew312.8356_neigh0.45658_cross0.6486_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_random_bw4508.95932_skew312.8356_neigh0.429_cross0.64869_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_random_bw4543.29273_skew312.8356_neigh0.41514_cross0.58411_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_random_bw4544.09158_skew312.8356_neigh0.4154_cross0.62714_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_random_bw4544.19255_skew312.8356_neigh0.41607_cross0.84262_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_random_bw4544.8615_skew312.8356_neigh0.41542_cross0.45413_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_random_bw4545.23971_skew312.8356_neigh0.41497_cross0.67016_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_random_bw4545.54195_skew312.8356_neigh0.41601_cross0.79982_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_random_bw4545.90204_skew312.8356_neigh0.41502_cross0.54071_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_random_bw4546.18642_skew312.8356_neigh0.4153_cross0.49753_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_random_bw4547.86071_skew312.8356_neigh0.41468_cross0.71352_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_random_bw4548.10776_skew312.8356_neigh0.41515_cross0.7566_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_random_bw4582.29087_skew312.8356_neigh0.40122_cross0.6487_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_random_bw4653.77434_skew312.8356_neigh0.37381_cross0.64862_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_random_bw4685.95172_skew312.8356_neigh0.41524_cross0.64878_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_random_bw4721.25686_skew312.8356_neigh0.34632_cross0.64851_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_random_bw4788.70054_skew312.8356_neigh0.31846_cross0.64863_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_random_bw4855.60684_skew312.8356_neigh0.29083_cross0.64871_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_random_bw5014.52865_skew312.8356_neigh0.41525_cross0.64877_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_random_bw5290.58011_skew312.8356_neigh0.41527_cross0.64873_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_random_bw5624.25737_skew312.8356_neigh0.41525_cross0.64872_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_random_bw5905.56921_skew312.8356_neigh0.4152_cross0.64868_n14.mtx","synthetic_4690002_4690002_18770121_avg4.00216_std1.17768_random_bw4545.45419_skew302.33648_neigh0.4153_cross0.6488_n14.mtx","synthetic_4690002_4690002_18770136_avg4.00216_std1.17836_random_bw4545.46735_skew302.5861_neigh0.4153_cross0.6488_n14.mtx","synthetic_4690002_4690002_18770459_avg4.00223_std1.21745_random_bw4545.43962_skew312.82517_neigh0.4153_cross0.6488_n14.mtx","synthetic_4690002_4690002_18770757_avg4.00229_std1.16774_random_bw4545.45494_skew281.58808_neigh0.4153_cross0.6488_n14.mtx","synthetic_4690002_4690002_18770794_avg4.0023_std1.16864_random_bw4545.45869_skew281.83738_neigh0.4153_cross0.6488_n14.mtx","synthetic_4690002_4690002_18771502_avg4.00245_std1.15845_random_bw4545.49654_skew260.58973_neigh0.41529_cross0.6488_n14.mtx","synthetic_4690002_4690002_18771567_avg4.00246_std1.15985_random_bw4545.48401_skew260.83867_neigh0.41529_cross0.6488_n14.mtx","synthetic_4690002_4690002_18771765_avg4.00251_std1.28647_random_bw4545.47248_skew312.80334_neigh0.41532_cross0.6488_n14.mtx","synthetic_4690002_4690002_18773155_avg4.0028_std1.35648_random_bw4545.50339_skew312.7801_neigh0.41532_cross0.6488_n14.mtx","synthetic_4690002_4690002_18773573_avg4.00289_std1.14442_random_bw4545.51523_skew219.34068_neigh0.4153_cross0.6488_n14.mtx","synthetic_4690002_4690002_18774622_avg4.00312_std1.42675_random_bw4545.47999_skew312.75559_neigh0.41534_cross0.6488_n14.mtx","synthetic_4690002_4690002_18776169_avg4.00345_std1.497_random_bw4545.44654_skew312.72973_neigh0.41535_cross0.6488_n14.mtx","synthetic_4690002_4690002_23458256_avg5.00176_std1.22307_random_bw4356.33337_skew308.6911_neigh0.44326_cross0.64833_n14.mtx","synthetic_4690002_4690002_23458625_avg5.00184_std1.21051_random_bw4356.3482_skew292.09232_neigh0.44326_cross0.64833_n14.mtx","synthetic_4690002_4690002_23459038_avg5.00192_std1.19839_random_bw4356.34599_skew275.29363_neigh0.44327_cross0.64833_n14.mtx","synthetic_4690002_4690002_28147894_avg6.00168_std1.23536_random_bw4777.70775_skew271.09045_neigh0.46196_cross0.6485_n14.mtx","synthetic_47014_47014_17979151_avg382.42121_std245.44364_random_bw21213.06834_skew1.57046_neigh1.97471_cross0.91776_n14.mtx","synthetic_4712_4712_10000031_avg2122.24767_std807.79622_random_bw4350.07343_skew1.22029_neigh1.71541_cross0.66543_n14.mtx","synthetic_47342_47342_5640490_avg119.14347_std31.78033_random_bw4829.64226_skew1.14028_neigh1.79166_cross0.81115_n14.mtx","synthetic_47684_47684_3704607_avg77.69078_std983.84259_random_bw1475.52561_skew612.76655_neigh1.33846_cross0.83857_n14.mtx","synthetic_4846335_4846335_19395478_avg4.00209_std1.18059_random_bw4694.95912_skew312.83587_neigh0.41556_cross0.64846_n14.mtx","synthetic_48942_48942_10847397_avg221.6378_std96.4853_random_bw40035.34643_skew15.40966_neigh1.70398_cross0.84816_n14.mtx","synthetic_49161_49161_5521770_avg112.32013_std88.82977_random_bw23368.76097_skew3.82549_neigh1.89374_cross0.62848_n14.mtx","synthetic_49556_49556_18951229_avg382.42047_std245.44344_random_bw21913.94158_skew1.57047_neigh1.97494_cross0.91632_n14.mtx","synthetic_4998_4998_10667079_avg2134.26951_std834.36333_random_bw4642.41957_skew1.34178_neigh1.70696_cross0.64169_n14.mtx","synthetic_50954_50954_7955612_avg156.13322_std368.01601_random_bw29951.25091_skew325.34951_neigh1.84107_cross0.94282_n14.mtx","synthetic_5159002_5159002_20646759_avg4.00208_std1.17597_random_bw5000.15374_skew312.8365_neigh0.41584_cross0.64845_n14.mtx","synthetic_51970_51970_5840112_avg112.37468_std88.844_random_bw24618.90708_skew3.82315_neigh1.89384_cross0.62918_n14.mtx","synthetic_52226_52226_4093416_avg78.37889_std1019.1394_random_bw1610.58794_skew665.32736_neigh1.33958_cross0.83862_n14.mtx","synthetic_5283_5283_11298374_avg2138.62843_std877.77445_random_bw4852.64547_skew1.47027_neigh1.70262_cross0.62729_n14.mtx","synthetic_53198_53198_11791218_avg221.64777_std96.43769_random_bw43503.74766_skew15.40892_neigh1.70456_cross0.84719_n14.mtx","synthetic_5471669_5471669_21898040_avg4.00208_std1.17221_random_bw5302.30865_skew312.83705_neigh0.41568_cross0.64833_n14.mtx","synthetic_54779_54779_6163317_avg112.5124_std88.83086_random_bw26024.3978_skew3.81725_neigh1.89353_cross0.62937_n14.mtx","synthetic_55385_55385_8650205_avg156.18317_std377.56925_random_bw32639.28862_skew353.61567_neigh1.83893_cross0.94232_n14.mtx","synthetic_5569_5569_11949345_avg2145.68953_std911.81863_random_bw5136.07919_skew1.59544_neigh1.69647_cross0.60886_n14.mtx","synthetic_56767_56767_4479663_avg78.91315_std1046.34011_random_bw1746.70014_skew718.36043_neigh1.34074_cross0.83789_n14.mtx","synthetic_57454_57454_12734908_avg221.65398_std96.39664_random_bw46842.79518_skew15.40846_neigh1.70371_cross0.85054_n14.mtx","synthetic_5784335_5784335_23149312_avg4.00207_std1.16847_random_bw5605.51869_skew312.83761_neigh0.41582_cross0.64832_n14.mtx","synthetic_58333_58333_4204101_avg72.07071_std19.00931_random_bw3352.53952_skew1.13679_neigh1.65318_cross0.7705_n14.mtx","synthetic_59816_59816_9344868_avg156.2269_std386.43932_random_bw35412.98975_skew381.87902_neigh1.83858_cross0.94305_n14.mtx","synthetic_603864_603864_13450351_avg22.27381_std30.02405_random_bw152862.83998_skew313.58473_neigh1.26422_cross0.75345_n14.mtx","synthetic_6097002_6097002_24400590_avg4.00206_std1.16523_random_bw5910.6367_skew312.83809_neigh0.41581_cross0.64818_n14.mtx","synthetic_61308_61308_4858098_avg79.24085_std1065.00271_random_bw1888.99212_skew772.69186_neigh1.34152_cross0.83808_n14.mtx","synthetic_61710_61710_13678136_avg221.65186_std96.36424_random_bw50412.03223_skew15.40862_neigh1.70378_cross0.84921_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_random_bw399.28715_skew0.93375_neigh1.57754_cross0.89197_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_random_bw427.54729_skew0.93375_neigh1.17987_cross0.83144_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_random_bw430.40577_skew0.93375_neigh1.2674_cross0.82956_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_random_bw430.53472_skew0.93375_neigh1.35298_cross0.83078_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_random_bw431.15425_skew0.93375_neigh1.56885_cross0.85081_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_random_bw432.49407_skew0.93375_neigh1.44041_cross0.82902_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_random_bw437.05305_skew0.93375_neigh1.52957_cross0.82898_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_random_bw443.57612_skew0.93375_neigh1.61781_cross0.82827_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_random_bw444.27433_skew0.93375_neigh1.57185_cross0.82753_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_random_bw453.26434_skew0.93375_neigh1.57391_cross0.80665_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_random_bw471.54689_skew0.93375_neigh1.58217_cross0.75806_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_random_bw472.59519_skew0.93375_neigh1.70735_cross0.82682_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_random_bw482.56044_skew0.93375_neigh1.58512_cross0.71559_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_random_bw493.41782_skew0.93375_neigh1.58831_cross0.66574_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_random_bw501.92205_skew0.93375_neigh1.59187_cross0.621_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_random_bw533.47514_skew0.93375_neigh1.8037_cross0.82219_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_random_bw573.2297_skew0.93375_neigh1.56189_cross0.82326_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_random_bw653.65655_skew0.93375_neigh1.90485_cross0.81986_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_random_bw683.09787_skew0.93375_neigh1.8984_cross0.81849_n14.mtx","synthetic_63838_63838_10847423_avg169.9211_std96.15794_random_bw50815.19321_skew15.38408_neigh1.69783_cross0.83843_n14.mtx","synthetic_63838_63838_11790856_avg184.69965_std96.20654_random_bw51110.24539_skew15.38877_neigh1.69913_cross0.84602_n14.mtx","synthetic_63838_63838_12734460_avg199.48087_std96.25772_random_bw51593.46717_skew15.39756_neigh1.70252_cross0.84507_n14.mtx","synthetic_63838_63838_13677970_avg214.26063_std96.31751_random_bw51973.46416_skew15.40525_neigh1.70373_cross0.84684_n14.mtx","synthetic_63838_63838_14148710_avg221.63461_std96.12924_random_bw51283.67085_skew11.06039_neigh1.70527_cross0.84272_n14.mtx","synthetic_63838_63838_14148875_avg221.63719_std96.16987_random_bw51535.13301_skew12.02579_neigh1.70395_cross0.8451_n14.mtx","synthetic_63838_63838_14148974_avg221.63874_std125.01008_random_bw50694.22474_skew15.38251_neigh1.706_cross0.83858_n14.mtx","synthetic_63838_63838_14149076_avg221.64034_std96.21451_random_bw51658.38851_skew12.99114_neigh1.70423_cross0.84718_n14.mtx","synthetic_63838_63838_14149188_avg221.64209_std118.63327_random_bw50889.44055_skew15.38678_neigh1.70517_cross0.84295_n14.mtx","synthetic_63838_63838_14149208_avg221.64241_std96.26387_random_bw51855.74965_skew13.96104_neigh1.70421_cross0.84669_n14.mtx","synthetic_63838_63838_14149296_avg221.64379_std112.26231_random_bw51274.57478_skew15.39568_neigh1.70495_cross0.84257_n14.mtx","synthetic_63838_63838_14149329_avg221.6443_std105.89153_random_bw51549.08313_skew15.40015_neigh1.70303_cross0.84806_n14.mtx","synthetic_63838_63838_14149335_avg221.6444_std96.31796_random_bw52140.13924_skew14.92641_neigh1.70443_cross0.84646_n14.mtx","synthetic_63838_63838_14149338_avg221.64444_std99.52693_random_bw52057.82066_skew15.40465_neigh1.70386_cross0.84887_n14.mtx","synthetic_63838_63838_14149538_avg221.64758_std93.16614_random_bw52341.45014_skew15.40893_neigh1.70422_cross0.84685_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_random_bw31553.90656_skew15.40883_neigh1.97927_cross0.90185_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_random_bw36435.81718_skew15.40883_neigh1.70415_cross0.85204_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_random_bw40001.91804_skew15.40883_neigh1.70412_cross0.84856_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_random_bw41994.67554_skew15.40883_neigh1.62327_cross0.97823_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_random_bw43525.69963_skew15.40883_neigh1.70435_cross0.8478_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_random_bw46863.79909_skew15.40883_neigh1.70408_cross0.85043_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_random_bw48556.48783_skew15.40883_neigh1.67822_cross0.93499_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_random_bw50400.40091_skew15.40883_neigh1.70389_cross0.84914_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_random_bw51440.8804_skew15.40883_neigh1.69826_cross0.87894_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_random_bw51860.61629_skew15.40883_neigh1.19383_cross0.8502_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_random_bw51860.91715_skew15.40883_neigh1.30912_cross0.84912_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_random_bw51934.71139_skew15.40883_neigh1.53425_cross0.84942_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_random_bw51974.98208_skew15.40883_neigh1.42134_cross0.8471_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_random_bw52133.12723_skew15.40883_neigh1.64881_cross0.84612_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_random_bw52424.50489_skew15.40883_neigh1.76191_cross0.85027_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_random_bw52642.58633_skew15.40883_neigh1.70867_cross0.81986_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_random_bw53211.08631_skew15.40883_neigh1.87632_cross0.84853_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_random_bw53321.33316_skew15.40883_neigh1.71205_cross0.7625_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_random_bw53759.27571_skew15.40883_neigh1.71483_cross0.70856_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_random_bw53829.21378_skew15.40883_neigh1.70392_cross0.84808_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_random_bw54127.70973_skew15.40883_neigh1.71701_cross0.64906_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_random_bw54345.60945_skew15.40883_neigh1.71839_cross0.59238_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_random_bw57263.41878_skew15.40883_neigh1.70375_cross0.85062_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_random_bw58199.76162_skew15.40883_neigh1.70478_cross0.85336_n14.mtx","synthetic_63838_63838_14149682_avg221.64983_std96.37657_random_bw52178.12439_skew15.887_neigh1.70295_cross0.84932_n14.mtx","synthetic_63838_63838_14149757_avg221.65101_std96.43499_random_bw52389.17814_skew16.85239_neigh1.70325_cross0.85077_n14.mtx","synthetic_63838_63838_14149836_avg221.65224_std86.81097_random_bw52747.35418_skew15.4131_neigh1.7044_cross0.85031_n14.mtx","synthetic_63838_63838_14149837_avg221.65226_std96.50454_random_bw52563.14853_skew17.81776_neigh1.70366_cross0.84972_n14.mtx","synthetic_63838_63838_14149882_avg221.65297_std80.46101_random_bw53023.30211_skew15.41756_neigh1.70467_cross0.84957_n14.mtx","synthetic_63838_63838_14149914_avg221.65347_std74.11814_random_bw53380.64075_skew15.42203_neigh1.70407_cross0.85405_n14.mtx","synthetic_63838_63838_14150121_avg221.65671_std67.78709_random_bw53636.29238_skew15.4263_neigh1.7049_cross0.85444_n14.mtx","synthetic_63838_63838_14150132_avg221.65688_std96.57304_random_bw52625.73423_skew18.78283_neigh1.70324_cross0.85136_n14.mtx","synthetic_63838_63838_14150259_avg221.65887_std96.64741_random_bw52705.54984_skew19.7481_neigh1.705_cross0.85168_n14.mtx","synthetic_63838_63838_14621335_avg229.03811_std96.37884_random_bw52333.38261_skew15.40775_neigh1.70591_cross0.8492_n14.mtx","synthetic_63838_63838_15564722_avg243.81594_std96.44368_random_bw52583.81193_skew15.41402_neigh1.70736_cross0.85221_n14.mtx","synthetic_63838_63838_16508065_avg258.59308_std96.5168_random_bw52719.01902_skew15.41575_neigh1.70764_cross0.8531_n14.mtx","synthetic_63838_63838_17451479_avg273.37133_std96.59121_random_bw53072.43958_skew15.41723_neigh1.70777_cross0.85229_n14.mtx","synthetic_63838_63838_18395041_avg288.1519_std96.67009_random_bw53224.04721_skew15.42189_neigh1.71047_cross0.85161_n14.mtx","synthetic_63838_63838_9903973_avg155.14228_std96.11613_random_bw50204.46286_skew15.37207_neigh1.69502_cross0.83641_n14.mtx","synthetic_63889_63889_4604376_avg72.06837_std18.98718_random_bw3672.48262_skew1.13686_neigh1.6535_cross0.76942_n14.mtx","synthetic_64247_64247_10039260_avg156.26037_std394.7431_random_bw37749.51279_skew410.15351_neigh1.83858_cross0.94311_n14.mtx","synthetic_65850_65850_5226388_avg79.36808_std1075.21227_random_bw2021.34421_skew828.67864_neigh1.34471_cross0.83794_n14.mtx","synthetic_65965_65965_14620660_avg221.64269_std96.33127_random_bw53946.07292_skew15.40929_neigh1.7042_cross0.84826_n14.mtx","synthetic_661375_661375_14730847_avg22.27306_std29.96346_random_bw167527.31499_skew313.59526_neigh1.26295_cross0.75385_n14.mtx","synthetic_66463_66463_10040261_avg151.06542_std395.7133_random_bw39089.18279_skew424.82214_neigh1.83705_cross0.94111_n14.mtx","synthetic_66463_66463_10375238_avg156.10547_std374.16577_random_bw37885.35229_skew297.97094_neigh1.84271_cross0.9369_n14.mtx","synthetic_66463_66463_10377678_avg156.14218_std378.88433_random_bw38300.96201_skew326.18897_neigh1.84277_cross0.93802_n14.mtx","synthetic_66463_66463_10378609_avg156.15619_std376.2538_random_bw40147.56841_skew424.61873_neigh1.83726_cross0.94556_n14.mtx","synthetic_66463_66463_10380253_avg156.18093_std381.55088_random_bw40132.11695_skew424.55132_neigh1.83814_cross0.94385_n14.mtx","synthetic_66463_66463_10380322_avg156.18197_std384.04911_random_bw38766.29347_skew354.38034_neigh1.84508_cross0.93857_n14.mtx","synthetic_66463_66463_10381996_avg156.20715_std386.69205_random_bw39470.08879_skew424.47988_neigh1.83953_cross0.9449_n14.mtx","synthetic_66463_66463_10382686_avg156.21753_std389.6586_random_bw38980.4995_skew382.5741_neigh1.84251_cross0.94018_n14.mtx","synthetic_66463_66463_10383864_avg156.23526_std391.65048_random_bw39602.28441_skew424.40333_neigh1.84213_cross0.94243_n14.mtx","synthetic_66463_66463_10384998_avg156.25232_std492.58539_random_bw37240.82965_skew424.35688_neigh1.85054_cross0.93288_n14.mtx","synthetic_66463_66463_10385329_avg156.2573_std395.72064_random_bw38976.54612_skew410.73756_neigh1.84231_cross0.94152_n14.mtx","synthetic_66463_66463_10385409_avg156.2585_std471.2363_random_bw37706.06005_skew424.34005_neigh1.84635_cross0.93626_n14.mtx","synthetic_66463_66463_10385572_avg156.26096_std396.41268_random_bw39282.12082_skew424.33337_neigh1.84007_cross0.94251_n14.mtx","synthetic_66463_66463_10385645_avg156.26206_std450.123_random_bw38391.32296_skew424.33038_neigh1.84642_cross0.93565_n14.mtx","synthetic_66463_66463_10386025_avg156.26777_std429.29508_random_bw38743.10672_skew424.31482_neigh1.84579_cross0.93782_n14.mtx","synthetic_66463_66463_10386363_avg156.27286_std408.80831_random_bw38921.95154_skew424.30098_neigh1.84281_cross0.94099_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_random_bw27380.53228_skew424.29959_neigh1.83971_cross0.94311_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_random_bw30115.66172_skew424.29959_neigh1.83841_cross0.94227_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_random_bw32802.33897_skew424.29959_neigh1.8391_cross0.94285_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_random_bw33891.3234_skew424.29959_neigh1.30566_cross0.9399_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_random_bw33914.40305_skew424.29959_neigh1.42021_cross0.9414_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_random_bw34216.51382_skew424.29959_neigh1.54427_cross0.94023_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_random_bw34386.70958_skew424.29959_neigh1.66268_cross0.94107_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_random_bw35355.46844_skew424.29959_neigh1.83904_cross0.94283_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_random_bw35880.75824_skew424.29959_neigh1.78051_cross0.94121_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_random_bw36358.1962_skew424.29959_neigh1.75925_cross0.97917_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_random_bw36828.67158_skew424.29959_neigh1.94766_cross0.95084_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_random_bw37162.55769_skew424.29959_neigh1.78345_cross0.9726_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_random_bw37752.50297_skew424.29959_neigh1.83874_cross0.94308_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_random_bw39749.89256_skew424.29959_neigh1.86406_cross0.91226_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_random_bw39793.17787_skew424.29959_neigh1.90076_cross0.72217_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_random_bw39841.02385_skew424.29959_neigh1.90364_cross0.65888_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_random_bw40073.12002_skew424.29959_neigh1.88547_cross0.84863_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_random_bw40078.15401_skew424.29959_neigh1.89472_cross0.78617_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_random_bw40874.04356_skew424.29959_neigh1.838_cross0.94255_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_random_bw43013.40304_skew424.29959_neigh1.92851_cross0.95044_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_random_bw43275.46909_skew424.29959_neigh1.8389_cross0.94246_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_random_bw45759.36917_skew424.29959_neigh1.83956_cross0.94233_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_random_bw48163.28926_skew424.29959_neigh1.84119_cross0.9421_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_random_bw50080.08679_skew424.29959_neigh1.84023_cross0.94336_n14.mtx","synthetic_66463_66463_10386663_avg156.27737_std388.73672_random_bw39544.24019_skew424.2887_neigh1.83892_cross0.9432_n14.mtx","synthetic_66463_66463_10387014_avg156.28265_std369.1724_random_bw40157.88734_skew424.27433_neigh1.8384_cross0.94372_n14.mtx","synthetic_66463_66463_10387408_avg156.28858_std350.24164_random_bw40130.74235_skew424.25819_neigh1.83783_cross0.94626_n14.mtx","synthetic_66463_66463_10387855_avg156.29531_std332.11193_random_bw40571.57874_skew424.23989_neigh1.8371_cross0.9482_n14.mtx","synthetic_66463_66463_10388533_avg156.30551_std315.01147_random_bw41077.64824_skew424.21214_neigh1.83814_cross0.94745_n14.mtx","synthetic_66463_66463_10730769_avg161.45478_std396.39605_random_bw39721.11865_skew410.65087_neigh1.8442_cross0.94219_n14.mtx","synthetic_66463_66463_11418942_avg171.80901_std391.59435_random_bw40372.93939_skew385.84235_neigh1.84536_cross0.94343_n14.mtx","synthetic_66463_66463_12107419_avg182.16781_std386.59381_random_bw38834.28252_skew363.84492_neigh1.84927_cross0.94558_n14.mtx","synthetic_66463_66463_12795815_avg192.52539_std381.413_random_bw39460.49056_skew344.2168_neigh1.85447_cross0.94527_n14.mtx","synthetic_66463_66463_13484307_avg202.88442_std376.07335_random_bw38668.20735_skew326.59046_neigh1.85481_cross0.94695_n14.mtx","synthetic_66463_66463_7269367_avg109.37464_std374.11902_random_bw41648.04419_skew424.85738_neigh1.81971_cross0.92957_n14.mtx","synthetic_66463_66463_7962072_avg119.79706_std378.84315_random_bw43046.45655_skew424.84518_neigh1.82641_cross0.93386_n14.mtx","synthetic_66463_66463_8654839_avg130.22041_std384.01598_random_bw39759.86055_skew424.83187_neigh1.83083_cross0.93703_n14.mtx","synthetic_66463_66463_9347522_avg140.64249_std389.6368_random_bw40907.57737_skew424.82436_neigh1.83448_cross0.93954_n14.mtx","synthetic_666542_666542_32559308_avg48.8481_std11.96425_random_bw120493.81988_skew1.12905_neigh1.69092_cross0.80241_n14.mtx","synthetic_68121_68121_3787555_avg55.6004_std1068.24938_random_bw1710.80564_skew856.74559_neigh1.75999_cross0.84954_n14.mtx","synthetic_68121_68121_4147401_avg60.88286_std1069.9982_random_bw1924.33426_skew857.05765_neigh1.62192_cross0.84311_n14.mtx","synthetic_68121_68121_4507272_avg66.16568_std1071.85693_random_bw2009.01365_skew857.28489_neigh1.51372_cross0.84049_n14.mtx","synthetic_68121_68121_4867182_avg71.44907_std1073.8353_random_bw2057.30139_skew857.47165_neigh1.43283_cross0.83917_n14.mtx","synthetic_68121_68121_5227217_avg76.7343_std1075.93952_random_bw2082.79121_skew857.61216_neigh1.37242_cross0.83802_n14.mtx","synthetic_68121_68121_5356033_avg78.62528_std1024.59921_random_bw2109.44585_skew865.40068_neigh1.23716_cross0.83754_n14.mtx","synthetic_68121_68121_5371179_avg78.84762_std1041.23556_random_bw2105.22274_skew862.95755_neigh1.25608_cross0.8375_n14.mtx","synthetic_68121_68121_5385572_avg79.05891_std1056.36091_random_bw2101.63207_skew860.64861_neigh1.27562_cross0.83773_n14.mtx","synthetic_68121_68121_5397801_avg79.23843_std1068.36979_random_bw2095.97678_skew858.6965_neigh1.29918_cross0.83772_n14.mtx","synthetic_68121_68121_5398333_avg79.24624_std1068.28806_random_bw2035.98597_skew601.36298_neigh1.51198_cross0.84041_n14.mtx","synthetic_68121_68121_5400082_avg79.27191_std1070.0369_random_bw2057.79974_skew658.4265_neigh1.46277_cross0.83918_n14.mtx","synthetic_68121_68121_5402340_avg79.30506_std1071.88875_random_bw2069.56856_skew715.38556_neigh1.42374_cross0.83882_n14.mtx","synthetic_68121_68121_5404162_avg79.33181_std1073.85709_random_bw2080.78824_skew772.35942_neigh1.38955_cross0.83863_n14.mtx","synthetic_68121_68121_5405884_avg79.35709_std1075.92117_random_bw2092.30701_skew857.41106_neigh1.32607_cross0.83815_n14.mtx","synthetic_68121_68121_5405960_avg79.3582_std834.61171_random_bw2118.46771_skew857.399_neigh1.17219_cross0.83969_n14.mtx","synthetic_68121_68121_5406317_avg79.36344_std1075.95011_random_bw2086.76499_skew829.23113_neigh1.35604_cross0.83848_n14.mtx","synthetic_68121_68121_5406423_avg79.365_std903.5954_random_bw2113.96985_skew857.32548_neigh1.20794_cross0.83738_n14.mtx","synthetic_68121_68121_5406595_avg79.36752_std972.82101_random_bw2105.47931_skew857.29818_neigh1.25674_cross0.83732_n14.mtx","synthetic_68121_68121_5406754_avg79.36986_std765.8736_random_bw2126.17506_skew857.27294_neigh1.12346_cross0.86663_n14.mtx","synthetic_68121_68121_5407048_avg79.37417_std1042.23736_random_bw2098.32533_skew857.22627_neigh1.3103_cross0.83794_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_random_bw1475.2481_skew857.20278_neigh1.35089_cross0.83825_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_random_bw1616.29047_skew857.20278_neigh1.3466_cross0.83808_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_random_bw1747.72823_skew857.20278_neigh1.34503_cross0.83777_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_random_bw1888.70535_skew857.20278_neigh1.34185_cross0.83764_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_random_bw2022.47778_skew857.20278_neigh1.34097_cross0.83789_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_random_bw2074.43904_skew857.20278_neigh1.34668_cross0.84315_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_random_bw2084.69378_skew857.20278_neigh1.47356_cross0.83912_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_random_bw2084.8877_skew857.20278_neigh1.41898_cross0.83878_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_random_bw2087.46808_skew857.20278_neigh1.44672_cross0.83882_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_random_bw2087.55843_skew857.20278_neigh1.38435_cross0.83848_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_random_bw2091.70347_skew857.20278_neigh1.24164_cross0.83704_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_random_bw2092.93644_skew857.20278_neigh1.27201_cross0.83748_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_random_bw2093.47124_skew857.20278_neigh1.29801_cross0.83744_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_random_bw2093.49917_skew857.20278_neigh1.35469_cross0.83806_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_random_bw2093.88641_skew857.20278_neigh1.32253_cross0.83718_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_random_bw2096.20556_skew857.20278_neigh1.21315_cross0.83699_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_random_bw2157.43003_skew857.20278_neigh1.33753_cross0.83793_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_random_bw2184.16415_skew857.20278_neigh1.33864_cross0.81059_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_random_bw2297.56153_skew857.20278_neigh1.34152_cross0.83789_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_random_bw2316.85899_skew857.20278_neigh1.33597_cross0.75893_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_random_bw2425.34644_skew857.20278_neigh1.33787_cross0.70358_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_random_bw2429.8828_skew857.20278_neigh1.33672_cross0.83792_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_random_bw2506.6977_skew857.20278_neigh1.33929_cross0.65052_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_random_bw2565.36768_skew857.20278_neigh1.33531_cross0.83829_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_random_bw2580.54001_skew857.20278_neigh1.34006_cross0.59378_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_random_bw2698.36137_skew857.20278_neigh1.33462_cross0.83799_n14.mtx","synthetic_68121_68121_5407447_avg79.38003_std1111.80732_random_bw2086.89475_skew857.16294_neigh1.369_cross0.83803_n14.mtx","synthetic_68121_68121_5407867_avg79.3862_std1181.49691_random_bw2070.29451_skew857.0963_neigh1.42499_cross0.83866_n14.mtx","synthetic_68121_68121_5408409_avg79.39415_std1251.28152_random_bw2052.9943_skew857.0103_neigh1.49396_cross0.83934_n14.mtx","synthetic_68121_68121_5409111_avg79.40446_std1321.14786_random_bw2021.97833_skew856.89895_neigh1.56017_cross0.84082_n14.mtx","synthetic_68121_68121_5409514_avg79.41037_std1391.07361_random_bw1983.1191_skew856.83504_neigh1.63329_cross0.84144_n14.mtx","synthetic_68121_68121_5584961_avg81.98589_std1075.92069_random_bw2099.22202_skew829.88685_neigh1.31053_cross0.83776_n14.mtx","synthetic_68121_68121_5934842_avg87.12206_std1068.36291_random_bw2110.99739_skew780.90298_neigh1.27284_cross0.83727_n14.mtx","synthetic_68121_68121_6280428_avg92.19518_std1056.3285_random_bw2123.14184_skew737.87809_neigh1.23532_cross0.83721_n14.mtx","synthetic_68121_68121_6623921_avg97.23758_std1041.17147_random_bw2125.39362_skew699.5625_neigh1.21008_cross0.83733_n14.mtx","synthetic_68121_68121_6966795_avg102.27089_std1024.4946_random_bw2134.44271_skew665.08399_neigh1.1864_cross0.83805_n14.mtx","synthetic_68678_68678_10731088_avg156.25219_std397.34109_random_bw40858.96562_skew424.88203_neigh1.83835_cross0.94238_n14.mtx","synthetic_690692_690692_50143771_avg72.59932_std15.84383_random_bw10372.13539_skew0.96972_neigh1.69572_cross0.81872_n14.mtx","synthetic_69445_69445_5001284_avg72.01791_std19.00309_random_bw3971.49729_skew1.13836_neigh1.65217_cross0.77151_n14.mtx","synthetic_70221_70221_15563153_avg221.63104_std96.30657_random_bw57348.75106_skew15.41016_neigh1.70385_cross0.85141_n14.mtx","synthetic_70391_70391_5585080_avg79.34367_std1076.52341_random_bw2156.59175_skew858.03517_neigh1.33809_cross0.83781_n14.mtx","synthetic_718886_718886_16011537_avg22.27271_std29.91357_random_bw181769.97437_skew313.60029_neigh1.26418_cross0.75622_n14.mtx","synthetic_730022_730022_35670999_avg48.86291_std11.95289_random_bw131807.11807_skew1.14887_neigh1.69169_cross0.8019_n14.mtx","synthetic_73109_73109_11420815_avg156.21627_std394.47456_random_bw43325.79208_skew424.97997_neigh1.83945_cross0.94217_n14.mtx","synthetic_74477_74477_16505933_avg221.62457_std96.28344_random_bw60807.87753_skew15.41064_neigh1.70429_cross0.84959_n14.mtx","synthetic_74933_74933_5942603_avg79.30555_std1075.58283_random_bw2296.59318_skew858.44801_neigh1.34268_cross0.83766_n14.mtx","synthetic_75000_75000_5399893_avg71.99857_std19.01052_random_bw4301.01067_skew1.13893_neigh1.6539_cross0.76935_n14.mtx","synthetic_756472_756472_54943929_avg72.63181_std15.80449_random_bw11344.89478_skew1.03767_neigh1.69516_cross0.81986_n14.mtx","synthetic_77540_77540_12111057_avg156.19109_std391.93861_random_bw45639.49656_skew425.04863_neigh1.84051_cross0.94184_n14.mtx","synthetic_776397_776397_17292444_avg22.27268_std29.86978_random_bw196406.45729_skew313.60063_neigh1.26218_cross0.75486_n14.mtx","synthetic_78733_78733_17449768_avg221.6322_std96.26215_random_bw64358.73616_skew15.41007_neigh1.70377_cross0.84821_n14.mtx","synthetic_793502_793502_38791634_avg48.88662_std11.94806_random_bw143383.51376_skew1.39329_neigh1.69098_cross0.8029_n14.mtx","synthetic_79474_79474_6302859_avg79.30718_std1074.75192_random_bw2429.36754_skew858.43036_neigh1.33896_cross0.83795_n14.mtx","synthetic_80556_80556_5802157_avg72.02638_std19.02772_random_bw4624.77567_skew1.13811_neigh1.6521_cross0.76889_n14.mtx","synthetic_81971_81971_12801291_avg156.16854_std389.68003_random_bw48091.50462_skew425.11016_neigh1.8392_cross0.9423_n14.mtx","synthetic_822252_822252_59710654_avg72.61844_std15.79674_random_bw12337.39137_skew1.02428_neigh1.69509_cross0.82013_n14.mtx","synthetic_82989_82989_18393238_avg221.63465_std96.24293_random_bw67768.63417_skew15.40989_neigh1.70426_cross0.84856_n14.mtx","synthetic_83334_83334_4201657_avg50.41948_std18.95017_random_bw4791.17217_skew1.61804_neigh1.63203_cross0.73132_n14.mtx","synthetic_83334_83334_4601152_avg55.21338_std18.98799_random_bw4722.67207_skew1.48128_neigh1.63833_cross0.74354_n14.mtx","synthetic_83334_83334_5001407_avg60.0164_std19.00589_random_bw4672.15651_skew1.36602_neigh1.64462_cross0.75232_n14.mtx","synthetic_83334_83334_5402033_avg64.82388_std19.01381_random_bw4770.34201_skew1.25226_neigh1.64891_cross0.75864_n14.mtx","synthetic_83334_83334_5802662_avg69.63139_std19.01513_random_bw4732.86769_skew1.16856_neigh1.65263_cross0.76629_n14.mtx","synthetic_83334_83334_6001725_avg72.02012_std24.68331_random_bw4665.54866_skew1.47153_neigh1.65687_cross0.7425_n14.mtx","synthetic_83334_83334_6001767_avg72.02063_std23.43414_random_bw4680.6887_skew1.40209_neigh1.65691_cross0.74737_n14.mtx","synthetic_83334_83334_6001894_avg72.02215_std22.17621_random_bw4719.37063_skew1.31873_neigh1.65546_cross0.7542_n14.mtx","synthetic_83334_83334_6002259_avg72.02653_std20.9136_random_bw4737.64665_skew1.24917_neigh1.65459_cross0.76108_n14.mtx","synthetic_83334_83334_6002747_avg72.03239_std19.64906_random_bw4762.0232_skew1.16569_neigh1.65259_cross0.76576_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_random_bw3350.26591_skew1.13786_neigh1.65428_cross0.77109_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_random_bw3665.49009_skew1.13786_neigh1.65236_cross0.77134_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_random_bw3982.36841_skew1.13786_neigh1.65216_cross0.77018_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_random_bw4220.40105_skew1.13786_neigh1.61436_cross0.87115_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_random_bw4298.73964_skew1.13786_neigh1.65361_cross0.7697_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_random_bw4395.07819_skew1.13786_neigh1.63026_cross0.84649_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_random_bw4592.28661_skew1.13786_neigh1.16508_cross0.77088_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_random_bw4606.03839_skew1.13786_neigh1.27327_cross0.76874_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_random_bw4608.30407_skew1.13786_neigh1.38168_cross0.77039_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_random_bw4614.10663_skew1.13786_neigh1.653_cross0.77104_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_random_bw4654.25229_skew1.13786_neigh1.4901_cross0.76949_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_random_bw4659.61285_skew1.13786_neigh1.6488_cross0.79591_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_random_bw4701.35313_skew1.13786_neigh1.59891_cross0.77081_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_random_bw4776.47572_skew1.13786_neigh1.65401_cross0.76988_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_random_bw4850.09408_skew1.13786_neigh1.65707_cross0.74566_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_random_bw4869.2196_skew1.13786_neigh1.70817_cross0.76996_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_random_bw4931.43106_skew1.13786_neigh1.65277_cross0.76964_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_random_bw4997.26823_skew1.13786_neigh1.66492_cross0.69556_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_random_bw5131.45013_skew1.13786_neigh1.66917_cross0.64549_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_random_bw5220.44701_skew1.13786_neigh1.67321_cross0.59335_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_random_bw5240.64593_skew1.13786_neigh1.65263_cross0.76939_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_random_bw5314.98523_skew1.13786_neigh1.67563_cross0.54557_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_random_bw5580.33847_skew1.13786_neigh1.65308_cross0.76857_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_random_bw5607.76149_skew1.13786_neigh1.81609_cross0.77014_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_random_bw5792.30158_skew1.13786_neigh1.92751_cross0.7686_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_random_bw5891.31062_skew1.13786_neigh1.65284_cross0.76938_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_random_bw5944.6548_skew1.13786_neigh1.92575_cross0.76959_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_random_bw6200.06394_skew1.13786_neigh1.65377_cross0.76946_n14.mtx","synthetic_83334_83334_6003209_avg72.03793_std18.38192_random_bw4782.87446_skew1.09612_neigh1.6521_cross0.77432_n14.mtx","synthetic_83334_83334_6003696_avg72.04378_std17.11472_random_bw4810.91187_skew1.01267_neigh1.65178_cross0.77955_n14.mtx","synthetic_83334_83334_6004305_avg72.05108_std15.84803_random_bw4849.17645_skew0.94307_neigh1.6506_cross0.7874_n14.mtx","synthetic_83334_83334_6004738_avg72.05628_std14.57979_random_bw4873.18033_skew0.87354_neigh1.6472_cross0.79352_n14.mtx","synthetic_83334_83334_6005175_avg72.06152_std13.31259_random_bw4900.61932_skew0.79014_neigh1.6502_cross0.79935_n14.mtx","synthetic_83334_83334_6203256_avg74.43848_std19.01659_random_bw4717.39456_skew1.09569_neigh1.65429_cross0.77391_n14.mtx","synthetic_83334_83334_6603858_avg79.24566_std19.01545_random_bw4770.86497_skew1.03166_neigh1.65742_cross0.7795_n14.mtx","synthetic_83334_83334_7004670_avg84.05537_std19.01725_random_bw4746.86476_skew0.97489_neigh1.66015_cross0.78394_n14.mtx","synthetic_83334_83334_7405486_avg88.86512_std19.01689_random_bw4748.30992_skew0.91301_neigh1.66314_cross0.78781_n14.mtx","synthetic_83334_83334_7806125_avg93.67275_std19.01502_random_bw4799.57248_skew0.86821_neigh1.66483_cross0.79526_n14.mtx","synthetic_833908_833908_18572841_avg22.27205_std29.833_random_bw210844.85277_skew313.60956_neigh1.26298_cross0.75563_n14.mtx","synthetic_84015_84015_6661114_avg79.28482_std1074.00684_random_bw2567.05804_skew858.67278_neigh1.33657_cross0.83743_n14.mtx","synthetic_856982_856982_41861338_avg48.84739_std11.94057_random_bw154675.82077_skew1.3338_neigh1.69011_cross0.80335_n14.mtx","synthetic_86111_86111_6203103_avg72.03613_std19.01524_random_bw4930.68174_skew1.13782_neigh1.65292_cross0.76969_n14.mtx","synthetic_862664_862664_13443097_avg15.58324_std29.57178_random_bw217449.07728_skew313.76132_neigh1.20059_cross0.75196_n14.mtx","synthetic_862664_862664_14783705_avg17.13727_std29.61764_random_bw211187.81291_skew312.46885_neigh1.21655_cross0.75346_n14.mtx","synthetic_862664_862664_15984607_avg18.52935_std29.67134_random_bw215438.01831_skew314.12161_neigh1.22333_cross0.75444_n14.mtx","synthetic_862664_862664_17373620_avg20.1395_std29.72434_random_bw211752.41659_skew312.11605_neigh1.23946_cross0.75537_n14.mtx","synthetic_862664_862664_18524582_avg21.47369_std29.78706_random_bw214344.29615_skew314.40918_neigh1.24035_cross0.75391_n14.mtx","synthetic_862664_862664_19089435_avg22.12847_std21.22646_random_bw218646.30801_skew315.65089_neigh1.27062_cross0.75619_n14.mtx","synthetic_862664_862664_19142097_avg22.18952_std25.02501_random_bw218419.55757_skew314.77975_neigh1.26903_cross0.75599_n14.mtx","synthetic_862664_862664_19162894_avg22.21362_std30.15004_random_bw218402.40102_skew408.47843_neigh1.26817_cross0.75575_n14.mtx","synthetic_862664_862664_19171989_avg22.22417_std26.93718_random_bw218359.93899_skew314.2874_neigh1.26727_cross0.7556_n14.mtx","synthetic_862664_862664_19173147_avg22.22551_std30.06905_random_bw218342.75064_skew387.38255_neigh1.26736_cross0.75562_n14.mtx","synthetic_862664_862664_19184120_avg22.23823_std29.99101_random_bw218334.93381_skew366.25046_neigh1.26648_cross0.75604_n14.mtx","synthetic_862664_862664_19195719_avg22.25168_std29.91773_random_bw218360.52038_skew345.17619_neigh1.26538_cross0.75561_n14.mtx","synthetic_862664_862664_19200670_avg22.25741_std28.85544_random_bw218339.10627_skew313.81644_neigh1.26454_cross0.75565_n14.mtx","synthetic_862664_862664_19207356_avg22.26516_std29.84926_random_bw218277.96985_skew324.12672_neigh1.26399_cross0.75549_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_random_bw152638.10671_skew313.61244_neigh1.26367_cross0.75505_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_random_bw167333.37581_skew313.61244_neigh1.26365_cross0.75511_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_random_bw181720.63529_skew313.61244_neigh1.26353_cross0.75502_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_random_bw196516.2028_skew313.61244_neigh1.26346_cross0.75516_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_random_bw211000.2943_skew313.61244_neigh1.26342_cross0.75507_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_random_bw213175.84954_skew313.61244_neigh0.96905_cross0.75596_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_random_bw213223.70119_skew313.61244_neigh0.88372_cross0.75578_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_random_bw213418.94648_skew313.61244_neigh1.05278_cross0.75588_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_random_bw213434.42462_skew313.61244_neigh1.13672_cross0.75617_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_random_bw213774.00756_skew313.61244_neigh1.22102_cross0.755_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_random_bw214411.79173_skew313.61244_neigh1.30492_cross0.75491_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_random_bw215370.12458_skew313.61244_neigh1.38935_cross0.75586_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_random_bw216786.80808_skew313.61244_neigh1.47408_cross0.75519_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_random_bw218507.74866_skew313.61244_neigh1.26845_cross0.52792_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_random_bw221443.99058_skew313.61244_neigh1.55823_cross0.75678_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_random_bw225527.89524_skew313.61244_neigh1.26339_cross0.75523_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_random_bw233014.20901_skew313.61244_neigh1.6426_cross0.75641_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_random_bw240175.84299_skew313.61244_neigh1.26362_cross0.75505_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_random_bw254512.86048_skew313.61244_neigh1.26353_cross0.7551_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_random_bw269234.4357_skew313.61244_neigh1.26372_cross0.75516_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_random_bw283733.51022_skew313.61244_neigh1.26362_cross0.75506_n14.mtx","synthetic_862664_862664_19218664_avg22.27827_std29.78506_random_bw218120.72798_skew303.10795_neigh1.26259_cross0.75534_n14.mtx","synthetic_862664_862664_19223712_avg22.28412_std30.77861_random_bw218094.95048_skew313.4391_neigh1.2615_cross0.75525_n14.mtx","synthetic_862664_862664_19229090_avg22.29036_std29.72467_random_bw217946.1315_skew282.12689_neigh1.26031_cross0.75517_n14.mtx","synthetic_862664_862664_19237224_avg22.29979_std29.6692_random_bw217791.40434_skew261.15496_neigh1.25666_cross0.75502_n14.mtx","synthetic_862664_862664_19238563_avg22.30134_std32.70575_random_bw217839.24923_skew313.19637_neigh1.25574_cross0.75505_n14.mtx","synthetic_862664_862664_19242590_avg22.30601_std29.61737_random_bw217546.12267_skew240.28029_neigh1.252_cross0.75446_n14.mtx","synthetic_862664_862664_19243461_avg22.30702_std38.50483_random_bw216241.00134_skew313.1164_neigh1.23809_cross0.7533_n14.mtx","synthetic_862664_862664_19244915_avg22.3087_std29.57096_random_bw217130.97238_skew219.45208_neigh1.24755_cross0.75425_n14.mtx","synthetic_862664_862664_19963054_avg23.14117_std29.84969_random_bw212519.73122_skew311.90553_neigh1.25719_cross0.75481_n14.mtx","synthetic_862664_862664_21064571_avg24.41805_std29.92055_random_bw214229.26362_skew314.66815_neigh1.25303_cross0.75497_n14.mtx","synthetic_862664_862664_22549917_avg26.13986_std29.9915_random_bw212791.27921_skew311.74076_neigh1.27038_cross0.75528_n14.mtx","synthetic_862664_862664_23605495_avg27.36349_std30.07249_random_bw214076.51411_skew314.82232_neigh1.26413_cross0.75466_n14.mtx","synthetic_862664_862664_25134553_avg29.13597_std30.15077_random_bw213148.94325_skew311.6376_neigh1.2775_cross0.75586_n14.mtx","synthetic_86401_86401_13491346_avg156.14803_std387.65694_random_bw50727.43053_skew425.16612_neigh1.83819_cross0.94244_n14.mtx","synthetic_88557_88557_7019496_avg79.26529_std1073.34158_random_bw2696.06683_skew858.88461_neigh1.33763_cross0.83799_n14.mtx","synthetic_888032_888032_64506105_avg72.63939_std15.82947_random_bw13315.63085_skew1.00993_neigh1.69523_cross0.81974_n14.mtx","synthetic_891419_891419_19853723_avg22.27204_std29.80105_random_bw225604.82234_skew313.60965_neigh1.26302_cross0.75611_n14.mtx","synthetic_91667_91667_6603697_avg72.04007_std19.01165_random_bw5241.08871_skew1.19322_neigh1.65278_cross0.76932_n14.mtx","synthetic_920462_920462_44975685_avg48.86208_std11.94475_random_bw166256.93189_skew1.2103_neigh1.69039_cross0.80408_n14.mtx","synthetic_948930_948930_21133875_avg22.27127_std29.77202_random_bw239983.61722_skew313.6206_neigh1.26346_cross0.75671_n14.mtx","synthetic_952203_952203_32554296_avg34.1884_std11.93673_random_bw208499.30602_skew1.69097_neigh1.6586_cross0.76805_n14.mtx","synthetic_952203_952203_35650826_avg37.44036_std11.95176_random_bw178182.3534_skew1.53737_neigh1.66758_cross0.77791_n14.mtx","synthetic_952203_952203_38750463_avg40.69559_std11.9572_random_bw182355.14926_skew1.4327_neigh1.67525_cross0.7865_n14.mtx","synthetic_952203_952203_41851500_avg43.95229_std11.95993_random_bw185907.68003_skew1.3207_neigh1.68123_cross0.79456_n14.mtx","synthetic_952203_952203_44952640_avg47.20909_std11.96013_random_bw170436.18015_skew1.22415_neigh1.68791_cross0.80043_n14.mtx","synthetic_952203_952203_46500626_avg48.83478_std13.95044_random_bw170085.21621_skew1.37536_neigh1.69447_cross0.78846_n14.mtx","synthetic_952203_952203_46500963_avg48.83514_std14.74428_random_bw169353.05265_skew1.45725_neigh1.69595_cross0.78191_n14.mtx","synthetic_952203_952203_46501138_avg48.83532_std15.536_random_bw168758.23927_skew1.53915_neigh1.69704_cross0.77638_n14.mtx","synthetic_952203_952203_46502122_avg48.83635_std13.15524_random_bw170806.64163_skew1.31385_neigh1.69248_cross0.79425_n14.mtx","synthetic_952203_952203_46502884_avg48.83715_std12.35905_random_bw171544.88603_skew1.23191_neigh1.69063_cross0.80082_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_random_bw120375.53352_skew1.19093_neigh1.69025_cross0.8034_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_random_bw131783.961_skew1.19093_neigh1.6902_cross0.80347_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_random_bw143412.56495_skew1.19093_neigh1.69053_cross0.8034_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_random_bw154853.41935_skew1.19093_neigh1.69028_cross0.80351_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_random_bw166308.11219_skew1.19093_neigh1.69031_cross0.80348_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_random_bw177753.75801_skew1.19093_neigh1.69007_cross0.80347_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_random_bw189177.26804_skew1.19093_neigh1.69023_cross0.80345_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_random_bw200570.66789_skew1.19093_neigh1.69027_cross0.80345_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_random_bw212041.87573_skew1.19093_neigh1.69043_cross0.80333_n14.mtx","synthetic_952203_952203_46504635_avg48.83899_std11.56217_random_bw172412.673_skew1.14992_neigh1.6892_cross0.80663_n14.mtx","synthetic_952203_952203_46504874_avg48.83924_std10.76526_random_bw173220.71164_skew1.06801_neigh1.68664_cross0.81356_n14.mtx","synthetic_952203_952203_46506617_avg48.84107_std9.96939_random_bw174360.33325_skew0.98603_neigh1.68579_cross0.81888_n14.mtx","synthetic_952203_952203_46507502_avg48.842_std9.17223_random_bw175367.02321_skew0.9041_neigh1.68327_cross0.82582_n14.mtx","synthetic_952203_952203_46508902_avg48.84347_std8.37556_random_bw177002.71627_skew0.82215_neigh1.68094_cross0.8324_n14.mtx","synthetic_952203_952203_48054277_avg50.46642_std11.96125_random_bw173372.13525_skew1.14004_neigh1.69321_cross0.80658_n14.mtx","synthetic_952203_952203_51155719_avg53.72354_std11.96081_random_bw176138.00989_skew1.08475_neigh1.69709_cross0.81142_n14.mtx","synthetic_952203_952203_54257059_avg56.98056_std11.96124_random_bw166937.37724_skew1.01823_neigh1.70097_cross0.81597_n14.mtx","synthetic_952203_952203_57358241_avg60.23741_std11.96072_random_bw169273.03323_skew0.95892_neigh1.70515_cross0.82049_n14.mtx","synthetic_952203_952203_60460102_avg63.49497_std11.9616_random_bw171406.13604_skew0.90566_neigh1.70892_cross0.82386_n14.mtx","synthetic_953812_953812_69251992_avg72.60549_std15.82731_random_bw14300.01986_skew1.05219_neigh1.69474_cross0.82044_n14.mtx","synthetic_968035_968035_11863180_avg12.25491_std37.64729_random_bw22809.30094_skew632.62368_neigh1.3222_cross0.78548_n14.mtx","synthetic_97223_97223_7006972_avg72.07114_std19.01384_random_bw5580.63884_skew1.19228_neigh1.65246_cross0.76899_n14.mtx","synthetic_983943_983943_48052837_avg48.83701_std11.95054_random_bw177774.01613_skew1.12953_neigh1.69013_cross0.804_n14.mtx","synthetic_98611_98611_5466825_avg55.43829_std11.03861_random_bw3858.831_skew0.894_neigh1.63088_cross0.78813_n14.mtx","synthetic_986703_986703_50142919_avg50.81865_std15.80386_random_bw14886.34812_skew1.44005_neigh1.67248_cross0.78656_n14.mtx","synthetic_986703_986703_54918080_avg55.65817_std15.81077_random_bw15208.31576_skew1.31772_neigh1.6788_cross0.79621_n14.mtx","synthetic_986703_986703_59694812_avg60.49927_std15.81312_random_bw14727.02983_skew1.2149_neigh1.68487_cross0.80451_n14.mtx","synthetic_986703_986703_64472911_avg65.34176_std15.81391_random_bw14462.32703_skew1.11197_neigh1.68893_cross0.81154_n14.mtx","synthetic_986703_986703_69250817_avg70.18405_std15.81417_random_bw14701.88268_skew1.0375_neigh1.6931_cross0.81692_n14.mtx","synthetic_986703_986703_71632861_avg72.5982_std20.55272_random_bw14534.71352_skew1.31411_neigh1.69992_cross0.79562_n14.mtx","synthetic_986703_986703_71633918_avg72.59927_std19.50045_random_bw14580.21846_skew1.2452_neigh1.69942_cross0.8008_n14.mtx","synthetic_986703_986703_71635339_avg72.60071_std18.448_random_bw14638.21943_skew1.17629_neigh1.6985_cross0.8063_n14.mtx","synthetic_986703_986703_71637178_avg72.60257_std17.39427_random_bw14693.79283_skew1.10736_neigh1.69733_cross0.81127_n14.mtx","synthetic_986703_986703_71638484_avg72.6039_std16.3415_random_bw14738.44751_skew1.03846_neigh1.69562_cross0.81775_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_random_bw10351.48927_skew1.0109_neigh1.69556_cross0.82059_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_random_bw11350.18676_skew1.0109_neigh1.69576_cross0.82027_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_random_bw13218.95443_skew1.0109_neigh1.63894_cross0.89577_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_random_bw13857.1301_skew1.0109_neigh1.1897_cross0.81891_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_random_bw13898.10027_skew1.0109_neigh1.30212_cross0.81916_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_random_bw14441.63744_skew1.0109_neigh1.63911_cross0.82048_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_random_bw14787.04212_skew1.0109_neigh1.69513_cross0.82047_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_random_bw15272.02993_skew1.0109_neigh1.69545_cross0.82033_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_random_bw15463.25016_skew1.0109_neigh1.75112_cross0.82045_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_random_bw15698.85861_skew1.0109_neigh1.71329_cross0.73895_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_random_bw16106.42649_skew1.0109_neigh1.72022_cross0.68496_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_random_bw19207.41674_skew1.0109_neigh1.69559_cross0.8201_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_random_bw20444.65782_skew1.0109_neigh1.91237_cross0.8206_n14.mtx","synthetic_986703_986703_71640715_avg72.60616_std15.28673_random_bw14825.68345_skew0.96953_neigh1.69453_cross0.82276_n14.mtx","synthetic_986703_986703_71642304_avg72.60777_std14.23366_random_bw14897.2269_skew0.90062_neigh1.69343_cross0.8287_n14.mtx","synthetic_986703_986703_71644194_avg72.60968_std13.17962_random_bw14980.02571_skew0.84548_neigh1.69135_cross0.83479_n14.mtx","synthetic_986703_986703_71646204_avg72.61172_std12.12567_random_bw15073.83895_skew0.77657_neigh1.68951_cross0.84052_n14.mtx","synthetic_986703_986703_71647231_avg72.61276_std11.07215_random_bw15188.71315_skew0.70769_neigh1.68786_cross0.84661_n14.mtx","synthetic_986703_986703_74029078_avg75.02671_std15.81428_random_bw14477.08036_skew0.97263_neigh1.69683_cross0.82287_n14.mtx","synthetic_986703_986703_78806613_avg79.86863_std15.81421_random_bw14655.966_skew0.91565_neigh1.69988_cross0.82778_n14.mtx","synthetic_986703_986703_83583783_avg84.71017_std15.81435_random_bw14520.34653_skew0.86518_neigh1.70339_cross0.83169_n14.mtx","synthetic_986703_986703_88361224_avg89.552_std15.81425_random_bw14683.12554_skew0.82017_neigh1.7057_cross0.83562_n14.mtx","synthetic_986703_986703_93139124_avg94.39428_std15.81414_random_bw14552.95978_skew0.76917_neigh1.70838_cross0.83944_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_random_bw35853.81968_skew632.66322_neigh1.32203_cross0.78644_n14.mtx","synthetic_268096_268096_19097540_avg71.23396_std101.70964_random_bw32428.45227_skew10.16041_neigh1.12478_cross0.8356_n14.mtx","synthetic_268096_268096_20328893_avg75.82692_std102.55303_random_bw32567.72158_skew10.2493_neigh1.12931_cross0.83183_n14.mtx","synthetic_4690002_4690002_18772478_avg4.00266_std1.15153_random_bw4545.51747_skew240.08974_neigh0.41531_cross0.6488_n14.mtx","synthetic_862664_862664_19114072_avg22.15703_std23.12053_random_bw218587.40518_skew315.24275_neigh1.26934_cross0.75615_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_random_bw218387.82176_skew313.61244_neigh1.26748_cross0.57934_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_random_bw223475.36954_skew1.19093_neigh1.69052_cross0.80348_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_random_bw37993.81919_skew632.66322_neigh1.32219_cross0.78642_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_random_bw40249.088_skew632.66322_neigh1.32209_cross0.7864_n14.mtx","synthetic_268096_268096_18479164_avg68.92741_std73.74951_random_bw32856.73895_skew10.4033_neigh1.1224_cross0.84096_n14.mtx","synthetic_268096_268096_18479645_avg68.92921_std80.36452_random_bw32647.3807_skew10.403_neigh1.12506_cross0.82761_n14.mtx","synthetic_268096_268096_18480123_avg68.93099_std86.42344_random_bw32612.95544_skew10.33017_neigh1.12468_cross0.82902_n14.mtx","synthetic_268096_268096_18480967_avg68.93414_std92.41882_random_bw32501.5288_skew10.25712_neigh1.12262_cross0.83284_n14.mtx","synthetic_268096_268096_18481691_avg68.93684_std98.3339_random_bw32411.34374_skew10.16964_neigh1.12316_cross0.83523_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_random_bw22631.95884_skew10.11157_neigh1.1236_cross0.83759_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_random_bw24810.92538_skew10.11157_neigh1.12535_cross0.83717_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_random_bw26946.62422_skew10.11157_neigh1.12451_cross0.83736_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_random_bw29104.95345_skew10.11157_neigh1.1245_cross0.83739_n14.mtx","synthetic_268096_268096_18482362_avg68.93934_std104.15593_random_bw32241.45427_skew10.0677_neigh1.12209_cross0.83937_n14.mtx","synthetic_268096_268096_18483075_avg68.942_std109.86851_random_bw32121.44382_skew9.95123_neigh1.1203_cross0.84327_n14.mtx","synthetic_268096_268096_18483963_avg68.94531_std115.46168_random_bw31912.0398_skew9.82017_neigh1.11692_cross0.84863_n14.mtx","synthetic_268096_268096_18484698_avg68.94806_std120.91939_random_bw31591.06942_skew9.68921_neigh1.11467_cross0.85538_n14.mtx","synthetic_268096_268096_18485774_avg68.95207_std126.22922_random_bw31186.72806_skew9.55806_neigh1.11278_cross0.86343_n14.mtx","synthetic_268096_268096_21560408_avg80.42048_std103.33442_random_bw32634.53398_skew10.30309_neigh1.13401_cross0.8296_n14.mtx","synthetic_268096_268096_22792018_avg85.01439_std104.0723_random_bw32673.47377_skew10.36278_neigh1.13741_cross0.82812_n14.mtx","synthetic_268096_268096_24023179_avg89.60663_std104.77794_random_bw32722.96392_skew10.40541_neigh1.1392_cross0.8264_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_random_bw209088.73821_skew313.61244_neigh1.15533_cross0.98278_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_random_bw216180.85999_skew313.61244_neigh1.23792_cross0.93242_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_random_bw217290.8255_skew313.61244_neigh1.2532_cross0.88202_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_random_bw217679.16493_skew313.61244_neigh1.25924_cross0.83135_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_random_bw218002.31666_skew313.61244_neigh1.26186_cross0.78113_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_random_bw218342.9789_skew313.61244_neigh1.26682_cross0.62984_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_random_bw218343.96868_skew313.61244_neigh1.26388_cross0.73052_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_random_bw218353.64903_skew313.61244_neigh1.26627_cross0.6806_n14.mtx","synthetic_862664_862664_19245171_avg22.309_std34.6357_random_bw217268.52825_skew313.08849_neigh1.25013_cross0.75439_n14.mtx","synthetic_862664_862664_19245832_avg22.30977_std36.57002_random_bw216933.33678_skew313.0777_neigh1.24283_cross0.75358_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_random_bw150854.23009_skew1.19093_neigh1.18385_cross0.80149_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_random_bw151895.95754_skew1.19093_neigh1.29667_cross0.80161_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_random_bw153481.11228_skew1.19093_neigh1.40932_cross0.80268_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_random_bw157146.27492_skew1.19093_neigh1.52169_cross0.80287_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_random_bw161158.68364_skew1.19093_neigh1.61052_cross0.88656_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_random_bw161480.87896_skew1.19093_neigh1.61762_cross0.88363_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_random_bw167259.59789_skew1.19093_neigh1.63354_cross0.80363_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_random_bw169055.00469_skew1.19093_neigh1.67727_cross0.83029_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_random_bw172056.91803_skew1.19093_neigh1.69026_cross0.80348_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_random_bw174466.89467_skew1.19093_neigh1.70003_cross0.77698_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_random_bw178531.20153_skew1.19093_neigh1.71349_cross0.72328_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_random_bw179918.61642_skew1.19093_neigh1.7464_cross0.80393_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_random_bw181221.45031_skew1.19093_neigh1.72367_cross0.66928_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_random_bw183413.76762_skew1.19093_neigh1.73057_cross0.61618_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_random_bw185134.26789_skew1.19093_neigh1.73599_cross0.56267_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_random_bw209833.19969_skew1.19093_neigh1.87978_cross0.80421_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_random_bw241430.72348_skew1.19093_neigh1.85806_cross0.80383_n14.mtx","synthetic_1060229_1060229_12992547_avg12.25447_std37.61142_diagonal_bw24877.30989_skew632.64621_neigh1.32175_cross0.78562_n14.mtx","synthetic_112749_112749_5727652_avg50.80002_std19.54637_diagonal_bw3425.86088_skew1.7559_neigh1.22834_cross0.71806_n14.mtx","synthetic_1152423_1152423_14123090_avg12.25513_std37.58092_diagonal_bw27066.48438_skew632.61237_neigh1.32182_cross0.78662_n14.mtx","synthetic_123486_123486_6272129_avg50.79223_std19.54882_diagonal_bw3753.58169_skew1.75633_neigh1.22934_cross0.71722_n14.mtx","synthetic_1244617_1244617_15251343_avg12.25384_std37.55512_diagonal_bw29219.49964_skew632.67869_neigh1.3223_cross0.78671_n14.mtx","synthetic_1336811_1336811_16381252_avg12.25398_std37.5327_diagonal_bw31339.89957_skew632.6718_neigh1.32267_cross0.78669_n14.mtx","synthetic_134225_134225_6816930_avg50.78733_std19.56874_diagonal_bw4082.86505_skew1.77628_neigh1.22794_cross0.71641_n14.mtx","synthetic_1382908_1382908_11844497_avg8.56492_std37.37396_diagonal_bw40250.63741_skew633.56515_neigh1.20442_cross0.7799_n14.mtx","synthetic_1382908_1382908_12976596_avg9.38356_std37.40231_diagonal_bw42376.82952_skew633.40762_neigh1.23873_cross0.78254_n14.mtx","synthetic_1382908_1382908_14113272_avg10.2055_std37.43376_diagonal_bw29741.04005_skew632.97168_neigh1.26021_cross0.78249_n14.mtx","synthetic_1382908_1382908_15242471_avg11.02204_std37.4678_diagonal_bw30995.34918_skew633.00226_neigh1.2701_cross0.78366_n14.mtx","synthetic_1382908_1382908_16349573_avg11.8226_std37.50438_diagonal_bw31873.17246_skew633.88554_neigh1.28656_cross0.78385_n14.mtx","synthetic_1382908_1382908_16848093_avg12.18309_std26.47929_diagonal_bw31429.05804_skew636.35882_neigh1.38049_cross0.79092_n14.mtx","synthetic_1382908_1382908_16896659_avg12.21821_std28.92386_diagonal_bw31501.87862_skew634.52686_neigh1.37656_cross0.79099_n14.mtx","synthetic_1382908_1382908_16922141_avg12.23664_std48.62398_diagonal_bw32239.82905_skew633.48814_neigh1.28004_cross0.7802_n14.mtx","synthetic_1382908_1382908_16923406_avg12.23755_std46.1541_diagonal_bw32382.62648_skew633.44071_neigh1.28391_cross0.78182_n14.mtx","synthetic_1382908_1382908_16926342_avg12.23967_std43.68598_diagonal_bw32452.39208_skew633.41236_neigh1.2921_cross0.78362_n14.mtx","synthetic_1382908_1382908_16931967_avg12.24374_std41.21881_diagonal_bw32524.36102_skew633.2016_neigh1.30009_cross0.7846_n14.mtx","synthetic_1382908_1382908_16933417_avg12.24479_std31.37528_diagonal_bw31740.52473_skew633.1473_neigh1.36561_cross0.7903_n14.mtx","synthetic_1382908_1382908_16941719_avg12.25079_std38.75385_diagonal_bw32512.7569_skew632.83654_neigh1.31404_cross0.78613_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_diagonal_bw22778.5047_skew632.66322_neigh1.32228_cross0.78682_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_diagonal_bw24917.9724_skew632.66322_neigh1.32217_cross0.78679_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_diagonal_bw27035.12761_skew632.66322_neigh1.32227_cross0.7868_n14.mtx","synthetic_1382908_1382908_16950422_avg12.25709_std36.29175_diagonal_bw32375.96458_skew632.51111_neigh1.33014_cross0.78815_n14.mtx","synthetic_1382908_1382908_16950660_avg12.25726_std33.83118_diagonal_bw32049.12593_skew632.50221_neigh1.34897_cross0.78969_n14.mtx","synthetic_1382908_1382908_17445616_avg12.61517_std37.54329_diagonal_bw32660.91296_skew635.05973_neigh1.31546_cross0.78564_n14.mtx","synthetic_1382908_1382908_18572951_avg13.43036_std37.58404_diagonal_bw33424.56483_skew634.94726_neigh1.34978_cross0.78747_n14.mtx","synthetic_1382908_1382908_19756490_avg14.28619_std37.62781_diagonal_bw28853.64125_skew633.10877_neigh1.36846_cross0.78954_n14.mtx","synthetic_1382908_1382908_20949793_avg15.14909_std37.6751_diagonal_bw29885.40906_skew631.18333_neigh1.36304_cross0.78832_n14.mtx","synthetic_1382908_1382908_22070012_avg15.95913_std37.72535_diagonal_bw30643.05341_skew631.49052_neigh1.34712_cross0.78696_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_diagonal_bw4732.21902_skew0.91217_neigh1.56188_cross0.89987_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_diagonal_bw5037.41918_skew0.91217_neigh1.59723_cross0.86565_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_diagonal_bw5204.83825_skew0.91217_neigh1.14624_cross0.78728_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_diagonal_bw5224.88197_skew0.91217_neigh1.25499_cross0.78723_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_diagonal_bw5232.21638_skew0.91217_neigh1.36051_cross0.78676_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_diagonal_bw5264.87446_skew0.91217_neigh1.46866_cross0.78769_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_diagonal_bw5313.69498_skew0.91217_neigh1.57687_cross0.78795_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_diagonal_bw5335.36823_skew0.91217_neigh1.6227_cross0.815_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_diagonal_bw5457.86719_skew0.91217_neigh1.63077_cross0.78851_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_diagonal_bw5549.51317_skew0.91217_neigh1.63866_cross0.76193_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_diagonal_bw5706.21311_skew0.91217_neigh1.64636_cross0.71043_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_diagonal_bw5738.5913_skew0.91217_neigh1.68499_cross0.78752_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_diagonal_bw5820.74673_skew0.91217_neigh1.65384_cross0.65812_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_diagonal_bw5912.46692_skew0.91217_neigh1.65889_cross0.60755_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_diagonal_bw5978.71195_skew0.91217_neigh1.66285_cross0.5548_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_diagonal_bw6442.07975_skew0.91217_neigh1.902_cross0.78882_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_diagonal_bw6597.81502_skew0.91217_neigh1.90014_cross0.78819_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_diagonal_bw7193.09661_skew0.91217_neigh1.79177_cross0.78885_n14.mtx","synthetic_140874_140874_7812605_avg55.4581_std11.08015_diagonal_bw6152.40279_skew2.11947_neigh1.6145_cross0.85299_n14.mtx","synthetic_140874_140874_7812630_avg55.45828_std11.08_diagonal_bw6149.91056_skew2.06537_neigh1.61493_cross0.85302_n14.mtx","synthetic_1429004_1429004_17510920_avg12.25393_std37.51291_diagonal_bw33558.90432_skew632.67408_neigh1.32147_cross0.78663_n14.mtx","synthetic_144963_144963_7361596_avg50.78259_std19.57819_diagonal_bw4399.88447_skew1.77654_neigh1.22836_cross0.71716_n14.mtx","synthetic_1521198_1521198_18638978_avg12.25283_std37.49579_diagonal_bw35682.83773_skew632.73123_neigh1.32136_cross0.78772_n14.mtx","synthetic_152542_152542_8133417_avg53.3192_std4.76067_diagonal_bw10659.33841_skew3.36991_neigh1.81127_cross0.94184_n14.mtx","synthetic_155701_155701_7905533_avg50.77381_std19.58895_diagonal_bw4730.89037_skew1.77702_neigh1.22735_cross0.71638_n14.mtx","synthetic_161070_161070_10085122_avg62.61329_std19.67482_diagonal_bw4970.48093_skew1.44357_neigh1.23583_cross0.74036_n14.mtx","synthetic_161070_161070_10630527_avg65.99942_std19.67949_diagonal_bw4997.9576_skew1.36366_neigh1.23787_cross0.74712_n14.mtx","synthetic_161070_161070_5763734_avg35.78403_std19.07947_diagonal_bw4691.45329_skew2.52112_neigh1.21157_cross0.6613_n14.mtx","synthetic_161070_161070_6293192_avg39.07116_std19.27381_diagonal_bw4754.31728_skew2.30167_neigh1.21361_cross0.6789_n14.mtx","synthetic_161070_161070_6828184_avg42.39265_std19.41502_diagonal_bw4805.07083_skew2.13734_neigh1.21786_cross0.69146_n14.mtx","synthetic_161070_161070_7366852_avg45.73696_std19.51164_diagonal_bw4846.14217_skew1.97353_neigh1.22207_cross0.70264_n14.mtx","synthetic_161070_161070_7908198_avg49.0979_std19.57809_diagonal_bw4854.49827_skew1.83108_neigh1.2254_cross0.71257_n14.mtx","synthetic_161070_161070_8177031_avg50.76694_std16.39176_diagonal_bw4982.53222_skew1.48193_neigh1.22352_cross0.73821_n14.mtx","synthetic_161070_161070_8177317_avg50.76872_std15.08847_diagonal_bw5015.21029_skew1.36366_neigh1.22314_cross0.74727_n14.mtx","synthetic_161070_161070_8177462_avg50.76962_std17.68784_diagonal_bw4942.64516_skew1.59998_neigh1.22734_cross0.72915_n14.mtx","synthetic_161070_161070_8177930_avg50.77252_std13.78024_diagonal_bw5053.39532_skew1.24531_neigh1.22227_cross0.75674_n14.mtx","synthetic_161070_161070_8178443_avg50.77571_std18.96773_diagonal_bw4913.07711_skew1.71784_neigh1.22757_cross0.72014_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_diagonal_bw3436.43137_skew1.77657_neigh1.22839_cross0.71742_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_diagonal_bw3766.73128_skew1.77657_neigh1.22745_cross0.71673_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_diagonal_bw4085.27859_skew1.77657_neigh1.22766_cross0.7165_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_diagonal_bw4397.09225_skew1.77657_neigh1.19684_cross0.82225_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_diagonal_bw4402.87749_skew1.77657_neigh1.22824_cross0.71675_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_diagonal_bw4582.03666_skew1.77657_neigh1.21427_cross0.78739_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_diagonal_bw4732.34129_skew1.77657_neigh1.22757_cross0.71639_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_diagonal_bw4806.31347_skew1.77657_neigh1.22333_cross0.7394_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_diagonal_bw4848.33162_skew1.77657_neigh0.945_cross0.71714_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_diagonal_bw4854.14888_skew1.77657_neigh0.86533_cross0.7155_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_diagonal_bw4858.05593_skew1.77657_neigh1.10606_cross0.71633_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_diagonal_bw4859.13659_skew1.77657_neigh1.02495_cross0.71552_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_diagonal_bw4887.50244_skew1.77657_neigh1.22794_cross0.7157_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_diagonal_bw4889.05284_skew1.77657_neigh1.18606_cross0.71625_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_diagonal_bw4905.36219_skew1.77657_neigh1.26702_cross0.71513_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_diagonal_bw4920.32821_skew1.77657_neigh1.34684_cross0.71421_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_diagonal_bw4938.19408_skew1.77657_neigh1.42871_cross0.71564_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_diagonal_bw4963.01172_skew1.77657_neigh1.50995_cross0.71659_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_diagonal_bw4974.48644_skew1.77657_neigh1.2296_cross0.6917_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_diagonal_bw5006.7359_skew1.77657_neigh1.58991_cross0.71725_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_diagonal_bw5047.52778_skew1.77657_neigh1.22518_cross0.71711_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_diagonal_bw5128.09611_skew1.77657_neigh1.23448_cross0.64524_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_diagonal_bw5252.15061_skew1.77657_neigh1.23809_cross0.60038_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_diagonal_bw5358.15437_skew1.77657_neigh1.24126_cross0.55385_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_diagonal_bw5368.754_skew1.77657_neigh1.22547_cross0.71707_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_diagonal_bw5466.81631_skew1.77657_neigh1.24378_cross0.50496_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_diagonal_bw5702.6109_skew1.77657_neigh1.22489_cross0.7154_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_diagonal_bw6025.13256_skew1.77657_neigh1.22428_cross0.71487_n14.mtx","synthetic_161070_161070_8179469_avg50.78208_std19.60099_diagonal_bw6352.66382_skew1.77657_neigh1.22335_cross0.71563_n14.mtx","synthetic_161070_161070_8181013_avg50.79166_std20.23119_diagonal_bw4876.39439_skew1.83511_neigh1.22713_cross0.71077_n14.mtx","synthetic_161070_161070_8181297_avg50.79343_std19.67479_diagonal_bw5249.20404_skew2.30751_neigh1.21368_cross0.79983_n14.mtx","synthetic_161070_161070_8184643_avg50.8142_std21.47457_diagonal_bw4835.93918_skew1.95193_neigh1.22927_cross0.70445_n14.mtx","synthetic_161070_161070_8190209_avg50.84876_std22.69412_diagonal_bw4805.6814_skew2.06792_neigh1.23025_cross0.69419_n14.mtx","synthetic_161070_161070_8197860_avg50.89626_std23.8911_diagonal_bw4766.49097_skew2.18295_neigh1.23139_cross0.68671_n14.mtx","synthetic_161070_161070_8207913_avg50.95867_std25.06294_diagonal_bw4739.27473_skew2.29679_neigh1.2307_cross0.67748_n14.mtx","synthetic_161070_161070_8451173_avg52.46895_std19.62109_diagonal_bw4901.00951_skew1.72542_neigh1.22729_cross0.72045_n14.mtx","synthetic_161070_161070_8995114_avg55.84599_std19.64654_diagonal_bw4934.95017_skew1.61433_neigh1.23106_cross0.72911_n14.mtx","synthetic_161070_161070_9539943_avg59.22855_std19.66505_diagonal_bw4956.28317_skew1.53256_neigh1.23358_cross0.73578_n14.mtx","synthetic_1613392_1613392_19770221_avg12.25382_std37.48086_diagonal_bw37889.56893_skew632.67976_neigh1.3217_cross0.78728_n14.mtx","synthetic_166438_166438_8452047_avg50.78195_std19.60687_diagonal_bw5049.85631_skew1.77658_neigh1.22495_cross0.71681_n14.mtx","synthetic_167070_167070_8908050_avg53.31927_std4.75919_diagonal_bw11705.28266_skew3.3699_neigh1.8115_cross0.9416_n14.mtx","synthetic_1705586_1705586_20897972_avg12.25266_std37.46728_diagonal_bw40049.59136_skew632.73974_neigh1.32185_cross0.78691_n14.mtx","synthetic_177177_177177_8998963_avg50.79081_std19.6143_diagonal_bw5379.20051_skew1.77609_neigh1.22643_cross0.71605_n14.mtx","synthetic_1797780_1797780_22027651_avg12.2527_std37.45466_diagonal_bw42214.60549_skew632.7381_neigh1.32143_cross0.78759_n14.mtx","synthetic_181598_181598_9682693_avg53.31938_std4.75813_diagonal_bw12669.60339_skew3.36989_neigh1.81184_cross0.9412_n14.mtx","synthetic_185639_185639_15001505_avg80.81009_std108.51788_diagonal_bw24304.53698_skew5.36059_neigh1.20249_cross0.88135_n14.mtx","synthetic_185639_185639_15005658_avg80.83247_std121.76167_diagonal_bw24068.62026_skew10.22074_neigh1.21908_cross0.83189_n14.mtx","synthetic_185639_185639_15005807_avg80.83327_std121.07944_diagonal_bw24098.67153_skew9.70104_neigh1.21713_cross0.83417_n14.mtx","synthetic_185639_185639_15006000_avg80.83431_std120.29592_diagonal_bw24022.966_skew9.18132_neigh1.21687_cross0.83669_n14.mtx","synthetic_185639_185639_15006242_avg80.83561_std119.38413_diagonal_bw24043.49472_skew8.66158_neigh1.21482_cross0.83974_n14.mtx","synthetic_185639_185639_15006279_avg80.83581_std111.39593_diagonal_bw24110.85286_skew5.93999_neigh1.20322_cross0.86853_n14.mtx","synthetic_185639_185639_15006681_avg80.83798_std118.31728_diagonal_bw24061.44702_skew8.14174_neigh1.21351_cross0.84318_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_diagonal_bw18670.03953_skew7.8818_neigh1.21456_cross0.84468_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_diagonal_bw20186.89682_skew7.8818_neigh1.2138_cross0.8455_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_diagonal_bw21725.13769_skew7.8818_neigh1.21452_cross0.84491_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_diagonal_bw22498.75533_skew7.8818_neigh1.19263_cross0.87688_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_diagonal_bw22743.18636_skew7.8818_neigh1.19716_cross0.87274_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_diagonal_bw23239.31109_skew7.8818_neigh1.21284_cross0.84509_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_diagonal_bw23967.12665_skew7.8818_neigh1.49579_cross0.84608_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_diagonal_bw24008.68622_skew7.8818_neigh1.33536_cross0.84607_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_diagonal_bw24018.14304_skew7.8818_neigh1.41334_cross0.84578_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_diagonal_bw24034.31555_skew7.8818_neigh1.25453_cross0.84477_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_diagonal_bw24106.64346_skew7.8818_neigh1.1756_cross0.84461_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_diagonal_bw24114.69933_skew7.8818_neigh1.57356_cross0.8475_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_diagonal_bw24131.56928_skew7.8818_neigh1.09712_cross0.84415_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_diagonal_bw24182.54428_skew7.8818_neigh1.01304_cross0.84375_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_diagonal_bw24230.56658_skew7.8818_neigh0.93581_cross0.84308_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_diagonal_bw24274.8686_skew7.8818_neigh0.85549_cross0.84272_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_diagonal_bw24811.92989_skew7.8818_neigh1.21348_cross0.84512_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_diagonal_bw24955.57252_skew7.8818_neigh1.2234_cross0.81646_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_diagonal_bw26276.50771_skew7.8818_neigh1.23325_cross0.75987_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_diagonal_bw26290.01987_skew7.8818_neigh1.21444_cross0.84532_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_diagonal_bw27182.66132_skew7.8818_neigh1.23712_cross0.7061_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_diagonal_bw27777.91443_skew7.8818_neigh1.2129_cross0.84488_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_diagonal_bw27910.49869_skew7.8818_neigh1.24252_cross0.65023_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_diagonal_bw28492.91056_skew7.8818_neigh1.24563_cross0.59619_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_diagonal_bw29291.3444_skew7.8818_neigh1.21419_cross0.84494_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_diagonal_bw30735.2641_skew7.8818_neigh1.21293_cross0.84528_n14.mtx","synthetic_185639_185639_15007163_avg80.84057_std117.05079_diagonal_bw24046.68677_skew7.60954_neigh1.21368_cross0.84709_n14.mtx","synthetic_185639_185639_15007417_avg80.84194_std113.68148_diagonal_bw24066.61476_skew6.50848_neigh1.20615_cross0.85925_n14.mtx","synthetic_185639_185639_15007427_avg80.84199_std115.53272_diagonal_bw24030.31379_skew7.06512_neigh1.21132_cross0.85201_n14.mtx","synthetic_187915_187915_9545196_avg50.79529_std19.60651_diagonal_bw5703.99496_skew1.77585_neigh1.22534_cross0.71637_n14.mtx","synthetic_196126_196126_10457351_avg53.31955_std4.75831_diagonal_bw13779.54019_skew3.36988_neigh1.81248_cross0.94112_n14.mtx","synthetic_198653_198653_10092676_avg50.80556_std19.60715_diagonal_bw6040.72041_skew1.77529_neigh1.2246_cross0.71507_n14.mtx","synthetic_205540_205540_14169500_avg68.93792_std101.25978_diagonal_bw23682.46024_skew10.08243_neigh1.12517_cross0.83739_n14.mtx","synthetic_209391_209391_10637567_avg50.80241_std19.60148_diagonal_bw6367.17197_skew1.77546_neigh1.22433_cross0.71571_n14.mtx","synthetic_210654_210654_11231930_avg53.31933_std4.75723_diagonal_bw14684.9664_skew3.3699_neigh1.81334_cross0.94103_n14.mtx","synthetic_217918_217918_10485592_avg48.11714_std4.75335_diagonal_bw15572.84676_skew3.36435_neigh1.77256_cross0.93985_n14.mtx","synthetic_217918_217918_11231492_avg51.53999_std4.76893_diagonal_bw15690.56769_skew3.36554_neigh1.75769_cross0.93992_n14.mtx","synthetic_217918_217918_11582841_avg53.15229_std3.34176_diagonal_bw14752.54676_skew3.38363_neigh1.83064_cross0.94044_n14.mtx","synthetic_217918_217918_11590559_avg53.18771_std3.65776_diagonal_bw14759.15174_skew3.38071_neigh1.82883_cross0.94023_n14.mtx","synthetic_217918_217918_11598747_avg53.22528_std3.97324_diagonal_bw14804.58821_skew3.37762_neigh1.82457_cross0.94082_n14.mtx","synthetic_217918_217918_11607157_avg53.26387_std4.28638_diagonal_bw14926.5797_skew3.37445_neigh1.82058_cross0.94051_n14.mtx","synthetic_217918_217918_11608217_avg53.26874_std4.76281_diagonal_bw14978.94192_skew4.08741_neigh1.8196_cross0.94057_n14.mtx","synthetic_217918_217918_11610489_avg53.27916_std4.76243_diagonal_bw14965.80924_skew3.93626_neigh1.8186_cross0.94068_n14.mtx","synthetic_217918_217918_11612905_avg53.29025_std4.75996_diagonal_bw15022.46198_skew3.76635_neigh1.8171_cross0.94082_n14.mtx","synthetic_217918_217918_11615389_avg53.30165_std4.60014_diagonal_bw15069.01749_skew3.37135_neigh1.81567_cross0.941_n14.mtx","synthetic_217918_217918_11615400_avg53.3017_std4.75902_diagonal_bw15068.40239_skew3.61524_neigh1.81564_cross0.94109_n14.mtx","synthetic_217918_217918_11617960_avg53.31345_std4.75796_diagonal_bw15125.67497_skew3.44541_neigh1.81377_cross0.94083_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_diagonal_bw10615.26121_skew3.36989_neigh1.81293_cross0.941_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_diagonal_bw11723.96417_skew3.36989_neigh1.81304_cross0.94098_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_diagonal_bw12548.56527_skew3.36989_neigh1.50637_cross0.94082_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_diagonal_bw12555.41407_skew3.36989_neigh1.26528_cross0.93926_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_diagonal_bw12561.18815_skew3.36989_neigh1.38503_cross0.94038_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_diagonal_bw12662.63745_skew3.36989_neigh1.81249_cross0.94104_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_diagonal_bw12875.08402_skew3.36989_neigh1.6263_cross0.94024_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_diagonal_bw13192.62211_skew3.36989_neigh1.74925_cross0.94027_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_diagonal_bw13589.06211_skew3.36989_neigh1.8131_cross0.94142_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_diagonal_bw14637.1457_skew3.36989_neigh1.81291_cross0.94097_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_diagonal_bw14639.64249_skew3.36989_neigh1.83771_cross0.66265_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_diagonal_bw14678.79878_skew3.36989_neigh1.83641_cross0.72459_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_diagonal_bw14692.70001_skew3.36989_neigh1.83507_cross0.78486_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_diagonal_bw14839.20089_skew3.36989_neigh1.8307_cross0.84682_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_diagonal_bw15050.52926_skew3.36989_neigh1.82143_cross0.90888_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_diagonal_bw15508.12602_skew3.36989_neigh1.7873_cross0.97163_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_diagonal_bw15626.79599_skew3.36989_neigh1.81205_cross0.94094_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_diagonal_bw15937.73698_skew3.36989_neigh1.74855_cross0.98962_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_diagonal_bw16599.98545_skew3.36989_neigh1.81308_cross0.9411_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_diagonal_bw16602.40328_skew3.36989_neigh1.87566_cross0.93981_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_diagonal_bw17572.60761_skew3.36989_neigh1.81311_cross0.94094_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_diagonal_bw18584.92687_skew3.36989_neigh1.81265_cross0.94092_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_diagonal_bw19575.60228_skew3.36989_neigh1.81285_cross0.94097_n14.mtx","synthetic_217918_217918_11619250_avg53.31937_std4.7566_diagonal_bw5775.00642_skew3.36989_neigh1.93946_cross0.94054_n14.mtx","synthetic_217918_217918_11620586_avg53.3255_std4.75691_diagonal_bw15174.3723_skew3.29438_neigh1.81188_cross0.94105_n14.mtx","synthetic_217918_217918_11622834_avg53.33581_std4.91358_diagonal_bw15211.80314_skew3.36855_neigh1.80941_cross0.94098_n14.mtx","synthetic_217918_217918_11623241_avg53.33768_std4.75547_diagonal_bw15225.00254_skew3.12466_neigh1.80948_cross0.9409_n14.mtx","synthetic_217918_217918_11625780_avg53.34933_std4.75397_diagonal_bw15317.68741_skew2.97381_neigh1.806_cross0.94062_n14.mtx","synthetic_217918_217918_11628184_avg53.36036_std4.75308_diagonal_bw15462.19266_skew2.80432_neigh1.8014_cross0.94016_n14.mtx","synthetic_217918_217918_11628620_avg53.36237_std5.22805_diagonal_bw15467.4387_skew3.36637_neigh1.80121_cross0.93991_n14.mtx","synthetic_217918_217918_11630299_avg53.37007_std4.7531_diagonal_bw15559.9952_skew2.65373_neigh1.79743_cross0.9399_n14.mtx","synthetic_217918_217918_11632309_avg53.37929_std5.54522_diagonal_bw15720.40304_skew3.36499_neigh1.78973_cross0.93981_n14.mtx","synthetic_217918_217918_11634099_avg53.38751_std5.86172_diagonal_bw15948.7068_skew3.36432_neigh1.77763_cross0.93909_n14.mtx","synthetic_217918_217918_11634741_avg53.39045_std6.17863_diagonal_bw16156.60265_skew3.36408_neigh1.76393_cross0.93894_n14.mtx","synthetic_217918_217918_12031582_avg55.21151_std4.74931_diagonal_bw15391.15638_skew3.36503_neigh1.82291_cross0.9415_n14.mtx","synthetic_217918_217918_12797820_avg58.72769_std4.7736_diagonal_bw13915.978_skew3.3591_neigh1.75599_cross0.93955_n14.mtx","synthetic_217918_217918_13561979_avg62.23432_std4.75526_diagonal_bw13183.43978_skew3.37058_neigh1.83856_cross0.94033_n14.mtx","synthetic_217918_217918_14374351_avg65.9622_std4.76726_diagonal_bw14447.91926_skew3.36614_neigh1.78633_cross0.93943_n14.mtx","synthetic_217918_217918_15093236_avg69.26108_std4.76589_diagonal_bw13990.39745_skew3.37475_neigh1.82922_cross0.94054_n14.mtx","synthetic_217918_217918_8143542_avg37.36975_std4.75333_diagonal_bw20866.53868_skew3.36182_neigh1.74733_cross0.9391_n14.mtx","synthetic_217918_217918_8924884_avg40.95524_std4.76049_diagonal_bw22294.09855_skew3.34621_neigh1.71612_cross0.93867_n14.mtx","synthetic_217918_217918_9684687_avg44.44189_std4.75752_diagonal_bw14468.16582_skew3.36525_neigh1.77178_cross0.93964_n14.mtx","synthetic_223413_223413_15400297_avg68.93196_std101.25897_diagonal_bw25758.67821_skew10.0979_neigh1.12654_cross0.83736_n14.mtx","synthetic_225181_225181_12006505_avg53.31935_std4.75652_diagonal_bw15512.77201_skew3.3699_neigh1.81265_cross0.94113_n14.mtx","synthetic_239709_239709_12781051_avg53.31903_std4.75618_diagonal_bw16764.10618_skew3.36992_neigh1.81204_cross0.94086_n14.mtx","synthetic_241286_241286_16633118_avg68.93528_std101.25723_diagonal_bw27818.08052_skew10.09737_neigh1.12625_cross0.83765_n14.mtx","synthetic_254237_254237_13555788_avg53.31949_std4.75562_diagonal_bw17672.99219_skew3.36988_neigh1.81182_cross0.94118_n14.mtx","synthetic_259159_259159_17866974_avg68.94213_std101.25583_diagonal_bw29844.7971_skew10.11077_neigh1.1228_cross0.83747_n14.mtx","synthetic_26684_26684_10204693_avg382.42741_std245.44826_diagonal_bw11448.26195_skew1.57042_neigh1.97454_cross0.92536_n14.mtx","synthetic_268096_268096_12941160_avg48.27062_std95.20699_diagonal_bw26357.31682_skew9.2754_neigh1.08518_cross0.89421_n14.mtx","synthetic_268096_268096_14172610_avg52.86394_std97.03631_diagonal_bw29123.12309_skew9.5554_neigh1.0936_cross0.86612_n14.mtx","synthetic_268096_268096_15403716_avg57.45597_std98.50696_diagonal_bw30125.15948_skew9.75606_neigh1.10358_cross0.85268_n14.mtx","synthetic_268096_268096_16634948_avg62.04848_std99.72897_diagonal_bw30499.11718_skew9.92694_neigh1.11411_cross0.84513_n14.mtx","synthetic_268096_268096_17866195_avg66.64104_std100.78017_diagonal_bw30751.27234_skew10.05925_neigh1.12044_cross0.83953_n14.mtx","synthetic_268096_268096_18479164_avg68.92741_std73.74951_diagonal_bw31377.65125_skew10.4033_neigh1.1237_cross0.84107_n14.mtx","synthetic_268096_268096_18479645_avg68.92921_std80.36452_diagonal_bw31191.46774_skew10.403_neigh1.12752_cross0.82739_n14.mtx","synthetic_268096_268096_18480123_avg68.93099_std86.42344_diagonal_bw31125.07403_skew10.33017_neigh1.12545_cross0.82944_n14.mtx","synthetic_268096_268096_18480967_avg68.93414_std92.41882_diagonal_bw31046.67467_skew10.25712_neigh1.12466_cross0.8325_n14.mtx","synthetic_268096_268096_18481691_avg68.93684_std98.3339_diagonal_bw30926.5338_skew10.16964_neigh1.12377_cross0.83538_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_diagonal_bw21923.44482_skew10.11157_neigh1.12703_cross0.83756_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_diagonal_bw23935.8137_skew10.11157_neigh1.12612_cross0.8371_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_diagonal_bw25895.422_skew10.11157_neigh1.12483_cross0.83763_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_diagonal_bw27899.12455_skew10.11157_neigh1.12567_cross0.83764_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_diagonal_bw29902.52638_skew10.11157_neigh1.12497_cross0.83752_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_diagonal_bw31924.98227_skew10.11157_neigh1.12445_cross0.83711_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_diagonal_bw33817.07914_skew10.11157_neigh1.12399_cross0.83739_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_diagonal_bw35779.99142_skew10.11157_neigh1.12322_cross0.83708_n14.mtx","synthetic_268096_268096_18482362_avg68.93934_std104.15593_diagonal_bw30765.9396_skew10.0677_neigh1.12333_cross0.83908_n14.mtx","synthetic_268096_268096_18483075_avg68.942_std109.86851_diagonal_bw30647.70205_skew9.95123_neigh1.12162_cross0.84333_n14.mtx","synthetic_268096_268096_18483963_avg68.94531_std115.46168_diagonal_bw30446.51037_skew9.82017_neigh1.11909_cross0.84887_n14.mtx","synthetic_268096_268096_18484698_avg68.94806_std120.91939_diagonal_bw30186.45871_skew9.68921_neigh1.11754_cross0.85523_n14.mtx","synthetic_268096_268096_18485774_avg68.95207_std126.22922_diagonal_bw29770.97031_skew9.55806_neigh1.11488_cross0.86335_n14.mtx","synthetic_268096_268096_19097540_avg71.23396_std101.70964_diagonal_bw30933.99411_skew10.16041_neigh1.12541_cross0.83599_n14.mtx","synthetic_268096_268096_20328893_avg75.82692_std102.55303_diagonal_bw31081.7585_skew10.2493_neigh1.1336_cross0.83222_n14.mtx","synthetic_268096_268096_21560408_avg80.42048_std103.33442_diagonal_bw31148.48253_skew10.30309_neigh1.13451_cross0.82948_n14.mtx","synthetic_268096_268096_22792018_avg85.01439_std104.0723_diagonal_bw31199.25223_skew10.36278_neigh1.13885_cross0.8282_n14.mtx","synthetic_268096_268096_24023179_avg89.60663_std104.77794_diagonal_bw31239.70547_skew10.40541_neigh1.14366_cross0.82667_n14.mtx","synthetic_268765_268765_14330411_avg53.31948_std4.75541_diagonal_bw18731.37211_skew3.36988_neigh1.81156_cross0.94066_n14.mtx","synthetic_277032_277032_19096094_avg68.931_std101.25944_diagonal_bw31953.72601_skew10.12707_neigh1.12352_cross0.83736_n14.mtx","synthetic_283293_283293_15104954_avg53.31919_std4.75508_diagonal_bw19749.01419_skew3.36991_neigh1.81186_cross0.9407_n14.mtx","synthetic_29225_29225_11176419_avg382.42666_std245.44724_diagonal_bw12425.97666_skew1.57043_neigh1.97381_cross0.92588_n14.mtx","synthetic_294905_294905_20328825_avg68.93347_std101.25836_diagonal_bw33931.21622_skew10.12667_neigh1.12413_cross0.83784_n14.mtx","synthetic_2998_2998_6123474_avg2042.51968_std621.91091_diagonal_bw2392.08572_skew0.46779_neigh1.915_cross0.85498_n14.mtx","synthetic_312778_312778_21563736_avg68.94262_std101.25558_diagonal_bw36065.10736_skew10.1397_neigh1.12116_cross0.83732_n14.mtx","synthetic_31766_31766_12148131_avg382.42558_std245.44713_diagonal_bw13817.64257_skew1.57043_neigh1.97384_cross0.91941_n14.mtx","synthetic_3284_3284_6742726_avg2053.20524_std666.12092_diagonal_bw2537.96711_skew0.59945_neigh1.8895_cross0.83825_n14.mtx","synthetic_330651_330651_22801392_avg68.95909_std101.25183_diagonal_bw38055.27574_skew10.13704_neigh1.1241_cross0.83714_n14.mtx","synthetic_34308_34308_13120227_avg382.42471_std245.44609_diagonal_bw14650.04739_skew1.57044_neigh1.97406_cross0.9232_n14.mtx","synthetic_348524_348524_24034194_avg68.95994_std101.25128_diagonal_bw40157.18736_skew10.1369_neigh1.12259_cross0.83707_n14.mtx","synthetic_3569_3569_7446679_avg2086.48893_std694.14116_diagonal_bw2698.16419_skew0.71053_neigh1.86781_cross0.82629_n14.mtx","synthetic_36849_36849_14091918_avg382.42335_std245.44508_diagonal_bw15781.99004_skew1.57045_neigh1.97399_cross0.92492_n14.mtx","synthetic_38120_38120_10204584_avg267.69633_std171.81206_diagonal_bw16895.89722_skew1.57008_neigh1.97152_cross0.92219_n14.mtx","synthetic_38120_38120_11176434_avg293.19082_std188.17435_diagonal_bw17618.12893_skew1.5717_neigh1.97171_cross0.9225_n14.mtx","synthetic_38120_38120_12148318_avg318.6862_std204.53731_diagonal_bw18097.15551_skew1.56993_neigh1.97265_cross0.92312_n14.mtx","synthetic_38120_38120_13120182_avg344.18106_std220.90077_diagonal_bw18722.69885_skew1.57132_neigh1.97302_cross0.92199_n14.mtx","synthetic_38120_38120_14004492_avg367.37912_std206.50003_diagonal_bw16517.99428_skew1.32185_neigh1.97296_cross0.92309_n14.mtx","synthetic_38120_38120_14092040_avg369.67576_std237.2634_diagonal_bw15900.27104_skew1.56982_neigh1.97398_cross0.92174_n14.mtx","synthetic_38120_38120_14186695_avg372.15884_std217.74813_diagonal_bw16610.97285_skew1.39145_neigh1.97336_cross0.92298_n14.mtx","synthetic_38120_38120_14353565_avg376.53633_std228.90355_diagonal_bw16362.02206_skew1.46191_neigh1.97392_cross0.9229_n14.mtx","synthetic_38120_38120_14506403_avg380.54572_std239.95763_diagonal_bw16221.47907_skew1.5332_neigh1.97371_cross0.92545_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_diagonal_bw11840.96207_skew1.57045_neigh1.97389_cross0.92172_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_diagonal_bw13051.93368_skew1.57045_neigh1.97364_cross0.9215_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_diagonal_bw14054.5128_skew1.57045_neigh1.97418_cross0.92287_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_diagonal_bw14901.13825_skew1.57045_neigh1.9738_cross0.92245_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_diagonal_bw15702.30456_skew1.57045_neigh1.97328_cross0.9903_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_diagonal_bw16005.2249_skew1.57045_neigh1.97376_cross0.92394_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_diagonal_bw16107.56653_skew1.57045_neigh1.90891_cross0.91992_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_diagonal_bw16165.12815_skew1.57045_neigh1.3922_cross0.92174_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_diagonal_bw16173.10614_skew1.57045_neigh1.64922_cross0.92071_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_diagonal_bw16181.2495_skew1.57045_neigh1.51874_cross0.92011_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_diagonal_bw16189.91522_skew1.57045_neigh1.77707_cross0.91956_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_diagonal_bw16251.80551_skew1.57045_neigh1.97384_cross0.89306_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_diagonal_bw16403.35273_skew1.57045_neigh1.97397_cross0.72233_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_diagonal_bw16412.02442_skew1.57045_neigh1.97389_cross0.66503_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_diagonal_bw16450.53809_skew1.57045_neigh1.9739_cross0.77998_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_diagonal_bw16454.6271_skew1.57045_neigh1.97336_cross0.95155_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_diagonal_bw16504.44638_skew1.57045_neigh1.97349_cross0.83444_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_diagonal_bw16601.00073_skew1.57045_neigh1.97376_cross0.92272_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_diagonal_bw16971.93463_skew1.57045_neigh1.9734_cross0.92126_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_diagonal_bw17802.86905_skew1.57045_neigh1.97345_cross0.9247_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_diagonal_bw18943.77943_skew1.57045_neigh1.97338_cross0.92328_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_diagonal_bw20020.58657_skew1.57045_neigh1.97348_cross0.92229_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_diagonal_bw21202.83549_skew1.57045_neigh1.97368_cross0.92139_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_diagonal_bw5783.47169_skew1.57045_neigh1.99394_cross0.9293_n14.mtx","synthetic_38120_38120_14646455_avg384.2197_std250.9036_diagonal_bw16369.04491_skew1.60788_neigh1.97362_cross0.92456_n14.mtx","synthetic_38120_38120_14774683_avg387.5835_std261.73289_diagonal_bw16271.8489_skew1.68071_neigh1.97337_cross0.92347_n14.mtx","synthetic_38120_38120_14892124_avg390.66432_std272.44037_diagonal_bw16642.15774_skew1.75428_neigh1.9735_cross0.92246_n14.mtx","synthetic_38120_38120_15009192_avg393.73536_std249.23315_diagonal_bw17334.85677_skew1.65915_neigh1.95771_cross0.90213_n14.mtx","synthetic_38120_38120_15029193_avg394.26005_std283.24138_diagonal_bw17593.26878_skew1.83569_neigh1.95985_cross0.92052_n14.mtx","synthetic_38120_38120_15063906_avg395.17067_std253.62681_diagonal_bw16677.30215_skew1.57104_neigh1.97408_cross0.92294_n14.mtx","synthetic_38120_38120_15246247_avg399.95401_std294.48908_diagonal_bw17717.6793_skew1.94534_neigh1.95613_cross0.91225_n14.mtx","synthetic_38120_38120_15509350_avg406.85598_std254.10081_diagonal_bw16470.84627_skew1.83638_neigh1.96076_cross0.87593_n14.mtx","synthetic_38120_38120_15831412_avg415.30462_std256.81933_diagonal_bw15755.80622_skew2.01225_neigh1.96337_cross0.85513_n14.mtx","synthetic_38120_38120_16013257_avg420.07495_std256.51996_diagonal_bw15554.95262_skew2.18991_neigh1.96477_cross0.83814_n14.mtx","synthetic_38120_38120_16417883_avg430.68948_std273.30431_diagonal_bw17866.36671_skew1.63763_neigh1.95894_cross0.90329_n14.mtx","synthetic_38120_38120_17911818_avg469.8798_std294.64729_diagonal_bw15437.19732_skew1.77305_neigh1.96314_cross0.88403_n14.mtx","synthetic_38120_38120_17943763_avg470.71781_std404.16901_diagonal_bw11533.34024_skew3.74169_neigh1.97374_cross0.57584_n14.mtx","synthetic_38120_38120_19284934_avg505.90068_std314.8817_diagonal_bw15240.20312_skew1.89583_neigh1.96535_cross0.86661_n14.mtx","synthetic_38120_38120_20568467_avg539.57154_std333.79776_diagonal_bw14935.54641_skew2.00609_neigh1.96694_cross0.85709_n14.mtx","synthetic_3855_3855_8084357_avg2097.10947_std723.99546_diagonal_bw2875.22931_skew0.83824_neigh1.84384_cross0.81071_n14.mtx","synthetic_39390_39390_15063634_avg382.4228_std245.44522_diagonal_bw17150.32615_skew1.57045_neigh1.97365_cross0.9222_n14.mtx","synthetic_4141_4141_8723864_avg2106.70466_std753.89371_diagonal_bw3055.31055_skew0.96563_neigh1.82243_cross0.79514_n14.mtx","synthetic_41932_41932_16035737_avg382.42242_std245.44481_diagonal_bw18021.62279_skew1.57046_neigh1.97387_cross0.92405_n14.mtx","synthetic_4284_4284_10161733_avg2372.01984_std415.11429_diagonal_bw3157.98039_skew0.80606_neigh1.8259_cross0.87786_n14.mtx","synthetic_4284_4284_10296320_avg2403.43604_std664.88425_diagonal_bw3183.64402_skew0.78245_neigh1.84578_cross0.84547_n14.mtx","synthetic_4284_4284_11128593_avg2597.71078_std612.25995_diagonal_bw3227.32843_skew0.64914_neigh1.86927_cross0.87428_n14.mtx","synthetic_4284_4284_11955257_avg2790.67624_std565.77456_diagonal_bw3299.09944_skew0.53511_neigh1.89073_cross0.89611_n14.mtx","synthetic_4284_4284_12776299_avg2982.32937_std524.4261_diagonal_bw3366.44678_skew0.43646_neigh1.91312_cross0.91453_n14.mtx","synthetic_4284_4284_5176835_avg1208.41153_std875.52905_diagonal_bw3144.46919_skew2.54515_neigh1.74013_cross0.56551_n14.mtx","synthetic_4284_4284_6045664_avg1411.21942_std813.05724_diagonal_bw3116.32236_skew2.03567_neigh1.73991_cross0.63076_n14.mtx","synthetic_4284_4284_6908449_avg1612.61648_std777.77715_diagonal_bw3104.0042_skew1.65655_neigh1.75254_cross0.68691_n14.mtx","synthetic_4284_4284_7606767_avg1775.62255_std826.63023_diagonal_bw3138.01167_skew1.41267_neigh1.77542_cross0.71893_n14.mtx","synthetic_4284_4284_7764407_avg1812.41993_std766.39496_diagonal_bw3107.61204_skew1.36369_neigh1.77486_cross0.73027_n14.mtx","synthetic_4284_4284_7948514_avg1855.39542_std808.46009_diagonal_bw3131.42437_skew1.30894_neigh1.78337_cross0.73476_n14.mtx","synthetic_4284_4284_8278029_avg1932.31303_std796.54977_diagonal_bw3133.04062_skew1.21703_neigh1.79149_cross0.75251_n14.mtx","synthetic_4284_4284_8399652_avg1960.70308_std840.91619_diagonal_bw3170.12582_skew1.18493_neigh1.79818_cross0.75117_n14.mtx","synthetic_4284_4284_8563923_avg1999.04832_std817.99013_diagonal_bw3151.49883_skew1.14302_neigh1.802_cross0.75986_n14.mtx","synthetic_4284_4284_8591948_avg2005.5901_std788.67453_diagonal_bw3136.71662_skew1.13603_neigh1.80156_cross0.76701_n14.mtx","synthetic_4284_4284_8613657_avg2010.65756_std774.7613_diagonal_bw3127.03525_skew1.13065_neigh1.8022_cross0.76595_n14.mtx","synthetic_4284_4284_8716487_avg2034.66083_std804.2751_diagonal_bw3151.36345_skew1.10551_neigh1.80589_cross0.76962_n14.mtx","synthetic_4284_4284_8853517_avg2066.64729_std794.27725_diagonal_bw3141.82166_skew1.07292_neigh1.80997_cross0.77415_n14.mtx","synthetic_4284_4284_8891530_avg2075.52054_std785.22497_diagonal_bw3124.03922_skew1.06406_neigh1.81154_cross0.77892_n14.mtx","synthetic_4284_4284_8977954_avg2095.69421_std780.20568_diagonal_bw3133.23389_skew1.04419_neigh1.81378_cross0.78051_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_diagonal_bw2587.0796_skew1.03117_neigh1.8919_cross0.84164_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_diagonal_bw2710.43301_skew1.03117_neigh1.87148_cross0.83111_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_diagonal_bw2823.22969_skew1.03117_neigh1.85445_cross0.8163_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_diagonal_bw2954.99697_skew1.03117_neigh1.83634_cross0.8091_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_diagonal_bw3067.13329_skew1.03117_neigh1.82242_cross0.79404_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_diagonal_bw3102.39846_skew1.03117_neigh1.81661_cross0.79393_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_diagonal_bw3107.22993_skew1.03117_neigh1.8162_cross0.79153_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_diagonal_bw3110.80742_skew1.03117_neigh1.81584_cross0.79256_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_diagonal_bw3114.21218_skew1.03117_neigh1.99853_cross0.70901_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_diagonal_bw3121.18417_skew1.03117_neigh1.81524_cross0.79023_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_diagonal_bw3126.92857_skew1.03117_neigh1.87269_cross0.76602_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_diagonal_bw3127.45448_skew1.03117_neigh1.91846_cross0.74602_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_diagonal_bw3130.74066_skew1.03117_neigh1.8144_cross0.78625_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_diagonal_bw3132.41083_skew1.03117_neigh1.8143_cross0.78794_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_diagonal_bw3133.53035_skew1.03117_neigh1.83285_cross0.77848_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_diagonal_bw3135.26284_skew1.03117_neigh1.79588_cross0.7941_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_diagonal_bw3135.64356_skew1.03117_neigh1.76094_cross0.80472_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_diagonal_bw3142.22782_skew1.03117_neigh1.81328_cross0.78516_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_diagonal_bw3144.31513_skew1.03117_neigh1.72858_cross0.81697_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_diagonal_bw3149.68371_skew1.03117_neigh1.69844_cross0.82642_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_diagonal_bw3156.4239_skew1.03117_neigh1.81211_cross0.78361_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_diagonal_bw3156.47082_skew1.03117_neigh1.96828_cross0.71812_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_diagonal_bw3162.04692_skew1.03117_neigh1.81086_cross0.78323_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_diagonal_bw3171.51494_skew1.03117_neigh1.66807_cross0.8333_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_diagonal_bw3178.75864_skew1.03117_neigh1.80961_cross0.78012_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_diagonal_bw3198.155_skew1.03117_neigh1.80553_cross0.78061_n14.mtx","synthetic_4284_4284_9035496_avg2109.12605_std755.93589_diagonal_bw3209.27381_skew1.03117_neigh1.80497_cross0.77778_n14.mtx","synthetic_4284_4284_9091186_avg2122.12558_std733.44441_diagonal_bw3133.95915_skew1.01873_neigh1.81407_cross0.79058_n14.mtx","synthetic_4284_4284_9176988_avg2142.15406_std713.01815_diagonal_bw3139.19981_skew0.99986_neigh1.8134_cross0.80089_n14.mtx","synthetic_4284_4284_9196527_avg2146.71499_std692.64997_diagonal_bw3137.96078_skew0.99561_neigh1.81239_cross0.80355_n14.mtx","synthetic_4284_4284_9292087_avg2169.02124_std655.40775_diagonal_bw3136.66106_skew0.97508_neigh1.81305_cross0.81195_n14.mtx","synthetic_4284_4284_9379803_avg2189.4965_std621.87428_diagonal_bw3140.22502_skew0.95661_neigh1.81248_cross0.82177_n14.mtx","synthetic_4284_4284_9448827_avg2205.60854_std630.77463_diagonal_bw3151.63002_skew0.94232_neigh1.81415_cross0.82352_n14.mtx","synthetic_4284_4284_9456745_avg2207.45682_std723.4564_diagonal_bw3150.4225_skew0.94069_neigh1.8231_cross0.80811_n14.mtx","synthetic_4284_4284_9460554_avg2208.34594_std591.75846_diagonal_bw3131.92063_skew0.93991_neigh1.81387_cross0.82831_n14.mtx","synthetic_4284_4284_9703417_avg2265.03665_std553.19719_diagonal_bw3154.62862_skew0.89136_neigh1.8169_cross0.84303_n14.mtx","synthetic_4284_4284_9941769_avg2320.67437_std481.87027_diagonal_bw3156.55415_skew0.84602_neigh1.82089_cross0.86146_n14.mtx","synthetic_4426_4426_9354099_avg2113.44306_std776.70231_diagonal_bw3225.23949_skew1.09421_neigh1.80628_cross0.77807_n14.mtx","synthetic_44473_44473_17007427_avg382.4214_std245.44406_diagonal_bw18588.49515_skew1.57046_neigh1.9742_cross0.92197_n14.mtx","synthetic_44686_44686_9904566_avg221.64808_std96.54092_diagonal_bw28996.12507_skew15.40889_neigh1.70385_cross0.85407_n14.mtx","synthetic_46524_46524_7261288_avg156.07618_std357.60767_diagonal_bw23117.62254_skew297.08521_neigh1.84312_cross0.94487_n14.mtx","synthetic_47014_47014_17979151_avg382.42121_std245.44364_diagonal_bw20028.40735_skew1.57046_neigh1.97384_cross0.92279_n14.mtx","synthetic_4712_4712_10000031_avg2122.24767_std807.79622_diagonal_bw3425.88222_skew1.22029_neigh1.78887_cross0.75824_n14.mtx","synthetic_48942_48942_10847397_avg221.6378_std96.4853_diagonal_bw31784.2128_skew15.40966_neigh1.7043_cross0.85189_n14.mtx","synthetic_49556_49556_18951229_avg382.42047_std245.44344_diagonal_bw21059.9188_skew1.57047_neigh1.97406_cross0.92093_n14.mtx","synthetic_4998_4998_10667079_avg2134.26951_std834.36333_diagonal_bw3626.9994_skew1.34178_neigh1.7719_cross0.74002_n14.mtx","synthetic_50954_50954_7955612_avg156.13322_std368.01601_diagonal_bw25406.9614_skew325.34951_neigh1.84494_cross0.94426_n14.mtx","synthetic_5283_5283_11298374_avg2138.62843_std877.77445_diagonal_bw3805.81677_skew1.47027_neigh1.7624_cross0.71965_n14.mtx","synthetic_53198_53198_11791218_avg221.64777_std96.43769_diagonal_bw34602.16726_skew15.40892_neigh1.70332_cross0.85286_n14.mtx","synthetic_55385_55385_8650205_avg156.18317_std377.56925_diagonal_bw27666.03716_skew353.61567_neigh1.84432_cross0.94383_n14.mtx","synthetic_5569_5569_11949345_avg2145.68953_std911.81863_diagonal_bw4008.83264_skew1.59544_neigh1.75126_cross0.70162_n14.mtx","synthetic_57454_57454_12734908_avg221.65398_std96.39664_diagonal_bw37281.76299_skew15.40846_neigh1.70414_cross0.85028_n14.mtx","synthetic_59816_59816_9344868_avg156.2269_std386.43932_diagonal_bw29761.58428_skew381.87902_neigh1.84288_cross0.94411_n14.mtx","synthetic_61710_61710_13678136_avg221.65186_std96.36424_diagonal_bw39991.18778_skew15.40862_neigh1.70436_cross0.85228_n14.mtx","synthetic_63838_63838_10847423_avg169.9211_std96.15794_diagonal_bw40401.27927_skew15.38408_neigh1.69606_cross0.84555_n14.mtx","synthetic_63838_63838_11790856_avg184.69965_std96.20654_diagonal_bw40776.4429_skew15.38877_neigh1.69867_cross0.84918_n14.mtx","synthetic_63838_63838_12734460_avg199.48087_std96.25772_diagonal_bw41109.50902_skew15.39756_neigh1.70139_cross0.84605_n14.mtx","synthetic_63838_63838_13677970_avg214.26063_std96.31751_diagonal_bw41355.45747_skew15.40525_neigh1.70246_cross0.84998_n14.mtx","synthetic_63838_63838_14148710_avg221.63461_std96.12924_diagonal_bw40820.71309_skew11.06039_neigh1.70496_cross0.84517_n14.mtx","synthetic_63838_63838_14148875_avg221.63719_std96.16987_diagonal_bw40978.10773_skew12.02579_neigh1.7049_cross0.84703_n14.mtx","synthetic_63838_63838_14148974_avg221.63874_std125.01008_diagonal_bw40403.16755_skew15.38251_neigh1.70576_cross0.84119_n14.mtx","synthetic_63838_63838_14149076_avg221.64034_std96.21451_diagonal_bw41078.1748_skew12.99114_neigh1.70372_cross0.8494_n14.mtx","synthetic_63838_63838_14149188_avg221.64209_std118.63327_diagonal_bw40496.69658_skew15.38678_neigh1.70525_cross0.84551_n14.mtx","synthetic_63838_63838_14149208_avg221.64241_std96.26387_diagonal_bw41309.92768_skew13.96104_neigh1.7036_cross0.84889_n14.mtx","synthetic_63838_63838_14149296_avg221.64379_std112.26231_diagonal_bw40884.59971_skew15.39568_neigh1.70481_cross0.84573_n14.mtx","synthetic_63838_63838_14149329_avg221.6443_std105.89153_diagonal_bw41036.28118_skew15.40015_neigh1.70463_cross0.8482_n14.mtx","synthetic_63838_63838_14149335_avg221.6444_std96.31796_diagonal_bw41384.37215_skew14.92641_neigh1.70468_cross0.85201_n14.mtx","synthetic_63838_63838_14149338_avg221.64444_std99.52693_diagonal_bw41338.26943_skew15.40465_neigh1.70463_cross0.85088_n14.mtx","synthetic_63838_63838_14149538_avg221.64758_std93.16614_diagonal_bw41616.56296_skew15.40893_neigh1.70332_cross0.8503_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_diagonal_bw31209.32828_skew15.40883_neigh1.70379_cross0.85318_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_diagonal_bw31263.79578_skew15.40883_neigh1.97907_cross0.90114_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_diagonal_bw33310.08077_skew15.40883_neigh1.62891_cross0.97834_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_diagonal_bw33741.33778_skew15.40883_neigh1.70407_cross0.84949_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_diagonal_bw36088.99159_skew15.40883_neigh1.70329_cross0.85235_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_diagonal_bw38238.34086_skew15.40883_neigh1.70401_cross0.8519_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_diagonal_bw38844.09753_skew15.40883_neigh1.68246_cross0.9336_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_diagonal_bw40356.3582_skew15.40883_neigh1.70389_cross0.85225_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_diagonal_bw40916.26257_skew15.40883_neigh1.69941_cross0.87831_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_diagonal_bw41070.66258_skew15.40883_neigh1.19535_cross0.84968_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_diagonal_bw41074.56236_skew15.40883_neigh1.30766_cross0.85115_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_diagonal_bw41141.45558_skew15.40883_neigh1.42075_cross0.85031_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_diagonal_bw41227.36727_skew15.40883_neigh1.53352_cross0.85087_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_diagonal_bw41364.42425_skew15.40883_neigh1.64744_cross0.85051_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_diagonal_bw41755.58307_skew15.40883_neigh1.76114_cross0.85034_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_diagonal_bw41906.82374_skew15.40883_neigh1.70747_cross0.82402_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_diagonal_bw42415.19562_skew15.40883_neigh1.71119_cross0.76651_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_diagonal_bw42495.96779_skew15.40883_neigh1.7039_cross0.85098_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_diagonal_bw42773.9898_skew15.40883_neigh1.8751_cross0.85514_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_diagonal_bw42815.11517_skew15.40883_neigh1.71375_cross0.7124_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_diagonal_bw43078.55498_skew15.40883_neigh1.71506_cross0.65509_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_diagonal_bw43268.40659_skew15.40883_neigh1.71684_cross0.60215_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_diagonal_bw44501.11614_skew15.40883_neigh1.70412_cross0.85016_n14.mtx","synthetic_63838_63838_14149627_avg221.64897_std96.34508_diagonal_bw46019.35751_skew15.40883_neigh1.70471_cross0.84937_n14.mtx","synthetic_63838_63838_14149682_avg221.64983_std96.37657_diagonal_bw41560.95078_skew15.887_neigh1.70358_cross0.84917_n14.mtx","synthetic_63838_63838_14149757_avg221.65101_std96.43499_diagonal_bw41589.99978_skew16.85239_neigh1.70335_cross0.8529_n14.mtx","synthetic_63838_63838_14149836_avg221.65224_std86.81097_diagonal_bw41852.38988_skew15.4131_neigh1.70445_cross0.8526_n14.mtx","synthetic_63838_63838_14149837_avg221.65226_std96.50454_diagonal_bw41795.62602_skew17.81776_neigh1.7041_cross0.85102_n14.mtx","synthetic_63838_63838_14149882_avg221.65297_std80.46101_diagonal_bw42155.13125_skew15.41756_neigh1.70314_cross0.85556_n14.mtx","synthetic_63838_63838_14149914_avg221.65347_std74.11814_diagonal_bw42395.55904_skew15.42203_neigh1.70343_cross0.855_n14.mtx","synthetic_63838_63838_14150121_avg221.65671_std67.78709_diagonal_bw42629.81854_skew15.4263_neigh1.70372_cross0.85703_n14.mtx","synthetic_63838_63838_14150132_avg221.65688_std96.57304_diagonal_bw41766.83688_skew18.78283_neigh1.70227_cross0.85422_n14.mtx","synthetic_63838_63838_14150259_avg221.65887_std96.64741_diagonal_bw41910.74145_skew19.7481_neigh1.70367_cross0.85371_n14.mtx","synthetic_63838_63838_14621335_avg229.03811_std96.37884_diagonal_bw41539.90302_skew15.40775_neigh1.70411_cross0.85184_n14.mtx","synthetic_63838_63838_15564722_avg243.81594_std96.44368_diagonal_bw41752.45562_skew15.41402_neigh1.70623_cross0.85371_n14.mtx","synthetic_63838_63838_16508065_avg258.59308_std96.5168_diagonal_bw41932.57262_skew15.41575_neigh1.70761_cross0.85252_n14.mtx","synthetic_63838_63838_17451479_avg273.37133_std96.59121_diagonal_bw42040.04983_skew15.41723_neigh1.70991_cross0.85482_n14.mtx","synthetic_63838_63838_18395041_avg288.1519_std96.67009_diagonal_bw42202.0742_skew15.42189_neigh1.71077_cross0.85457_n14.mtx","synthetic_63838_63838_9903973_avg155.14228_std96.11613_diagonal_bw40026.20256_skew15.37207_neigh1.69445_cross0.84169_n14.mtx","synthetic_64247_64247_10039260_avg156.26037_std394.7431_diagonal_bw31668.91797_skew410.15351_neigh1.84448_cross0.94465_n14.mtx","synthetic_65965_65965_14620660_avg221.64269_std96.33127_diagonal_bw42883.51398_skew15.40929_neigh1.70416_cross0.8507_n14.mtx","synthetic_66463_66463_10040261_avg151.06542_std395.7133_diagonal_bw32962.39217_skew424.82214_neigh1.844_cross0.94181_n14.mtx","synthetic_66463_66463_10375238_avg156.10547_std374.16577_diagonal_bw32112.74231_skew297.97094_neigh1.84818_cross0.93876_n14.mtx","synthetic_66463_66463_10377678_avg156.14218_std378.88433_diagonal_bw32368.34001_skew326.18897_neigh1.84976_cross0.93964_n14.mtx","synthetic_66463_66463_10378609_avg156.15619_std376.2538_diagonal_bw33885.40087_skew424.61873_neigh1.84267_cross0.94663_n14.mtx","synthetic_66463_66463_10380253_avg156.18093_std381.55088_diagonal_bw33648.55795_skew424.55132_neigh1.84118_cross0.94627_n14.mtx","synthetic_66463_66463_10380322_avg156.18197_std384.04911_diagonal_bw32719.54181_skew354.38034_neigh1.847_cross0.94116_n14.mtx","synthetic_66463_66463_10381996_avg156.20715_std386.69205_diagonal_bw33367.73913_skew424.47988_neigh1.84_cross0.94697_n14.mtx","synthetic_66463_66463_10382686_avg156.21753_std389.6586_diagonal_bw32926.14501_skew382.5741_neigh1.84812_cross0.94145_n14.mtx","synthetic_66463_66463_10383864_avg156.23526_std391.65048_diagonal_bw33368.29388_skew424.40333_neigh1.84441_cross0.94475_n14.mtx","synthetic_66463_66463_10384998_avg156.25232_std492.58539_diagonal_bw31613.53189_skew424.35688_neigh1.85286_cross0.93515_n14.mtx","synthetic_66463_66463_10385329_avg156.2573_std395.72064_diagonal_bw32889.72072_skew410.73756_neigh1.8454_cross0.94344_n14.mtx","synthetic_66463_66463_10385409_avg156.2585_std471.2363_diagonal_bw32083.54764_skew424.34005_neigh1.85336_cross0.93664_n14.mtx","synthetic_66463_66463_10385572_avg156.26096_std396.41268_diagonal_bw33072.52193_skew424.33337_neigh1.84327_cross0.94513_n14.mtx","synthetic_66463_66463_10385645_avg156.26206_std450.123_diagonal_bw32496.96564_skew424.33038_neigh1.85048_cross0.93744_n14.mtx","synthetic_66463_66463_10386025_avg156.26777_std429.29508_diagonal_bw32700.40045_skew424.31482_neigh1.84744_cross0.94073_n14.mtx","synthetic_66463_66463_10386363_avg156.27286_std408.80831_diagonal_bw32967.75383_skew424.30098_neigh1.84617_cross0.94247_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_diagonal_bw24478.41774_skew424.29959_neigh1.8439_cross0.94323_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_diagonal_bw26438.0019_skew424.29959_neigh1.84659_cross0.94266_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_diagonal_bw28604.35611_skew424.29959_neigh1.8441_cross0.94365_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_diagonal_bw29170.51087_skew424.29959_neigh1.30538_cross0.94141_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_diagonal_bw29263.20915_skew424.29959_neigh1.42161_cross0.9415_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_diagonal_bw29455.90226_skew424.29959_neigh1.54434_cross0.94126_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_diagonal_bw29562.86522_skew424.29959_neigh1.66172_cross0.94313_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_diagonal_bw30146.88541_skew424.29959_neigh1.76709_cross0.97938_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_diagonal_bw30341.85309_skew424.29959_neigh1.8421_cross0.94456_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_diagonal_bw30743.21286_skew424.29959_neigh1.7836_cross0.94242_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_diagonal_bw30957.89412_skew424.29959_neigh1.78881_cross0.97358_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_diagonal_bw32012.39481_skew424.29959_neigh1.84485_cross0.94356_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_diagonal_bw33940.03652_skew424.29959_neigh1.8657_cross0.91479_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_diagonal_bw34285.62295_skew424.29959_neigh1.84502_cross0.94387_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_diagonal_bw34484.39386_skew424.29959_neigh1.88647_cross0.85463_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_diagonal_bw34739.67447_skew424.29959_neigh1.89449_cross0.79398_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_diagonal_bw34819.1538_skew424.29959_neigh1.89891_cross0.73344_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_diagonal_bw34936.4192_skew424.29959_neigh1.90192_cross0.6744_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_diagonal_bw35702.1289_skew424.29959_neigh1.84209_cross0.94568_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_diagonal_bw36237.71714_skew424.29959_neigh1.94624_cross0.95035_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_diagonal_bw37525.80485_skew424.29959_neigh1.84482_cross0.94433_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_diagonal_bw39170.09238_skew424.29959_neigh1.84527_cross0.94403_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_diagonal_bw40665.0048_skew424.29959_neigh1.84538_cross0.94433_n14.mtx","synthetic_66463_66463_10386397_avg156.27337_std398.71424_diagonal_bw41811.7269_skew424.29959_neigh1.92884_cross0.94905_n14.mtx","synthetic_66463_66463_10386663_avg156.27737_std388.73672_diagonal_bw33293.45057_skew424.2887_neigh1.84329_cross0.94546_n14.mtx","synthetic_66463_66463_10387014_avg156.28265_std369.1724_diagonal_bw33710.6116_skew424.27433_neigh1.84189_cross0.94604_n14.mtx","synthetic_66463_66463_10387408_avg156.28858_std350.24164_diagonal_bw33963.66553_skew424.25819_neigh1.84411_cross0.94677_n14.mtx","synthetic_66463_66463_10387855_avg156.29531_std332.11193_diagonal_bw34351.45063_skew424.23989_neigh1.8405_cross0.94886_n14.mtx","synthetic_66463_66463_10388533_avg156.30551_std315.01147_diagonal_bw34714.95685_skew424.21214_neigh1.83978_cross0.94968_n14.mtx","synthetic_66463_66463_10730769_avg161.45478_std396.39605_diagonal_bw33389.55323_skew410.65087_neigh1.84718_cross0.94432_n14.mtx","synthetic_66463_66463_11418942_avg171.80901_std391.59435_diagonal_bw33904.10221_skew385.84235_neigh1.8474_cross0.94666_n14.mtx","synthetic_66463_66463_12107419_avg182.16781_std386.59381_diagonal_bw33078.68366_skew363.84492_neigh1.85114_cross0.94649_n14.mtx","synthetic_66463_66463_12795815_avg192.52539_std381.413_diagonal_bw33483.01449_skew344.2168_neigh1.85524_cross0.94648_n14.mtx","synthetic_66463_66463_13484307_avg202.88442_std376.07335_diagonal_bw32893.08663_skew326.59046_neigh1.85894_cross0.94691_n14.mtx","synthetic_66463_66463_7269367_avg109.37464_std374.11902_diagonal_bw34146.31739_skew424.85738_neigh1.83213_cross0.93119_n14.mtx","synthetic_66463_66463_7962072_avg119.79706_std378.84315_diagonal_bw35198.97952_skew424.84518_neigh1.835_cross0.93592_n14.mtx","synthetic_66463_66463_8654839_avg130.22041_std384.01598_diagonal_bw33135.07615_skew424.83187_neigh1.83541_cross0.94038_n14.mtx","synthetic_66463_66463_9347522_avg140.64249_std389.6368_diagonal_bw33861.58836_skew424.82436_neigh1.84023_cross0.94139_n14.mtx","synthetic_68678_68678_10731088_avg156.25219_std397.34109_diagonal_bw34504.32861_skew424.88203_neigh1.844_cross0.94379_n14.mtx","synthetic_70221_70221_15563153_avg221.63104_std96.30657_diagonal_bw45552.93294_skew15.41016_neigh1.70514_cross0.85192_n14.mtx","synthetic_73109_73109_11420815_avg156.21627_std394.47456_diagonal_bw36296.88892_skew424.97997_neigh1.84425_cross0.94399_n14.mtx","synthetic_74477_74477_16505933_avg221.62457_std96.28344_diagonal_bw48331.68664_skew15.41064_neigh1.70415_cross0.85307_n14.mtx","synthetic_77540_77540_12111057_avg156.19109_std391.93861_diagonal_bw38852.76655_skew425.04863_neigh1.84276_cross0.94411_n14.mtx","synthetic_78733_78733_17449768_avg221.6322_std96.26215_diagonal_bw51104.86271_skew15.41007_neigh1.7037_cross0.8514_n14.mtx","synthetic_81971_81971_12801291_avg156.16854_std389.68003_diagonal_bw40877.20774_skew425.11016_neigh1.84172_cross0.94425_n14.mtx","synthetic_82989_82989_18393238_avg221.63465_std96.24293_diagonal_bw53916.13093_skew15.40989_neigh1.70419_cross0.85143_n14.mtx","synthetic_86401_86401_13491346_avg156.14803_std387.65694_diagonal_bw43065.81676_skew425.16612_neigh1.845_cross0.94312_n14.mtx","synthetic_968035_968035_11863180_avg12.25491_std37.64729_diagonal_bw22702.75661_skew632.62368_neigh1.32267_cross0.78592_n14.mtx","synthetic_1382908_1382908_16923784_avg12.23782_std37.37393_diagonal_bw32417.85806_skew443.68692_neigh1.28774_cross0.78284_n14.mtx","synthetic_1382908_1382908_16927023_avg12.24017_std37.40259_diagonal_bw32489.12672_skew485.75812_neigh1.2937_cross0.78368_n14.mtx","synthetic_1382908_1382908_16931896_avg12.24369_std37.43372_diagonal_bw32563.1309_skew527.76219_neigh1.30041_cross0.78501_n14.mtx","synthetic_1382908_1382908_16938002_avg12.2481_std37.46714_diagonal_bw32505.8507_skew569.78219_neigh1.30908_cross0.78582_n14.mtx","synthetic_1382908_1382908_16943948_avg12.2524_std37.50354_diagonal_bw32473.5113_skew611.69607_neigh1.31734_cross0.7865_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_diagonal_bw16934.48122_skew632.66322_neigh1.72397_cross0.78921_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_diagonal_bw26955.89151_skew632.66322_neigh1.63424_cross0.788_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_diagonal_bw28433.29645_skew632.66322_neigh0.52815_cross0.98565_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_diagonal_bw28482.64659_skew632.66322_neigh1.10244_cross0.78637_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_diagonal_bw28699.51453_skew632.66322_neigh0.92649_cross0.7846_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_diagonal_bw29202.3537_skew632.66322_neigh1.32237_cross0.78681_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_diagonal_bw29676.91177_skew632.66322_neigh1.01331_cross0.78531_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_diagonal_bw30053.31884_skew632.66322_neigh1.19029_cross0.78701_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_diagonal_bw30933.71381_skew632.66322_neigh1.36619_cross0.78741_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_diagonal_bw31363.81879_skew632.66322_neigh1.32155_cross0.78695_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_diagonal_bw31759.41838_skew632.66322_neigh1.36147_cross0.55271_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_diagonal_bw31850.6332_skew632.66322_neigh1.35679_cross0.60476_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_diagonal_bw31986.10955_skew632.66322_neigh1.35083_cross0.65686_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_diagonal_bw32172.68164_skew632.66322_neigh1.34169_cross0.70926_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_diagonal_bw32389.12793_skew632.66322_neigh1.32961_cross0.76125_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_diagonal_bw32518.92977_skew632.66322_neigh1.31159_cross0.81268_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_diagonal_bw32792.31039_skew632.66322_neigh0.99726_cross0.96686_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_diagonal_bw32847.57488_skew632.66322_neigh1.28282_cross0.8647_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_diagonal_bw33206.98578_skew632.66322_neigh1.21973_cross0.91658_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_diagonal_bw33535.67693_skew632.66322_neigh1.32147_cross0.78686_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_diagonal_bw33796.51893_skew632.66322_neigh1.27722_cross0.78689_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_diagonal_bw34964.23666_skew632.66322_neigh1.54388_cross0.78867_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_diagonal_bw35676.67346_skew632.66322_neigh1.32177_cross0.78687_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_diagonal_bw37797.05653_skew632.66322_neigh1.32178_cross0.78697_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_diagonal_bw40050.68177_skew632.66322_neigh1.32136_cross0.78693_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_diagonal_bw41281.79686_skew632.66322_neigh1.45465_cross0.78781_n14.mtx","synthetic_1382908_1382908_16946353_avg12.25414_std37.52234_diagonal_bw42127.99755_skew632.66322_neigh1.32137_cross0.787_n14.mtx","synthetic_1382908_1382908_16948048_avg12.25537_std37.72372_diagonal_bw31935.00008_skew822.06782_neigh1.35574_cross0.78975_n14.mtx","synthetic_1382908_1382908_16948709_avg12.25585_std37.54211_diagonal_bw32403.26495_skew653.62631_neigh1.32622_cross0.78731_n14.mtx","synthetic_1382908_1382908_16951601_avg12.25794_std37.67423_diagonal_bw32052.58183_skew779.80014_neigh1.34903_cross0.78953_n14.mtx","synthetic_1382908_1382908_16951652_avg12.25798_std37.58358_diagonal_bw32324.44939_skew695.60771_neigh1.33364_cross0.78844_n14.mtx","synthetic_1382908_1382908_16952713_avg12.25874_std37.62729_diagonal_bw32145.64971_skew737.65652_neigh1.34221_cross0.78918_n14.mtx","synthetic_187667_187667_12936634_avg68.93398_std101.26087_diagonal_bw21604.74821_skew10.08307_neigh1.12835_cross0.83707_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_diagonal_bw37687.1288_skew10.11157_neigh1.12382_cross0.83773_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_diagonal_bw39604.98722_skew10.11157_neigh1.12032_cross0.83705_n14.mtx","synthetic_268096_268096_18483826_avg68.9448_std96.40124_diagonal_bw30839.36433_skew7.10794_neigh1.1166_cross0.85528_n14.mtx","synthetic_1000005_1000005_2190550_avg2.19054_std25.45314_diagonal_bw114108.9876_skew1501.82644_neigh0.08498_cross0.84657_n14.mtx","synthetic_1000005_1000005_2378888_avg2.37888_std25.47359_diagonal_bw121468.2977_skew1514.4215_neigh0.10196_cross0.883_n14.mtx","synthetic_1000005_1000005_2551748_avg2.55174_std25.499_diagonal_bw114016.12502_skew1534.81764_neigh0.10873_cross0.87763_n14.mtx","synthetic_1000005_1000005_2793946_avg2.79393_std25.52481_diagonal_bw131905.46734_skew1513.71115_neigh0.10444_cross0.86052_n14.mtx","synthetic_1000005_1000005_3068666_avg3.06865_std25.54963_diagonal_bw112236.356_skew1480.43289_neigh0.12104_cross0.88326_n14.mtx","synthetic_1000005_1000005_3110917_avg3.1109_std33.11754_diagonal_bw99822.95915_skew1510.459_neigh0.11796_cross0.85632_n14.mtx","synthetic_1000005_1000005_3116965_avg3.11695_std31.43703_diagonal_bw103495.94606_skew1507.52625_neigh0.11943_cross0.86424_n14.mtx","synthetic_1000005_1000005_3121903_avg3.12189_std25.45302_diagonal_bw106120.25943_skew1053.81063_neigh0.11898_cross0.86904_n14.mtx","synthetic_1000005_1000005_3127972_avg3.12796_std29.75772_diagonal_bw108092.68293_skew1502.5376_neigh0.12362_cross0.87298_n14.mtx","synthetic_1000005_1000005_3132968_avg3.13295_std25.47429_diagonal_bw109696.38688_skew1150.31021_neigh0.12322_cross0.87652_n14.mtx","synthetic_1000005_1000005_3136144_avg3.13613_std18.05255_diagonal_bw131504.88603_skew1498.61976_neigh0.20987_cross0.9344_n14.mtx","synthetic_1000005_1000005_3144021_avg3.14401_std28.07823_diagonal_bw112779.06701_skew1494.86263_neigh0.12564_cross0.88411_n14.mtx","synthetic_1000005_1000005_3144795_avg3.14478_std25.49712_diagonal_bw113241.01438_skew1245.51038_neigh0.12721_cross0.885_n14.mtx","synthetic_1000005_1000005_3156371_avg3.15636_std25.52223_diagonal_bw116391.07562_skew1340.10381_neigh0.13204_cross0.89292_n14.mtx","synthetic_1000005_1000005_3160475_avg3.16046_std19.71515_diagonal_bw131174.03454_skew1487.0749_neigh0.20681_cross0.93372_n14.mtx","synthetic_1000005_1000005_3162276_avg3.16226_std26.40103_diagonal_bw118193.42178_skew1486.2274_neigh0.13377_cross0.89745_n14.mtx","synthetic_1000005_1000005_3166512_avg3.1665_std25.54915_diagonal_bw119317.10886_skew1434.65625_neigh0.13844_cross0.90081_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_diagonal_bw100875.1064_skew1482.18441_neigh0.14415_cross0.90421_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_diagonal_bw108834.1731_skew1482.18441_neigh0.14336_cross0.90436_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_diagonal_bw116472.78808_skew1482.18441_neigh0.14374_cross0.90463_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_diagonal_bw117096.97406_skew1482.18441_neigh0.06224_cross0.96168_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_diagonal_bw117677.40758_skew1482.18441_neigh0.18675_cross0.90502_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_diagonal_bw118297.28426_skew1482.18441_neigh0.17603_cross0.90478_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_diagonal_bw118732.2225_skew1482.18441_neigh0.11799_cross0.93285_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_diagonal_bw118876.87938_skew1482.18441_neigh0.164_cross0.9047_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_diagonal_bw119416.85487_skew1482.18441_neigh0.1575_cross0.90469_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_diagonal_bw119981.2095_skew1482.18441_neigh0.14756_cross0.90455_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_diagonal_bw120655.04241_skew1482.18441_neigh0.13912_cross0.90438_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_diagonal_bw121157.45525_skew1482.18441_neigh0.12935_cross0.90421_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_diagonal_bw121505.44901_skew1482.18441_neigh0.15658_cross0.87562_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_diagonal_bw121800.16162_skew1482.18441_neigh0.1198_cross0.90401_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_diagonal_bw122167.3375_skew1482.18441_neigh0.10989_cross0.90394_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_diagonal_bw122547.34699_skew1482.18441_neigh0.1007_cross0.90373_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_diagonal_bw122970.89533_skew1482.18441_neigh0.17404_cross0.8183_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_diagonal_bw123852.15593_skew1482.18441_neigh0.18429_cross0.76047_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_diagonal_bw124316.39867_skew1482.18441_neigh0.14315_cross0.90464_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_diagonal_bw124333.68929_skew1482.18441_neigh0.19183_cross0.70272_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_diagonal_bw124629.28556_skew1482.18441_neigh0.19573_cross0.64408_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_diagonal_bw132643.35795_skew1482.18441_neigh0.14112_cross0.90484_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_diagonal_bw139742.14787_skew1482.18441_neigh0.14113_cross0.9048_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_diagonal_bw147748.47168_skew1482.18441_neigh0.14113_cross0.90477_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_diagonal_bw154767.97818_skew1482.18441_neigh0.14355_cross0.90514_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_diagonal_bw85508.89004_skew1482.18441_neigh0.14435_cross0.90404_n14.mtx","synthetic_1000005_1000005_3170896_avg3.17088_std25.56324_diagonal_bw92882.01052_skew1482.18441_neigh0.14352_cross0.90404_n14.mtx","synthetic_1000005_1000005_3174621_avg3.17461_std25.57736_diagonal_bw121783.7238_skew1529.58406_neigh0.14525_cross0.90761_n14.mtx","synthetic_1000005_1000005_3177914_avg3.1779_std24.72495_diagonal_bw122735.04959_skew1478.909_neigh0.15049_cross0.91097_n14.mtx","synthetic_1000005_1000005_3179023_avg3.17901_std21.38191_diagonal_bw130097.76418_skew1478.39273_neigh0.19486_cross0.93033_n14.mtx","synthetic_1000005_1000005_3180578_avg3.18056_std25.60874_diagonal_bw123725.69417_skew1625.12766_neigh0.15445_cross0.91356_n14.mtx","synthetic_1000005_1000005_3184160_avg3.18414_std25.64113_diagonal_bw125738.34862_skew1721.91199_neigh0.16506_cross0.91841_n14.mtx","synthetic_1000005_1000005_3185020_avg3.185_std23.052_diagonal_bw127168.47608_skew1475.60722_neigh0.17295_cross0.92261_n14.mtx","synthetic_1000005_1000005_3185310_avg3.18529_std25.71304_diagonal_bw128318.0595_skew1917.81812_neigh0.18146_cross0.92577_n14.mtx","synthetic_1000005_1000005_3185554_avg3.18554_std25.67645_diagonal_bw127162.91658_skew1819.4146_neigh0.17343_cross0.92255_n14.mtx","synthetic_1000005_1000005_3238430_avg3.23841_std25.57761_diagonal_bw124163.77369_skew1499.42591_neigh0.16758_cross0.91813_n14.mtx","synthetic_1000005_1000005_3338044_avg3.33803_std25.60993_diagonal_bw115730.26217_skew1548.7177_neigh0.16947_cross0.91526_n14.mtx","synthetic_1000005_1000005_3554394_avg3.55438_std25.64567_diagonal_bw109077.20175_skew1542.73078_neigh0.13309_cross0.88625_n14.mtx","synthetic_1000005_1000005_3911756_avg3.91174_std25.67945_diagonal_bw128127.29214_skew1481.71748_neigh0.14126_cross0.8869_n14.mtx","synthetic_1000005_1000005_4154204_avg4.15418_std25.71363_diagonal_bw152102.90185_skew1470.76946_neigh0.19404_cross0.9225_n14.mtx","synthetic_1006441_1006441_22414673_avg22.27122_std29.74727_diagonal_bw241757.44752_skew313.62123_neigh1.25654_cross0.76073_n14.mtx","synthetic_100993_100993_2185846_avg21.64354_std13.77891_diagonal_bw47261.74273_skew4.03614_neigh1.00741_cross0.57221_n14.mtx","synthetic_1019593_1019593_74066178_avg72.64289_std15.80935_diagonal_bw15214.38873_skew1.03736_neigh1.69545_cross0.82041_n14.mtx","synthetic_102778_102778_7408141_avg72.07905_std19.01282_diagonal_bw5790.43171_skew1.19204_neigh1.65271_cross0.77107_n14.mtx","synthetic_1033338_1033338_3275830_avg3.17014_std25.55637_diagonal_bw124512.44793_skew1482.52894_neigh0.14371_cross0.90431_n14.mtx","synthetic_1047423_1047423_51169710_avg48.85296_std11.94333_diagonal_bw179790.95101_skew1.10837_neigh1.69181_cross0.80579_n14.mtx","synthetic_1063952_1063952_23695040_avg22.27078_std29.72463_diagonal_bw255536.35333_skew313.62752_neigh1.25753_cross0.76084_n14.mtx","synthetic_108003_108003_5987303_avg55.43645_std11.04356_diagonal_bw4191.65781_skew0.89406_neigh1.63079_cross0.7877_n14.mtx","synthetic_108334_108334_7808284_avg72.07602_std19.0204_diagonal_bw6122.43647_skew1.19213_neigh1.65216_cross0.76954_n14.mtx","synthetic_1085373_1085373_78800672_avg72.60239_std15.80629_diagonal_bw16200.45996_skew1.02473_neigh1.69539_cross0.82034_n14.mtx","synthetic_109072_109072_2360564_avg21.64225_std13.77936_diagonal_bw51027.24323_skew4.08265_neigh1.00811_cross0.57094_n14.mtx","synthetic_1100005_1100005_3487516_avg3.17045_std25.54298_diagonal_bw132402.99662_skew1482.38345_neigh0.14166_cross0.90462_n14.mtx","synthetic_1110903_1110903_54291613_avg48.87161_std11.93583_diagonal_bw190510.69059_skew1.10756_neigh1.69155_cross0.80595_n14.mtx","synthetic_1121463_1121463_24976074_avg22.27097_std29.7046_diagonal_bw269393.05866_skew313.62476_neigh1.2569_cross0.76099_n14.mtx","synthetic_1151153_1151153_83612943_avg72.63408_std15.81129_diagonal_bw17174.81048_skew1.01008_neigh1.69486_cross0.82103_n14.mtx","synthetic_1166672_1166672_3699263_avg3.17078_std25.53274_diagonal_bw140872.67348_skew1482.23015_neigh0.13992_cross0.90484_n14.mtx","synthetic_117152_117152_2535420_avg21.64214_std13.77711_diagonal_bw54858.54236_skew4.08268_neigh1.00527_cross0.57159_n14.mtx","synthetic_117394_117394_6507908_avg55.43646_std11.04171_diagonal_bw4548.03296_skew0.89406_neigh1.63141_cross0.78802_n14.mtx","synthetic_1174383_1174383_57377479_avg48.85755_std11.94355_diagonal_bw201337.3527_skew1.10817_neigh1.69149_cross0.80605_n14.mtx","synthetic_119698_119698_670830_avg5.60435_std4.47718_diagonal_bw35851.2119_skew62.8789_neigh0.59319_cross0.62674_n14.mtx","synthetic_121192_121192_1836052_avg15.14994_std12.36381_diagonal_bw54781.70924_skew3.8185_neigh0.97072_cross0.58007_n14.mtx","synthetic_121192_121192_2010928_avg16.59291_std12.82631_diagonal_bw54606.41729_skew3.94187_neigh0.98248_cross0.57348_n14.mtx","synthetic_121192_121192_2185768_avg18.03558_std13.23353_diagonal_bw55828.6145_skew4.04558_neigh0.98987_cross0.56777_n14.mtx","synthetic_121192_121192_2360629_avg19.47842_std13.60481_diagonal_bw55574.55169_skew4.13389_neigh0.99709_cross0.56533_n14.mtx","synthetic_121192_121192_2535413_avg20.92063_std13.77469_diagonal_bw56884.2582_skew4.11457_neigh1.00373_cross0.57077_n14.mtx","synthetic_121192_121192_2622629_avg21.64028_std13.77138_diagonal_bw56242.87207_skew3.85206_neigh1.00809_cross0.56863_n14.mtx","synthetic_121192_121192_2622663_avg21.64056_std13.63311_diagonal_bw55697.66942_skew3.52853_neigh1.00738_cross0.56431_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_diagonal_bw41449.60518_skew4.08299_neigh1.00709_cross0.56924_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_diagonal_bw44969.04646_skew4.08299_neigh1.00918_cross0.5684_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_diagonal_bw48375.95076_skew4.08299_neigh1.0066_cross0.57186_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_diagonal_bw51808.58859_skew4.08299_neigh1.00555_cross0.57153_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_diagonal_bw52786.48945_skew4.08299_neigh0.98178_cross0.73314_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_diagonal_bw53882.38648_skew4.08299_neigh0.98833_cross0.69862_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_diagonal_bw54816.63871_skew4.08299_neigh0.99503_cross0.66136_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_diagonal_bw55046.60667_skew4.08299_neigh1.00618_cross0.57223_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_diagonal_bw55588.08012_skew4.08299_neigh1.00197_cross0.62866_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_diagonal_bw56289.99224_skew4.08299_neigh0.77156_cross0.57111_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_diagonal_bw56335.34997_skew4.08299_neigh1.00541_cross0.59144_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_diagonal_bw56445.73291_skew4.08299_neigh0.83909_cross0.57135_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_diagonal_bw56550.54337_skew4.08299_neigh0.70497_cross0.57228_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_diagonal_bw56640.70898_skew4.08299_neigh1.10568_cross0.57621_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_diagonal_bw56646.25647_skew4.08299_neigh0.90696_cross0.57222_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_diagonal_bw56981.50968_skew4.08299_neigh1.00923_cross0.55476_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_diagonal_bw57143.44331_skew4.08299_neigh1.1732_cross0.57642_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_diagonal_bw57206.83955_skew4.08299_neigh0.97383_cross0.57171_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_diagonal_bw57392.3198_skew4.08299_neigh1.03836_cross0.57342_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_diagonal_bw57680.27683_skew4.08299_neigh1.01286_cross0.52046_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_diagonal_bw57736.54601_skew4.08299_neigh1.2401_cross0.57851_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_diagonal_bw58118.42279_skew4.08299_neigh1.01366_cross0.48629_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_diagonal_bw58305.84042_skew4.08299_neigh1.00621_cross0.5718_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_diagonal_bw58608.29432_skew4.08299_neigh1.01533_cross0.452_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_diagonal_bw59091.75387_skew4.08299_neigh1.01816_cross0.41458_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_diagonal_bw59170.98197_skew4.08299_neigh1.30576_cross0.57913_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_diagonal_bw61351.56958_skew4.08299_neigh1.00459_cross0.57351_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_diagonal_bw64498.77718_skew4.08299_neigh1.00458_cross0.57593_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_diagonal_bw67553.05734_skew4.08299_neigh1.00396_cross0.57603_n14.mtx","synthetic_121192_121192_2622694_avg21.64082_std13.77751_diagonal_bw70322.56861_skew4.08299_neigh1.00409_cross0.57935_n14.mtx","synthetic_121192_121192_2622724_avg21.64107_std13.77599_diagonal_bw56586.74825_skew3.99051_neigh1.00718_cross0.57135_n14.mtx","synthetic_121192_121192_2622736_avg21.64116_std14.23425_diagonal_bw56154.62111_skew4.08291_neigh1.00762_cross0.56767_n14.mtx","synthetic_121192_121192_2622750_avg21.64128_std13.768_diagonal_bw55889.48449_skew3.71321_neigh1.00743_cross0.56293_n14.mtx","synthetic_121192_121192_2622777_avg21.6415_std15.59481_diagonal_bw55368.29813_skew4.08283_neigh1.00631_cross0.56573_n14.mtx","synthetic_121192_121192_2622792_avg21.64163_std13.77856_diagonal_bw56873.57123_skew4.129_neigh1.0086_cross0.57384_n14.mtx","synthetic_121192_121192_2622812_avg21.64179_std16.1555_diagonal_bw55349.84176_skew3.99034_neigh1.00627_cross0.56748_n14.mtx","synthetic_121192_121192_2622828_avg21.64192_std13.43338_diagonal_bw55856.38154_skew3.29722_neigh1.00759_cross0.56535_n14.mtx","synthetic_121192_121192_2622837_avg21.642_std15.01875_diagonal_bw55585.0516_skew4.12892_neigh1.00673_cross0.56331_n14.mtx","synthetic_121192_121192_2622876_avg21.64232_std16.69445_diagonal_bw55240.42113_skew3.94402_neigh1.00765_cross0.5674_n14.mtx","synthetic_121192_121192_2622972_avg21.64311_std13.78148_diagonal_bw57067.76991_skew4.31347_neigh1.00707_cross0.57783_n14.mtx","synthetic_121192_121192_2623047_avg21.64373_std13.32114_diagonal_bw57121.20154_skew4.0361_neigh1.00729_cross0.57944_n14.mtx","synthetic_121192_121192_2623109_avg21.64424_std13.78493_diagonal_bw57338.33986_skew4.4518_neigh1.00545_cross0.58243_n14.mtx","synthetic_121192_121192_2623156_avg21.64463_std11.48856_diagonal_bw58934.13194_skew3.89729_neigh1.00538_cross0.59657_n14.mtx","synthetic_121192_121192_2623189_avg21.6449_std13.78718_diagonal_bw57991.06353_skew4.77503_neigh1.00635_cross0.58424_n14.mtx","synthetic_121192_121192_2623197_avg21.64497_std13.78635_diagonal_bw57626.49167_skew4.59021_neigh1.00568_cross0.58329_n14.mtx","synthetic_121192_121192_2623249_avg21.6454_std12.40537_diagonal_bw57996.48538_skew3.98951_neigh1.0059_cross0.58819_n14.mtx","synthetic_121192_121192_2623583_avg21.64815_std9.65791_diagonal_bw60405.36362_skew3.8503_neigh1.00451_cross0.61091_n14.mtx","synthetic_121192_121192_2623613_avg21.6484_std10.57459_diagonal_bw59660.52937_skew3.85024_neigh1.00452_cross0.60326_n14.mtx","synthetic_121192_121192_2710314_avg22.3638_std13.78076_diagonal_bw57516.41338_skew4.05281_neigh1.00876_cross0.57867_n14.mtx","synthetic_121192_121192_2885490_avg23.80924_std13.78209_diagonal_bw57922.69442_skew3.99806_neigh1.01487_cross0.58481_n14.mtx","synthetic_121192_121192_3060321_avg25.25184_std13.78548_diagonal_bw58954.02529_skew3.95013_neigh1.0179_cross0.59132_n14.mtx","synthetic_121192_121192_3235599_avg26.69812_std13.78831_diagonal_bw59260.45423_skew3.86926_neigh1.02285_cross0.59628_n14.mtx","synthetic_121192_121192_3410529_avg28.14154_std13.78985_diagonal_bw59475.58642_skew3.86825_neigh1.02443_cross0.59913_n14.mtx","synthetic_1216933_1216933_88363297_avg72.61147_std15.79644_diagonal_bw18153.04703_skew1.03825_neigh1.69455_cross0.82106_n14.mtx","synthetic_1233339_1233339_3909668_avg3.16999_std25.52165_diagonal_bw148870.18881_skew1482.60253_neigh0.14197_cross0.90505_n14.mtx","synthetic_1237863_1237863_60496296_avg48.87156_std11.94811_diagonal_bw211983.92253_skew1.20987_neigh1.69135_cross0.80663_n14.mtx","synthetic_125231_125231_2710355_avg21.64284_std13.77742_diagonal_bw58546.37263_skew4.08251_neigh1.00665_cross0.57313_n14.mtx","synthetic_126786_126786_7028939_avg55.43939_std11.04404_diagonal_bw4906.49056_skew0.89396_neigh1.63134_cross0.78947_n14.mtx","synthetic_1282713_1282713_93191153_avg72.6516_std15.81312_diagonal_bw19135.30286_skew1.10594_neigh1.69446_cross0.821_n14.mtx","synthetic_129947_129947_10504776_avg80.83893_std117.70867_diagonal_bw16837.76377_skew7.85712_neigh1.21519_cross0.8459_n14.mtx","synthetic_1300006_1300006_4120758_avg3.1698_std25.51264_diagonal_bw157091.52358_skew1482.69019_neigh0.13981_cross0.90518_n14.mtx","synthetic_131098_131098_734679_avg5.60404_std4.47302_diagonal_bw39029.11606_skew62.88244_neigh0.59297_cross0.62541_n14.mtx","synthetic_133311_133311_2884795_avg21.63959_std13.78017_diagonal_bw62354.80178_skew4.08328_neigh1.00679_cross0.57368_n14.mtx","synthetic_136178_136178_7548659_avg55.43229_std11.05483_diagonal_bw5281.69958_skew0.91224_neigh1.63102_cross0.78738_n14.mtx","synthetic_140874_140874_10153211_avg72.07299_std11.05163_diagonal_bw5512.27589_skew0.7066_neigh1.64608_cross0.80473_n14.mtx","synthetic_140874_140874_5465261_avg38.79538_std11.0515_diagonal_bw5412.9335_skew1.31986_neigh1.60389_cross0.75869_n14.mtx","synthetic_140874_140874_5986164_avg42.49304_std11.0536_diagonal_bw5548.31213_skew1.18859_neigh1.61095_cross0.76635_n14.mtx","synthetic_140874_140874_6506839_avg46.18907_std11.05195_diagonal_bw5412.39753_skew1.10006_neigh1.61828_cross0.77328_n14.mtx","synthetic_140874_140874_7027827_avg49.88732_std11.05353_diagonal_bw5512.6976_skew1.02456_neigh1.62363_cross0.78108_n14.mtx","synthetic_140874_140874_7548897_avg53.58616_std11.05321_diagonal_bw5415.30582_skew0.9408_neigh1.62912_cross0.7833_n14.mtx","synthetic_140874_140874_7807971_avg55.42521_std14.36638_diagonal_bw5335.16197_skew1.18312_neigh1.63649_cross0.76697_n14.mtx","synthetic_140874_140874_7808175_avg55.42666_std13.63005_diagonal_bw5359.29341_skew1.12894_neigh1.63441_cross0.77184_n14.mtx","synthetic_140874_140874_7808374_avg55.42807_std12.89549_diagonal_bw5396.31618_skew1.07476_neigh1.63344_cross0.77504_n14.mtx","synthetic_140874_140874_7808713_avg55.43048_std12.15675_diagonal_bw5408.74805_skew1.00251_neigh1.63324_cross0.78138_n14.mtx","synthetic_140874_140874_7809044_avg55.43283_std11.42187_diagonal_bw5439.57739_skew0.9483_neigh1.6324_cross0.78435_n14.mtx","synthetic_140874_140874_7809186_avg55.43383_std10.68354_diagonal_bw5480.44304_skew0.89415_neigh1.62983_cross0.78974_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_diagonal_bw3838.69228_skew0.91217_neigh1.63011_cross0.78907_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_diagonal_bw4204.41167_skew0.91217_neigh1.62943_cross0.78923_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_diagonal_bw4558.67931_skew0.91217_neigh1.63021_cross0.78941_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_diagonal_bw4913.35727_skew0.91217_neigh1.63146_cross0.78927_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_diagonal_bw5282.63594_skew0.91217_neigh1.63072_cross0.78718_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_diagonal_bw5457.86719_skew0.91217_neigh1.63077_cross0.78851_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_diagonal_bw5648.31754_skew0.91217_neigh1.62967_cross0.78702_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_diagonal_bw6002.23969_skew0.91217_neigh1.63068_cross0.78728_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_diagonal_bw6356.58993_skew0.91217_neigh1.62903_cross0.78853_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_diagonal_bw6710.26111_skew0.91217_neigh1.62896_cross0.78828_n14.mtx","synthetic_140874_140874_7809286_avg55.43454_std11.05282_diagonal_bw7054.0657_skew0.91217_neigh1.63058_cross0.78783_n14.mtx","synthetic_140874_140874_7809480_avg55.43592_std9.94668_diagonal_bw5490.06378_skew0.82192_neigh1.63009_cross0.79615_n14.mtx","synthetic_140874_140874_7809970_avg55.4394_std9.2113_diagonal_bw5566.8335_skew0.7677_neigh1.62757_cross0.7999_n14.mtx","synthetic_140874_140874_7810161_avg55.44076_std8.47606_diagonal_bw5572.74971_skew0.6955_neigh1.62616_cross0.80687_n14.mtx","synthetic_140874_140874_7810358_avg55.44215_std7.74028_diagonal_bw5620.76373_skew0.64135_neigh1.62766_cross0.81104_n14.mtx","synthetic_140874_140874_8069568_avg57.28217_std11.05207_diagonal_bw5516.71932_skew0.8854_neigh1.63387_cross0.78936_n14.mtx","synthetic_140874_140874_8590481_avg60.97989_std11.05315_diagonal_bw5437.17752_skew0.83667_neigh1.63703_cross0.79384_n14.mtx","synthetic_140874_140874_9111507_avg64.67841_std11.05222_diagonal_bw5508.99952_skew0.77803_neigh1.6398_cross0.79875_n14.mtx","synthetic_140874_140874_9632262_avg68.37502_std11.0533_diagonal_bw5447.32524_skew0.7404_neigh1.64497_cross0.80063_n14.mtx","synthetic_141390_141390_3059797_avg21.64083_std13.77882_diagonal_bw66168.99875_skew4.08298_neigh1.00521_cross0.57194_n14.mtx","synthetic_142323_142323_11504673_avg80.83495_std117.71151_diagonal_bw18462.18834_skew7.85755_neigh1.21247_cross0.84559_n14.mtx","synthetic_142498_142498_798541_avg5.60388_std4.46852_diagonal_bw42535.5183_skew62.88436_neigh0.59531_cross0.62546_n14.mtx","synthetic_144550_144550_890999_avg6.16395_std4.44624_diagonal_bw249.77002_skew7.27392_neigh0.18253_cross0.33392_n14.mtx","synthetic_145569_145569_8068734_avg55.42893_std11.05658_diagonal_bw5649.76329_skew0.91236_neigh1.62963_cross0.7868_n14.mtx","synthetic_14839_14839_1040332_avg70.10796_std6.31579_diagonal_bw1132.91226_skew0.34079_neigh1.79083_cross0.91451_n14.mtx","synthetic_149470_149470_3234141_avg21.63739_std13.78086_diagonal_bw69927.19965_skew4.08379_neigh1.00664_cross0.57176_n14.mtx","synthetic_153898_153898_862342_avg5.60333_std4.46589_diagonal_bw45848.56757_skew62.89053_neigh0.59749_cross0.62567_n14.mtx","synthetic_154699_154699_12503850_avg80.82696_std117.71492_diagonal_bw20044.28137_skew7.85843_neigh1.21606_cross0.84532_n14.mtx","synthetic_154961_154961_8588351_avg55.42266_std11.06218_diagonal_bw6007.51607_skew0.91258_neigh1.63019_cross0.78673_n14.mtx","synthetic_157549_157549_3408821_avg21.63658_std13.78244_diagonal_bw73653.17583_skew4.08398_neigh1.00489_cross0.5734_n14.mtx","synthetic_158316_158316_975794_avg6.16358_std4.44586_diagonal_bw273.64732_skew7.27441_neigh0.1798_cross0.33007_n14.mtx","synthetic_16253_16253_1139653_avg70.11955_std6.30855_diagonal_bw1264.2658_skew0.34057_neigh1.78351_cross0.91717_n14.mtx","synthetic_164353_164353_9109408_avg55.42587_std11.0688_diagonal_bw6365.94759_skew0.91246_neigh1.6294_cross0.78797_n14.mtx","synthetic_165298_165298_926272_avg5.60365_std4.4624_diagonal_bw49243.13259_skew62.88694_neigh0.59497_cross0.62573_n14.mtx","synthetic_167075_167075_13504757_avg80.83051_std117.71597_diagonal_bw21622.74045_skew7.87041_neigh1.21592_cross0.84505_n14.mtx","synthetic_170998_170998_1008507_avg5.89777_std3.14995_diagonal_bw45171.22177_skew59.87046_neigh0.62353_cross0.6314_n14.mtx","synthetic_170998_170998_1037773_avg6.06892_std4.43265_diagonal_bw45901.38609_skew59.96638_neigh0.64574_cross0.63979_n14.mtx","synthetic_170998_170998_1043193_avg6.10062_std4.44271_diagonal_bw46214.72752_skew63.58365_neigh0.6538_cross0.64274_n14.mtx","synthetic_170998_170998_1092151_avg6.38692_std4.47337_diagonal_bw46053.98994_skew64.44623_neigh0.62763_cross0.62819_n14.mtx","synthetic_170998_170998_1211686_avg7.08597_std4.4649_diagonal_bw49555.83123_skew64.76379_neigh0.67173_cross0.6434_n14.mtx","synthetic_170998_170998_1212281_avg7.08945_std4.45045_diagonal_bw49559.75803_skew61.3462_neigh0.67177_cross0.64357_n14.mtx","synthetic_170998_170998_703429_avg4.11367_std4.4083_diagonal_bw62525.67356_skew60.01611_neigh0.55916_cross0.6516_n14.mtx","synthetic_170998_170998_709969_avg4.15191_std4.42013_diagonal_bw62829.82587_skew65.23451_neigh0.57062_cross0.65616_n14.mtx","synthetic_170998_170998_813246_avg4.75588_std4.44226_diagonal_bw46482.54362_skew61.86954_neigh0.56085_cross0.62458_n14.mtx","synthetic_170998_170998_875832_avg5.12188_std4.42226_diagonal_bw49819.5485_skew62.06273_neigh0.62237_cross0.64457_n14.mtx","synthetic_170998_170998_882159_avg5.15888_std4.44207_diagonal_bw49851.31179_skew66.06876_neigh0.61393_cross0.64338_n14.mtx","synthetic_170998_170998_944721_avg5.52475_std5.76053_diagonal_bw50240.49201_skew63.79933_neigh0.59642_cross0.62742_n14.mtx","synthetic_170998_170998_945632_avg5.53008_std5.47116_diagonal_bw50240.27614_skew63.7369_neigh0.5981_cross0.62739_n14.mtx","synthetic_170998_170998_946661_avg5.53609_std4.44002_diagonal_bw50440.90686_skew44.88072_neigh0.59713_cross0.62583_n14.mtx","synthetic_170998_170998_947518_avg5.54111_std5.18255_diagonal_bw50420.37847_skew63.60804_neigh0.59685_cross0.62681_n14.mtx","synthetic_170998_170998_948659_avg5.54778_std4.44165_diagonal_bw50506.82008_skew48.9299_neigh0.59655_cross0.62663_n14.mtx","synthetic_170998_170998_950710_avg5.55977_std4.8941_diagonal_bw50483.86054_skew63.39112_neigh0.59518_cross0.62663_n14.mtx","synthetic_170998_170998_951053_avg5.56178_std4.44852_diagonal_bw50556.26173_skew53.11938_neigh0.59346_cross0.62678_n14.mtx","synthetic_170998_170998_953667_avg5.57707_std4.45309_diagonal_bw50785.42323_skew57.09507_neigh0.59646_cross0.62658_n14.mtx","synthetic_170998_170998_955297_avg5.5866_std4.60347_diagonal_bw50921.85081_skew63.08194_neigh0.596_cross0.6275_n14.mtx","synthetic_170998_170998_956707_avg5.59484_std4.45769_diagonal_bw51000.98471_skew61.0214_neigh0.59632_cross0.62507_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_diagonal_bw36461.87693_skew62.88573_neigh0.59695_cross0.61997_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_diagonal_bw39610.08149_skew62.88573_neigh0.59906_cross0.62096_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_diagonal_bw43105.47571_skew62.88573_neigh0.59816_cross0.62237_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_diagonal_bw44031.98084_skew62.88573_neigh0.5372_cross0.62197_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_diagonal_bw44847.35264_skew62.88573_neigh0.49711_cross0.62325_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_diagonal_bw45384.2646_skew62.88573_neigh0.4588_cross0.62138_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_diagonal_bw45971.97334_skew62.88573_neigh0.41681_cross0.61967_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_diagonal_bw46159.87112_skew62.88573_neigh0.59674_cross0.62414_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_diagonal_bw46483.06345_skew62.88573_neigh0.77274_cross0.62785_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_diagonal_bw47421.7408_skew62.88573_neigh0.73666_cross0.62676_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_diagonal_bw48584.83352_skew62.88573_neigh0.69732_cross0.62642_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_diagonal_bw49003.35613_skew62.88573_neigh0.54697_cross0.79644_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_diagonal_bw49449.25941_skew62.88573_neigh0.59289_cross0.62664_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_diagonal_bw49562.93434_skew62.88573_neigh0.65726_cross0.6251_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_diagonal_bw49598.09636_skew62.88573_neigh0.56075_cross0.75943_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_diagonal_bw50162.74852_skew62.88573_neigh0.57537_cross0.72046_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_diagonal_bw50491.14134_skew62.88573_neigh0.58432_cross0.6826_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_diagonal_bw50571.15461_skew62.88573_neigh0.61583_cross0.62382_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_diagonal_bw50893.01638_skew62.88573_neigh0.5929_cross0.64621_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_diagonal_bw51129.15176_skew62.88573_neigh0.59642_cross0.60766_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_diagonal_bw51234.36906_skew62.88573_neigh0.60413_cross0.5687_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_diagonal_bw51395.69692_skew62.88573_neigh0.61075_cross0.52993_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_diagonal_bw51420.69302_skew62.88573_neigh0.61218_cross0.49298_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_diagonal_bw51535.92532_skew62.88573_neigh0.57482_cross0.62495_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_diagonal_bw51655.22982_skew62.88573_neigh0.61451_cross0.45451_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_diagonal_bw52368.46746_skew62.88573_neigh0.59522_cross0.62658_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_diagonal_bw55725.08564_skew62.88573_neigh0.59126_cross0.62765_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_diagonal_bw58848.50641_skew62.88573_neigh0.5924_cross0.62814_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_diagonal_bw61752.83746_skew62.88573_neigh0.59197_cross0.62973_n14.mtx","synthetic_170998_170998_958231_avg5.60376_std4.45931_diagonal_bw64753.49829_skew62.88573_neigh0.59013_cross0.63179_n14.mtx","synthetic_170998_170998_959835_avg5.61314_std4.46266_diagonal_bw51077.39694_skew64.91681_neigh0.59508_cross0.62613_n14.mtx","synthetic_170998_170998_961623_avg5.62359_std4.31537_diagonal_bw51156.06217_skew62.66038_neigh0.59268_cross0.62607_n14.mtx","synthetic_170998_170998_963061_avg5.632_std4.4671_diagonal_bw51143.10828_skew68.77981_neigh0.59568_cross0.62558_n14.mtx","synthetic_170998_170998_966597_avg5.65268_std4.47351_diagonal_bw51344.14215_skew72.59341_neigh0.59618_cross0.6253_n14.mtx","synthetic_170998_170998_970106_avg5.6732_std4.48093_diagonal_bw51465.29233_skew76.55763_neigh0.59896_cross0.62591_n14.mtx","synthetic_170998_170998_970315_avg5.67442_std4.02645_diagonal_bw51483.87979_skew62.09011_neigh0.59847_cross0.62615_n14.mtx","synthetic_170998_170998_973557_avg5.69338_std4.48437_diagonal_bw51658.71521_skew80.32248_neigh0.59938_cross0.6276_n14.mtx","synthetic_170998_170998_981215_avg5.73817_std3.73499_diagonal_bw52085.53194_skew61.38927_neigh0.60333_cross0.62711_n14.mtx","synthetic_170998_170998_994346_avg5.81496_std3.44543_diagonal_bw52702.41379_skew60.73734_neigh0.6117_cross0.63171_n14.mtx","synthetic_172083_172083_1060867_avg6.16486_std4.44424_diagonal_bw297.53188_skew7.2727_neigh0.17686_cross0.32932_n14.mtx","synthetic_173744_173744_9631043_avg55.43238_std11.0756_diagonal_bw6722.25909_skew0.91224_neigh1.62924_cross0.7879_n14.mtx","synthetic_17666_17666_1238955_avg70.13217_std6.31199_diagonal_bw1367.70486_skew0.34033_neigh1.78108_cross0.91647_n14.mtx","synthetic_176697_176697_990167_avg5.60376_std4.45819_diagonal_bw52568.63684_skew62.88571_neigh0.59492_cross0.62595_n14.mtx","synthetic_179451_179451_14506771_avg80.83973_std117.71232_diagonal_bw23249.97453_skew7.8694_neigh1.21265_cross0.84464_n14.mtx","synthetic_183136_183136_10152501_avg55.43695_std11.07295_diagonal_bw7090.3351_skew0.91208_neigh1.631_cross0.78675_n14.mtx","synthetic_185639_185639_10500260_avg56.56279_std99.5812_diagonal_bw18098.88818_skew7.18559_neigh1.25585_cross0.95445_n14.mtx","synthetic_185639_185639_11501046_avg61.95382_std109.06293_diagonal_bw18425.33593_skew7.19966_neigh1.25674_cross0.95406_n14.mtx","synthetic_185639_185639_12507698_avg67.37646_std112.75287_diagonal_bw23036.91435_skew7.4154_neigh1.18654_cross0.88126_n14.mtx","synthetic_185639_185639_13507262_avg72.76091_std115.00904_diagonal_bw23636.41108_skew7.61726_neigh1.19914_cross0.86016_n14.mtx","synthetic_185639_185639_14507140_avg78.14705_std116.8824_diagonal_bw23970.8135_skew7.80392_neigh1.21053_cross0.84821_n14.mtx","synthetic_185639_185639_15003294_avg80.81973_std87.6573_diagonal_bw24360.81189_skew8.39127_neigh1.21918_cross0.82425_n14.mtx","synthetic_185639_185639_15003763_avg80.82226_std142.24862_diagonal_bw19300.06598_skew7.19081_neigh1.26239_cross0.95424_n14.mtx","synthetic_185639_185639_15004156_avg80.82437_std94.60907_diagonal_bw24270.69581_skew8.30412_neigh1.21969_cross0.82786_n14.mtx","synthetic_185639_185639_15004647_avg80.82702_std101.43013_diagonal_bw24276.21957_skew8.20484_neigh1.21772_cross0.83109_n14.mtx","synthetic_185639_185639_15005689_avg80.83263_std108.09033_diagonal_bw24209.38904_skew8.08049_neigh1.21736_cross0.83603_n14.mtx","synthetic_185639_185639_15006431_avg80.83663_std114.55767_diagonal_bw24067.75636_skew7.94397_neigh1.21737_cross0.84232_n14.mtx","synthetic_185639_185639_15006960_avg80.83948_std117.7123_diagonal_bw17089.64436_skew7.8818_neigh1.21539_cross0.84521_n14.mtx","synthetic_185639_185639_15007305_avg80.84134_std120.8084_diagonal_bw23932.70372_skew7.80738_neigh1.21348_cross0.84884_n14.mtx","synthetic_185639_185639_15008187_avg80.84609_std126.81395_diagonal_bw23820.75271_skew7.64606_neigh1.20923_cross0.85805_n14.mtx","synthetic_185639_185639_15008702_avg80.84886_std137.983_diagonal_bw23110.0529_skew7.32417_neigh1.20249_cross0.89205_n14.mtx","synthetic_185639_185639_15008859_avg80.84971_std132.54683_diagonal_bw23516.89351_skew7.48488_neigh1.20404_cross0.87118_n14.mtx","synthetic_185639_185639_15506741_avg83.5317_std118.48272_diagonal_bw24157.25407_skew7.94271_neigh1.21788_cross0.84141_n14.mtx","synthetic_185639_185639_16506181_avg88.91548_std119.88133_diagonal_bw24248.66297_skew8.06479_neigh1.22306_cross0.83613_n14.mtx","synthetic_185639_185639_17505764_avg94.30003_std121.13382_diagonal_bw24308.77245_skew8.16224_neigh1.22615_cross0.83329_n14.mtx","synthetic_185639_185639_18505682_avg99.68639_std122.27517_diagonal_bw24387.0841_skew8.23897_neigh1.22956_cross0.82911_n14.mtx","synthetic_185639_185639_19505424_avg105.0718_std123.33624_diagonal_bw24435.45274_skew8.2984_neigh1.23248_cross0.82657_n14.mtx","synthetic_185850_185850_1145706_avg6.16468_std4.44467_diagonal_bw321.4122_skew7.27293_neigh0.17353_cross0.32676_n14.mtx","synthetic_188097_188097_1054064_avg5.60383_std4.45656_diagonal_bw55856.29168_skew62.88486_neigh0.59686_cross0.62548_n14.mtx","synthetic_19080_19080_1337993_avg70.12542_std6.31221_diagonal_bw1466.79911_skew0.34046_neigh1.78524_cross0.91492_n14.mtx","synthetic_191826_191826_15508397_avg80.84617_std117.70962_diagonal_bw24854.19831_skew7.88106_neigh1.21284_cross0.84477_n14.mtx","synthetic_199497_199497_1118014_avg5.60416_std4.45398_diagonal_bw59520.93639_skew62.88107_neigh0.59561_cross0.62565_n14.mtx","synthetic_199616_199616_1230682_avg6.16525_std4.44318_diagonal_bw345.00688_skew7.27217_neigh0.17135_cross0.32384_n14.mtx","synthetic_204202_204202_16508082_avg80.84192_std117.71063_diagonal_bw26469.08637_skew7.88153_neigh1.21323_cross0.84417_n14.mtx","synthetic_20493_20493_1437302_avg70.13624_std6.31204_diagonal_bw1570.36622_skew0.34025_neigh1.78301_cross0.915_n14.mtx","synthetic_206500_206500_1060353_avg5.13488_std4.44292_diagonal_bw335.07862_skew7.3741_neigh0.16054_cross0.3088_n14.mtx","synthetic_206500_206500_1145534_avg5.54738_std4.44341_diagonal_bw356.74644_skew7.2922_neigh0.16406_cross0.31454_n14.mtx","synthetic_206500_206500_1230425_avg5.95847_std4.44436_diagonal_bw374.57129_skew7.22358_neigh0.16743_cross0.31868_n14.mtx","synthetic_206500_206500_1272393_avg6.16171_std5.76962_diagonal_bw330.69652_skew7.43922_neigh0.17725_cross0.30587_n14.mtx","synthetic_206500_206500_1272581_avg6.16262_std4.44309_diagonal_bw352.66015_skew6.30209_neigh0.16953_cross0.32032_n14.mtx","synthetic_206500_206500_1272606_avg6.16274_std4.44382_diagonal_bw349.99388_skew5.97741_neigh0.16985_cross0.31715_n14.mtx","synthetic_206500_206500_1272648_avg6.16294_std5.47591_diagonal_bw338.10897_skew7.43753_neigh0.17526_cross0.31073_n14.mtx","synthetic_206500_206500_1272653_avg6.16297_std5.1813_diagonal_bw344.20462_skew7.27523_neigh0.17301_cross0.31335_n14.mtx","synthetic_206500_206500_1272686_avg6.16313_std4.88791_diagonal_bw350.18601_skew7.27502_neigh0.1719_cross0.3183_n14.mtx","synthetic_206500_206500_1272775_avg6.16356_std4.44367_diagonal_bw347.38147_skew5.48976_neigh0.17032_cross0.31578_n14.mtx","synthetic_206500_206500_1272904_avg6.16418_std4.4462_diagonal_bw361.59791_skew8.73365_neigh0.17074_cross0.32864_n14.mtx","synthetic_206500_206500_1272958_avg6.16445_std4.59126_diagonal_bw354.70395_skew7.27325_neigh0.171_cross0.32279_n14.mtx","synthetic_206500_206500_1272981_avg6.16456_std4.44526_diagonal_bw362.3522_skew9.05749_neigh0.17062_cross0.32883_n14.mtx","synthetic_206500_206500_1272983_avg6.16457_std4.4441_diagonal_bw354.25261_skew6.62422_neigh0.17027_cross0.32275_n14.mtx","synthetic_206500_206500_1272990_avg6.1646_std4.44515_diagonal_bw357.75317_skew7.43526_neigh0.17014_cross0.3244_n14.mtx","synthetic_206500_206500_1272993_avg6.16462_std4.44589_diagonal_bw358.92702_skew7.92189_neigh0.17019_cross0.32662_n14.mtx","synthetic_206500_206500_1272999_avg6.16464_std4.44572_diagonal_bw360.42425_skew8.24628_neigh0.17053_cross0.3283_n14.mtx","synthetic_206500_206500_1273034_avg6.16481_std4.29748_diagonal_bw358.86067_skew7.27276_neigh0.17064_cross0.32624_n14.mtx","synthetic_206500_206500_1273051_avg6.1649_std3.71025_diagonal_bw366.16904_skew7.27265_neigh0.16791_cross0.33052_n14.mtx","synthetic_206500_206500_1273053_avg6.16491_std4.44447_diagonal_bw355.79495_skew7.11042_neigh0.17039_cross0.3241_n14.mtx","synthetic_206500_206500_1273074_avg6.16501_std4.00298_diagonal_bw362.83666_skew7.2725_neigh0.16845_cross0.33021_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_diagonal_bw250.50201_skew7.27211_neigh0.1828_cross0.332_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_diagonal_bw272.67229_skew7.27211_neigh0.18005_cross0.32953_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_diagonal_bw297.25621_skew7.27211_neigh0.17513_cross0.32858_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_diagonal_bw321.39573_skew7.27211_neigh0.17392_cross0.32618_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_diagonal_bw345.22322_skew7.27211_neigh0.17126_cross0.32461_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_diagonal_bw352.99268_skew7.27211_neigh0.16832_cross0.41146_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_diagonal_bw353.69447_skew7.27211_neigh0.17026_cross0.39266_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_diagonal_bw354.69502_skew7.27211_neigh0.19369_cross0.32315_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_diagonal_bw354.75358_skew7.27211_neigh0.16996_cross0.37247_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_diagonal_bw355.71822_skew7.27211_neigh0.18424_cross0.32217_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_diagonal_bw355.76259_skew7.27211_neigh0.16942_cross0.35332_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_diagonal_bw356.3296_skew7.27211_neigh0.17524_cross0.3238_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_diagonal_bw356.8881_skew7.27211_neigh0.16993_cross0.33263_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_diagonal_bw357.32531_skew7.27211_neigh0.17048_cross0.3136_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_diagonal_bw357.60403_skew7.27211_neigh0.16581_cross0.32313_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_diagonal_bw358.15523_skew7.27211_neigh0.17117_cross0.29493_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_diagonal_bw358.35721_skew7.27211_neigh0.15654_cross0.322_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_diagonal_bw358.64557_skew7.27211_neigh0.17114_cross0.27625_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_diagonal_bw359.47191_skew7.27211_neigh0.17162_cross0.25643_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_diagonal_bw359.53662_skew7.27211_neigh0.14621_cross0.32227_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_diagonal_bw360.09546_skew7.27211_neigh0.17221_cross0.23667_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_diagonal_bw360.26387_skew7.27211_neigh0.13732_cross0.32285_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_diagonal_bw360.84579_skew7.27211_neigh0.12827_cross0.32319_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_diagonal_bw367.88503_skew7.27211_neigh0.17018_cross0.32346_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_diagonal_bw378.11669_skew7.27211_neigh0.21111_cross0.3229_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_diagonal_bw379.08464_skew7.27211_neigh0.20234_cross0.32283_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_diagonal_bw392.32879_skew7.27211_neigh0.16826_cross0.32093_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_diagonal_bw416.65222_skew7.27211_neigh0.16705_cross0.32007_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_diagonal_bw440.35505_skew7.27211_neigh0.16563_cross0.32035_n14.mtx","synthetic_206500_206500_1273134_avg6.1653_std4.44455_diagonal_bw463.79818_skew7.27211_neigh0.16463_cross0.31842_n14.mtx","synthetic_206500_206500_1273351_avg6.16635_std3.41298_diagonal_bw369.12411_skew7.2707_neigh0.16658_cross0.33288_n14.mtx","synthetic_206500_206500_1274354_avg6.17121_std3.11913_diagonal_bw371.78899_skew7.10214_neigh0.1662_cross0.33678_n14.mtx","synthetic_206500_206500_1315280_avg6.36939_std4.44461_diagonal_bw364.33156_skew7.32104_neigh0.17162_cross0.32684_n14.mtx","synthetic_206500_206500_1400260_avg6.78092_std4.44568_diagonal_bw375.85394_skew7.25847_neigh0.17409_cross0.33062_n14.mtx","synthetic_206500_206500_1485240_avg7.19245_std4.44584_diagonal_bw368.30232_skew7.20305_neigh0.17875_cross0.33711_n14.mtx","synthetic_206500_206500_1570037_avg7.60308_std4.44659_diagonal_bw376.69061_skew7.15458_neigh0.18122_cross0.33823_n14.mtx","synthetic_206500_206500_1655223_avg8.01561_std4.44684_diagonal_bw384.76944_skew7.23394_neigh0.18229_cross0.34249_n14.mtx","synthetic_206500_206500_890883_avg4.3142_std4.3742_diagonal_bw314.74509_skew7.34453_neigh0.14985_cross0.29741_n14.mtx","synthetic_206500_206500_975718_avg4.72503_std4.44295_diagonal_bw342.29814_skew7.46556_neigh0.15394_cross0.29927_n14.mtx","synthetic_210897_210897_1181860_avg5.60397_std4.45232_diagonal_bw62903.73817_skew62.88331_neigh0.59456_cross0.6267_n14.mtx","synthetic_21200_21200_1040239_avg49.06788_std6.3116_diagonal_bw2199.9967_skew0.48774_neigh1.7444_cross0.90083_n14.mtx","synthetic_21200_21200_1139550_avg53.75236_std6.31054_diagonal_bw2242.21934_skew0.4325_neigh1.75417_cross0.90404_n14.mtx","synthetic_21200_21200_1238795_avg58.43373_std6.30753_diagonal_bw2270.23382_skew0.4033_neigh1.7655_cross0.90641_n14.mtx","synthetic_21200_21200_1338002_avg63.1133_std6.31089_diagonal_bw1589.57203_skew0.37847_neigh1.76861_cross0.91301_n14.mtx","synthetic_21200_21200_1437342_avg67.79915_std6.31153_diagonal_bw1595.57099_skew0.35695_neigh1.77813_cross0.91446_n14.mtx","synthetic_21200_21200_1486357_avg70.11118_std8.20133_diagonal_bw1585.25472_skew0.44057_neigh1.79532_cross0.90224_n14.mtx","synthetic_21200_21200_1486477_avg70.11684_std7.77973_diagonal_bw1583.02868_skew0.41193_neigh1.79079_cross0.90683_n14.mtx","synthetic_21200_21200_1486586_avg70.12198_std7.35959_diagonal_bw1593.45448_skew0.39756_neigh1.78432_cross0.91026_n14.mtx","synthetic_21200_21200_1486691_avg70.12693_std6.94018_diagonal_bw1598.67321_skew0.36895_neigh1.79102_cross0.91039_n14.mtx","synthetic_21200_21200_1486777_avg70.13099_std6.51934_diagonal_bw1631.45156_skew0.35461_neigh1.78321_cross0.91501_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_diagonal_bw1081.97071_skew0.3403_neigh1.27549_cross0.91799_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_diagonal_bw1084.54524_skew0.3403_neigh1.38938_cross0.91731_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_diagonal_bw1117.74425_skew0.3403_neigh1.49561_cross0.91247_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_diagonal_bw1145.98717_skew0.3403_neigh1.61232_cross0.9156_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_diagonal_bw1149.7934_skew0.3403_neigh1.7861_cross0.91524_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_diagonal_bw1271.02222_skew0.3403_neigh1.78132_cross0.91674_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_diagonal_bw1300.1284_skew0.3403_neigh1.73006_cross0.9152_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_diagonal_bw1368.9767_skew0.3403_neigh1.77995_cross0.91561_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_diagonal_bw1468.32679_skew0.3403_neigh1.78426_cross0.91509_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_diagonal_bw1537.10948_skew0.3403_neigh1.72718_cross0.94892_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_diagonal_bw1564.65613_skew0.3403_neigh1.73429_cross0.94452_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_diagonal_bw1571.45608_skew0.3403_neigh1.78136_cross0.91469_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_diagonal_bw1625.49264_skew0.3403_neigh1.77718_cross0.91652_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_diagonal_bw1634.23774_skew0.3403_neigh1.86126_cross0.70933_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_diagonal_bw1635.55807_skew0.3403_neigh1.83717_cross0.82731_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_diagonal_bw1642.04132_skew0.3403_neigh1.85064_cross0.77163_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_diagonal_bw1646.38675_skew0.3403_neigh1.86599_cross0.6495_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_diagonal_bw1648.95406_skew0.3403_neigh1.80659_cross0.88791_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_diagonal_bw1682.76604_skew0.3403_neigh1.77909_cross0.91593_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_diagonal_bw1801.4084_skew0.3403_neigh1.77944_cross0.91386_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_diagonal_bw1870.38679_skew0.3403_neigh1.77964_cross0.91599_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_diagonal_bw1950.42142_skew0.3403_neigh1.85569_cross0.91574_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_diagonal_bw1986.16745_skew0.3403_neigh1.78558_cross0.9125_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_diagonal_bw2072.52415_skew0.3403_neigh1.78223_cross0.91562_n14.mtx","synthetic_21200_21200_1486835_avg70.13373_std6.31005_diagonal_bw2091.85245_skew0.3403_neigh1.83963_cross0.91531_n14.mtx","synthetic_21200_21200_1487000_avg70.14151_std6.10324_diagonal_bw1634.66854_skew0.32589_neigh1.77365_cross0.91984_n14.mtx","synthetic_21200_21200_1487077_avg70.14514_std5.67801_diagonal_bw1633.00311_skew0.29731_neigh1.77437_cross0.91951_n14.mtx","synthetic_21200_21200_1487192_avg70.15057_std5.25576_diagonal_bw1652.73608_skew0.28295_neigh1.78041_cross0.92082_n14.mtx","synthetic_21200_21200_1487315_avg70.15637_std4.84276_diagonal_bw1652.45113_skew0.25434_neigh1.77351_cross0.92458_n14.mtx","synthetic_21200_21200_1487478_avg70.16406_std4.4186_diagonal_bw1687.16137_skew0.23995_neigh1.76694_cross0.92745_n14.mtx","synthetic_21200_21200_1536543_avg72.47844_std6.30853_diagonal_bw1645.73835_skew0.32453_neigh1.78455_cross0.91665_n14.mtx","synthetic_21200_21200_1635721_avg77.15665_std6.31046_diagonal_bw1656.96292_skew0.30903_neigh1.79294_cross0.9192_n14.mtx","synthetic_21200_21200_1735090_avg81.84387_std6.31393_diagonal_bw1680.13108_skew0.29515_neigh1.79626_cross0.9206_n14.mtx","synthetic_21200_21200_1834315_avg86.52429_std6.30851_diagonal_bw1434.02858_skew0.27132_neigh1.79882_cross0.92217_n14.mtx","synthetic_21200_21200_1933465_avg91.20118_std6.31043_diagonal_bw1436.35245_skew0.26095_neigh1.80915_cross0.92347_n14.mtx","synthetic_213383_213383_1315556_avg6.16523_std4.44392_diagonal_bw368.56408_skew7.27219_neigh0.16933_cross0.32183_n14.mtx","synthetic_216578_216578_17508080_avg80.83961_std117.71328_diagonal_bw28056.02979_skew7.89416_neigh1.2146_cross0.84473_n14.mtx","synthetic_21906_21906_1536320_avg70.13238_std6.31756_diagonal_bw1682.27098_skew0.34032_neigh1.78153_cross0.91497_n14.mtx","synthetic_222297_222297_1245675_avg5.60365_std4.45068_diagonal_bw66275.4602_skew62.88691_neigh0.59474_cross0.62554_n14.mtx","synthetic_227150_227150_1400378_avg6.16499_std4.4444_diagonal_bw392.40004_skew7.27252_neigh0.16863_cross0.32173_n14.mtx","synthetic_228954_228954_18508358_avg80.83876_std117.71434_diagonal_bw29655.86157_skew7.89425_neigh1.21497_cross0.8448_n14.mtx","synthetic_23320_23320_1635398_avg70.12856_std6.31946_diagonal_bw1799.99889_skew0.3404_neigh1.78021_cross0.91361_n14.mtx","synthetic_240916_240916_1485276_avg6.16512_std4.4437_diagonal_bw416.70601_skew7.27235_neigh0.16679_cross0.32047_n14.mtx","synthetic_241330_241330_19509069_avg80.8398_std117.71435_diagonal_bw31247.32631_skew7.89413_neigh1.21166_cross0.84524_n14.mtx","synthetic_24733_24733_1734543_avg70.13072_std6.31636_diagonal_bw1883.50532_skew0.34035_neigh1.77994_cross0.91558_n14.mtx","synthetic_254683_254683_1570192_avg6.16528_std4.4433_diagonal_bw440.76115_skew7.27213_neigh0.1672_cross0.32015_n14.mtx","synthetic_25491_25491_3028356_avg118.80099_std31.7722_diagonal_bw2553.87309_skew1.00335_neigh1.79544_cross0.81183_n14.mtx","synthetic_26146_26146_1833536_avg70.12683_std6.31243_diagonal_bw2003.83967_skew0.34043_neigh1.78323_cross0.91235_n14.mtx","synthetic_268096_268096_18481037_avg68.9344_std103.42196_diagonal_bw30970.77096_skew12.99882_neigh1.12598_cross0.8301_n14.mtx","synthetic_268096_268096_18481246_avg68.93518_std102.1488_diagonal_bw30884.51728_skew11.06931_neigh1.12482_cross0.83411_n14.mtx","synthetic_268096_268096_18481318_avg68.93545_std102.6344_diagonal_bw30907.47443_skew11.72205_neigh1.12565_cross0.83282_n14.mtx","synthetic_268096_268096_18481428_avg68.93586_std103.0557_diagonal_bw30967.38712_skew12.36025_neigh1.12626_cross0.83123_n14.mtx","synthetic_268096_268096_18481631_avg68.93662_std101.58036_diagonal_bw30873.57231_skew10.43079_neigh1.12458_cross0.83658_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_diagonal_bw29370.46931_skew10.11157_neigh1.10661_cross0.8633_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_diagonal_bw30799.97611_skew10.11157_neigh1.16245_cross0.83769_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_diagonal_bw30825.7074_skew10.11157_neigh1.23495_cross0.83815_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_diagonal_bw30839.54861_skew10.11157_neigh1.38307_cross0.83895_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_diagonal_bw30893.42836_skew10.11157_neigh1.01276_cross0.83715_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_diagonal_bw30904.3262_skew10.11157_neigh1.31024_cross0.83886_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_diagonal_bw30928.04662_skew10.11157_neigh1.08835_cross0.83649_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_diagonal_bw30929.83701_skew10.11157_neigh0.93859_cross0.83647_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_diagonal_bw30967.23835_skew10.11157_neigh0.86484_cross0.83608_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_diagonal_bw30999.29783_skew10.11157_neigh1.45859_cross0.83966_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_diagonal_bw31009.21338_skew10.11157_neigh0.79088_cross0.83554_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_diagonal_bw32084.18859_skew10.11157_neigh1.13434_cross0.80865_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_diagonal_bw33853.27533_skew10.11157_neigh1.14358_cross0.7543_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_diagonal_bw35112.29857_skew10.11157_neigh1.14998_cross0.69937_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_diagonal_bw36191.33218_skew10.11157_neigh1.15329_cross0.64297_n14.mtx","synthetic_268096_268096_18481779_avg68.93717_std101.2577_diagonal_bw37019.2485_skew10.11157_neigh1.15704_cross0.58827_n14.mtx","synthetic_268096_268096_18482060_avg68.93822_std100.90606_diagonal_bw30904.29893_skew9.79227_neigh1.12389_cross0.83844_n14.mtx","synthetic_268096_268096_18482363_avg68.93935_std100.10125_diagonal_bw30812.69345_skew9.13935_neigh1.12221_cross0.84194_n14.mtx","synthetic_268096_268096_18482725_avg68.9407_std99.12285_diagonal_bw30864.65616_skew8.47191_neigh1.12038_cross0.84492_n14.mtx","synthetic_268096_268096_18483147_avg68.94227_std97.91705_diagonal_bw30780.25248_skew7.80447_neigh1.11844_cross0.84993_n14.mtx","synthetic_268450_268450_1655008_avg6.16505_std4.44456_diagonal_bw464.64015_skew7.27244_neigh0.16504_cross0.31934_n14.mtx","synthetic_27120_27120_1234696_avg45.52714_std31.4396_diagonal_bw652.95889_skew2.64618_neigh1.23287_cross0.77761_n14.mtx","synthetic_27560_27560_1933075_avg70.1406_std6.31297_diagonal_bw2085.91422_skew0.34017_neigh1.78453_cross0.915_n14.mtx","synthetic_27919_27919_3319237_avg118.8881_std31.78107_diagonal_bw2786.27282_skew1.00188_neigh1.79352_cross0.80999_n14.mtx","synthetic_29496_29496_3296630_avg111.76532_std88.66285_diagonal_bw11717.2995_skew3.35734_neigh1.89319_cross0.64015_n14.mtx","synthetic_29703_29703_1352183_avg45.52345_std31.43457_diagonal_bw714.4414_skew2.64647_neigh1.23245_cross0.77394_n14.mtx","synthetic_30347_30347_3608195_avg118.89791_std31.76143_diagonal_bw3042.07187_skew1.00172_neigh1.79272_cross0.81233_n14.mtx","synthetic_32286_32286_1469958_avg45.52927_std31.42835_diagonal_bw776.30704_skew2.66797_neigh1.22078_cross0.77601_n14.mtx","synthetic_32305_32305_3613009_avg111.84055_std88.75337_diagonal_bw12837.01436_skew3.35441_neigh1.89289_cross0.64054_n14.mtx","synthetic_32775_32775_3898242_avg118.9395_std31.78937_diagonal_bw3266.08442_skew1.00102_neigh1.79344_cross0.81117_n14.mtx","synthetic_32784_32784_1657863_avg50.56927_std27.82469_diagonal_bw4979.98978_skew3.09339_neigh1.63543_cross0.76823_n14.mtx","synthetic_3283001_3283001_13139084_avg4.00216_std1.21615_diagonal_bw3180.2682_skew312.8308_neigh0.41642_cross0.64828_n14.mtx","synthetic_34406_34406_1341834_avg39.0_std0.0_diagonal_bw7973.05903_skew0.0_neigh1.39882_cross0.80525_n14.mtx","synthetic_34869_34869_1587838_avg45.53724_std31.4194_diagonal_bw841.85663_skew2.66733_neigh1.22513_cross0.77194_n14.mtx","synthetic_35115_35115_3931279_avg111.95441_std88.67056_diagonal_bw13934.66499_skew3.34999_neigh1.89327_cross0.64172_n14.mtx","synthetic_35203_35203_4188100_avg118.96997_std31.72558_diagonal_bw3517.51751_skew1.0005_neigh1.79419_cross0.8104_n14.mtx","synthetic_35906_35906_1816148_avg50.58063_std27.80623_diagonal_bw5428.79923_skew3.09248_neigh1.63383_cross0.77123_n14.mtx","synthetic_3595668_3595668_14390363_avg4.00214_std1.20637_diagonal_bw3482.69612_skew312.83218_neigh0.41594_cross0.64805_n14.mtx","synthetic_36417_36417_3031503_avg83.24417_std31.62901_diagonal_bw3659.90647_skew1.4266_neigh1.77779_cross0.7712_n14.mtx","synthetic_36417_36417_3320284_avg91.17401_std31.69422_diagonal_bw3723.96647_skew1.30329_neigh1.78294_cross0.78319_n14.mtx","synthetic_36417_36417_3609556_avg99.11734_std31.72484_diagonal_bw3679.72145_skew1.19941_neigh1.78657_cross0.7906_n14.mtx","synthetic_36417_36417_3899087_avg107.06777_std31.73599_diagonal_bw3640.28097_skew1.11081_neigh1.79005_cross0.79883_n14.mtx","synthetic_36417_36417_4188724_avg115.02112_std31.74031_diagonal_bw3682.55345_skew1.03441_neigh1.79122_cross0.80743_n14.mtx","synthetic_36417_36417_4330804_avg118.92259_std41.20033_diagonal_bw3566.11415_skew1.30402_neigh1.79774_cross0.77959_n14.mtx","synthetic_36417_36417_4331206_avg118.93363_std39.11615_diagonal_bw3582.20153_skew1.23654_neigh1.79553_cross0.78805_n14.mtx","synthetic_36417_36417_4331707_avg118.94739_std37.01905_diagonal_bw3596.97108_skew1.16903_neigh1.79583_cross0.79348_n14.mtx","synthetic_36417_36417_4332360_avg118.96532_std34.9075_diagonal_bw3586.20139_skew1.10145_neigh1.79703_cross0.80194_n14.mtx","synthetic_36417_36417_4333163_avg118.98737_std32.79928_diagonal_bw3623.78502_skew1.03383_neigh1.79496_cross0.80499_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_diagonal_bw2579.24335_skew1.00003_neigh1.79431_cross0.81183_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_diagonal_bw2808.21114_skew1.00003_neigh1.79322_cross0.81044_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_diagonal_bw3060.87028_skew1.00003_neigh1.7932_cross0.81204_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_diagonal_bw3277.14828_skew1.00003_neigh1.79298_cross0.81293_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_diagonal_bw3375.93338_skew1.00003_neigh1.77805_cross0.86843_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_diagonal_bw3480.48944_skew1.00003_neigh1.38997_cross0.81257_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_diagonal_bw3491.53049_skew1.00003_neigh1.50396_cross0.81194_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_diagonal_bw3492.88901_skew1.00003_neigh1.27007_cross0.81135_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_diagonal_bw3503.97943_skew1.00003_neigh1.62041_cross0.81058_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_diagonal_bw3512.30892_skew1.00003_neigh1.7876_cross0.83745_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_diagonal_bw3516.78812_skew1.00003_neigh1.79411_cross0.811_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_diagonal_bw3588.56273_skew1.00003_neigh1.73452_cross0.81056_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_diagonal_bw3629.41058_skew1.00003_neigh1.79306_cross0.81098_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_diagonal_bw3710.52736_skew1.00003_neigh1.79803_cross0.78467_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_diagonal_bw3750.91707_skew1.00003_neigh1.79431_cross0.80913_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_diagonal_bw3858.63267_skew1.00003_neigh1.80498_cross0.73111_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_diagonal_bw3893.78263_skew1.00003_neigh1.85196_cross0.8121_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_diagonal_bw3967.32032_skew1.00003_neigh1.80868_cross0.68014_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_diagonal_bw3977.20534_skew1.00003_neigh1.79315_cross0.81223_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_diagonal_bw4055.86613_skew1.00003_neigh1.81065_cross0.62863_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_diagonal_bw4135.85089_skew1.00003_neigh1.81242_cross0.57712_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_diagonal_bw4215.83134_skew1.00003_neigh1.79293_cross0.81131_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_diagonal_bw4432.89683_skew1.00003_neigh1.79479_cross0.81053_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_diagonal_bw4655.25722_skew1.00003_neigh1.79359_cross0.80909_n14.mtx","synthetic_36417_36417_4333559_avg118.99824_std31.74163_diagonal_bw5224.60291_skew1.00003_neigh1.94938_cross0.81229_n14.mtx","synthetic_36417_36417_4333866_avg119.00667_std30.68416_diagonal_bw3624.47093_skew0.96628_neigh1.79345_cross0.81402_n14.mtx","synthetic_36417_36417_4334685_avg119.02916_std28.56617_diagonal_bw3659.71673_skew0.89869_neigh1.79021_cross0.8221_n14.mtx","synthetic_36417_36417_4335436_avg119.04978_std26.44817_diagonal_bw3656.97312_skew0.83117_neigh1.79127_cross0.82819_n14.mtx","synthetic_36417_36417_4336151_avg119.06942_std24.33643_diagonal_bw3688.3458_skew0.76368_neigh1.79072_cross0.83561_n14.mtx","synthetic_36417_36417_4336875_avg119.0893_std22.21912_diagonal_bw3701.03968_skew0.7046_neigh1.78927_cross0.84338_n14.mtx","synthetic_36417_36417_4478356_avg122.97433_std31.74172_diagonal_bw3655.42475_skew0.96789_neigh1.79462_cross0.81529_n14.mtx","synthetic_36417_36417_4767999_avg130.92784_std31.7426_diagonal_bw3634.76154_skew0.90945_neigh1.79683_cross0.8202_n14.mtx","synthetic_36417_36417_5057685_avg138.88253_std31.74387_diagonal_bw3658.71557_skew0.85769_neigh1.80018_cross0.82673_n14.mtx","synthetic_36417_36417_5347290_avg146.83499_std31.74228_diagonal_bw3634.01996_skew0.81156_neigh1.80021_cross0.83143_n14.mtx","synthetic_36417_36417_5636923_avg154.78823_std31.74188_diagonal_bw3643.12294_skew0.77016_neigh1.80217_cross0.8324_n14.mtx","synthetic_368077_368077_1472308_avg4.0_std0.0_diagonal_bw504.58307_skew0.0_neigh0.37345_cross0.98911_n14.mtx","synthetic_37452_37452_1706036_avg45.5526_std31.4087_diagonal_bw902.84756_skew2.66609_neigh1.22018_cross0.77042_n14.mtx","synthetic_37630_37630_4477645_avg118.99136_std31.72824_diagonal_bw3752.49753_skew1.00015_neigh1.79415_cross0.80957_n14.mtx","synthetic_37683_37683_1469637_avg39.0_std0.0_diagonal_bw8731.54935_skew0.0_neigh1.39924_cross0.81006_n14.mtx","synthetic_37924_37924_4245700_avg111.95285_std88.65794_diagonal_bw15018.56603_skew3.35005_neigh1.89355_cross0.6412_n14.mtx","synthetic_38120_38120_14186695_avg372.15884_std217.74813_diagonal_bw16610.97285_skew1.39145_neigh1.97336_cross0.92298_n14.mtx","synthetic_38120_38120_14353565_avg376.53633_std228.90355_diagonal_bw16362.02206_skew1.46191_neigh1.97392_cross0.9229_n14.mtx","synthetic_38120_38120_14506403_avg380.54572_std239.95763_diagonal_bw16221.47907_skew1.5332_neigh1.97371_cross0.92545_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_diagonal_bw16107.56653_skew1.57045_neigh1.90891_cross0.91992_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_diagonal_bw16165.12815_skew1.57045_neigh1.3922_cross0.92174_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_diagonal_bw16173.10614_skew1.57045_neigh1.64922_cross0.92071_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_diagonal_bw16181.2495_skew1.57045_neigh1.51874_cross0.92011_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_diagonal_bw16189.91522_skew1.57045_neigh1.77707_cross0.91956_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_diagonal_bw16403.35273_skew1.57045_neigh1.97397_cross0.72233_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_diagonal_bw16412.02442_skew1.57045_neigh1.97389_cross0.66503_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_diagonal_bw16450.53809_skew1.57045_neigh1.9739_cross0.77998_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_diagonal_bw16504.44638_skew1.57045_neigh1.97349_cross0.83444_n14.mtx","synthetic_38120_38120_14577958_avg382.42282_std245.44525_diagonal_bw5783.47169_skew1.57045_neigh1.99394_cross0.9293_n14.mtx","synthetic_38120_38120_14646455_avg384.2197_std250.9036_diagonal_bw16369.04491_skew1.60788_neigh1.97362_cross0.92456_n14.mtx","synthetic_38120_38120_14774683_avg387.5835_std261.73289_diagonal_bw16271.8489_skew1.68071_neigh1.97337_cross0.92347_n14.mtx","synthetic_38120_38120_14892124_avg390.66432_std272.44037_diagonal_bw16642.15774_skew1.75428_neigh1.9735_cross0.92246_n14.mtx","synthetic_38120_38120_15029193_avg394.26005_std283.24138_diagonal_bw17593.26878_skew1.83569_neigh1.95985_cross0.92052_n14.mtx","synthetic_38120_38120_15246247_avg399.95401_std294.48908_diagonal_bw17717.6793_skew1.94534_neigh1.95613_cross0.91225_n14.mtx","synthetic_38744_38744_1161488_avg29.97853_std22.5272_diagonal_bw1029.10133_skew1.93543_neigh1.22753_cross0.85246_n14.mtx","synthetic_38744_38744_1307882_avg33.75702_std24.88267_diagonal_bw994.82547_skew2.08084_neigh1.17474_cross0.82692_n14.mtx","synthetic_38744_38744_1449216_avg37.40491_std27.07152_diagonal_bw955.52689_skew2.28834_neigh1.19116_cross0.80482_n14.mtx","synthetic_38744_38744_1579684_avg40.77235_std28.98643_diagonal_bw944.42949_skew2.45823_neigh1.20967_cross0.78737_n14.mtx","synthetic_38744_38744_1703850_avg43.97713_std30.65815_diagonal_bw937.00284_skew2.61552_neigh1.2153_cross0.77679_n14.mtx","synthetic_38744_38744_1707520_avg44.07186_std32.46486_diagonal_bw992.99381_skew2.10856_neigh1.20651_cross0.82436_n14.mtx","synthetic_38744_38744_1730367_avg44.66155_std32.51286_diagonal_bw967.85456_skew2.22425_neigh1.21385_cross0.81258_n14.mtx","synthetic_38744_38744_1736329_avg44.81543_std26.74959_diagonal_bw952.63904_skew1.96773_neigh1.21351_cross0.79326_n14.mtx","synthetic_38744_38744_1746236_avg45.07113_std32.41542_diagonal_bw950.25712_skew2.35026_neigh1.21262_cross0.79957_n14.mtx","synthetic_38744_38744_1747768_avg45.11068_std28.0118_diagonal_bw947.08319_skew2.12565_neigh1.21504_cross0.78804_n14.mtx","synthetic_38744_38744_1755242_avg45.30358_std29.13201_diagonal_bw940.57346_skew2.28892_neigh1.21712_cross0.78411_n14.mtx","synthetic_38744_38744_1756611_avg45.33892_std32.15335_diagonal_bw942.33254_skew2.48486_neigh1.21428_cross0.78987_n14.mtx","synthetic_38744_38744_1760377_avg45.43612_std30.12952_diagonal_bw941.83703_skew2.43339_neigh1.22146_cross0.77671_n14.mtx","synthetic_38744_38744_1762627_avg45.49419_std31.70473_diagonal_bw939.03988_skew2.60486_neigh1.21755_cross0.77579_n14.mtx","synthetic_38744_38744_1763457_avg45.51562_std31.01391_diagonal_bw935.46596_skew2.60316_neigh1.22021_cross0.77524_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_diagonal_bw1007.61049_skew2.6669_neigh1.23901_cross0.60466_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_diagonal_bw1020.39872_skew2.6669_neigh1.23921_cross0.56066_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_diagonal_bw1025.26425_skew2.6669_neigh1.21375_cross0.77219_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_diagonal_bw1083.3129_skew2.6669_neigh1.21379_cross0.77025_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_diagonal_bw1146.61932_skew2.6669_neigh1.21114_cross0.77133_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_diagonal_bw1206.53601_skew2.6669_neigh1.21863_cross0.77168_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_diagonal_bw658.37758_skew2.6669_neigh1.22947_cross0.77507_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_diagonal_bw718.69366_skew2.6669_neigh1.22855_cross0.77459_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_diagonal_bw780.82743_skew2.6669_neigh1.22346_cross0.77387_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_diagonal_bw812.95816_skew2.6669_neigh1.18464_cross0.89437_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_diagonal_bw816.16013_skew2.6669_neigh1.18813_cross0.89061_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_diagonal_bw837.69265_skew2.6669_neigh1.22334_cross0.77418_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_diagonal_bw873.34813_skew2.6669_neigh1.20844_cross0.84314_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_diagonal_bw897.51727_skew2.6669_neigh1.2208_cross0.77498_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_diagonal_bw915.8694_skew2.6669_neigh1.21725_cross0.79709_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_diagonal_bw930.70419_skew2.6669_neigh1.10614_cross0.77727_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_diagonal_bw930.71709_skew2.6669_neigh1.18489_cross0.77237_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_diagonal_bw931.48136_skew2.6669_neigh1.03297_cross0.77123_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_diagonal_bw932.33357_skew2.6669_neigh0.96264_cross0.77189_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_diagonal_bw932.7993_skew2.6669_neigh1.25769_cross0.77371_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_diagonal_bw934.61429_skew2.6669_neigh0.88819_cross0.7721_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_diagonal_bw937.26642_skew2.6669_neigh1.33096_cross0.77342_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_diagonal_bw940.71841_skew2.6669_neigh1.40988_cross0.77173_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_diagonal_bw950.09139_skew2.6669_neigh1.48389_cross0.77368_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_diagonal_bw950.26476_skew2.6669_neigh1.2235_cross0.74756_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_diagonal_bw961.91957_skew2.6669_neigh1.22023_cross0.77204_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_diagonal_bw968.81104_skew2.6669_neigh1.56264_cross0.77172_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_diagonal_bw970.44453_skew2.6669_neigh1.2313_cross0.70063_n14.mtx","synthetic_38744_38744_1764502_avg45.54259_std31.41249_diagonal_bw991.09919_skew2.6669_neigh1.23319_cross0.6528_n14.mtx","synthetic_38744_38744_1765368_avg45.56494_std31.78636_diagonal_bw930.97393_skew2.75289_neigh1.22265_cross0.77205_n14.mtx","synthetic_38744_38744_1765877_avg45.57808_std31.07073_diagonal_bw931.07658_skew2.72986_neigh1.22085_cross0.76806_n14.mtx","synthetic_38744_38744_1766496_avg45.59405_std32.481_diagonal_bw927.24148_skew2.88209_neigh1.22172_cross0.76806_n14.mtx","synthetic_38744_38744_1767073_avg45.60895_std26.86577_diagonal_bw920.15365_skew3.122_neigh1.22525_cross0.74801_n14.mtx","synthetic_38744_38744_1767160_avg45.61119_std30.25452_diagonal_bw926.58649_skew2.83678_neigh1.22133_cross0.7642_n14.mtx","synthetic_38744_38744_1767330_avg45.61558_std33.08901_diagonal_bw927.56388_skew3.03371_neigh1.22246_cross0.76288_n14.mtx","synthetic_38744_38744_1767431_avg45.61819_std29.2665_diagonal_bw927.4427_skew2.94579_neigh1.22246_cross0.75424_n14.mtx","synthetic_38744_38744_1767460_avg45.61893_std28.1289_diagonal_bw919.38078_skew3.03341_neigh1.22097_cross0.75479_n14.mtx","synthetic_38744_38744_1767651_avg45.62386_std33.63482_diagonal_bw924.81207_skew3.16449_neigh1.22491_cross0.76276_n14.mtx","synthetic_38744_38744_1767760_avg45.62668_std34.12282_diagonal_bw922.18836_skew3.31765_neigh1.22536_cross0.76056_n14.mtx","synthetic_38744_38744_1824565_avg47.09284_std32.11946_diagonal_bw936.71239_skew2.7373_neigh1.22296_cross0.76775_n14.mtx","synthetic_38744_38744_1943737_avg50.16872_std33.41096_diagonal_bw932.86026_skew2.82709_neigh1.22921_cross0.76384_n14.mtx","synthetic_38744_38744_2062005_avg53.22127_std34.5782_diagonal_bw922.50232_skew2.90821_neigh1.23925_cross0.76092_n14.mtx","synthetic_38744_38744_2179882_avg56.26373_std35.63951_diagonal_bw923.68405_skew2.98125_neigh1.24163_cross0.75814_n14.mtx","synthetic_38744_38744_2297582_avg59.30162_std36.62621_diagonal_bw923.20411_skew3.03024_neigh1.25019_cross0.75495_n14.mtx","synthetic_39029_39029_1974236_avg50.58382_std27.80163_diagonal_bw5889.07769_skew3.11199_neigh1.63432_cross0.77123_n14.mtx","synthetic_3908334_3908334_15641636_avg4.00212_std1.19836_diagonal_bw3787.4883_skew312.83338_neigh0.41515_cross0.64842_n14.mtx","synthetic_40035_40035_1824084_avg45.56223_std31.3971_diagonal_bw964.72262_skew2.66532_neigh1.22009_cross0.77447_n14.mtx","synthetic_40058_40058_4768887_avg119.04955_std31.78281_diagonal_bw3989.12307_skew0.99917_neigh1.79379_cross0.8114_n14.mtx","synthetic_403132_403132_1612528_avg4.0_std0.0_diagonal_bw553.42608_skew0.0_neigh0.37448_cross0.98906_n14.mtx","synthetic_40733_40733_4573744_avg112.28596_std88.81893_diagonal_bw16070.31044_skew3.33714_neigh1.8936_cross0.64178_n14.mtx","synthetic_40960_40960_1597440_avg39.0_std0.0_diagonal_bw9495.98555_skew0.0_neigh1.40202_cross0.80537_n14.mtx","synthetic_42138_42138_3666754_avg87.01775_std81.65352_diagonal_bw17018.78651_skew4.24031_neigh1.89191_cross0.58167_n14.mtx","synthetic_42138_42138_3893764_avg92.40505_std83.38574_diagonal_bw15269.74446_skew4.01055_neigh1.89198_cross0.59237_n14.mtx","synthetic_42138_42138_3928503_avg93.22946_std52.30654_diagonal_bw21754.9879_skew1.38122_neigh1.71883_cross0.97165_n14.mtx","synthetic_42138_42138_4006220_avg95.07381_std54.75201_diagonal_bw21199.48559_skew1.47176_neigh1.73848_cross0.96424_n14.mtx","synthetic_42138_42138_4072421_avg96.64486_std57.11798_diagonal_bw20552.35225_skew1.5661_neigh1.75712_cross0.95786_n14.mtx","synthetic_42138_42138_4126809_avg97.93557_std85.02631_diagonal_bw15674.99532_skew3.79907_neigh1.89239_cross0.60629_n14.mtx","synthetic_42138_42138_4128960_avg97.98662_std59.40057_diagonal_bw20131.7263_skew1.65342_neigh1.77031_cross0.9522_n14.mtx","synthetic_42138_42138_4176699_avg99.11954_std61.5954_diagonal_bw19886.75485_skew1.74416_neigh1.78051_cross0.94591_n14.mtx","synthetic_42138_42138_4365370_avg103.59699_std86.57531_diagonal_bw16089.94143_skew3.60438_neigh1.89274_cross0.62091_n14.mtx","synthetic_42138_42138_4491637_avg106.5935_std66.96099_diagonal_bw17746.25056_skew2.48989_neigh1.88275_cross0.71376_n14.mtx","synthetic_42138_42138_4533678_avg107.5912_std72.07832_diagonal_bw17488.78198_skew2.69919_neigh1.88644_cross0.69523_n14.mtx","synthetic_42138_42138_4582731_avg108.7553_std77.01103_diagonal_bw17151.27085_skew2.88947_neigh1.88851_cross0.67805_n14.mtx","synthetic_42138_42138_4608940_avg109.37728_std88.04095_diagonal_bw16405.47544_skew3.42505_neigh1.89321_cross0.63399_n14.mtx","synthetic_42138_42138_4638390_avg110.07618_std81.79532_diagonal_bw16937.41371_skew3.07899_neigh1.88999_cross0.6614_n14.mtx","synthetic_42138_42138_4699816_avg111.53391_std86.4582_diagonal_bw16776.08987_skew3.24983_neigh1.89212_cross0.64722_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_diagonal_bw12401.69942_skew3.33605_neigh1.89352_cross0.6363_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_diagonal_bw13394.34783_skew3.33605_neigh1.89357_cross0.63833_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_diagonal_bw13613.90223_skew3.33605_neigh1.32672_cross0.62991_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_diagonal_bw13637.14168_skew3.33605_neigh1.45441_cross0.63031_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_diagonal_bw13744.74961_skew3.33605_neigh1.57876_cross0.6321_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_diagonal_bw13939.96504_skew3.33605_neigh1.70489_cross0.63265_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_diagonal_bw14040.32778_skew3.33605_neigh1.96986_cross0.65808_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_diagonal_bw14295.7245_skew3.33605_neigh1.89407_cross0.63944_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_diagonal_bw14505.18821_skew3.33605_neigh1.83046_cross0.63541_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_diagonal_bw15310.55024_skew3.33605_neigh1.89386_cross0.6408_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_diagonal_bw16210.1421_skew3.33605_neigh1.89362_cross0.64161_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_diagonal_bw16611.63949_skew3.33605_neigh1.89329_cross0.64481_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_diagonal_bw16636.23077_skew3.33605_neigh1.89346_cross0.6403_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_diagonal_bw16806.17125_skew3.33605_neigh1.89434_cross0.62136_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_diagonal_bw17086.3858_skew3.33605_neigh1.89638_cross0.5859_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_diagonal_bw17169.41374_skew3.33605_neigh1.89337_cross0.64205_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_diagonal_bw17388.06735_skew3.33605_neigh1.89731_cross0.5472_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_diagonal_bw17574.48287_skew3.33605_neigh1.89818_cross0.5104_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_diagonal_bw17859.63285_skew3.33605_neigh1.89909_cross0.46925_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_diagonal_bw17951.80365_skew3.33605_neigh1.9579_cross0.66188_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_diagonal_bw17999.08176_skew3.33605_neigh1.89332_cross0.64342_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_diagonal_bw18723.11078_skew3.33605_neigh1.89352_cross0.64341_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_diagonal_bw19493.96504_skew3.33605_neigh1.89374_cross0.64434_n14.mtx","synthetic_42138_42138_4732695_avg112.31418_std88.74122_diagonal_bw20097.98809_skew3.33605_neigh1.89382_cross0.64549_n14.mtx","synthetic_42138_42138_4766804_avg113.12364_std90.99737_diagonal_bw16569.85666_skew3.41994_neigh1.8942_cross0.63355_n14.mtx","synthetic_42138_42138_4838257_avg114.81933_std95.4594_diagonal_bw16396.72595_skew3.5724_neigh1.89578_cross0.62195_n14.mtx","synthetic_42138_42138_4857446_avg115.27472_std89.41484_diagonal_bw16809.13779_skew3.25939_neigh1.89318_cross0.64791_n14.mtx","synthetic_42138_42138_4913422_avg116.60311_std99.85384_diagonal_bw16243.33808_skew3.72543_neigh1.89696_cross0.61156_n14.mtx","synthetic_42138_42138_4991872_avg118.46485_std104.19584_diagonal_bw14758.12336_skew3.8622_neigh1.89775_cross0.60004_n14.mtx","synthetic_42138_42138_5073389_avg120.39938_std108.48726_diagonal_bw14653.196_skew4.00003_neigh1.89942_cross0.59075_n14.mtx","synthetic_42138_42138_5111019_avg121.2924_std90.68096_diagonal_bw15825.42845_skew3.10578_neigh1.89371_cross0.65868_n14.mtx","synthetic_42138_42138_5368820_avg127.41041_std91.85872_diagonal_bw16050.79954_skew2.96357_neigh1.89482_cross0.67034_n14.mtx","synthetic_42138_42138_5630518_avg133.62091_std92.94843_diagonal_bw16358.79403_skew2.83174_neigh1.89447_cross0.682_n14.mtx","synthetic_42138_42138_5895869_avg139.9181_std93.95081_diagonal_bw16631.46988_skew2.70931_neigh1.89472_cross0.69266_n14.mtx","synthetic_42151_42151_2133812_avg50.62305_std27.78297_diagonal_bw6371.02387_skew3.12855_neigh1.63402_cross0.77144_n14.mtx","synthetic_4221001_4221001_16892920_avg4.00211_std1.19165_diagonal_bw4085.33261_skew312.83427_neigh0.41605_cross0.64818_n14.mtx","synthetic_42486_42486_5059907_avg119.09587_std31.75092_diagonal_bw4233.6327_skew1.14113_neigh1.79264_cross0.81065_n14.mtx","synthetic_42618_42618_1942443_avg45.578_std31.38541_diagonal_bw1023.09081_skew2.66405_neigh1.21726_cross0.77332_n14.mtx","synthetic_43542_43542_4892832_avg112.3704_std88.7642_diagonal_bw17276.74331_skew3.82333_neigh1.89356_cross0.64204_n14.mtx","synthetic_43715_43715_2801362_avg64.0824_std14.01594_diagonal_bw317.77484_skew0.93501_neigh1.58611_cross0.83222_n14.mtx","synthetic_438187_438187_1752748_avg4.0_std0.0_diagonal_bw605.21006_skew0.0_neigh0.36936_cross0.98955_n14.mtx","synthetic_44236_44236_1725204_avg39.0_std0.0_diagonal_bw10260.92608_skew0.0_neigh1.40155_cross0.80735_n14.mtx","synthetic_44914_44914_5350806_avg119.13448_std31.75488_diagonal_bw4462.1276_skew1.14044_neigh1.7944_cross0.81059_n14.mtx","synthetic_45201_45201_2060190_avg45.57842_std31.38204_diagonal_bw1086.85529_skew2.68596_neigh1.21635_cross0.77336_n14.mtx","synthetic_45273_45273_2292008_avg50.62638_std27.77577_diagonal_bw6880.17573_skew3.12828_neigh1.63398_cross0.77235_n14.mtx","synthetic_4533668_4533668_18144199_avg4.0021_std1.18568_diagonal_bw4390.98196_skew312.83513_neigh0.41656_cross0.64825_n14.mtx","synthetic_46351_46351_5209319_avg112.38849_std88.86396_diagonal_bw18374.76208_skew3.82256_neigh1.89341_cross0.64137_n14.mtx","synthetic_46835_46835_1659331_avg35.42929_std24.19453_diagonal_bw7302.21917_skew2.92331_neigh1.5963_cross0.7705_n14.mtx","synthetic_46835_46835_1817944_avg38.81593_std25.43637_diagonal_bw7388.90979_skew3.04473_neigh1.61055_cross0.76366_n14.mtx","synthetic_46835_46835_1976036_avg42.19144_std26.52871_diagonal_bw7052.0227_skew3.14776_neigh1.62131_cross0.75772_n14.mtx","synthetic_46835_46835_2134025_avg45.56475_std27.52318_diagonal_bw7179.05959_skew3.21378_neigh1.62818_cross0.75329_n14.mtx","synthetic_46835_46835_2292218_avg48.94241_std27.77072_diagonal_bw6998.59428_skew3.16816_neigh1.62975_cross0.76816_n14.mtx","synthetic_46835_46835_2370905_avg50.6225_std27.74621_diagonal_bw6973.90507_skew2.81253_neigh1.63594_cross0.75611_n14.mtx","synthetic_46835_46835_2371001_avg50.62455_std27.39541_diagonal_bw6984.37081_skew2.69386_neigh1.6379_cross0.75534_n14.mtx","synthetic_46835_46835_2371113_avg50.62695_std32.27909_diagonal_bw6994.42105_skew3.10848_neigh1.63552_cross0.75717_n14.mtx","synthetic_46835_46835_2371156_avg50.62786_std27.75732_diagonal_bw7042.80683_skew2.91089_neigh1.63466_cross0.76079_n14.mtx","synthetic_46835_46835_2371164_avg50.62803_std30.41679_diagonal_bw6939.09181_skew3.22691_neigh1.63687_cross0.75372_n14.mtx","synthetic_46835_46835_2371201_avg50.62882_std33.12436_diagonal_bw6969.84853_skew3.04908_neigh1.63376_cross0.76209_n14.mtx","synthetic_46835_46835_2371202_avg50.62885_std27.76569_diagonal_bw7070.97963_skew3.00957_neigh1.6345_cross0.76617_n14.mtx","synthetic_46835_46835_2371250_avg50.62987_std31.37317_diagonal_bw6975.62833_skew3.1675_neigh1.6373_cross0.75463_n14.mtx","synthetic_46835_46835_2371308_avg50.63111_std28.69655_diagonal_bw7047.95307_skew3.1674_neigh1.63521_cross0.76523_n14.mtx","synthetic_46835_46835_2371384_avg50.63273_std27.77434_diagonal_bw7064.33473_skew3.08826_neigh1.63349_cross0.76857_n14.mtx","synthetic_46835_46835_2371440_avg50.63393_std27.7783_diagonal_bw7114.30981_skew3.18692_neigh1.63074_cross0.77345_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_diagonal_bw5050.44975_skew3.12766_neigh1.63384_cross0.7683_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_diagonal_bw5519.37801_skew3.12766_neigh1.63425_cross0.76948_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_diagonal_bw5974.71957_skew3.12766_neigh1.63482_cross0.76832_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_diagonal_bw6289.96966_skew3.12766_neigh1.56424_cross0.88608_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_diagonal_bw6432.35471_skew3.12766_neigh1.63211_cross0.77136_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_diagonal_bw6665.6247_skew3.12766_neigh1.60147_cross0.846_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_diagonal_bw6698.93127_skew3.12766_neigh1.14856_cross0.76971_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_diagonal_bw6757.40436_skew3.12766_neigh1.25689_cross0.76811_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_diagonal_bw6780.66794_skew3.12766_neigh1.36547_cross0.76752_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_diagonal_bw6866.76471_skew3.12766_neigh1.57918_cross0.77036_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_diagonal_bw6870.58687_skew3.12766_neigh1.47124_cross0.76996_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_diagonal_bw6888.78317_skew3.12766_neigh1.63511_cross0.76873_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_diagonal_bw6998.11442_skew3.12766_neigh1.62573_cross0.79459_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_diagonal_bw7163.16152_skew3.12766_neigh1.68835_cross0.77345_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_diagonal_bw7218.82703_skew3.12766_neigh1.6394_cross0.74525_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_diagonal_bw7311.94737_skew3.12766_neigh1.63543_cross0.76939_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_diagonal_bw7364.02791_skew3.12766_neigh1.64601_cross0.69592_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_diagonal_bw7473.56761_skew3.12766_neigh1.65519_cross0.64484_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_diagonal_bw7584.44582_skew3.12766_neigh1.66065_cross0.59521_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_diagonal_bw7668.03327_skew3.12766_neigh1.66479_cross0.54423_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_diagonal_bw7752.28955_skew3.12766_neigh1.63462_cross0.76859_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_diagonal_bw8144.77457_skew3.12766_neigh1.8993_cross0.77798_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_diagonal_bw8228.79031_skew3.12766_neigh1.63299_cross0.77184_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_diagonal_bw8649.96035_skew3.12766_neigh1.63342_cross0.77078_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_diagonal_bw8734.98268_skew3.12766_neigh1.79476_cross0.77224_n14.mtx","synthetic_46835_46835_2371446_avg50.63406_std27.77501_diagonal_bw9093.75454_skew3.12766_neigh1.63511_cross0.76942_n14.mtx","synthetic_46835_46835_2371578_avg50.63687_std27.78811_diagonal_bw7158.8859_skew3.36441_neigh1.63233_cross0.77814_n14.mtx","synthetic_46835_46835_2371669_avg50.63882_std26.85583_diagonal_bw7120.5289_skew3.10752_neigh1.63199_cross0.77811_n14.mtx","synthetic_46835_46835_2371693_avg50.63933_std27.78141_diagonal_bw7136.54039_skew3.26546_neigh1.63148_cross0.77945_n14.mtx","synthetic_46835_46835_2371771_avg50.64099_std27.79721_diagonal_bw7216.51397_skew3.60102_neigh1.63057_cross0.78376_n14.mtx","synthetic_46835_46835_2371874_avg50.64319_std27.79128_diagonal_bw7207.36039_skew3.48234_neigh1.63236_cross0.78304_n14.mtx","synthetic_46835_46835_2371944_avg50.64469_std25.00834_diagonal_bw7255.44428_skew3.02806_neigh1.63083_cross0.78597_n14.mtx","synthetic_46835_46835_2372249_avg50.6512_std23.16377_diagonal_bw7332.95584_skew3.0078_neigh1.63031_cross0.79742_n14.mtx","synthetic_46835_46835_2372391_avg50.65423_std21.31556_diagonal_bw7497.52574_skew2.98782_neigh1.6247_cross0.80892_n14.mtx","synthetic_46835_46835_2372900_avg50.6651_std19.46618_diagonal_bw7594.71372_skew2.96723_neigh1.62451_cross0.818_n14.mtx","synthetic_46835_46835_2450659_avg52.32538_std27.78086_diagonal_bw7165.93817_skew3.1089_neigh1.6353_cross0.77626_n14.mtx","synthetic_46835_46835_2609003_avg55.70627_std27.78687_diagonal_bw7091.41887_skew3.03904_neigh1.63996_cross0.78555_n14.mtx","synthetic_46835_46835_2767329_avg59.08677_std27.79319_diagonal_bw7277.89561_skew3.01105_neigh1.64136_cross0.79423_n14.mtx","synthetic_46835_46835_2925939_avg62.47334_std27.79699_diagonal_bw7235.67689_skew3.00171_neigh1.64415_cross0.80009_n14.mtx","synthetic_46835_46835_3084303_avg65.85466_std27.80201_diagonal_bw7327.42293_skew2.97846_neigh1.64707_cross0.80559_n14.mtx","synthetic_4690002_4690002_14083544_avg3.00289_std1.14359_diagonal_bw5391.75836_skew291.71835_neigh0.36891_cross0.64826_n14.mtx","synthetic_4690002_4690002_18765165_avg4.0011_std0.88465_diagonal_bw4544.61296_skew312.91371_neigh0.41523_cross0.64886_n14.mtx","synthetic_4690002_4690002_18766060_avg4.00129_std0.94875_diagonal_bw4544.61433_skew312.89874_neigh0.41523_cross0.64886_n14.mtx","synthetic_4690002_4690002_18767037_avg4.0015_std1.01413_diagonal_bw4544.62177_skew312.88239_neigh0.41524_cross0.64886_n14.mtx","synthetic_4690002_4690002_18767925_avg4.00169_std1.23629_diagonal_bw4544.6141_skew406.578_neigh0.4153_cross0.64886_n14.mtx","synthetic_4690002_4690002_18768094_avg4.00172_std1.08092_diagonal_bw4544.60917_skew312.86472_neigh0.41525_cross0.64886_n14.mtx","synthetic_4690002_4690002_18768255_avg4.00176_std1.22278_diagonal_bw4544.60003_skew385.58006_neigh0.41528_cross0.64886_n14.mtx","synthetic_4690002_4690002_18768639_avg4.00184_std1.21074_diagonal_bw4544.61468_skew364.83169_neigh0.41531_cross0.64886_n14.mtx","synthetic_4690002_4690002_18769069_avg4.00193_std1.19917_diagonal_bw4544.58175_skew344.08333_neigh0.41529_cross0.64886_n14.mtx","synthetic_4690002_4690002_18769236_avg4.00197_std1.1487_diagonal_bw4544.58612_skew312.84562_neigh0.41528_cross0.64886_n14.mtx","synthetic_4690002_4690002_18769566_avg4.00204_std1.18832_diagonal_bw4544.53443_skew323.33476_neigh0.41529_cross0.64886_n14.mtx","synthetic_4690002_4690002_18769585_avg4.00204_std1.18853_diagonal_bw4544.55293_skew323.33443_neigh0.41529_cross0.64886_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_diagonal_bw3184.88357_skew312.8356_neigh0.41551_cross0.64901_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_diagonal_bw3466.39633_skew312.8356_neigh0.41544_cross0.64895_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_diagonal_bw3795.79694_skew312.8356_neigh0.41543_cross0.64893_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_diagonal_bw4074.65853_skew312.8356_neigh0.41535_cross0.64891_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_diagonal_bw4200.67402_skew312.8356_neigh0.54031_cross0.6488_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_diagonal_bw4279.94263_skew312.8356_neigh0.51216_cross0.6488_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_diagonal_bw4358.28843_skew312.8356_neigh0.48436_cross0.64888_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_diagonal_bw4407.10934_skew312.8356_neigh0.41525_cross0.64888_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_diagonal_bw4433.9589_skew312.8356_neigh0.45655_cross0.64867_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_diagonal_bw4508.13803_skew312.8356_neigh0.42901_cross0.64876_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_diagonal_bw4542.4258_skew312.8356_neigh0.41512_cross0.58418_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_diagonal_bw4543.20029_skew312.8356_neigh0.41608_cross0.84265_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_diagonal_bw4543.24698_skew312.8356_neigh0.41538_cross0.62721_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_diagonal_bw4543.90921_skew312.8356_neigh0.41541_cross0.45424_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_diagonal_bw4544.3722_skew312.8356_neigh0.41496_cross0.67021_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_diagonal_bw4544.551_skew312.8356_neigh0.41603_cross0.79985_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_diagonal_bw4544.99823_skew312.8356_neigh0.415_cross0.54079_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_diagonal_bw4545.19427_skew312.8356_neigh0.4153_cross0.49762_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_diagonal_bw4546.90824_skew312.8356_neigh0.41469_cross0.71357_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_diagonal_bw4547.16626_skew312.8356_neigh0.41515_cross0.75665_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_diagonal_bw4581.4107_skew312.8356_neigh0.40122_cross0.64877_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_diagonal_bw4652.81349_skew312.8356_neigh0.37381_cross0.64869_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_diagonal_bw4685.0085_skew312.8356_neigh0.41522_cross0.64883_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_diagonal_bw4720.29248_skew312.8356_neigh0.3463_cross0.64858_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_diagonal_bw4787.73638_skew312.8356_neigh0.31846_cross0.64869_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_diagonal_bw4854.49698_skew312.8356_neigh0.29083_cross0.64876_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_diagonal_bw5013.47671_skew312.8356_neigh0.41526_cross0.64883_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_diagonal_bw5289.58717_skew312.8356_neigh0.41524_cross0.64878_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_diagonal_bw5622.90244_skew312.8356_neigh0.41522_cross0.64879_n14.mtx","synthetic_4690002_4690002_18769835_avg4.0021_std1.18299_diagonal_bw5904.00292_skew312.8356_neigh0.41517_cross0.64878_n14.mtx","synthetic_4690002_4690002_18770121_avg4.00216_std1.17768_diagonal_bw4544.61405_skew302.33648_neigh0.41531_cross0.64886_n14.mtx","synthetic_4690002_4690002_18770136_avg4.00216_std1.17836_diagonal_bw4544.58465_skew302.5861_neigh0.41528_cross0.64886_n14.mtx","synthetic_4690002_4690002_18770757_avg4.00229_std1.16774_diagonal_bw4544.58986_skew281.58808_neigh0.41528_cross0.64886_n14.mtx","synthetic_4690002_4690002_18771502_avg4.00245_std1.15845_diagonal_bw4544.58937_skew260.58973_neigh0.41528_cross0.64886_n14.mtx","synthetic_4690002_4690002_18772478_avg4.00266_std1.15153_diagonal_bw4544.58274_skew240.08974_neigh0.41532_cross0.64886_n14.mtx","synthetic_4690002_4690002_18773573_avg4.00289_std1.14442_diagonal_bw4544.5891_skew219.34068_neigh0.4153_cross0.64886_n14.mtx","synthetic_4690002_4690002_18774622_avg4.00312_std1.42675_diagonal_bw4544.59601_skew312.75559_neigh0.41536_cross0.64886_n14.mtx","synthetic_4690002_4690002_18776169_avg4.00345_std1.497_diagonal_bw4544.58594_skew312.72973_neigh0.41537_cross0.64886_n14.mtx","synthetic_4690002_4690002_23458256_avg5.00176_std1.22307_diagonal_bw4355.44656_skew308.6911_neigh0.44325_cross0.64839_n14.mtx","synthetic_4690002_4690002_23458625_avg5.00184_std1.21051_diagonal_bw4355.4414_skew292.09232_neigh0.44326_cross0.64839_n14.mtx","synthetic_4690002_4690002_23459038_avg5.00192_std1.19839_diagonal_bw4355.43174_skew275.29363_neigh0.44326_cross0.64839_n14.mtx","synthetic_4690002_4690002_28147894_avg6.00168_std1.23536_diagonal_bw4776.66629_skew271.09045_neigh0.46196_cross0.64855_n14.mtx","synthetic_473242_473242_1892968_avg4.0_std0.0_diagonal_bw643.09685_skew0.0_neigh0.37802_cross0.99028_n14.mtx","synthetic_47342_47342_5640490_avg119.14347_std31.78033_diagonal_bw4706.12927_skew1.14028_neigh1.79368_cross0.80973_n14.mtx","synthetic_47513_47513_1853007_avg39.0_std0.0_diagonal_bw11035.73908_skew0.0_neigh1.40111_cross0.80716_n14.mtx","synthetic_47684_47684_3704607_avg77.69078_std983.84259_diagonal_bw1484.67635_skew612.76655_neigh1.33477_cross0.83815_n14.mtx","synthetic_47784_47784_2177960_avg45.57927_std31.38054_diagonal_bw1146.70877_skew2.68589_neigh1.21315_cross0.77186_n14.mtx","synthetic_47879_47879_3068257_avg64.08356_std14.01465_diagonal_bw346.42014_skew0.93497_neigh1.583_cross0.83014_n14.mtx","synthetic_48396_48396_2450061_avg50.62528_std27.78261_diagonal_bw7308.68938_skew3.14813_neigh1.63518_cross0.76979_n14.mtx","synthetic_4846335_4846335_19395478_avg4.00209_std1.18059_diagonal_bw4694.1316_skew312.83587_neigh0.41556_cross0.64854_n14.mtx","synthetic_49152_49152_1327104_avg27.0_std0.0_diagonal_bw11307.3231_skew0.0_neigh1.38197_cross0.80419_n14.mtx","synthetic_49152_49152_1474560_avg30.0_std0.0_diagonal_bw11595.80863_skew0.0_neigh1.39239_cross0.80817_n14.mtx","synthetic_49152_49152_1622016_avg33.0_std0.0_diagonal_bw11493.20247_skew0.0_neigh1.39477_cross0.80691_n14.mtx","synthetic_49152_49152_1720320_avg35.0_std0.0_diagonal_bw11394.44574_skew0.0_neigh1.39367_cross0.80647_n14.mtx","synthetic_49152_49152_1867776_avg38.0_std0.0_diagonal_bw11349.18608_skew0.0_neigh1.40287_cross0.8065_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_diagonal_bw10308.18809_skew0.0_neigh1.40356_cross0.80771_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_diagonal_bw11012.95563_skew0.0_neigh1.40324_cross0.80742_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_diagonal_bw11346.284_skew0.0_neigh1.1685_cross0.80302_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_diagonal_bw11390.72874_skew0.0_neigh0.98082_cross0.80701_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_diagonal_bw11391.50515_skew0.0_neigh1.07508_cross0.8087_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_diagonal_bw11399.78432_skew0.0_neigh1.25841_cross0.80863_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_diagonal_bw11414.96842_skew0.0_neigh1.40418_cross0.99013_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_diagonal_bw11418.68166_skew0.0_neigh1.40287_cross0.78307_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_diagonal_bw11422.21698_skew0.0_neigh1.40004_cross0.5727_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_diagonal_bw11424.32371_skew0.0_neigh1.35394_cross0.80634_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_diagonal_bw11425.76263_skew0.0_neigh1.40146_cross0.67683_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_diagonal_bw11434.58527_skew0.0_neigh1.40111_cross0.62362_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_diagonal_bw11435.72972_skew0.0_neigh1.40014_cross0.7295_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_diagonal_bw11437.37392_skew0.0_neigh1.40223_cross0.80454_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_diagonal_bw11439.83738_skew0.0_neigh1.40359_cross0.83214_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_diagonal_bw11448.35966_skew0.0_neigh1.4023_cross0.93723_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_diagonal_bw11458.2605_skew0.0_neigh1.39946_cross0.88672_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_diagonal_bw11460.26939_skew0.0_neigh1.44716_cross0.80489_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_diagonal_bw11518.20732_skew0.0_neigh1.54189_cross0.80766_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_diagonal_bw11741.60107_skew0.0_neigh1.6337_cross0.80807_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_diagonal_bw11774.41903_skew0.0_neigh1.40065_cross0.80951_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_diagonal_bw12457.04529_skew0.0_neigh1.40302_cross0.81002_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_diagonal_bw12492.34208_skew0.0_neigh1.72608_cross0.80667_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_diagonal_bw13195.13298_skew0.0_neigh1.39937_cross0.80881_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_diagonal_bw13899.08816_skew0.0_neigh1.39772_cross0.80832_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_diagonal_bw14638.79781_skew0.0_neigh1.39901_cross0.80937_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_diagonal_bw17395.89585_skew0.0_neigh1.81611_cross0.81026_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_diagonal_bw8111.08537_skew0.0_neigh1.40272_cross0.80591_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_diagonal_bw8847.37626_skew0.0_neigh1.40198_cross0.80857_n14.mtx","synthetic_49152_49152_1916928_avg39.0_std0.0_diagonal_bw9576.764_skew0.0_neigh1.40409_cross0.80825_n14.mtx","synthetic_49152_49152_1966080_avg40.0_std0.0_diagonal_bw11481.6866_skew0.0_neigh1.4051_cross0.80332_n14.mtx","synthetic_49152_49152_2113536_avg43.0_std0.0_diagonal_bw11425.25747_skew0.0_neigh1.40405_cross0.80294_n14.mtx","synthetic_49152_49152_2260992_avg46.0_std0.0_diagonal_bw11375.92244_skew0.0_neigh1.407_cross0.80755_n14.mtx","synthetic_49152_49152_2359296_avg48.0_std0.0_diagonal_bw11455.144_skew0.0_neigh1.40962_cross0.8078_n14.mtx","synthetic_49152_49152_2506752_avg51.0_std0.0_diagonal_bw11427.36776_skew0.0_neigh1.41299_cross0.80878_n14.mtx","synthetic_49161_49161_5521770_avg112.32013_std88.82977_diagonal_bw19423.22797_skew3.82549_neigh1.89372_cross0.64143_n14.mtx","synthetic_50367_50367_2296192_avg45.58922_std31.37796_diagonal_bw1208.68088_skew2.68508_neigh1.2113_cross0.76966_n14.mtx","synthetic_50790_50790_1980810_avg39.0_std0.0_diagonal_bw11784.56686_skew0.0_neigh1.40283_cross0.80853_n14.mtx","synthetic_508297_508297_2033188_avg4.0_std0.0_diagonal_bw694.94548_skew0.0_neigh0.36737_cross0.99098_n14.mtx","synthetic_51518_51518_2608832_avg50.63923_std27.762_diagonal_bw7791.12322_skew3.14698_neigh1.63399_cross0.76903_n14.mtx","synthetic_5159002_5159002_20646759_avg4.00208_std1.17597_diagonal_bw4999.05211_skew312.8365_neigh0.41586_cross0.64852_n14.mtx","synthetic_51970_51970_5840112_avg112.37468_std88.844_diagonal_bw20596.85007_skew3.82315_neigh1.89334_cross0.64183_n14.mtx","synthetic_52042_52042_3335826_avg64.09873_std14.0_diagonal_bw375.19427_skew0.93452_neigh1.57868_cross0.8294_n14.mtx","synthetic_52226_52226_4093416_avg78.37889_std1019.1394_diagonal_bw1620.53483_skew665.32736_neigh1.34078_cross0.83814_n14.mtx","synthetic_525825_525825_1577449_avg2.99995_std0.00703_diagonal_bw858.65423_skew2e-05_neigh0.32898_cross0.99111_n14.mtx","synthetic_525825_525825_1577475_avg3.0_std0.0_diagonal_bw858.11898_skew0.0_neigh0.3297_cross0.99112_n14.mtx","synthetic_525825_525825_1583911_avg3.01224_std0.10995_diagonal_bw794.94425_skew0.32792_neigh0.26356_cross0.98852_n14.mtx","synthetic_525825_525825_2046881_avg3.8927_std0.30949_diagonal_bw644.63048_skew0.02756_neigh0.14022_cross0.96875_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_diagonal_bw505.65007_skew0.0_neigh0.37404_cross0.99123_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_diagonal_bw554.71674_skew0.0_neigh0.37192_cross0.99123_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_diagonal_bw606.72183_skew0.0_neigh0.37221_cross0.99122_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_diagonal_bw644.28962_skew0.0_neigh0.37132_cross0.99121_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_diagonal_bw670.65649_skew0.0_neigh0.48388_cross0.99123_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_diagonal_bw683.48363_skew0.0_neigh0.45899_cross0.99121_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_diagonal_bw695.19178_skew0.0_neigh0.43396_cross0.9912_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_diagonal_bw702.15381_skew0.0_neigh0.37156_cross0.99122_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_diagonal_bw705.67664_skew0.0_neigh0.40817_cross0.99123_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_diagonal_bw714.97854_skew0.0_neigh0.38388_cross0.99124_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_diagonal_bw717.19496_skew0.0_neigh0.37708_cross0.95839_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_diagonal_bw718.53995_skew0.0_neigh0.37405_cross0.89258_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_diagonal_bw719.73706_skew0.0_neigh0.37722_cross0.76298_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_diagonal_bw719.84723_skew0.0_neigh0.37147_cross0.99224_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_diagonal_bw720.36711_skew0.0_neigh0.37694_cross0.69678_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_diagonal_bw720.4043_skew0.0_neigh0.37129_cross0.99122_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_diagonal_bw720.62767_skew0.0_neigh0.3775_cross0.8279_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_diagonal_bw726.02105_skew0.0_neigh0.35715_cross0.99121_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_diagonal_bw737.10622_skew0.0_neigh0.37107_cross0.99122_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_diagonal_bw737.13597_skew0.0_neigh0.33298_cross0.9912_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_diagonal_bw744.1915_skew0.0_neigh0.30973_cross0.99122_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_diagonal_bw756.74982_skew0.0_neigh0.28172_cross0.99121_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_diagonal_bw766.27818_skew0.0_neigh0.25988_cross0.99121_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_diagonal_bw792.21338_skew0.0_neigh0.37115_cross0.99122_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_diagonal_bw832.70023_skew0.0_neigh0.371_cross0.9912_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_diagonal_bw890.5806_skew0.0_neigh0.37102_cross0.99121_n14.mtx","synthetic_525825_525825_2103300_avg4.0_std0.0_diagonal_bw939.37529_skew0.0_neigh0.37252_cross0.9912_n14.mtx","synthetic_525825_525825_2146618_avg4.08238_std0.27494_diagonal_bw492.86623_skew0.22478_neigh0.18979_cross0.97725_n14.mtx","synthetic_525825_525825_2619767_avg4.9822_std0.13221_diagonal_bw661.36508_skew0.00357_neigh0.31619_cross0.98832_n14.mtx","synthetic_525825_525825_2629125_avg5.0_std0.0_diagonal_bw685.10273_skew0.0_neigh0.39925_cross0.99119_n14.mtx","synthetic_525825_525825_2629143_avg5.00003_std0.00585_diagonal_bw685.04151_skew0.19999_neigh0.39906_cross0.99119_n14.mtx","synthetic_54067_54067_2108613_avg39.0_std0.0_diagonal_bw12526.00168_skew0.0_neigh1.40239_cross0.80938_n14.mtx","synthetic_543352_543352_2173408_avg4.0_std0.0_diagonal_bw737.33837_skew0.0_neigh0.38232_cross0.99144_n14.mtx","synthetic_54640_54640_2767687_avg50.65313_std27.75459_diagonal_bw8285.22183_skew3.14584_neigh1.63356_cross0.77112_n14.mtx","synthetic_5471669_5471669_21898040_avg4.00208_std1.17221_diagonal_bw5301.34866_skew312.83705_neigh0.41566_cross0.6484_n14.mtx","synthetic_54779_54779_6163317_avg112.5124_std88.83086_diagonal_bw21771.99144_skew3.81725_neigh1.89359_cross0.64245_n14.mtx","synthetic_56205_56205_3603711_avg64.11727_std13.99803_diagonal_bw403.30752_skew0.93396_neigh1.57631_cross0.8294_n14.mtx","synthetic_56767_56767_4479663_avg78.91315_std1046.34011_diagonal_bw1754.34908_skew718.36043_neigh1.3421_cross0.83778_n14.mtx","synthetic_57344_57344_2236416_avg39.0_std0.0_diagonal_bw13320.73858_skew0.0_neigh1.39839_cross0.80795_n14.mtx","synthetic_57763_57763_2926741_avg50.66809_std27.74629_diagonal_bw8733.73693_skew3.16436_neigh1.63025_cross0.77273_n14.mtx","synthetic_578407_578407_2313628_avg4.0_std0.0_diagonal_bw786.09292_skew0.0_neigh0.37349_cross0.99188_n14.mtx","synthetic_5784335_5784335_23149312_avg4.00207_std1.16847_diagonal_bw5604.48836_skew312.83761_neigh0.4158_cross0.6484_n14.mtx","synthetic_58333_58333_4204101_avg72.07071_std19.00931_diagonal_bw3296.83076_skew1.13679_neigh1.65316_cross0.77161_n14.mtx","synthetic_60369_60369_3871327_avg64.12773_std13.99319_diagonal_bw432.23492_skew0.93364_neigh1.57393_cross0.82891_n14.mtx","synthetic_603864_603864_13450351_avg22.27381_std30.02405_diagonal_bw145119.2978_skew313.58473_neigh1.25727_cross0.75786_n14.mtx","synthetic_60620_60620_2364180_avg39.0_std0.0_diagonal_bw14073.89657_skew0.0_neigh1.40012_cross0.80617_n14.mtx","synthetic_60885_60885_3084540_avg50.66174_std27.74397_diagonal_bw9209.10073_skew3.16488_neigh1.63202_cross0.77149_n14.mtx","synthetic_6097002_6097002_24400590_avg4.00206_std1.16523_diagonal_bw5909.43124_skew312.83809_neigh0.41578_cross0.64825_n14.mtx","synthetic_61308_61308_4858098_avg79.24085_std1065.00271_diagonal_bw1887.76817_skew772.69186_neigh1.34799_cross0.83808_n14.mtx","synthetic_613462_613462_2453848_avg4.0_std0.0_diagonal_bw842.69066_skew0.0_neigh0.3823_cross0.9922_n14.mtx","synthetic_62451_62451_2802494_avg44.87509_std13.98221_diagonal_bw436.8282_skew1.33983_neigh1.53945_cross0.78907_n14.mtx","synthetic_62451_62451_3069539_avg49.15116_std13.98917_diagonal_bw444.92629_skew1.21765_neigh1.54779_cross0.80067_n14.mtx","synthetic_62451_62451_3336828_avg53.43114_std13.99111_diagonal_bw443.55866_skew1.11487_neigh1.55347_cross0.81011_n14.mtx","synthetic_62451_62451_3603802_avg57.70607_std13.99012_diagonal_bw443.04206_skew1.04485_neigh1.56426_cross0.8182_n14.mtx","synthetic_62451_62451_3870981_avg61.98429_std13.98816_diagonal_bw442.47014_skew0.96824_neigh1.56874_cross0.82542_n14.mtx","synthetic_62451_62451_4003815_avg64.1113_std18.18532_diagonal_bw439.05052_skew1.2149_neigh1.57718_cross0.80166_n14.mtx","synthetic_62451_62451_4004004_avg64.11433_std17.25509_diagonal_bw440.54065_skew1.1524_neigh1.57552_cross0.80816_n14.mtx","synthetic_62451_62451_4004101_avg64.11588_std16.32075_diagonal_bw441.99681_skew1.08997_neigh1.5736_cross0.81519_n14.mtx","synthetic_62451_62451_4004374_avg64.12025_std15.38744_diagonal_bw441.76455_skew1.02744_neigh1.57521_cross0.82106_n14.mtx","synthetic_62451_62451_4004540_avg64.12291_std14.45551_diagonal_bw446.48393_skew0.96498_neigh1.57187_cross0.8254_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_diagonal_bw318.44834_skew0.93375_neigh1.5857_cross0.8319_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_diagonal_bw347.21382_skew0.93375_neigh1.58282_cross0.83047_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_diagonal_bw375.5891_skew0.93375_neigh1.5788_cross0.82967_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_diagonal_bw403.32614_skew0.93375_neigh1.57688_cross0.82914_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_diagonal_bw428.19497_skew0.93375_neigh1.56047_cross0.89117_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_diagonal_bw431.64225_skew0.93375_neigh1.17959_cross0.83144_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_diagonal_bw432.20602_skew0.93375_neigh1.57402_cross0.82868_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_diagonal_bw432.28425_skew0.93375_neigh1.26448_cross0.83122_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_diagonal_bw434.25807_skew0.93375_neigh1.3496_cross0.82903_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_diagonal_bw435.77275_skew0.93375_neigh1.43957_cross0.82958_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_diagonal_bw436.2092_skew0.93375_neigh1.56891_cross0.85105_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_diagonal_bw441.89184_skew0.93375_neigh1.52756_cross0.82773_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_diagonal_bw446.0811_skew0.93375_neigh1.57318_cross0.82708_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_diagonal_bw448.47362_skew0.93375_neigh1.61679_cross0.82621_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_diagonal_bw455.82892_skew0.93375_neigh1.57514_cross0.8052_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_diagonal_bw460.20173_skew0.93375_neigh1.57143_cross0.82713_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_diagonal_bw471.01044_skew0.93375_neigh1.58044_cross0.75754_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_diagonal_bw474.11936_skew0.93375_neigh1.70896_cross0.82572_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_diagonal_bw482.92841_skew0.93375_neigh1.58602_cross0.71177_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_diagonal_bw488.82353_skew0.93375_neigh1.5698_cross0.82576_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_diagonal_bw492.24525_skew0.93375_neigh1.58863_cross0.66769_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_diagonal_bw501.27634_skew0.93375_neigh1.592_cross0.62297_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_diagonal_bw518.7649_skew0.93375_neigh1.56528_cross0.82577_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_diagonal_bw534.51368_skew0.93375_neigh1.80272_cross0.82363_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_diagonal_bw545.14101_skew0.93375_neigh1.56484_cross0.82581_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_diagonal_bw573.04011_skew0.93375_neigh1.56402_cross0.82536_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_diagonal_bw655.89393_skew0.93375_neigh1.90447_cross0.82019_n14.mtx","synthetic_62451_62451_4004623_avg64.12424_std13.99039_diagonal_bw688.45913_skew0.93375_neigh1.89805_cross0.8202_n14.mtx","synthetic_62451_62451_4004768_avg64.12656_std12.59183_diagonal_bw449.42601_skew0.84011_neigh1.56907_cross0.83623_n14.mtx","synthetic_62451_62451_4004839_avg64.1277_std13.52372_diagonal_bw447.77336_skew0.90245_neigh1.56888_cross0.83178_n14.mtx","synthetic_62451_62451_4005045_avg64.131_std11.65916_diagonal_bw451.49143_skew0.77761_neigh1.56649_cross0.83952_n14.mtx","synthetic_62451_62451_4005321_avg64.13542_std10.72592_diagonal_bw453.45016_skew0.71512_neigh1.56305_cross0.84869_n14.mtx","synthetic_62451_62451_4005505_avg64.13836_std9.79513_diagonal_bw456.12735_skew0.65268_neigh1.56103_cross0.85395_n14.mtx","synthetic_62451_62451_4138184_avg66.26289_std13.99067_diagonal_bw443.34518_skew0.90152_neigh1.57487_cross0.83169_n14.mtx","synthetic_62451_62451_4405420_avg70.54202_std13.98985_diagonal_bw444.61993_skew0.85705_neigh1.57908_cross0.83695_n14.mtx","synthetic_62451_62451_4672484_avg74.8184_std13.9889_diagonal_bw448.7849_skew0.80437_neigh1.58559_cross0.8421_n14.mtx","synthetic_62451_62451_4939666_avg79.09667_std13.99063_diagonal_bw451.01291_skew0.75734_neigh1.58874_cross0.84589_n14.mtx","synthetic_62451_62451_5206981_avg83.37706_std13.99111_diagonal_bw451.49187_skew0.7151_neigh1.5926_cross0.8518_n14.mtx","synthetic_63889_63889_4604376_avg72.06837_std18.98718_diagonal_bw3617.55279_skew1.13686_neigh1.65392_cross0.76978_n14.mtx","synthetic_63897_63897_2491983_avg39.0_std0.0_diagonal_bw14829.98433_skew0.0_neigh1.40105_cross0.80703_n14.mtx","synthetic_64532_64532_4137790_avg64.11997_std13.98737_diagonal_bw460.49369_skew0.93387_neigh1.57096_cross0.82708_n14.mtx","synthetic_648517_648517_2594068_avg4.0_std0.0_diagonal_bw886.29581_skew0.0_neigh0.37472_cross0.99258_n14.mtx","synthetic_65850_65850_5226388_avg79.36808_std1075.21227_diagonal_bw2024.76418_skew828.67864_neigh1.34353_cross0.83818_n14.mtx","synthetic_661375_661375_14730847_avg22.27306_std29.96346_diagonal_bw159071.45445_skew313.59526_neigh1.25597_cross0.75815_n14.mtx","synthetic_666542_666542_32559308_avg48.8481_std11.96425_diagonal_bw114344.90623_skew1.12905_neigh1.69253_cross0.80431_n14.mtx","synthetic_68121_68121_3787555_avg55.6004_std1068.24938_diagonal_bw1718.51627_skew856.74559_neigh1.78059_cross0.84932_n14.mtx","synthetic_68121_68121_4147401_avg60.88286_std1069.9982_diagonal_bw1926.13858_skew857.05765_neigh1.62569_cross0.84303_n14.mtx","synthetic_68121_68121_4507272_avg66.16568_std1071.85693_diagonal_bw2010.64137_skew857.28489_neigh1.51921_cross0.84027_n14.mtx","synthetic_68121_68121_4867182_avg71.44907_std1073.8353_diagonal_bw2058.69186_skew857.47165_neigh1.43412_cross0.83891_n14.mtx","synthetic_68121_68121_5227217_avg76.7343_std1075.93952_diagonal_bw2086.64904_skew857.61216_neigh1.36941_cross0.83829_n14.mtx","synthetic_68121_68121_5356033_avg78.62528_std1024.59921_diagonal_bw2111.50808_skew865.40068_neigh1.23427_cross0.83732_n14.mtx","synthetic_68121_68121_5371179_avg78.84762_std1041.23556_diagonal_bw2110.12014_skew862.95755_neigh1.25133_cross0.8377_n14.mtx","synthetic_68121_68121_5385572_avg79.05891_std1056.36091_diagonal_bw2104.37146_skew860.64861_neigh1.27611_cross0.83748_n14.mtx","synthetic_68121_68121_5397801_avg79.23843_std1068.36979_diagonal_bw2101.31551_skew858.6965_neigh1.30166_cross0.83769_n14.mtx","synthetic_68121_68121_5398333_avg79.24624_std1068.28806_diagonal_bw2041.94523_skew601.36298_neigh1.52107_cross0.83991_n14.mtx","synthetic_68121_68121_5400082_avg79.27191_std1070.0369_diagonal_bw2059.09281_skew658.4265_neigh1.46808_cross0.83939_n14.mtx","synthetic_68121_68121_5402340_avg79.30506_std1071.88875_diagonal_bw2075.75188_skew715.38556_neigh1.42245_cross0.83852_n14.mtx","synthetic_68121_68121_5404162_avg79.33181_std1073.85709_diagonal_bw2085.38637_skew772.35942_neigh1.3883_cross0.8379_n14.mtx","synthetic_68121_68121_5405884_avg79.35709_std1075.92117_diagonal_bw2096.53019_skew857.41106_neigh1.32589_cross0.83788_n14.mtx","synthetic_68121_68121_5405960_avg79.3582_std834.61171_diagonal_bw2119.37251_skew857.399_neigh1.16523_cross0.8395_n14.mtx","synthetic_68121_68121_5406317_avg79.36344_std1075.95011_diagonal_bw2092.50296_skew829.23113_neigh1.35417_cross0.83799_n14.mtx","synthetic_68121_68121_5406423_avg79.365_std903.5954_diagonal_bw2114.95495_skew857.32548_neigh1.21021_cross0.83745_n14.mtx","synthetic_68121_68121_5406595_avg79.36752_std972.82101_diagonal_bw2111.91293_skew857.29818_neigh1.25842_cross0.83751_n14.mtx","synthetic_68121_68121_5406754_avg79.36986_std765.8736_diagonal_bw2124.58844_skew857.27294_neigh1.12605_cross0.86678_n14.mtx","synthetic_68121_68121_5407048_avg79.37417_std1042.23736_diagonal_bw2101.43831_skew857.22627_neigh1.31378_cross0.83792_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_diagonal_bw1491.43754_skew857.20278_neigh1.35156_cross0.83798_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_diagonal_bw1624.0197_skew857.20278_neigh1.34962_cross0.8382_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_diagonal_bw1758.02102_skew857.20278_neigh1.3463_cross0.83749_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_diagonal_bw1895.79908_skew857.20278_neigh1.34593_cross0.83806_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_diagonal_bw2025.50456_skew857.20278_neigh1.3433_cross0.83803_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_diagonal_bw2082.54704_skew857.20278_neigh1.34143_cross0.84309_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_diagonal_bw2086.37071_skew857.20278_neigh1.41767_cross0.83887_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_diagonal_bw2086.77817_skew857.20278_neigh1.47784_cross0.83958_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_diagonal_bw2091.20798_skew857.20278_neigh1.38952_cross0.83839_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_diagonal_bw2093.83269_skew857.20278_neigh1.44388_cross0.83897_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_diagonal_bw2093.9416_skew857.20278_neigh1.35558_cross0.83786_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_diagonal_bw2094.95959_skew857.20278_neigh1.29648_cross0.83766_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_diagonal_bw2094.9781_skew857.20278_neigh1.32585_cross0.83766_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_diagonal_bw2095.49089_skew857.20278_neigh1.23838_cross0.83723_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_diagonal_bw2096.11441_skew857.20278_neigh1.27082_cross0.83753_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_diagonal_bw2101.47777_skew857.20278_neigh1.21125_cross0.83652_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_diagonal_bw2157.03003_skew857.20278_neigh1.34041_cross0.83822_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_diagonal_bw2169.50548_skew857.20278_neigh1.34139_cross0.8107_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_diagonal_bw2293.06146_skew857.20278_neigh1.33571_cross0.83819_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_diagonal_bw2300.29781_skew857.20278_neigh1.33945_cross0.75919_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_diagonal_bw2404.30044_skew857.20278_neigh1.34229_cross0.7026_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_diagonal_bw2427.21701_skew857.20278_neigh1.33918_cross0.83764_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_diagonal_bw2485.72726_skew857.20278_neigh1.34463_cross0.6511_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_diagonal_bw2552.48722_skew857.20278_neigh1.34607_cross0.59496_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_diagonal_bw2557.58023_skew857.20278_neigh1.34055_cross0.83779_n14.mtx","synthetic_68121_68121_5407196_avg79.37635_std1077.00512_diagonal_bw2689.25582_skew857.20278_neigh1.33474_cross0.83821_n14.mtx","synthetic_68121_68121_5407447_avg79.38003_std1111.80732_diagonal_bw2088.98344_skew857.16294_neigh1.36649_cross0.83852_n14.mtx","synthetic_68121_68121_5407867_avg79.3862_std1181.49691_diagonal_bw2075.59383_skew857.0963_neigh1.43167_cross0.83884_n14.mtx","synthetic_68121_68121_5408409_avg79.39415_std1251.28152_diagonal_bw2053.19145_skew857.0103_neigh1.49371_cross0.83947_n14.mtx","synthetic_68121_68121_5409111_avg79.40446_std1321.14786_diagonal_bw2024.25072_skew856.89895_neigh1.56267_cross0.84086_n14.mtx","synthetic_68121_68121_5409514_avg79.41037_std1391.07361_diagonal_bw1990.12473_skew856.83504_neigh1.64055_cross0.84129_n14.mtx","synthetic_68121_68121_5584961_avg81.98589_std1075.92069_diagonal_bw2103.42705_skew829.88685_neigh1.31388_cross0.83785_n14.mtx","synthetic_68121_68121_5934842_avg87.12206_std1068.36291_diagonal_bw2115.43957_skew780.90298_neigh1.27178_cross0.83756_n14.mtx","synthetic_68121_68121_6280428_avg92.19518_std1056.3285_diagonal_bw2124.28281_skew737.87809_neigh1.23335_cross0.83743_n14.mtx","synthetic_68121_68121_6623921_avg97.23758_std1041.17147_diagonal_bw2129.92795_skew699.5625_neigh1.20508_cross0.83709_n14.mtx","synthetic_68121_68121_6966795_avg102.27089_std1024.4946_diagonal_bw2136.61749_skew665.08399_neigh1.18226_cross0.8378_n14.mtx","synthetic_683572_683572_2734288_avg4.0_std0.0_diagonal_bw924.57479_skew0.0_neigh0.37964_cross0.9929_n14.mtx","synthetic_68696_68696_4403097_avg64.09539_std13.99668_diagonal_bw488.31048_skew0.93462_neigh1.5701_cross0.82639_n14.mtx","synthetic_690692_690692_50143771_avg72.59932_std15.84383_diagonal_bw10332.89385_skew0.96972_neigh1.69604_cross0.81876_n14.mtx","synthetic_69445_69445_5001284_avg72.01791_std19.00309_diagonal_bw3910.75892_skew1.13836_neigh1.65289_cross0.77119_n14.mtx","synthetic_700003_700003_2220092_avg3.17155_std25.65674_diagonal_bw84360.35744_skew1481.87283_neigh0.14269_cross0.90247_n14.mtx","synthetic_70391_70391_5585080_avg79.34367_std1076.52341_diagonal_bw2157.78507_skew858.03517_neigh1.33819_cross0.83827_n14.mtx","synthetic_718886_718886_16011537_avg22.27271_std29.91357_diagonal_bw172575.43963_skew313.60029_neigh1.25733_cross0.76047_n14.mtx","synthetic_72859_72859_4668546_avg64.07645_std14.00516_diagonal_bw518.58045_skew0.93519_neigh1.56556_cross0.82561_n14.mtx","synthetic_730022_730022_35670999_avg48.86291_std11.95289_diagonal_bw125143.89165_skew1.14887_neigh1.69318_cross0.80372_n14.mtx","synthetic_74933_74933_5942603_avg79.30555_std1075.58283_diagonal_bw2295.08486_skew858.44801_neigh1.33914_cross0.83797_n14.mtx","synthetic_75000_75000_5399893_avg71.99857_std19.01052_diagonal_bw4226.28011_skew1.13893_neigh1.65498_cross0.76971_n14.mtx","synthetic_756472_756472_54943929_avg72.63181_std15.80449_diagonal_bw11306.62304_skew1.03767_neigh1.69544_cross0.81974_n14.mtx","synthetic_766670_766670_2431595_avg3.17163_std25.62988_diagonal_bw92517.8455_skew1481.83288_neigh0.14291_cross0.90344_n14.mtx","synthetic_77022_77022_4936326_avg64.08982_std14.01488_diagonal_bw546.71648_skew0.93478_neigh1.56521_cross0.82414_n14.mtx","synthetic_776397_776397_17292444_avg22.27268_std29.86978_diagonal_bw186487.71815_skew313.60063_neigh1.25606_cross0.75904_n14.mtx","synthetic_793502_793502_38791634_avg48.88662_std11.94806_diagonal_bw136182.3121_skew1.39329_neigh1.69181_cross0.80518_n14.mtx","synthetic_79474_79474_6302859_avg79.30718_std1074.75192_diagonal_bw2429.79547_skew858.43036_neigh1.33761_cross0.83764_n14.mtx","synthetic_80556_80556_5802157_avg72.02638_std19.02772_diagonal_bw4548.81679_skew1.13811_neigh1.65123_cross0.77098_n14.mtx","synthetic_81186_81186_5203811_avg64.09739_std14.01954_diagonal_bw573.41229_skew0.93456_neigh1.56394_cross0.82507_n14.mtx","synthetic_822252_822252_59710654_avg72.61844_std15.79674_diagonal_bw12289.27629_skew1.02428_neigh1.69505_cross0.82015_n14.mtx","synthetic_833337_833337_2642605_avg3.17111_std25.60679_diagonal_bw100178.12772_skew1482.07595_neigh0.14558_cross0.90327_n14.mtx","synthetic_83334_83334_4201657_avg50.41948_std18.95017_diagonal_bw4709.6572_skew1.61804_neigh1.63244_cross0.73261_n14.mtx","synthetic_83334_83334_4601152_avg55.21338_std18.98799_diagonal_bw4646.03415_skew1.48128_neigh1.63903_cross0.74386_n14.mtx","synthetic_83334_83334_5001407_avg60.0164_std19.00589_diagonal_bw4595.50832_skew1.36602_neigh1.64466_cross0.7532_n14.mtx","synthetic_83334_83334_5402033_avg64.82388_std19.01381_diagonal_bw4698.8203_skew1.25226_neigh1.64878_cross0.75926_n14.mtx","synthetic_83334_83334_5802662_avg69.63139_std19.01513_diagonal_bw4668.44682_skew1.16856_neigh1.65019_cross0.76698_n14.mtx","synthetic_83334_83334_6001725_avg72.02012_std24.68331_diagonal_bw4597.09978_skew1.47153_neigh1.65763_cross0.74295_n14.mtx","synthetic_83334_83334_6001767_avg72.02063_std23.43414_diagonal_bw4605.04571_skew1.40209_neigh1.65672_cross0.74782_n14.mtx","synthetic_83334_83334_6001894_avg72.02215_std22.17621_diagonal_bw4646.80877_skew1.31873_neigh1.65465_cross0.75393_n14.mtx","synthetic_83334_83334_6002259_avg72.02653_std20.9136_diagonal_bw4656.17762_skew1.24917_neigh1.65449_cross0.76067_n14.mtx","synthetic_83334_83334_6002747_avg72.03239_std19.64906_diagonal_bw4673.90798_skew1.16569_neigh1.65312_cross0.76886_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_diagonal_bw3305.20832_skew1.13786_neigh1.65315_cross0.77231_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_diagonal_bw3620.90018_skew1.13786_neigh1.65349_cross0.77196_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_diagonal_bw3937.58802_skew1.13786_neigh1.65156_cross0.77221_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_diagonal_bw4166.4266_skew1.13786_neigh1.61515_cross0.87118_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_diagonal_bw4233.14868_skew1.13786_neigh1.65482_cross0.76951_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_diagonal_bw4319.26886_skew1.13786_neigh1.63126_cross0.84629_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_diagonal_bw4526.66129_skew1.13786_neigh1.27337_cross0.77072_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_diagonal_bw4530.37209_skew1.13786_neigh1.16582_cross0.76816_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_diagonal_bw4534.10717_skew1.13786_neigh1.38187_cross0.76916_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_diagonal_bw4547.8539_skew1.13786_neigh1.65373_cross0.7709_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_diagonal_bw4579.65029_skew1.13786_neigh1.48964_cross0.77146_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_diagonal_bw4585.62608_skew1.13786_neigh1.65019_cross0.79589_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_diagonal_bw4637.47048_skew1.13786_neigh1.59841_cross0.77139_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_diagonal_bw4689.47064_skew1.13786_neigh1.65378_cross0.77101_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_diagonal_bw4780.79023_skew1.13786_neigh1.65879_cross0.74352_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_diagonal_bw4789.06709_skew1.13786_neigh1.70747_cross0.77176_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_diagonal_bw4857.24227_skew1.13786_neigh1.65353_cross0.76955_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_diagonal_bw4925.35613_skew1.13786_neigh1.6643_cross0.6948_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_diagonal_bw5058.46251_skew1.13786_neigh1.66965_cross0.64539_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_diagonal_bw5147.50469_skew1.13786_neigh1.67236_cross0.59361_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_diagonal_bw5154.62121_skew1.13786_neigh1.65272_cross0.77108_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_diagonal_bw5233.28398_skew1.13786_neigh1.67487_cross0.54727_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_diagonal_bw5476.98466_skew1.13786_neigh1.65336_cross0.76948_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_diagonal_bw5507.02461_skew1.13786_neigh1.81577_cross0.77147_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_diagonal_bw5678.54487_skew1.13786_neigh1.92809_cross0.77038_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_diagonal_bw5771.62843_skew1.13786_neigh1.65209_cross0.77094_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_diagonal_bw5838.52381_skew1.13786_neigh1.92627_cross0.77149_n14.mtx","synthetic_83334_83334_6002931_avg72.0346_std19.01655_diagonal_bw6077.07144_skew1.13786_neigh1.65366_cross0.76991_n14.mtx","synthetic_83334_83334_6003209_avg72.03793_std18.38192_diagonal_bw4707.48488_skew1.09612_neigh1.65203_cross0.77498_n14.mtx","synthetic_83334_83334_6003696_avg72.04378_std17.11472_diagonal_bw4728.66163_skew1.01267_neigh1.65174_cross0.7812_n14.mtx","synthetic_83334_83334_6004305_avg72.05108_std15.84803_diagonal_bw4777.9355_skew0.94307_neigh1.65015_cross0.78619_n14.mtx","synthetic_83334_83334_6004738_avg72.05628_std14.57979_diagonal_bw4790.14851_skew0.87354_neigh1.64947_cross0.79306_n14.mtx","synthetic_83334_83334_6005175_avg72.06152_std13.31259_diagonal_bw4831.11382_skew0.79014_neigh1.64887_cross0.7998_n14.mtx","synthetic_83334_83334_6203256_avg74.43848_std19.01659_diagonal_bw4654.6223_skew1.09569_neigh1.65385_cross0.77339_n14.mtx","synthetic_83334_83334_6603858_avg79.24566_std19.01545_diagonal_bw4699.25025_skew1.03166_neigh1.65768_cross0.78094_n14.mtx","synthetic_83334_83334_7004670_avg84.05537_std19.01725_diagonal_bw4679.09844_skew0.97489_neigh1.65994_cross0.78535_n14.mtx","synthetic_83334_83334_7405486_avg88.86512_std19.01689_diagonal_bw4672.23001_skew0.91301_neigh1.66199_cross0.79152_n14.mtx","synthetic_83334_83334_7806125_avg93.67275_std19.01502_diagonal_bw4727.18715_skew0.86821_neigh1.66453_cross0.79468_n14.mtx","synthetic_833908_833908_18572841_avg22.27205_std29.833_diagonal_bw200221.84865_skew313.60956_neigh1.2563_cross0.76034_n14.mtx","synthetic_84015_84015_6661114_avg79.28482_std1074.00684_diagonal_bw2558.388_skew858.67278_neigh1.33638_cross0.83771_n14.mtx","synthetic_84834_84834_1835279_avg21.63377_std13.7861_diagonal_bw39708.14271_skew4.03842_neigh1.00596_cross0.57165_n14.mtx","synthetic_856982_856982_41861338_avg48.84739_std11.94057_diagonal_bw146875.5352_skew1.3338_neigh1.69178_cross0.8051_n14.mtx","synthetic_86111_86111_6203103_avg72.03613_std19.01524_diagonal_bw4859.13722_skew1.13782_neigh1.65351_cross0.76961_n14.mtx","synthetic_862664_862664_13443097_avg15.58324_std29.57178_diagonal_bw205914.8262_skew313.76132_neigh1.19467_cross0.75677_n14.mtx","synthetic_862664_862664_14783705_avg17.13727_std29.61764_diagonal_bw200544.4475_skew312.46885_neigh1.21062_cross0.75788_n14.mtx","synthetic_862664_862664_15984607_avg18.52935_std29.67134_diagonal_bw204031.16331_skew314.12161_neigh1.21772_cross0.75849_n14.mtx","synthetic_862664_862664_17373620_avg20.1395_std29.72434_diagonal_bw201184.47121_skew312.11605_neigh1.23462_cross0.75917_n14.mtx","synthetic_862664_862664_18524582_avg21.47369_std29.78706_diagonal_bw202931.99997_skew314.40918_neigh1.23555_cross0.75791_n14.mtx","synthetic_862664_862664_19089435_avg22.12847_std21.22646_diagonal_bw207932.34557_skew315.65089_neigh1.26294_cross0.76006_n14.mtx","synthetic_862664_862664_19114072_avg22.15703_std23.12053_diagonal_bw207837.85303_skew315.24275_neigh1.26195_cross0.76014_n14.mtx","synthetic_862664_862664_19142097_avg22.18952_std25.02501_diagonal_bw207559.16437_skew314.77975_neigh1.2614_cross0.76033_n14.mtx","synthetic_862664_862664_19162894_avg22.21362_std30.15004_diagonal_bw207436.8044_skew408.47843_neigh1.26104_cross0.76006_n14.mtx","synthetic_862664_862664_19171989_avg22.22417_std26.93718_diagonal_bw207418.24466_skew314.2874_neigh1.26045_cross0.75991_n14.mtx","synthetic_862664_862664_19173147_avg22.22551_std30.06905_diagonal_bw207404.3214_skew387.38255_neigh1.26037_cross0.75992_n14.mtx","synthetic_862664_862664_19184120_avg22.23823_std29.99101_diagonal_bw207352.46946_skew366.25046_neigh1.25968_cross0.76002_n14.mtx","synthetic_862664_862664_19195719_avg22.25168_std29.91773_diagonal_bw207364.02997_skew345.17619_neigh1.25896_cross0.7599_n14.mtx","synthetic_862664_862664_19200670_avg22.25741_std28.85544_diagonal_bw207358.48533_skew313.81644_neigh1.25804_cross0.75992_n14.mtx","synthetic_862664_862664_19207356_avg22.26516_std29.84926_diagonal_bw207298.49088_skew324.12672_neigh1.25745_cross0.75965_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_diagonal_bw147175.94882_skew313.61244_neigh1.25927_cross0.75819_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_diagonal_bw160722.70732_skew313.61244_neigh1.25886_cross0.75853_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_diagonal_bw174086.80059_skew313.61244_neigh1.2581_cross0.75845_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_diagonal_bw187595.73649_skew313.61244_neigh1.25764_cross0.75905_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_diagonal_bw197107.06742_skew313.61244_neigh1.15768_cross0.98285_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_diagonal_bw200512.30622_skew313.61244_neigh1.25755_cross0.75937_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_diagonal_bw201450.72793_skew313.61244_neigh0.87919_cross0.75907_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_diagonal_bw201648.55655_skew313.61244_neigh0.96351_cross0.75931_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_diagonal_bw202106.02353_skew313.61244_neigh1.04694_cross0.75957_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_diagonal_bw202474.16303_skew313.61244_neigh1.13071_cross0.75965_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_diagonal_bw202922.36711_skew313.61244_neigh1.21508_cross0.75942_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_diagonal_bw203998.92204_skew313.61244_neigh1.29803_cross0.759_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_diagonal_bw204574.41148_skew313.61244_neigh1.23281_cross0.9336_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_diagonal_bw205344.97895_skew313.61244_neigh1.38267_cross0.76047_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_diagonal_bw205907.14271_skew313.61244_neigh1.24839_cross0.88389_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_diagonal_bw206626.31443_skew313.61244_neigh1.2532_cross0.83425_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_diagonal_bw206974.26599_skew313.61244_neigh1.25565_cross0.78408_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_diagonal_bw207389.25109_skew313.61244_neigh1.25761_cross0.73493_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_diagonal_bw207439.23549_skew313.61244_neigh1.46644_cross0.76053_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_diagonal_bw207459.39564_skew313.61244_neigh1.25937_cross0.68571_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_diagonal_bw207488.93911_skew313.61244_neigh1.25981_cross0.63655_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_diagonal_bw207554.94279_skew313.61244_neigh1.26046_cross0.587_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_diagonal_bw207595.91908_skew313.61244_neigh1.2611_cross0.53735_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_diagonal_bw212590.02941_skew313.61244_neigh1.54974_cross0.76249_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_diagonal_bw213723.771_skew313.61244_neigh1.25664_cross0.75921_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_diagonal_bw225205.18654_skew313.61244_neigh1.63328_cross0.76333_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_diagonal_bw226779.75408_skew313.61244_neigh1.25626_cross0.75971_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_diagonal_bw239405.59411_skew313.61244_neigh1.25615_cross0.75997_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_diagonal_bw252280.20232_skew313.61244_neigh1.2553_cross0.76024_n14.mtx","synthetic_862664_862664_19213120_avg22.27185_std29.81604_diagonal_bw264965.46748_skew313.61244_neigh1.25507_cross0.76101_n14.mtx","synthetic_862664_862664_19218664_avg22.27827_std29.78506_diagonal_bw207102.49027_skew303.10795_neigh1.25635_cross0.75926_n14.mtx","synthetic_862664_862664_19223712_avg22.28412_std30.77861_diagonal_bw207046.68591_skew313.4391_neigh1.25463_cross0.75893_n14.mtx","synthetic_862664_862664_19229090_avg22.29036_std29.72467_diagonal_bw206867.28901_skew282.12689_neigh1.25356_cross0.759_n14.mtx","synthetic_862664_862664_19237224_avg22.29979_std29.6692_diagonal_bw206653.68064_skew261.15496_neigh1.25095_cross0.7591_n14.mtx","synthetic_862664_862664_19238563_avg22.30134_std32.70575_diagonal_bw206629.43331_skew313.19637_neigh1.25007_cross0.75903_n14.mtx","synthetic_862664_862664_19242590_avg22.30601_std29.61737_diagonal_bw206307.07674_skew240.28029_neigh1.2466_cross0.75831_n14.mtx","synthetic_862664_862664_19243461_avg22.30702_std38.50483_diagonal_bw204657.66387_skew313.1164_neigh1.23352_cross0.75708_n14.mtx","synthetic_862664_862664_19244915_avg22.3087_std29.57096_diagonal_bw205734.95503_skew219.45208_neigh1.24265_cross0.75782_n14.mtx","synthetic_862664_862664_19245171_avg22.309_std34.6357_diagonal_bw205994.25147_skew313.08849_neigh1.24488_cross0.75815_n14.mtx","synthetic_862664_862664_19245832_avg22.30977_std36.57002_diagonal_bw205514.46748_skew313.0777_neigh1.23854_cross0.7575_n14.mtx","synthetic_862664_862664_19963054_avg23.14117_std29.84969_diagonal_bw201699.12835_skew311.90553_neigh1.2515_cross0.75854_n14.mtx","synthetic_862664_862664_21064571_avg24.41805_std29.92055_diagonal_bw202624.20371_skew314.66815_neigh1.24936_cross0.75798_n14.mtx","synthetic_862664_862664_22549917_avg26.13986_std29.9915_diagonal_bw201802.77435_skew311.74076_neigh1.26527_cross0.75886_n14.mtx","synthetic_862664_862664_23605495_avg27.36349_std30.07249_diagonal_bw202516.58382_skew314.82232_neigh1.2598_cross0.75787_n14.mtx","synthetic_862664_862664_25134553_avg29.13597_std30.15077_diagonal_bw202078.38456_skew311.6376_neigh1.27289_cross0.7586_n14.mtx","synthetic_88557_88557_7019496_avg79.26529_std1073.34158_diagonal_bw2695.50047_skew858.88461_neigh1.33448_cross0.83815_n14.mtx","synthetic_888032_888032_64506105_avg72.63939_std15.82947_diagonal_bw13252.2913_skew1.00993_neigh1.69563_cross0.81974_n14.mtx","synthetic_891419_891419_19853723_avg22.27204_std29.80105_diagonal_bw214143.5371_skew313.60965_neigh1.25649_cross0.76024_n14.mtx","synthetic_900004_900004_2853888_avg3.17097_std25.58712_diagonal_bw108539.4849_skew1482.14118_neigh0.14041_cross0.90359_n14.mtx","synthetic_91667_91667_6603697_avg72.04007_std19.01165_diagonal_bw5166.06684_skew1.19322_neigh1.65306_cross0.77077_n14.mtx","synthetic_920462_920462_44975685_avg48.86208_std11.94475_diagonal_bw157813.69929_skew1.2103_neigh1.69158_cross0.8057_n14.mtx","synthetic_92913_92913_2009995_avg21.63309_std13.78454_diagonal_bw43437.75323_skew4.03858_neigh1.00444_cross0.57612_n14.mtx","synthetic_948930_948930_21133875_avg22.27127_std29.77202_diagonal_bw227994.28681_skew313.6206_neigh1.25605_cross0.76054_n14.mtx","synthetic_952203_952203_32554296_avg34.1884_std11.93673_diagonal_bw195297.92349_skew1.69097_neigh1.66003_cross0.77069_n14.mtx","synthetic_952203_952203_35650826_avg37.44036_std11.95176_diagonal_bw168585.46646_skew1.53737_neigh1.6689_cross0.78029_n14.mtx","synthetic_952203_952203_38750463_avg40.69559_std11.9572_diagonal_bw172420.33981_skew1.4327_neigh1.67668_cross0.78884_n14.mtx","synthetic_952203_952203_41851500_avg43.95229_std11.95993_diagonal_bw175566.75547_skew1.3207_neigh1.68288_cross0.79655_n14.mtx","synthetic_952203_952203_44952640_avg47.20909_std11.96013_diagonal_bw161847.45785_skew1.22415_neigh1.68913_cross0.80243_n14.mtx","synthetic_952203_952203_46500626_avg48.83478_std13.95044_diagonal_bw161361.82335_skew1.37536_neigh1.696_cross0.7901_n14.mtx","synthetic_952203_952203_46500963_avg48.83514_std14.74428_diagonal_bw160687.29213_skew1.45725_neigh1.69741_cross0.78406_n14.mtx","synthetic_952203_952203_46501138_avg48.83532_std15.536_diagonal_bw160058.71086_skew1.53915_neigh1.6982_cross0.77843_n14.mtx","synthetic_4690002_4690002_14082483_avg3.00266_std1.15153_diagonal_bw5391.75085_skew319.7156_neigh0.3689_cross0.64826_n14.mtx","synthetic_4690002_4690002_18770459_avg4.00223_std1.21745_diagonal_bw4544.58663_skew312.82517_neigh0.41529_cross0.64886_n14.mtx","synthetic_4690002_4690002_18770794_avg4.0023_std1.16864_diagonal_bw4544.54744_skew281.83738_neigh0.4153_cross0.64886_n14.mtx","synthetic_4690002_4690002_18771567_avg4.00246_std1.15985_diagonal_bw4544.56125_skew260.83867_neigh0.4153_cross0.64886_n14.mtx","synthetic_4690002_4690002_18771765_avg4.00251_std1.28647_diagonal_bw4544.58181_skew312.80334_neigh0.41533_cross0.64886_n14.mtx","synthetic_4690002_4690002_18773155_avg4.0028_std1.35648_diagonal_bw4544.56842_skew312.7801_neigh0.41535_cross0.64886_n14.mtx","synthetic_952203_952203_46501138_avg48.83532_std15.536_diagonal_bw160058.71086_skew1.53915_neigh1.6982_cross0.77843_n14.mtx","synthetic_952203_952203_46502122_avg48.83635_std13.15524_diagonal_bw162100.43586_skew1.31385_neigh1.69445_cross0.79612_n14.mtx","synthetic_952203_952203_46502884_avg48.83715_std12.35905_diagonal_bw162891.6186_skew1.23191_neigh1.69248_cross0.80224_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_diagonal_bw116106.75277_skew1.19093_neigh1.69117_cross0.80488_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_diagonal_bw126698.85453_skew1.19093_neigh1.69122_cross0.80482_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_diagonal_bw137306.6029_skew1.19093_neigh1.69135_cross0.80527_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_diagonal_bw143715.18864_skew1.19093_neigh1.18395_cross0.80203_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_diagonal_bw144653.44632_skew1.19093_neigh1.29656_cross0.80259_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_diagonal_bw146143.79431_skew1.19093_neigh1.40968_cross0.80337_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_diagonal_bw147733.42467_skew1.19093_neigh1.69116_cross0.8053_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_diagonal_bw149667.71742_skew1.19093_neigh1.52206_cross0.80376_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_diagonal_bw150291.80563_skew1.19093_neigh1.61648_cross0.88662_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_diagonal_bw150742.69167_skew1.19093_neigh1.62479_cross0.88374_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_diagonal_bw158141.44775_skew1.19093_neigh1.69161_cross0.80521_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_diagonal_bw158785.70138_skew1.19093_neigh1.63527_cross0.80486_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_diagonal_bw159871.29872_skew1.19093_neigh1.68017_cross0.83168_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_diagonal_bw163210.79241_skew1.19093_neigh1.69195_cross0.80513_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_diagonal_bw165912.04361_skew1.19093_neigh1.70043_cross0.77937_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_diagonal_bw168260.36222_skew1.19093_neigh1.69196_cross0.80515_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_diagonal_bw170143.42422_skew1.19093_neigh1.71296_cross0.72708_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_diagonal_bw170449.52862_skew1.19093_neigh1.74831_cross0.80609_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_diagonal_bw173244.29895_skew1.19093_neigh1.72242_cross0.67391_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_diagonal_bw175577.55656_skew1.19093_neigh1.72872_cross0.62205_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_diagonal_bw177436.06958_skew1.19093_neigh1.73407_cross0.56915_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_diagonal_bw178609.376_skew1.19093_neigh1.69178_cross0.80552_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_diagonal_bw188630.00924_skew1.19093_neigh1.69167_cross0.80566_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_diagonal_bw196253.24669_skew1.19093_neigh1.88306_cross0.80917_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_diagonal_bw198626.45229_skew1.19093_neigh1.6924_cross0.80549_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_diagonal_bw208660.18746_skew1.19093_neigh1.69223_cross0.80594_n14.mtx","synthetic_952203_952203_46503364_avg48.83766_std11.96106_diagonal_bw224378.247_skew1.19093_neigh1.86131_cross0.80899_n14.mtx","synthetic_952203_952203_46504635_avg48.83899_std11.56217_diagonal_bw163741.45383_skew1.14992_neigh1.6908_cross0.80838_n14.mtx","synthetic_952203_952203_46504874_avg48.83924_std10.76526_diagonal_bw164534.32716_skew1.06801_neigh1.68826_cross0.81515_n14.mtx","synthetic_952203_952203_46506617_avg48.84107_std9.96939_diagonal_bw165495.48128_skew0.98603_neigh1.68708_cross0.82079_n14.mtx","synthetic_952203_952203_46507502_avg48.842_std9.17223_diagonal_bw166478.11101_skew0.9041_neigh1.68492_cross0.82769_n14.mtx","synthetic_952203_952203_46508902_avg48.84347_std8.37556_diagonal_bw167919.97397_skew0.82215_neigh1.68255_cross0.83397_n14.mtx","synthetic_952203_952203_48054277_avg50.46642_std11.96125_diagonal_bw164498.00225_skew1.14004_neigh1.69477_cross0.8085_n14.mtx","synthetic_952203_952203_51155719_avg53.72354_std11.96081_diagonal_bw166947.78914_skew1.08475_neigh1.69857_cross0.81333_n14.mtx","synthetic_952203_952203_54257059_avg56.98056_std11.96124_diagonal_bw158641.62561_skew1.01823_neigh1.70257_cross0.81769_n14.mtx","synthetic_952203_952203_57358241_avg60.23741_std11.96072_diagonal_bw160780.13898_skew0.95892_neigh1.70685_cross0.82181_n14.mtx","synthetic_952203_952203_60460102_avg63.49497_std11.9616_diagonal_bw162722.07884_skew0.90566_neigh1.71034_cross0.82524_n14.mtx","synthetic_953812_953812_69251992_avg72.60549_std15.82731_diagonal_bw14247.53739_skew1.05219_neigh1.69481_cross0.82031_n14.mtx","synthetic_966671_966671_3064966_avg3.17064_std25.57082_diagonal_bw116665.94473_skew1482.29662_neigh0.14335_cross0.9042_n14.mtx","synthetic_97223_97223_7006972_avg72.07114_std19.01384_diagonal_bw5487.10435_skew1.19228_neigh1.65371_cross0.7705_n14.mtx","synthetic_983943_983943_48052837_avg48.83701_std11.95054_diagonal_bw168639.17559_skew1.12953_neigh1.69189_cross0.80596_n14.mtx","synthetic_98611_98611_5466825_avg55.43829_std11.03861_diagonal_bw3819.4861_skew0.894_neigh1.63102_cross0.78874_n14.mtx","synthetic_986703_986703_50142919_avg50.81865_std15.80386_diagonal_bw14822.43122_skew1.44005_neigh1.67248_cross0.78706_n14.mtx","synthetic_986703_986703_54918080_avg55.65817_std15.81077_diagonal_bw15138.82145_skew1.31772_neigh1.67905_cross0.79644_n14.mtx","synthetic_986703_986703_59694812_avg60.49927_std15.81312_diagonal_bw14664.0448_skew1.2149_neigh1.68455_cross0.80453_n14.mtx","synthetic_986703_986703_64472911_avg65.34176_std15.81391_diagonal_bw14414.58333_skew1.11197_neigh1.68893_cross0.81154_n14.mtx","synthetic_986703_986703_69250817_avg70.18405_std15.81417_diagonal_bw14641.24259_skew1.0375_neigh1.69312_cross0.81698_n14.mtx","synthetic_986703_986703_71632861_avg72.5982_std20.55272_diagonal_bw14466.90673_skew1.31411_neigh1.70018_cross0.79544_n14.mtx","synthetic_986703_986703_71633918_avg72.59927_std19.50045_diagonal_bw14532.04367_skew1.2452_neigh1.69963_cross0.80057_n14.mtx","synthetic_986703_986703_71635339_avg72.60071_std18.448_diagonal_bw14581.49631_skew1.17629_neigh1.69833_cross0.80629_n14.mtx","synthetic_986703_986703_71637178_avg72.60257_std17.39427_diagonal_bw14634.97505_skew1.10736_neigh1.69727_cross0.81167_n14.mtx","synthetic_986703_986703_71638484_avg72.6039_std16.3415_diagonal_bw14690.20318_skew1.03846_neigh1.69598_cross0.81731_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_diagonal_bw10335.21324_skew1.0109_neigh1.69533_cross0.82034_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_diagonal_bw11318.36353_skew1.0109_neigh1.69541_cross0.82033_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_diagonal_bw12275.00965_skew1.0109_neigh1.69568_cross0.82032_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_diagonal_bw13203.20771_skew1.0109_neigh1.63897_cross0.89577_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_diagonal_bw13249.95843_skew1.0109_neigh1.69575_cross0.82075_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_diagonal_bw13805.02825_skew1.0109_neigh1.19003_cross0.81894_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_diagonal_bw13844.57814_skew1.0109_neigh1.30228_cross0.81902_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_diagonal_bw13882.13942_skew1.0109_neigh1.41423_cross0.81952_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_diagonal_bw13951.55398_skew1.0109_neigh1.52662_cross0.82014_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_diagonal_bw14238.03579_skew1.0109_neigh1.69538_cross0.82019_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_diagonal_bw14278.82315_skew1.0109_neigh1.68517_cross0.84736_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_diagonal_bw14385.45717_skew1.0109_neigh1.63958_cross0.82034_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_diagonal_bw14735.70059_skew1.0109_neigh1.69514_cross0.82041_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_diagonal_bw15076.26174_skew1.0109_neigh1.70327_cross0.79318_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_diagonal_bw15212.0391_skew1.0109_neigh1.6958_cross0.82068_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_diagonal_bw15394.12525_skew1.0109_neigh1.75112_cross0.82054_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_diagonal_bw15634.92651_skew1.0109_neigh1.71345_cross0.73894_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_diagonal_bw16040.9703_skew1.0109_neigh1.72027_cross0.68529_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_diagonal_bw16190.01108_skew1.0109_neigh1.69515_cross0.82012_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_diagonal_bw16364.90593_skew1.0109_neigh1.72531_cross0.63012_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_diagonal_bw16621.34957_skew1.0109_neigh1.72913_cross0.57601_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_diagonal_bw17168.96391_skew1.0109_neigh1.69549_cross0.82015_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_diagonal_bw18143.13335_skew1.0109_neigh1.69549_cross0.82027_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_diagonal_bw19086.21946_skew1.0109_neigh1.69561_cross0.82051_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_diagonal_bw20302.1187_skew1.0109_neigh1.91272_cross0.82094_n14.mtx","synthetic_986703_986703_71639053_avg72.60447_std15.81435_diagonal_bw27082.55582_skew1.0109_neigh1.86325_cross0.82124_n14.mtx","synthetic_986703_986703_71640715_avg72.60616_std15.28673_diagonal_bw14762.27652_skew0.96953_neigh1.69488_cross0.82275_n14.mtx","synthetic_986703_986703_71642304_avg72.60777_std14.23366_diagonal_bw14828.49584_skew0.90062_neigh1.6931_cross0.829_n14.mtx","synthetic_986703_986703_71644194_avg72.60968_std13.17962_diagonal_bw14914.32275_skew0.84548_neigh1.6914_cross0.83495_n14.mtx","synthetic_986703_986703_71646204_avg72.61172_std12.12567_diagonal_bw15020.25928_skew0.77657_neigh1.68947_cross0.84053_n14.mtx","synthetic_986703_986703_71647231_avg72.61276_std11.07215_diagonal_bw15141.16106_skew0.70769_neigh1.68759_cross0.84656_n14.mtx","synthetic_986703_986703_74029078_avg75.02671_std15.81428_diagonal_bw14414.50158_skew0.97263_neigh1.69708_cross0.82306_n14.mtx","synthetic_986703_986703_78806613_avg79.86863_std15.81421_diagonal_bw14604.1321_skew0.91565_neigh1.70019_cross0.82763_n14.mtx","synthetic_986703_986703_83583783_avg84.71017_std15.81435_diagonal_bw14458.40041_skew0.86518_neigh1.70355_cross0.83196_n14.mtx","synthetic_986703_986703_88361224_avg89.552_std15.81425_diagonal_bw14614.23314_skew0.82017_neigh1.70575_cross0.83603_n14.mtx","synthetic_986703_986703_93139124_avg94.39428_std15.81414_diagonal_bw14500.66288_skew0.76917_neigh1.70841_cross0.8393_n14.mtx"]))


















Counter({'synthetic_185639_185639_18505682_avg99.68639_std122.27517_random_bw25744.70036_skew8.23897_neigh1.22874_cross0.82863_n14.mtx': 2,
         'synthetic_38120_38120_17911818_avg469.8798_std294.64729_random_bw17768.78305_skew1.77305_neigh1.96177_cross0.87832_n14.mtx': 2,
         'synthetic_986703_986703_71639053_avg72.60447_std15.81435_random_bw13218.95443_skew1.0109_neigh1.63894_cross0.89577_n14.mtx': 2,
         'synthetic_986703_986703_71639053_avg72.60447_std15.81435_random_bw14787.04212_skew1.0109_neigh1.69513_cross0.82047_n14.mtx': 2,
         'synthetic_986703_986703_71639053_avg72.60447_std15.81435_random_bw20444.65782_skew1.0109_neigh1.91237_cross0.8206_n14.mtx': 2,
         'synthetic_140874_140874_7809286_avg55.43454_std11.05282_diagonal_bw5457.86719_skew0.91217_neigh1.63077_cross0.78851_n14.mtx': 2,
         'synthetic_38120_38120_14186695_avg372.15884_std217.74813_diagonal_bw16610.97285_skew1.39145_neigh1.97336_cross0.92298_n14.mtx': 2,
         'synthetic_38120_381

         'synthetic_161070_161070_8179469_avg50.78208_std19.60099_diagonal_bw4938.19408_skew1.77657_neigh1.42871_cross0.71564_n14.mtx': 1,
         'synthetic_161070_161070_8179469_avg50.78208_std19.60099_diagonal_bw4963.01172_skew1.77657_neigh1.50995_cross0.71659_n14.mtx': 1,
         'synthetic_161070_161070_8179469_avg50.78208_std19.60099_diagonal_bw4974.48644_skew1.77657_neigh1.2296_cross0.6917_n14.mtx': 1,
         'synthetic_161070_161070_8179469_avg50.78208_std19.60099_diagonal_bw5006.7359_skew1.77657_neigh1.58991_cross0.71725_n14.mtx': 1,
         'synthetic_161070_161070_8179469_avg50.78208_std19.60099_diagonal_bw5047.52778_skew1.77657_neigh1.22518_cross0.71711_n14.mtx': 1,
         'synthetic_161070_161070_8179469_avg50.78208_std19.60099_diagonal_bw5128.09611_skew1.77657_neigh1.23448_cross0.64524_n14.mtx': 1,
         'synthetic_161070_161070_8179469_avg50.78208_std19.60099_diagonal_bw5252.15061_skew1.77657_neigh1.23809_cross0.60038_n14.mtx': 1,
         'synthetic_161070_161